In [ ]:
### Timing Template
start = time()
runtime = time() - start
runtime_rd = round(digits=2, runtime)
runtime1, runtime2 = seconds_to_hrs_min_sec(runtime)
println("Runtime v0 = $(runtime) seconds")
println("Runtime v1 = $(runtime1)")
println("Runtime v2 = $(runtime2)")

In [ ]:
############################## This Version ########################################
# • Changed seq_pango check to seq_clade check for related groups of sequences (pairwise check for new groups only——existing-group check was already by clade)
##############################################################################################################################################################
################# IMPORTANT NOTE: In order to run the chronic % DQ'ed version, you first have to run the main fx on "1_1_1" qc format ########################
################# Then you have to load the "1_1_1" qc data. After tha, you can run the main fx on whatever qc-filter level you desire. ######################
##############################################################################################################################################################

In [27]:
cell1_start_time = time(); start_load_pkgs = time()
    using Pkg;  
#    Pkg.add("Plots"); using Plots;  
    Pkg.add("JSON3"); using JSON3;
    Pkg.add("Dates"); using Dates;
    Pkg.add("JLD2"); using JLD2
    Pkg.add("HypothesisTests"); using HypothesisTests
    Pkg.add("DataFrames"); using DataFrames
    Pkg.add("CSV"); using CSV
    Pkg.add("FASTX"); using FASTX
#    Pkg.add("Combinatorics"); using Combinatorics
    Pkg.add("StatsBase"); using StatsBase
    using Statistics
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
println(pwd()); cd("/Users/ryhisner"); println(pwd())
#    Pkg.add("LaTeXStrings");  using LaTeXStrings
#    Pkg.add("PyPlot"); using PyPlot
#    Pkg.add("PlotlyJS"); using PlotlyJS
#    Pkg.add("PGFPlotsX"); using PGFPlotsX
#    Pkg.add("UnicodePlots"); using UnicodePlots
#    Pkg.add("InspectDR"); using InspectDR
#    Pkg.add("GLMakie"); using GLMakie
#    Pkg.add("CairoMakie"); using CairoMakie
#    Pkg.add("WGLMakie"); using WGLMakie
#    Pkg.add("GMT"); using GMT
#####################################################################################################################################
### Turns time in seconds to hours, minutes, & seconds
function seconds_to_hrs_min_sec(t)
    hours = 0
    minutes = 0
    seconds = 0
    hours = t÷3600
    minutes = (t%3600)÷60
    if t > 0.0001
        seconds = (t%3600)%60
    end
    hours_int = Int(hours)
    minutes_int = Int(minutes)
    minutes_str = split(string(minutes_int), ".")[1]
    hours_fin = split(string(hours_int), ".")[1]
    minutes_fin = ""
    minutes_fin = lpad(minutes_str, 2, "0")
    seconds_rd = round(digits=2, seconds)
    seconds_1 = string(split(string(seconds_rd), ".")[1])
    seconds_2 = string(split(string(seconds_rd), ".")[2])
    seconds_left = lpad(seconds_1, 2, "0")
    seconds_right = rpad(seconds_2, 2, "0")
    seconds_fin = "$(seconds_left).$(seconds_right)"
    final_time = "$hours_int:$minutes_fin:$seconds_fin"
    final_time2 = "$hours_int hr, $minutes_int min, $seconds_fin sec"
    return final_time, final_time2
end
#################################################################################
function add_leading_zero(int_str::String)
    int_str2 = int_str
    if length(int_str) == 1 && int_str ≠ "0"
        int_str2 = "0"*int_str
    end
    return int_str2
end     
######################################################################################################################################
### Adds zero to truncated digit so all numbers have same # of digits & line up nicely E.g. if number = 9.1 & digits_rd = 3, it returns 9.100
function add_zeros_to_rounded(number::Float64, digits_rd::Int)
    if number ≥ 1
        num_final = 0.0
        num_whole = number*10^digits_rd
        num_whole_str = split(string(num_whole), ".")[1]
        len = length(num_whole_str)
        if len ≤ 2
            num_final = "."*"0"^(digits_rd-len)*num_whole_str
        else
            num_final = num_whole_str[1:end-digits_rd]*"."*num_whole_str[end-digits_rd+1:end]
        end
    return num_final
    end
end
#####################################################################################################################################
## Makes all digits go to 2 decimal places so they line up nicely
function number_to_string_to_two_decimal_places(number::Float64, decimals::Int)  
    num_str = string(number)
    before_dec = split(num_str, ".")[1]
    before_dec_str = string(before_dec)
    after_dec = split(num_str, ".")[2]
    after_dec_str = ""
    if length(after_dec) ≥ decimals
        after_dec_str = after_dec[1:decimals]
    else
        after_dec_len = length(after_dec)
        zero_array = Vector{String}()
        missing_zeros_to_fill = decimals - after_dec_len
        for i in 1:missing_zeros_to_fill
            push!(zero_array, "0")
        end
        zeros_str = join(zero_array)
        after_dec_str = after_dec[1:after_dec_len]*zeros_str
    end
    final_num_string = before_dec_str*"."*after_dec_str
    return final_num_string
end
#####################################################################################################################################
load_pkgs_runtime = time() - start_load_pkgs
load_pkgs_hms1, load_pkgs_hms2 = seconds_to_hrs_min_sec(load_pkgs_runtime)
println("Time to Load Packages = ", load_pkgs_hms1); println("Time to Load Packages = ", load_pkgs_hms2)
#####################################################################################################################################
hydrophobic_index_dict = Dict{String, Int}("L"=>97, "I"=>99, "F"=>100, "W"=>97, "V"=>76, "M"=>74, "C"=>49, "Y"=>63, "A"=>41, "T"=>13, "E"=>-31, "H"=>8, "G"=>0, "S"=>-5, "Q"=>-10, "D"=>-55, "R"=>-14, "K"=>-23, "N"=>-28, "P"=>5, "*"=>0)
#####################################################################################################################################
wuhan_ref_seq = "..."
ref_seq = wuhan_ref_seq
#####################################################################################################################################
######################################################################################################################################
clade_set_complete = Set(["recombinant", "19A", "19B", "20A", "20B", "20C", "20D", "20E", "20F", "20G", "20H", "20I", "20J", "21A", "21B", "21C", "21D", "21E", "21F", "21G", "21H", "21I", "21J", "21K", "21L", "21M", "22A", "22B", "22C", "22D", "22E", "22F", "23A", "23B", "23C", "23D", "23E", "23F", "23G", "23H", "23I", "24A", "24B", "24C", "24D", "24E", "24F", "24G", "24H", "24I", "25A", "25B", "25C", "25D", "25E", "25F", "25G", "25H", "25I"])
clade_arr_complete = ["recombinant", "19A", "19B", "20A", "20B", "20C", "20D", "20E", "20F", "20G", "20H", "20I", "20J", "21A", "21B", "21C", "21D", "21E", "21F", "21G", "21H", "21I", "21J", "21K", "21L", "21M", "22A", "22B", "22C", "22D", "22E", "22F", "23A", "23B", "23C", "23D", "23E", "23F", "23G", "23H", "23I", "24A", "24B", "24C", "24D", "24E", "24F", "24G", "24H", "24I", "25A", "25B", "25C", "25D", "25E", "25F", "25G", "25H", "25I"]
clade_to_pango = Dict("recombinant"=>"recombinant", "19A"=>"B", "19B"=>"A", "20A"=>"B.1", "20B"=>"B.1.1", "20C"=>"B.1", "20D"=>"B.1.1.1", "20E"=>"B.1.177", "20F"=>"D.2", "20G"=>"B.1.2", "20H"=>"B.1.351", "20I"=>"B.1.1.7", "20J"=>"P.1", "21A"=>"B.1.617.2", "21B"=>"B.1.617.1", "21C"=>"B.1.427", "21D"=>"B.1.525", "21E"=>"P.3", "21F"=>"B.1.526", "21G"=>"C.37", "21H"=>"B.1.621", "21I"=>"B.1.617.2", "21J"=>"B.1.617.2", "21K"=>"BA.1", "21L"=>"BA.2", "21M"=>"BA.3", "22A"=>"BA.4", "22B"=>"BA.5", "22C"=>"BA.2.12.1", "22D"=>"BA.2.75", "22E"=>"BQ.1", "22F"=>"XBB", "23A"=>"XBB.1.5", "23B"=>"XBB.1.16", "23C"=>"CH.1.1", "23D"=>"XBB.1.9", "23E"=>"XBB.2.3", "23F"=>"EG.5.1", "23G"=>"XBB.1.5.70", "23H"=>"HK.3", "23I"=>"BA.2.86", "24A"=>"JN.1", "24B"=>"JN.1.11.1", "24C"=>"KP.3", "24D"=>"XDV.1", "24E"=>"KP.3.1.1", "24F"=>"XEC", "24G"=>"KP.2.3", "24H"=>"LF.7", "24I"=>"MV.1", "25A"=>"LP.8.1", "25B"=>"NB.1.8.1", "25C"=>"XFG", "25D"=>"MC.10.2.1", "25E"=>"PY.1", "25F"=>"NW.1.2", "25G"=>"XFC", "25H"=>"XFJ", "25I"=>"BA.3.2")
######################################################################################################################################
EPI_ISL(n) = split(n, "|")[2]
country(n) = split(n, "/")[2]
sequence_date(n) = split(n, "|")[3]
seq_lab(n) = split(n, "/")[3]
US_state(n) = split(split(n, "/")[3], "-")[1]
mut_gene(n) = split(string(n), ":")[1]
######################################################################################################################################
######################################################################################################################################
AA_mut_to_AA_pos(n) = aa_gene_comprehensive_dict[n]*":"*"$(aa_pos_comprehensive_dict[n])"
AAsub_gene(n) = aa_gene_comprehensive_dict[n]
AAsub_gene_num(n) = [aa_gene_comprehensive_dict[n], aa_pos_comprehensive_dict[n]]
mut_num_pos_only(n) = aa_pos_comprehensive_dict[n]
AAsub_gene_num_pos_only(n) = [aa_gene_comprehensive_dict[n], aa_pos_comprehensive_dict[n]]
mut_gene_Dict = Dict{String, Int}("ORF1a"=>1, "ORF1b"=>2, "S"=>3, "E"=>4, "M"=>5, "N"=>6, "ORF3a"=>7, "ORF6"=>8, "ORF7a"=>9, "ORF7b"=>10, "ORF8"=>11, "ORF9b"=>12)
#################################
AA_gene_sortKey_2(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n]], aa_pos_comprehensive_dict[n])
AA_gene_sortKey(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
AA_ct_sortKey1(n) = (1000÷mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
AA_ct_sortKey2(n) = (n[2], AA_ct_sortKey1(n))
AA_gene_pos_sortKey(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
AA_gene_pos_sortKey_2(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n]], aa_pos_comprehensive_dict[n])
AA_ct_pos_sortKey1(n) = (1000÷mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
AA_ct_pos_sortKey2(n) = (n[2], AA_ct_pos_sortKey1(n))
######################################################################################################################################
function AA_order_key(mut)
    gene = aa_gene_comprehensive_dict[mut]
    AApos = aa_pos_comprehensive_dict[mut]
    gene_pos = gene_print_order[gene]
    return (gene_pos, AApos)
end
#####################################################################################################################################
function pango_variant_sort_key(pango::String)
    dotparts = split(pango, ".")
    k1 = string(dotparts[1])
    k2 = 0
    k3 = 0
    k4 = 0
    if length(dotparts) ≥ 2
        k2 = parse(Int, string(dotparts[2]))
    end
    if length(dotparts) ≥ 3
        k3 = parse(Int, string(dotparts[3]))
    end
    if length(dotparts) ≥ 4
        k4 = parse(Int, string(dotparts[4]))
    end
    return (k1, k2, k3, k4)
end
######################################################################################################################################
gene_print_order = Dict{String, Int}("S"=>1, "N"=>2, "E"=>3, "M"=>4, "ORF3a"=>5, "ORF6"=>6, "ORF7a"=>7, "ORF7b"=>8, "ORF8"=>9, "ORF9b"=>10, "ORF1a"=>12, "ORF1b"=>13)
######################################################################################################################################
function pango_minus_X_fx(pango::String, minus::Int)
    unaliased = pango_to_pango_unaliased_v2[pango]
    dot_ct = count(".", unaliased)
    println(dot_ct)
    if dot_ct ≥ minus
        dotsplits = split(unaliased, ".")
        minus_X_unaliased = join(dotsplits[1:dot_ct+1-minus], ".")
        minus_X_pango = pango_unaliased_to_pango[minus_X_unaliased]
        println("$(pango), $(unaliased), minus-$(minus) = $(minus_X_unaliased)")
        return minus_X_pango
    else
        return pango
    end
end
####################################################################################
function pango_unaliased_minus_X_fx(unaliased::String, minus::Int)
    dot_ct = count(".", unaliased)
    if dot_ct ≥ minus 
        dotsplits = split(unaliased, ".")
        println(dotsplits)
        minus_X_unaliased = join(dotsplits[1:dot_ct+1-minus], ".")
        minus_X_pango = pango_unaliased_to_pango[minus_X_unaliased]
        println("$(unaliased), minus-$(minus) = $(minus_X_unaliased)")
        return minus_X
    else
        return minus_X_pango
    end
end
##########################################################################################################################################################################
function read_fasta(filepath::String)
    reader = FASTX.FASTA.Reader(open(filepath, "r"))
    fasta_in = [record for record in reader]
    close(reader)
    return[String(FASTX.FASTX.description(rec)) for rec in fasta_in],
    [uppercase(String(FASTX.FASTA.sequence(rec))) for rec in fasta_in]
end
##########################################################################################################################################################################
wuhan_ref_seq = "..."
refAA_ORF1a = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV"
refAA_ORF1b = "NRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
refAA_S = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
refAA_ORF3a = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
refAA_E = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
refAA_M = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
refAA_ORF6 = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
refAA_ORF7a = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
refAA_ORF7b = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
refAA_ORF8 = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
refAA_N = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
refAA_ORF9b = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
######################################################################################################################################
gene_array = ["ORF1a", "ORF1b", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF9b"]
gene_order_dict = Dict{String, Int}("ORF1a"=>1, "ORF1b"=>2, "S"=>3, "ORF3a"=>4, "E"=>4, "M"=>6, "ORF6"=>7, "ORF7a"=>8, "ORF7b"=>9, "ORF8"=>10, "N"=>11, "ORF9b"=>12)
gene_order_dict = Dict{String, Int}("ORF1a"=>1, "ORF1b"=>2, "S"=>3, "ORF3a"=>4, "E"=>4, "M"=>6, "ORF6"=>7, "ORF7a"=>8, "ORF7b"=>9, "ORF8"=>10, "N"=>11, "ORF9b"=>12)
gene_AA_sortKey(m) = (gene_order_dict[aa_gene_comprehensive_dict[m]], aa_pos_comprehensive_dict[m])
gene_AApos_sortKey(m) = (gene_order_dict[aa_gene_comprehensive_dict[m]], aa_pos_comprehensive_dict[m])
######################################################################################################################################
gene_AA_dict = Dict{String, String}()
gene_AA_dict["ORF1a"] = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV*"
gene_AA_dict["ORF1b"] = "RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
gene_AA_dict["S"] = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
gene_AA_dict["E"] = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
gene_AA_dict["M"] = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
gene_AA_dict["N"] = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
gene_AA_dict["ORF3a"] = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
gene_AA_dict["ORF6"] = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
gene_AA_dict["ORF7a"] = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
gene_AA_dict["ORF7b"] = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
gene_AA_dict["ORF8"] = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
gene_AA_dict["ORF9b"] = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
######################################################################################################################################
NSP_AA_size = Dict{String, Int}("NSP1"=>180, "NSP2"=>638, "NSP3"=>1945, "NSP4"=>500, "NSP5"=>306, "NSP6"=>290, "NSP7"=>83, "NSP8"=>198, "NSP9"=>113, "NSP10"=>139, "NSP11"=>0, "NSP12"=>932, "NSP13"=>601, "NSP14"=>527, "NSP15"=>346, "NSP16"=>299)                                                                # "NSP12"=>BitSet(1:923), 
NSP_ranges = Dict{String, String}("NSP1"=>"ORF1a:1-180", "NSP2"=>"ORF1a:181-818", "NSP3"=>"ORF1a:819-2763", "NSP4"=>"ORF1a:2764-3263", "NSP5"=>"ORF1a:3264-3569", "NSP6"=>"ORF1a:3570-3859", "NSP7"=>"ORF1a:3860-3942", "NSP8"=>"ORF1a:3943-4140", "NSP9"=>"ORF1a:4141-4253", "NSP10"=>"ORF1a:4254-4392", "NSP11"=>"", "NSP12"=>"1a:4393-1b:923", "NSP13"=>"ORF1b:924-1524", "NSP14"=>"ORF1b:1525-2051", "NSP15"=>"ORF1b:2052-2397", "NSP16"=>"ORF1b:2398-2696", "S"=>"S:1-1274", "ORF3a"=>"ORF3a:1-276", "E"=>"E:1-76", "M"=>"M:1-223", "ORF6"=>"ORF6:1-62", "ORF7a"=>"ORF7a:1-122", "ORF7b"=>"ORF7b:1-44", "ORF8"=>"ORF8:1-122", "N"=>"N:1-420", "ORF9b"=>"ORF9b:1-98")
NSP_ranges_num_only = Dict{String, BitSet}("NSP1"=>BitSet(1:180), "NSP2"=>BitSet(181:818), "NSP3"=>BitSet(819:2763), "NSP4"=>BitSet(2764:3263), "NSP5"=>BitSet(3264:3569), "NSP6"=>BitSet(3570:3859), "NSP7"=>BitSet(3860:3942), "NSP8"=>BitSet(3943:4140), "NSP9"=>BitSet(4141:4253), "NSP10"=>BitSet(4254:4392), "NSP11"=>BitSet(), "NSP12"=>BitSet([4393:4401; 1:923]), "NSP13"=>BitSet(924:1524), "NSP14"=>BitSet(1525:2051), "NSP15"=>BitSet(2052:2397), "NSP16"=>BitSet(2398:2696), "S"=>BitSet(1:1274), "ORF3a"=>BitSet(1:276), "E"=>BitSet(1:76), "M"=>BitSet(1:223), "ORF6"=>BitSet(1:62), "ORF7a"=>BitSet(1:122), "ORF7b"=>BitSet(1:44), "ORF8"=>BitSet(1:122), "N"=>BitSet(1:420), "ORF9b"=>BitSet(1:98))
NSP_ranges1a = Dict{Int, BitSet}(1=>BitSet(1:180), 2=>BitSet(181:818), 3=>BitSet(819:2763), 4=>BitSet(2764:3263), 5=>BitSet(3264:3569), 6=>BitSet(3570:3859), 7=>BitSet(3860:3942), 8=>BitSet(3943:4140), 9=>BitSet(4141:4253), 10=>BitSet(4254:4392), 12=>BitSet(4393:4401))
NSP_ranges1b = Dict{Int, BitSet}(12=>BitSet(1:923), 13=>BitSet(924:1524), 14=>BitSet(1525:2051), 15=>BitSet(2052:2397), 16=>BitSet(2398:2696))
NSP1a_add = Dict{Int, Int}(1=>0, 2=>180, 3=>818, 4=>2763, 5=>3263, 6=>3569, 7=>3859, 8=>3942, 9=>4140, 10=>4253, 11=>0, 12=>4392)
NSP1b_add = Dict{Int, Int}(12=>-9, 13=>923, 14=>1524, 15=>2051, 16=>2397)
NSP1ab_add = Dict{Int, Int}(1=>0, 2=>180, 3=>818, 4=>2763, 5=>3263, 6=>3569, 7=>3859, 8=>3942, 9=>4140, 10=>4353, 11=>0, 12=>-9, 13=>923, 14=>1524, 15=>2051, 16=>2397)
######################################################################################################################################
ORF_size_dict = Dict{String, Int}()
for (orf, aaseq) in gene_AA_dict
    orf_len = length(aaseq)
    ORF_size_dict[orf] = orf_len
end
###########################################################################################################################################################################
###########################################################################################################################################################################
AA_residues = Set(["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y", "-", "*", "X"])
aa_pos_comprehensive_dict = Dict{String, Int}()
aa_gene_comprehensive_dict = Dict{String, String}()
aa_gene_and_pos_comprehensive_dict = Dict{String, String}()
refAA_comprehensive_dict = Dict{String, String}()
qryAA_comprehensive_dict = Dict{String, String}()
global nonspike_muts = Set{String}()
for (orf, orf_len) in ORF_size_dict
    for aa1 in AA_residues
        for aa2 in AA_residues
            for i in 1:orf_len
                orf_mut = "$(orf):$(aa1)$(i)$(aa2)"
                orf_pos_mut = "$(orf):$(i)"
                aa_pos_comprehensive_dict[orf_mut] = i
                aa_pos_comprehensive_dict[orf_pos_mut] = i
                aa_gene_comprehensive_dict[orf_mut] = orf
                aa_gene_comprehensive_dict[orf_pos_mut] = orf
                aa_gene_and_pos_comprehensive_dict[orf_mut] = orf_pos_mut
                aa_gene_and_pos_comprehensive_dict[orf_pos_mut] = orf_pos_mut
                if orf ≠ "S"
                    push!(nonspike_muts, orf_mut)
                    push!(nonspike_muts, orf_pos_mut)
                end
                refAA_comprehensive_dict[orf_mut] = aa1
                qryAA_comprehensive_dict[orf_mut] = aa2
            end
        end
    end
end
aa_gene_comprehensive_dict["NTD_disulfide"] = "S"
aa_pos_comprehensive_dict["NTD_disulfide"] = 1
aa_gene_and_pos_comprehensive_dict["NTD_disulfide"] = "S:NTD_disulfide"
refAA_comprehensive_dict["NTD_disulfide"] = "NTDdisulfide"
qryAA_comprehensive_dict["NTD_disulfide"] = "NTD_disulfide"
aa_gene_comprehensive_dict["NTD:disulfide"] = "S"
aa_pos_comprehensive_dict["NTD:disulfide"] = 1
aa_gene_and_pos_comprehensive_dict["NTD:disulfide"] = "S:NTD_disulfide"
refAA_comprehensive_dict["NTD:disulfide"] = "NTDdisulfide"
qryAA_comprehensive_dict["NTD:disulfide"] = "NTD_disulfide"
######################################################## Below: 100% comprehensive ref_nuc & qry_nuc dicts #################################################################
ref_nuc_comprehensive_dict = Dict{String, String}()
qry_nuc_comprehensive_dict = Dict{String, String}()
nuc_mut_int_comprehensive_dict = Dict{String, Int}()
nuc_mut_int_string_comprehensive_dict = Dict{String, String}()
nuc_residues1 = ["T", "C", "A", "G"]
nuc_residues2 = ["T", "C", "A", "G", "Y", "R", "K", "W", "M", "S", "-", "N"]
for nres1 in nuc_residues1
    for nres2 in nuc_residues2
        for i in 1:30000
            mut = "$(nres1)$(i)$(nres2)"
            nucmpos = i
            ref_nuc_comprehensive_dict[mut] = nres1
            qry_nuc_comprehensive_dict[mut] = nres2
            nuc_mut_int_comprehensive_dict[mut] = i
            nuc_mut_int_string_comprehensive_dict[mut] = string(i)
        end
    end
end
##########################################################################################################################################################################
##########################################################################################################################################################################
NSP_muts_pos_dict = Dict{String, Int}()
NSP_muts_gene_dict = Dict{String, String}()
NSP_ref_AA_dict = Dict{String, String}()
NSP_qry_AA_dict = Dict{String, String}()
NSP_set = Set(["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"])
for nsp in NSP_set
    nsp_len = NSP_AA_size[nsp]
    for aa1 in AA_residues
        for aa2 in AA_residues
            for i in 1:nsp_len
                nspmut = "$(nsp):$(aa1)$(i)$(aa2)"
                nsppos = "$(nsp):$(i)"
                NSP_muts_gene_dict[nspmut] = nsp
                NSP_muts_pos_dict[nspmut] = i
                NSP_muts_gene_dict[nsppos] = nsp
                NSP_muts_pos_dict[nsppos] = i
                NSP_ref_AA_dict[nspmut] = aa1
                NSP_qry_AA_dict[nspmut] = aa2  
            end
        end
    end
end
###########################################################################################################################################################################
###########################################################################################################################################################################
function AAmut_to_AApos(m)
    gn = split(m, ":")[1]
    mt = split(m, ":")[2]
    pos = mt[2:end-1]
    AApos = gn*":"*pos
    return AApos
end
########################################################################################################################################################################
########################################################################################################################################################################
########################################################################################################################################################################
function mixed2nuc(mix_mut)
    mut = mix_mut
    qry_nuc = qry_nuc_comprehensive_dict[mut]
    ref_nuc = ref_nuc_comprehensive_dict[mut]
    nuc_int = nuc_mut_int_comprehensive_dict[mut]
    ref_n_pos = "$(ref_nuc)$(nuc_int)"
    if length(mix_mut) ≥ 4
        if qry_nuc == "T"
            mut = mix_mut
        elseif qry_nuc == "C"
            mut = mix_mut
        elseif qry_nuc == "A"
            mut = mix_mut
        elseif qry_nuc == "G"
            mut = mix_mut
        elseif qry_nuc == "N"
            mut = mix_mut
        end   
        if ref_nuc == "T"
            if qry_nuc == "Y"
                mut = ref_n_pos*"C"
            elseif qry_nuc == "W"
                mut = ref_n_pos*"A"
            elseif qry_nuc == "K"
                mut = ref_n_pos*"G"
            elseif qry_nuc == "M"
                mut = "$(ref_n_pos)C, $(ref_n_pos)A"
            elseif qry_nuc == "S"
                mut = "$(ref_n_pos)C, $(ref_n_pos)G"
            elseif qry_nuc == "R"
                mut = "$(ref_n_pos)A, $(ref_n_pos)G"
            end
        end
        if ref_nuc == "C"
            if qry_nuc == "Y"
                mut = ref_n_pos*"T"
            elseif qry_nuc == "M"
                mut = ref_n_pos*"A"
            elseif qry_nuc == "S"
                mut = ref_n_pos*"G"
            elseif qry_nuc == "R"
                mut = "$(ref_n_pos)A, $(ref_n_pos)G"
            elseif qry_nuc == "W"
                mut = "$(ref_n_pos)T, $(ref_n_pos)A"
            elseif qry_nuc == "K"
                mut = "$(ref_n_pos)T, $(ref_n_pos)G"
            end
        end
        if ref_nuc == "A"
            if qry_nuc == "R"
                mut = ref_n_pos*"G"
            elseif qry_nuc == "W"
                mut = ref_n_pos*"T"
            elseif qry_nuc == "M"
                mut = ref_n_pos*"C"
            elseif qry_nuc == "Y"
                mut = "$(ref_n_pos)T, $(ref_n_pos)C"
            elseif qry_nuc == "K"
                mut = "$(ref_n_pos)T, $(ref_n_pos)G"
            elseif qry_nuc == "S"
                mut = "$(ref_n_pos)C, $(ref_n_pos)G"
            end
        end
        if ref_nuc == "G"
            if qry_nuc == "R"
                mut = ref_n_pos*"A"
            elseif qry_nuc == "K"
                mut = ref_n_pos*"T"
            elseif qry_nuc == "S"
                mut = ref_n_pos*"C"
            elseif qry_nuc == "Y"
                mut = "$(ref_n_pos)T, $(ref_n_pos)C"
            elseif qry_nuc == "W"
                mut = "$(ref_n_pos)T, $(ref_n_pos)A"
            elseif qry_nuc == "M"
                mut = "$(ref_n_pos)C, $(ref_n_pos)A"
            end
        end
    end
    return mut
end
#####################################################################################################################################
####################### Making gene AA & nuc references for all designated variants #################################################
#####################################################################################################################################
gene_AA_pango_dict = Dict{String, Dict{String, String}}()
nuc_genome_pango_dict = Dict{String, String}()
pango_consensus_set = Set{String}()
headers1a, seqs1a = read_fasta("___pango_consensus_sequences/pango_consensus_AA_ORF1a_2025_06_25_NNL.fasta")
for i in 1:length(headers1a)
    pango = headers1a[i]
    push!(pango_consensus_set, pango)
end
for pango in pango_consensus_set
    gene_AA_pango_dict[pango] = Dict{String, String}()
end
################################################################################################
for gene in gene_array
    aa_file = "___pango_consensus_sequences/pango_consensus_AA_$(gene)_2025_06_25_NNL.fasta"
    headers, seqs = read_fasta(aa_file)
    for i in 1:length(headers)
        pango = headers[i]
        aa_seq = seqs[i]
        gene_AA_pango_dict[pango][gene] = aa_seq
    end
end
nuc_file = "___pango_consensus_sequences/pango_consensus_sequences_genome_nuc_2025_06_25_NNL.fasta"
nuc_headers, nuc_seqs = read_fasta(nuc_file)
for i in 1:length(nuc_headers)
    pango = nuc_headers[i]
    nuc_seq = nuc_seqs[i]
    if length(nuc_seq) ≥ 28000
        nuc_genome_pango_dict[pango] = nuc_seq
    end
end
seqs_in_nuc_genome_pango_dict = length(nuc_genome_pango_dict)
println("seqs_in_nuc_genome_pango_dict = $(seqs_in_nuc_genome_pango_dict)")
######################################################################################################################################
######################################################################################################################################
gene_hydrophobic_dict = Dict{String, Float64}()
for (gene, aaseq) in gene_AA_dict
    gene_hydrophobe_sum = 0
    for aa in aaseq
        hydrophobe_score = hydrophobic_index_dict[string(aa)]
        gene_hydrophobe_sum += hydrophobe_score
    end
    gene_hydrophobe_score = gene_hydrophobe_sum/length(aaseq)
    gene_hydrophobic_dict[gene] = gene_hydrophobe_score
end 
######################################################################################################################################
AA_res_set = Set(["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y", "-", "*"])
AA_res_set_noDel = Set(["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y", "*"])
AA_res_pairs = Set{Tuple{String, String}}()
for res1 in AA_res_set_noDel
    for res2 in AA_res_set
        push!(AA_res_pairs, (res1, res2) )
    end
end
###########################################################################################################################################################################
AA_triplets = Dict{String, String}("TTT"=>"F", "TTC"=>"F", "TTA"=>"L", "TTG"=>"L", "TCT"=>"S", "TCC"=>"S", "TCA"=>"S", "TCG"=>"S", "TAT"=>"Y", "TAC"=>"Y", "TAA"=>"*", "TAG"=>"*", "TGT"=>"C", "TGC"=>"C", "TGA"=>"*", "TGG"=>"W", "CTT"=>"L", "CTC"=>"L", "CTA"=>"L", "CTG"=>"L", "CCT"=>"P", "CCC"=>"P", "CCA"=>"P", "CCG"=>"P", "CAT"=>"H", "CAC"=>"H", "CAA"=>"Q", "CAG"=>"Q", "CGT"=>"R", "CGC"=>"R", "CGA"=>"R", "CGG"=>"R", "ATT"=>"I", "ATC"=>"I", "ATA"=>"I", "ATG"=>"M", "ACT"=>"T", "ACC"=>"T", "ACA"=>"T", "ACG"=>"T", "AAT"=>"N", "AAC"=>"N", "AAA"=>"K", "AAG"=>"K", "AGT"=>"S", "AGC"=>"S", "AGA"=>"R", "AGG"=>"R", "GTT"=>"V", "GTC"=>"V", "GTA"=>"V", "GTG"=>"V", "GCT"=>"A", "GCC"=>"A", "GCA"=>"A", "GCG"=>"A", "GAT"=>"D", "GAC"=>"D", "GAA"=>"E", "GAG"=>"E", "GGT"=>"G", "GGC"=>"G", "GGA"=>"G", "GGG"=>"G", "TT-"=>"X", "TC-"=>"X", "TA-"=>"X", "TG-"=>"X", "T-T"=>"X", "T-C"=>"X", "T-A"=>"X", "T-G"=>"X", "T--"=>"X", "CT-"=>"X", "CC-"=>"X", "CA-"=>"X", "CG-"=>"X", "C-T"=>"X", "C-C"=>"X", "C-A"=>"X", "C-G"=>"X", "C--"=>"X", "AT-"=>"X", "AC-"=>"X", "AA-"=>"X", "AG-"=>"X", "A-T"=>"X", "A-C"=>"X", "A-A"=>"X", "A-G"=>"X", "A--"=>"X", "GT-"=>"X", "GC-"=>"X", "GA-"=>"X", "GG-"=>"X", "G-T"=>"X", "G-C"=>"X", "G-A"=>"X", "G-G"=>"X", "G--"=>"X", "-TT"=>"X", "-TC"=>"X", "-TA"=>"X", "-TG"=>"X", "-T-"=>"X", "-CT"=>"X", "-CC"=>"X", "-CA"=>"X", "-CG"=>"X", "-C-"=>"X", "-AT"=>"X", "-AC"=>"X", "-AA"=>"X", "-AG"=>"X", "-A-"=>"X", "-GT"=>"X", "-GC"=>"X", "-GA"=>"X", "-GG"=>"X", "-G-"=>"X", "--T"=>"X", "--C"=>"X", "--A"=>"X", "--G"=>"X", "---"=>"X", "NTT"=>"X", "TNT"=>"X", "TTN"=>"X", "NTC"=>"X", "TNC"=>"X", "TCN"=>"X", "NTA"=>"X", "TNA"=>"X", "TAN"=>"X", "NTG"=>"X", "TNG"=>"X", "TGN"=>"X", "NT-"=>"X", "TN-"=>"X", "T-N"=>"X", "NTN"=>"X", "TNN"=>"X", "NCT"=>"X", "CNT"=>"X", "CTN"=>"X", "NCC"=>"X", "CNC"=>"X", "CCN"=>"X", "NCA"=>"X", "CNA"=>"X", "CAN"=>"X", "NCG"=>"X", "CNG"=>"X", "CGN"=>"X", "NC-"=>"X", "CN-"=>"X", "C-N"=>"X", "NCN"=>"X", "CNN"=>"X", "NAT"=>"X", "ANT"=>"X", "ATN"=>"X", "NAC"=>"X", "ANC"=>"X", "ACN"=>"X", "NAA"=>"X", "ANA"=>"X", "AAN"=>"X", "NAG"=>"X", "ANG"=>"X", "AGN"=>"X", "NA-"=>"X", "AN-"=>"X", "A-N"=>"X", "NAN"=>"X", "ANN"=>"X", "NGT"=>"X", "GNT"=>"X", "GTN"=>"X", "NGC"=>"X", "GNC"=>"X", "GCN"=>"X", "NGA"=>"X", "GNA"=>"X", "GAN"=>"X", "NGG"=>"X", "GNG"=>"X", "GGN"=>"X", "NG-"=>"X", "GN-"=>"X", "G-N"=>"X", "NGN"=>"X", "GNN"=>"X", "N-T"=>"X", "-NT"=>"X", "-TN"=>"X", "N-C"=>"X", "-NC"=>"X", "-CN"=>"X", "N-A"=>"X", "-NA"=>"X", "-AN"=>"X", "N-G"=>"X", "-NG"=>"X", "-GN"=>"X", "N--"=>"X", "-N-"=>"X", "--N"=>"X", "N-N"=>"X", "-NN"=>"X", "NNT"=>"X", "NNC"=>"X", "NNA"=>"X", "NNG"=>"X", "NN-"=>"X", "NNN"=>"X")                 
AA_triplet_dels = Dict{String, String}("TTT"=>"F", "TTC"=>"F", "TTA"=>"L", "TTG"=>"L", "TCT"=>"S", "TCC"=>"S", "TCA"=>"S", "TCG"=>"S", "TAT"=>"Y", "TAC"=>"Y", "TAA"=>"*", "TAG"=>"*", "TGT"=>"C", "TGC"=>"C", "TGA"=>"*", "TGG"=>"W", "CTT"=>"L", "CTC"=>"L", "CTA"=>"L", "CTG"=>"L", "CCT"=>"P", "CCC"=>"P", "CCA"=>"P", "CCG"=>"P", "CAT"=>"H", "CAC"=>"H", "CAA"=>"Q", "CAG"=>"Q", "CGT"=>"R", "CGC"=>"R", "CGA"=>"R", "CGG"=>"R", "ATT"=>"I", "ATC"=>"I", "ATA"=>"I", "ATG"=>"M", "ACT"=>"T", "ACC"=>"T", "ACA"=>"T", "ACG"=>"T", "AAT"=>"N", "AAC"=>"N", "AAA"=>"K", "AAG"=>"K", "AGT"=>"S", "AGC"=>"S", "AGA"=>"R", "AGG"=>"R", "GTT"=>"V", "GTC"=>"V", "GTA"=>"V", "GTG"=>"V", "GCT"=>"A", "GCC"=>"A", "GCA"=>"A", "GCG"=>"A", "GAT"=>"D", "GAC"=>"D", "GAA"=>"E", "GAG"=>"E", "GGT"=>"G", "GGC"=>"G", "GGA"=>"G", "GGG"=>"G", "TT-"=>"X", "TC-"=>"X", "TA-"=>"X", "TG-"=>"X", "T-T"=>"X", "T-C"=>"X", "T-A"=>"X", "T-G"=>"X", "T--"=>"-", "CT-"=>"X", "CC-"=>"X", "CA-"=>"X", "CG-"=>"X", "C-T"=>"X", "C-C"=>"X", "C-A"=>"X", "C-G"=>"X", "C--"=>"-", "AT-"=>"X", "AC-"=>"X", "AA-"=>"X", "AG-"=>"X", "A-T"=>"X", "A-C"=>"X", "A-A"=>"X", "A-G"=>"X", "A--"=>"-", "GT-"=>"X", "GC-"=>"X", "GA-"=>"X", "GG-"=>"X", "G-T"=>"X", "G-C"=>"X", "G-A"=>"X", "G-G"=>"X", "G--"=>"-", "-TT"=>"X", "-TC"=>"X", "-TA"=>"X", "-TG"=>"X", "-T-"=>"X", "-CT"=>"X", "-CC"=>"X", "-CA"=>"X", "-CG"=>"X", "-C-"=>"X", "-AT"=>"X", "-AC"=>"X", "-AA"=>"X", "-AG"=>"X", "-A-"=>"X", "-GT"=>"X", "-GC"=>"X", "-GA"=>"X", "-GG"=>"X", "-G-"=>"X", "--T"=>"-", "--C"=>"-", "--A"=>"-", "--G"=>"-", "---"=>"-", "NTT"=>"X", "TNT"=>"X", "TTN"=>"X", "NTC"=>"X", "TNC"=>"X", "TCN"=>"X", "NTA"=>"X", "TNA"=>"X", "TAN"=>"X", "NTG"=>"X", "TNG"=>"X", "TGN"=>"X", "NT-"=>"X", "TN-"=>"X", "T-N"=>"X", "NTN"=>"X", "TNN"=>"X", "NCT"=>"X", "CNT"=>"X", "CTN"=>"X", "NCC"=>"X", "CNC"=>"X", "CCN"=>"X", "NCA"=>"X", "CNA"=>"X", "CAN"=>"X", "NCG"=>"X", "CNG"=>"X", "CGN"=>"X", "NC-"=>"X", "CN-"=>"X", "C-N"=>"X", "NCN"=>"X", "CNN"=>"X", "NAT"=>"X", "ANT"=>"X", "ATN"=>"X", "NAC"=>"X", "ANC"=>"X", "ACN"=>"X", "NAA"=>"X", "ANA"=>"X", "AAN"=>"X", "NAG"=>"X", "ANG"=>"X", "AGN"=>"X", "NA-"=>"X", "AN-"=>"X", "A-N"=>"X", "NAN"=>"X", "ANN"=>"X", "NGT"=>"X", "GNT"=>"X", "GTN"=>"X", "NGC"=>"X", "GNC"=>"X", "GCN"=>"X", "NGA"=>"X", "GNA"=>"X", "GAN"=>"X", "NGG"=>"X", "GNG"=>"X", "GGN"=>"X", "NG-"=>"X", "GN-"=>"X", "G-N"=>"X", "NGN"=>"X", "GNN"=>"X", "N-T"=>"X", "-NT"=>"X", "-TN"=>"X", "N-C"=>"X", "-NC"=>"X", "-CN"=>"X", "N-A"=>"X", "-NA"=>"X", "-AN"=>"X", "N-G"=>"X", "-NG"=>"X", "-GN"=>"X", "N--"=>"X", "-N-"=>"X", "--N"=>"X", "N-N"=>"X", "-NN"=>"X", "NNT"=>"X", "NNC"=>"X", "NNA"=>"X", "NNG"=>"X", "NN-"=>"X", "NNN"=>"X")                 
############################################################################################################################################################################
############################################################################################################################################################################
nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
###################
index_to_tuple = Dict{Int, Tuple{Int, Int, Int}}()
tuple_to_index = Dict{Tuple{Int, Int, Int}, Int}()        
###########################################################
index = 0
for year in 2020:2027
    for month in 1:12
        if year%4 == 0
            month_days = leap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        else
            month_days = nonleap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        end
    end
end
for (index, date) in index_to_tuple
    tuple_to_index[date] = index
end
for y in 2020:2027
    tuple_to_index[(y, 0, 0)] = y*1000000
    index_to_tuple[y*1000000] = (y, 0, 0)
    for m in 1:12
        tuple_to_index[(y, m, 0)] = y*1000000 + m*1000
        index_to_tuple[y*1000000 + m*1000] = (y, m, 0)
    end
end
tuple_to_index[(0, 0, 0)] = 1000000000
index_to_tuple[1000000000] = (0, 0, 0)
############################################################################################################################################################################
############################################################################################################################################################################
function mut_ct_by_date_range_all_UNIVERSAL(date1::Int, date2::Int, all_dict::Dict{Int, Dict{String, Int}})
    for i in 1:3000
        if !haskey(all_dict, i)
            all_dict[i] = Dict{String, Int}()
        end
    end
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        for (mut, count) in all_dict[i]
            date1_to_date2_ct[mut] = get!(date1_to_date2_ct, mut, 0) + count
        end
    end
    return date1_to_date2_ct
end
############################################################################################################################################################################
############################################################################################################################################################################
function nuc_mut_pos_only_ct_by_date_range_all_UNIVERSAL(date1::Int, date2::Int, all_dict::Dict{Int, Dict{Int, Int}})
    for i in 1:3000
        if !haskey(all_dict, i)
            all_dict[i] = Dict{Int, Int}()
        end
    end
    date1_to_date2_ct = Dict()
    for i in date1:date2
        for (mut, count) in all_dict[i]
            date1_to_date2_ct[mut] = get!(date1_to_date2_ct, mut, 0) + count
        end
    end
    return date1_to_date2_ct
end
############################################################################################################################################################################
############################################################################################################################################################################
function ORF1abMut_to_NSP(mut::String)
    NSPmut = ""
#    NSP_muts = Dict("NSP$(i)" => Dict{String, Int}() for i in 1:16 if i ≠ 11)
    gene = aa_gene_comprehensive_dict[mut]
    pos = aa_pos_comprehensive_dict[mut]
    refAA = ref_AA_dict[mut]
    qryAA = qry_AA_dict[mut]
    if gene == "ORF1a"
        for (NSP, range) in NSP_ranges1a
            if pos in range
                NSPpos = pos - NSP1a_add[NSP]
                NSPmut = "NSP$(NSP):$(refAA)$(NSPpos)$(qryAA)"
            end
        end
    end
    if gene == "ORF1b"
        for (NSP, range) in NSP_ranges1b
            if pos in range
                NSPpos = pos - NSP1b_add[NSP]
                NSPmut = "NSP$(NSP):$(refAA)$(NSPpos)$(qryAA)"
            end
        end
    end
    return NSPmut
end
#####################################################################################################################################
function NSPmut_to_ORF1ab(NSPmut::String)
    ORFmut = ""
    ORFpos = ""
    NSP_num = parse(Int, split(NSPmut, ":")[1][4:end])
    NSP_pos = NSP_muts_pos_dict[NSPmut]
    refAA = NSP_ref_AA_dict[NSPmut]
    qryAA = NSP_qry_AA_dict[NSPmut]
    if NSPnum in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        ORF1_pos = NSP1a_add[NSP_num] + NSP_pos
        ORFmut = "ORF1a:$(refAA)$(ORF1_pos)$(qryAA)"
    end
    if NSPnum in [13, 14, 15, 16]
        ORF1_pos = NSP1b_add[NSP_num] + NSP_pos
        ORFmut = "ORF1b:$(refAA)$(ORF1_pos)$(qryAA)"
    end
    if NSP_num == 12
        if NSP_pos in [1:8...]
            ORF1_pos = NSP1a_add[NSP_num] + NSP_pos
            ORFmut = "ORF1a:$(refAA)$(ORF1_pos)$(qryAA)"
        end
        if NSP_pos in [10:932...]
            ORF1_pos = NSP1b_add[NSP_num] + NSP_pos
            ORFmut = "ORF1b:$(refAA)$(ORF1_pos)$(qryAA)"
        end
    end
    ORF1pos = parse(Int, ORFpos)
    return ORFmut
end
#####################################################################################################################################
function NSP_range_to_ORF1ab_range(NSP_range::String)
    NSP = split(NSP_range, ":")[1]
    if length(NSP) < 3
        return ""
    end
    if NSP[1:3] ≠ "NSP"
        return ""
    end
    NSP_int = parse(Int, NSP[4:end])
    range = split(NSP_range, ":")[2]
    NSPpos1_int = parse(Int, split(range, "-")[1])
    NSPpos2_int = parse(Int, split(range, "-")[2])
    ORF_int1 = 0
    ORF_int2 = 0
    ORF1_AorB_pos1 = ""
    ORF1_AorB_pos2 = ""
    if NSP_int in [1:10...]
        ORF_int1 = NSPpos1_int + NSP1a_add[NSP_int]
        ORF_int2 = NSPpos2_int + NSP1a_add[NSP_int]
        ORF1_AorB_pos1 = "ORF1a"
        ORF1_AorB_pos2 = ""
    end
    if NSP_int in [13:16...]
        ORF_int1 = NSPpos1_int + NSP1b_add[NSP_int]
        ORF_int2 = NSPpos2_int + NSP1b_add[NSP_int]
        ORF1_AorB_pos1 = "ORF1b"
        ORF1_AorB_pos2 = ""
    end
    if NSP_int == 12
        if NSPpos1_int in [1:9...]
            ORF_int1 = NSPpos1_int + NSP1a_add[NSP_int]
            ORF1_AorB_pos1 = "1a"
        else
            ORF_int1 = NSPpos1_int + NSP1b_add[NSP_int]
            ORF1_AorB_pos1 = "ORF1b"
        end
        if NSPpos2_int in [1:9...]
            ORF_int2 = NSPpos2_int + NSP1a_add[NSP_int]
            ORF1_AorB_pos2 = "1a"
        end
        if !(NSPpos2_int in [1:9...])
            ORF_int2 = NSPpos2_int + NSP1b_add[NSP_int]
            if ORF1_AorB_pos1 == "1a"
                ORF1_AorB_pos2 = "1b:"
            else
            ORF1_AorB_pos2 = ""
            end
        end
    end
    ORF_int1_str = string(ORF_int1)
    ORF_int2_str = string(ORF_int2)
    ORF_range = ORF1_AorB_pos1*":"*ORF_int1_str*"-"*ORF1_AorB_pos2*ORF_int2_str
    return ORF_range
end
#####################################################################################################################################
function ORF1ab_range_to_NSP_range(ab_range::String)
    ab = split(ab_range, ":")[1]
    range = split(ab_range, ":")[2]
    pos1 = split(range, "-")[1]
    pos2 = split(range, "-")[2]
    pos1_int = parse(Int, pos1)
    pos2_int = parse(Int, pos2)
    NSPint1 = 0
    NSPint2 = 0
    NSPpos1 = ""
    NSPpos2 = ""
    NSPrange = ""
    NSPrange_pt1 = ""
    top_ct = 0
    if ab == "ORF1a"
        ct = 0
        for (NSP, rng) in NSP_ranges1a
            if pos1_int in rng && pos2_int in rng
                NSPint1 = pos1_int - NSP1a_add[NSP]
                NSPint2 = pos2_int - NSP1a_add[NSP]
                NSPpos1 = string(NSPint1)
                NSPpos2 = string(NSPint2)
                NSPstr = string(NSP)
                NSPrange = "NSP"*NSPstr*":"*NSPpos1*"-"*NSPpos2
            end
            if pos1_int in rng && !(pos2_int in rng)
                ct += 1
                NSPint1 = pos1_int - NSP1a_add[NSP]
                NSPpos1 = string(NSPint1)
                NSPstr = string(NSP)
                NSPrange_pt1 = "NSP"*NSPstr*":"*NSPpos1*"-"
            end
            if ct > 0
                top_ct += 1
                if pos2_int in rng
                    NSPint2 = pos2_int - NSP1a_add[NSP]
                    NSPpos2 = string(NSPint2)
                    NSPstr = string(NSP)
                    NSPrange_pt2 = "NSP"*NSPstr*":"*NSPpos2
                    NSPrange = NSPrange_pt1*NSPrange_pt2
                end
            end
        end
    end
    if ab == "ORF1b"
        ct = 0
        for (NSP, rng) in NSP_ranges1b
            if pos1_int in rng && pos2_int in rng
                NSPint1 = pos1_int - NSP1b_add[NSP]
                NSPint2 = pos2_int - NSP1b_add[NSP]
                NSPpos1 = string(NSPint1)
                NSPpos2 = string(NSPint2)
                NSPstr = string(NSP)
                NSPrange = "NSP"*NSPstr*":"*NSPpos1*"-"*NSPpos2
            end
            if pos1_int in rng && !(pos2_int in rng)
                ct += 1
                NSPint1 = pos1_int - NSP1b_add[NSP]
                NSPpos1 = string(NSPint1)
                NSPstr = string(NSP)
                NSPrange_pt1 = "NSP"*NSPstr*":"*NSPpos1*"-"
            end
            if ct > 0
                top_ct += 1
                if pos2_int in rng
                    NSPint2 = pos2_int - NSP1b_add[NSP]
                    NSPpos2 = string(NSPint2)
                    NSPstr = string(NSP)
                    NSPrange_pt2 = "NSP"*NSPstr*":"*NSPpos2
                    NSPrange = NSPrange_pt1*NSPrange_pt2
                end
            end
        end
    end 
    return NSPrange
end
######################################################################################################################################
function multiepi_to_epis(multi)  
    epi_num_only_pre(n) = split(n, "_")[3]
    epi_num_only_first(n) = parse(Int, split(epi_num_only_pre(n), "-")[1])
    epi_num_only_last(n) = parse(Int, split(epi_num_only_pre(n), "-")[2])
    first = epi_num_only_first(multi)
    last = epi_num_only_last(multi)
    epi_arr = Vector{String}()
    for i in first:last
        i_str = string(i)
        epi = "EPI_ISL_"*i_str
        push!(epi_arr, epi)
    end
    return epi_arr
end
######################################################################################################################################
function stringlist_to_strings(txt::String)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    arr_of_strings1 = Vector{String}()
    arr_of_strings2 = Vector{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(arr_of_strings2, mseq)
            end
        else 
            push!(arr_of_strings2, seq)
        end
    end
    sort_arr_of_strings2 = sort(collect(arr_of_strings2), by = x -> epi_sortkey(x))    
    return sort_arr_of_strings2
end
######################################################################################################################################
function stringlist_to_strings_set(txt::String)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    set_of_strings = Set{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(set_of_strings, mseq)
            end
        else 
            push!(set_of_strings, seq)
        end
    end
    return set_of_strings
end  
######################################################################################################################################
function stringlist_to_set(txt::String)
    set_of_strings = Set{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(set_of_strings, mseq)
            end
        else 
            push!(set_of_strings, seq)
        end
    end   
    return set_of_strings
end
#######################################################################################################################################
function list_to_strings(list::String)
    string_vec = string.(split(list, ", "))
    return string_vec
end
##################################################
function list_to_strings_set(list::String)
    string_vec = string.(split(list, ", "))
    string_set = Set{String}()
    for str in string_vec
        push!(string_set, str)
    end
    return string_set
end
#####################################################################################################################################
function cross_check_set(old_seq_set::Set{String}, new_seq_set::Set{String})
    new_set2 = Set{String}()
    old_set2 = Set{String}()
    old_seq_set_len = length(old_seq_set)
    new_seq_set_len = length(new_seq_set)
    difference = new_seq_set_len - old_seq_set_len
    println("Number of Sequences in Old List = $(old_seq_set_len)")
    println("Number of Sequences in New List = $(new_seq_set_len)")
    println("Difference between Old & New Sets = $(difference)")
    println()
    for seq in new_seq_set
        if !(seq in old_seq_set)
            push!(new_set2, seq)
        end
        if seq in old_seq_set
            push!(old_set2, seq)
        end
    end
    new_len = length(new_set2)
    old_len = length(old_set2)
    if difference == new_len
        println("The numbers check out: difference between old & new sets = Number of new seqs")
    else
        println("The numbers don't check out: difference between old & new sets not equal to Number of new seqs")
    end
    println()
    println("Number of New Sequences = $(new_len)")
    println("Number of Old Sequences = $(old_len)")
    new_set2_sort = sort(collect(new_set2), by = x -> (length(x), x))
    return new_set2, new_set2_sort
end
############################################################################################################################################################################
############################################################################################################################################################################
cell1_runtime = round(digits=1, time() - cell1_start_time)
c1runtime1, c1runtime2 = seconds_to_hrs_min_sec(cell1_runtime)
println("Cell1 Runtime v0 = $(cell1_runtime) seconds")
println("Cell1 Runtime v1 = $(c1runtime1)")
println("Cell1 Runtime v2 = $(c1runtime2)"); println()
######################################################################################################################################

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.to

2026_02_20__1100PM
/Users/ryhisner
/Users/ryhisner
Time to Load Packages = 0:00:09.13
Time to Load Packages = 0 hr, 0 min, 09.13 sec
seqs_in_nuc_genome_pango_dict = 3586
Cell1 Runtime v0 = 39.6 seconds
Cell1 Runtime v1 = 0:00:39.60
Cell1 Runtime v2 = 0 hr, 0 min, 39.60 sec



In [7]:
### Get confirmed BAL sequences and BAL Fx sequences
function string_to_set(txt::String)
    set_of_strings = Set{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        push!(set_of_strings, seq)
    end   
    return set_of_strings
end
BAL_seqs_known = stringlist_to_strings("EPI_ISL_2036230, EPI_ISL_1517099, EPI_ISL_1908476, EPI_ISL_2484152, EPI_ISL_3982251, EPI_ISL_4072038, EPI_ISL_4935949, EPI_ISL_9873278, EPI_ISL_10590270, EPI_ISL_11349763, EPI_ISL_13503362, EPI_ISL_14439530, EPI_ISL_14479735, EPI_ISL_14553998, EPI_ISL_14935931, EPI_ISL_15511842, EPI_ISL_15687965, EPI_ISL_16157875, EPI_ISL_16356910, EPI_ISL_16691487, EPI_ISL_16868655, EPI_ISL_16957015, EPI_ISL_17374807, EPI_ISL_17408352, EPI_ISL_17634290, EPI_ISL_17709926, EPI_ISL_17719124, EPI_ISL_17777729, EPI_ISL_17826285, EPI_ISL_17960749, EPI_ISL_17964486, EPI_ISL_18111040, EPI_ISL_18111041, EPI_ISL_18116015, EPI_ISL_19147578, EPI_ISL_19302363, EPI_ISL_19364322, EPI_ISL_19742521, EPI_ISL_19878655, EPI_ISL_19977768")
BAL_mut_list_old = string_to_set("ORF1a:T2183I, ORF1a:S2972F, ORF1a:S2972P, ORF1a:A3023V, ORF1a:A3049V, ORF1a:T3058I, ORF1a:A3070V, ORF1a:G3072C, ORF1a:H3076Y, ORF1a:L3123F, ORF1a:S3195G, ORF1a:P3359L, ORF1a:K3365R, ORF1a:A3454V, ORF1a:A3456V, ORF1a:S3946L, ORF1a:Q4110R, ORF1a:T4175I, ORF1a:K4176R, ORF1a:P4197S, ORF1a:I4205V, ORF1a:T4207A, ORF1a:R4387S, ORF1b:L314P, ORF1b:I1181T, ORF1b:Y1247C, ORF1b:T1424I, ORF1b:S2339F, ORF1b:K2340N, ORF1b:T2537I, S:S50L, S:T274I, S:P330S, S:N354D, S:V367F, S:F374L, S:F375-, S:A376V, S:N405S, S:N405D, S:Y508H, S:V551I, S:T573I, S:T618A, S:A647S, S:A653V, S:N657K, S:S659P, S:A668V, S:I693V, S:T859I, S:A944T, S:A958D, S:N978D, S:I1169T, S:S1175P, S:I1179T, S:I1183T, S:L1186F, E:V5A, E:V5F, E:T9I, E:G10S, E:G10C, E:I13-, E:V14-, E:S16N, E:F23S, E:L27S, E:T30I, E:A32V, E:A36V, E:Y42C, E:S68F, M:A2V, M:S4P, M:V10I, M:L17F, M:F28S, M:T77N, M:S94R, M:S94N, M:H125Y, M:H148R, M:A188T, N:P80L, N:S416L, N:T417I, ORF7a:T115I, ORF9b:M1T, ORF9b:Q77*")
BAL_mut_list_2025_12_10 = string_to_set("ORF1a:T2183I, ORF1a:T2967I, ORF1a:S2972F, ORF1a:S2972P, ORF1a:A3023V, ORF1a:T3032I, ORF1a:A3049V, ORF1a:T3058I, ORF1a:A3070V, ORF1a:G3072C, ORF1a:H3076Y, ORF1a:V3077A, ORF1a:F3089L, ORF1a:S3195G, ORF1a:I3255V, ORF1a:P3359L, ORF1a:K3365R, ORF1a:A3454V, ORF1a:A3456V, ORF1a:Q4110R, ORF1a:T4175I, ORF1a:K4176R, ORF1a:P4197S, ORF1a:I4205V, ORF1a:T4207A, ORF1a:R4387S, ORF1b:L314P, ORF1b:I1181T, ORF1b:Y1247C, ORF1b:T1424I, ORF1b:K1957R, ORF1b:S2339F, ORF1b:K2340N, ORF1b:T2537I, S:S50L, S:T274I, S:P330S, S:N354D, S:V367F, S:Y369H, S:A372T, S:F374L, S:F375-, S:A376V, S:N405S, S:N405D, S:Y508H, S:V551I, S:T573I, S:A647T, S:A647S, S:A653V, S:N657D, S:N657K, S:S659A, S:S659P, S:A668V, S:S691F, S:S691P, S:I693V, S:T859I, S:A944T, S:A944V, S:A958D, S:N978D, S:I1169T, S:I1179T, S:I1183T, S:L1186F, E:V5A, E:V5F, E:S6L, E:T9I, E:G10S, E:G10C, E:I13-, E:V14-, E:S16N, E:L19I, E:F23S, E:L27S, E:T30I, E:A32V, E:A36V, E:Y42C, E:I46V, E:S68F, E:P71L, M:N3K, M:S4P, M:F28S, M:T77N, M:S94R, M:S94N, M:H125Y, M:H148R, M:S173P, M:A188T, M:G189S, N:S416L, N:T417I, ORF7a:T115I, ORF9b:M1T, ORF9b:Q77*")
############################################################################################################################################
############################################################################################################################################

Set{String} with 91 elements:
  "E:S68F"
  "S:I1169T"
  "ORF1a:K3365R"
  "S:A376V"
  "S:A958D"
  "S:S1175P"
  "M:S4P"
  "E:T30I"
  "ORF1a:S3195G"
  "S:Y508H"
  "ORF1a:K4176R"
  "ORF1b:I1181T"
  "E:A36V"
  "ORF1a:S2972F"
  "ORF1b:L314P"
  "ORF1a:S2972P"
  "N:P80L"
  "S:F374L"
  "S:I1179T"
  "ORF7a:T115I"
  "ORF1a:I4205V"
  "ORF1b:T2537I"
  "S:A668V"
  "ORF9b:M1T"
  "S:N978D"
  ⋮ 

In [48]:
### Fx: load_all_seq_dicts, 2026_02_08   | many large ones not needed & therefore commented out  
load_all_start = time();   date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
function load_all_seq_dicts(date::String, fx_name::String, ndjson_name::String, max_AA_mut::Int, revs_thresh::Int, qc_max::Int)
    start_all = time()
    filename = "$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)"
############################################################################################################################################################################    
    global seq_ct_by_year_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year.jld2", "seq_ct_by_year")
    global seq_ct_by_year_month_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year_month.jld2", "seq_ct_by_year_month")
    global seq_ct_by_year_month_day_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day")
    global pango_date_index_ct = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_date_index_ct.jld2", "pango_date_index_ct")
############################################################################################################################################################################
    global avg_private_AA_per_circ_seq = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_avg_private_AA_per_circ_seq.jld2", "avg_private_AA_per_circ_seq")
    global all_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_all_seq_ct.jld2", "all_seq_ct")
    global qualifying_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_qualifying_seq_ct.jld2", "qualifying_seq_ct")
    global nuc_muts_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct.jld2", "nuc_muts_ct")
    global nuc_muts_ct_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels")
    global AA_muts_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct.jld2", "AA_muts_ct")
    global AA_dels_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_dels_ct.jld2", "AA_dels_ct")
    global AA_muts_ct_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels")
    global AA_muts_ct_pos_only_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only")
    global AA_muts_ct_pos_only_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels")
    global gene_mut_density_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density.jld2", "gene_mut_density")
    global domain_mut_density_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density.jld2", "domain_mut_density")
    global nuc_muts_ct_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_sort.jld2", "nuc_muts_ct_sort")
    global nuc_muts_ct_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_sort_by_seq_ct.jld2", "nuc_muts_ct_sort_by_seq_ct")
    global nuc_muts_ct_no_dels_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels_sort.jld2", "nuc_muts_ct_no_dels_sort")
    global nuc_muts_ct_no_dels_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels_sort_by_seq_ct.jld2", "nuc_muts_ct_no_dels_sort_by_seq_ct")
    global AA_muts_ct_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_sort.jld2", "AA_muts_ct_sort")
    global AA_muts_ct_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_sort_by_seq_ct.jld2", "AA_muts_ct_sort_by_seq_ct")
    global AA_muts_ct_pos_only_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_sort.jld2", "AA_muts_ct_pos_only_sort")
    global AA_muts_ct_pos_only_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_sort_by_seq_ct")
    global AA_muts_ct_no_dels_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels_sort.jld2", "AA_muts_ct_no_dels_sort")
    global AA_muts_ct_no_dels_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_sort_by_seq_ct")
    global AA_muts_ct_pos_only_no_dels_sort_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels_sort.jld2", "AA_muts_ct_pos_only_no_dels_sort")
    global AA_muts_ct_pos_only_no_dels_sort_by_seq_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_seq_ct")
    global gene_mut_density_sort_by_gene_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density_sort_by_gene.jld2", "gene_mut_density_sort_by_gene")
    global gene_mut_density_sort_by_density_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density_sort_by_density.jld2", "gene_mut_density_sort_by_density")
    global domain_mut_density_sort_by_gene_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density_sort_by_gene.jld2", "domain_mut_density_sort_by_gene")
    global domain_mut_density_sort_by_density_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density_sort_by_density.jld2", "domain_mut_density_sort_by_density")
    global AA_muts_seq_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_seq.jld2", "AA_muts_seq")
    global nuc_dels_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_dels_ct.jld2", "nuc_dels_ct")
############################################################################################################################################################################
    global AA_no_dels_sub_count_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_no_dels_sub_count.jld2", "AA_no_dels_sub_count")
    global total_AA_subs_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_total_AA_subs.jld2", "total_AA_subs")
#    global date_nuc_mut_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_nuc_mut_ct.jld2", "date_nuc_mut_ct")
#    global date_nuc_mut_ct_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels")
#    global date_AA_mut_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct.jld2", "date_AA_mut_ct")
#    global date_AA_mut_ct_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels")
#    global date_AA_mut_ct_pos_only_no_dels_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels")
#    global seq_date_index_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_date_index.jld2", "seq_date_index")
    global pango_date_index_ct = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_date_index_ct.jld2", "pango_date_index_ct")
    global clade_date_index_ct = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_clade_date_index_ct.jld2", "clade_date_index_ct")
############################################################################################################################################################################
######################################################## Below: semi-permanent, unt#il new ndjson made #####################################################################
############################################################################################################################################################################
## Changed mind about pango_date_index_ct--better to have high qc filters in place for those to weed out shitty misassigned seqs
#    global pango_date_index_ct = load("2025-11-12_all_private_muts_EPI_ISL_400000_20080000_v5_maxAAmut7_maxRevs2_qcMax10_pango_date_index_ct.jld2", "pango_date_index_ct")
#    global pango_date_index_ct = load("2025_08_25_all_private_muts_EPI_ISL_400000_20080000_maxAAmut90_maxRevs5_qcMax8000_pango_date_index_ct.jld2", "pango_date_index_ct")
#    global seq_date_index_all = load("dictionaries/2025-11-12_all_private_muts_EPI_ISL_400000_20080000_v5_maxAAmut90_maxRevs5_qcMax8000_seq_date_index.jld2", "seq_date_index")
#    global seq_date_tuple_all = load("dictionaries/2025-11-12_all_private_muts_EPI_ISL_400000_20080000_v5_maxAAmut90_maxRevs5_qcMax8000_seq_date_tuple.jld2", "seq_date_tuple")
#    global seq_pango_all = load("dictionaries/2025_08_25_all_private_muts_EPI_ISL_400000_20080000_maxAAmut90_maxRevs5_qcMax8000_seq_pango.jld2", "seq_pango")



    
#####################################################################################################################################
    global country_set = load("2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000/2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000_country_set.jld2", "country_set")
    global clade_set = load("2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000/2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000_clade_set.jld2", "clade_set")
    global pango_set = load("2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000/2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000_pango_set.jld2", "pango_set")
#    global seq_lab_set = load("2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000/2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000_seq_lab_set.jld2", "seq_lab_set")
#####################################################################################################################################
    
    
    
    
    #    date_temp = "2025-08-03"; max_AA_mut_temp = 5; revs_thresh_temp = 1; qc_max_temp = 5; ndjson_name_temp = "EPI_ISL_400000_20080000"
#    global country_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_country_set.jld2", "country_set")
#    global clade_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_clade_set.jld2", "clade_set")
#    global pango_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_pango_set.jld2", "pango_set")
#    global pango_unaliased_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_pango_unaliased_set.jld2", "pango_unaliased_set")
#    global pango_to_pango_unaliased = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_pango_to_pango_unaliased.jld2", "pango_to_pango_unaliased")
#    global clade_pango_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_clade_pango_set.jld2", "clade_pango_set")
#    global clade_pango_unaliased_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_clade_pango_unaliased_set.jld2", "clade_pango_unaliased_set")
#####################################################################################################################################
#    global seq_country_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_country.jld2", "seq_country")
#    global seq_US_state_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_US_state.jld2", "seq_US_state")
#    global seq_clade_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_clade.jld2", "seq_clade")
############ Currently using low-qc seq_pango_all as it's more comprehensive ###########
#    global seq_pango_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_pango.jld2", "seq_pango")
#    global seq_pango_unaliased_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_pango_unaliased.jld2", "seq_pango_unaliased")
############ Currently using low-qc seq_date_index_all as it's more comprehensive ########################
#    global seq_date_index_all = load("dictionaries/2025-11-12_all_private_muts_EPI_ISL_400000_20080000_v5_maxAAmut90_maxRevs5_qcMax8000_seq_date_index.jld2", "seq_date_index")
#    global seq_date_tuple_all = load("dictionaries/2025-11-12_all_private_muts_EPI_ISL_400000_20080000_v5_maxAAmut90_maxRevs5_qcMax8000_seq_date_tuple.jld2", "seq_date_tuple")
#    global seq_collection_date_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_collection_date.jld2", "seq_collection_date")
#    global seq_lab_dict_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_lab_dict.jld2", "seq_lab_dict")
#    global seq_nuc_del_ranges_ct_all = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_nuc_del_ranges_ct.jld2", "seq_nuc_del_ranges_ct")
#    global seq_lab_set = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_seq_lab_set.jld2", "seq_lab_set")
#####################################################################################################################################    
#    global pango_unaliased_date_index_ct = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_pango_unaliased_date_index_ct.jld2", "pango_unaliased_date_index_ct")
#    global country_clade_date_index_ct = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_country_clade_date_index_ct.jld2", "country_clade_date_index_ct")
#    global country_pango_date_index_ct = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_country_pango_date_index_ct.jld2", "country_pango_date_index_ct")
#    global country_pango_unaliased_date_index_ct = load("dictionaries/$(date_temp)_$(fx_name)_$(ndjson_name_temp)_maxAAmut$(max_AA_mut_temp)_maxRevs$(revs_thresh_temp)_qcMax$(qc_max_temp)_country_pango_unaliased_date_index_ct.jld2", "country_pango_unaliased_date_index_ct")
############################################################################################################################################################################
######################################################## Above: semi-permanent, unt#il new ndjson made #####################################################################
############################################################################################################################################################################
#    global ORF9b_CTD_muts_seq_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_ORF9b_CTD_muts_seq.jld2", "ORF9b_CTD_muts_seq")
    global multi_ORF9b_CTD_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD.jld2", "multi_ORF9b_CTD")
    global multi_ORF9b_CTD_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD_ct.jld2", "multi_ORF9b_CTD_ct")
#    global qualifying_ORF9b_double_ct_all = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_qualifying_ORF9b_double_ct.jld2", "qualifying_ORF9b_double_ct")
    
#    global ORF9b_CTD_muts_seq_all_relaxed_qc_10_1_30 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut8_maxRevs1_qcMax30_ORF9b_CTD_muts_seq_relaxed_qc_10_1_30.jld2", "ORF9b_CTD_muts_seq_relaxed_qc_10_1_30")
#    global multi_ORF9b_CTD_all_relaxed_qc_10_1_30 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut8_maxRevs1_qcMax30_multi_ORF9b_CTD_relaxed_qc_10_1_30.jld2", "multi_ORF9b_CTD_relaxed_qc_10_1_30")
#    global multi_ORF9b_CTD_ct_all_relaxed_qc_10_1_30 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut8_maxRevs1_qcMax30_multi_ORF9b_CTD_ct_relaxed_qc_10_1_30.jld2", "multi_ORF9b_CTD_ct_relaxed_qc_10_1_30")
#    global qualifying_ORF9b_double_ct_all_relaxed_qc_10_1_30 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut8_maxRevs1_qcMax30_qualifying_ORF9b_double_ct_relaxed_qc_10_1_30.jld2", "qualifying_ORF9b_double_ct_relaxed_qc_10_1_30")

#    global ORF9b_CTD_muts_seq_all_relaxed_qc_15_1_50 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut15_maxRevs1_qcMax50_ORF9b_CTD_muts_seq_relaxed_qc_15_1_50.jld2", "ORF9b_CTD_muts_seq_relaxed_qc_50_1_15")
#    global multi_ORF9b_CTD_all_relaxed_qc_15_1_50 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut15_maxRevs1_qcMax50_multi_ORF9b_CTD_relaxed_qc_15_1_50.jld2", "multi_ORF9b_CTD_relaxed_qc_50_1_15")
#    global multi_ORF9b_CTD_ct_all_relaxed_qc_15_1_50 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut15_maxRevs1_qcMax50_multi_ORF9b_CTD_ct_relaxed_qc_15_1_50.jld2", "multi_ORF9b_CTD_ct_relaxed_qc_50_1_15")
#    global qualifying_ORF9b_double_ct_all_relaxed_qc_15_1_50 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut15_maxRevs1_qcMax50_qualifying_ORF9b_double_ct_relaxed_qc_15_1_50.jld2", "qualifying_ORF9b_double_ct_relaxed_qc_50_1_15")

#    global ORF9b_CTD_muts_seq_all_relaxed_qc_25_3_200 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut25_maxRevs3_qcMax200_ORF9b_CTD_muts_seq_relaxed_qc_25_3_200.jld2", "ORF9b_CTD_muts_seq_relaxed_qc_200_3_25")
#    global multi_ORF9b_CTD_all_relaxed_qc_25_3_200 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut25_maxRevs3_qcMax200_multi_ORF9b_CTD_relaxed_qc_25_3_200.jld2", "multi_ORF9b_CTD_relaxed_qc_200_3_25")
#    global multi_ORF9b_CTD_ct_all_relaxed_qc_25_3_200 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut25_maxRevs3_qcMax200_multi_ORF9b_CTD_ct_relaxed_qc_25_3_200.jld2", "multi_ORF9b_CTD_ct_relaxed_qc_200_3_25")
#    global qualifying_ORF9b_double_ct_all_relaxed_qc_25_3_200 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut25_maxRevs3_qcMax200_qualifying_ORF9b_double_ct_relaxed_qc_25_3_200.jld2", "qualifying_ORF9b_double_ct_relaxed_qc_200_3_25")
#    global avg_private_AA_per_circ_seq2 = load("$(filename)/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_avg_private_AA_per_circ_seq2.jld2", "avg_private_AA_per_circ_seq2")
    ##################### Below: Dicts for all seqs using higher (less restrictive) QC filters ##########################################
#    global AA_muts_seq_all_8000qc_filter = load("2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000/2026_01_07_all_private_muts_EPI_ISL_400001_20300000_maxAAmut90_maxRevs5_qcMax8000_AA_muts_seq.jld2", "AA_muts_seq")
#    global AA_muts_seq_all_999qc_filter = load("dictionaries/2025-08-24_all_private_muts_EPI_ISL_400000_20080000_maxAAmut90_maxRevs4_qcMax999_AA_muts_seq.jld2", "AA_muts_seq")
#####################################################################################################################################
    println("Dictionaries loaded!");   println()
    finish_all = time() - start_all;    finish_all_rd = round(digits=1, finish_all)
    load_hms1, load_hms2 = seconds_to_hrs_min_sec(finish_all)
    println("Total Time to Load ALL Dictionaries = $(finish_all_rd) seconds")  # println("Total Time to Load ALL Dictionaries = $(load_hms1)")
    println("Total Time to Load ALL Dictionaries = $(load_hms2)")
end
######################################################################################################################################
date = "2026_02_08"
clade_pct_date_index = load("dictionaries/$(date)__clade_pct_date_index.jld2", "clade_pct_date_index")
clade_pct_date_tuple = load("dictionaries/$(date)__clade_pct_date_tuple.jld2", "clade_pct_date_tuple")
clade_pct_date_string = load("dictionaries/$(date)__clade_pct_date_string.jld2", "clade_pct_date_string")
pango_pct_date_index = load("dictionaries/$(date)__pango_pct_date_index.jld2", "pango_pct_date_index")
pango_pct_date_tuple = load("dictionaries/$(date)__pango_pct_date_tuple.jld2", "pango_pct_date_tuple")
pango_pct_date_string = load("dictionaries/$(date)__pango_pct_date_string.jld2", "pango_pct_date_string")
pango_unaliased_pct_date_index = load("dictionaries/$(date)__pango_unaliased_pct_date_index.jld2", "pango_unaliased_pct_date_index")
pango_unaliased_pct_date_tuple = load("dictionaries/$(date)__pango_unaliased_pct_date_tuple.jld2", "pango_unaliased_pct_date_tuple")
pango_unaliased_pct_date_string = load("dictionaries/$(date)__pango_unaliased_pct_date_string.jld2", "pango_unaliased_pct_date_string")
pango_to_pango_unaliased_v2 = load("dictionaries/$(date)__pango_to_pango_unaliased_v2.jld2", "pango_to_pango_unaliased_v2")
pango_unaliased_to_pango_prefix = load("dictionaries/$(date)__pango_unaliased_to_pango_prefix.jld2", "pango_unaliased_to_pango_prefix")
pango_unaliased_to_pango = load("dictionaries/$(date)__pango_unaliased_to_pango.jld2", "pango_unaliased_to_pango")
pango_unaliased_predecessor_meta_dict = load("dictionaries/$(date)__pango_unaliased_predecessor_meta_dict.jld2", "pango_unaliased_predecessor_meta_dict")
pango_predecessor_meta_dict = load("dictionaries/$(date)__pango_predecessor_meta_dict.jld2", "pango_predecessor_meta_dict")
pango_unaliased_set = load("dictionaries/$(date)__pango_unaliased_set.jld2", "pango_unaliased_set")
pango_unaliased_date_index_ct = load("dictionaries/$(date)__pango_unaliased_date_index_ct.jld2", "pango_unaliased_date_index_ct")
clade_date_index_cumul = load("dictionaries/$(date)__clade_date_index_cumul.jld2", "clade_date_index_cumul")
pango_date_index_cumul = load("dictionaries/$(date)__pango_date_index_cumul.jld2", "pango_date_index_cumul")
pango_unaliased_date_index_cumul = load("dictionaries/$(date)__pango_unaliased_date_index_cumul.jld2", "pango_unaliased_date_index_cumul")
clade_total = load("dictionaries/$(date)__clade_total.jld2", "clade_total")
pango_total = load("dictionaries/$(date)__pango_total.jld2", "pango_total")
pango_unaliased_total = load("dictionaries/$(date)__pango_unaliased_total.jld2", "pango_unaliased_total")
AAmut_date_index_cumul = load("dictionaries/$(date)__AAmut_date_index_cumul.jld2", "AAmut_date_index_cumul")
nucmut_date_index_cumul = load("dictionaries/$(date)__nucmut_date_index_cumul.jld2", "nucmut_date_index_cumul")
pango_nuc_sub_WT = load("dictionaries/$(date)__pango_nuc_sub_WT.jld2", "pango_nuc_sub_WT")
pango_nuc_del_WT = load("dictionaries/$(date)__pango_nuc_del_WT.jld2", "pango_nuc_del_WT")
pango_nuc_sub_private = load("dictionaries/$(date)__pango_nuc_sub_private.jld2", "pango_nuc_sub_private")
pango_nuc_del_private = load("dictionaries/$(date)__pango_nuc_del_private.jld2", "pango_nuc_del_private")
pango_nuc_sub_revs = load("dictionaries/$(date)__pango_nuc_sub_revs.jld2", "pango_nuc_sub_revs")
pango_nuc_del_revs = load("dictionaries/$(date)__pango_nuc_del_revs.jld2", "pango_nuc_del_revs")
pango_AAsub_WT = load("dictionaries/$(date)__pango_AAsub_WT.jld2", "pango_AAsub_WT")
pango_AAsub_WT_pos_only = load("dictionaries/$(date)__pango_AAsub_WT_pos_only.jld2", "pango_AAsub_WT_pos_only")
pango_AAdel_WT = load("dictionaries/$(date)__pango_AAdel_WT.jld2", "pango_AAdel_WT")
pango_AAsub_private = load("dictionaries/$(date)__pango_AAsub_private.jld2", "pango_AAsub_private")
pango_AAdel_private = load("dictionaries/$(date)__pango_AAdel_private.jld2", "pango_AAdel_private")
pango_AAsub_revs = load("dictionaries/$(date)__pango_AAsub_revs.jld2", "pango_AAsub_revs")
pango_AAdel_revs = load("dictionaries/$(date)__pango_AAdel_revs.jld2", "pango_AAdel_revs")
pango_frameshifts_WT = load("dictionaries/$(date)__pango_frameshifts_WT.jld2", "pango_frameshifts_WT")
pango_designation_date = load("dictionaries/$(date)__pango_designation_date.jld2", "pango_designation_date")
clade_pango_set = load("dictionaries/$(date)__clade_pango_set.jld2", "clade_pango_set")
println("Done!")
load_all_fx_runtime = time() - load_all_start; load_all_fx_runtime_rd = round(digits=3, load_all_fx_runtime)
load_all_fx_hms1, load_all_fx_hms2 = seconds_to_hrs_min_sec(load_all_fx_runtime)
println("Total Time to Run load_all Fx = $(load_all_fx_runtime_rd) seconds")  # println("Total Time to Run load_all Fx = $(load_all_fx_hms1)") 
println("Total Time to Run load_all Fx = $(load_all_fx_hms2)")
######################################################################################################################################
######################################################################################################################################

2026_02_21__807AM
Done!
Total Time to Run load_all Fx = 6.202 seconds
Total Time to Run load_all Fx = 0 hr, 0 min, 06.20 sec


In [4]:
### Execute Load All Dicts Fx, EPI_ISL_400000_20300000 | 2026_02_08 | QC--5-1-5 | Runtime = 38 sec #########################
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
start_load_all_dicts = time()
#                     date          fx_name            ndjson_name     max_AA_mut,revs_thresh,qc_max
HQCS_date = "2026_01_06"
HQCS_fx_name = "all_private_muts"
HQCS_ndjson_name = "EPI_ISL_400001_20300000"
HQCS_max_AA_mut = 5
HQCS_revs_thresh = 1
HQCS_qc_max = 5
HQCS_qc_string = "$(HQCS_max_AA_mut)_$(HQCS_revs_thresh)_$(HQCS_qc_max)"
load_all_seq_dicts(HQCS_date, HQCS_fx_name, HQCS_ndjson_name, HQCS_max_AA_mut, HQCS_revs_thresh, HQCS_qc_max)

gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)
domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP6_AmphHlx"=>20, "NSP6_MAE"=>21, "NSP6_cyto_CTD"=>22, "NSP12_NiRAN"=>23, "NSP12_intrfce"=>24, "NSP12_fingers"=>25, "NSP12_palm"=>26, "NSP12_palmLnk"=>27, "NSP12_thumb"=>28, "NSP13_ZBD"=>29, "NSP13_stalk"=>30, "NSP13_1B"=>31, "NSP13_RecA1"=>32, "NSP13_RecA2"=>33, "NSP14_nsp10"=>34, "NSP14_EXON"=>35, "NSP14_hinge1"=>36, "NSP14_hinge2"=>37, "NSP14_N7MTase"=>38, "NSP15_NTD"=>39, "NSP15_MD"=>40, "NSP15_endoU"=>41, "S_S1"=>42, "S_S2"=>43, "S_NTD"=>44, "S_N2R"=>45, "S_RBD"=>46, "S_RBM"=>47, "S_SD1"=>48, "S_SD2"=>49, "S_630_loop"=>50, "S_FCS_region"=>51, "S_Beta1"=>52, "S_3H"=>53, "S_IL770"=>54, "S_FPPR"=>55, "S_FP"=>56, "S_HR1"=>57, "S_CH"=>58, "S_CD"=>59, "S_Beta2"=>60, "S_2turnHelix"=>61, "S_HR2"=>62, "S_TM"=>63, "S_CT"=>64, "ORF3a_SignalP"=>65, "ORF3a_NTD"=>66, "ORF3a_TM1"=>67, "ORF3a_TM12Lnk"=>68, "ORF3a_TM2"=>69, "ORF3a_TM3"=>70, "ORF3a_cytosl1"=>71, "ORF3a_Loop"=>72, "ORF3a_3DB"=>73, "ORF3a_CTD"=>74, "E_TM"=>75, "E_cytosol"=>76, "E_CTD"=>77, "N_N1"=>78, "N_N2"=>79, "N_N3"=>80, "N_N4"=>81, "N_N5"=>82, "N_SR"=>83, "N_L_helix"=>84, "N_CBP"=>85, "N_9b_overlap"=>86)    

finish_load_all_dicts = time() - start_load_all_dicts
finish_load_all_dicts_rd = round(digits=1, finish_load_all_dicts)
println("Total Time to Load ALL Dictionaries = $(finish_load_all_dicts_rd)")
####################################################################################################################################
#################################### Create  date_index_seq_ct_all  Dictionary #####################################################
############################# Only needed for special calculations, so commented out here ##########################################
####################################################################################################################################
#date_index_seq_ct_all = Dict{Int, Int}()
#for d_index in values(seq_date_index_all)
#    date_index_seq_ct_all[d_index] = get(date_index_seq_ct_all, d_index, 0) + 1
#end
function date_index_range_seq_ct(date1::Int, date2::Int)
    date1_date2_seq_ct = 0
    for d_index in date1:date2
        date1_date2_seq_ct += date_index_seq_ct_all[d_index]
    end
    return date1_date2_seq_ct
end   
###################################################################################################################################
############################### Create  nuc_muts_ct_pos_only_no_dels_all  Dictionary ##############################################
###################################################################################################################################
nuc_sort_key(m) = parse(Int, m[2:end-1])
dict_make_start = time()
nuc_muts_ct_pos_only_no_dels_all = Dict{Int, Int}()
for i in 1:30000
    nuc_muts_ct_pos_only_no_dels_all[i] = 0
end
for (nuc_mut, count) in nuc_muts_ct_no_dels_all
    pos = nuc_sort_key(nuc_mut)
    nuc_muts_ct_pos_only_no_dels_all[pos] += count
end
#############################################__Create   pango_index_date_set    ###################################################
pango_index_date_set = Set{String}()
for pango in keys(pango_date_index_ct)
    push!(pango_index_date_set, pango)
end
println(length(pango_index_date_set))
####################################################################################################################################
#function pango_prefix_to_unaliased_fx(fx_pango::String)
#    prefix_unaliased = ""
#    dotpts_ct = count('.', fx_pango)
#    if dotpts_ct ≥ 1
#        dotsplits = split(fx_pango, ".")
#        prefix = string(dotsplits[1])
#        prefix_unaliased = get(pango_prefix_to_pango_unaliased, prefix, prefix)
#        last_pts = join(dotsplits[2:end], ".")
#        unaliased = "$(prefix_unaliased).$(last_pts)"
#        return unaliased, prefix_unaliased
#    else
#        return fx_pango, prefix_unaliased
#    end
#end
########################################################################################################################
#for pango in pango_set
#    unali, new_prfx = pango_prefix_to_unaliased_fx(pango)
#    pango_to_pango_unaliased_v2[pango] = unali
#end
#################################################################################################################################
println("length(keys(pango_to_pango_unaliased_v2) = $(length(keys(pango_to_pango_unaliased_v2)))")
dict_make_runtime = round(digits=2, time() - dict_make_start)
println("Total Time to Make nuc_muts_ct_pos_only_no_dels_all Dict = $(dict_make_runtime) seconds"); print("\n"^1)
#################################################################################################################################
#################################################################################################################################

2026_02_20__1009PM
Dictionaries loaded!

Total Time to Load ALL Dictionaries = 38.8 seconds
Total Time to Load ALL Dictionaries = 0 hr, 0 min, 38.83 sec
Total Time to Load ALL Dictionaries = 38.9
4297
length(keys(pango_to_pango_unaliased_v2) = 5657
Total Time to Make nuc_muts_ct_pos_only_no_dels_all Dict = 0.11 seconds



In [58]:
### Fx: chronic_load_dicts2_DQ, 2026_02_22 version ###############################
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
start_chr_load_fx = time()
function chronic_load_dicts2_DQ(ndjson_name::String, folder_name::String, date::String, rep_thresh::Int, revs_thresh::Int, print_ct_thresh::Int, DQ_mut_thresh::Int, DatePctDQThresh::Int, abs_min_mut_thresh::Int)
    start_date = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(start_date)
######################################################################################################################################
#    global cumulative_AA_mut_ct_by_date_index
#        cumulative_AA_mut_ct_by_date_index = load("$(folder_name)/$(folder_name)__cumulative_AA_mut_ct_by_date_index.jld2", "cumulative_AA_mut_ct_by_date_index")
#    global cumulative_nuc_mut_ct_by_date_index
#        cumulative_nuc_mut_ct_by_date_index = load("$(folder_name)/$(folder_name)__cumulative_nuc_mut_ct_by_date_index.jld2", "cumulative_nuc_mut_ct_by_date_index")
######################################################################################################################################
    global seq_AA_insertions_WT
        seq_AA_insertions_WT = load("$(folder_name)/$(folder_name)__seq_AA_insertions_WT.jld2", "seq_AA_insertions_WT")
    global seq_nuc_insertions_WT
        seq_nuc_insertions_WT = load("$(folder_name)/$(folder_name)__seq_nuc_insertions_WT.jld2", "seq_nuc_insertions_WT")    
######################################################################################################################################
    global total_chr_AA_subs
        total_chr_AA_subs = load("$(folder_name)/$(folder_name)__total_chr_AA_subs.jld2", "total_chr_AA_subs")
######################################################################################################################################
    global total_nuc_revs_seq
        total_nuc_revs_seq = load("$(folder_name)/$(folder_name)__total_nuc_revs_seq.jld2", "total_nuc_revs_seq")
    global seq_nuc_total_revs
        seq_nuc_total_revs = load("$(folder_name)/$(folder_name)__seq_nuc_total_revs.jld2", "seq_nuc_total_revs")
    global total_AA_revs_seq
        total_AA_revs_seq = load("$(folder_name)/$(folder_name)__total_AA_revs_seq.jld2", "total_AA_revs_seq")
    global seq_AA_total_revs
        seq_AA_total_revs = load("$(folder_name)/$(folder_name)__seq_AA_total_revs.jld2", "seq_AA_total_revs")
    global seq_AA_revs
        seq_AA_revs = load("$(folder_name)/$(folder_name)__seq_AA_revs.jld2", "seq_AA_revs")
######################################################################################################################################
    global AA_muts_ct_chr_all_ratio
        AA_muts_ct_chr_all_ratio = load("$(folder_name)/$(folder_name)__AA_muts_ct_chr_all_ratio.jld2", "AA_muts_ct_chr_all_ratio")
    global AA_muts_ct_no_dels_chr_all_ratio
        AA_muts_ct_no_dels_chr_all_ratio = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_chr_all_ratio.jld2", "AA_muts_ct_no_dels_chr_all_ratio")
    global AA_muts_ct_pos_only_no_dels_chr_all_ratio
        AA_muts_ct_pos_only_no_dels_chr_all_ratio = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio")
    global AA_muts_ct_no_dels_no_revs_chr_all_ratio
        AA_muts_ct_no_dels_no_revs_chr_all_ratio = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_no_dels_no_revs_chr_all_ratio")
    global AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio
        AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio")
###################################################################################################################################### 
    global nuc_muts_ct_no_dels_chr_all_ratio
        nuc_muts_ct_no_dels_chr_all_ratio = load("$(folder_name)/$(folder_name)__nuc_muts_ct_no_dels_chr_all_ratio.jld2", "nuc_muts_ct_no_dels_chr_all_ratio")
    global nuc_muts_ct_pos_only_no_dels_chr_all_ratio
        nuc_muts_ct_pos_only_no_dels_chr_all_ratio = load("$(folder_name)/$(folder_name)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio")
    global nuc_muts_ct_no_dels_no_revs_chr_all_ratio
        nuc_muts_ct_no_dels_no_revs_chr_all_ratio = load("$(folder_name)/$(folder_name)__nuc_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "nuc_muts_ct_no_dels_no_revs_chr_all_ratio")
    global nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio
        nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio = load("$(folder_name)/$(folder_name)__nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio")
###################################################################################################################################### 
####################################################################################################################################
    global AA_muts_ct_pos_only_adj_score
        AA_muts_ct_pos_only_adj_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score.jld2", "AA_muts_ct_pos_only_adj_score")
    global AA_muts_ct_adj
        AA_muts_ct_adj = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj.jld2", "AA_muts_ct_adj")
    global AA_muts_ct_pos_only_adj_score_no_dels
        AA_muts_ct_pos_only_adj_score_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score_no_dels.jld2", "AA_muts_ct_pos_only_adj_score_no_dels")
    global nuc_muts_ct_adj
        nuc_muts_ct_adj = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj.jld2", "nuc_muts_ct_adj")
    global AA_muts_ct_adj_score
        AA_muts_ct_adj_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score.jld2", "AA_muts_ct_adj_score")
    global AA_muts_ct_adj_score_no_dels
        AA_muts_ct_adj_score_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score_no_dels.jld2", "AA_muts_ct_adj_score_no_dels")
    global AA_muts_ct_pos_only_adj
        AA_muts_ct_pos_only_adj = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj.jld2", "AA_muts_ct_pos_only_adj")
    global AA_muts_ct_pos_only_adj_no_dels
        AA_muts_ct_pos_only_adj_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_no_dels.jld2", "AA_muts_ct_pos_only_adj_no_dels")
    global nuc_muts_ct_adj_score_no_dels
        nuc_muts_ct_adj_score_no_dels = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score_no_dels.jld2", "nuc_muts_ct_adj_score_no_dels")
    global nuc_muts_ct_adj_score
        nuc_muts_ct_adj_score = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score.jld2", "nuc_muts_ct_adj_score")
    global nuc_muts_ct_adj_no_dels
        nuc_muts_ct_adj_no_dels = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_no_dels.jld2", "nuc_muts_ct_adj_no_dels")
    global AA_muts_ct_adj_no_dels
        AA_muts_ct_adj_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_no_dels.jld2", "AA_muts_ct_adj_no_dels")
####################################################################################################################################
####################################################################################################################################
    global seq_ct_by_year
        seq_ct_by_year = load("$(folder_name)/$(folder_name)__seq_ct_by_year.jld2", "seq_ct_by_year")
    global seq_ct_by_year_month
        seq_ct_by_year_month = load("$(folder_name)/$(folder_name)__seq_ct_by_year_month.jld2", "seq_ct_by_year_month")
    global seq_ct_by_year_month_day
        seq_ct_by_year_month_day = load("$(folder_name)/$(folder_name)__seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day")
    global seq_date_tuple
        seq_date_tuple = load("$(folder_name)/$(folder_name)__seq_date_tuple.jld2", "seq_date_tuple")
    global date_nuc_mut_ct_no_dels
        date_nuc_mut_ct_no_dels = load("$(folder_name)/$(folder_name)__date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels")
    global date_AA_mut_ct_no_dels
        date_AA_mut_ct_no_dels = load("$(folder_name)/$(folder_name)__date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels")
    global date_AA_mut_ct
        date_AA_mut_ct = load("$(folder_name)/$(folder_name)__date_AA_mut_ct.jld2", "date_AA_mut_ct")
    global date_AA_mut_ct_pos_only_no_dels
        date_AA_mut_ct_pos_only_no_dels = load("$(folder_name)/$(folder_name)__date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels")
####################################################################################################################################
    global seq_collection_date
        seq_collection_date = load("$(folder_name)/$(folder_name)__seq_collection_date.jld2", "seq_collection_date")
    global seq_date_index
        seq_date_index = load("$(folder_name)/$(folder_name)__seq_date_index.jld2", "seq_date_index")
    global date_nuc_mut_ct
        date_nuc_mut_ct = load("$(folder_name)/$(folder_name)__date_nuc_mut_ct.jld2", "date_nuc_mut_ct")
####################################################################################################################################
    global seq_AA_dels
        seq_AA_dels = load("$(folder_name)/$(folder_name)__seq_AA_dels.jld2", "seq_AA_dels")
    global seq_AA_muts_pos_only_no_dels
        seq_AA_muts_pos_only_no_dels = load("$(folder_name)/$(folder_name)__seq_AA_muts_pos_only_no_dels.jld2", "seq_AA_muts_pos_only_no_dels")
    global seq_AA_muts_WT_pos_only
        seq_AA_muts_WT_pos_only = load("$(folder_name)/$(folder_name)__seq_AA_muts_WT_pos_only.jld2", "seq_AA_muts_WT_pos_only")
    global seq_AA_dels_WT
        seq_AA_dels_WT = load("$(folder_name)/$(folder_name)__seq_AA_dels_WT.jld2", "seq_AA_dels_WT")
    global seq_AA_muts_WT
        seq_AA_muts_WT = load("$(folder_name)/$(folder_name)__seq_AA_muts_WT.jld2", "seq_AA_muts_WT")
    global seq_AA_muts
        seq_AA_muts = load("$(folder_name)/$(folder_name)__seq_AA_muts.jld2", "seq_AA_muts")
    global seq_AA_muts_no_dels
        seq_AA_muts_no_dels = load("$(folder_name)/$(folder_name)__seq_AA_muts_no_dels.jld2", "seq_AA_muts_no_dels")
    global seq_AA_muts_pos_only
        seq_AA_muts_pos_only = load("$(folder_name)/$(folder_name)__seq_AA_muts_pos_only.jld2", "seq_AA_muts_pos_only")
    global seq_mixed_AA_muts
        seq_mixed_AA_muts = load("$(folder_name)/$(folder_name)__seq_mixed_AA_muts.jld2", "seq_mixed_AA_muts")
    global seq_unknown_AA
        seq_unknown_AA = load("$(folder_name)/$(folder_name)__seq_unknown_AA.jld2", "seq_unknown_AA")
    global seq_unknown_AA_ranges
        seq_unknown_AA_ranges = load("$(folder_name)/$(folder_name)__seq_unknown_AA_ranges.jld2", "seq_unknown_AA_ranges")
####################################################################################################################################
    global seq_nuc_muts
        seq_nuc_muts = load("$(folder_name)/$(folder_name)__seq_nuc_muts.jld2", "seq_nuc_muts")
    global seq_nuc_muts_WT
        seq_nuc_muts_WT = load("$(folder_name)/$(folder_name)__seq_nuc_muts_WT.jld2", "seq_nuc_muts_WT")
    global seq_nuc_del_ranges_WT
        seq_nuc_del_ranges_WT = load("$(folder_name)/$(folder_name)__seq_nuc_del_ranges_WT.jld2", "seq_nuc_del_ranges_WT")
    global seq_nuc_dropout
        seq_nuc_dropout = load("$(folder_name)/$(folder_name)__seq_nuc_dropout.jld2", "seq_nuc_dropout")
    global seq_nuc_del_ranges
        seq_nuc_del_ranges = load("$(folder_name)/$(folder_name)__seq_nuc_del_ranges.jld2", "seq_nuc_del_ranges")
    global seq_mixed_nucs
        seq_mixed_nucs = load("$(folder_name)/$(folder_name)__seq_mixed_nucs.jld2", "seq_mixed_nucs")
###################################################################################################################################
    global seq_clade
        seq_clade = load("$(folder_name)/$(folder_name)__seq_clade.jld2", "seq_clade")
    global seq_pango
        seq_pango = load("$(folder_name)/$(folder_name)__seq_pango.jld2", "seq_pango")
    global seq_pango_unaliased
        seq_pango_unaliased = load("$(folder_name)/$(folder_name)__seq_pango_unaliased.jld2", "seq_pango_unaliased")    
    global seq_clade_display
        seq_clade_display = load("$(folder_name)/$(folder_name)__seq_clade_display.jld2", "seq_clade_display")
    global seq_clade_ct
        seq_clade_ct = load("$(folder_name)/$(folder_name)__seq_clade_ct.jld2", "seq_clade_ct")
    global seq_pango_ct
        seq_pango_ct = load("$(folder_name)/$(folder_name)__seq_pango_ct.jld2", "seq_pango_ct")
    global seq_pango_unaliased_ct
        seq_pango_unaliased_ct = load("$(folder_name)/$(folder_name)__seq_pango_unaliased_ct.jld2", "seq_pango_unaliased_ct")
####################################################################################################################################
    global seq_US_state
        seq_US_state = load("$(folder_name)/$(folder_name)__seq_US_state.jld2", "seq_US_state")
    global seq_country
        seq_country = load("$(folder_name)/$(folder_name)__seq_country.jld2", "seq_country")
    global seq_lab_dict
        seq_lab_dict = load("$(folder_name)/$(folder_name)__seq_lab_dict.jld2", "seq_lab_dict")
####################################################################################################################################
    global gene_mut_density
        gene_mut_density = load("$(folder_name)/$(folder_name)__gene_mut_density.jld2", "gene_mut_density")
    global domain_mut_density
        domain_mut_density = load("$(folder_name)/$(folder_name)__domain_mut_density.jld2", "domain_mut_density")
####################################################################################################################################
    global nuc_muts_ct
        nuc_muts_ct = load("$(folder_name)/$(folder_name)__nuc_muts_ct.jld2", "nuc_muts_ct")
    global nuc_dels_ct
        nuc_dels_ct = load("$(folder_name)/$(folder_name)__nuc_dels_ct.jld2", "nuc_dels_ct")
    global nuc_muts_ct_no_dels
        nuc_muts_ct_no_dels = load("$(folder_name)/$(folder_name)__nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels")
######################
    global nuc_dels_seq
        nuc_dels_seq = load("$(folder_name)/$(folder_name)__nuc_dels_seq.jld2", "nuc_dels_seq")
    global nuc_muts_seq
        nuc_muts_seq = load("$(folder_name)/$(folder_name)__nuc_muts_seq.jld2", "nuc_muts_seq")
    global nuc_dels_seq_WT
        nuc_dels_seq_WT = load("$(folder_name)/$(folder_name)__nuc_dels_seq_WT.jld2", "nuc_dels_seq_WT")
    global nuc_muts_seq_WT
        nuc_muts_seq_WT = load("$(folder_name)/$(folder_name)__nuc_muts_seq_WT.jld2", "nuc_muts_seq_WT")
##################################################################
    global AA_muts_ct
        AA_muts_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct.jld2", "AA_muts_ct")
    global AA_muts_ct_no_dels_no_revs
        AA_muts_ct_no_dels_no_revs = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_no_revs.jld2", "AA_muts_ct_no_dels_no_revs")
    global AA_muts_ct_pos_only
        AA_muts_ct_pos_only = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only")
    global AA_dels_ct
        AA_dels_ct = load("$(folder_name)/$(folder_name)__AA_dels_ct.jld2", "AA_dels_ct")
    global AA_muts_ct_pos_only_no_dels
        AA_muts_ct_pos_only_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels")
    global AA_muts_ct_no_dels
        AA_muts_ct_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels")
############################## 
    global AA_muts_seq
        AA_muts_seq = load("$(folder_name)/$(folder_name)__AA_muts_seq.jld2", "AA_muts_seq")
    global AA_muts_seq_pos_only
        AA_muts_seq_pos_only = load("$(folder_name)/$(folder_name)__AA_muts_seq_pos_only.jld2", "AA_muts_seq_pos_only")
    global AA_muts_seq_pos_only_no_dels
        AA_muts_seq_pos_only_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_seq_pos_only_no_dels.jld2", "AA_muts_seq_pos_only_no_dels")
    global AA_dels_seq
        AA_dels_seq = load("$(folder_name)/$(folder_name)__AA_dels_seq.jld2", "AA_dels_seq")
##############################
    global AA_muts_seq_WT
        AA_muts_seq_WT = load("$(folder_name)/$(folder_name)__AA_muts_seq_WT.jld2", "AA_muts_seq_WT")
    global AA_muts_seq_WT_pos_only
        AA_muts_seq_WT_pos_only = load("$(folder_name)/$(folder_name)__AA_muts_seq_WT_pos_only.jld2", "AA_muts_seq_WT_pos_only")
    global AA_dels_seq_WT
        AA_dels_seq_WT = load("$(folder_name)/$(folder_name)__AA_dels_seq_WT.jld2", "AA_dels_seq_WT")
#####################################################################################################################################
#####################################################################################################################################
    global NSP_muts
        NSP_muts = load("$(folder_name)/$(folder_name)__NSP_muts.jld2", "NSP_muts")
    global NSP_muts_no_dels
        NSP_muts_no_dels = load("$(folder_name)/$(folder_name)__NSP_muts_no_dels.jld2", "NSP_muts_no_dels")
#####################################################################################################################################
    global non_rep_seqs_AA
        non_rep_seqs_AA = load("$(folder_name)/$(folder_name)__non_rep_seqs_AA.jld2", "non_rep_seqs_AA")
    global non_rep_seqs_AA_pos_only
        non_rep_seqs_AA_pos_only = load("$(folder_name)/$(folder_name)__non_rep_seqs_AA_pos_only.jld2", "non_rep_seqs_AA_pos_only")
    global non_rep_seqs_AA_pos_only_no_dels
        non_rep_seqs_AA_pos_only_no_dels = load("$(folder_name)/$(folder_name)__non_rep_seqs_AA_pos_only_no_dels.jld2", "non_rep_seqs_AA_pos_only_no_dels")
#####################################################################################################################################
    global rep_seq_grps_AA_muts_WT
        rep_seq_grps_AA_muts_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_muts_WT.jld2", "rep_seq_grps_AA_muts_WT")
    global rep_seq_grps_AA_muts_WT_pos_only
        rep_seq_grps_AA_muts_WT_pos_only = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_muts_WT_pos_only.jld2", "rep_seq_grps_AA_muts_WT_pos_only")
    global rep_seq_grps_AA_dels_WT
        rep_seq_grps_AA_dels_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_dels_WT.jld2", "rep_seq_grps_AA_dels_WT")
    global rep_seq_grps_nuc_muts_WT
        rep_seq_grps_nuc_muts_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_nuc_muts_WT.jld2", "rep_seq_grps_nuc_muts_WT")
    global rep_seq_grps_nuc_dels_WT
        rep_seq_grps_nuc_dels_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_nuc_dels_WT.jld2", "rep_seq_grps_nuc_dels_WT")
    global nuc_muts_rep_seq_grps_WT
        nuc_muts_rep_seq_grps_WT = load("$(folder_name)/$(folder_name)__nuc_muts_rep_seq_grps_WT.jld2", "nuc_muts_rep_seq_grps_WT")
    global nuc_dels_rep_seq_grps_WT
        nuc_dels_rep_seq_grps_WT = load("$(folder_name)/$(folder_name)__nuc_dels_rep_seq_grps_WT.jld2", "nuc_dels_rep_seq_grps_WT")
    global AA_dels_rep_seq_grps_WT
        AA_dels_rep_seq_grps_WT = load("$(folder_name)/$(folder_name)__AA_dels_rep_seq_grps_WT.jld2", "AA_dels_rep_seq_grps_WT")
    global AA_muts_rep_seq_grps_WT
        AA_muts_rep_seq_grps_WT = load("$(folder_name)/$(folder_name)__AA_muts_rep_seq_grps_WT.jld2", "AA_muts_rep_seq_grps_WT")
    global AA_muts_rep_seq_grps_WT_pos_only
        AA_muts_rep_seq_grps_WT_pos_only = load("$(folder_name)/$(folder_name)__AA_muts_rep_seq_grps_WT_pos_only.jld2", "AA_muts_rep_seq_grps_WT_pos_only")
#####################################################################################################################################
#####################################################################################################################################
    global rep_seq_grps_maxmut_pango
        rep_seq_grps_maxmut_pango = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_pango.jld2", "rep_seq_grps_maxmut_pango")
    global rep_seq_grps_maxmut_clade
        rep_seq_grps_maxmut_clade = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_clade.jld2", "rep_seq_grps_maxmut_clade")
    global rep_seq_grps_maxmut_pango_unaliased
        rep_seq_grps_maxmut_pango_unaliased = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_pango_unaliased.jld2", "rep_seq_grps_maxmut_pango_unaliased")
#####################################################################################################################################
    global rep_seq_grps_maxmut_dels
        rep_seq_grps_maxmut_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_dels.jld2", "rep_seq_grps_maxmut_dels")
    global rep_seq_grps_maxmut_AA_pos_only_no_dels
        rep_seq_grps_maxmut_AA_pos_only_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_pos_only_no_dels.jld2", "rep_seq_grps_maxmut_AA_pos_only_no_dels")
    global rep_seq_grps_maxmut_nuc
        rep_seq_grps_maxmut_nuc = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_nuc.jld2", "rep_seq_grps_maxmut_nuc")
    global rep_seq_grps_maxmut_AA_pos_only
        rep_seq_grps_maxmut_AA_pos_only = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_pos_only.jld2", "rep_seq_grps_maxmut_AA_pos_only")
    global rep_seq_grps_maxmut_nuc_dropout
        rep_seq_grps_maxmut_nuc_dropout = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_nuc_dropout.jld2", "rep_seq_grps_maxmut_nuc_dropout")
    global rep_seq_grps_maxmut_nuc_no_dels
        rep_seq_grps_maxmut_nuc_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_nuc_no_dels.jld2", "rep_seq_grps_maxmut_nuc_no_dels")
    global rep_seq_grps_maxmut_mixed_nucs
        rep_seq_grps_maxmut_mixed_nucs = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_mixed_nucs.jld2", "rep_seq_grps_maxmut_mixed_nucs")
    global rep_seq_grps_maxmut_AA_dels
        rep_seq_grps_maxmut_AA_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_dels.jld2", "rep_seq_grps_maxmut_AA_dels")
    global rep_seq_grps_maxmut_del_ranges_ct
        rep_seq_grps_maxmut_del_ranges_ct = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_del_ranges_ct.jld2", "rep_seq_grps_maxmut_del_ranges_ct")
    global rep_seq_grps_maxmut_AA
        rep_seq_grps_maxmut_AA = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA.jld2", "rep_seq_grps_maxmut_AA")
    global rep_seq_grps_maxmut_mixed_AA_muts
        rep_seq_grps_maxmut_mixed_AA_muts = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_mixed_AA_muts.jld2", "rep_seq_grps_maxmut_mixed_AA_muts")
    global rep_seq_grps_maxmut_unknown_AA
        rep_seq_grps_maxmut_unknown_AA = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_unknown_AA.jld2", "rep_seq_grps_maxmut_unknown_AA")
    global rep_seq_grps_maxmut_unknown_AA_ranges
        rep_seq_grps_maxmut_unknown_AA_ranges = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_unknown_AA_ranges.jld2", "rep_seq_grps_maxmut_unknown_AA_ranges")
    global rep_seq_grps_maxmut_seqs
        rep_seq_grps_maxmut_seqs = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_seqs.jld2", "rep_seq_grps_maxmut_seqs")
    global rep_seq_grps_maxmut_AA_no_dels
        rep_seq_grps_maxmut_AA_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_no_dels.jld2", "rep_seq_grps_maxmut_AA_no_dels")
    global rep_seq_grps_maxmut_nuc_muts_WT
        rep_seq_grps_maxmut_nuc_muts_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_nuc_muts_WT.jld2", "rep_seq_grps_maxmut_nuc_muts_WT")
    global rep_seq_grps_maxmut_nuc_dels_WT
        rep_seq_grps_maxmut_nuc_dels_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_nuc_dels_WT.jld2", "rep_seq_grps_maxmut_nuc_dels_WT")
    global rep_seq_grps_maxmut_AA_muts_WT
        rep_seq_grps_maxmut_AA_muts_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_muts_WT.jld2", "rep_seq_grps_maxmut_AA_muts_WT")
    global rep_seq_grps_maxmut_AA_dels_WT
        rep_seq_grps_maxmut_AA_dels_WT = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_dels_WT.jld2", "rep_seq_grps_maxmut_AA_dels_WT")
    global rep_seq_grps_maxmut_AA_muts_WT_pos_only
        rep_seq_grps_maxmut_AA_muts_WT_pos_only = load("$(folder_name)/$(folder_name)__rep_seq_grps_maxmut_AA_muts_WT_pos_only.jld2", "rep_seq_grps_maxmut_AA_muts_WT_pos_only")
################################################################################################################################
#####################################################################################################################################
    global rep_seq_grps_seqs
        rep_seq_grps_seqs = load("$(folder_name)/$(folder_name)__rep_seq_grps_seqs.jld2", "rep_seq_grps_seqs")
    global rep_seq_grps_pango
        rep_seq_grps_pango = load("$(folder_name)/$(folder_name)__rep_seq_grps_pango.jld2", "rep_seq_grps_pango")
    global rep_seq_grps_clade
        rep_seq_grps_clade = load("$(folder_name)/$(folder_name)__rep_seq_grps_clade.jld2", "rep_seq_grps_clade")
    global rep_seq_grps_pango_unaliased
        rep_seq_grps_pango_unaliased = load("$(folder_name)/$(folder_name)__rep_seq_grps_pango_unaliased.jld2", "rep_seq_grps_pango_unaliased")
    global rep_seq_grps_muts
        rep_seq_grps_muts = load("$(folder_name)/$(folder_name)__rep_seq_grps_muts.jld2", "rep_seq_grps_muts")
    global rep_seq_grps_mixed_nucs
        rep_seq_grps_mixed_nucs = load("$(folder_name)/$(folder_name)__rep_seq_grps_mixed_nucs.jld2", "rep_seq_grps_mixed_nucs")
    global rep_seq_grps_muts_no_dels
        rep_seq_grps_muts_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_muts_no_dels.jld2", "rep_seq_grps_muts_no_dels")
    global rep_seq_grps_AA_pos_only
        rep_seq_grps_AA_pos_only = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_pos_only.jld2", "rep_seq_grps_AA_pos_only")
    global rep_seq_grps_AA_no_dels
        rep_seq_grps_AA_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_no_dels.jld2", "rep_seq_grps_AA_no_dels")
    global rep_seq_grps_AA_dels
        rep_seq_grps_AA_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_dels.jld2", "rep_seq_grps_AA_dels")
    global rep_seq_grps_dels
        rep_seq_grps_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_dels.jld2", "rep_seq_grps_dels")
    global rep_seq_grps_AA_pos_only_no_dels
        rep_seq_grps_AA_pos_only_no_dels = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA_pos_only_no_dels.jld2", "rep_seq_grps_AA_pos_only_no_dels")
    global rep_seq_grps_del_ranges_ct
        rep_seq_grps_del_ranges_ct = load("$(folder_name)/$(folder_name)__rep_seq_grps_del_ranges_ct.jld2", "rep_seq_grps_del_ranges_ct")
    global rep_seq_grps_AA
        rep_seq_grps_AA = load("$(folder_name)/$(folder_name)__rep_seq_grps_AA.jld2", "rep_seq_grps_AA")
    global rep_seq_grps_nuc_dropout
        rep_seq_grps_nuc_dropout = load("$(folder_name)/$(folder_name)__rep_seq_grps_nuc_dropout.jld2", "rep_seq_grps_nuc_dropout")
#    global rep_seq_grps_unknown_AA
#        rep_seq_grps_unknown_AA = load("$(folder_name)/$(folder_name)__rep_seq_grps_unknown_AA.jld2", "rep_seq_grps_unknown_AA")
#    global rep_seq_grps_mixed_AA_muts
#        rep_seq_grps_mixed_AA_muts = load("$(folder_name)/$(folder_name)__rep_seq_grps_mixed_AA_muts.jld2", "rep_seq_grps_mixed_AA_muts")
    global AA_dels_rep_seq_grps
        AA_dels_rep_seq_grps = load("$(folder_name)/$(folder_name)__AA_dels_rep_seq_grps.jld2", "AA_dels_rep_seq_grps")
    global AA_muts_rep_seq_grps
        AA_muts_rep_seq_grps = load("$(folder_name)/$(folder_name)__AA_muts_rep_seq_grps.jld2", "AA_muts_rep_seq_grps")
    global AA_muts_rep_seq_grps_pos_only
        AA_muts_rep_seq_grps_pos_only = load("$(folder_name)/$(folder_name)__AA_muts_rep_seq_grps_pos_only.jld2", "AA_muts_rep_seq_grps_pos_only")
    global AA_muts_rep_seq_grps_no_dels
        AA_muts_rep_seq_grps_no_dels = load("$(folder_name)/$(folder_name)__AA_muts_rep_seq_grps_no_dels.jld2", "AA_muts_rep_seq_grps_no_dels")
    global nuc_muts_rep_seq_grps
        nuc_muts_rep_seq_grps = load("$(folder_name)/$(folder_name)__nuc_muts_rep_seq_grps.jld2", "nuc_muts_rep_seq_grps")
    global nuc_muts_rep_seq_grps_no_dels
        nuc_muts_rep_seq_grps_no_dels = load("$(folder_name)/$(folder_name)__nuc_muts_rep_seq_grps_no_dels.jld2", "nuc_muts_rep_seq_grps_no_dels")
    global nuc_dels_rep_seq_grps
        nuc_dels_rep_seq_grps = load("$(folder_name)/$(folder_name)__nuc_dels_rep_seq_grps.jld2", "nuc_dels_rep_seq_grps")    
#####################################################################################################################################
#####################################################################################################################################
################################### Below: Used to be in ungodly @save form, now changed ############################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
    global AA_muts_ct_pos_only_adj_sort_by_site
        AA_muts_ct_pos_only_adj_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_sort_by_site")
    global nuc_muts_ct_adj_score_no_dels_sort_by_site
        nuc_muts_ct_adj_score_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score_no_dels_sort_by_site.jld2", "nuc_muts_ct_adj_score_no_dels_sort_by_site")
    global nuc_muts_ct_adj_no_dels_sort_by_site
        nuc_muts_ct_adj_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_no_dels_sort_by_site.jld2", "nuc_muts_ct_adj_no_dels_sort_by_site")
    global nuc_muts_ct_adj_sort_by_seq_ct
        nuc_muts_ct_adj_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_sort_by_seq_ct.jld2", "nuc_muts_ct_adj_sort_by_seq_ct")
    global nuc_muts_ct_adj_score_sort_by_score
        nuc_muts_ct_adj_score_sort_by_score = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score_sort_by_score.jld2", "nuc_muts_ct_adj_score_sort_by_score")
    global AA_muts_ct_adj_score_sort_by_site
        AA_muts_ct_adj_score_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score_sort_by_site.jld2", "AA_muts_ct_adj_score_sort_by_site")
    global AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct
        AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct")
    global AA_muts_ct_pos_only_adj_score_sort_by_site
        AA_muts_ct_pos_only_adj_score_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_score_sort_by_site")
    global AA_muts_ct_pos_only_adj_sort_by_seq_ct
        AA_muts_ct_pos_only_adj_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_adj_sort_by_seq_ct")
    global AA_muts_ct_adj_score_no_dels_sort_by_site
        AA_muts_ct_adj_score_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score_no_dels_sort_by_site.jld2", "AA_muts_ct_adj_score_no_dels_sort_by_site")
    global AA_muts_ct_adj_sort_by_seq_ct
        AA_muts_ct_adj_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_sort_by_seq_ct.jld2", "AA_muts_ct_adj_sort_by_seq_ct")
    global AA_muts_ct_adj_no_dels_sort_by_site
        AA_muts_ct_adj_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_no_dels_sort_by_site.jld2", "AA_muts_ct_adj_no_dels_sort_by_site")
    global AA_muts_ct_pos_only_adj_score_sort_by_score
        AA_muts_ct_pos_only_adj_score_sort_by_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score_sort_by_score.jld2", "AA_muts_ct_pos_only_adj_score_sort_by_score")
    global AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score
        AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score.jld2", "AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score")
    global nuc_muts_ct_adj_no_dels_sort_by_seq_ct
        nuc_muts_ct_adj_no_dels_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", "nuc_muts_ct_adj_no_dels_sort_by_seq_ct")
    global AA_muts_ct_adj_sort_by_site
        AA_muts_ct_adj_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_sort_by_site.jld2", "AA_muts_ct_adj_sort_by_site")
    global AA_muts_ct_pos_only_adj_no_dels_sort_by_site
        AA_muts_ct_pos_only_adj_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_no_dels_sort_by_site")
    global AA_muts_ct_adj_score_sort_by_score
        AA_muts_ct_adj_score_sort_by_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score_sort_by_score.jld2", "AA_muts_ct_adj_score_sort_by_score")
    global nuc_muts_ct_adj_score_no_dels_sort_by_score
        nuc_muts_ct_adj_score_no_dels_sort_by_score = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score_no_dels_sort_by_score.jld2", "nuc_muts_ct_adj_score_no_dels_sort_by_score")
    global nuc_muts_ct_adj_sort_by_site
        nuc_muts_ct_adj_sort_by_site = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_sort_by_site.jld2", "nuc_muts_ct_adj_sort_by_site")
    global AA_muts_ct_adj_score_no_dels_sort_by_score
        AA_muts_ct_adj_score_no_dels_sort_by_score = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_score_no_dels_sort_by_score.jld2", "AA_muts_ct_adj_score_no_dels_sort_by_score")
    global AA_muts_ct_adj_no_dels_sort_by_seq_ct
        AA_muts_ct_adj_no_dels_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_adj_no_dels_sort_by_seq_ct")
    global AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site
        AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site")
    global nuc_muts_ct_adj_score_sort_by_site
        nuc_muts_ct_adj_score_sort_by_site = load("$(folder_name)/$(folder_name)__nuc_muts_ct_adj_score_sort_by_site.jld2", "nuc_muts_ct_adj_score_sort_by_site")
#####################################################################################################################################
#####################################################################################################################################
    global AA_muts_ct_no_dels_sort_by_site
        AA_muts_ct_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_sort_by_site.jld2", "AA_muts_ct_no_dels_sort_by_site")
    global AA_muts_ct_pos_only_sort_by_seq_ct
        AA_muts_ct_pos_only_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_sort_by_seq_ct")
    global nuc_muts_ct_sort_by_site
        nuc_muts_ct_sort_by_site = load("$(folder_name)/$(folder_name)__nuc_muts_ct_sort_by_site.jld2", "nuc_muts_ct_sort_by_site")
    global excluded_AA
        excluded_AA = load("$(folder_name)/$(folder_name)__excluded_AA.jld2", "excluded_AA")
    global chronic_search_muts
        chronic_search_muts = load("$(folder_name)/$(folder_name)__chronic_search_muts.jld2", "chronic_search_muts")
    global excluded_pos
        excluded_pos = load("$(folder_name)/$(folder_name)__excluded_pos.jld2", "excluded_pos")
    global rep_seqs
        rep_seqs = load("$(folder_name)/$(folder_name)__rep_seqs.jld2", "rep_seqs")
    global non_rep_seqs
        non_rep_seqs = load("$(folder_name)/$(folder_name)__non_rep_seqs.jld2", "non_rep_seqs")
    global all_seqs
        all_seqs = load("$(folder_name)/$(folder_name)__all_seqs.jld2", "all_seqs")
    global domain_mut_density_sort_by_gene
        domain_mut_density_sort_by_gene = load("$(folder_name)/$(folder_name)__domain_mut_density_sort_by_gene.jld2", "domain_mut_density_sort_by_gene")
    global gene_mut_density_sort_by_density
        gene_mut_density_sort_by_density = load("$(folder_name)/$(folder_name)__gene_mut_density_sort_by_density.jld2", "gene_mut_density_sort_by_density")
    global nuc_muts_ct_sort_by_seq_ct
        nuc_muts_ct_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__nuc_muts_ct_sort_by_seq_ct.jld2", "nuc_muts_ct_sort_by_seq_ct")
    global AA_muts_ct_sort_by_seq_ct
        AA_muts_ct_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_sort_by_seq_ct.jld2", "AA_muts_ct_sort_by_seq_ct")
    global AA_muts_ct_no_dels_sort_by_seq_ct
        AA_muts_ct_no_dels_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_sort_by_seq_ct")
    global AA_muts_ct_pos_only_no_dels_sort_by_site
        AA_muts_ct_pos_only_no_dels_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_site")
    global gene_mut_density_sort_by_gene
        gene_mut_density_sort_by_gene = load("$(folder_name)/$(folder_name)__gene_mut_density_sort_by_gene.jld2", "gene_mut_density_sort_by_gene")
    global AA_muts_ct_sort_by_site
        AA_muts_ct_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_sort_by_site.jld2", "AA_muts_ct_sort_by_site")
    global AA_muts_ct_pos_only_sort_by_site
        AA_muts_ct_pos_only_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_sort_by_site.jld2", "AA_muts_ct_pos_only_sort_by_site")
    global domain_mut_density_sort_by_density
        domain_mut_density_sort_by_density = load("$(folder_name)/$(folder_name)__domain_mut_density_sort_by_density.jld2", "domain_mut_density_sort_by_density")
    global too_many_reversions
        too_many_reversions = load("$(folder_name)/$(folder_name)__too_many_reversions.jld2", "too_many_reversions")
    global AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
        AA_muts_ct_pos_only_no_dels_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_seq_ct")

    global AA_muts_ct_chr_all_ratio_ct_sort
        AA_muts_ct_chr_all_ratio_ct_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_chr_all_ratio_ct_sort")
    global AA_muts_ct_chr_all_ratio_pos_sort
        AA_muts_ct_chr_all_ratio_pos_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_chr_all_ratio_pos_sort")    

    global AA_muts_ct_no_dels_chr_all_ratio_ct_sort
        AA_muts_ct_no_dels_chr_all_ratio_ct_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_no_dels_chr_all_ratio_ct_sort")
    global AA_muts_ct_no_dels_chr_all_ratio_pos_sort
        AA_muts_ct_no_dels_chr_all_ratio_pos_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_no_dels_chr_all_ratio_pos_sort")
    
    global AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort
        AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort")
    global AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort
        AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort = load("$(folder_name)/$(folder_name)__AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort")
    
    global avg_AA_subs_per_chr_seq
        avg_AA_subs_per_chr_seq = load("$(folder_name)/$(folder_name)__avg_AA_subs_per_chr_seq.jld2", "avg_AA_subs_per_chr_seq")
    global avg_AA_subs_per_circ_seq
        avg_AA_subs_per_circ_seq = load("$(folder_name)/$(folder_name)__avg_AA_subs_per_circ_seq.jld2", "avg_AA_subs_per_circ_seq")
######################################################################################################################################
######################################################################################################################################
    global NSP_muts_sortByCt_Arr
        NSP_muts_sortByCt_Arr = load("$(folder_name)/$(folder_name)__NSP_muts_sortByCt_Arr.jld2", "NSP_muts_sortByCt_Arr")
    global NSP_muts_sortByPos_Arr
        NSP_muts_sortByPos_Arr = load("$(folder_name)/$(folder_name)__NSP_muts_sortByPos_Arr.jld2", "NSP_muts_sortByPos_Arr")
    global NSP_muts_no_dels_sortByCt_Arr
        NSP_muts_no_dels_sortByCt_Arr = load("$(folder_name)/$(folder_name)__NSP_muts_no_dels_sortByCt_Arr.jld2", "NSP_muts_no_dels_sortByCt_Arr")
    global NSP_muts_no_dels_sortByPos_Arr
        NSP_muts_no_dels_sortByPos_Arr = load("$(folder_name)/$(folder_name)__NSP_muts_no_dels_sortByPos_Arr.jld2", "NSP_muts_no_dels_sortByPos_Arr")
    global NSP1_muts_sortByCt
        NSP1_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP1_muts_sortByCt.jld2", "NSP1_muts_sortByCt")
    global NSP1_muts_sortByPos
        NSP1_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP1_muts_sortByPos.jld2", "NSP1_muts_sortByPos")
    global NSP2_muts_sortByCt
        NSP2_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP2_muts_sortByCt.jld2", "NSP2_muts_sortByCt")
    global NSP2_muts_sortByPos
        NSP2_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP2_muts_sortByPos.jld2", "NSP2_muts_sortByPos")
    global NSP3_muts_sortByCt
        NSP3_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP3_muts_sortByCt.jld2", "NSP3_muts_sortByCt")
    global NSP3_muts_sortByPos
        NSP3_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP3_muts_sortByPos.jld2", "NSP3_muts_sortByPos")
    global NSP4_muts_sortByCt
        NSP4_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP4_muts_sortByCt.jld2", "NSP4_muts_sortByCt")
    global NSP4_muts_sortByPos
        NSP4_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP4_muts_sortByPos.jld2", "NSP4_muts_sortByPos")
    global NSP5_muts_sortByCt
        NSP5_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP5_muts_sortByCt.jld2", "NSP5_muts_sortByCt")
    global NSP5_muts_sortByPos
        NSP5_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP5_muts_sortByPos.jld2", "NSP5_muts_sortByPos")
    global NSP6_muts_sortByCt
        NSP6_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP6_muts_sortByCt.jld2", "NSP6_muts_sortByCt")
    global NSP6_muts_sortByPos
        NSP6_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP6_muts_sortByPos.jld2", "NSP6_muts_sortByPos")
    global NSP7_muts_sortByCt
        NSP7_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP7_muts_sortByCt.jld2", "NSP7_muts_sortByCt")
    global NSP7_muts_sortByPos
        NSP7_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP7_muts_sortByPos.jld2", "NSP7_muts_sortByPos")
    global NSP8_muts_sortByCt
        NSP8_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP8_muts_sortByCt.jld2", "NSP8_muts_sortByCt")
    global NSP8_muts_sortByPos
        NSP8_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP8_muts_sortByPos.jld2", "NSP8_muts_sortByPos")
    global NSP9_muts_sortByCt
        NSP9_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP9_muts_sortByCt.jld2", "NSP9_muts_sortByCt")
    global NSP9_muts_sortByPos
        NSP9_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP9_muts_sortByPos.jld2", "NSP9_muts_sortByPos")
    global NSP10_muts_sortByCt
        NSP10_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP10_muts_sortByCt.jld2", "NSP10_muts_sortByCt")
    global NSP10_muts_sortByPos
        NSP10_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP10_muts_sortByPos.jld2", "NSP10_muts_sortByPos")
    global NSP11_muts_sortByCt
        NSP11_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP11_muts_sortByCt.jld2", "NSP11_muts_sortByCt")
    global NSP11_muts_sortByPos
        NSP11_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP11_muts_sortByPos.jld2", "NSP11_muts_sortByPos")
    global NSP12_muts_sortByCt
        NSP12_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP12_muts_sortByCt.jld2", "NSP12_muts_sortByCt")
    global NSP12_muts_sortByPos
        NSP12_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP12_muts_sortByPos.jld2", "NSP12_muts_sortByPos")
    global NSP13_muts_sortByCt
        NSP13_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP13_muts_sortByCt.jld2", "NSP13_muts_sortByCt")
    global NSP13_muts_sortByPos
        NSP13_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP13_muts_sortByPos.jld2", "NSP13_muts_sortByPos")
    global NSP14_muts_sortByCt
        NSP14_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP14_muts_sortByCt.jld2", "NSP14_muts_sortByCt")
    global NSP14_muts_sortByPos
        NSP14_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP14_muts_sortByPos.jld2", "NSP14_muts_sortByPos")
    global NSP15_muts_sortByCt
        NSP15_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP15_muts_sortByCt.jld2", "NSP15_muts_sortByCt")
    global NSP15_muts_sortByPos
        NSP15_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP15_muts_sortByPos.jld2", "NSP15_muts_sortByPos")
    global NSP16_muts_sortByCt
        NSP16_muts_sortByCt = load("$(folder_name)/$(folder_name)__NSP16_muts_sortByCt.jld2", "NSP16_muts_sortByCt")
    global NSP16_muts_sortByPos
        NSP16_muts_sortByPos = load("$(folder_name)/$(folder_name)__NSP16_muts_sortByPos.jld2", "NSP16_muts_sortByPos")
    global NSP1_muts_no_dels_sortByCt
        NSP1_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP1_muts_no_dels_sortByCt.jld2", "NSP1_muts_no_dels_sortByCt")
    global NSP1_muts_no_dels_sortByPos
        NSP1_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP1_muts_no_dels_sortByPos.jld2", "NSP1_muts_no_dels_sortByPos")
    global NSP2_muts_no_dels_sortByCt
        NSP2_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP2_muts_no_dels_sortByCt.jld2", "NSP2_muts_no_dels_sortByCt")
    global NSP2_muts_no_dels_sortByPos
        NSP2_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP2_muts_no_dels_sortByPos.jld2", "NSP2_muts_no_dels_sortByPos")
    global NSP3_muts_no_dels_sortByCt
        NSP3_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP3_muts_no_dels_sortByCt.jld2", "NSP3_muts_no_dels_sortByCt")
    global NSP3_muts_no_dels_sortByPos
        NSP3_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP3_muts_no_dels_sortByPos.jld2", "NSP3_muts_no_dels_sortByPos")
    global NSP4_muts_no_dels_sortByCt
        NSP4_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP4_muts_no_dels_sortByCt.jld2", "NSP4_muts_no_dels_sortByCt")
    global NSP4_muts_no_dels_sortByPos
        NSP4_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP4_muts_no_dels_sortByPos.jld2", "NSP4_muts_no_dels_sortByPos")
    global NSP5_muts_no_dels_sortByCt
        NSP5_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP5_muts_no_dels_sortByCt.jld2", "NSP5_muts_no_dels_sortByCt")
    global NSP5_muts_no_dels_sortByPos
        NSP5_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP5_muts_no_dels_sortByPos.jld2", "NSP5_muts_no_dels_sortByPos")
    global NSP6_muts_no_dels_sortByCt
        NSP6_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP6_muts_no_dels_sortByCt.jld2", "NSP6_muts_no_dels_sortByCt")
    global NSP6_muts_no_dels_sortByPos
        NSP6_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP6_muts_no_dels_sortByPos.jld2", "NSP6_muts_no_dels_sortByPos")
    global NSP7_muts_no_dels_sortByCt
        NSP7_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP7_muts_no_dels_sortByCt.jld2", "NSP7_muts_no_dels_sortByCt")
    global NSP7_muts_no_dels_sortByPos
        NSP7_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP7_muts_no_dels_sortByPos.jld2", "NSP7_muts_no_dels_sortByPos")
    global NSP8_muts_no_dels_sortByCt
        NSP8_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP8_muts_no_dels_sortByCt.jld2", "NSP8_muts_no_dels_sortByCt")
    global NSP8_muts_no_dels_sortByPos
        NSP8_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP8_muts_no_dels_sortByPos.jld2", "NSP8_muts_no_dels_sortByPos")
    global NSP9_muts_no_dels_sortByCt
        NSP9_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP9_muts_no_dels_sortByCt.jld2", "NSP9_muts_no_dels_sortByCt")
    global NSP9_muts_no_dels_sortByPos
        NSP9_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP9_muts_no_dels_sortByPos.jld2", "NSP9_muts_no_dels_sortByPos")
    global NSP10_muts_no_dels_sortByCt
        NSP10_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP10_muts_no_dels_sortByCt.jld2", "NSP10_muts_no_dels_sortByCt")
    global NSP10_muts_no_dels_sortByPos
        NSP10_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP10_muts_no_dels_sortByPos.jld2", "NSP10_muts_no_dels_sortByPos")
    global NSP11_muts_no_dels_sortByCt
        NSP11_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP11_muts_no_dels_sortByCt.jld2", "NSP11_muts_no_dels_sortByCt")
    global NSP11_muts_no_dels_sortByPos
        NSP11_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP11_muts_no_dels_sortByPos.jld2", "NSP11_muts_no_dels_sortByPos")
    global NSP12_muts_no_dels_sortByCt
        NSP12_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP12_muts_no_dels_sortByCt.jld2", "NSP12_muts_no_dels_sortByCt")
    global NSP12_muts_no_dels_sortByPos
        NSP12_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP12_muts_no_dels_sortByPos.jld2", "NSP12_muts_no_dels_sortByPos")
    global NSP13_muts_no_dels_sortByCt
        NSP13_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP13_muts_no_dels_sortByCt.jld2", "NSP13_muts_no_dels_sortByCt")
    global NSP13_muts_no_dels_sortByPos
        NSP13_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP13_muts_no_dels_sortByPos.jld2", "NSP13_muts_no_dels_sortByPos")
    global NSP14_muts_no_dels_sortByCt
        NSP14_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP14_muts_no_dels_sortByCt.jld2", "NSP14_muts_no_dels_sortByCt")
    global NSP14_muts_no_dels_sortByPos
        NSP14_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP14_muts_no_dels_sortByPos.jld2", "NSP14_muts_no_dels_sortByPos")
    global NSP15_muts_no_dels_sortByCt
        NSP15_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP15_muts_no_dels_sortByCt.jld2", "NSP15_muts_no_dels_sortByCt")
    global NSP15_muts_no_dels_sortByPos
        NSP15_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP15_muts_no_dels_sortByPos.jld2", "NSP15_muts_no_dels_sortByPos")
    global NSP16_muts_no_dels_sortByCt
        NSP16_muts_no_dels_sortByCt = load("$(folder_name)/$(folder_name)__NSP16_muts_no_dels_sortByCt.jld2", "NSP16_muts_no_dels_sortByCt")
    global NSP16_muts_no_dels_sortByPos
        NSP16_muts_no_dels_sortByPos = load("$(folder_name)/$(folder_name)__NSP16_muts_no_dels_sortByPos.jld2", "NSP16_muts_no_dels_sortByPos")
####################################################################################################
    global all_seqs_set
        all_seqs_set = load("$(folder_name)/$(folder_name)__all_seqs_set.jld2", "all_seqs_set")
    global all_qualifying_seqs
        all_qualifying_seqs = load("$(folder_name)/$(folder_name)__all_qualifying_seqs.jld2", "all_qualifying_seqs")
    global all_qualifying_seqs_set
        all_qualifying_seqs_set = load("$(folder_name)/$(folder_name)__all_qualifying_seqs_set.jld2", "all_qualifying_seqs_set")
    global all_nonqualifying_seqs
        all_nonqualifying_seqs = load("$(folder_name)/$(folder_name)__all_nonqualifying_seqs.jld2", "all_nonqualifying_seqs")
    global all_nonqualifying_seqs_set
        all_nonqualifying_seqs_set = load("$(folder_name)/$(folder_name)__all_nonqualifying_seqs_set.jld2", "all_nonqualifying_seqs_set")
######################################################################################################################################
    global AA_muts_ct_no_dels_no_revs_sort_by_site
        AA_muts_ct_no_dels_no_revs_sort_by_site = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_no_revs_sort_by_site.jld2", "AA_muts_ct_no_dels_no_revs_sort_by_site")
    global AA_muts_ct_no_dels_no_revs_sort_by_seq_ct
        AA_muts_ct_no_dels_no_revs_sort_by_seq_ct = load("$(folder_name)/$(folder_name)__AA_muts_ct_no_dels_no_revs_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_no_revs_sort_by_seq_ct")
    global chronic_search_muts_v2
        chronic_search_muts_v2 = load("$(folder_name)/$(folder_name)__chronic_search_muts_v2.jld2", "chronic_search_muts_v2")
    global avg_AA_subs_per_chr_seq_no_revs
        avg_AA_subs_per_chr_seq_no_revs = load("$(folder_name)/$(folder_name)__avg_AA_subs_per_chr_seq_no_revs.jld2", "avg_AA_subs_per_chr_seq_no_revs")
######################################################################################################################################
    global nuc_muts_ct_no_dels_chr_all_ratio_ct_sort
        nuc_muts_ct_no_dels_chr_all_ratio_ct_sort = load("$(folder_name)/$(folder_name)__nuc_muts_ct_no_dels_chr_all_ratio_ct_sort.jld2", "nuc_muts_ct_no_dels_chr_all_ratio_ct_sort")
    global nuc_muts_ct_no_dels_chr_all_ratio_pos_sort
        nuc_muts_ct_no_dels_chr_all_ratio_pos_sort = load("$(folder_name)/$(folder_name)__nuc_muts_ct_no_dels_chr_all_ratio_pos_sort.jld2", "nuc_muts_ct_no_dels_chr_all_ratio_pos_sort")
    global nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort
        nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort = load("$(folder_name)/$(folder_name)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort")
    global nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort
        nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort = load("$(folder_name)/$(folder_name)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort")
    global avg_nuc_subs_per_chr_seq
        avg_nuc_subs_per_chr_seq = load("$(folder_name)/$(folder_name)__avg_nuc_subs_per_chr_seq.jld2", "avg_nuc_subs_per_chr_seq")
    global avg_nuc_subs_per_circ_seq
        avg_nuc_subs_per_circ_seq = load("$(folder_name)/$(folder_name)__avg_nuc_subs_per_circ_seq.jld2", "avg_nuc_subs_per_circ_seq")
######################################################################################################################################
######################################################################################################################################
    return date_nuc_mut_ct, date_nuc_mut_ct_no_dels, date_AA_mut_ct, date_AA_mut_ct_no_dels, date_AA_mut_ct_pos_only_no_dels, 
    seq_ct_by_year, seq_ct_by_year_month, seq_ct_by_year_month_day, 
    seq_collection_date, seq_date_index, seq_date_tuple, 
###################################################################################################################################
    seq_clade, seq_clade_display, seq_pango, 
    seq_pango_unaliased, seq_clade_ct, seq_pango_ct, seq_pango_unaliased_ct, 
###################################################################################################################################
    too_many_reversions, 
###################################################################################################################################
    seq_nuc_muts, seq_nuc_del_ranges, seq_nuc_dropout, seq_nuc_muts_WT, seq_nuc_del_ranges_WT, 
################################
    seq_AA_insertions_WT, seq_nuc_insertions_WT,
################################
    seq_AA_muts, seq_AA_muts_no_dels, seq_AA_muts_pos_only, seq_AA_dels, seq_mixed_AA_muts, 
    seq_AA_muts_WT, seq_AA_muts_WT_pos_only, seq_AA_dels_WT, seq_AA_muts_pos_only_no_dels, 
########################################################
    nuc_muts_seq, nuc_muts_seq_WT, 
    nuc_dels_seq, nuc_dels_seq_WT, 
    AA_muts_seq, AA_muts_seq_pos_only, AA_muts_seq_WT, AA_muts_seq_WT_pos_only, AA_muts_seq_pos_only_no_dels, 
    AA_dels_seq, AA_dels_seq_WT, 
######################################################## 
    seq_unknown_AA, seq_unknown_AA_ranges, seq_mixed_nucs, 
############################
    nuc_dels_ct, nuc_muts_ct, nuc_muts_ct_no_dels, 
###############
    AA_dels_ct, AA_muts_ct, AA_muts_ct_no_dels, AA_muts_ct_pos_only, AA_muts_ct_pos_only_no_dels, AA_muts_ct_no_dels_no_revs, 
############################
    nuc_muts_ct_sort_by_site, nuc_muts_ct_sort_by_seq_ct, 
############################ 
    AA_muts_ct_sort_by_site, AA_muts_ct_sort_by_seq_ct, AA_muts_ct_no_dels_sort_by_site, AA_muts_ct_no_dels_no_revs_sort_by_site, 
    AA_muts_ct_no_dels_no_revs_sort_by_seq_ct, AA_muts_ct_no_dels_sort_by_seq_ct,  AA_muts_ct_pos_only_sort_by_site, 
    AA_muts_ct_pos_only_sort_by_seq_ct, AA_muts_ct_pos_only_no_dels_sort_by_site, AA_muts_ct_pos_only_no_dels_sort_by_seq_ct, 
###################################################################################################################################
################################################################################################################################### 
    nuc_muts_ct_adj, nuc_muts_ct_adj_no_dels, nuc_muts_ct_adj_score, nuc_muts_ct_adj_score_no_dels,   
    nuc_muts_ct_adj_sort_by_seq_ct, nuc_muts_ct_adj_no_dels_sort_by_site, nuc_muts_ct_adj_no_dels_sort_by_seq_ct,
    nuc_muts_ct_adj_sort_by_site, nuc_muts_ct_adj_score_no_dels_sort_by_score, AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score, 
    nuc_muts_ct_adj_score_sort_by_site, nuc_muts_ct_adj_score_sort_by_score, nuc_muts_ct_adj_score_no_dels_sort_by_site, 
    AA_muts_ct_adj, AA_muts_ct_adj_no_dels, AA_muts_ct_pos_only_adj, AA_muts_ct_pos_only_adj_no_dels, 
    AA_muts_ct_adj_score, AA_muts_ct_adj_score_no_dels, AA_muts_ct_pos_only_adj_score, AA_muts_ct_pos_only_adj_score_no_dels,
    AA_muts_ct_adj_sort_by_site, AA_muts_ct_adj_sort_by_seq_ct, AA_muts_ct_adj_no_dels_sort_by_site,
    AA_muts_ct_adj_no_dels_sort_by_seq_ct, AA_muts_ct_adj_score_sort_by_score, AA_muts_ct_adj_score_sort_by_site, 
    AA_muts_ct_adj_score_no_dels_sort_by_score, AA_muts_ct_adj_score_no_dels_sort_by_site, 
    AA_muts_ct_pos_only_adj_sort_by_site, AA_muts_ct_pos_only_adj_sort_by_seq_ct, AA_muts_ct_pos_only_adj_no_dels_sort_by_site, 
    AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct, AA_muts_ct_pos_only_adj_score_sort_by_site, 
    AA_muts_ct_pos_only_adj_score_sort_by_score, AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site, 
####################################################################################################################################
    AA_muts_ct_chr_all_ratio, AA_muts_ct_no_dels_chr_all_ratio, AA_muts_ct_no_dels_no_revs_chr_all_ratio, AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio, 
########################
    AA_muts_ct_chr_all_ratio_ct_sort, AA_muts_ct_chr_all_ratio_pos_sort, 
    AA_muts_ct_no_dels_chr_all_ratio_ct_sort, AA_muts_ct_no_dels_chr_all_ratio_pos_sort, 
    AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort, AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort, 
###################################################### 
    nuc_muts_ct_no_dels_chr_all_ratio, nuc_muts_ct_no_dels_no_revs_chr_all_ratio, nuc_muts_ct_pos_only_no_dels_chr_all_ratio, nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio, 
#######################
    nuc_muts_ct_no_dels_chr_all_ratio_ct_sort, nuc_muts_ct_no_dels_chr_all_ratio_pos_sort, 
    nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort, nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort, 
####################################################################################################################################
    NSP_muts, NSP_muts_sortByCt_Arr, NSP_muts_sortByPos_Arr, NSP_muts_no_dels_sortByCt_Arr, NSP_muts_no_dels_sortByPos_Arr, 
    NSP1_muts_sortByCt, NSP2_muts_sortByCt, NSP3_muts_sortByCt, NSP4_muts_sortByCt, NSP5_muts_sortByCt, NSP6_muts_sortByCt,
    NSP7_muts_sortByCt, NSP8_muts_sortByCt, NSP9_muts_sortByCt, NSP10_muts_sortByCt, NSP11_muts_sortByCt, NSP12_muts_sortByCt, 
    NSP13_muts_sortByCt, NSP14_muts_sortByCt, NSP15_muts_sortByCt, NSP16_muts_sortByCt, NSP1_muts_sortByPos, NSP2_muts_sortByPos, 
    NSP3_muts_sortByPos, NSP4_muts_sortByPos, NSP5_muts_sortByPos, NSP6_muts_sortByPos, NSP7_muts_sortByPos, NSP8_muts_sortByPos, 
    NSP9_muts_sortByPos, NSP10_muts_sortByPos, NSP11_muts_sortByPos, NSP12_muts_sortByPos, NSP13_muts_sortByPos, NSP14_muts_sortByPos, 
    NSP15_muts_sortByPos, NSP16_muts_sortByPos, NSP1_muts_no_dels_sortByCt, NSP2_muts_no_dels_sortByCt, NSP3_muts_no_dels_sortByCt, 
    NSP4_muts_no_dels_sortByCt, NSP5_muts_no_dels_sortByCt, NSP6_muts_no_dels_sortByCt, NSP7_muts_no_dels_sortByCt, 
    NSP8_muts_no_dels_sortByCt, NSP9_muts_no_dels_sortByCt, NSP10_muts_no_dels_sortByCt, NSP11_muts_no_dels_sortByCt, 
    NSP12_muts_no_dels_sortByCt, NSP13_muts_no_dels_sortByCt, NSP14_muts_no_dels_sortByCt, NSP15_muts_no_dels_sortByCt, 
    NSP16_muts_no_dels_sortByCt, NSP1_muts_no_dels_sortByPos, NSP2_muts_no_dels_sortByPos, NSP3_muts_no_dels_sortByPos, 
    NSP4_muts_no_dels_sortByPos, NSP5_muts_no_dels_sortByPos, NSP6_muts_no_dels_sortByPos, NSP7_muts_no_dels_sortByPos, 
    NSP8_muts_no_dels_sortByPos, NSP9_muts_no_dels_sortByPos, NSP10_muts_no_dels_sortByPos, NSP11_muts_no_dels_sortByPos, 
    NSP12_muts_no_dels_sortByPos, NSP13_muts_no_dels_sortByPos, NSP14_muts_no_dels_sortByPos, NSP15_muts_no_dels_sortByPos, 
    NSP16_muts_no_dels_sortByPos,
####################################################################################################################################
    chronic_search_muts, chronic_search_muts_v2, 
####################################################################################################################################
    avg_AA_subs_per_circ_seq, avg_nuc_subs_per_circ_seq, avg_AA_subs_per_chr_seq, avg_nuc_subs_per_chr_seq,
####################################################################################################################################
    nuc_muts_rep_seq_grps, nuc_dels_rep_seq_grps, 
    nuc_muts_rep_seq_grps_no_dels, rep_seq_grps_dels, rep_seq_grps_muts_no_dels, rep_seq_grps_del_ranges_ct, rep_seq_grps_nuc_dropout, 
    rep_seq_grps_mixed_nucs, rep_seq_grps_AA_dels, rep_seq_grps_AA_no_dels, AA_muts_rep_seq_grps, AA_dels_rep_seq_grps, 
    AA_muts_rep_seq_grps_no_dels, AA_muts_rep_seq_grps_pos_only,  
    rep_seq_grps_nuc_muts_WT, rep_seq_grps_nuc_dels_WT, rep_seq_grps_AA_muts_WT, rep_seq_grps_AA_dels_WT, nuc_muts_rep_seq_grps_WT, 
    nuc_dels_rep_seq_grps_WT, AA_muts_rep_seq_grps_WT, AA_dels_rep_seq_grps_WT, rep_seq_grps_AA_muts_WT_pos_only, 
    AA_muts_rep_seq_grps_WT_pos_only, rep_seq_grps_pango, rep_seq_grps_pango_unaliased, excluded_pos, excluded_AA, all_seqs, rep_seqs, 
    non_rep_seqs, rep_seq_grps_seqs, rep_seq_grps_muts, rep_seq_grps_clade, rep_seq_grps_AA, rep_seq_grps_AA_pos_only, 
    rep_seq_grps_AA_pos_only_no_dels, non_rep_seqs_AA, non_rep_seqs_AA_pos_only, non_rep_seqs_AA_pos_only_no_dels, 
####################################################################################################################################
    rep_seq_grps_maxmut_nuc_muts_WT, rep_seq_grps_maxmut_nuc_dels_WT, 
    rep_seq_grps_maxmut_AA_muts_WT, rep_seq_grps_maxmut_AA_dels_WT, rep_seq_grps_maxmut_AA_muts_WT_pos_only,
    rep_seq_grps_maxmut_seqs, rep_seq_grps_maxmut_nuc, rep_seq_grps_maxmut_nuc_no_dels, rep_seq_grps_maxmut_dels, 
    rep_seq_grps_maxmut_nuc_dropout, rep_seq_grps_maxmut_mixed_nucs, rep_seq_grps_maxmut_AA, rep_seq_grps_maxmut_AA_no_dels, 
    rep_seq_grps_maxmut_AA_dels, rep_seq_grps_maxmut_AA_pos_only, rep_seq_grps_maxmut_AA_pos_only_no_dels, 
    rep_seq_grps_maxmut_unknown_AA, rep_seq_grps_maxmut_unknown_AA_ranges, rep_seq_grps_maxmut_mixed_AA_muts, rep_seq_grps_maxmut_del_ranges_ct,
####################################################################################################################################
    gene_mut_density_sort_by_gene, gene_mut_density_sort_by_density, domain_mut_density_sort_by_gene, domain_mut_density_sort_by_density, 
    gene_mut_density, domain_mut_density, 
####################################################################################################################################
    all_seqs_set, all_qualifying_seqs, all_qualifying_seqs_set, all_nonqualifying_seqs, all_nonqualifying_seqs_set
####################################################################################################################################
    total_chr_AA_subs, total_nuc_revs_seq, seq_nuc_total_revs, total_AA_revs_seq, seq_AA_total_revs, seq_AA_revs
end
# REMOVED: rep_seq_grps_unknown_AA, rep_seq_grps_mixed_AA_muts,
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
function gene_sub_pos_ranks_chr(gene_array::Vector{String}, gene_AA_lengths::Dict{String, Int}, AA_muts_ct_pos_only_no_dels::Dict{String, Int} )
    ORF1a_pos_ct = Dict{String, Int}()
    ORF1b_pos_ct = Dict{String, Int}()
    S_pos_ct = Dict{String, Int}()
    ORF3a_pos_ct = Dict{String, Int}()
    E_pos_ct = Dict{String, Int}()
    M_pos_ct = Dict{String, Int}()
    ORF6_pos_ct = Dict{String, Int}()
    ORF7a_pos_ct = Dict{String, Int}()
    ORF7b_pos_ct = Dict{String, Int}()
    ORF8_pos_ct = Dict{String, Int}()
    N_pos_ct = Dict{String, Int}()
    ORF9b_pos_ct = Dict{String, Int}()
    gene_pos_ct_dict_arr = [ORF1a_pos_ct, ORF1b_pos_ct, S_pos_ct, ORF3a_pos_ct, E_pos_ct, M_pos_ct, ORF6_pos_ct, ORF7a_pos_ct, ORF7b_pos_ct, ORF8_pos_ct, N_pos_ct, ORF9b_pos_ct]
######################################################
    ORF1a_pos_ct_v1 = Dict{Int, Int}()
    ORF1b_pos_ct_v1 = Dict{Int, Int}()
    S_pos_ct_v1 = Dict{Int, Int}()
    ORF3a_pos_ct_v1 = Dict{Int, Int}()
    E_pos_ct_v1 = Dict{Int, Int}()
    M_pos_ct_v1 = Dict{Int, Int}()
    ORF6_pos_ct_v1 = Dict{Int, Int}()
    ORF7a_pos_ct_v1 = Dict{Int, Int}()
    ORF7b_pos_ct_v1 = Dict{Int, Int}()
    ORF8_pos_ct_v1 = Dict{Int, Int}()
    N_pos_ct_v1 = Dict{Int, Int}()
    ORF9b_pos_ct_v1 = Dict{Int, Int}()
    gene_pos_ct_v1_dict_arr = [ORF1a_pos_ct_v1, ORF1b_pos_ct_v1, S_pos_ct_v1, ORF3a_pos_ct_v1, E_pos_ct_v1, M_pos_ct_v1, ORF6_pos_ct_v1, ORF7a_pos_ct_v1, ORF7b_pos_ct_v1, ORF8_pos_ct_v1, N_pos_ct_v1, ORF9b_pos_ct_v1]
######################################################
    for i in 1:length(gene_pos_ct_dict_arr)
        dict = gene_pos_ct_dict_arr[i]
        dict_v1 = gene_pos_ct_v1_dict_arr[i]
        gene = gene_array[i]
        gene_len = gene_AA_lengths[gene]
        for j in 1:gene_len
            site = gene*":"*"$(j)"
            dict[site] = 0
            dict_v1[j] = 0
        end
    end
#############################################################
#############################################################
    for (mut, ct) in AA_muts_ct_pos_only_no_dels
        pos = aa_pos_comprehensive_dict[mut]
        if aa_gene_comprehensive_dict[mut] == "ORF1a"
            ORF1a_pos_ct[mut] = ct
            ORF1a_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF1b"
            ORF1b_pos_ct[mut] = ct
            ORF1b_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "S"
            S_pos_ct[mut] = ct
            S_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF3a"
            ORF3a_pos_ct[mut] = ct
            ORF3a_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "E"
            E_pos_ct[mut] = ct
            E_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "M"
            M_pos_ct[mut] = ct
            M_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF6"
            ORF6_pos_ct[mut] = ct
            ORF6_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7a"
            ORF7a_pos_ct[mut] = ct
            ORF7a_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7b"
            ORF7b_pos_ct[mut] = ct
            ORF7b_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF8"
            ORF8_pos_ct[mut] = ct
            ORF8_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "N"
            N_pos_ct[mut] = ct
            N_pos_ct_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF9b"
            ORF9b_pos_ct[mut] = ct
            ORF9b_pos_ct_v1[pos] = ct
        end
    end
######################### 
    global ORF1a_pos_ct_sort_by_pos = sort(collect(ORF1a_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF1b_pos_ct_sort_by_pos = sort(collect(ORF1b_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global S_pos_ct_sort_by_pos = sort(collect(S_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF3a_pos_ct_sort_by_pos = sort(collect(ORF3a_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global E_pos_ct_sort_by_pos = sort(collect(E_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global M_pos_ct_sort_by_pos = sort(collect(M_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF6_pos_ct_sort_by_pos = sort(collect(ORF6_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF7a_pos_ct_sort_by_pos = sort(collect(ORF7a_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF7b_pos_ct_sort_by_pos = sort(collect(ORF7b_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF8_pos_ct_sort_by_pos = sort(collect(ORF8_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global N_pos_ct_sort_by_pos = sort(collect(N_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF9b_pos_ct_sort_by_pos = sort(collect(ORF9b_pos_ct), by = x -> aa_pos_comprehensive_dict[x[1]])
#########################
    global ORF1a_pos_ct_sort_by_ct = sort(collect(ORF1a_pos_ct), by = x -> x[2], rev=true)
    global ORF1b_pos_ct_sort_by_ct = sort(collect(ORF1b_pos_ct), by = x -> x[2], rev=true)
    global S_pos_ct_sort_by_ct = sort(collect(S_pos_ct), by = x -> x[2], rev=true)
    global ORF3a_pos_ct_sort_by_ct = sort(collect(ORF3a_pos_ct), by = x -> x[2], rev=true)
    global E_pos_ct_sort_by_ct = sort(collect(E_pos_ct), by = x -> x[2], rev=true)
    global M_pos_ct_sort_by_ct = sort(collect(M_pos_ct), by = x -> x[2], rev=true)
    global ORF6_pos_ct_sort_by_ct = sort(collect(ORF6_pos_ct), by = x -> x[2], rev=true)
    global ORF7a_pos_ct_sort_by_ct = sort(collect(ORF7a_pos_ct), by = x -> x[2], rev=true)
    global ORF7b_pos_ct_sort_by_ct = sort(collect(ORF7b_pos_ct), by = x -> x[2], rev=true)
    global ORF8_pos_ct_sort_by_ct = sort(collect(ORF8_pos_ct), by = x -> x[2], rev=true)
    global N_pos_ct_sort_by_ct = sort(collect(N_pos_ct), by = x -> x[2], rev=true)
    global ORF9b_pos_ct_sort_by_ct = sort(collect(ORF9b_pos_ct), by = x -> x[2], rev=true)
#########################
    global ORF1a_pos_ct_sort_by_pos_v1 = sort(collect(ORF1a_pos_ct_v1), by = x -> x[1])
    global ORF1b_pos_ct_sort_by_pos_v1 = sort(collect(ORF1b_pos_ct_v1), by = x -> x[1])
    global S_pos_ct_sort_by_pos_v1 = sort(collect(S_pos_ct_v1), by = x -> x[1])
    global ORF3a_pos_ct_sort_by_pos_v1 = sort(collect(ORF3a_pos_ct_v1), by = x -> x[1])
    global E_pos_ct_sort_by_pos_v1 = sort(collect(E_pos_ct_v1), by = x -> x[1])
    global M_pos_ct_sort_by_pos_v1 = sort(collect(M_pos_ct_v1), by = x -> x[1])
    global ORF6_pos_ct_sort_by_pos_v1 = sort(collect(ORF6_pos_ct_v1), by = x -> x[1])
    global ORF7a_pos_ct_sort_by_pos_v1 = sort(collect(ORF7a_pos_ct_v1), by = x -> x[1])
    global ORF7b_pos_ct_sort_by_pos_v1 = sort(collect(ORF7b_pos_ct_v1), by = x -> x[1])
    global ORF8_pos_ct_sort_by_pos_v1 = sort(collect(ORF8_pos_ct_v1), by = x -> x[1])
    global N_pos_ct_sort_by_pos_v1 = sort(collect(N_pos_ct_v1), by = x -> x[1])
    global ORF9b_pos_ct_sort_by_pos_v1 = sort(collect(ORF9b_pos_ct_v1), by = x -> x[1])
#########################
    global ORF1a_pos_ct_sort_by_ct_v1 = sort(collect(ORF1a_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF1b_pos_ct_sort_by_ct_v1 = sort(collect(ORF1b_pos_ct_v1), by = x -> x[2], rev=true)
    global S_pos_ct_sort_by_ct_v1 = sort(collect(S_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF3a_pos_ct_sort_by_ct_v1 = sort(collect(ORF3a_pos_ct_v1), by = x -> x[2], rev=true)
    global E_pos_ct_sort_by_ct_v1 = sort(collect(E_pos_ct_v1), by = x -> x[2], rev=true)
    global M_pos_ct_sort_by_ct_v1 = sort(collect(M_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF6_pos_ct_sort_by_ct_v1 = sort(collect(ORF6_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF7a_pos_ct_sort_by_ct_v1 = sort(collect(ORF7a_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF7b_pos_ct_sort_by_ct_v1 = sort(collect(ORF7b_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF8_pos_ct_sort_by_ct_v1 = sort(collect(ORF8_pos_ct_v1), by = x -> x[2], rev=true)
    global N_pos_ct_sort_by_ct_v1 = sort(collect(N_pos_ct_v1), by = x -> x[2], rev=true)
    global ORF9b_pos_ct_sort_by_ct_v1 = sort(collect(ORF9b_pos_ct_v1), by = x -> x[2], rev=true)
end
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
function gene_sub_ranks_chr(gene_array::Vector{String}, sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_no_dels::Dict{String, Int} )
    ORF1a_ct = Dict{String, Int}()
    ORF1b_ct = Dict{String, Int}()
    S_ct = Dict{String, Int}()
    ORF3a_ct = Dict{String, Int}()
    E_ct = Dict{String, Int}()
    M_ct = Dict{String, Int}()
    ORF6_ct = Dict{String, Int}()
    ORF7a_ct = Dict{String, Int}()
    ORF7b_ct = Dict{String, Int}()
    ORF8_ct = Dict{String, Int}()
    N_ct = Dict{String, Int}()
    ORF9b_ct = Dict{String, Int}()
#############################################################
    gene_ct_dict_arr = [ORF1a_ct, ORF1b_ct, S_ct, ORF3a_ct, E_ct, M_ct, ORF6_ct, ORF7a_ct, ORF7b_ct, ORF8_ct, N_ct, ORF9b_ct] 
#############################################################
#                                        Gene      AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
# sub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()
    for i in 1:length(gene_array)
        gene = gene_array[i]
        gene_ct_dict = gene_ct_dict_arr[i]
        for (AAsite, mut_type_set) in sub_types_at_every_site_combined[gene]
            for mut_type in mut_type_set
                sub = gene*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                gene_ct_dict[sub] = 0
            end
        end
    end
#############################################################
    for (mut, ct) in AA_muts_ct_no_dels
        if aa_gene_comprehensive_dict[mut] == "ORF1a"
            ORF1a_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF1b"
            ORF1b_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "S"
            S_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF3a"
            ORF3a_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "E"
            E_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "M"
            M_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF6"
            ORF6_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7a"
            ORF7a_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7b"
            ORF7b_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF8"
            ORF8_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "N"
            N_ct[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF9b"
            ORF9b_ct[mut] = ct
        end
    end
    fin_sortkey(m) = (aa_pos_comprehensive_dict[m], refAA_comprehensive_dict[m]*qryAA_comprehensive_dict[m])
########################################## 
    global ORF1a_ct_sort_by_pos = sort(collect(ORF1a_ct), by = x -> fin_sortkey(x[1]))
    global ORF1b_ct_sort_by_pos = sort(collect(ORF1b_ct), by = x -> fin_sortkey(x[1]))
    global S_ct_sort_by_pos = sort(collect(S_ct), by = x -> fin_sortkey(x[1]))
    global ORF3a_ct_sort_by_pos = sort(collect(ORF3a_ct), by = x -> fin_sortkey(x[1]))
    global E_ct_sort_by_pos = sort(collect(E_ct), by = x -> fin_sortkey(x[1]))
    global M_ct_sort_by_pos = sort(collect(M_ct), by = x -> fin_sortkey(x[1]))
    global ORF6_ct_sort_by_pos = sort(collect(ORF6_ct), by = x -> fin_sortkey(x[1]))
    global ORF7a_ct_sort_by_pos = sort(collect(ORF7a_ct), by = x -> fin_sortkey(x[1]))
    global ORF7b_ct_sort_by_pos = sort(collect(ORF7b_ct), by = x -> fin_sortkey(x[1]))
    global ORF8_ct_sort_by_pos = sort(collect(ORF8_ct), by = x -> fin_sortkey(x[1]))
    global N_ct_sort_by_pos = sort(collect(N_ct), by = x -> fin_sortkey(x[1]))
    global ORF9b_ct_sort_by_pos = sort(collect(ORF9b_ct), by = x -> fin_sortkey(x[1]))
##########################################
    global ORF1a_ct_sort_by_ct = sort(collect(ORF1a_ct), by = x -> x[2], rev=true)
    global ORF1b_ct_sort_by_ct = sort(collect(ORF1b_ct), by = x -> x[2], rev=true)
    global S_ct_sort_by_ct = sort(collect(S_ct), by = x -> x[2], rev=true)
    global ORF3a_ct_sort_by_ct = sort(collect(ORF3a_ct), by = x -> x[2], rev=true)
    global E_ct_sort_by_ct = sort(collect(E_ct), by = x -> x[2], rev=true)
    global M_ct_sort_by_ct = sort(collect(M_ct), by = x -> x[2], rev=true)
    global ORF6_ct_sort_by_ct = sort(collect(ORF6_ct), by = x -> x[2], rev=true)
    global ORF7a_ct_sort_by_ct = sort(collect(ORF7a_ct), by = x -> x[2], rev=true)
    global ORF7b_ct_sort_by_ct = sort(collect(ORF7b_ct), by = x -> x[2], rev=true)
    global ORF8_ct_sort_by_ct = sort(collect(ORF8_ct), by = x -> x[2], rev=true)
    global N_ct_sort_by_ct = sort(collect(N_ct), by = x -> x[2], rev=true)
    global ORF9b_ct_sort_by_ct = sort(collect(ORF9b_ct), by = x -> x[2], rev=true)
end
#####################################################################################################################################
function NSP_sub_ranks_chr(NSP_array::Vector{String}, NSP_sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_no_dels::Dict{String, Int} )
    NSP1_ct = Dict{String, Int}()
    NSP2_ct = Dict{String, Int}()
    NSP3_ct = Dict{String, Int}()
    NSP4_ct = Dict{String, Int}()
    NSP5_ct = Dict{String, Int}()
    NSP6_ct = Dict{String, Int}()
    NSP7_ct = Dict{String, Int}()
    NSP8_ct = Dict{String, Int}()
    NSP9_ct = Dict{String, Int}()
    NSP10_ct = Dict{String, Int}()
    NSP11_ct = Dict{String, Int}()
    NSP12_ct = Dict{String, Int}()
    NSP13_ct = Dict{String, Int}()
    NSP14_ct = Dict{String, Int}()
    NSP15_ct = Dict{String, Int}()
    NSP16_ct = Dict{String, Int}()
#############################################################
    NSP_ct_dict_arr = [NSP1_ct, NSP2_ct, NSP3_ct, NSP4_ct, NSP5_ct, NSP6_ct, NSP7_ct, NSP8_ct, NSP9_ct, NSP10_ct, NSP11_ct, NSP12_ct, NSP13_ct, NSP14_ct, NSP15_ct, NSP16_ct]
#                                              NSP       AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
# NSP_sub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()   
    for i in 1:length(NSP_array)
        NSP = NSP_array[i]
        NSP_ct_dict = NSP_ct_dict_arr[i]
        for (AAsite, mut_type_set) in NSP_sub_types_at_every_site_combined[NSP]
            for mut_type in mut_type_set
                sub = NSP*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                NSP_ct_dict[sub] = 0
            end
        end
    end   
    for (sub, ct) in AA_muts_ct_no_dels
        gene = aa_gene_comprehensive_dict[sub]
        if gene == "ORF1a" || gene == "ORF1b"
            AA_site = aa_pos_comprehensive_dict[sub]
            if AA_site < 4402
                NSP_sub = ORF1abMut_to_NSP(sub)
                NSP = NSP_muts_gene_dict[NSP_sub]
                NSP_pos = NSP_muts_pos_dict[NSP_sub]
                NSP_num = parse(Int, split(NSP, "P")[2])
                NSP_dict = NSP_ct_dict_arr[NSP_num]
                NSP_dict[NSP_sub] = ct
            end
        end
    end
    fin_sortkey(m) = (aa_pos_comprehensive_dict[m], NSP_ref_AA_dict[m]*NSP_qry_AA_dict[m])
#############################
    global NSP1_ct_sort_by_pos = sort(collect(NSP1_ct), by = x -> fin_sortkey(x[1]))
    global NSP2_ct_sort_by_pos = sort(collect(NSP2_ct), by = x -> fin_sortkey(x[1]))
    global NSP3_ct_sort_by_pos = sort(collect(NSP3_ct), by = x -> fin_sortkey(x[1]))
    global NSP4_ct_sort_by_pos = sort(collect(NSP4_ct), by = x -> fin_sortkey(x[1]))
    global NSP5_ct_sort_by_pos = sort(collect(NSP5_ct), by = x -> fin_sortkey(x[1]))
    global NSP6_ct_sort_by_pos = sort(collect(NSP6_ct), by = x -> fin_sortkey(x[1]))
    global NSP7_ct_sort_by_pos = sort(collect(NSP7_ct), by = x -> fin_sortkey(x[1]))
    global NSP8_ct_sort_by_pos = sort(collect(NSP8_ct), by = x -> fin_sortkey(x[1]))
    global NSP9_ct_sort_by_pos = sort(collect(NSP9_ct), by = x -> fin_sortkey(x[1]))
    global NSP10_ct_sort_by_pos = sort(collect(NSP10_ct), by = x -> fin_sortkey(x[1]))
    global NSP11_ct_sort_by_pos = sort(collect(NSP11_ct), by = x -> fin_sortkey(x[1]))
    global NSP12_ct_sort_by_pos = sort(collect(NSP12_ct), by = x -> fin_sortkey(x[1]))
    global NSP13_ct_sort_by_pos = sort(collect(NSP13_ct), by = x -> fin_sortkey(x[1]))
    global NSP14_ct_sort_by_pos = sort(collect(NSP14_ct), by = x -> fin_sortkey(x[1]))
    global NSP15_ct_sort_by_pos = sort(collect(NSP15_ct), by = x -> fin_sortkey(x[1]))
    global NSP16_ct_sort_by_pos = sort(collect(NSP16_ct), by = x -> fin_sortkey(x[1]))
#############################
    global NSP1_ct_sort_by_ct = sort(collect(NSP1_ct), by = x -> x[2], rev=true)
    global NSP2_ct_sort_by_ct = sort(collect(NSP2_ct), by = x -> x[2], rev=true)
    global NSP3_ct_sort_by_ct = sort(collect(NSP3_ct), by = x -> x[2], rev=true)
    global NSP4_ct_sort_by_ct = sort(collect(NSP4_ct), by = x -> x[2], rev=true)
    global NSP5_ct_sort_by_ct = sort(collect(NSP5_ct), by = x -> x[2], rev=true)
    global NSP6_ct_sort_by_ct = sort(collect(NSP6_ct), by = x -> x[2], rev=true)
    global NSP7_ct_sort_by_ct = sort(collect(NSP7_ct), by = x -> x[2], rev=true)
    global NSP8_ct_sort_by_ct = sort(collect(NSP8_ct), by = x -> x[2], rev=true)
    global NSP9_ct_sort_by_ct = sort(collect(NSP9_ct), by = x -> x[2], rev=true)
    global NSP10_ct_sort_by_ct = sort(collect(NSP10_ct), by = x -> x[2], rev=true)
    global NSP11_ct_sort_by_ct = sort(collect(NSP11_ct), by = x -> x[2], rev=true)
    global NSP12_ct_sort_by_ct = sort(collect(NSP12_ct), by = x -> x[2], rev=true)
    global NSP13_ct_sort_by_ct = sort(collect(NSP13_ct), by = x -> x[2], rev=true)
    global NSP14_ct_sort_by_ct = sort(collect(NSP14_ct), by = x -> x[2], rev=true)
    global NSP15_ct_sort_by_ct = sort(collect(NSP15_ct), by = x -> x[2], rev=true)
    global NSP16_ct_sort_by_ct = sort(collect(NSP16_ct), by = x -> x[2], rev=true)
end
#####################################################################################################################################
function NSP_sub_pos_ranks_chr(NSP_array::Vector{String}, NSP_sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_pos_only_no_dels::Dict{String, Int} )
    NSP1_pos_ct = Dict{String, Int}()
    NSP2_pos_ct = Dict{String, Int}()
    NSP3_pos_ct = Dict{String, Int}()
    NSP4_pos_ct = Dict{String, Int}()
    NSP5_pos_ct = Dict{String, Int}()
    NSP6_pos_ct = Dict{String, Int}()
    NSP7_pos_ct = Dict{String, Int}()
    NSP8_pos_ct = Dict{String, Int}()
    NSP9_pos_ct = Dict{String, Int}()
    NSP10_pos_ct = Dict{String, Int}()
    NSP11_pos_ct = Dict{String, Int}()
    NSP12_pos_ct = Dict{String, Int}()
    NSP13_pos_ct = Dict{String, Int}()
    NSP14_pos_ct = Dict{String, Int}()
    NSP15_pos_ct = Dict{String, Int}()
    NSP16_pos_ct = Dict{String, Int}()
    NSP_pos_ct_array = [NSP1_pos_ct, NSP2_pos_ct, NSP3_pos_ct, NSP4_pos_ct, NSP5_pos_ct, NSP6_pos_ct, NSP7_pos_ct, NSP8_pos_ct, NSP9_pos_ct, NSP10_pos_ct, NSP11_pos_ct, NSP12_pos_ct, NSP13_pos_ct, NSP14_pos_ct, NSP15_pos_ct, NSP16_pos_ct]
##########################################
    NSP1_pos_ct_v1 = Dict{Int, Int}()
    NSP2_pos_ct_v1 = Dict{Int, Int}()
    NSP3_pos_ct_v1 = Dict{Int, Int}()
    NSP4_pos_ct_v1 = Dict{Int, Int}()
    NSP5_pos_ct_v1 = Dict{Int, Int}()
    NSP6_pos_ct_v1 = Dict{Int, Int}()
    NSP7_pos_ct_v1 = Dict{Int, Int}()
    NSP8_pos_ct_v1 = Dict{Int, Int}()
    NSP9_pos_ct_v1 = Dict{Int, Int}()
    NSP10_pos_ct_v1 = Dict{Int, Int}()
    NSP11_pos_ct_v1 = Dict{Int, Int}()
    NSP12_pos_ct_v1 = Dict{Int, Int}()
    NSP13_pos_ct_v1 = Dict{Int, Int}()
    NSP14_pos_ct_v1 = Dict{Int, Int}()
    NSP15_pos_ct_v1 = Dict{Int, Int}()
    NSP16_pos_ct_v1 = Dict{Int, Int}()
    NSP_pos_ct_v1_array = [NSP1_pos_ct_v1, NSP2_pos_ct_v1, NSP3_pos_ct_v1, NSP4_pos_ct_v1, NSP5_pos_ct_v1, NSP6_pos_ct_v1, NSP7_pos_ct_v1, NSP8_pos_ct_v1, NSP9_pos_ct_v1, NSP10_pos_ct_v1, NSP11_pos_ct_v1, NSP12_pos_ct_v1, NSP13_pos_ct_v1, NSP14_pos_ct_v1, NSP15_pos_ct_v1, NSP16_pos_ct_v1]
#######################################################
    NSP_pos_ct = Dict{String, Int}()
    for i in 1:length(NSP_pos_ct_array)
        NSP_dict = NSP_pos_ct_array[i]
        NSP_dict_v1 = NSP_pos_ct_v1_array[i]
        NSP = NSP_array[i]
        NSP_len = NSP_AA_size[NSP]
        for j in 1:NSP_len
            NSP_pos = NSP*":"*"$(j)"
            NSP_dict[NSP_pos] = 0
            NSP_dict_v1[j] = 0
            NSP_pos_ct[NSP_pos] = 0
        end
    end
##########################################################################################################################
##########################################################################################################################
    NSP1_ct = Dict{String, Int}()
    NSP2_ct = Dict{String, Int}()
    NSP3_ct = Dict{String, Int}()
    NSP4_ct = Dict{String, Int}()
    NSP5_ct = Dict{String, Int}()
    NSP6_ct = Dict{String, Int}()
    NSP7_ct = Dict{String, Int}()
    NSP8_ct = Dict{String, Int}()
    NSP9_ct = Dict{String, Int}()
    NSP10_ct = Dict{String, Int}()
    NSP11_ct = Dict{String, Int}()
    NSP12_ct = Dict{String, Int}()
    NSP13_ct = Dict{String, Int}()
    NSP14_ct = Dict{String, Int}()
    NSP15_ct = Dict{String, Int}()
    NSP16_ct = Dict{String, Int}()
#############################################################
    NSP_ct_dict_arr = [NSP1_ct, NSP2_ct, NSP3_ct, NSP4_ct, NSP5_ct, NSP6_ct, NSP7_ct, NSP8_ct, NSP9_ct, NSP10_ct, NSP11_ct, NSP12_ct, NSP13_ct, NSP14_ct, NSP15_ct, NSP16_ct]
    for i in 1:length(NSP_array)
        NSP = NSP_array[i]
        NSP_ct_dict = NSP_ct_dict_arr[i]
        for (AAsite, mut_type_set) in NSP_sub_types_at_every_site_combined[NSP]
            for mut_type in mut_type_set
                sub = NSP*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                NSP_ct_dict[sub] = 0
            end
        end
    end   
    for (sub, ct) in AA_muts_ct_no_dels
        gene = aa_gene_comprehensive_dict[sub]
        if gene == "ORF1a" || gene == "ORF1b"
            NSP_sub = ORF1abMut_to_NSP(sub)
            NSP = NSP_muts_gene_dict[NSP_sub]
            NSP_pos = NSP_muts_pos_dict[NSP_sub]
            NSP_num = parse(Int, split(NSP, "P")[2])
            NSP_dict = NSP_ct_dict_arr[NSP_num]
            NSP_dict[NSP_sub] = ct
        end
    end
##########################################################################################################################
##########################################################################################################################
    for i in 1:length(NSP_ct_dict_arr)
        NSP = NSP_array[i]
        NSP_dict = NSP_ct_dict_arr[i]
        NSP_pos_dict = NSP_pos_ct_array[i]
        NSP_pos_dict_v1 = NSP_pos_ct_v1_array[i]
        for (sub, ct) in NSP_dict
            pos = NSP_muts_pos_dict[sub]
            sub_pos = NSP*":"*"$(pos)"
            NSP_pos_dict[sub_pos] += ct
            NSP_pos_dict_v1[pos] += ct
            NSP_pos_ct[sub_pos] += ct
        end
    end
    global NSP1_pos_ct_sort_by_pos = sort(collect(NSP1_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP2_pos_ct_sort_by_pos = sort(collect(NSP2_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP3_pos_ct_sort_by_pos = sort(collect(NSP3_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP4_pos_ct_sort_by_pos = sort(collect(NSP4_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP5_pos_ct_sort_by_pos = sort(collect(NSP5_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP6_pos_ct_sort_by_pos = sort(collect(NSP6_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP7_pos_ct_sort_by_pos = sort(collect(NSP7_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP8_pos_ct_sort_by_pos = sort(collect(NSP8_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP9_pos_ct_sort_by_pos = sort(collect(NSP9_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP10_pos_ct_sort_by_pos = sort(collect(NSP10_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP11_pos_ct_sort_by_pos = sort(collect(NSP11_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP12_pos_ct_sort_by_pos = sort(collect(NSP12_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP13_pos_ct_sort_by_pos = sort(collect(NSP13_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP14_pos_ct_sort_by_pos = sort(collect(NSP14_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP15_pos_ct_sort_by_pos = sort(collect(NSP15_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP16_pos_ct_sort_by_pos = sort(collect(NSP16_pos_ct), by = x -> NSP_muts_pos_dict[x[1]])
#########################
    global NSP1_pos_ct_sort_by_ct = sort(collect(NSP1_pos_ct), by = x -> x[2], rev=true)
    global NSP2_pos_ct_sort_by_ct = sort(collect(NSP2_pos_ct), by = x -> x[2], rev=true)
    global NSP3_pos_ct_sort_by_ct = sort(collect(NSP3_pos_ct), by = x -> x[2], rev=true)
    global NSP4_pos_ct_sort_by_ct = sort(collect(NSP4_pos_ct), by = x -> x[2], rev=true)
    global NSP5_pos_ct_sort_by_ct = sort(collect(NSP5_pos_ct), by = x -> x[2], rev=true)
    global NSP6_pos_ct_sort_by_ct = sort(collect(NSP6_pos_ct), by = x -> x[2], rev=true)
    global NSP7_pos_ct_sort_by_ct = sort(collect(NSP7_pos_ct), by = x -> x[2], rev=true)
    global NSP8_pos_ct_sort_by_ct = sort(collect(NSP8_pos_ct), by = x -> x[2], rev=true)
    global NSP9_pos_ct_sort_by_ct = sort(collect(NSP9_pos_ct), by = x -> x[2], rev=true)
    global NSP10_pos_ct_sort_by_ct = sort(collect(NSP10_pos_ct), by = x -> x[2], rev=true)
    global NSP11_pos_ct_sort_by_ct = sort(collect(NSP11_pos_ct), by = x -> x[2], rev=true)
    global NSP12_pos_ct_sort_by_ct = sort(collect(NSP12_pos_ct), by = x -> x[2], rev=true)
    global NSP13_pos_ct_sort_by_ct = sort(collect(NSP13_pos_ct), by = x -> x[2], rev=true)
    global NSP14_pos_ct_sort_by_ct = sort(collect(NSP14_pos_ct), by = x -> x[2], rev=true)
    global NSP15_pos_ct_sort_by_ct = sort(collect(NSP15_pos_ct), by = x -> x[2], rev=true)
    global NSP16_pos_ct_sort_by_ct = sort(collect(NSP16_pos_ct), by = x -> x[2], rev=true)
#########################
    global NSP1_pos_ct_sort_by_pos_v1 = sort(collect(NSP1_pos_ct_v1), by = x -> x[1])
    global NSP2_pos_ct_sort_by_pos_v1 = sort(collect(NSP2_pos_ct_v1), by = x -> x[1])
    global NSP3_pos_ct_sort_by_pos_v1 = sort(collect(NSP3_pos_ct_v1), by = x -> x[1])
    global NSP4_pos_ct_sort_by_pos_v1 = sort(collect(NSP4_pos_ct_v1), by = x -> x[1])
    global NSP5_pos_ct_sort_by_pos_v1 = sort(collect(NSP5_pos_ct_v1), by = x -> x[1])
    global NSP6_pos_ct_sort_by_pos_v1 = sort(collect(NSP6_pos_ct_v1), by = x -> x[1])
    global NSP7_pos_ct_sort_by_pos_v1 = sort(collect(NSP7_pos_ct_v1), by = x -> x[1])
    global NSP8_pos_ct_sort_by_pos_v1 = sort(collect(NSP8_pos_ct_v1), by = x -> x[1])
    global NSP9_pos_ct_sort_by_pos_v1 = sort(collect(NSP9_pos_ct_v1), by = x -> x[1])
    global NSP10_pos_ct_sort_by_pos_v1 = sort(collect(NSP10_pos_ct_v1), by = x -> x[1])
    global NSP11_pos_ct_sort_by_pos_v1 = sort(collect(NSP11_pos_ct_v1), by = x -> x[1])
    global NSP12_pos_ct_sort_by_pos_v1 = sort(collect(NSP12_pos_ct_v1), by = x -> x[1])
    global NSP13_pos_ct_sort_by_pos_v1 = sort(collect(NSP13_pos_ct_v1), by = x -> x[1])
    global NSP14_pos_ct_sort_by_pos_v1 = sort(collect(NSP14_pos_ct_v1), by = x -> x[1])
    global NSP15_pos_ct_sort_by_pos_v1 = sort(collect(NSP15_pos_ct_v1), by = x -> x[1])
    global NSP16_pos_ct_sort_by_pos_v1 = sort(collect(NSP16_pos_ct_v1), by = x -> x[1])
end
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
function gene_sub_ranks_all(gene_array::Vector{String}, sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_no_dels_all::Dict{String, Int} )
    ORF1a_ct_all = Dict{String, Int}()
    ORF1b_ct_all = Dict{String, Int}()
    S_ct_all = Dict{String, Int}()
    ORF3a_ct_all = Dict{String, Int}()
    E_ct_all = Dict{String, Int}()
    M_ct_all = Dict{String, Int}()
    ORF6_ct_all = Dict{String, Int}()
    ORF7a_ct_all = Dict{String, Int}()
    ORF7b_ct_all = Dict{String, Int}()
    ORF8_ct_all = Dict{String, Int}()
    N_ct_all = Dict{String, Int}()
    ORF9b_ct_all = Dict{String, Int}()
#############################################################
    #                                        Gene      AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
# sub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()
    gene_ct_dict_all_arr = [ORF1a_ct_all, ORF1b_ct_all, S_ct_all, ORF3a_ct_all, E_ct_all, M_ct_all, ORF6_ct_all, ORF7a_ct_all, ORF7b_ct_all, ORF8_ct_all, N_ct_all, ORF9b_ct_all]
    for i in 1:length(gene_array)
        gene = gene_array[i]
        gene_ct_dict = gene_ct_dict_all_arr[i]
        for (AAsite, mut_type_set) in sub_types_at_every_site_combined[gene]
            for mut_type in mut_type_set
                sub = gene*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                gene_ct_dict[sub] = 0
            end
        end
    end
#############################################################
    for (mut, ct) in AA_muts_ct_no_dels_all
        if aa_gene_comprehensive_dict[mut] == "ORF1a"
            ORF1a_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF1b"
            ORF1b_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "S"
            S_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF3a"
            ORF3a_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "E"
            E_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "M"
            M_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF6"
            ORF6_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7a"
            ORF7a_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7b"
            ORF7b_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF8"
            ORF8_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "N"
            N_ct_all[mut] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF9b"
            ORF9b_ct_all[mut] = ct
        end
    end
    fin_sortkey(m) = (aa_pos_comprehensive_dict[m], refAA_comprehensive_dict[m]*qryAA_comprehensive_dict[m])
    global ORF1a_ct_all_sort_by_pos = sort(collect(ORF1a_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF1b_ct_all_sort_by_pos = sort(collect(ORF1b_ct_all), by = x -> fin_sortkey(x[1]))
    global S_ct_all_sort_by_pos = sort(collect(S_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF3a_ct_all_sort_by_pos = sort(collect(ORF3a_ct_all), by = x -> fin_sortkey(x[1]))
    global E_ct_all_sort_by_pos = sort(collect(E_ct_all), by = x -> fin_sortkey(x[1]))
    global M_ct_all_sort_by_pos = sort(collect(M_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF6_ct_all_sort_by_pos = sort(collect(ORF6_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF7a_ct_all_sort_by_pos = sort(collect(ORF7a_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF7b_ct_all_sort_by_pos = sort(collect(ORF7b_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF8_ct_all_sort_by_pos = sort(collect(ORF8_ct_all), by = x -> fin_sortkey(x[1]))
    global N_ct_all_sort_by_pos = sort(collect(N_ct_all), by = x -> fin_sortkey(x[1]))
    global ORF9b_ct_all_sort_by_pos = sort(collect(ORF9b_ct_all), by = x -> fin_sortkey(x[1]))

    global ORF1b_ct_all_sort_by_ct = sort(collect(ORF1b_ct_all), by = x -> x[2], rev=true)
    global ORF1a_ct_all_sort_by_ct = sort(collect(ORF1a_ct_all), by = x -> x[2], rev=true)
    global S_ct_all_sort_by_ct = sort(collect(S_ct_all), by = x -> x[2], rev=true)
    global ORF3a_ct_all_sort_by_ct = sort(collect(ORF3a_ct_all), by = x -> x[2], rev=true)
    global E_ct_all_sort_by_ct = sort(collect(E_ct_all), by = x -> x[2], rev=true)
    global M_ct_all_sort_by_ct = sort(collect(M_ct_all), by = x -> x[2], rev=true)
    global ORF6_ct_all_sort_by_ct = sort(collect(ORF6_ct_all), by = x -> x[2], rev=true)
    global ORF7a_ct_all_sort_by_ct = sort(collect(ORF7a_ct_all), by = x -> x[2], rev=true)
    global ORF7b_ct_all_sort_by_ct = sort(collect(ORF7b_ct_all), by = x -> x[2], rev=true)
    global ORF8_ct_all_sort_by_ct = sort(collect(ORF8_ct_all), by = x -> x[2], rev=true)
    global N_ct_all_sort_by_ct = sort(collect(N_ct_all), by = x -> x[2], rev=true)
    global ORF9b_ct_all_sort_by_ct = sort(collect(ORF9b_ct_all), by = x -> x[2], rev=true)
end
#####################################################################################################################################
#####################################################################################################################################
function gene_sub_pos_ranks_all(gene_array::Vector{String}, gene_AA_lengths::Dict{String, Int},  AA_muts_ct_pos_only_no_dels_all::Dict{String, Int})
    ORF1a_pos_ct_all = Dict{String, Int}()
    ORF1b_pos_ct_all = Dict{String, Int}()
    S_pos_ct_all = Dict{String, Int}()
    ORF3a_pos_ct_all = Dict{String, Int}()
    E_pos_ct_all = Dict{String, Int}()
    M_pos_ct_all = Dict{String, Int}()
    ORF6_pos_ct_all = Dict{String, Int}()
    ORF7a_pos_ct_all = Dict{String, Int}()
    ORF7b_pos_ct_all = Dict{String, Int}()
    ORF8_pos_ct_all = Dict{String, Int}()
    N_pos_ct_all = Dict{String, Int}()
    ORF9b_pos_ct_all = Dict{String, Int}()
    gene_ct_pos_all_dict_arr = [ORF1a_pos_ct_all, ORF1b_pos_ct_all, S_pos_ct_all, ORF3a_pos_ct_all, E_pos_ct_all, M_pos_ct_all, ORF6_pos_ct_all, ORF7a_pos_ct_all, ORF7b_pos_ct_all, ORF8_pos_ct_all, N_pos_ct_all, ORF9b_pos_ct_all] 
###################################################### 
    ORF1a_pos_ct_all_v1 = Dict{Int, Int}()
    ORF1b_pos_ct_all_v1 = Dict{Int, Int}()
    S_pos_ct_all_v1 = Dict{Int, Int}()
    ORF3a_pos_ct_all_v1 = Dict{Int, Int}()
    E_pos_ct_all_v1 = Dict{Int, Int}()
    M_pos_ct_all_v1 = Dict{Int, Int}()
    ORF6_pos_ct_all_v1 = Dict{Int, Int}()
    ORF7a_pos_ct_all_v1 = Dict{Int, Int}()
    ORF7b_pos_ct_all_v1 = Dict{Int, Int}()
    ORF8_pos_ct_all_v1 = Dict{Int, Int}()
    N_pos_ct_all_v1 = Dict{Int, Int}()
    ORF9b_pos_ct_all_v1 = Dict{Int, Int}()
    gene_pos_ct_all_v1_dict_arr = [ORF1a_pos_ct_all_v1, ORF1b_pos_ct_all_v1, S_pos_ct_all_v1, ORF3a_pos_ct_all_v1, E_pos_ct_all_v1, M_pos_ct_all_v1, ORF6_pos_ct_all_v1, ORF7a_pos_ct_all_v1, ORF7b_pos_ct_all_v1, ORF8_pos_ct_all_v1, N_pos_ct_all_v1, ORF9b_pos_ct_all_v1]
###################################################### 
    for i in 1:length(gene_ct_pos_all_dict_arr)
        dict = gene_ct_pos_all_dict_arr[i]
        dict_v1 = gene_pos_ct_all_v1_dict_arr[i]
        gene = gene_array[i]
        gene_len = gene_AA_lengths[gene]
        for j in 1:gene_len
            site = gene*":"*"$(j)"
            dict[site] = 0
            dict_v1[j] = 0
        end
    end
#############################################################
    for (mut, ct) in AA_muts_ct_pos_only_no_dels_all
        pos = aa_pos_comprehensive_dict[mut]
        if aa_gene_comprehensive_dict[mut] == "ORF1a"
            ORF1a_pos_ct_all[mut] = ct
            ORF1a_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF1b"
            ORF1b_pos_ct_all[mut] = ct
            ORF1b_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "S"
            S_pos_ct_all[mut] = ct
            S_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF3a"
            ORF3a_pos_ct_all[mut] = ct
            ORF3a_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "E"
            E_pos_ct_all[mut] = ct
            E_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "M"
            M_pos_ct_all[mut] = ct
            M_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF6"
            ORF6_pos_ct_all[mut] = ct
            ORF6_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7a"
            ORF7a_pos_ct_all[mut] = ct
            ORF7a_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF7b"
            ORF7b_pos_ct_all[mut] = ct
            ORF7b_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF8"
            ORF8_pos_ct_all[mut] = ct
            ORF8_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "N"
            N_pos_ct_all[mut] = ct
            N_pos_ct_all_v1[pos] = ct
            elseif aa_gene_comprehensive_dict[mut] == "ORF9b"
            ORF9b_pos_ct_all[mut] = ct
            ORF9b_pos_ct_all_v1[pos] = ct
        end
    end
    global ORF1a_pos_ct_all_sort_by_pos = sort(collect(ORF1a_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF1b_pos_ct_all_sort_by_pos = sort(collect(ORF1b_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global S_pos_ct_all_sort_by_pos = sort(collect(S_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF3a_pos_ct_all_sort_by_pos = sort(collect(ORF3a_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global E_pos_ct_all_sort_by_pos = sort(collect(E_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]]) 
    global M_pos_ct_all_sort_by_pos = sort(collect(M_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF6_pos_ct_all_sort_by_pos = sort(collect(ORF6_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF7a_pos_ct_all_sort_by_pos = sort(collect(ORF7a_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF7b_pos_ct_all_sort_by_pos = sort(collect(ORF7b_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global ORF8_pos_ct_all_sort_by_pos = sort(collect(ORF8_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
    global N_pos_ct_all_sort_by_pos = sort(collect(N_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])   
    global ORF9b_pos_ct_all_sort_by_pos = sort(collect(ORF9b_pos_ct_all), by = x -> aa_pos_comprehensive_dict[x[1]])
################################## 
    global ORF1a_pos_ct_all_sort_by_ct = sort(collect(ORF1a_pos_ct_all), by = x -> x[2], rev=true)
    global ORF1b_pos_ct_all_sort_by_ct = sort(collect(ORF1b_pos_ct_all), by = x -> x[2], rev=true)
    global S_pos_ct_all_sort_by_ct = sort(collect(S_pos_ct_all), by = x -> x[2], rev=true)
    global ORF3a_pos_ct_all_sort_by_ct = sort(collect(ORF3a_pos_ct_all), by = x -> x[2], rev=true)
    global E_pos_ct_all_sort_by_ct = sort(collect(E_pos_ct_all), by = x -> x[2], rev=true)
    global M_pos_ct_all_sort_by_ct = sort(collect(M_pos_ct_all), by = x -> x[2], rev=true)
    global ORF6_pos_ct_all_sort_by_ct = sort(collect(ORF6_pos_ct_all), by = x -> x[2], rev=true)
    global ORF7a_pos_ct_all_sort_by_ct = sort(collect(ORF7a_pos_ct_all), by = x -> x[2], rev=true)
    global ORF7b_pos_ct_all_sort_by_ct = sort(collect(ORF7b_pos_ct_all), by = x -> x[2], rev=true)
    global ORF8_pos_ct_all_sort_by_ct = sort(collect(ORF8_pos_ct_all), by = x -> x[2], rev=true)
    global N_pos_ct_all_sort_by_ct = sort(collect(N_pos_ct_all), by = x -> x[2], rev=true)
    global ORF9b_pos_ct_all_sort_by_ct = sort(collect(ORF9b_pos_ct_all), by = x -> x[2], rev=true)
##################################
    global ORF1a_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF1a_pos_ct_all_v1), by = x -> x[1])
    global ORF1b_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF1b_pos_ct_all_v1), by = x -> x[1])
    global S_pos_ct_all_sort_by_pos_v1 = sort(collect(S_pos_ct_all_v1), by = x -> x[1])
    global ORF3a_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF3a_pos_ct_all_v1), by = x -> x[1])
    global E_pos_ct_all_sort_by_pos_v1 = sort(collect(E_pos_ct_all_v1), by = x -> x[1]) 
    global M_pos_ct_all_sort_by_pos_v1 = sort(collect(M_pos_ct_all_v1), by = x -> x[1])
    global ORF6_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF6_pos_ct_all_v1), by = x -> x[1])
    global ORF7a_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF7a_pos_ct_all_v1), by = x -> x[1])
    global ORF7b_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF7b_pos_ct_all_v1), by = x -> x[1])
    global ORF8_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF8_pos_ct_all_v1), by = x -> x[1])
    global N_pos_ct_all_sort_by_pos_v1 = sort(collect(N_pos_ct_all_v1), by = x -> x[1])   
    global ORF9b_pos_ct_all_sort_by_pos_v1 = sort(collect(ORF9b_pos_ct_all_v1), by = x -> x[1])
################################## 
    global ORF1a_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF1a_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF1b_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF1b_pos_ct_all_v1), by = x -> x[2], rev=true)
    global S_pos_ct_all_sort_by_ct_v1 = sort(collect(S_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF3a_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF3a_pos_ct_all_v1), by = x -> x[2], rev=true)
    global E_pos_ct_all_sort_by_ct_v1 = sort(collect(E_pos_ct_all_v1), by = x -> x[2], rev=true)
    global M_pos_ct_all_sort_by_ct_v1 = sort(collect(M_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF6_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF6_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF7a_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF7a_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF7b_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF7b_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF8_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF8_pos_ct_all_v1), by = x -> x[2], rev=true)
    global N_pos_ct_all_sort_by_ct_v1 = sort(collect(N_pos_ct_all_v1), by = x -> x[2], rev=true)
    global ORF9b_pos_ct_all_sort_by_ct_v1 = sort(collect(ORF9b_pos_ct_all_v1), by = x -> x[2], rev=true)
end
#####################################################################################################################################
function NSP_sub_ranks_all(NSP_array::Vector{String}, NSP_sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_no_dels_all::Dict{String, Int} )
    NSP1_ct_all = Dict{String, Int}()
    NSP2_ct_all = Dict{String, Int}()
    NSP3_ct_all = Dict{String, Int}()
    NSP4_ct_all = Dict{String, Int}()
    NSP5_ct_all = Dict{String, Int}()
    NSP6_ct_all = Dict{String, Int}()
    NSP7_ct_all = Dict{String, Int}()
    NSP8_ct_all = Dict{String, Int}()
    NSP9_ct_all = Dict{String, Int}()
    NSP10_ct_all = Dict{String, Int}()
    NSP11_ct_all = Dict{String, Int}()
    NSP12_ct_all = Dict{String, Int}()
    NSP13_ct_all = Dict{String, Int}()
    NSP14_ct_all = Dict{String, Int}()
    NSP15_ct_all = Dict{String, Int}()
    NSP16_ct_all = Dict{String, Int}()
#############################################################
    NSP_ct_all_dict_arr = [NSP1_ct_all, NSP2_ct_all, NSP3_ct_all, NSP4_ct_all, NSP5_ct_all, NSP6_ct_all, NSP7_ct_all, NSP8_ct_all, NSP9_ct_all, NSP10_ct_all, NSP11_ct_all, NSP12_ct_all, NSP13_ct_all, NSP14_ct_all, NSP15_ct_all, NSP16_ct_all]
#                                              NSP       AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
# NSPsub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()   
    for i in 1:length(NSP_array)
        NSP = NSP_array[i]
        NSP_ct_dict = NSP_ct_all_dict_arr[i]
        for (AAsite, mut_type_set) in NSP_sub_types_at_every_site_combined[NSP]
            for mut_type in mut_type_set
                sub = NSP*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                NSP_ct_dict[sub] = 0
            end
        end
    end   
    for (sub, ct) in AA_muts_ct_no_dels_all
        AA_site = aa_pos_comprehensive_dict[sub]
        if AA_site < 4402
            aa_gene_comprehensive_dict[sub]
            if gene == "ORF1a" || gene == "ORF1b"
                NSP_sub = ORF1abMut_to_NSP(sub)
                NSP_muts_pos_dict[NSP_sub]
                NSP_muts_pos_dict[NSP_sub]
                NSP_num = parse(Int, split(NSP, "P")[2])
                NSP_dict = NSP_ct_all_dict_arr[NSP_num]
                NSP_dict[NSP_sub] = ct
            end
        end
    end
#############################
    fin_sortkey(m) = (aa_pos_comprehensive_dict[m], mNSP_ref_AA_dict[m]*NSP_qry_AA_dict[m])
    global NSP1_ct_all_sort_by_pos = sort(collect(NSP1_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP2_ct_all_sort_by_pos = sort(collect(NSP2_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP3_ct_all_sort_by_pos = sort(collect(NSP3_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP4_ct_all_sort_by_pos = sort(collect(NSP4_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP5_ct_all_sort_by_pos = sort(collect(NSP5_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP6_ct_all_sort_by_pos = sort(collect(NSP6_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP7_ct_all_sort_by_pos = sort(collect(NSP7_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP8_ct_all_sort_by_pos = sort(collect(NSP8_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP9_ct_all_sort_by_pos = sort(collect(NSP9_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP10_ct_all_sort_by_pos = sort(collect(NSP10_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP11_ct_all_sort_by_pos = sort(collect(NSP11_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP12_ct_all_sort_by_pos = sort(collect(NSP12_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP13_ct_all_sort_by_pos = sort(collect(NSP13_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP14_ct_all_sort_by_pos = sort(collect(NSP14_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP15_ct_all_sort_by_pos = sort(collect(NSP15_ct_all), by = x -> fin_sortkey(x[1]))
    global NSP16_ct_all_sort_by_pos = sort(collect(NSP16_ct_all), by = x -> fin_sortkey(x[1]))
#############################
    global NSP1_ct_all_sort_by_ct = sort(collect(NSP1_ct_all), by = x -> x[2], rev=true)
    global NSP2_ct_all_sort_by_ct = sort(collect(NSP2_ct_all), by = x -> x[2], rev=true)
    global NSP3_ct_all_sort_by_ct = sort(collect(NSP3_ct_all), by = x -> x[2], rev=true)
    global NSP4_ct_all_sort_by_ct = sort(collect(NSP4_ct_all), by = x -> x[2], rev=true)
    global NSP5_ct_all_sort_by_ct = sort(collect(NSP5_ct_all), by = x -> x[2], rev=true)
    global NSP6_ct_all_sort_by_ct = sort(collect(NSP6_ct_all), by = x -> x[2], rev=true)
    global NSP7_ct_all_sort_by_ct = sort(collect(NSP7_ct_all), by = x -> x[2], rev=true)
    global NSP8_ct_all_sort_by_ct = sort(collect(NSP8_ct_all), by = x -> x[2], rev=true)
    global NSP9_ct_all_sort_by_ct = sort(collect(NSP9_ct_all), by = x -> x[2], rev=true)
    global NSP10_ct_all_sort_by_ct = sort(collect(NSP10_ct_all), by = x -> x[2], rev=true)
    global NSP11_ct_all_sort_by_ct = sort(collect(NSP11_ct_all), by = x -> x[2], rev=true)
    global NSP12_ct_all_sort_by_ct = sort(collect(NSP12_ct_all), by = x -> x[2], rev=true)
    global NSP13_ct_all_sort_by_ct = sort(collect(NSP13_ct_all), by = x -> x[2], rev=true)
    global NSP14_ct_all_sort_by_ct = sort(collect(NSP14_ct_all), by = x -> x[2], rev=true)
    global NSP15_ct_all_sort_by_ct = sort(collect(NSP15_ct_all), by = x -> x[2], rev=true)
    global NSP16_ct_all_sort_by_ct = sort(collect(NSP16_ct_all), by = x -> x[2], rev=true)
end
#####################################################################################################################################
# Removed from parameters: NSP_AA_size::Dict{String, Int}
function NSP_sub_pos_ranks_all(NSP_array::Vector{String}, NSP_sub_types_at_every_site_combined::Dict{String, Dict{Int, Set{String}}}, AA_muts_ct_no_dels_all::Dict{String, Int} )
    NSP1_pos_ct_all = Dict{String, Int}()
    NSP2_pos_ct_all = Dict{String, Int}()
    NSP3_pos_ct_all = Dict{String, Int}()
    NSP4_pos_ct_all = Dict{String, Int}()
    NSP5_pos_ct_all = Dict{String, Int}()
    NSP6_pos_ct_all = Dict{String, Int}()
    NSP7_pos_ct_all = Dict{String, Int}()
    NSP8_pos_ct_all = Dict{String, Int}()
    NSP9_pos_ct_all = Dict{String, Int}()
    NSP10_pos_ct_all = Dict{String, Int}()
    NSP11_pos_ct_all = Dict{String, Int}()
    NSP12_pos_ct_all = Dict{String, Int}()
    NSP13_pos_ct_all = Dict{String, Int}()
    NSP14_pos_ct_all = Dict{String, Int}()
    NSP15_pos_ct_all = Dict{String, Int}()
    NSP16_pos_ct_all = Dict{String, Int}()
    NSP_pos_ct_all_array = [NSP1_pos_ct_all, NSP2_pos_ct_all, NSP3_pos_ct_all, NSP4_pos_ct_all, NSP5_pos_ct_all, NSP6_pos_ct_all, NSP7_pos_ct_all, NSP8_pos_ct_all, NSP9_pos_ct_all, NSP10_pos_ct_all, NSP11_pos_ct_all, NSP12_pos_ct_all, NSP13_pos_ct_all, NSP14_pos_ct_all, NSP15_pos_ct_all, NSP16_pos_ct_all]
#######################################################
    NSP1_pos_ct_all_v1 = Dict{Int, Int}()
    NSP2_pos_ct_all_v1 = Dict{Int, Int}()
    NSP3_pos_ct_all_v1 = Dict{Int, Int}()
    NSP4_pos_ct_all_v1 = Dict{Int, Int}()
    NSP5_pos_ct_all_v1 = Dict{Int, Int}()
    NSP6_pos_ct_all_v1 = Dict{Int, Int}()
    NSP7_pos_ct_all_v1 = Dict{Int, Int}()
    NSP8_pos_ct_all_v1 = Dict{Int, Int}()
    NSP9_pos_ct_all_v1 = Dict{Int, Int}()
    NSP10_pos_ct_all_v1 = Dict{Int, Int}()
    NSP11_pos_ct_all_v1 = Dict{Int, Int}()
    NSP12_pos_ct_all_v1 = Dict{Int, Int}()
    NSP13_pos_ct_all_v1 = Dict{Int, Int}()
    NSP14_pos_ct_all_v1 = Dict{Int, Int}()
    NSP15_pos_ct_all_v1 = Dict{Int, Int}()
    NSP16_pos_ct_all_v1 = Dict{Int, Int}()
    NSP_pos_ct_all_v1_array = [NSP1_pos_ct_all_v1, NSP2_pos_ct_all_v1, NSP3_pos_ct_all_v1, NSP4_pos_ct_all_v1, NSP5_pos_ct_all_v1, NSP6_pos_ct_all_v1, NSP7_pos_ct_all_v1, NSP8_pos_ct_all_v1, NSP9_pos_ct_all_v1, NSP10_pos_ct_all_v1, NSP11_pos_ct_all_v1, NSP12_pos_ct_all_v1, NSP13_pos_ct_all_v1, NSP14_pos_ct_all_v1, NSP15_pos_ct_all_v1, NSP16_pos_ct_all_v1]
#######################################################
    NSP_pos_ct_all = Dict{String, Int}()
    for i in 1:length(NSP_pos_ct_all_array)
        NSP_all_dict = NSP_pos_ct_all_array[i]
        NSP_all_dict_v1 = NSP_pos_ct_all_v1_array[i]
        NSP = NSP_array[i]
        NSP_len = NSP_AA_size[NSP]
        for j in 1:NSP_len
            NSP_pos = NSP*":"*"$(j)"
            NSP_all_dict[NSP_pos] = 0
            NSP_pos_ct_all[NSP_pos] = 0
            NSP_all_dict_v1[j] = 0
        end
    end
############################################################
    NSP1_ct_all = Dict{String, Int}()
    NSP2_ct_all = Dict{String, Int}()
    NSP3_ct_all = Dict{String, Int}()
    NSP4_ct_all = Dict{String, Int}()
    NSP5_ct_all = Dict{String, Int}()
    NSP6_ct_all = Dict{String, Int}()
    NSP7_ct_all = Dict{String, Int}()
    NSP8_ct_all = Dict{String, Int}()
    NSP9_ct_all = Dict{String, Int}()
    NSP10_ct_all = Dict{String, Int}()
    NSP11_ct_all = Dict{String, Int}()
    NSP12_ct_all = Dict{String, Int}()
    NSP13_ct_all = Dict{String, Int}()
    NSP14_ct_all = Dict{String, Int}()
    NSP15_ct_all = Dict{String, Int}()
    NSP16_ct_all = Dict{String, Int}()
##################################################################################################################################
##################################################################################################################################
    NSP_ct_all_dict_arr = [NSP1_ct_all, NSP2_ct_all, NSP3_ct_all, NSP4_ct_all, NSP5_ct_all, NSP6_ct_all, NSP7_ct_all, NSP8_ct_all, NSP9_ct_all, NSP10_ct_all, NSP11_ct_all, NSP12_ct_all, NSP13_ct_all, NSP14_ct_all, NSP15_ct_all, NSP16_ct_all]
#                                              NSP       AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
# NSPsub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()   
    for i in 1:length(NSP_array)
        NSP = NSP_array[i]
        NSP_ct_dict = NSP_ct_all_dict_arr[i]
        for (AAsite, mut_type_set) in NSP_sub_types_at_every_site_combined[NSP]
            for mut_type in mut_type_set
                sub = NSP*":"*string(mut_type[1])*"$(AAsite)"*string(mut_type[end])
                NSP_ct_dict[sub] = 0
            end
        end
    end   
    for (sub, ct) in AA_muts_ct_no_dels_all
        aa_gene_comprehensive_dict[sub]
        if gene == "ORF1a" || gene == "ORF1b"
            AA_site = parse(Int, split(sub, ":")[2][2:end-1])
            if AA_site < 4402
                NSP_sub = ORF1abMut_to_NSP(sub)
                NSP_muts_pos_dict[NSP_sub]
                NSP_muts_pos_dict[NSP_sub]
                NSP_num = parse(Int, split(NSP, "P")[2])
                NSP_dict = NSP_ct_all_dict_arr[NSP_num]
                NSP_dict[NSP_sub] = ct
            end
        end
    end
###################################################################################################################################
###################################################################################################################################
    for i in 1:length(NSP_pos_ct_all_array)
        NSP = NSP_array[i]
        NSP_dict = NSP_ct_all_dict_arr[i]
        NSP_pos_dict = NSP_pos_ct_all_array[i]
        NSP_pos_dict_v1 = NSP_pos_ct_all_v1_array[i]
        for (sub, ct) in NSP_dict
            pos = parse(Int, split(sub, ":")[2][2:end-1])
            sub_pos = NSP*":"*"$(pos)"
            NSP_pos_dict[sub_pos] += ct
            NSP_pos_dict_v1[pos] += ct
            NSP_pos_ct_all[sub_pos] += ct
        end
    end
##############################################################
    global NSP1_pos_ct_all_sort_by_pos = sort(collect(NSP1_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP2_pos_ct_all_sort_by_pos = sort(collect(NSP2_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP3_pos_ct_all_sort_by_pos = sort(collect(NSP3_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP4_pos_ct_all_sort_by_pos = sort(collect(NSP4_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP5_pos_ct_all_sort_by_pos = sort(collect(NSP5_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP6_pos_ct_all_sort_by_pos = sort(collect(NSP6_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP7_pos_ct_all_sort_by_pos = sort(collect(NSP7_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP8_pos_ct_all_sort_by_pos = sort(collect(NSP8_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP9_pos_ct_all_sort_by_pos = sort(collect(NSP9_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP10_pos_ct_all_sort_by_pos = sort(collect(NSP10_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP11_pos_ct_all_sort_by_pos = sort(collect(NSP11_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP12_pos_ct_all_sort_by_pos = sort(collect(NSP12_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP13_pos_ct_all_sort_by_pos = sort(collect(NSP13_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP14_pos_ct_all_sort_by_pos = sort(collect(NSP14_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP15_pos_ct_all_sort_by_pos = sort(collect(NSP15_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
    global NSP16_pos_ct_all_sort_by_pos = sort(collect(NSP16_pos_ct_all), by = x -> NSP_muts_pos_dict[x[1]])
#############################
    global NSP1_pos_ct_all_sort_by_ct = sort(collect(NSP1_pos_ct_all), by = x -> x[2], rev=true)
    global NSP2_pos_ct_all_sort_by_ct = sort(collect(NSP2_pos_ct_all), by = x -> x[2], rev=true)
    global NSP3_pos_ct_all_sort_by_ct = sort(collect(NSP3_pos_ct_all), by = x -> x[2], rev=true)
    global NSP4_pos_ct_all_sort_by_ct = sort(collect(NSP4_pos_ct_all), by = x -> x[2], rev=true)
    global NSP5_pos_ct_all_sort_by_ct = sort(collect(NSP5_pos_ct_all), by = x -> x[2], rev=true)
    global NSP6_pos_ct_all_sort_by_ct = sort(collect(NSP6_pos_ct_all), by = x -> x[2], rev=true)
    global NSP7_pos_ct_all_sort_by_ct = sort(collect(NSP7_pos_ct_all), by = x -> x[2], rev=true)
    global NSP8_pos_ct_all_sort_by_ct = sort(collect(NSP8_pos_ct_all), by = x -> x[2], rev=true)
    global NSP9_pos_ct_all_sort_by_ct = sort(collect(NSP9_pos_ct_all), by = x -> x[2], rev=true)
    global NSP10_pos_ct_all_sort_by_ct = sort(collect(NSP10_pos_ct_all), by = x -> x[2], rev=true)
    global NSP11_pos_ct_all_sort_by_ct = sort(collect(NSP11_pos_ct_all), by = x -> x[2], rev=true)
    global NSP12_pos_ct_all_sort_by_ct = sort(collect(NSP12_pos_ct_all), by = x -> x[2], rev=true)
    global NSP13_pos_ct_all_sort_by_ct = sort(collect(NSP13_pos_ct_all), by = x -> x[2], rev=true)
    global NSP14_pos_ct_all_sort_by_ct = sort(collect(NSP14_pos_ct_all), by = x -> x[2], rev=true)
    global NSP15_pos_ct_all_sort_by_ct = sort(collect(NSP15_pos_ct_all), by = x -> x[2], rev=true)
    global NSP16_pos_ct_all_sort_by_ct = sort(collect(NSP16_pos_ct_all), by = x -> x[2], rev=true)
###############################
    global NSP1_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP1_pos_ct_all_v1), by = x -> x[1])
    global NSP2_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP2_pos_ct_all_v1), by = x -> x[1])
    global NSP3_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP3_pos_ct_all_v1), by = x -> x[1])
    global NSP4_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP4_pos_ct_all_v1), by = x -> x[1])
    global NSP5_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP5_pos_ct_all_v1), by = x -> x[1])
    global NSP6_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP6_pos_ct_all_v1), by = x -> x[1])
    global NSP7_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP7_pos_ct_all_v1), by = x -> x[1])
    global NSP8_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP8_pos_ct_all_v1), by = x -> x[1])
    global NSP9_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP9_pos_ct_all_v1), by = x -> x[1])
    global NSP10_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP10_pos_ct_all_v1), by = x -> x[1])
    global NSP11_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP11_pos_ct_all_v1), by = x -> x[1])
    global NSP12_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP12_pos_ct_all_v1), by = x -> x[1])
    global NSP13_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP13_pos_ct_all_v1), by = x -> x[1])
    global NSP14_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP14_pos_ct_all_v1), by = x -> x[1])
    global NSP15_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP15_pos_ct_all_v1), by = x -> x[1])
    global NSP16_pos_ct_all_sort_by_pos_v1 = sort(collect(NSP16_pos_ct_all_v1), by = x -> x[1])
end
######################################################################################################################################
runtime = time() - start_chr_load_fx
runtime_rd = round(digits=2, runtime)
runtime1, runtime2 = seconds_to_hrs_min_sec(runtime); print("\n"^1)
println("Runtime v0 = $(runtime) seconds")
println("Runtime v2 = $(runtime2)"); print("\n"^1)
######################################################################################################################################
######################################################################################################################################

2026_02_21__929AM


NSP_sub_pos_ranks_all (generic function with 1 method)

In [69]:
### Execute Load Chronic Dicts DQ Fx | 2026_02_22 version | chronics_2026_02_01__6153seq | 95—8—6 | Runtime: 1 min, 00.26 sec | 2026_2_13
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
chr_dict_load_start = time()

abs_min_mut_thresh = 8
DQ_mut_thresh = 10
date_pct_DQ_thresh = 95

rep_thresh = 5
revs_thresh = 6
HQCS_qc_str = HQCS_qc_string
EPCI_qc_str = "$(abs_min_mut_thresh)_$(DQ_mut_thresh)_$(date_pct_DQ_thresh)"

print_ct_thresh = 1
date = "2026_02_20"
ndjson_name = "chronics_2026_02_01__6153seq_v2"
folder_name = "$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_string)"

println("EPCI_qc_str = $(EPCI_qc_str) | HQCS_qc_str = $(HQCS_qc_str)")

chronic_load_dicts2_DQ(ndjson_name, folder_name, date, rep_thresh, revs_thresh, print_ct_thresh, DQ_mut_thresh, date_pct_DQ_thresh, abs_min_mut_thresh)
# ndjson_name::String, folder_name::String, date::String, rep_thresh::Int, revs_thresh::Int, print_ct_thresh::Int, DQ_mut_thresh::Int, DatePctDQThresh::Float64, abs_min_mut_thresh::Int
######################################################################################################################################
seq_AA_muts["EPI_ISL_8725398"] = Set{String}();             seq_AA_muts["EPI_ISL_949208"] = Set{String}();   seq_pango["EPI_ISL_6281381"] = "AY.4"
######################################################################################################################################
### Create  all_unique_chr_seqs  &  all_unique_chr_seqs_set 
rep_seq_grps_maxmut_seqs_arr = collect(values(rep_seq_grps_maxmut_seqs))
all_unique_chr_seqs = union(rep_seq_grps_maxmut_seqs_arr, non_rep_seqs)
all_unique_chr_seqs_ct = length(all_unique_chr_seqs)
all_unique_chr_seqs_set = Set{String}()
for seq in all_unique_chr_seqs
    push!(all_unique_chr_seqs_set, seq)
end
println("length(all_unique_chr_seqs) = $(length(all_unique_chr_seqs))")
println("length(all_unique_chr_seqs_set)  $(length(all_unique_chr_seqs_set))"); print("\n"^2)
########################################################################################################################################
for seq in all_unique_chr_seqs_set 
    for del in seq_AA_dels[seq]
        aa_gene_comprehensive_dict[del] = string(split(del, ":")[1])
        firstdel = string(split(del, "-")[1])
        aa_pos_comprehensive_dict[del] = aa_pos_comprehensive_dict[firstdel]
    end
end
#############################################################################################################################################################################
#### Removing clearly artifactual reversions that occur directly next to dropout (and are almost always from labs that frequently have this problem)
FCS_fake_revs = Set(["S:K679N", "S:H681P", "S:R681P"])
FCS_fake_revs_pos = Set(["S:679", "S:681"])
for seq in all_qualifying_seqs_set
    if "S:683" in seq_unknown_AA[seq] || "S:676" in seq_unknown_AA[seq]
        if seq in all_unique_chr_seqs_set
            for mut in FCS_fake_revs
                if mut in seq_AA_muts[seq]
                    mut_pos = aa_gene_and_pos_comprehensive_dict[mut]
                    AA_muts_ct[mut] -= 1
                    AA_muts_ct_no_dels[mut] -= 1
                    AA_muts_ct_pos_only[mut_pos] -= 1
                    AA_muts_ct_pos_only_no_dels[mut_pos] -= 1
                end
            end
        end
        setdiff!(seq_AA_muts[seq], FCS_fake_revs)
        setdiff!(seq_AA_muts_no_dels[seq], FCS_fake_revs)
        setdiff!(seq_AA_muts_pos_only[seq], FCS_fake_revs_pos)
        setdiff!(seq_AA_muts_pos_only_no_dels[seq], FCS_fake_revs_pos)
    end
end
######################################################################################################################################
if haskey(AA_muts_ct_pos_only_no_dels, "")
    delete!(AA_muts_ct_pos_only_no_dels, "")
end
if haskey(AA_muts_ct_no_dels, "")
    delete!(AA_muts_ct_no_dels, "")
end
######################################################################################################################################
## Deletions royally screw up any attempt to find correlated mutations since they frequently occur in bunches, which are, of course, highly correlated, but only in the most trivial way. 
## The code below is a preliminary attempt to include common deletions by only allowing the inclusion of a single AA deletion, though most contain multiple consecutive 
## AA deletions. Also included are mutations that only occur as 1-AA deletions, such as E:I13- and E:V14-. It can be removed or inserted without substantially changing the results 
deletion_exceptions = list_to_strings_set("E:I13-, E:V14-, M:G6-, S:C15-, S:C136-, S:D138-, S:A243-, S:F371-, S:F375-, S:V483-, S:A484-, S:E484-, S:D1257-, ORF3a:T14-, ORF3a:V255-, ORF3a:V256-, ORF3a:N257-, ORF7a:I103-, ORF7a:*122-, ORF1a:N2081-, ORF1a:D2136-, ORF1a:S4398-")
pos_only_deletion_exception_ct_dict = Dict{String, Int}("E:I13-"=>0, "E:V14-"=>0, "M:G6-"=>0, "S:C15-"=>0, "S:C136-"=>0, "S:D138-"=>0, "S:A243-"=>0, "S:F371-"=>0, "S:F375-"=>0, "S:V483-"=>0, "S:A484-"=>0, "S:E484-"=>0, "S:D1257-"=>0, "ORF3a:T14-"=>0, "ORF3a:V255-"=>0, "ORF3a:V256-"=>0, "ORF3a:N257-"=>0, "ORF7a:I103-"=>0, "ORF7a:*122-"=>0, "ORF1a:N2081-"=>0, "ORF1a:D2136-"=>0, "ORF1a:S4398-"=>0)    
del_to_del_pos = Dict{String, String}("E:I13-"=>"E:13", "E:V14-"=>"E:14", "M:G6-"=>"M:6", "S:C15-"=>"S:15", "S:C136-"=>"S:136", "S:D138-"=>"S:138", "S:A243-"=>"S:243", "S:F371-"=>"S:371", "S:F375-"=>"S:375", "S:V483-"=>"S:483", "S:A484-"=>"S:484", "S:E484-"=>"S:484", "S:D1257-"=>"S:1257", "ORF3a:T14-"=>"ORF3a:14", "ORF3a:V255-"=>"ORF3a:255", "ORF3a:V256-"=>"ORF3a:256", "ORF3a:N257-"=>"ORF3a:257", "ORF7a:I103-"=>"ORF7a:103", "ORF7a:*122-"=>"ORF7a:122", "ORF1a:N2081-"=>"ORF1a:2081", "ORF1a:D2136-"=>"ORF1a:2136", "ORF1a:S4398-"=>"ORF1a:4398")
pos_only_exception_count::Int = 0
for seq in all_seqs_set
    for del in deletion_exceptions
        if del in seq_AA_muts[seq]
            del_pos = del_to_del_pos[del]
            push!(seq_AA_muts_pos_only_no_dels[seq], del_pos)
            pos_only_exception_count += 1
            pos_only_deletion_exception_ct_dict[del] += 1
        end
    end
end
######################################################################################################################################
for del in deletion_exceptions
    del_pos = del_to_del_pos[del]
    if !haskey(AA_muts_ct_pos_only_no_dels, del_pos)
        AA_muts_ct_pos_only_no_dels[del_pos] = 0
    end
    AA_muts_ct_pos_only_no_dels[del_pos] += pos_only_deletion_exception_ct_dict[del]
    AA_muts_ct_pos_only_no_dels_chr_all_ratio[del_pos] = 999
end
######################################################################################################################################
deletion_exception_ct_dict = Dict{String, Int}("E:I13-"=>0, "E:V14-"=>0, "M:G6-"=>0, "S:C15-"=>0, "S:C136-"=>0, "S:D138-"=>0, "S:A243-"=>0, "S:F371-"=>0, "S:F375-"=>0, "S:V483-"=>0, "S:A484-"=>0, "S:E484-"=>0, "S:D1257-"=>0, "ORF3a:T14-"=>0, "ORF3a:V255-"=>0, "ORF3a:V256-"=>0, "ORF3a:N257-"=>0, "ORF7a:I103-"=>0, "ORF7a:*122-"=>0, "ORF1a:N2081-"=>0, "ORF1a:D2136-"=>0, "ORF1a:S4398-"=>0)    
exception_count::Int = 0
for seq in all_unique_chr_seqs_set
    for del in deletion_exceptions
        if del in seq_AA_muts[seq]
            push!(seq_AA_muts_no_dels[seq], del)
            exception_count += 1
            deletion_exception_ct_dict[del] += 1
        end
    end
end
######################################################################################################################################
for del in deletion_exceptions
    AA_muts_ct_no_dels[del] = get(AA_muts_ct_no_dels, del, 0)
    AA_muts_ct_no_dels[del] += deletion_exception_ct_dict[del]
    AA_muts_ct_chr_all_ratio[del] = 999
end
######################################################################################################################################
blank_mutstrings = Set(["", " ", "  ", "   ", "    ", "     ", "      ", "       ", "        ", "         ", "          ", "           ", "            "])
for blnk in blank_mutstrings
    AA_muts_ct_chr_all_ratio[blnk] = 0
    AA_muts_ct_no_dels_chr_all_ratio[blnk] = 0
    AA_muts_ct_pos_only_no_dels_chr_all_ratio[blnk] = 0
end
#######################################################################################################################################
mut_gene_Dict = Dict{String, Int}("ORF1a"=>1, "ORF1b"=>2, "S"=>3, "E"=>4, "M"=>5, "N"=>6, "ORF3a"=>7, "ORF6"=>8, "ORF7a"=>9, "ORF7b"=>10, "ORF8"=>11, "ORF9b"=>12)
######################################################################################################################################
mp_AA_gene_sortKey(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
mp_AA_gene_sortKey_2(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n]], aa_pos_comprehensive_dict[n])
mp_AA_ct_sortKey1(n) = (1000÷mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
mp_AA_ct_sortKey2(n) = (n[2], mp_AA_ct_sortKey1(n))
#########
mp_AA_gene_pos_only_sortKey(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
mp_AA_gene_pos_only_sortKey_2(n) = (mut_gene_Dict[aa_gene_comprehensive_dict[n]], aa_pos_comprehensive_dict[n])
mp_AA_ct_pos_only_sortKey1(n) = (1000÷mut_gene_Dict[aa_gene_comprehensive_dict[n[1]]], aa_pos_comprehensive_dict[n[1]])
mp_AA_ct_pos_only_sortKey2(n) = (n[2], mp_AA_ct_pos_only_sortKey1(n))
#################################
function mp_AApos_sort_key(n)
    if haskey(aa_pos_comprehensive_dict, n)
        return (aa_pos_comprehensive_dict[n], n)
    else
        return (9999, n)
    end
end
function sel_muts_pt1_sort_key(n)
    if n == "" || isempty(n)
        return (0, 0)  # Return a default sort key for empty strings
    else
        return mp_AA_gene_sortKey_2(string(split(n, ", ")[1]))
    end
end
##################################################################
println("Number of Deletion Exceptions Made = $(exception_count)")
deletion_exception_ct_dict_sort = sort(collect(deletion_exception_ct_dict), by = x -> x[2])
for del__ct in deletion_exception_ct_dict_sort
    del = del__ct[1]
    ct = del__ct[2]
    println("$(del) = $(ct)")
end
###########################################################################################
seq_privAA_len = Dict{String, Int}()
for (seq, AAsubvec) in seq_AA_muts_no_dels
    seqAAlen = length(AAsubvec)
    seq_privAA_len[seq] = seqAAlen
end
############################################################################################################################################################################
###########################################################
for (seq, date) in seq_collection_date
    date_arr = string.(collect(date))
    if sum(date_arr .== "-") ≠ 2
        println("Doesn't have two dashes in date = $(seq)")
    else
        year = string(split(date, "-")[1])
        month = string(split(date, "-")[2])
        day = string(split(date, "-")[3])
        if length(month) == 1 && month ≠ "0"
            month = add_leading_zero(month)
        end
        if length(day) == 1 && day ≠ "0"
            day = add_leading_zero(day)
        end
        seq_collection_date[seq] = year*"-"*month*"-"*day
    end
end 
############################################################################################################################################################################
corrected_count = 0
noncorrected_ct = 0
for seq in all_seqs_set
    if seq_date_index[seq] > 4000
#        println("seq_date_tuple = $(seq_date_tuple[seq]); seq_date_tuple[seq][1] = $(seq_date_tuple[seq][1]); seq_date_tuple[seq][2] = $(seq_date_tuple[seq][2]); seq_date_tuple[seq][3] = $(seq_date_tuple[seq][3])")
        if seq_date_tuple[seq][1] ≠ 0 && seq_date_tuple[seq][2] ≠ 0
            new_date_tuple = (seq_date_tuple[seq][1], seq_date_tuple[seq][2], 15)
            seq_date_tuple[seq] = new_date_tuple
            seq_date_index[seq] = tuple_to_index[new_date_tuple]
            corrected_count += 1
        elseif seq_date_tuple[seq][2] == 0
            noncorrected_ct += 1
        end
    end
end
total_baddie_ct = corrected_count + noncorrected_ct
println("seq_date_index corrected = $(corrected_count)")
println("seq_date_index not corrected = $(noncorrected_ct)")
println("total_baddie_ct = $(total_baddie_ct)")
######################################################################################################################################################
######################################################################################################################################  
chr_load_runtime = time() - chr_dict_load_start
chr_load_runtime_rd = round(digits=1, chr_load_runtime)
chr_load_hms1, chr_load_hms2 = seconds_to_hrs_min_sec(chr_load_runtime)
println("Total Time to Load Chronic Dictionaries = $(chr_load_runtime_rd)")
println("Total Time to Load Chronic Dictionaries = $(chr_load_hms1)")
println("Total Time to Load Chronic Dictionaries = $(chr_load_hms2)"); print("\n"^1)
println("Finished!")
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now); print("\n"^1)
######################################################################################################################################
######################################################################################################################################

2026_02_21__150PM
EPCI_qc_str = 1_1_1 | HQCS_qc_str = 5_1_5
2026_02_21__150PM
length(all_unique_chr_seqs) = 3483
length(all_unique_chr_seqs_set)  3483


Number of Deletion Exceptions Made = 853
S:E484- = 1
ORF1a:D2136- = 2
S:F371- = 4
ORF1a:N2081- = 6
ORF1a:S4398- = 7
E:I13- = 8
S:D1257- = 8
S:V483- = 9
ORF7a:I103- = 10
M:G6- = 12
ORF7a:*122- = 16
S:A484- = 23
S:F375- = 25
S:C136- = 32
ORF3a:V256- = 39
ORF3a:T14- = 40
ORF3a:N257- = 44
ORF3a:V255- = 51
S:D138- = 91
E:V14- = 96
S:C15- = 105
S:A243- = 224
seq_date_index corrected = 72
seq_date_index not corrected = 52
total_baddie_ct = 124
Total Time to Load Chronic Dictionaries = 16.8
Total Time to Load Chronic Dictionaries = 0:00:16.77
Total Time to Load Chronic Dictionaries = 0 hr, 0 min, 16.77 sec

Finished!
2026_02_21__150PM



"10_1_30"

In [371]:
### index_to_tuple, tuple_to_index + date_index assignment for year-month-(no day) seqs
start = time()
nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
###################
index_to_tuple = Dict{Int, Tuple{Int, Int, Int}}()
tuple_to_index = Dict{Tuple{Int, Int, Int}, Int}()        
###########################################################
for (seq, date) in seq_collection_date
    date_arr = string.(collect(date))
    if sum(date_arr .== "-") ≠ 2
        println("Doesn't have two dashes in date = $(seq)")
    else
        year = string(split(date, "-")[1])
        month = string(split(date, "-")[2])
        day = string(split(date, "-")[3])
        if length(month) == 1 && month ≠ "0"
            month = add_leading_zero(month)
        end
        if length(day) == 1 && day ≠ "0"
            day = add_leading_zero(day)
        end
        seq_collection_date[seq] = year*"-"*month*"-"*day
    end
end 
###################
index = 0
for year in 2020:2027
    for month in 1:12
        if year%4 == 0
            month_days = leap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        else
            month_days = nonleap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        end
    end
end
for (index, date) in index_to_tuple
    tuple_to_index[date] = index
end
for y in 2020:2027
    tuple_to_index[(y, 0, 0)] = y*1000000
    index_to_tuple[y*1000000] = (y, 0, 0)
    for m in 1:12
        tuple_to_index[(y, m, 0)] = y*1000000 + m*1000
        index_to_tuple[y*1000000 + m*1000] = (y, m, 0)
    end
end
tuple_to_index[(0, 0, 0)] = 1000000000
index_to_tuple[1000000000] = (0, 0, 0)
######################################################################################################################################################
corrected_count = 0
noncorrected_ct = 0
for seq in all_seqs_set
    if seq_date_index[seq] > 4000
#        println("seq_date_tuple = $(seq_date_tuple[seq]); seq_date_tuple[seq][1] = $(seq_date_tuple[seq][1]); seq_date_tuple[seq][2] = $(seq_date_tuple[seq][2]); seq_date_tuple[seq][3] = $(seq_date_tuple[seq][3])")
        if seq_date_tuple[seq][1] ≠ 0 && seq_date_tuple[seq][2] ≠ 0
            new_date_tuple = (seq_date_tuple[seq][1], seq_date_tuple[seq][2], 15)
            seq_date_tuple[seq] = new_date_tuple
            seq_date_index[seq] = tuple_to_index[new_date_tuple]
            corrected_count += 1
        elseif seq_date_tuple[seq][2] == 0
            noncorrected_ct += 1
        end
    end
end
total_baddie_ct = corrected_count + noncorrected_ct
println("seq_date_index corrected = $(corrected_count)")
println("seq_date_index not corrected = $(noncorrected_ct)")
println("total_baddie_ct = $(total_baddie_ct)")
######################################################################################################################################################
runtime = time() - start
runtime_rd = round(digits=2, runtime)
runtime1, runtime2 = seconds_to_hrs_min_sec(runtime)
println("Runtime v0 = $(runtime) seconds")
println("Runtime v2 = $(runtime2)")
###################################################################################################################

seq_date_index corrected = 72
seq_date_index not corrected = 52
total_baddie_ct = 124
Runtime v0 = 0.3152179718017578 seconds
Runtime v2 = 0 hr, 0 min, 00.32 sec


In [7]:
### NEW | 2026_02_06 | Many, many functions (mixed_nuc, nuc_to_AA, etc) | Runtime: 2 min, 20.79 sec | 2026_02_13
start = time()
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
######################################################################################################################################
AA_triplets = Dict{String, String}("TTT"=>"F", "TTC"=>"F", "TTA"=>"L", "TTG"=>"L", "TCT"=>"S", "TCC"=>"S", "TCA"=>"S", "TCG"=>"S", "TAT"=>"Y", "TAC"=>"Y", "TAA"=>"*", "TAG"=>"*", "TGT"=>"C", "TGC"=>"C", "TGA"=>"*", "TGG"=>"W", "CTT"=>"L", "CTC"=>"L", "CTA"=>"L", "CTG"=>"L", "CCT"=>"P", "CCC"=>"P", "CCA"=>"P", "CCG"=>"P", "CAT"=>"H", "CAC"=>"H", "CAA"=>"Q", "CAG"=>"Q", "CGT"=>"R", "CGC"=>"R", "CGA"=>"R", "CGG"=>"R", "ATT"=>"I", "ATC"=>"I", "ATA"=>"I", "ATG"=>"M", "ACT"=>"T", "ACC"=>"T", "ACA"=>"T", "ACG"=>"T", "AAT"=>"N", "AAC"=>"N", "AAA"=>"K", "AAG"=>"K", "AGT"=>"S", "AGC"=>"S", "AGA"=>"R", "AGG"=>"R", "GTT"=>"V", "GTC"=>"V", "GTA"=>"V", "GTG"=>"V", "GCT"=>"A", "GCC"=>"A", "GCA"=>"A", "GCG"=>"A", "GAT"=>"D", "GAC"=>"D", "GAA"=>"E", "GAG"=>"E", "GGT"=>"G", "GGC"=>"G", "GGA"=>"G", "GGG"=>"G", "TT-"=>"X", "TC-"=>"X", "TA-"=>"X", "TG-"=>"X", "T-T"=>"X", "T-C"=>"X", "T-A"=>"X", "T-G"=>"X", "T--"=>"X", "CT-"=>"X", "CC-"=>"X", "CA-"=>"X", "CG-"=>"X", "C-T"=>"X", "C-C"=>"X", "C-A"=>"X", "C-G"=>"X", "C--"=>"X", "AT-"=>"X", "AC-"=>"X", "AA-"=>"X", "AG-"=>"X", "A-T"=>"X", "A-C"=>"X", "A-A"=>"X", "A-G"=>"X", "A--"=>"X", "GT-"=>"X", "GC-"=>"X", "GA-"=>"X", "GG-"=>"X", "G-T"=>"X", "G-C"=>"X", "G-A"=>"X", "G-G"=>"X", "G--"=>"X", "-TT"=>"X", "-TC"=>"X", "-TA"=>"X", "-TG"=>"X", "-T-"=>"X", "-CT"=>"X", "-CC"=>"X", "-CA"=>"X", "-CG"=>"X", "-C-"=>"X", "-AT"=>"X", "-AC"=>"X", "-AA"=>"X", "-AG"=>"X", "-A-"=>"X", "-GT"=>"X", "-GC"=>"X", "-GA"=>"X", "-GG"=>"X", "-G-"=>"X", "--T"=>"X", "--C"=>"X", "--A"=>"X", "--G"=>"X", "---"=>"X", "NTT"=>"X", "TNT"=>"X", "TTN"=>"X", "NTC"=>"X", "TNC"=>"X", "TCN"=>"X", "NTA"=>"X", "TNA"=>"X", "TAN"=>"X", "NTG"=>"X", "TNG"=>"X", "TGN"=>"X", "NT-"=>"X", "TN-"=>"X", "T-N"=>"X", "NTN"=>"X", "TNN"=>"X", "NCT"=>"X", "CNT"=>"X", "CTN"=>"X", "NCC"=>"X", "CNC"=>"X", "CCN"=>"X", "NCA"=>"X", "CNA"=>"X", "CAN"=>"X", "NCG"=>"X", "CNG"=>"X", "CGN"=>"X", "NC-"=>"X", "CN-"=>"X", "C-N"=>"X", "NCN"=>"X", "CNN"=>"X", "NAT"=>"X", "ANT"=>"X", "ATN"=>"X", "NAC"=>"X", "ANC"=>"X", "ACN"=>"X", "NAA"=>"X", "ANA"=>"X", "AAN"=>"X", "NAG"=>"X", "ANG"=>"X", "AGN"=>"X", "NA-"=>"X", "AN-"=>"X", "A-N"=>"X", "NAN"=>"X", "ANN"=>"X", "NGT"=>"X", "GNT"=>"X", "GTN"=>"X", "NGC"=>"X", "GNC"=>"X", "GCN"=>"X", "NGA"=>"X", "GNA"=>"X", "GAN"=>"X", "NGG"=>"X", "GNG"=>"X", "GGN"=>"X", "NG-"=>"X", "GN-"=>"X", "G-N"=>"X", "NGN"=>"X", "GNN"=>"X", "N-T"=>"X", "-NT"=>"X", "-TN"=>"X", "N-C"=>"X", "-NC"=>"X", "-CN"=>"X", "N-A"=>"X", "-NA"=>"X", "-AN"=>"X", "N-G"=>"X", "-NG"=>"X", "-GN"=>"X", "N--"=>"X", "-N-"=>"X", "--N"=>"X", "N-N"=>"X", "-NN"=>"X", "NNT"=>"X", "NNC"=>"X", "NNA"=>"X", "NNG"=>"X", "NN-"=>"X", "NNN"=>"X")                 
AA_triplet_dels = Dict{String, String}("TTT"=>"F", "TTC"=>"F", "TTA"=>"L", "TTG"=>"L", "TCT"=>"S", "TCC"=>"S", "TCA"=>"S", "TCG"=>"S", "TAT"=>"Y", "TAC"=>"Y", "TAA"=>"*", "TAG"=>"*", "TGT"=>"C", "TGC"=>"C", "TGA"=>"*", "TGG"=>"W", "CTT"=>"L", "CTC"=>"L", "CTA"=>"L", "CTG"=>"L", "CCT"=>"P", "CCC"=>"P", "CCA"=>"P", "CCG"=>"P", "CAT"=>"H", "CAC"=>"H", "CAA"=>"Q", "CAG"=>"Q", "CGT"=>"R", "CGC"=>"R", "CGA"=>"R", "CGG"=>"R", "ATT"=>"I", "ATC"=>"I", "ATA"=>"I", "ATG"=>"M", "ACT"=>"T", "ACC"=>"T", "ACA"=>"T", "ACG"=>"T", "AAT"=>"N", "AAC"=>"N", "AAA"=>"K", "AAG"=>"K", "AGT"=>"S", "AGC"=>"S", "AGA"=>"R", "AGG"=>"R", "GTT"=>"V", "GTC"=>"V", "GTA"=>"V", "GTG"=>"V", "GCT"=>"A", "GCC"=>"A", "GCA"=>"A", "GCG"=>"A", "GAT"=>"D", "GAC"=>"D", "GAA"=>"E", "GAG"=>"E", "GGT"=>"G", "GGC"=>"G", "GGA"=>"G", "GGG"=>"G", "TT-"=>"X", "TC-"=>"X", "TA-"=>"X", "TG-"=>"X", "T-T"=>"X", "T-C"=>"X", "T-A"=>"X", "T-G"=>"X", "T--"=>"-", "CT-"=>"X", "CC-"=>"X", "CA-"=>"X", "CG-"=>"X", "C-T"=>"X", "C-C"=>"X", "C-A"=>"X", "C-G"=>"X", "C--"=>"-", "AT-"=>"X", "AC-"=>"X", "AA-"=>"X", "AG-"=>"X", "A-T"=>"X", "A-C"=>"X", "A-A"=>"X", "A-G"=>"X", "A--"=>"-", "GT-"=>"X", "GC-"=>"X", "GA-"=>"X", "GG-"=>"X", "G-T"=>"X", "G-C"=>"X", "G-A"=>"X", "G-G"=>"X", "G--"=>"-", "-TT"=>"X", "-TC"=>"X", "-TA"=>"X", "-TG"=>"X", "-T-"=>"X", "-CT"=>"X", "-CC"=>"X", "-CA"=>"X", "-CG"=>"X", "-C-"=>"X", "-AT"=>"X", "-AC"=>"X", "-AA"=>"X", "-AG"=>"X", "-A-"=>"X", "-GT"=>"X", "-GC"=>"X", "-GA"=>"X", "-GG"=>"X", "-G-"=>"X", "--T"=>"-", "--C"=>"-", "--A"=>"-", "--G"=>"-", "---"=>"-", "NTT"=>"X", "TNT"=>"X", "TTN"=>"X", "NTC"=>"X", "TNC"=>"X", "TCN"=>"X", "NTA"=>"X", "TNA"=>"X", "TAN"=>"X", "NTG"=>"X", "TNG"=>"X", "TGN"=>"X", "NT-"=>"X", "TN-"=>"X", "T-N"=>"X", "NTN"=>"X", "TNN"=>"X", "NCT"=>"X", "CNT"=>"X", "CTN"=>"X", "NCC"=>"X", "CNC"=>"X", "CCN"=>"X", "NCA"=>"X", "CNA"=>"X", "CAN"=>"X", "NCG"=>"X", "CNG"=>"X", "CGN"=>"X", "NC-"=>"X", "CN-"=>"X", "C-N"=>"X", "NCN"=>"X", "CNN"=>"X", "NAT"=>"X", "ANT"=>"X", "ATN"=>"X", "NAC"=>"X", "ANC"=>"X", "ACN"=>"X", "NAA"=>"X", "ANA"=>"X", "AAN"=>"X", "NAG"=>"X", "ANG"=>"X", "AGN"=>"X", "NA-"=>"X", "AN-"=>"X", "A-N"=>"X", "NAN"=>"X", "ANN"=>"X", "NGT"=>"X", "GNT"=>"X", "GTN"=>"X", "NGC"=>"X", "GNC"=>"X", "GCN"=>"X", "NGA"=>"X", "GNA"=>"X", "GAN"=>"X", "NGG"=>"X", "GNG"=>"X", "GGN"=>"X", "NG-"=>"X", "GN-"=>"X", "G-N"=>"X", "NGN"=>"X", "GNN"=>"X", "N-T"=>"X", "-NT"=>"X", "-TN"=>"X", "N-C"=>"X", "-NC"=>"X", "-CN"=>"X", "N-A"=>"X", "-NA"=>"X", "-AN"=>"X", "N-G"=>"X", "-NG"=>"X", "-GN"=>"X", "N--"=>"X", "-N-"=>"X", "--N"=>"X", "N-N"=>"X", "-NN"=>"X", "NNT"=>"X", "NNC"=>"X", "NNA"=>"X", "NNG"=>"X", "NN-"=>"X", "NNN"=>"X")                 
######################################################################################################################################
function list_to_strings_no_spaces(list::String)
    string_vec = string.(split(list, ","))
    return string_vec
end
######################################################################################################################################
function stringlist_to_strings_nonEPI(txt::String)
    arr_of_strings = Vector{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        push!(arr_of_strings, seq)
    end
    sort_arr_of_strings = sort(collect(arr_of_strings), by = x -> nuc_mut_int_comprehensive_dict[x])  
    return sort_arr_of_strings
end
######################################################################################################################################
function list_to_string_array(txt::String) # similar to stringlist_to_strings but not for EPIs
    no_newlines = replace(txt, "\n" =>" ")
    string_array = string.(split(no_newlines, ", "))
    return string_array
end
######################################################################################################################################
function AA_sub_to_AA_pos(mut)
    gene = aa_gene_comprehensive_dict[mut]
    pos = aa_pos_comprehensive_dict[mut]
    AA_pos = aa_gene_and_pos_comprehensive_dict[mut]
    return AA_pos
end
###########################################################################################################################################################################
###########################################################################################################################################################################
###########################################################################################################################################################################
function get_ref_pango_nucseq_and_geneseqs(ref_pango::String)
    ref_seq = "..."
    refAA_ORF1a = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV"
    refAA_ORF1b = "NRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
    refAA_S = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
    refAA_ORF3a = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
    refAA_E = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
    refAA_M = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
    refAA_ORF6 = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
    refAA_ORF7a = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
    refAA_ORF7b = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
    refAA_ORF8 = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
    refAA_N = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
    refAA_ORF9b = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
    if ref_pango ≠ "Wuhan"
        if haskey(nuc_genome_pango_dict, ref_pango)
            ref_seq = nuc_genome_pango_dict[ref_pango]
        elseif haskey(pango_predecessor_meta_dict, ref_pango)
            minus1_pango = ""
            minus2_pango = ""
            minus3_pango = ""
            minus4_pango = ""
            if haskey(pango_predecessor_meta_dict[ref_pango], 1)
                minus1_pango = pango_predecessor_meta_dict[ref_pango][1]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 2)
                minus2_pango = pango_predecessor_meta_dict[ref_pango][2]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 3)
                minus3_pango = pango_predecessor_meta_dict[ref_pango][3]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 4)
                minus4_pango = pango_predecessor_meta_dict[ref_pango][4]
            end
            minus_pango_vec = [minus1_pango, minus2_pango, minus3_pango, minus4_pango]
            for minus_pango_X in minus_pango_vec
                if haskey(nuc_genome_pango_dict, minus_pango_X)
                    ref_seq = nuc_genome_pango_dict[minus_pango_X]
                    break
                end
            end
        else
            for x in 1:5
                minus_pango_X = pango_minus_X_fx(ref_pango, x)
                if haskey(nuc_genome_pango_dict, minus_pango_X)
                    ref_seq = nuc_genome_pango_dict[minus_pango_X]
                    break
                end
            end
        end
##########################################################################################
        if haskey(gene_AA_pango_dict, ref_pango)
            refAA_ORF1a = gene_AA_pango_dict[ref_pango]["ORF1a"]
            refAA_ORF1b = gene_AA_pango_dict[ref_pango]["ORF1b"]
            refAA_S = gene_AA_pango_dict[ref_pango]["S"]
            refAA_ORF3a = gene_AA_pango_dict[ref_pango]["ORF3a"]
            refAA_E = gene_AA_pango_dict[ref_pango]["E"]
            refAA_M = gene_AA_pango_dict[ref_pango]["M"]
            refAA_ORF6 = gene_AA_pango_dict[ref_pango]["ORF6"]
            refAA_ORF7a = gene_AA_pango_dict[ref_pango]["ORF7a"]
            refAA_ORF7b = gene_AA_pango_dict[ref_pango]["ORF7b"]
            refAA_ORF8 = gene_AA_pango_dict[ref_pango]["ORF8"]
            refAA_N = gene_AA_pango_dict[ref_pango]["N"]
            refAA_ORF9b = gene_AA_pango_dict[ref_pango]["ORF9b"]
        elseif haskey(pango_predecessor_meta_dict, ref_pango)
            minus1_pango = ""
            minus2_pango = ""
            minus3_pango = ""
            minus4_pango = ""
            if haskey(pango_predecessor_meta_dict[ref_pango], 1)
                minus1_pango = pango_predecessor_meta_dict[ref_pango][1]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 2)
                minus2_pango = pango_predecessor_meta_dict[ref_pango][2]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 3)
                minus3_pango = pango_predecessor_meta_dict[ref_pango][3]
            end
            if haskey(pango_predecessor_meta_dict[ref_pango], 4)
                minus4_pango = pango_predecessor_meta_dict[ref_pango][4]
            end
            minus_pango_vec = [minus1_pango, minus2_pango, minus3_pango, minus4_pango]
            for minus_pango_X in minus_pango_vec
                if haskey(gene_AA_pango_dict, minus_pango_X)
                    refAA_ORF1a = gene_AA_pango_dict[minus_pango_X]["ORF1a"]
                    refAA_ORF1b = gene_AA_pango_dict[minus_pango_X]["ORF1b"]
                    refAA_S = gene_AA_pango_dict[minus_pango_X]["S"]
                    refAA_ORF3a = gene_AA_pango_dict[minus_pango_X]["ORF3a"]
                    refAA_E = gene_AA_pango_dict[minus_pango_X]["E"]
                    refAA_M = gene_AA_pango_dict[minus_pango_X]["M"]
                    refAA_ORF6 = gene_AA_pango_dict[minus_pango_X]["ORF6"]
                    refAA_ORF7a = gene_AA_pango_dict[minus_pango_X]["ORF7a"]
                    refAA_ORF7b = gene_AA_pango_dict[minus_pango_X]["ORF7b"]
                    refAA_ORF8 = gene_AA_pango_dict[minus_pango_X]["ORF8"]
                    refAA_N = gene_AA_pango_dict[minus_pango_X]["N"]
                    refAA_ORF9b = gene_AA_pango_dict[minus_pango_X]["ORF9b"]
                    break
                end
            end
        else
            for x in 1:5
                minus_pango_X = pango_minus_X_fx(ref_pango, x)
                if haskey(gene_AA_pango_dict, minus_pango_X)
                    refAA_ORF1a = gene_AA_pango_dict[minus_pango_X]["ORF1a"]
                    refAA_ORF1b = gene_AA_pango_dict[minus_pango_X]["ORF1b"]
                    refAA_S = gene_AA_pango_dict[minus_pango_X]["S"]
                    refAA_ORF3a = gene_AA_pango_dict[minus_pango_X]["ORF3a"]
                    refAA_E = gene_AA_pango_dict[minus_pango_X]["E"]
                    refAA_M = gene_AA_pango_dict[minus_pango_X]["M"]
                    refAA_ORF6 = gene_AA_pango_dict[minus_pango_X]["ORF6"]
                    refAA_ORF7a = gene_AA_pango_dict[minus_pango_X]["ORF7a"]
                    refAA_ORF7b = gene_AA_pango_dict[minus_pango_X]["ORF7b"]
                    refAA_ORF8 = gene_AA_pango_dict[minus_pango_X]["ORF8"]
                    refAA_N = gene_AA_pango_dict[minus_pango_X]["N"]
                    refAA_ORF9b = gene_AA_pango_dict[minus_pango_X]["ORF9b"]
                    break
                end
            end
        end
    end
    return ref_seq, refAA_ORF1a, refAA_ORF1b, refAA_S, refAA_ORF3a, refAA_E, refAA_M, refAA_ORF6, refAA_ORF7a, refAA_ORF7b, refAA_ORF8, refAA_N, refAA_ORF9b
end
###########################################################################################################################################################################
###########################################################################################################################################################################
function mixed_nucs_filter(mut_arr)
    mixed_nuc_arr = Vector{String}()
    mixed_nuc_res_list = Set(["Y", "R", "K", "M", "W", "S"])
    for mut in mut_arr
        if string(mut[end]) in mixed_nuc_res_list
            push!(mixed_nuc_arr, mut)
        end
    end
    return mixed_nuc_arr
end
#####################################################################################################################################
N3_syn = ["TCT", "TCC", "TCA", "TCG", "CTT", "CTC", "CTA", "CTG", "CCT", "CCC", "CCA", "CCG", "CGT", "CGC", "CGA", "CGG", "ACT", "ACC", "ACA", "ACG", "GTT", "GTC", "GTA", "GTG", "GCT", "GCC", "GCA", "GCG", "GGT", "GGC", "GGA", "GGG"]
N3_tv = ["TTT", "TTC", "TTA", "TTG", "TAT", "TAC", "TAA", "TAG", "AAT", "AAC", "AAA", "AAG", "AGT", "AGC", "AGA", "AGG", "GAT", "GAC", "GAA", "GAG"]
#####################################################################################################################################
function mixed2nuc(mix_mut::String)
    nuc_mut = mix_mut
    qrynuc = qry_nuc_comprehensive_dict[mix_mut]
    refnuc = ref_nuc_comprehensive_dict[mix_mut]
    pos_str = nuc_mut_int_string_comprehensive_dict[mix_mut]
    ref_n_pos = refnuc*pos_str
    if length(mix_mut) ≥ 4
        if qrynuc == "T"
            nuc_mut = mix_mut
        elseif qrynuc == "C"
            nuc_mut = mix_mut
        elseif qrynuc == "A"
            nuc_mut = mix_mut
        elseif qrynuc == "G"
            nuc_mut = mix_mut
        elseif qrynuc == "N"
            nuc_mut = mix_mut
        end   
        if refnuc == "T"
            if qrynuc == "Y"
                nuc_mut = ref_n_pos*"C"
            elseif qrynuc == "W"
                nuc_mut = ref_n_pos*"A"
            elseif qrynuc == "K"
                nuc_mut = ref_n_pos*"G"
            elseif qrynuc == "M"
                nuc_mut = "$(ref_n_pos)C, $(ref_n_pos)A"
            elseif qrynuc == "S"
                nuc_mut = "$(ref_n_pos)C, $(ref_n_pos)G"
            elseif qrynuc == "R"
                nuc_mut = "$(ref_n_pos)A, $(ref_n_pos)G"
            end
        end
        if refnuc == "C"
            if qrynuc == "Y"
                nuc_mut = ref_n_pos*"T"
            elseif qrynuc == "M"
                nuc_mut = ref_n_pos*"A"
            elseif qrynuc == "S"
                nuc_mut = ref_n_pos*"G"
            elseif qrynuc == "R"
                nuc_mut = "$(ref_n_pos)A, $(ref_n_pos)G"
            elseif qrynuc == "W"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)A"
            elseif qrynuc == "K"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)G"
            end
        end
        if refnuc == "A"
            if qrynuc == "R"
                nuc_mut = ref_n_pos*"G"
            elseif qrynuc == "W"
                nuc_mut = ref_n_pos*"T"
            elseif qrynuc == "M"
                nuc_mut = ref_n_pos*"C"
            elseif qrynuc == "Y"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)C"
            elseif qrynuc == "K"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)G"
            elseif qrynuc == "S"
                nuc_mut = "$(ref_n_pos)C, $(ref_n_pos)G"
            end
        end
        if refnuc == "G"
            if qrynuc == "R"
                nuc_mut = ref_n_pos*"A"
            elseif qrynuc == "K"
                nuc_mut = ref_n_pos*"T"
            elseif qrynuc == "S"
                nuc_mut = ref_n_pos*"C"
            elseif qrynuc == "Y"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)C"
            elseif qrynuc == "W"
                nuc_mut = "$(ref_n_pos)T, $(ref_n_pos)A"
            elseif qrynuc == "M"
                nuc_mut = "$(ref_n_pos)C, $(ref_n_pos)A"
            end
        end
    end
    return nuc_mut
end
######################################################################################################################################
function muts_to_strings(mut_list_in_string_form::String)
    mut_arr = split(mut_list_in_string_form, ", ")
    mut_array = Vector{String}()
    for mut in mut_arr
#        if !haskey(qry_nuc_comprehensive_dict, mut)
#            println("No qry_nuc_comprehensive_dict key for $(mut)")
#            qry_nuc_comprehensive_dict[mut] = string(mut[end])
#            ref_nuc_comprehensive_dict[mut] = string(mut[1])
#            nuc_mut_int_comprehensive_dict[mut] = parse(Int, mut[2:end-1])
#            nuc_mut_int_string_comprehensive_dict = string(mut[2:end-1])
#        end
        if string(mut[end]) ≠ "-"
            mutstr = string(mut)
            push!(mut_array, mutstr)
        end
    end
#    mut_to_int = nuc_mut_int_comprehensive_dict[nuc_mut]
#    mut_to_int(n) = parse(Int, n[2:end-1])               ## Fx ##
    sortKey(n) = (length(n), n[2:end-1])  ## Fx ##
    mut_array_sort = sort(mut_array, by = x -> sortKey(x))
#    mut_array_sort = sort(mut_array, by = x -> (length(x[2:end-1]), x[2:end-1]))
#    mixed_mut_arr = mixed_nucs_filter(mut_arr)
    ct = 0
    return mut_array_sort
end
######################################################################################################################################
function mixed_mut_to_regular_mut(mixed_nuc_muts)            ### New, 2025-1-26 (entire function)
    ct = 0
    mixed_regular_muts = Vector{String}()
    for i in 1:length(mixed_nuc_muts)
        mut = mixed_nuc_muts[i]
        qrynuc = qry_nuc_comprehensive_dict[mut]
        refnuc = ref_nuc_comprehensive_dict[mut]
        pos_str = nuc_mut_int_string_comprehensive_dict[mut]
        ref_n_pos = refnuc*pos_str
        if refnuc == "T"
            if qrynuc == "Y"
                new_end = "C"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "R"
                new_end1 = "A"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "K"
                new_end = "G"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "M"
                new_end1 = "C"
                new_end2 = "A"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "W"
                new_end = "A"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "S"
                new_end1 = "C"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
        end
        if refnuc == "C"
            if qrynuc == "Y"
                new_end = "T"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "R"
                new_end1 = "A"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "K"
                new_end1 = "T"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "M"
                new_end = "A"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "W"
                new_end1 = "T"
                new_end2 = "A"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "S"
                new_end = "G"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
        end
        if refnuc == "A"
            if qrynuc == "Y"
                new_end1 = "T"
                new_end2 = "C"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "R"
                new_end = "G"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "K"
                new_end1 = "T"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "M"
                new_end = "C"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "W"
                new_end = "T"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "S"
                new_end1 = "C"
                new_end2 = "G"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
        end
        if refnuc == "G"
            if qrynuc == "Y"
                new_end1 = "T"
                new_end2 = "C"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "R"
                new_end = "A"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "K"
                new_end = "T"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
            if qrynuc == "M"
                new_end1 = "C"
                new_end2 = "A"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "W"
                new_end1 = "T"
                new_end2 = "A"
                new_mut1 = ref_n_pos*new_end1
                new_mut2 = ref_n_pos*new_end2
                push!(mixed_regular_muts, new_mut1)
                push!(mixed_regular_muts, new_mut2)
            end
            if qrynuc == "S"
                new_end = "C"
                new_mut = ref_n_pos*new_end
                push!(mixed_regular_muts, new_mut)
            end
        end
    end
    return mixed_regular_muts
end
######################################################################################################################################
gene_print_array = ["S", "N", "E", "M", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "ORF1a", "ORF1b"]
#######################################################################################################################################
function nuc_to_AA(ref_pango::String, muts::Vector{String})
    muts_filtered = filter(!isempty, muts)
#    all_muts_sort = sort(muts_filtered, by = x -> nuc_mut_int_comprehensive_dict[x])
    ref_seq, refAA_ORF1a, refAA_ORF1b, refAA_S, refAA_ORF3a, refAA_E, refAA_M, refAA_ORF6, refAA_ORF7a, refAA_ORF7b, refAA_ORF8, refAA_N, refAA_ORF9b = get_ref_pango_nucseq_and_geneseqs(ref_pango)
    coding_ranges = BitSet([266:13467..., 13469:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533..., 28284:28577...])
    noncoding_ranges = BitSet([1:265..., 21556:21562..., 25385:25392..., 26221:26244..., 26473:26522..., 27192:27201..., 27388:27393..., 27888:27893..., 28260:28273..., 29534:29830...])
    coding_range_9b = BitSet([28284:28577...])
    gene_nuc_starts = Dict{Int, Int}(0=>263, 1=>13465, 2=>21560, 3=>25390, 4=>26242, 5=>26520, 6=>27199, 7=>27391, 8=>27753, 9=>27891, 10=>28271, 11=>28281)
    ref_AA_genes = Dict{Int, String}(0=>refAA_ORF1a, 1=>refAA_ORF1b, 2=>refAA_S, 3=>refAA_ORF3a, 4=>refAA_E, 5=>refAA_M, 6=>refAA_ORF6, 7=>refAA_ORF7a, 8=>refAA_ORF7b, 9=>refAA_ORF8, 10=>refAA_N, 11=>refAA_ORF9b)
    gene_strings = Dict{Int, String}(0=>"ORF1a", 1=>"ORF1b", 2=>"S", 3=>"ORF3a", 4=>"E", 5=>"M", 6=>"ORF6", 7=>"ORF7a", 8=>"ORF7b", 9=>"ORF8", 10=>"N", 11=>"ORF9b")
    nuc_gene_num = Dict{Int, Int}()
    nuc_gene_num_9b = Dict{Int, Int}()
    nonsynonymous_nuc_to_AA_dict = Dict{String, String}()
    nonsynonymous_nuc_to_context_dict = Dict{String, String}()
    all_nuc_to_AA_dict = Dict{String, String}()
    all_nuc_to_context_dict = Dict{String, String}()
    synonymous_nuc_to_AA_dict = Dict{String, String}()
    synonymous_nuc_to_context_dict = Dict{String, String}()
    noncoding_nuc_to_context_dict = Dict{String, String}()
    noncoding_to_noncoding_region_dict = Dict{String, String}()
    noncoding_range_dict = Dict{Vector{Int}, String}([1, 265]=>"5' UTR", [21556, 21562]=>"Spike TRS", [25385, 25392]=>"ORF3a TRS", [26221, 26234]=>"ORF3a-E UTR", [26235, 26244]=>"E TRS", [26473, 26522]=>"E-M UTR", [27192, 27201]=>"M-ORF6 UTR", [27388, 27393]=>"ORF7a TRS", [27888, 27893]=>"ORF8 TRS", [28260, 28273]=>"N/ORF9b TRS", [29534, 29830]=>"3' UTR", [29794, 29801]=>"ONM")
    noncoding_nuc_vector = Vector{String}()
################################################        
    gene_nuc_arr = [[266:13467...], [13469:21555...], [21563:25384...], [25393:26220...], [26245:26472...], [26523:27191...], [27202:27387...], [27394:27755...], [27760:27887...], [27894:28259...], [28274:29533...], [28284:28577...]]
    for i in 1:length(gene_nuc_arr)-1
        for nuc_pos in gene_nuc_arr[i]
            nuc_gene_num[nuc_pos] = i-1
        end
    end
    for nuc_pos in gene_nuc_arr[end]
        nuc_gene_num_9b[nuc_pos] = 11
    end

    rem0_gene = [5, 8, 9, 11]
    rem1_gene = [1, 3, 4, 6, 7]
    rem2_gene = [0, 2, 10]
    rem0 = BitSet([26523:27191..., 27760:27887..., 27894:28259..., 28284:28577...])
    rem1 = BitSet([13469:21555..., 25393:26220..., 26245:26472..., 27202:27387..., 27394:27755...])
    rem2 = BitSet([266:13467..., 21563:25384..., 28274:29533...])
    rem9b = BitSet([28284:28577...])
    rem7ab = BitSet([27756:27759...])
    
    gene_num(nuc_mut) = nuc_gene_num[nuc_mut_int_comprehensive_dict[nuc_mut]]                                                          ### FUNCTION #####################
    nuc_to_AA_pos(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - gene_nuc_starts[gene_num(nuc_mut)])÷3)                  ### FUNCTION #####################
    nuc_to_AA_pos_9b(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - 28281)÷3)                                            ### FUNCTION #####################
    nuc2AA_ORF1a(nuc_mut, refAA, qryAA) = gene_strings[gene_num(nuc_mut)]*":"*refAA*nuc_to_AA_pos(nuc_mut)*qryAA   ### FUNCTION #####################
    nuc2AA_ORF9b(nuc_mut, refAA, qryAA) = "ORF9b:"*refAA*nuc_to_AA_pos_9b(nuc_mut)*qryAA
    
    nuc_codon_pos_dict = Dict{Int, Int}()
    for nuc_pos in coding_ranges
        gene_number = nuc_gene_num[nuc_pos]
        gene_start = gene_nuc_starts[gene_number]
        codon_num = (nuc_pos-gene_start)%3 + 1
        nuc_codon_pos_dict[nuc_pos] = codon_num
    end
    nuc_codon_pos_dict_9b = Dict{Int, Int}()
    for nuc_pos in coding_range_9b
        gene_number = 11
        gene_start = gene_nuc_starts[gene_number]
        codon_num = (nuc_pos-gene_start)%3 + 1
        nuc_codon_pos_dict_9b[nuc_pos] = codon_num
    end
#######################################################################################################################
    for nuc_mut in muts
        mut = mixed2nuc(nuc_mut)
        if ',' in mut
            mut1 = string(split(mut, ",")[1])
            mut2 = string(split(mut, ",")[2])
            push!(muts, mut1)
            push!(muts, mut2)
            filter!(x -> !(length(x)>6), muts)
        end
    end
    coding_ranges = BitSet([266:13467..., 13468:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533...])
    N_9b_synonymous = Set(["C28379A", "C28394A", "T28406C", "C28475A", "C28535A", "A28547C"])
    AA_mut_set = Set{String}()
    AA_mut = ""
    for nuc_mut in muts
        pos = nuc_mut_int_comprehensive_dict[nuc_mut]
        if pos in coding_ranges
            mut = mixed2nuc(nuc_mut)  
            gene_number = gene_num(mut)
            ref_triple = ""
            qry_triple = ""
            ref_triple_context = ""
            qry_triple_context = ""
            ref2qry_context = ""
            if nuc_codon_pos_dict[pos] == 1
                ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])  *string(ref_seq[pos+2])
                qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
                ref_triple_context = string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])*string(ref_seq[pos+7])*string(ref_seq[pos+8])
                qry_triple_context = string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])*string(ref_seq[pos+7])*string(ref_seq[pos+8])
            elseif nuc_codon_pos_dict[pos] == 2
                ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]  *string(ref_seq[pos+1])
                qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                ref_triple_context = string(ref_seq[pos-7])*string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*ref_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])*string(ref_seq[pos+7])
                qry_triple_context = string(ref_seq[pos-7])*string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*qry_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])*string(ref_seq[pos+7])
            elseif nuc_codon_pos_dict[pos] == 3
                ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                ref_triple_context = string(ref_seq[pos-8])*string(ref_seq[pos-7])*string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*ref_triple*string(ref_seq[pos+1])*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])
                qry_triple_context = string(ref_seq[pos-8])*string(ref_seq[pos-7])*string(ref_seq[pos-6])*string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*qry_triple*string(ref_seq[pos+1])*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])*string(ref_seq[pos+6])
            end
            refAA = AA_triplets[ref_triple]
            qryAA = AA_triplets[qry_triple]
            AA_mut = nuc2AA_ORF1a(mut, refAA, qryAA)
            push!(AA_mut_set, AA_mut)
            ref2qry_context = ref_triple_context*"-->"*qry_triple_context
            all_nuc_to_AA_dict[mut] = AA_mut
            if refAA == qryAA && !(pos in rem9b)
                synonymous_nuc_to_AA_dict[mut] = AA_mut
                synonymous_nuc_to_context_dict[mut] = ref2qry_context
            elseif refAA == qryAA && pos in rem9b && mut in N_9b_synonymous
                synonymous_nuc_to_AA_dict[mut] = AA_mut
                synonymous_nuc_to_context_dict[mut] = ref2qry_context
            else
                nonsynonymous_nuc_to_AA_dict[mut] = AA_mut
                nonsynonymous_nuc_to_context_dict[mut] = ref2qry_context
#                push!(nonsynonymous_nuc_muts, mut)
            end
            all_nuc_to_context_dict[mut] = ref2qry_context
###################################
            for nuc_mut2 in muts
                mut2 = mixed2nuc(nuc_mut2)
                pos2 = nuc_mut_int_comprehensive_dict[mut2]
                if pos2 in coding_ranges
                    gene_number2 = gene_num(mut2)
                    if mut ≠ mut2 && gene_number == gene_number2 && nuc_to_AA_pos(mut) == nuc_to_AA_pos(mut2)
                        if nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 2
                            ref_triple = ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                            qry_triple = qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                            ref_triple_context = string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])
                            qry_triple_context = string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])
                        elseif nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 3
                            ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*ref_nuc_comprehensive_dict[mut2]
                            qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*qry_nuc_comprehensive_dict[mut2]
                            ref_triple_context = string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])
                            qry_triple_context = string(ref_seq[pos-3])*string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_triple*string(ref_seq[pos+3])*string(ref_seq[pos+4])*string(ref_seq[pos+5])
                        elseif nuc_codon_pos_dict[pos] == 2 && nuc_codon_pos_dict[pos2] == 3
                            ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]
                            qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]
                            ref_triple_context = string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*ref_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])
                            qry_triple_context = string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*qry_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])
                        elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 2
                            ref_triple = ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                            qry_triple = qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                            ref_triple_context = string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*ref_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])
                            qry_triple_context = string(ref_seq[pos-4])*string(ref_seq[pos-3])*string(ref_seq[pos-2])*qry_triple*string(ref_seq[pos+2])*string(ref_seq[pos+3])*string(ref_seq[pos+4])
                        elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 3
                            ref_triple = ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                            qry_triple = qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                            ref_triple_context = string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*ref_triple*string(ref_seq[pos+1])*string(ref_seq[pos+2])*string(ref_seq[pos+3])
                            qry_triple_context = string(ref_seq[pos-5])*string(ref_seq[pos-4])*string(ref_seq[pos-3])*qry_triple*string(ref_seq[pos+1])*string(ref_seq[pos+2])*string(ref_seq[pos+3])
                        elseif nuc_codon_pos_dict[pos2] == 2 && nuc_codon_pos_dict[pos] == 3
                            ref_triple = string(ref_seq[pos-2])*ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]
                            qry_triple = string(ref_seq[pos-2])*qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]
                        end
                        refAA2 = AA_triplets[ref_triple]
                        qryAA2 = AA_triplets[qry_triple]
                        AA_mut2 = nuc2AA_ORF1a(mut2, refAA2, qryAA2)
                        push!(AA_mut_set, AA_mut2)
                        ref2qry_context = ref_triple_context*"-->"*qry_triple_context
                        all_nuc_to_AA_dict[mut2] = AA_mut2
                        all_nuc_to_AA_dict[mut] = AA_mut2
                        if refAA2 == qryAA2 && !(pos2 in rem9b)
                            synonymous_nuc_to_AA_dict[mut2] = AA_mut2
                            synonymous_nuc_to_context_dict[mut2] = ref2qry_context 
                        else
                            nonsynonymous_nuc_to_AA_dict[mut2] = AA_mut2
                            nonsynonymous_nuc_to_context_dict[mut2] = ref2qry_context
                            nonsynonymous_nuc_to_AA_dict[mut] = AA_mut2
                            nonsynonymous_nuc_to_context_dict[mut] = ref2qry_context
                        end
                        all_nuc_to_context_dict[mut] = ref2qry_context
                        all_nuc_to_context_dict[mut2] = ref2qry_context
                    end
                end
            end
        else                  
            npos = nuc_mut_int_comprehensive_dict[nuc_mut]
            qry_nuc = qry_nuc_comprehensive_dict[nuc_mut]
            ref_nc_nuc_context = string(ref_seq[npos-8])*string(ref_seq[npos-7])*string(ref_seq[npos-6])*string(ref_seq[npos-5])*string(ref_seq[npos-4])*string(ref_seq[npos-3])*string(ref_seq[npos-2])*string(ref_seq[npos-1])*string(ref_seq[npos])*string(ref_seq[npos+1])*string(ref_seq[npos+2])*string(ref_seq[npos+3])*string(ref_seq[npos+4])*string(ref_seq[npos+5])*string(ref_seq[npos+6])*string(ref_seq[npos+7])*string(ref_seq[npos+8])
            qry_nc_nuc_context = string(ref_seq[npos-8])*string(ref_seq[npos-7])*string(ref_seq[npos-6])*string(ref_seq[npos-5])*string(ref_seq[npos-4])*string(ref_seq[npos-3])*string(ref_seq[npos-2])*string(ref_seq[npos-1])*qry_nuc*string(ref_seq[npos+1])*string(ref_seq[npos+2])*string(ref_seq[npos+3])*string(ref_seq[npos+4])*string(ref_seq[npos+5])*string(ref_seq[npos+6])*string(ref_seq[npos+7])*string(ref_seq[npos+8])
            full_nc_context = ref_nc_nuc_context*"|"*qry_nc_nuc_context
            noncoding_nuc_to_context_dict[nuc_mut] = full_nc_context
            for (start_end, place) in noncoding_range_dict
                frst = start_end[1]
                last = start_end[2]
                if npos ≥ frst && npos ≤ last
                    noncoding_to_noncoding_region_dict[nuc_mut] = place
                    mut_vec = [nuc_mut, place]
                    push!(noncoding_nuc_vector, nuc_mut)
                end
            end
        end
    end
#########################################################################################################
    for nuc_mut in muts
        pos_9b = nuc_mut_int_comprehensive_dict[nuc_mut]
        if pos_9b in rem9b
            mut_9b = mixed2nuc(nuc_mut)
            pos_9b = nuc_mut_int_comprehensive_dict[mut_9b]   
            gene_number_9b = 11
            ref_triple_9b = ""
            qry_triple_9b = ""
            ref_triple_context_9b = ""
            qry_triple_context_9b = ""
            ref2qry_context_9b = ""
            if nuc_codon_pos_dict_9b[pos_9b] == 1
                ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])  *string(ref_seq[pos_9b+2])
                qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])
                ref_triple_context_9b = string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])*string(ref_seq[pos_9b+7])*string(ref_seq[pos_9b+8])
                qry_triple_context_9b = string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])*string(ref_seq[pos_9b+7])*string(ref_seq[pos_9b+8])
            elseif nuc_codon_pos_dict_9b[pos_9b] == 2
                ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]  *string(ref_seq[pos_9b+1])
                qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                ref_triple_context_9b = string(ref_seq[pos_9b-7])*string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*ref_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])*string(ref_seq[pos_9b+7])
                qry_triple_context_9b = string(ref_seq[pos_9b-7])*string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*qry_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])*string(ref_seq[pos_9b+7])
            elseif nuc_codon_pos_dict_9b[pos_9b] == 3
                ref_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                qry_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                ref_triple_context_9b = string(ref_seq[pos_9b-8])*string(ref_seq[pos_9b-7])*string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*ref_triple_9b*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])
                qry_triple_context_9b = string(ref_seq[pos_9b-8])*string(ref_seq[pos_9b-7])*string(ref_seq[pos_9b-6])*string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*qry_triple_9b*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])*string(ref_seq[pos_9b+6])
            end
            refAA_9b = AA_triplets[ref_triple_9b]
            qryAA_9b = AA_triplets[qry_triple_9b]
            AA_mut_9b = nuc2AA_ORF9b(mut_9b, refAA_9b, qryAA_9b)
            push!(AA_mut_set, AA_mut_9b)
            ref2qry_context_9b = ref_triple_context_9b*"-->"*qry_triple_context_9b
            all_nuc_to_AA_dict[mut_9b] = AA_mut_9b
            if refAA_9b == qryAA_9b && nuc_mut in N_9b_synonymous
                synonymous_nuc_to_AA_dict[mut_9b] = AA_mut_9b
                synonymous_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
            end
            if refAA_9b ≠ qryAA_9b
                nonsynonymous_nuc_to_AA_dict[mut_9b] = AA_mut_9b
                nonsynonymous_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
#                push!(nonsynonymous_nuc_muts, mut_9b)
            end
            all_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
###################################
            for nuc_mut2_9b in muts
                mut2_9b = mixed2nuc(nuc_mut2_9b)
                pos2_9b = nuc_mut_int_comprehensive_dict[mut2_9b]
                if pos2_9b in rem9b
                    gene_number2_9b = 11
                    if mut_9b ≠ mut2_9b && nuc_to_AA_pos_9b(mut_9b) == nuc_to_AA_pos_9b(mut2_9b)
                        if nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 2
                            ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                            qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                            ref_triple_context_9b = string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])
                            qry_triple_context_9b = string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])
                        elseif nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                            ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*ref_nuc_comprehensive_dict[mut2_9b]
                            qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*qry_nuc_comprehensive_dict[mut2_9b]
                            ref_triple_context_9b = string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])
                            qry_triple_context_9b = string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_triple_9b*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])*string(ref_seq[pos_9b+5])
                        elseif nuc_codon_pos_dict_9b[pos_9b] == 2 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                            ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]
                            qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]
                            ref_triple_context_9b = string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*ref_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])
                            qry_triple_context_9b = string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*qry_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])
                        elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 2
                            ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                            qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                            ref_triple_context_9b = string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*ref_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])
                            qry_triple_context_9b = string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*string(ref_seq[pos_9b-2])*qry_triple_9b*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])*string(ref_seq[pos_9b+4])
                        elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 3
                            ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                            qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                            ref_triple_context_9b = string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*ref_triple_9b*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])
                            qry_triple_context_9b = string(ref_seq[pos_9b-5])*string(ref_seq[pos_9b-4])*string(ref_seq[pos_9b-3])*qry_triple_9b*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])*string(ref_seq[pos_9b+3])
                        elseif nuc_codon_pos_dict_9b[pos2_9b] == 2 && nuc_codon_pos_dict_9b[pos_9b] == 3
                            ref_triple_9b = string(ref_seq[pos_9b-2])*ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]
                            qry_triple_9b = string(ref_seq[pos_9b-2])*qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]
                        end
                        refAA2_9b = AA_triplets[ref_triple_9b]
                        qryAA2_9b = AA_triplets[qry_triple_9b]
                        AA_mut2_9b = nuc2AA_ORF9b(mut2_9b, refAA2_9b, qryAA2_9b)
                        push!(AA_mut_set, AA_mut2_9b)
                        ref2qry_context_9b = ref_triple_context_9b*"-->"*qry_triple_context_9b
                        if refAA2_9b == qryAA2_9b && nuc_mut2_9b in N_9b_synonymous 
                            synonymous_nuc_to_AA_dict[mut2_9b] = AA_mut2_9b
                            synonymous_nuc_to_context_dict[mut2_9b] = ref2qry_context_9b
                            synonymous_nuc_to_AA_dict[mut_9b] = AA_mut2_9b
                            synonymous_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
                        else
                            nonsynonymous_nuc_to_AA_dict[mut2_9b] = AA_mut2_9b
                            nonsynonymous_nuc_to_context_dict[mut2_9b] = ref2qry_context_9b
                            nonsynonymous_nuc_to_AA_dict[mut_9b] = AA_mut2_9b
                            nonsynonymous_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
                        end
                        all_nuc_to_context_dict[mut_9b] = ref2qry_context_9b
                        all_nuc_to_context_dict[mut2_9b] = ref2qry_context_9b
                    end
                end
            end
        end
    end
#################################################################################
#    mut_vec_dict = Dict{String, Vector{String}}()
#    for gene in gene_print_array
#        mut_vec_dict[gene] = Vector{String}()
#    end
#    for mut in AA_mut_set
#        jeen = aa_gene_comprehensive_dict[mut]
#        mut_only = string(split(mut, ":")[2])
#        push!(mut_vec_dict[jeen], mut_only) 
#    end
#    for gene in keys(mut_vec_dict)
#        if !isempty(mut_vec_dict[gene])
#            sort!(mut_vec_dict[gene], by = x -> x[2:end-1])
#        end
#    end
#####################################################################################################################################
#    aasortkeyhere(m) = (gene_order_dict[aa_gene_comprehensive_dict[m]], aa_pos_comprehensive_dict[m])
#    AA_sort = sort(collect(AA_mut_set), by = x -> aasortkeyhere(x))
#    AA_sort2 = Vector{String}()
#    for mut in AA_sort
#        refAA = ref_AA_dict[mut]
#        qryAA = qry_AA_dict[mut]
#        if !(refAA == qryAA)
#            push!(AA_sort2, mut)
#        end
#    end
#    print("\n"^2)
#    ct = 0
#    println("##################### Amino Acid Mutations ######################")
#    for i in 1:length(AA_sort2)
#        mut = AA_sort2[i]
#        gene = aa_gene_comprehensive_dict[mut]
#        non_gene = string(split(mut, ":")[2])
#        if i == 1
#            print("               ", AA_sort2[i])
#        elseif i > 1
#            lastmut = AA_sort2[i-1]
#            last_gene = string(split(lastmut, ":")[1])
#            last_non_gene = string(split(lastmut, ":")[2])
#            if gene == last_gene
#                print(", $(non_gene)")
#            else
#                println()
#                print("               $(mut)")
#            end
#        end
#    end
#    print("\n"^2)
#####################################################################################################################################
    all_nuc_to_AA_dict_sort = sort(collect(all_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
    all_nuc_to_context_dict_sort = sort(collect(all_nuc_to_context_dict), by = x -> nuc_pos_fx(x[1]))
    nonsynonymous_nuc_to_AA_dict_sort = sort(collect(nonsynonymous_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
    nonsynonymous_nuc_sort = sort(collect(keys(nonsynonymous_nuc_to_AA_dict)), by = x -> nuc_mut_int_comprehensive_dict[x])
    nonsynonymous_AA_sort = sort(collect(values(nonsynonymous_nuc_to_AA_dict)), by = x -> AA_gene_sortKey_2(x))
    nonsynonymous_nuc_to_context_dict_sort = sort(collect(nonsynonymous_nuc_to_context_dict), by = x -> nuc_pos_fx(x[1]))
    synonymous_nuc_to_AA_dict_sort = sort(collect(synonymous_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
    synonymous_nuc_sort = sort(collect(keys(synonymous_nuc_to_AA_dict)), by = x -> nuc_mut_int_comprehensive_dict[x])
    synonymous_AA_sort = sort(collect(values(synonymous_nuc_to_AA_dict)), by = x -> AA_gene_sortKey_2(x))
    synonymous_nuc_to_context_dict_sort = sort(collect(synonymous_nuc_to_context_dict), by = x -> nuc_pos_fx(x[1]))
    noncoding_nuc_to_context_dict_sort = sort(collect(noncoding_nuc_to_context_dict), by = x -> nuc_pos_fx(x[1]))
    noncoding_nuc_vector_sort = sort(collect(noncoding_nuc_vector), by = x -> nuc_mut_int_comprehensive_dict[x])
    noncoding_to_noncoding_region_dict_sort = sort(collect(noncoding_to_noncoding_region_dict), by = x -> nuc_pos_fx(x[1]))
####################################################################################################################################
    nc_len = length(noncoding_to_noncoding_region_dict_sort)
#    println("NONCODING NUC MUTATIONS - Total:$(nc_len)")
    if !isempty(noncoding_to_noncoding_region_dict_sort)
        for i in 1:length(noncoding_to_noncoding_region_dict_sort)
            nc_nuc = noncoding_to_noncoding_region_dict_sort[i][1]
            nc_nuc_pad = rpad(nc_nuc, 7)
            nc_region = noncoding_to_noncoding_region_dict_sort[i][2]
            nc_region_len = length(nc_region)
            ncpad1 = (13 - nc_region_len)÷2
            ncpad12 = " "^ncpad1*nc_region
            nc_region_pad2 = rpad(ncpad12, 13)
            nc_context = noncoding_nuc_to_context_dict_sort[i][2]
            premut_context = ""
            postmut_context = ""
            context = split(nc_context, "-->")
            if !isempty(context)
                if length(context) >1
                    premut_context = split(nc_context, "-->")[1]
                    postmut_context = split(nc_context, "-->")[2]
                end
            end    
            postpad = lpad(postmut_context, 39)
#            println("$(nc_nuc_pad)|$(nc_region_pad2)|$(premut_context)")
#            println(postpad)
#                println(g, "$(nc_nuc_pad)|$(nc_region_pad2)|$(premut_context)")
#                println(g, postpad)
        end
#        println("#"^94); println() 
#            println(g, "#"^94); println(g)
    end
######################################################################################################
    total_syn = length(synonymous_nuc_to_AA_dict_sort)
#    println("SYNONYMOUS NUC MUTATIONS — Total:$(total_syn)")
    for i in 1:length(synonymous_nuc_to_AA_dict_sort)
        synnuc = synonymous_nuc_to_AA_dict_sort[i][1]
        synnuc_pad = rpad(synnuc, 7)
        synAA = synonymous_nuc_to_AA_dict_sort[i][2]
        AAlen = length(synAA)
        pad1 = (14 - AAlen)÷2
        pad12 = " "^pad1*synAA
        synAA_pad = rpad(pad12, 14)
        syncontext = synonymous_nuc_to_context_dict_sort[i][2]
        premut_context = split(syncontext, "-->")[1]
        postmut_context = split(syncontext, "-->")[2]
        postpad = lpad(postmut_context, 38)
#        println("$(synnuc_pad)|$(synAA_pad)|$(premut_context)")
#        println(postpad)
#            println(g, "$(synnuc_pad)|$(synAA_pad)|$(premut_context)")
#            println(g, postpad)
    end
#    println()
#################################################################
    total_syn = length(nonsynonymous_nuc_to_AA_dict_sort)
#    println("NONSYNONYMOUS NUC MUTATIONS — Total:$(total_syn)")
    for i in 1:length(nonsynonymous_nuc_to_AA_dict_sort)
        synnuc = nonsynonymous_nuc_to_AA_dict_sort[i][1]
        synnuc_pad = rpad(synnuc, 7)
        synAA = nonsynonymous_nuc_to_AA_dict_sort[i][2]
        AAlen = length(synAA)
        pad1 = (14 - AAlen)÷2
        pad12 = " "^pad1*synAA
        synAA_pad = rpad(pad12, 14)
        syncontext = nonsynonymous_nuc_to_context_dict_sort[i][2]
        premut_context = split(syncontext, "-->")[1]
        postmut_context = split(syncontext, "-->")[2]
        postpad = lpad(postmut_context, 38)
#        println("$(synnuc_pad)|$(synAA_pad)|$(premut_context)")
#        println(postpad)
#            println(g, "$(synnuc_pad)|$(synAA_pad)|$(premut_context)")
#            println(g, postpad)
    end
#    println()
###################################################################
    nonsynonymous_nuc_total = length(nonsynonymous_nuc_sort)
#    println("        Total Number of Non-synonymous Nuc Muts = $(nonsynonymous_nuc_total)")
    nonsynonymous_nuc_sort_join = join(nonsynonymous_nuc_sort, ", ")
#    println("################ Nonsynonymous Nuc Mutations ################")
#    println(nonsynonymous_nuc_sort_join)
#    print("\n"^1)
    for i in 1:length(nonsynonymous_nuc_sort)
#        println("               $(nonsynonymous_nuc_to_AA_dict_sort[i][1]) | $(nonsynonymous_nuc_to_AA_dict_sort[i][2])")
        premut_nucseq = split(nonsynonymous_nuc_to_context_dict_sort[i][2], "-->")[1]
        postmut_nucseq = split(nonsynonymous_nuc_to_context_dict_sort[i][2], "-->")[2]
#        println("                 $(premut_nucseq)")
#        println("                 $(postmut_nucseq)")
    end
#synonymous_nuc_to_context_dict[mut] = ref_triple_context*"-->"*qry_triple_context
    return synonymous_nuc_to_AA_dict_sort, nonsynonymous_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort
end
###########################################################################################################################################################################
###########################################################################################################################################################################
function mixed_muts_to_AA(ref_pango::String, muts::String)
    mut_strings = muts_to_strings(muts)
    mixed_muts = mixed_nucs_filter(mut_strings)             ### New, 2025-1-26
    mixed_muts_regular = mixed_mut_to_regular_mut(mixed_muts)    ### New, 2025-1-26
    ct = 0
    total_mixed_muts = length(mixed_muts)
#    println("Total Mixed Nucs = $(total_mixed_muts)")
#    println(); println()
    for i in 1:length(mixed_muts)
        if ct == 0
#            print(mixed_muts[i])
            ct = 1
        else
#            print(", ", mixed_muts[i])
        end
    end
    ct2 = 0
#    println(); println()
    for i in 1:length(mixed_muts_regular)
        if ct2 == 0
#            print(mixed_muts_regular[i])
            ct2 = 1
#        else
#            print(", ", mixed_muts_regular[i])
        end
    end
#    println()
    syn_nuc_to_AA_dict_sort, nonsyn_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort = nuc_to_AA(ref_pango, mixed_muts_regular)
    AA__AAprintlen__vec = []
    nonsynnuc__nonsynprintlen__vec = []
    for nuc___AA in nonsyn_nuc_to_AA_dict_sort
        nucmut = nuc___AA[1]
        nucmutlen = length(nucmut)
        AAmut = nuc___AA[2]
        AAmutlen = length(AAmut)
        push!(AA__AAprintlen__vec, [AAmut, AAmutlen])
        push!(nonsynnuc__nonsynprintlen__vec, [nucmut, nucmutlen])
    end
    aa_pad_vec = String[]
    nuc_pad_vec = String[]
    for i in 1:length(AA__AAprintlen__vec)
        aa = AA__AAprintlen__vec[i][1]
        nuc = nonsynnuc__nonsynprintlen__vec[i][1]
        aapad = AA__AAprintlen__vec[i][2]
        nucpad = nonsynnuc__nonsynprintlen__vec[i][2]
        pads = [nucpad, aapad]
        pad = maximum(pads)
        push!(aa_pad_vec, rpad(aa, pad))
        push!(nuc_pad_vec, rpad(nuc, pad))
    end
    aapad_join = join(aa_pad_vec, ", ")
    nucpad_join = join(nuc_pad_vec, ", ")
#    println("\n"^1)
#    println(aapad_join)
#    println(nucpad_join)
#    println("\n"^1)
    return syn_nuc_to_AA_dict_sort, nonsyn_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort
end
######################################################################################################################################
function count_nuc_mut_types(mut_strings::Vector{String})
    mut_types_arr = ["TC", "TA", "TG", "CT", "CA", "CG", "AT", "AC", "AG", "GT", "GC", "GA"]
    mut_type_cts = Dict{String, Int}(mut_nuc_type=>0 for mut_nuc_type in mut_types_arr)
    for nuc_mut in mut_strings
        ref = ref_nuc_comprehensive_dict[nuc_mut]
        qry = qry_nuc_comprehensive_dict[nuc_mut]
        if ref == "T"
            if qry == "C"
                mut_type_cts["TC"] += 1
            elseif qry == "A"
                mut_type_cts["TA"] += 1
            elseif qry == "G"
                mut_type_cts["TG"] += 1
            end
        end
        if ref == "C"
            if qry == "T"
                mut_type_cts["CT"] += 1
            elseif qry == "A"
                mut_type_cts["CA"] += 1
            elseif qry == "G"
                mut_type_cts["CG"] += 1
            end
        end 
        if ref == "A"
            if qry == "T"
                mut_type_cts["AT"] += 1
            elseif qry == "C"
                mut_type_cts["AC"] += 1
            elseif qry == "G"
                mut_type_cts["AG"] += 1
            end
        end   
        if ref == "G"
            if qry == "T"
                mut_type_cts["GT"] += 1
            elseif qry == "C"
                mut_type_cts["GC"] += 1
            elseif qry == "A"
                mut_type_cts["GA"] += 1
            end
        end
    end
    mut_type_cts_sort_by_type = sort(collect(mut_type_cts), by = x -> x[1])
    mut_type_cts_sort_by_count = sort(collect(mut_type_cts), by = x -> x[2], rev=true)
    return mut_type_cts_sort_by_count
end
######################################################################################################################################
function AA_triple(pos::Int, rem0::BitSet, rem1::BitSet, rem2::BitSet, mut::String, ref_pango::String)
    ref_seq, refAA_ORF1a, refAA_ORF1b, refAA_S, refAA_ORF3a, refAA_E, refAA_M, refAA_ORF6, refAA_ORF7a, refAA_ORF7b, refAA_ORF8, refAA_N, refAA_ORF9b = get_ref_pango_nucseq_and_geneseqs(ref_pango)
    ref_triple = ""
    qry_triple = ""
    if pos in rem0 && pos%3 == 0
        ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
        qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
    elseif pos in rem1 && pos%3 == 1
        ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
        qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
    elseif pos in rem2 && pos%3 == 2
        ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
        qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
    elseif pos in rem0 && pos%3 == 1
        ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
        qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
    elseif pos in rem1 && pos%3 == 2
        ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
        qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
    elseif pos in rem2 && pos%3 == 0
        ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
        qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
    elseif pos in rem0 && pos%3 == 2
        ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
        qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
    elseif pos in rem1 && pos%3 == 0
        ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
        qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
    elseif pos in rem2 && pos%3 == 1
        ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
        qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
    end
    return ref_triple, qry_triple
end
######################################################################################################################################
### Fx: SIMPLER_nuc_to_AA (no context)
function SIMPLER_nuc_to_AA(ref_pango::String, muts::Vector{String})
    muts = filter(!isempty, muts)
    if !isempty(muts)
#        all_muts_sort = sort(collect(muts), by = x -> nuc_mut_int_comprehensive_dict[x])
        ref_seq, refAA_ORF1a, refAA_ORF1b, refAA_S, refAA_ORF3a, refAA_E, refAA_M, refAA_ORF6, refAA_ORF7a, refAA_ORF7b, refAA_ORF8, refAA_N, refAA_ORF9b = get_ref_pango_nucseq_and_geneseqs(ref_pango)
    ###############################################################################
        coding_ranges = BitSet([266:13467..., 13469:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533..., 28284:28577...])
        noncoding_ranges = BitSet([1:265..., 21556:21562..., 25385:25392..., 26221:26244..., 26473:26522..., 27192:27201..., 27388:27393..., 27888:27893..., 28260:28273..., 29534:29830...])
        coding_range_9b = BitSet([28284:28577...])
        gene_nuc_starts = Dict{Int, Int}(0=>263, 1=>13465, 2=>21560, 3=>25390, 4=>26242, 5=>26520, 6=>27199, 7=>27391, 8=>27753, 9=>27891, 10=>28271, 11=>28281)
        ref_AA_genes = Dict{Int, String}(0=>refAA_ORF1a, 1=>refAA_ORF1b, 2=>refAA_S, 3=>refAA_ORF3a, 4=>refAA_E, 5=>refAA_M, 6=>refAA_ORF6, 7=>refAA_ORF7a, 8=>refAA_ORF7b, 9=>refAA_ORF8, 10=>refAA_N, 11=>refAA_ORF9b)
        gene_strings = Dict{Int, String}(0=>"ORF1a", 1=>"ORF1b", 2=>"S", 3=>"ORF3a", 4=>"E", 5=>"M", 6=>"ORF6", 7=>"ORF7a", 8=>"ORF7b", 9=>"ORF8", 10=>"N", 11=>"ORF9b")
        nuc_gene_num = Dict{Int, Int}()
        nuc_gene_num_9b = Dict{Int, Int}()
        synonymous_nuc_to_AA_dict = Dict{String, String}()
        nonsynonymous_nuc_to_AA_dict = Dict{String, String}()
        all_nuc_to_AA_dict = Dict{String, String}()
        noncoding_range_dict = Dict{Vector{Int}, String}([1, 265]=>"5' UTR", [21556, 21562]=>"Spike TRS", [25385, 25392]=>"ORF3a TRS", [26221, 26234]=>"ORF3a-E UTR", [26235, 26244]=>"E TRS", [26473, 26522]=>"E-M UTR", [27192, 27201]=>"M-ORF6 UTR", [27388, 27393]=>"ORF7a TRS", [27888, 27893]=>"ORF8 TRS", [28260, 28273]=>"N/ORF9b TRS", [29534, 29830]=>"3' UTR", [29794, 29801]=>"Octanucleotide Motif")
################################################
        noncoding_nuc_vector = Vector{String}()
################################################ 
        gene_nuc_arr = [[266:13467...], [13469:21555...], [21563:25384...], [25393:26220...], [26245:26472...], [26523:27191...], [27202:27387...], [27394:27755...], [27760:27887...], [27894:28259...], [28274:29533...], [28284:28577...]]
        for i in 1:length(gene_nuc_arr)-1
            for nuc_pos in gene_nuc_arr[i]
                nuc_gene_num[nuc_pos] = i-1
            end
        end
        for nuc_pos in gene_nuc_arr[end]
            nuc_gene_num_9b[nuc_pos] = 11
        end
        rem0_gene = [5, 8, 9, 11]
        rem1_gene = [1, 3, 4, 6, 7]
        rem2_gene = [0, 2, 10]
        rem0 = BitSet([26523:27191..., 27760:27887..., 27894:28259..., 28284:28577...])
        rem1 = BitSet([13469:21555..., 25393:26220..., 26245:26472..., 27202:27387..., 27394:27755...])
        rem2 = BitSet([266:13467..., 21563:25384..., 28274:29533...])
        rem9b = BitSet([28284:28577...])
        rem7ab = BitSet([27756:27759...])

        gene_num(nuc_mut) = nuc_gene_num[nuc_mut_int_comprehensive_dict[nuc_mut]] ## Fx ##
        nuc_to_AA_pos(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - gene_nuc_starts[gene_num(nuc_mut)])÷3) ## Fx ##
        nuc_to_AA_pos_9b(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - 28281)÷3) ## Fx ##
        nuc2AA_ORF1a(nuc_mut, refAA, qryAA) = gene_strings[gene_num(nuc_mut)]*":"*refAA*nuc_to_AA_pos(nuc_mut)*qryAA ## Fx ##
        nuc2AA_ORF9b(nuc_mut, refAA, qryAA) = "ORF9b:"*refAA*nuc_to_AA_pos_9b(nuc_mut)*qryAA
        nuc_codon_pos_dict = Dict{Int, Int}()
        for nuc_pos in coding_ranges
            gene_number = nuc_gene_num[nuc_pos]
            gene_start = gene_nuc_starts[gene_number]
            codon_num = (nuc_pos-gene_start)%3 + 1
            nuc_codon_pos_dict[nuc_pos] = codon_num
        end
        nuc_codon_pos_dict_9b = Dict{Int, Int}()
        for nuc_pos in coding_range_9b
            gene_number = 11
            gene_start = gene_nuc_starts[gene_number]
            codon_num = (nuc_pos-gene_start)%3 + 1
            nuc_codon_pos_dict_9b[nuc_pos] = codon_num
        end    
        N3_syn = ["TCT", "TCC", "TCA", "TCG", "CTT", "CTC", "CTA", "CTG", "CCT", "CCC", "CCA", "CCG", "CGT", "CGC", "CGA", "CGG", "ACT", "ACC", "ACA", "ACG", "GTT", "GTC", "GTA", "GTG", "GCT", "GCC", "GCA", "GCG", "GGT", "GGC", "GGA", "GGG"]
        N3_tv = ["TTT", "TTC", "TTA", "TTG", "TAT", "TAC", "TAA", "TAG", "AAT", "AAC", "AAA", "AAG", "AGT", "AGC", "AGA", "AGG", "GAT", "GAC", "GAA", "GAG"]
        for nuc_mut in muts
            if !isempty(muts)
                mut = mixed2nuc(nuc_mut)
                if ',' in mut
                    mut1 = string(split(mut, ",")[1])
                    mut2 = string(split(mut, ",")[2])
                    push!(muts, mut1)
                    push!(muts, mut2)
                    filter!(x -> !(length(x)>6), muts)
                end
            end
        end
        coding_ranges = BitSet([266:13467..., 13468:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533...])
        N_9b_synonymous = Set(["C28379A", "C28394A", "T28406C", "C28475A", "C28535A", "A28547C"])
        AA_mut_set = Set{String}()
        AA_mut = ""
        for nuc_mut in muts
            pos = nuc_mut_int_comprehensive_dict[nuc_mut]
            if pos in coding_ranges
                mut = mixed2nuc(nuc_mut)  
                gene_number = gene_num(mut)
                ref_triple = ""
                qry_triple = ""
                if nuc_codon_pos_dict[pos] == 1
                    ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
                    qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
                elseif nuc_codon_pos_dict[pos] == 2
                    ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                    qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                elseif nuc_codon_pos_dict[pos] == 3
                    ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                    qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                end
                refAA = AA_triplets[ref_triple]
                qryAA = AA_triplets[qry_triple]
                AA_mut = nuc2AA_ORF1a(mut, refAA, qryAA)
                push!(AA_mut_set, AA_mut)
                all_nuc_to_AA_dict[mut] = AA_mut
                if refAA == qryAA && !(pos in rem9b)
                    synonymous_nuc_to_AA_dict[mut] = AA_mut
                elseif refAA == qryAA && pos in rem9b && mut in N_9b_synonymous
                    synonymous_nuc_to_AA_dict[mut] = AA_mut
                else
                    nonsynonymous_nuc_to_AA_dict[mut] = AA_mut
#                    push!(nonsynonymous_nuc_muts, mut)
                end
###################################
                for nuc_mut2 in muts
                    mut2 = mixed2nuc(nuc_mut2)
                    pos2 = nuc_mut_int_comprehensive_dict[mut2]
                    if pos2 in coding_ranges
                        gene_number2 = gene_num(mut2)
                        if mut ≠ mut2 && gene_number == gene_number2 && nuc_to_AA_pos(mut) == nuc_to_AA_pos(mut2)
                            if nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 2
                                ref_triple = ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                                qry_triple = qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                            elseif nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 3
                                ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*ref_nuc_comprehensive_dict[mut2]
                                qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*qry_nuc_comprehensive_dict[mut2]
                            elseif nuc_codon_pos_dict[pos] == 2 && nuc_codon_pos_dict[pos2] == 3
                                ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]
                                qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]
                            elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 2
                                ref_triple = ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                                qry_triple = qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                            elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 3
                                ref_triple = ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                                qry_triple = qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                            elseif nuc_codon_pos_dict[pos2] == 2 && nuc_codon_pos_dict[pos] == 3
                                ref_triple = string(ref_seq[pos-2])*ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]
                                qry_triple = string(ref_seq[pos-2])*qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]
                            end
                            refAA2 = AA_triplets[ref_triple]
                            qryAA2 = AA_triplets[qry_triple]
                            AA_mut2 = nuc2AA_ORF1a(mut2, refAA2, qryAA2)
                            push!(AA_mut_set, AA_mut2)
                            delete!(AA_mut_set, AA_mut)
                            all_nuc_to_AA_dict[mut2] = AA_mut2
                            all_nuc_to_AA_dict[mut] = AA_mut2
                            if refAA2 == qryAA2 && !(pos2 in rem9b)
                                synonymous_nuc_to_AA_dict[mut2] = AA_mut2
                            else
                                nonsynonymous_nuc_to_AA_dict[mut2] = AA_mut2
                                nonsynonymous_nuc_to_AA_dict[mut] = AA_mut2
                            end
                        end
                    end
                end
            elseif !isempty(nuc_mut)
                qry_nuc = qry_nuc_comprehensive_dict[nuc_mut]
                for (start_end, place) in noncoding_range_dict
                    frst = start_end[1]
                    last = start_end[2]
                    if pos ≥ frst && pos ≤ last
                        mut_vec = [nuc_mut, place]
                        push!(noncoding_nuc_vector, nuc_mut)
                    end
                end
            end
        end
#########################################################################################################
        for nuc_mut in muts
            pos_9b = nuc_mut_int_comprehensive_dict[nuc_mut]
            if pos_9b in rem9b
                mut_9b = mixed2nuc(nuc_mut)
                pos_9b = nuc_mut_int_comprehensive_dict[mut_9b]   
                gene_number_9b = 11
                ref_triple_9b = ""
                qry_triple_9b = ""
                if nuc_codon_pos_dict_9b[pos_9b] == 1
                    ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])
                    qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])
                elseif nuc_codon_pos_dict_9b[pos_9b] == 2
                    ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                    qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                elseif nuc_codon_pos_dict_9b[pos_9b] == 3
                    ref_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                    qry_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                end
                refAA_9b = AA_triplets[ref_triple_9b]
                qryAA_9b = AA_triplets[qry_triple_9b]
                AA_mut_9b = nuc2AA_ORF9b(mut_9b, refAA_9b, qryAA_9b)
                push!(AA_mut_set, AA_mut_9b)
                all_nuc_to_AA_dict[mut_9b] = AA_mut_9b
                if refAA_9b == qryAA_9b && nuc_mut in N_9b_synonymous
                    synonymous_nuc_to_AA_dict[mut_9b] = AA_mut_9b
                end
                if refAA_9b ≠ qryAA_9b
                    nonsynonymous_nuc_to_AA_dict[mut_9b] = AA_mut_9b
#                    push!(nonsynonymous_nuc_muts, mut_9b)
                end
###################################
                for nuc_mut2_9b in muts
                    mut2_9b = mixed2nuc(nuc_mut2_9b)
                    pos2_9b = nuc_mut_int_comprehensive_dict[mut2_9b]
                    if pos2_9b in rem9b
                        gene_number2_9b = 11
                        if mut_9b ≠ mut2_9b && nuc_to_AA_pos_9b(mut_9b) == nuc_to_AA_pos_9b(mut2_9b)
                            if nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 2
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                            elseif nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*ref_nuc_comprehensive_dict[mut2_9b]
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*qry_nuc_comprehensive_dict[mut2_9b]
                            elseif nuc_codon_pos_dict_9b[pos_9b] == 2 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                                ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]
                                qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 2
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 3
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 2 && nuc_codon_pos_dict_9b[pos_9b] == 3
                                ref_triple_9b = string(ref_seq[pos_9b-2])*ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]
                                qry_triple_9b = string(ref_seq[pos_9b-2])*qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]
                            end
                            refAA2_9b = AA_triplets[ref_triple_9b]
                            qryAA2_9b = AA_triplets[qry_triple_9b]
                            AA_mut2_9b = nuc2AA_ORF9b(mut2_9b, refAA2_9b, qryAA2_9b)
                            push!(AA_mut_set, AA_mut2_9b)
                            delete!(AA_mut_set, AA_mut_9b)
                            if refAA2_9b == qryAA2_9b && nuc_mut2_9b in N_9b_synonymous 
                                synonymous_nuc_to_AA_dict[mut2_9b] = AA_mut2_9b
                                synonymous_nuc_to_AA_dict[mut_9b] = AA_mut2_9b
                            else
                                nonsynonymous_nuc_to_AA_dict[mut2_9b] = AA_mut2_9b
                                nonsynonymous_nuc_to_AA_dict[mut_9b] = AA_mut2_9b
                            end
                        end
                    end
                end
            end
        end
#####################################################################################################################################
#        for aamoot in AA_mut_set
#            if !haskey(aa_gene_comprehensive_dict, aamoot)
#                push!(missing_AA_dict_keys, aamoot)
#                mutgene = split_gene(aamoot)
#                mutAApos = split_pos(aamoot)
#                ref_pos_qry = split_ref_pos_qry(aamoot)
#                pos_n_gene = "$(mutgene):$(mutAApos)"
#                aa_gene_comprehensive_dict[aamoot] = mutgene
#                aa_pos_comprehensive_dict[aamoot] = mutAApos
#                aa_gene_and_pos_comprehensive_dict[aamoot] = pos_n_gene
#                refAA = string(ref_pos_qry[1])
#                qryAA = string(ref_pos_qry[end])
#                ref_AA_dict[aamoot] = refAA
#                qry_AA_dict[aamoot] = qryAA
#            end
#        end
        AA_sort = sort(collect(AA_mut_set), by = x -> AA_order_key(x))
        AA_sort2 = Vector{String}()
        for aa in AA_sort
            refAA = 
            aa_refqry = refAA_comprehensive_dict[aa]
            if !(string(aa_refqry[1]) == string(aa_refqry[end]))
                push!(AA_sort2, aa)
            end
        end
#        print("\n"^1)
#        ct = 0
#        println("############# AA Mutations #############")
#        for i in 1:length(AA_sort2)
#            mut = AA_sort2[i]
#            gene = aa_gene_comprehensive_dict[mut]
#            non_gene = string(split(mut, ":")[2])
#            if i == 1
#                print("        ", AA_sort2[i])
#            elseif i > 1
#                lastmut = AA_sort2[i-1]
#                last_gene = string(split(lastmut, ":")[1])
#                last_non_gene = string(split(lastmut, ":")[2])
#                if gene == last_gene
#                    print(", $(non_gene)")
#                else
#                    print("        $(mut)")
#                end
#            end
#        end
#####################################################################################################################################
        all_nuc_to_AA_dict_sort = sort(collect(all_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
        nonsynonymous_nuc_to_AA_dict_sort = sort(collect(nonsynonymous_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
        nonsynonymous_nuc_sort = sort(collect(keys(nonsynonymous_nuc_to_AA_dict)), by = x -> nuc_mut_int_comprehensive_dict[x])
        nonsynonymous_AA_sort = sort(collect(values(nonsynonymous_nuc_to_AA_dict)), by = x -> AA_gene_sortKey_2(x))
        synonymous_nuc_to_AA_dict_sort = sort(collect(synonymous_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
        synonymous_nuc_sort = sort(collect(keys(synonymous_nuc_to_AA_dict)), by = x -> nuc_mut_int_comprehensive_dict[x])
        synonymous_AA_sort = sort(collect(values(synonymous_nuc_to_AA_dict)), by = x -> AA_gene_sortKey_2(x))
        noncoding_nuc_vector_sort = sort(collect(noncoding_nuc_vector), by = x -> nuc_mut_int_comprehensive_dict[x])
        synonymous_nuc_total = length(synonymous_nuc_sort)
        for i in 1:length(synonymous_nuc_sort)
            nucpad = rpad("$(synonymous_nuc_to_AA_dict_sort[i][1])", 10)
        end
#        return synonymous_nuc_to_AA_dict, nonsynonymous_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort
        return synonymous_nuc_to_AA_dict_sort, nonsynonymous_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort
    end
    if isempty(muts)
        aa = Vector{Pair{String, String}}()
        bb = Vector{Pair{String, String}}()
        cc = Vector{Pair{String, String}}()
        return aa, bb, cc
    end 
end
###########################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
function pango_minus_1_fx(pango::String)
    if '.' in pango
        dot_ct = count(".", pango)
        dotsplits = split(pango, ".")
        minus_1 = join(dotsplits(1:dot_ct-1), ".")
        return minus_1
    else
        return pango
    end
end
############################################################################################################################################################################
### Fx: SIMPLER_syn_noncoding_nonsyn_nuc (no context)
function SIMPLER_syn_noncoding_nuc(ref_pango::String, muts::Set{String})
    B_1_1_list = ["B.1.1.53", "B.1.1.273"]
    if ref_pango in B_1_1_list
        ref_pango = "B.1.1"
    end
    if ref_pango == "XBB.1.5.82"
         ref_pango = "XBB.1.5"
    end
    if !isempty(muts)
#        all_muts_sort = sort(collect(muts), by = x -> nuc_mut_int_comprehensive_dict[x])
        ref_seq, refAA_ORF1a, refAA_ORF1b, refAA_S, refAA_ORF3a, refAA_E, refAA_M, refAA_ORF6, refAA_ORF7a, refAA_ORF7b, refAA_ORF8, refAA_N, refAA_ORF9b = get_ref_pango_nucseq_and_geneseqs(ref_pango)
###################################################################################################################################
        coding_ranges = BitSet([266:13467..., 13469:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533..., 28284:28577...])
        noncoding_ranges = BitSet([1:265..., 21556:21562..., 25385:25392..., 26221:26244..., 26473:26522..., 27192:27201..., 27388:27393..., 27888:27893..., 28260:28273..., 29534:29830...])
        coding_range_9b = BitSet([28284:28577...])
        gene_nuc_starts = Dict{Int, Int}(0=>263, 1=>13465, 2=>21560, 3=>25390, 4=>26242, 5=>26520, 6=>27199, 7=>27391, 8=>27753, 9=>27891, 10=>28271, 11=>28281)
        ref_AA_genes = Dict{Int, String}(0=>refAA_ORF1a, 1=>refAA_ORF1b, 2=>refAA_S, 3=>refAA_ORF3a, 4=>refAA_E, 5=>refAA_M, 6=>refAA_ORF6, 7=>refAA_ORF7a, 8=>refAA_ORF7b, 9=>refAA_ORF8, 10=>refAA_N, 11=>refAA_ORF9b)
        gene_strings = Dict{Int, String}(0=>"ORF1a", 1=>"ORF1b", 2=>"S", 3=>"ORF3a", 4=>"E", 5=>"M", 6=>"ORF6", 7=>"ORF7a", 8=>"ORF7b", 9=>"ORF8", 10=>"N", 11=>"ORF9b")
        nuc_gene_num = Dict{Int, Int}()
        nuc_gene_num_9b = Dict{Int, Int}()
        synonymous_nuc_to_AA_dict = Dict{String, String}()
        noncoding_range_dict = Dict{Vector{Int}, String}([1, 265]=>"5' UTR", [21556, 21562]=>"Spike TRS", [25385, 25392]=>"ORF3a TRS", [26221, 26234]=>"ORF3a-E UTR", [26235, 26244]=>"E TRS", [26473, 26522]=>"E-M UTR", [27192, 27201]=>"M-ORF6 UTR", [27388, 27393]=>"ORF7a TRS", [27888, 27893]=>"ORF8 TRS", [28260, 28273]=>"N/ORF9b TRS", [29534, 29830]=>"3' UTR", [29794, 29801]=>"Octanucleotide Motif")
################################################
        noncoding_nuc_vector = Vector{String}()
################################################ 
        gene_nuc_arr = [[266:13467...], [13469:21555...], [21563:25384...], [25393:26220...], [26245:26472...], [26523:27191...], [27202:27387...], [27394:27755...], [27760:27887...], [27894:28259...], [28274:29533...], [28284:28577...]]
        for i in 1:length(gene_nuc_arr)-1
            for nuc_pos in gene_nuc_arr[i]
                nuc_gene_num[nuc_pos] = i-1
            end
        end
        for nuc_pos in gene_nuc_arr[end]
            nuc_gene_num_9b[nuc_pos] = 11
        end
        rem0_gene = [5, 8, 9, 11]
        rem1_gene = [1, 3, 4, 6, 7]
        rem2_gene = [0, 2, 10]
        rem0 = BitSet([26523:27191..., 27760:27887..., 27894:28259..., 28284:28577...])
        rem1 = BitSet([13469:21555..., 25393:26220..., 26245:26472..., 27202:27387..., 27394:27755...])
        rem2 = BitSet([266:13467..., 21563:25384..., 28274:29533...])
        rem9b = BitSet([28284:28577...])
        rem7ab = BitSet([27756:27759...])

        gene_num(nuc_mut) = nuc_gene_num[nuc_mut_int_comprehensive_dict[nuc_mut]] ## Fx ##
        nuc_to_AA_pos(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - gene_nuc_starts[gene_num(nuc_mut)])÷3) ## Fx ##
        nuc_to_AA_pos_9b(nuc_mut) = string((nuc_mut_int_comprehensive_dict[nuc_mut] - 28281)÷3) ## Fx ##
        nuc2AA_ORF1a(nuc_mut, refAA, qryAA) = gene_strings[gene_num(nuc_mut)]*":"*refAA*nuc_to_AA_pos(nuc_mut)*qryAA ## Fx ##
        nuc2AA_ORF9b(nuc_mut, refAA, qryAA) = "ORF9b:"*refAA*nuc_to_AA_pos_9b(nuc_mut)*qryAA
        nuc_codon_pos_dict = Dict{Int, Int}()
        for nuc_pos in coding_ranges
            gene_number = nuc_gene_num[nuc_pos]
            gene_start = gene_nuc_starts[gene_number]
            codon_num = (nuc_pos-gene_start)%3 + 1
            nuc_codon_pos_dict[nuc_pos] = codon_num
        end
        nuc_codon_pos_dict_9b = Dict{Int, Int}()
        for nuc_pos in coding_range_9b
            gene_number = 11
            gene_start = gene_nuc_starts[gene_number]
            codon_num = (nuc_pos-gene_start)%3 + 1
            nuc_codon_pos_dict_9b[nuc_pos] = codon_num
        end    
        for nuc_mut in muts
            if !isempty(muts)
                mut = mixed2nuc(nuc_mut)
                if ',' in mut
                    mut1 = string(split(mut, ",")[1])
                    mut2 = string(split(mut, ",")[2])
                    push!(muts, mut1)
                    push!(muts, mut2)
                    filter!(x -> !(length(x)>6), muts)
                end
            end
        end
        coding_ranges = BitSet([266:13467..., 13468:21555..., 21563:25384..., 25393:26220..., 26245:26472..., 26523:27191..., 27202:27387..., 27394:27755..., 27760:27887..., 27894:28259..., 28274:29533...])
        N_9b_synonymous = Set(["C28379A", "C28394A", "T28406C", "C28475A", "C28535A", "A28547C"])
        AA_mut = ""
        for nuc_mut in muts
            if !isempty(muts) && !isempty(ref_seq) && nuc_mut ≠ ""
                pos = nuc_mut_int_comprehensive_dict[nuc_mut]
                if pos in coding_ranges
                    mut = mixed2nuc(nuc_mut)
                    if isempty(mut)
                        println(nuc_mut)
                        println(mut)
                        println("$(pango)")
                    end
                    try
                        nuc_mut_int_comprehensive_dict[mut]
                    catch e
                        println("Problematic mutation string: ", repr(mut))
                        println("Length: ", length(mut))
                        println("Extracted substring: ", repr(mut[2:end-1]))
                        rethrow(e)
                    end
                    gene_number = nuc_gene_num[pos]
                    ref_triple = ""
                    qry_triple = ""
                    if nuc_codon_pos_dict[pos] == 1
                        ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
                        qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*string(ref_seq[pos+2])
                    elseif nuc_codon_pos_dict[pos] == 2
                        ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                        qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                    elseif nuc_codon_pos_dict[pos] == 3
                        ref_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                        qry_triple = string(ref_seq[pos-2])*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                    end
                    refAA = AA_triplets[ref_triple]
                    qryAA = AA_triplets[qry_triple]
                    AA_mut = nuc2AA_ORF1a(mut, refAA, qryAA)
                    if refAA == qryAA && !(pos in rem9b)
                        synonymous_nuc_to_AA_dict[mut] = AA_mut
                    elseif refAA == qryAA && pos in rem9b && mut in N_9b_synonymous
                        synonymous_nuc_to_AA_dict[mut] = AA_mut
                    end
###################################
                    for nuc_mut2 in muts
                        mut2 = mixed2nuc(nuc_mut2)
                        pos2 = nuc_pos_check_fx(mut2)
                        if pos2 in coding_ranges
                            gene_number2 = gene_num(mut2)
                            if mut ≠ mut2 && gene_number == gene_number2 && nuc_to_AA_pos(mut) == nuc_to_AA_pos(mut2)
                                if nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 2
                                    ref_triple = ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                                    qry_triple = qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos+2])
                                elseif nuc_codon_pos_dict[pos] == 1 && nuc_codon_pos_dict[pos2] == 3
                                    ref_triple = ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*ref_nuc_comprehensive_dict[mut2]
                                    qry_triple = qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])*qry_nuc_comprehensive_dict[mut2]
                                elseif nuc_codon_pos_dict[pos] == 2 && nuc_codon_pos_dict[pos2] == 3
                                    ref_triple = string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]*ref_nuc_comprehensive_dict[mut2]
                                    qry_triple = string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]*qry_nuc_comprehensive_dict[mut2]
                                elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 2
                                    ref_triple = ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                                    qry_triple = qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]*string(ref_seq[pos+1])
                                elseif nuc_codon_pos_dict[pos2] == 1 && nuc_codon_pos_dict[pos] == 3
                                    ref_triple = ref_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*ref_nuc_comprehensive_dict[mut]
                                    qry_triple = qry_nuc_comprehensive_dict[mut2]*string(ref_seq[pos-1])*qry_nuc_comprehensive_dict[mut]
                                elseif nuc_codon_pos_dict[pos2] == 2 && nuc_codon_pos_dict[pos] == 3
                                    ref_triple = string(ref_seq[pos-2])*ref_nuc_comprehensive_dict[mut2]*ref_nuc_comprehensive_dict[mut]
                                    qry_triple = string(ref_seq[pos-2])*qry_nuc_comprehensive_dict[mut2]*qry_nuc_comprehensive_dict[mut]
                                end
                                refAA2 = AA_triplets[ref_triple]
                                qryAA2 = AA_triplets[qry_triple]
                                AA_mut2 = nuc2AA_ORF1a(mut2, refAA2, qryAA2)
                                if refAA2 == qryAA2 && !(pos2 in rem9b)
                                    synonymous_nuc_to_AA_dict[mut2] = AA_mut2
                                end
                            end
                        end
                    end
                else
                    qry_nuc = qry_nuc_comprehensive_dict[nuc_mut]
                    for (start_end, place) in noncoding_range_dict
                        frst = start_end[1]
                        last = start_end[2]
                        if pos ≥ frst && pos ≤ last
                            mut_vec = [nuc_mut, place]
                            push!(noncoding_nuc_vector, nuc_mut)
                        end
                    end
                end
            end
        end
#########################################################################################################
        for nuc_mut in muts
            pos_9b = nuc_mut_int_comprehensive_dict[nuc_mut]
            if pos_9b in rem9b && !isempty(ref_seq) && nuc_mut ≠ ""
                mut_9b = mixed2nuc(nuc_mut)
                pos_9b = nuc_mut_int_comprehensive_dict[mut_9b]   
                gene_number_9b = 11
                ref_triple_9b = ""
                qry_triple_9b = ""
                if nuc_codon_pos_dict_9b[pos_9b] == 1
                    ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])
                    qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*string(ref_seq[pos_9b+2])
                elseif nuc_codon_pos_dict_9b[pos_9b] == 2
                    ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                    qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                elseif nuc_codon_pos_dict_9b[pos_9b] == 3
                    ref_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                    qry_triple_9b = string(ref_seq[pos_9b-2])*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                end
                refAA_9b = AA_triplets[ref_triple_9b]
                qryAA_9b = AA_triplets[qry_triple_9b]
                AA_mut_9b = nuc2AA_ORF9b(mut_9b, refAA_9b, qryAA_9b)
                if refAA_9b == qryAA_9b && nuc_mut in N_9b_synonymous
                    synonymous_nuc_to_AA_dict[mut_9b] = AA_mut_9b
                end
###################################
                for nuc_mut2_9b in muts
                    mut2_9b = mixed2nuc(nuc_mut2_9b)
                    pos2_9b = nuc_mut_int_comprehensive_dict[mut2_9b]
                    if pos2_9b in rem9b
                        gene_number2_9b = 11
                        if mut_9b ≠ mut2_9b && nuc_to_AA_pos_9b(mut_9b) == nuc_to_AA_pos_9b(mut2_9b)
                            if nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 2
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b+2])
                            elseif nuc_codon_pos_dict_9b[pos_9b] == 1 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*ref_nuc_comprehensive_dict[mut2_9b]
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])*qry_nuc_comprehensive_dict[mut2_9b]
                            elseif nuc_codon_pos_dict_9b[pos_9b] == 2 && nuc_codon_pos_dict_9b[pos2_9b] == 3
                                ref_triple_9b = string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]*ref_nuc_comprehensive_dict[mut2_9b]
                                qry_triple_9b = string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]*qry_nuc_comprehensive_dict[mut2_9b]
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 2
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]*string(ref_seq[pos_9b+1])
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 1 && nuc_codon_pos_dict_9b[pos_9b] == 3
                                ref_triple_9b = ref_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*ref_nuc_comprehensive_dict[mut_9b]
                                qry_triple_9b = qry_nuc_comprehensive_dict[mut2_9b]*string(ref_seq[pos_9b-1])*qry_nuc_comprehensive_dict[mut_9b]
                            elseif nuc_codon_pos_dict_9b[pos2_9b] == 2 && nuc_codon_pos_dict_9b[pos_9b] == 3
                                ref_triple_9b = string(ref_seq[pos_9b-2])*ref_nuc_comprehensive_dict[mut2_9b]*ref_nuc_comprehensive_dict[mut_9b]
                                qry_triple_9b = string(ref_seq[pos_9b-2])*qry_nuc_comprehensive_dict[mut2_9b]*qry_nuc_comprehensive_dict[mut_9b]
                            end
                            refAA2_9b = AA_triplets[ref_triple_9b]
                            qryAA2_9b = AA_triplets[qry_triple_9b]
                            AA_mut2_9b = nuc2AA_ORF9b(mut2_9b, refAA2_9b, qryAA2_9b)
                            if refAA2_9b == qryAA2_9b && nuc_mut2_9b in N_9b_synonymous 
                                synonymous_nuc_to_AA_dict[mut2_9b] = AA_mut2_9b
                                synonymous_nuc_to_AA_dict[mut_9b] = AA_mut2_9b
                            end
                        end
                    end
                end
            end
        end
#####################################################################################################################################
        synonymous_nuc_to_AA_dict_sort = sort(collect(synonymous_nuc_to_AA_dict), by = x -> nuc_pos_fx(x[1]))
        synonymous_nuc_sort = sort(collect(keys(synonymous_nuc_to_AA_dict)), by = x -> nuc_mut_int_comprehensive_dict[x])
        synonymous_AA_sort = sort(collect(values(synonymous_nuc_to_AA_dict)), by = x -> AA_gene_sortKey_2(x))
        noncoding_nuc_vector_sort = sort(collect(noncoding_nuc_vector), by = x -> nuc_mut_int_comprehensive_dict[x])
        synonymous_nuc_total = length(synonymous_nuc_sort)
#        for i in 1:length(synonymous_nuc_sort)
#            nucpad = rpad("$(synonymous_nuc_to_AA_dict_sort[i][1])", 10)
#        end
        return synonymous_nuc_sort, noncoding_nuc_vector_sort
    else
        aa = Vector{Pair{String, String}}()
        bb = Vector{Pair{String, String}}()
        cc = Vector{Pair{String, String}}()
        return aa, bb, cc
    end 
end
#################################################################################
function add_leading_zero(int_str::String)
    int_str2 = int_str
    if length(int_str) == 1 && int_str ≠ "0"
        int_str2 = "0"*int_str
    end
    return int_str2
end     
##############################################################################################################################
###################################################################################################################
index_to_date_str = Dict{Int, String}()
date_str_to_index = Dict{String, Int}()
function convert_date_to_date_index(date_str::String)
    date_arr = string.(collect(date_str))
    date_tuple = nothing
### This counts the number of times "-" appears in the date_arr ---> sum(date_arr .== "-")
    if sum(date_arr .== "-") == 0
        year = parse(Int, date_str)
        date_tuple = (year, 0, 0)
    end
    if sum(date_arr .== "-") > 0
        year = parse(Int, split(date_str, "-")[1])
        month = parse(Int, split(date_str, "-")[2])
        if sum(date_arr .== "-") == 1
            date_tuple = (year, month, 0)
        else
            day = parse(Int, split(date_str, "-")[3])
            date_tuple = (year, month, day)
        end
    end
    date_index = tuple_to_index[date_tuple]
    return date_index
end
println("Done running all functions in cell.")
time_run = round(digits=1, time() - start); println("$(time_run) seconds"); print("\n"^1)
for tuple in keys(tuple_to_index)
    one = add_leading_zero(string(tuple[1]))
    two = add_leading_zero(string(tuple[2]))
    three = add_leading_zero(string(tuple[3]))
    date_string = one*"-"*two*"-"*three
    date_str_to_index[date_string] = convert_date_to_date_index(date_string)
end
for (index, tuple) in index_to_tuple
    one = add_leading_zero(string(tuple[1]))
    two = add_leading_zero(string(tuple[2]))
    three = add_leading_zero(string(tuple[3]))
    date_string = one*"-"*two*"-"*three
    index_to_date_str[index] = date_string
end
println("Done making tuple_to_index and index_to_tuple dicts.")
time_run = round(digits=1, time() - start); println("$(time_run) seconds"); print("\n"^1)
##################################################################
date_to_tuple = Dict{String, Tuple{Int, Int, Int}}()
tuple_to_date = Dict{Tuple{Int, Int, Int}, String}()
function convert_date_to_date_tuple(date_str::String)
    date_arr = string.(collect(date_str))
    date_tuple = nothing
### This counts the number of times "-" appears in the date_arr ---> sum(date_arr .== "-")
    if sum(date_arr .== "-") == 0
        year = parse(Int, date_str)
        date_tuple = (year, 0, 0)
    end
    if sum(date_arr .== "-") > 0
        year = parse(Int, split(date_str, "-")[1])
        month = parse(Int, split(date_str, "-")[2])
        if sum(date_arr .== "-") == 1
            date_tuple = (year, month, 0)
        else
            day = parse(Int, split(date_str, "-")[3])
            date_tuple = (year, month, day)
        end
    end
    return date_tuple
end
for date in keys(date_str_to_index)
    date_to_tuple[date] = convert_date_to_date_tuple(date)
end
for (date, date_tuple) in date_to_tuple
    tuple_to_date[date_tuple] = date
end
println("Done making date_to_tuple and tuple_to_date dicts.")
time_run = round(digits=1, time() - start); println("$(time_run) seconds"); print("\n"^1)
##################################################################################################################
function find_X_pct_date(clade_pango::String, pct::Float64, cp_date_cumul_dict::Dict{String, Dict{Int, Int}}, cp_total_dict::Dict{String, Int})
    cp_total = cp_total_dict[clade_pango]
    pct_date_index = 0
    pct_date_tuple = nothing
    for date_index in 1:2500
        cumulative_ct = cp_date_cumul_dict[clade_pango][date_index]
        if 100*cumulative_ct/cp_total ≥ pct
            pct_date_index = date_index
            pct_date_tuple = index_to_tuple[date_index]
            break
        end
    end
    return pct_date_index, pct_date_tuple
end
##################################################################################################################
### Truly necessary stuff from old megacell | 2026_02_08
############################################################################################################################################################################
###########################################################################################################################################################################
###########################################################################################################################################################################
all_chr_seqs_pangos = Dict{String, Int}()
all_chr_seqs_inherited = Dict{String, Int}()
for seq in all_unique_chr_seqs
    pango = seq_pango[seq]
    all_chr_seqs_pangos[pango] = get(all_chr_seqs_pangos, pango, 0) + 1
    if !haskey(pango_AAsub_WT, pango)
        if haskey(pango_predecessor_meta_dict, pango)
            if haskey(pango_predecessor_meta_dict[pango], 2)
                pango = pango_predecessor_meta_dict[pango][2]
                if !haskey(pango_AAsub_WT, pango)
                    if haskey(pango_predecessor_meta_dict, pango)
                        if haskey(pango_predecessor_meta_dict[pango], 3)
                            pango = pango_predecessor_meta_dict[pango][3]
                        end
                    end
                end
            end
        end
    end
    if pango == "B.1.1.529"
        pango_AAsub_WT[pango] = union(pango_AAsub_WT["BA.1"], pango_AAsub_WT["BA.2"])
        for mut in pango_AAsub_WT[pango]
            all_chr_seqs_inherited[mut] = get(all_chr_seqs_inherited, mut, 0) + 1
        end
    elseif haskey(pango_AAsub_WT, pango)
        for mut in pango_AAsub_WT[pango]
            all_chr_seqs_inherited[mut] = get(all_chr_seqs_inherited, mut, 0) + 1
        end
    elseif haskey(pango_predecessor_meta_dict, pango)
        if haskey(pango_predecessor_meta_dict[pango], 1)
            pango_pre1 = pango_predecessor_meta_dict[pango][1]
            for mut in pango_AAsub_WT[pango_pre1]
                all_chr_seqs_inherited[mut] = get(all_chr_seqs_inherited, mut, 0) + 1
            end
        end
    end
end
println("Done Filling all_chr_seqs_pangos, all_chr_seqs_inherited, pango_AAsub_WT (leftovers)!")
time_run = round(digits=1, time() - start); println("$(time_run) seconds"); print("\n"^1)
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
seq_syn_nucs = Dict{String, Vector{String}}()
seq_noncoding_nucs = Dict{String, Vector{String}}()
for (seq, nuc_set) in seq_nuc_muts
    pango = seq_pango[seq]
#    if !haskey(nuc_genome_pango_dict, pango)
#        println("No nuc_genome_pango_dict key, $(pango)")
#    end
    synonymous_nucmuts, noncoding_nucmuts = SIMPLER_syn_noncoding_nuc(pango, nuc_set)
    seq_syn_nucs[seq] = synonymous_nucmuts
    seq_noncoding_nucs[seq] = noncoding_nucmuts
end
println("Done Filling seq_syn_nucs, seq_noncoding_nucs!")
println()
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
############################################################################################################################################################################
runtime = time() - start
runtime_rd = round(digits=2, runtime)
runtime1, runtime2 = seconds_to_hrs_min_sec(runtime)
println("Runtime v0 = $(runtime) seconds")
println("Runtime v2 = $(runtime2)")
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
println("Finished!"); println()

2026_02_20__1011PM
Done running all functions in cell.
0.3 seconds

Done making tuple_to_index and index_to_tuple dicts.
0.6 seconds

Done making date_to_tuple and tuple_to_date dicts.
0.8 seconds

Done Filling all_chr_seqs_pangos, all_chr_seqs_inherited, pango_AAsub_WT (leftovers)!
0.9 seconds

Done Filling seq_syn_nucs, seq_noncoding_nucs!

2026_02_20__1012PM
Runtime v0 = 93.24316120147705 seconds
Runtime v2 = 0 hr, 1 min, 33.24 sec
Finished!



In [146]:
### This is not always needed ##### RUN THIS CELL EVERY TIME YOU LOAD DICTS!!!! ######################
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
start_time_post_load_cell = time()
######################################################################################################################################
gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)

### domain_order **WITH** RBD (added 2025-08-31)
domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP6_AmphHlx"=>20, "NSP6_MAE"=>21, "NSP6_cyto_CTD"=>22, "NSP12_NiRAN"=>23, "NSP12_intrfce"=>24, "NSP12_fingers"=>25, "NSP12_palm"=>26, "NSP12_palmLnk"=>27, "NSP12_thumb"=>28, "NSP13_ZBD"=>29, "NSP13_stalk"=>30, "NSP13_1B"=>31, "NSP13_RecA1"=>32, "NSP13_RecA2"=>33, "NSP14_nsp10"=>34, "NSP14_EXON"=>35, "NSP14_hinge1"=>36, "NSP14_hinge2"=>37, "NSP14_N7MTase"=>38, "NSP15_NTD"=>39, "NSP15_MD"=>40, "NSP15_endoU"=>41, "S_S1"=>42, "S_S2"=>43, "S_NTD"=>44, "S_N2R"=>45, "S_RBD"=>46, "S_RBM"=>47, "S_SD1"=>48, "S_SD2"=>49, "S_630_loop"=>50, "S_FCS_region"=>51, "S_Beta1"=>52, "S_3H"=>53, "S_IL770"=>54, "S_FPPR"=>55, "S_FP"=>56, "S_HR1"=>57, "S_CH"=>58, "S_CD"=>59, "S_Beta2"=>60, "S_2turnHelix"=>61, "S_HR2"=>62, "S_TM"=>63, "S_CT"=>64, "ORF3a_SignalP"=>65, "ORF3a_NTD"=>66, "ORF3a_TM1"=>67, "ORF3a_TM12Lnk"=>68, "ORF3a_TM2"=>69, "ORF3a_TM3"=>70, "ORF3a_cytosl1"=>71, "ORF3a_Loop"=>72, "ORF3a_3DB"=>73, "ORF3a_CTD"=>74, "E_TM"=>75, "E_cytosol"=>76, "E_CTD"=>77, "N_N1"=>78, "N_N2"=>79, "N_N3"=>80, "N_N4"=>81, "N_N5"=>82, "N_SR"=>83, "N_L_helix"=>84, "N_CBP"=>85, "N_9b_overlap"=>86)    

### domain_order **WITHOUT** RBD (added 2025-08-31)
domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP6_AmphHlx"=>20, "NSP6_MAE"=>21, "NSP6_cyto_CTD"=>22, "NSP12_NiRAN"=>23, "NSP12_intrfce"=>24, "NSP12_fingers"=>25, "NSP12_palm"=>26, "NSP12_palmLnk"=>27, "NSP12_thumb"=>28, "NSP13_ZBD"=>29, "NSP13_stalk"=>30, "NSP13_1B"=>31, "NSP13_RecA1"=>32, "NSP13_RecA2"=>33, "NSP14_nsp10"=>34, "NSP14_EXON"=>35, "NSP14_hinge1"=>36, "NSP14_hinge2"=>37, "NSP14_N7MTase"=>38, "NSP15_NTD"=>39, "NSP15_MD"=>40, "NSP15_endoU"=>41, "S_S1"=>42, "S_S2"=>43, "S_NTD"=>44, "S_N2R"=>45, "S_RBD"=>46, "S_SD1"=>47, "S_SD2"=>48, "S_630_loop"=>49, "S_FCS_region"=>50, "S_Beta1"=>51, "S_3H"=>52, "S_IL770"=>53, "S_FPPR"=>54, "S_FP"=>55, "S_HR1"=>56, "S_CH"=>57, "S_CD"=>58, "S_Beta2"=>59, "S_2turnHelix"=>60, "S_HR2"=>61, "S_TM"=>62, "S_CT"=>63, "ORF3a_SignalP"=>64, "ORF3a_NTD"=>65, "ORF3a_TM1"=>66, "ORF3a_TM12Lnk"=>67, "ORF3a_TM2"=>68, "ORF3a_TM3"=>69, "ORF3a_cytosl1"=>70, "ORF3a_Loop"=>71, "ORF3a_3DB"=>72, "ORF3a_CTD"=>73, "E_TM"=>74, "E_cytosol"=>75, "E_CTD"=>76, "N_N1"=>77, "N_N2"=>78, "N_N3"=>79, "N_N4"=>80, "N_N5"=>81, "N_SR"=>82, "N_L_helix"=>83, "N_CBP"=>84, "N_9b_overlap"=>85)    

######################################################################################################################################
gene_array = ["ORF1a", "ORF1b", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF9b"]
#                                        Gene      AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
sub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()
#                                           Gene      AAsite sub_type_ct
sub_types_at_every_site_combined_ct = Dict{String, Dict{Int, Int}}()
# gene_array = ["ORF1a", "ORF1b", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF9b"]
for gene in gene_array
    sub_types_at_every_site_combined[gene] = Dict{Int, Set{String}}()
    sub_types_at_every_site_combined_ct[gene] = Dict{Int, Int}()
end
gene_AA_lengths = Dict{String, Int}()
for (gene, AAseq) in gene_AA_dict
    gene_AA_lengths[gene] = length(AAseq)
end
for gene in gene_array
    gene_len = gene_AA_lengths[gene]
    for i in 1:gene_len
        sub_types_at_every_site_combined[gene][i] = Set{String}()
        sub_types_at_every_site_combined_ct[gene][i] = 0
    end
end
for sub in keys(AA_muts_ct_no_dels_all)
    gene = aa_gene_comprehensive_dict[sub]
    pos = parse(Int, split(sub, ":")[2][2:end-1])
    mut_type = "$(refAA_comprehensive_dict[sub])$(qryAA_comprehensive_dict[sub])"
    push!(sub_types_at_every_site_combined[gene][pos], mut_type)
end
for sub in keys(AA_muts_ct_no_dels)
    gene = aa_gene_comprehensive_dict[sub]
    pos = parse(Int, split(sub, ":")[2][2:end-1])
    mut_type = "$(refAA_comprehensive_dict[sub])$(qryAA_comprehensive_dict[sub])"
    push!(sub_types_at_every_site_combined[gene][pos], mut_type)
end
for gene in gene_array
    for (AApos, mut_type_set) in sub_types_at_every_site_combined[gene]
        sub_type_ct = length(mut_type_set)
        sub_types_at_every_site_combined_ct[gene][AApos] = sub_type_ct
    end
end
######################################################################################################################################
#                                             NSP      AAsite  all_sub_types_at_AAsite (e.g., AV, AT, TA, etc)
NSP_sub_types_at_every_site_combined = Dict{String, Dict{Int, Set{String}}}()
#                                               NSP      AAsite sub_type_ct
NSP_sub_types_at_every_site_combined_ct = Dict{String, Dict{Int, Int}}()
NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
for NSP in NSP_array
    NSP_sub_types_at_every_site_combined[NSP] = Dict{Int, Set{String}}()
    NSP_sub_types_at_every_site_combined_ct[NSP] = Dict{Int, Int}()
end
for NSP in NSP_array
    NSP_len = NSP_AA_size[NSP]
    for i in 1:NSP_len
        NSP_sub_types_at_every_site_combined[NSP][i] = Set{String}()
        NSP_sub_types_at_every_site_combined_ct[NSP][i] = 0
    end
end
for sub in keys(AA_muts_ct_no_dels_all)
    gene = aa_gene_comprehensive_dict[sub]
    if gene == "ORF1a" || gene == "ORF1b"
        AA_site = aa_pos_comprehensive_dict[sub]
        if AA_site < 4402
            NSP_sub = ORF1abMut_to_NSP(sub)
            NSP = NSP_muts_gene_dict[NSP_sub]
            NSP_pos = NSP_muts_pos_dict[NSP_sub]
            sub_type = "$(refAA_comprehensive_dict[sub])$(qryAA_comprehensive_dict[sub])"
            push!(NSP_sub_types_at_every_site_combined[NSP][NSP_pos], sub_type)
        end
    end
end
for sub in keys(AA_muts_ct_no_dels)
    gene = aa_gene_comprehensive_dict[sub]
    if gene == "ORF1a" || gene == "ORF1b"
        AA_site = aa_pos_comprehensive_dict[sub]
        if AA_site < 4402
            NSP_sub = ORF1abMut_to_NSP(sub)
            NSP = NSP_muts_gene_dict[NSP_sub]
            NSP_pos = NSP_muts_pos_dict[NSP_sub]
            sub_type = "$(refAA_comprehensive_dict[sub])$(qryAA_comprehensive_dict[sub])"
            push!(NSP_sub_types_at_every_site_combined[NSP][NSP_pos], sub_type)
        end
    end
end
for NSP in NSP_array
    for (NSP_pos, mut_type_set) in NSP_sub_types_at_every_site_combined[NSP]
        sub_type_ct = length(mut_type_set)
        NSP_sub_types_at_every_site_combined_ct[NSP][NSP_pos] = sub_type_ct
    end
end
#####################################################################################################################################
counted_seq_total_private_AA_subs = Dict{String, Int}()
total_private_AA_subs_all_counted_chronics::Int = 0
total_counted_sequences::Int = 0
for name in all_unique_chr_seqs_set
    total_counted_sequences += 1
    total_AA = length(seq_AA_muts_no_dels[name])
    total_private_AA_subs_all_counted_chronics += total_AA
    counted_seq_total_private_AA_subs[name] = total_AA
end
global chronic_avg_AA_subs_per_seq = total_private_AA_subs_all_counted_chronics/total_counted_sequences
chronic_avg_AA_subs_per_seq_rd = round(digits=2, chronic_avg_AA_subs_per_seq)
avg_private_AA_per_circ_seq_rd = round(digits=3, avg_private_AA_per_circ_seq)
println()
println("Avg AA Subs per Chronic Sequence = ", chronic_avg_AA_subs_per_seq_rd)
println("Avg AA Subs per Circulating Sequence = ", avg_private_AA_per_circ_seq_rd)
date = Dates.format(today(), "yyyy_mm_dd")
fx_name = "chr_to_circ_mut_density_ratios"
chr_to_circ_gene_density_ratio = Dict{String, Float64}()
chr_to_circ_domain_density_ratio = Dict{String, Float64}()
# The adjusted dict below adjust for the greater number of AA muts per chronic seq compared to circ seq
adj_factor = avg_private_AA_per_circ_seq/chronic_avg_AA_subs_per_seq
adj_factor_rd = round(digits=4, adj_factor)

simple_chr_to_circ_adj_factor = total_AA_subs_all/total_chr_AA_subs
simple_chr_to_circ_adj_factor_rd = round(digits=2, simple_chr_to_circ_adj_factor)

println("simple_chr_to_circ_adj_factor = $(simple_chr_to_circ_adj_factor_rd)")
println("Adjustment Factor for greater number of AA muts per chronic seq compared to circ seq = $(adj_factor_rd)")
#####################################################################################################################################
tot_singlets = length(non_rep_seqs)
tot_groups = length(keys(rep_seq_grps_maxmut_seqs) )
tot_chronics = tot_singlets + tot_groups
tot_all = qualifying_seq_ct_all
circ_to_chronic_seq_total_ratio = qualifying_seq_ct_all/tot_chronics
combined_adjustment_factor = circ_to_chronic_seq_total_ratio*adj_factor
combined_adjustment_factor_check = (avg_private_AA_per_circ_seq*qualifying_seq_ct_all)/(chronic_avg_AA_subs_per_seq*tot_chronics)
combined_adjustment_factor_rd = round(digits=3, combined_adjustment_factor)
combined_adjustment_factor_check_rd = round(digits=3, combined_adjustment_factor_check)
println()
print("Total Singlets = $(tot_singlets) | ")
print("Total Groups = $(tot_groups) | ")
println("Total Independent Chronics = $(tot_chronics)")
println("Total Qualifying Circulating Sequences = $(qualifying_seq_ct_all)")
circ_to_chronic_seq_total_ratio_rd = round(digits=2, circ_to_chronic_seq_total_ratio)
println("(Total Qualifying Circulating Sequences)/(Total Independent Chronics) = $(circ_to_chronic_seq_total_ratio_rd)")
print("Combined Adjustment Factor = $(combined_adjustment_factor_rd)  |  ")
println("Combined Adjustment Factor Check = $(combined_adjustment_factor_check_rd)")
#####################################################################################################################################
############ Note: the 1/1000 factor below is to make up for the 1000 factor that's used in the current chronic function ############
#####################################################################################################################################
for (gene, chr_density) in gene_mut_density
    all_density = gene_mut_density_all[gene]
    chr_circ_ratio = (chr_density/all_density)*simple_chr_to_circ_adj_factor
    ratio_rd = round(digits=2, chr_circ_ratio)
    chr_to_circ_gene_density_ratio[gene] = ratio_rd
end
for (domain, chr_density) in domain_mut_density
    all_density = domain_mut_density_all[domain]
    chr_circ_ratio = chr_density/all_density*simple_chr_to_circ_adj_factor
    ratio_rd = round(digits=2, chr_circ_ratio)
    chr_to_circ_domain_density_ratio[domain] = ratio_rd
end
######################################################################################################################################
chr_to_circ_gene_density_ratio_sort_by_gene = sort(collect(chr_to_circ_gene_density_ratio), by = x -> gene_protein_order[x[1]])
chr_to_circ_gene_density_ratio_sort_by_ratio = sort(collect(chr_to_circ_gene_density_ratio), by = x -> x[2], rev=true)
chr_to_circ_domain_density_ratio_sort_by_domain = sort(collect(chr_to_circ_domain_density_ratio), by = x -> domain_order[x[1]])
chr_to_circ_domain_density_ratio_sort_by_ratio = sort(collect(chr_to_circ_domain_density_ratio), by = x -> x[2], rev=true)
######################################################################################################################################
save("$(date)_chr_to_circ_gene_density_ratio.jld2", "chr_to_circ_gene_density_ratio", chr_to_circ_gene_density_ratio)
save("$(date)_chr_to_circ_domain_density_ratio.jld2", "chr_to_circ_domain_density_ratio", chr_to_circ_domain_density_ratio)
@save("$(date)_chr_to_circ_gene_density_ratio_sort_by_gene.jld2", chr_to_circ_gene_density_ratio_sort_by_gene)
@save("$(date)_chr_to_circ_gene_density_ratio_sort_by_ratio.jld2", chr_to_circ_gene_density_ratio_sort_by_ratio)
@save("$(date)_chr_to_circ_domain_density_ratio_sort_by_domain.jld2", chr_to_circ_domain_density_ratio_sort_by_domain)
@save("$(date)_chr_to_circ_domain_density_ratio_sort_by_ratio.jld2", chr_to_circ_domain_density_ratio_sort_by_ratio)
# save("$(date)_.jld2", "")
# @save("$(date)_.jld2")
######################################################################################################################################

##### domain_residues_NSP/ORF *WITH* RBM (added 2025-08-31)
domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM234HLX"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP6_AmphHlx"=>"NSP6:219-236", "NSP6_MAE"=>"NSP6:237-276", "NSP6_cyto_CTD"=>"NSP6:277-290", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "NSP13_ZBD"=>"NSP13:1-101", "NSP13_stalk"=>"NSP13:102-150", "NSP13_1B"=>"NSP13:151-226", "NSP13_RecA1"=>"NSP13:261-442", "NSP13_RecA2"=>"NSP13:443-601", "NSP14_nsp10"=>"NSP14:1-58", "NSP14_EXON"=>"NSP14:67-282 ", "NSP14_hinge1"=>"NSP14:286-300", "NSP14_hinge2"=>"NSP14:411-434", "NSP14_N7MTase"=>"NSP14:302-527", "NSP15_NTD"=>"NSP15:1-64", "NSP15_MD"=>"NSP15:65-182", "NSP15_endoU"=>"NSP15:207-347", "S_S1"=>"S:2-686", "S_S2"=>"S:687-1273", "S_NTD"=>"S:2-305", "S_N2R"=>"S:306-334", "S_RBD"=>"S:335-528", "S_RBM"=>"S:438-506", "S_SD1"=>"S:529-591", "S_SD2"=>"S:592-674", "S_630_loop"=>"S:619-642", "S_FCS_region"=>"S:675-691", "S_Beta1"=>"S:718-729", "S_3H"=>"S:747-769", "S_IL770"=>"S:770-831", "S_FPPR"=>"S:832-866", "S_FP"=>"S:867-909", "S_HR1"=>"S:910-985", "S_CH"=>"S:986-1035", "S_CD"=>"S:1036-1068", "S_Beta2"=>"S:1127-1135", "S_2turnHelix"=>"S:1148-1155", "S_HR2"=>"S:1164-1211", "S_TM"=>"S:1212-1234", "S_CT"=>"S:1235-1273", "ORF3a_SignalP"=>"ORF3a:1-14", "ORF3a_NTD"=>"ORF3a:15-33", "ORF3a_TM1"=>"ORF3a:34-56", "ORF3a_TM12Lnk"=>"ORF3a:57-76", "ORF3a_TM2"=>"ORF3a:77-99", "ORF3a_TM3"=>"ORF3a:103-125", "ORF3a_cytosl1"=>"ORF3a:126-169", "ORF3a_Loop"=>"ORF3a:170-183", "ORF3a_3DB"=>"ORF3a:170-222", "ORF3a_CTD"=>"ORF3a:223-275", "E_TM"=>"E:8-38", "E_cytosol"=>"E:54-65", "E_CTD"=>"E:69-75", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
domain_residues_ORF = Dict{String, String}("ORF3a_SignalP"=>"ORF3a:1-14", "ORF3a_NTD"=>"ORF3a:15-33", "ORF3a_TM1"=>"ORF3a:34-56", "ORF3a_TM12Lnk"=>"ORF3a:57-76", "ORF3a_TM2"=>"ORF3a:77-99", "ORF3a_TM3"=>"ORF3a:103-125", "ORF3a_cytosl1"=>"ORF3a:126-169", "ORF3a_Loop"=>"ORF3a:170-183", "ORF3a_3DB"=>"ORF3a:170-222", "ORF3a_CTD"=>"ORF3a:223-275", "NSP3_Ubl1"=>"ORF1a:819-929", "NSP3_HVR"=>"ORF1a:930-1024", "NSP3_Mac1"=>"ORF1a:1025-1192", "NSP3_Mac2"=>"ORF1a:1193-1369", "NSP3_Mac3"=>"ORF1a:1370-1491", "NSP3_DPUP"=>"ORF1a:1492-1561", "NSP3_Ubl2"=>"ORF1a:1562-1621", "NSP3_PLpro"=>"ORF1a:1622-1870", "NSP3_NAB"=>"ORF1a:1871-2021", "NSP3_BSM"=>"ORF1a:2022-2152", "NSP3_TM1"=>"ORF1a:2153-2258", "NSP3_Ecto3"=>"ORF1a:2259-2317", "NSP3_TM234HLX"=>"ORF1a:2318-2404", "NSP3_Y1"=>"ORF1a:2405-2582", "NSP3_CoVY"=>"ORF1a:2583-2763", "NSP4_TM1"=>"ORF1a:2764-2794", "NSP4_Ecto4"=>"ORF1a:2795-3034", "NSP4_TM2_TM6"=>"ORF1a:3035-3173", "NSP4_CTD"=>"ORF1a:3174-3263", "NSP6_AmphHlx"=>"ORF1a:3788-3805", "NSP6_MAE"=>"ORF1a:3806-3845", "NSP6_cyto_CTD"=>"ORF1a:3846-3859", "NSP12_NiRAN"=>"1a:4393-1b:241", "NSP12_intrfce"=>"ORF1b:242-389", "NSP12_fingers"=>"ORF1b:390-572", "NSP12_palm"=>"ORF1b:573-618", "NSP12_palmLnk"=>"ORF1b:619-678", "NSP12_thumb"=>"ORF1b:804-923", "NSP13_ZBD"=>"ORF1b:924-1024", "NSP13_stalk"=>"ORF1b:1025-1073", "NSP13_1B"=>"ORF1b:1074-1149", "NSP13_RecA1"=>"ORF1b:1184-1365", "NSP13_RecA2"=>"ORF1b:1366-1524", "NSP14_nsp10"=>"ORF1b:1525-1582", "NSP14_EXON"=>"ORF1b:1591-1806", "NSP14_hinge1"=>"ORF1b:1810-1824", "NSP14_hinge2"=>"ORF1b:1935-1958", "NSP14_N7MTase"=>"ORF1b:1826-2051", "NSP15_NTD"=>"ORF1b:2052-2115", "NSP15_MD"=>"ORF1b:2116-2233", "NSP15_endoU"=>"ORF1b:2258-2398", "S_S1"=>"S:2-686", "S_NTD"=>"S:2-305", "S_N2R"=>"S:306-334", "S_RBD"=>"S:335-528", "S_RBM"=>"S:438-506", "S_SD1"=>"S:529-591", "S_SD2"=>"S:592-674", "S_630_loop"=>"S:619-642", "S_FCS_region"=>"S:675-691", "S_S2"=>"S:687-1273", "S_Beta1"=>"S:718-729", "S_3H"=>"S:747-769", "S_IL770"=>"S:770-831", "S_FPPR"=>"S:832-866", "S_FP"=>"S:867-909", "S_HR1"=>"S:910-985", "S_CH"=>"S:986-1035", "S_CD"=>"S:1036-1068", "S_Beta2"=>"S:1127-1135", "S_2turnHelix"=>"S:1148-1155", "S_HR2"=>"S:1164-1211", "S_TM"=>"S:1212-1234", "S_CT"=>"S:1235-1273", "E_TM"=>"E:8-38", "E_cytosol"=>"E:54-65", "E_CTD"=>"E:69-75", "N_9b_overlap"=>"N:4-101", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_SR"=>"N:176-206", "N_N3"=>"N:176-246", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419" )

##### domain_residues_NSP/ORF *WITHOUT* RBM (added 2025-08-31)
domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM234HLX"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP6_AmphHlx"=>"NSP6:219-236", "NSP6_MAE"=>"NSP6:237-276", "NSP6_cyto_CTD"=>"NSP6:277-290", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "NSP13_ZBD"=>"NSP13:1-101", "NSP13_stalk"=>"NSP13:102-150", "NSP13_1B"=>"NSP13:151-226", "NSP13_RecA1"=>"NSP13:261-442", "NSP13_RecA2"=>"NSP13:443-601", "NSP14_nsp10"=>"NSP14:1-58", "NSP14_EXON"=>"NSP14:67-282 ", "NSP14_hinge1"=>"NSP14:286-300", "NSP14_hinge2"=>"NSP14:411-434", "NSP14_N7MTase"=>"NSP14:302-527", "NSP15_NTD"=>"NSP15:1-64", "NSP15_MD"=>"NSP15:65-182", "NSP15_endoU"=>"NSP15:207-347", "S_S1"=>"S:2-686", "S_S2"=>"S:687-1273", "S_NTD"=>"S:2-305", "S_N2R"=>"S:306-334", "S_RBD"=>"S:335-528", "S_SD1"=>"S:529-591", "S_SD2"=>"S:592-674", "S_630_loop"=>"S:619-642", "S_FCS_region"=>"S:675-691", "S_Beta1"=>"S:718-729", "S_3H"=>"S:747-769", "S_IL770"=>"S:770-831", "S_FPPR"=>"S:832-866", "S_FP"=>"S:867-909", "S_HR1"=>"S:910-985", "S_CH"=>"S:986-1035", "S_CD"=>"S:1036-1068", "S_Beta2"=>"S:1127-1135", "S_2turnHelix"=>"S:1148-1155", "S_HR2"=>"S:1164-1211", "S_TM"=>"S:1212-1234", "S_CT"=>"S:1235-1273", "ORF3a_SignalP"=>"ORF3a:1-14", "ORF3a_NTD"=>"ORF3a:15-33", "ORF3a_TM1"=>"ORF3a:34-56", "ORF3a_TM12Lnk"=>"ORF3a:57-76", "ORF3a_TM2"=>"ORF3a:77-99", "ORF3a_TM3"=>"ORF3a:103-125", "ORF3a_cytosl1"=>"ORF3a:126-169", "ORF3a_Loop"=>"ORF3a:170-183", "ORF3a_3DB"=>"ORF3a:170-222", "ORF3a_CTD"=>"ORF3a:223-275", "E_TM"=>"E:8-38", "E_cytosol"=>"E:54-65", "E_CTD"=>"E:69-75", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
domain_residues_ORF = Dict{String, String}("ORF3a_SignalP"=>"ORF3a:1-14", "ORF3a_NTD"=>"ORF3a:15-33", "ORF3a_TM1"=>"ORF3a:34-56", "ORF3a_TM12Lnk"=>"ORF3a:57-76", "ORF3a_TM2"=>"ORF3a:77-99", "ORF3a_TM3"=>"ORF3a:103-125", "ORF3a_cytosl1"=>"ORF3a:126-169", "ORF3a_Loop"=>"ORF3a:170-183", "ORF3a_3DB"=>"ORF3a:170-222", "ORF3a_CTD"=>"ORF3a:223-275", "NSP3_Ubl1"=>"ORF1a:819-929", "NSP3_HVR"=>"ORF1a:930-1024", "NSP3_Mac1"=>"ORF1a:1025-1192", "NSP3_Mac2"=>"ORF1a:1193-1369", "NSP3_Mac3"=>"ORF1a:1370-1491", "NSP3_DPUP"=>"ORF1a:1492-1561", "NSP3_Ubl2"=>"ORF1a:1562-1621", "NSP3_PLpro"=>"ORF1a:1622-1870", "NSP3_NAB"=>"ORF1a:1871-2021", "NSP3_BSM"=>"ORF1a:2022-2152", "NSP3_TM1"=>"ORF1a:2153-2258", "NSP3_Ecto3"=>"ORF1a:2259-2317", "NSP3_TM234HLX"=>"ORF1a:2318-2404", "NSP3_Y1"=>"ORF1a:2405-2582", "NSP3_CoVY"=>"ORF1a:2583-2763", "NSP4_TM1"=>"ORF1a:2764-2794", "NSP4_Ecto4"=>"ORF1a:2795-3034", "NSP4_TM2_TM6"=>"ORF1a:3035-3173", "NSP4_CTD"=>"ORF1a:3174-3263", "NSP6_AmphHlx"=>"ORF1a:3788-3805", "NSP6_MAE"=>"ORF1a:3806-3845", "NSP6_cyto_CTD"=>"ORF1a:3846-3859", "NSP12_NiRAN"=>"1a:4393-1b:241", "NSP12_intrfce"=>"ORF1b:242-389", "NSP12_fingers"=>"ORF1b:390-572", "NSP12_palm"=>"ORF1b:573-618", "NSP12_palmLnk"=>"ORF1b:619-678", "NSP12_thumb"=>"ORF1b:804-923", "NSP13_ZBD"=>"ORF1b:924-1024", "NSP13_stalk"=>"ORF1b:1025-1073", "NSP13_1B"=>"ORF1b:1074-1149", "NSP13_RecA1"=>"ORF1b:1184-1365", "NSP13_RecA2"=>"ORF1b:1366-1524", "NSP14_nsp10"=>"ORF1b:1525-1582", "NSP14_EXON"=>"ORF1b:1591-1806", "NSP14_hinge1"=>"ORF1b:1810-1824", "NSP14_hinge2"=>"ORF1b:1935-1958", "NSP14_N7MTase"=>"ORF1b:1826-2051", "NSP15_NTD"=>"ORF1b:2052-2115", "NSP15_MD"=>"ORF1b:2116-2233", "NSP15_endoU"=>"ORF1b:2258-2398", "S_S1"=>"S:2-686", "S_NTD"=>"S:2-305", "S_N2R"=>"S:306-334", "S_RBD"=>"S:335-528", "S_SD1"=>"S:529-591", "S_SD2"=>"S:592-674", "S_630_loop"=>"S:619-642", "S_FCS_region"=>"S:675-691", "S_S2"=>"S:687-1273", "S_Beta1"=>"S:718-729", "S_3H"=>"S:747-769", "S_IL770"=>"S:770-831", "S_FPPR"=>"S:832-866", "S_FP"=>"S:867-909", "S_HR1"=>"S:910-985", "S_CH"=>"S:986-1035", "S_CD"=>"S:1036-1068", "S_Beta2"=>"S:1127-1135", "S_2turnHelix"=>"S:1148-1155", "S_HR2"=>"S:1164-1211", "S_TM"=>"S:1212-1234", "S_CT"=>"S:1235-1273", "E_TM"=>"E:8-38", "E_cytosol"=>"E:54-65", "E_CTD"=>"E:69-75", "N_9b_overlap"=>"N:4-101", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_SR"=>"N:176-206", "N_N3"=>"N:176-246", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419" )

######################################################################################################################################
    function NSP_to_ORF(domain::String, NSP::Int)
        ORFadd = NSP1ab_add[NSP]
        range = split(domain, ":")[2]
        first1 = split(range, "-")[1]
        first = parse(Int, first1)
        last1 = split(range, "-")[2]
        last = parse(Int, last1)
        ORFfirst1 = first + ORFadd
        ORFfirst = string(ORFfirst1)
        ORFlast1 = last + ORFadd
        ORFlast = string(ORFlast1)
        ORF_range = ""
        if NSP < 12
            ORF_range = "ORF1a:$(ORFfirst)-$(ORFlast)"
        else
            ORF_range = "ORF1b:$(ORFfirst)-$(ORFlast)"
        end
        return ORF_range
    end
    domain_residues_ORF["NSP12_NiRAN"] = "1a:4393-1b:241"
    for (dom, range) in domain_residues_NSP
        if range[1:4] == "NSP3"
            ORF = NSP_to_ORF(range, 3)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:4] == "NSP4"
            ORF = NSP_to_ORF(range, 4)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:4] == "NSP6"
            ORF = NSP_to_ORF(range, 4)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:5] == "NSP12"
            ORF = NSP_to_ORF(range, 12)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:5] == "NSP13"
            ORF = NSP_to_ORF(range, 12)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:5] == "NSP14"
            ORF = NSP_to_ORF(range, 12)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:5] == "NSP15"
            ORF = NSP_to_ORF(range, 12)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:3] ≠ "NSP"
            domain_residues_ORF[dom] = range
        end
    end
#####################################################################################################################################
sorted_ratio_gene_dicts = [chr_to_circ_gene_density_ratio_sort_by_gene, chr_to_circ_gene_density_ratio_sort_by_ratio] 
sorted_ratio_domain_dicts = [chr_to_circ_domain_density_ratio_sort_by_domain, chr_to_circ_domain_density_ratio_sort_by_ratio]
headers_gene = ["###### Chronic-to-Circulating Gene Mutation Density Ratios, Sorted by Gene ######", "###### Chronic-to-Circulating Gene Mutation Density Ratios, Sorted by Ratio ######"]
headers_domain = ["######### Chronic-to-Circulating Domain Mutation Density Ratios, Sorted by Domain #########", "######### Chronic-to-Circulating Domain Mutation Density Ratios, Sorted by Ratio #########"]
open("$(date)_$(fx_name).txt", "w") do g
    ct::Int = 0
    for sorted_dict in sorted_ratio_gene_dicts
        ct += 1
        println()
        println(headers_gene[ct])
        println(g)
        println(g, headers_gene[ct])
        for w in sorted_dict
            gene = w[1]
            ratio1 = string(Int(w[2]÷1) )
            ratio21 = string(round(digits=2, w[2]%1) )
            ratio22 = split(ratio21, ".")[2]
            if length(ratio22) == 1
                ratio22 = "$(ratio22)0"
            end
            ratio = "$(ratio1).$(ratio22)"
            println("                  ", rpad("$(gene)", 5), " = ", lpad("$(ratio)", 5), "       ", lpad("$(NSP_ranges[gene])", 16) )
            println(g, "                  ", rpad("$(gene)", 5), " = ", lpad("$(ratio)", 5), "       ", lpad("$(NSP_ranges[gene])", 16) )
        end
    end
    ct2::Int = 0
    for sorted_dict in sorted_ratio_domain_dicts
        ct2 += 1
        println()
        println(headers_domain[ct2])
        println(g)
        println(g, headers_domain[ct2])
        for w in sorted_dict
            gene = w[1]
            ratio1 = string(Int(w[2]÷1) )
            ratio21 = string(round(digits=2, w[2]%1) )
            ratio22 = split(ratio21, ".")[2]
            if length(ratio22) == 1
                ratio22 = "$(ratio22)0"
            end
            ratio = "$(ratio1).$(ratio22)"
            println("               ", rpad("$(gene)", 13), " = ", lpad("$(ratio)", 5), "   ", rpad("$(domain_residues_NSP[gene])", 15), "  ", rpad("$(domain_residues_ORF[gene])", 17) )
            println(g, "               ", rpad("$(gene)", 13), " = ", lpad("$(ratio)", 5), "   ", rpad("$(domain_residues_NSP[gene])", 15), "  ", rpad("$(domain_residues_ORF[gene])", 17) )
        end
    end
end
#####################################################################################################################################
AApos(m) = parse(Int, split(m, ":")[2][2:end-1])
AA_sub_type(m) = "$(m[1])$(m[end])"
gene_name(m) = split(m, ":")[1]
######################################################################################################################################
gene_sub_ranks_chr()
gene_sub_ranks_all()
gene_sub_pos_ranks_all()
gene_sub_pos_ranks_chr()
NSP_sub_ranks_chr()
NSP_sub_ranks_all()
NSP_sub_pos_ranks_all()
NSP_sub_pos_ranks_chr()
######################################################################################################################################
println()
print(sub_types_at_every_site_combined["ORF9b"][2])
print(" | ", sub_types_at_every_site_combined_ct["ORF9b"][2]);    println()
print(sub_types_at_every_site_combined["ORF9b"][92])
print(" | ", sub_types_at_every_site_combined_ct["ORF9b"][92]);   println()
#####################################################################################################################################
gene_AA_sites = Dict{String, BitSet}()
for (gene, length) in gene_AA_lengths
    gene_AA_sites[gene] = BitSet([1:length...])
end
#for (gene, bitset) in gene_AA_sites; println(gene); println(bitset); end
############################################################
gene_AA_sites_used = Dict{String, BitSet}()
for gene in gene_array
    gene_AA_sites_used[gene] = BitSet()
end
for (sub, ct) in AA_muts_ct_no_dels_all
    gene = aa_gene_comprehensive_dict[sub]
    subby = split(sub, ":")[2]
    AA_site = parse(Int, subby[2:end-1])
    push!(gene_AA_sites_used[gene], AA_site)
end
############################################################
gene_AA_sites_not_used = Dict{String, BitSet}()
for gene in gene_array
    gene_AA_sites_not_used[gene] = BitSet()
end
for (gene, bitset) in gene_AA_sites
    bitset_used = gene_AA_sites_used[gene]
    unused_AA_sites = setdiff(bitset, bitset_used)
    gene_AA_sites_not_used[gene] = unused_AA_sites
end
###########################################################
for (gene, unused_AA_sites) in gene_AA_sites_not_used
    print("$(gene) = ")
    print(" | ")
    for unused in unused_AA_sites
        print("$(unused) |")
    end
end
for (gene, unused_AA_sites) in gene_AA_sites_not_used
    for AA_site in unused_AA_sites
        placeholder_sub = "$(gene):W$(AA_site)W"
        AA_muts_ct_no_dels_all[placeholder_sub] = 0
    end
end
#####################################################################################################################################
#NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
NSP1_muts_ct_no_dels_all = Dict{String, Int}()
NSP2_muts_ct_no_dels_all = Dict{String, Int}()
NSP3_muts_ct_no_dels_all = Dict{String, Int}()
NSP4_muts_ct_no_dels_all = Dict{String, Int}()
NSP5_muts_ct_no_dels_all = Dict{String, Int}()
NSP6_muts_ct_no_dels_all = Dict{String, Int}()
NSP7_muts_ct_no_dels_all = Dict{String, Int}()
NSP8_muts_ct_no_dels_all = Dict{String, Int}()
NSP9_muts_ct_no_dels_all = Dict{String, Int}()
NSP10_muts_ct_no_dels_all = Dict{String, Int}()
NSP11_muts_ct_no_dels_all = Dict{String, Int}()
NSP12_muts_ct_no_dels_all = Dict{String, Int}()
NSP13_muts_ct_no_dels_all = Dict{String, Int}()
NSP14_muts_ct_no_dels_all = Dict{String, Int}()
NSP15_muts_ct_no_dels_all = Dict{String, Int}()
NSP16_muts_ct_no_dels_all = Dict{String, Int}()
NSP_muts_ct_no_dels_all_array = [NSP1_muts_ct_no_dels_all, NSP2_muts_ct_no_dels_all, NSP3_muts_ct_no_dels_all, NSP4_muts_ct_no_dels_all, NSP5_muts_ct_no_dels_all, NSP6_muts_ct_no_dels_all, NSP7_muts_ct_no_dels_all, NSP8_muts_ct_no_dels_all, NSP9_muts_ct_no_dels_all, NSP10_muts_ct_no_dels_all, NSP11_muts_ct_no_dels_all, NSP12_muts_ct_no_dels_all, NSP13_muts_ct_no_dels_all, NSP14_muts_ct_no_dels_all, NSP15_muts_ct_no_dels_all, NSP16_muts_ct_no_dels_all]       
NSP_muts_ct_no_dels_all = Dict{String, Int}()
for (sub, count) in AA_muts_ct_no_dels_all
#    println(sub)
    gene = aa_gene_comprehensive_dict[sub]
    if gene == "ORF1a" || gene == "ORF1b"
        AA_site = aa_pos_comprehensive_dict[sub]
        if AA_site < 4402
            NSP_sub = ORF1abMut_to_NSP(sub)
            NSP_muts_ct_no_dels_all[NSP_sub] = count
            NSP = NSP_muts_gene_dict[NSP_sub]
#            println(NSP_sub)
            NSP_num = parse(Int, NSP[4:end])
            NSP_dict = NSP_muts_ct_no_dels_all_array[NSP_num]
            NSP_dict[NSP_sub] = count
        end
    end
end
###################################################################################################################################
NSP1_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP2_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP3_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP4_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP5_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP6_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP7_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP8_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP9_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP10_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP11_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP12_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP13_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP14_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP15_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP16_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
NSP_muts_ct_pos_only_no_dels_all_array = [NSP1_muts_ct_pos_only_no_dels_all, NSP2_muts_ct_pos_only_no_dels_all, NSP3_muts_ct_pos_only_no_dels_all, NSP4_muts_ct_pos_only_no_dels_all, NSP5_muts_ct_pos_only_no_dels_all, NSP6_muts_ct_pos_only_no_dels_all, NSP7_muts_ct_pos_only_no_dels_all, NSP8_muts_ct_pos_only_no_dels_all, NSP9_muts_ct_pos_only_no_dels_all, NSP10_muts_ct_pos_only_no_dels_all, NSP11_muts_ct_pos_only_no_dels_all, NSP12_muts_ct_pos_only_no_dels_all, NSP13_muts_ct_pos_only_no_dels_all, NSP14_muts_ct_pos_only_no_dels_all, NSP15_muts_ct_pos_only_no_dels_all, NSP16_muts_ct_pos_only_no_dels_all]
NSP_muts_ct_pos_only_no_dels_all = Dict{String, Int}()
for i in 1:length(NSP_muts_ct_pos_only_no_dels_all_array)
    NSP_all_dict = NSP_muts_ct_pos_only_no_dels_all_array[i]
    NSP = NSP_array[i]
    NSP_len = NSP_AA_size[NSP]
    for j in 1:NSP_len
        NSP_pos = "$(NSP):$(j)"
        NSP_all_dict[NSP_pos] = 0
        NSP_muts_ct_pos_only_no_dels_all[NSP_pos] = 0
    end
end
for i in 1:length(NSP_muts_ct_pos_only_no_dels_all_array)
    NSP = NSP_array[i]
    NSP_dict = NSP_muts_ct_no_dels_all_array[i]
    NSP_pos_dict = NSP_muts_ct_pos_only_no_dels_all_array[i]
    for (sub, ct) in NSP_dict
        pos = parse(Int, split(sub, ":")[2][2:end-1])
        sub_pos = "$(NSP):$(pos)"
        NSP_pos_dict[sub_pos] += ct
        NSP_muts_ct_pos_only_no_dels_all[sub_pos] += ct
    end
end
total_time_post_load_cell = time() - start_time_post_load_cell
total_time_post_load_cell_rd = round(digits=1, total_time_post_load_cell)
println()
println("Total Time for Post-Load Cell = $(total_time_post_load_cell_rd) Seconds")

2025-08-19__549PM

Avg AA Subs per Chronic Sequence = 17.68
Avg AA Subs per Circulating Sequence = 2.198
Adjustment Factor for greater number of AA muts per chronic seq compared to circ seq = 0.1243

Total Singlets = 2222 | Total Groups = 917 | Total Independent Chronics = 3139
Total Qualifying Circulating Sequences = 11857354
(Total Qualifying Circulating Sequences)/(Total Independent Chronics) = 3777.43
Combined Adjustment Factor = 469.574  |  Combined Adjustment Factor Check = 469.574

###### Chronic-to-Circulating Gene Mutation Density Ratios, Sorted by Gene ######
                  NSP1  =  1.13            ORF1a:1-180
                  NSP2  =  0.48          ORF1a:181-818
                  NSP3  =  0.90         ORF1a:819-2763
                  NSP4  =  1.02        ORF1a:2764-3263
                  NSP5  =  0.75        ORF1a:3264-3569
                  NSP6  =  0.92        ORF1a:3570-3859
                  NSP7  =  0.98        ORF1a:3860-3942
                  NSP8  =  1.69        

LoadError: InterruptException:

In [245]:
### NO NEED TO RUN MOST OF THE TIME NOW!!!!! | NEW | 2026_02_06 v6 | Making pango_to_pango_unaliased_v2 & pango consensus Dicts + MANY Others | Runtime = 154 seconds
####### Making pango_to_pango_unaliased_v2 —— using pango_variant_alias_key_2026_01_03_update.json from https://github.com/cov-lineages/pango-designation/blob/master/pango_designation/alias_key.json
start = time(); date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
println("nuc_genome_pango_dict length = $(length(nuc_genome_pango_dict))")

missing_AA_dict_keys = Set{String}()
############################################################################################################################################################################
#################### Making/filling index_to_tuple, tuple_to_index, index_to_date_str, date_str_to_index...etc dictionaries ################################################
############################################################################################################################################################################
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
###################################
chronic_pango_set = Set{String}()
chronic_pango_unaliased_set = Set{String}()
chronic_clade_set = Set{String}()
for seq in all_qualifying_seqs_set
    push!(chronic_pango_set, seq_pango[seq])
    push!(chronic_pango_unaliased_set, seq_pango_unaliased[seq])
    push!(chronic_clade_set, seq_clade[seq])
end
chronic_pango_set_len = length(chronic_pango_set)
chronic_pango_unaliased_set_len = length(chronic_pango_unaliased_set)
chronic_clade_set_len = length(chronic_clade_set)
println("Number of Different Pango Lineages in Chronics = $(chronic_pango_set_len)")
println("Number of Different Pango_Unaliased Lineages in Chronics = $(chronic_pango_unaliased_set_len)")
println("Number of Different Clades in Chronics = $(chronic_clade_set_len)")
println()
#####################################################################################################################################
all_fucking_pangos = union(pango_set, pango_index_date_set, chronic_pango_set)
push!(all_fucking_pangos, "B.1.640")
push!(all_fucking_pangos, "B.1.640.1")
push!(all_fucking_pangos, "B.1.640.2")
push!(all_fucking_pangos, "B.1.258.2")
push!(all_fucking_pangos, "G.1")
#####################################################################################################################################
nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
###################
index_to_tuple = Dict{Int, Tuple{Int, Int, Int}}()
tuple_to_index = Dict{Tuple{Int, Int, Int}, Int}()        
###########################################################
for (seq, date) in seq_collection_date
    date_arr = string.(collect(date))
    if sum(date_arr .== "-") ≠ 2
        println("Doesn't have two dashes in date = $(seq)")
    else
        year = string(split(date, "-")[1])
        month = string(split(date, "-")[2])
        day = string(split(date, "-")[3])
        if length(month) == 1 && month ≠ "0"
            month = add_leading_zero(month)
        end
        if length(day) == 1 && day ≠ "0"
            day = add_leading_zero(day)
        end
        seq_collection_date[seq] = year*"-"*month*"-"*day
    end
end 
###################
index = 0
for year in 2020:2027
    for month in 1:12
        if year%4 == 0
            month_days = leap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        else
            month_days = nonleap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_tuple[index] = (year, month, day)
            end
        end
    end
end
for (index, date) in index_to_tuple
    tuple_to_index[date] = index
end
for y in 2020:2027
    tuple_to_index[(y, 0, 0)] = y*1000000
    index_to_tuple[y*1000000] = (y, 0, 0)
    for m in 1:12
        tuple_to_index[(y, m, 0)] = y*1000000 + m*1000
        index_to_tuple[y*1000000 + m*1000] = (y, m, 0)
    end
end
tuple_to_index[(0, 0, 0)] = 1000000000
index_to_tuple[1000000000] = (0, 0, 0)
###################################################################################################################
############################################################################################################################################################################
pango_to_pango_unaliased_v2["G.1"] = "B.1.258.2.1"
pgct = 0
for (pngo, unal) in pango_to_pango_unaliased_v2
    pgct += 1
    if pgct < 15
        println("$(pngo) = $(unal)")
    end
end
println("length(keys(pango_to_pango_unaliased_v2) = $(length(keys(pango_to_pango_unaliased_v2)))")
########################################################################################################################################################################
########################################################################################################################################################################
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
date_hour = Dates.format(now(), "yyyy_mm_dd_Ip")
missing_keys = String[]
ndjson = "___pango_consensus_sequences/pango_consensus_sequences_summary_2025_06_25.ndjson"
for line in eachline(ndjson)
    j = JSON3.read(line)
    pango = j.lineage
    if !(pango in pango_set)
        push!(missing_keys, pango)
    end
    push!(pango_set, pango)
end
for pango in keys(pango_date_index_ct)
    if !(pango in keys(pango_to_pango_unaliased_v2))
        push!(pango_set, pango)
    end
end
date_now = Dates.format(now(), "yyyy_mm_dd__IMMp"); println(date_now)
#pango_to_pango_unaliased = Dict{String, String}()
clade_pango_set = Dict{String, Set{String}}()
#################################################################################
pango_nuc_sub_WT = Dict{String, Set{String}}()
pango_nuc_del_WT = Dict{String, Set{String}}()
pango_nuc_sub_private = Dict{String, Set{String}}()
pango_nuc_del_private = Dict{String, Set{String}}()
pango_nuc_sub_revs = Dict{String, Set{String}}()
pango_nuc_del_revs = Dict{String, Set{String}}()

pango_AAsub_WT = Dict{String, Set{String}}()
pango_AAsub_WT_pos_only = Dict{String, Set{String}}()
pango_AAdel_WT = Dict{String, Set{String}}()
pango_AAsub_private = Dict{String, Set{String}}()
pango_AAdel_private = Dict{String, Set{String}}()
pango_AAsub_revs = Dict{String, Set{String}}()
pango_AAdel_revs = Dict{String, Set{String}}()

pango_frameshifts_WT = Dict{String, Set{String}}()
pango_designation_date = Dict{String, String}()
#####################################################
for pango in pango_set
    pango_nuc_sub_WT[pango] = Set{String}()
    pango_nuc_del_WT[pango] = Set{String}()
    pango_nuc_sub_private[pango] = Set{String}()
    pango_nuc_del_private[pango] = Set{String}()
    pango_nuc_sub_revs[pango] = Set{String}()
    pango_nuc_del_revs[pango] = Set{String}()
########
    pango_AAsub_WT[pango] = Set{String}()
    pango_AAsub_WT_pos_only[pango] = Set{String}()
    pango_AAdel_WT[pango] = Set{String}()
    pango_AAsub_private[pango] = Set{String}()
    pango_AAdel_private[pango] = Set{String}()
    pango_AAsub_revs[pango] = Set{String}()
    pango_AAdel_revs[pango] = Set{String}()
#########
    pango_frameshifts_WT[pango] = Set{String}()
end
#################################################################################
for clade in clade_set
    clade_pango_set[clade] = Set{String}()
end
ndjson = "___pango_consensus_sequences/pango_consensus_sequences_summary_2025_06_25.ndjson"
missing_keys_for_WT = Set{String}()
###################################################################
#pango_consensus_nuc_subs_WT = 
#pango_consensus_nuc_dels_WT = 
#pango_consensus_nuc_subs = 
#pango_consensus_nuc_dels = 
#pango_consensus_nuc_revs = 
#pango_consensus_nuc_del_revs = 
#pango_consensus_AA_subs_WT = 
#pango_consensus_AA_dels_WT = 
#pango_consensus_AA_subs = 
#pango_consensus_AA_dels = 
#pango_consensus_AA_revs = 
#pango_consensus_AA_del_revs = 
#pango_consensus_
###################################################################
for line in eachline(ndjson)
    j = JSON3.read(line)
    pango = j.lineage
    unaliased = j.unaliased
    clade = j.nextstrainClade
    push!(clade_pango_set[clade], pango)
#    pango_to_pango_unaliased[pango] = unaliased
####################
    parent = j.parent
    children = j.children
####################
    nuc_subs_WT = j.nucSubstitutions
    nuc_dels_WT = j.nucDeletions
    nuc_subs = j.nucSubstitutionsNew
    nuc_dels = j.nucDeletionsNew
    nuc_revs = j.nucSubstitutionsReverted
    nuc_del_revs = j.nucDeletionsReverted
#####################
    for nuc_sub in nuc_subs_WT
        push!(pango_nuc_sub_WT[pango], nuc_sub)
    end
    for ndwt in nuc_dels_WT
        push!(pango_nuc_del_WT[pango], ndwt)
    end
    for n in nuc_subs
        push!(pango_nuc_sub_private[pango], n)
    end
    for n in nuc_dels
        push!(pango_nuc_del_private[pango], n)
    end
    for n in nuc_revs
        push!(pango_nuc_sub_revs[pango], n)
    end
    for n in nuc_del_revs
        push!(pango_nuc_del_revs[pango], n)
    end
########################################################
    AA_subs_WT = j.aaSubstitutions
    AA_dels_WT = Set(j.aaDeletions)
    AA_subs = j.aaSubstitutionsNew
    AA_dels = j.aaDeletionsNew
    AA_revs = j.aaSubstitutionsReverted
    AA_del_revs = j.aaDeletionsReverted
##################################################
    AA_subs_WT_pos_only = Set{String}()
    delete!(AA_dels_WT, "")
    for sub in AA_subs_WT
        if !haskey(AA_gene_and_pos_dict, sub)
            push!(missing_keys_for_WT, sub)
            continue
        end
        sub_po = AA_gene_and_pos_dict[sub]
        push!(AA_subs_WT_pos_only, sub_po)
    end 
    for AA_sub_po in AA_subs_WT_pos_only
        push!(pango_AAsub_WT_pos_only[pango], AA_sub_po)
    end
##################################################
    for AAsub in AA_subs_WT
        push!(pango_AAsub_WT[pango], AAsub)
    end
    for AAdel in AA_dels_WT
        push!(pango_AAdel_WT[pango], AAdel)
    end
    for AAsub in AA_subs
        push!(pango_AAsub_private[pango], AAsub)
    end
    for AAdel in AA_dels
        push!(pango_AAdel_private[pango], AAdel)
    end
    for AArev in AA_revs
        push!(pango_AAsub_revs[pango], AArev)
    end
    for AAdelrev in AA_del_revs
        push!(pango_AAdel_revs[pango], AAdelrev)
    end
    frameshiftsWT = j.frameShifts
    for fs in frameshiftsWT
        push!(pango_frameshifts_WT[pango], fs)
    end
    designation_date = j.designationDate
    pango_designation_date[pango] = designation_date
end
#####################################################################################################################################################################
#####################################################################################################################################################################
push!(pango_nuc_del_WT["B.1.617.2"], "28271")
push!(pango_nuc_del_private["B.1.617.2"], "28271")
#####################################################################################################################################################################
#####################################################################################################################################################################
println("Done Making Pango Consensus Sequences! (Part 1)"); print("\n"^1)
#####################################################################################################################################################################
println("Total Missing Keys for AA_pos_only_gene_and_pos_dict = $(length(missing_keys_for_WT))")
#missing_keys_for_WT_sort = sort(collect(missing_keys_for_WT), by = x -> mp_AA_gene_sortKey_2(x))
missing_keys_for_WT_join = join(collect(missing_keys_for_WT), ", ")
println("missing_keys_for_WT_join = ", "|", missing_keys_for_WT_join, "|"); print("\n"^1)
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################# NEW SECTION (2026-01-03)   #################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
### Making consensus muts for new sequences (using Angie Hinrichs' tree mut TSV file) —— MUST RUN load_all_seq_dicts FIRST
df_tree_muts = CSV.read("___pango_consensus_sequences/pango_clade_mutations_from_Angie_Hinrichs_2026_01_03.tsv", DataFrame, delim='\t')
##################synonymous_nuc_AA_sorted_dict, nonsynonymous_nuc_AA_sorted_dict, all_nuc_AA_sorted_dict = nuc_to_AA(pango, new_muts)########################################################################################################################################
total_rows = nrow(df_tree_muts)
tree_pangos = df_tree_muts[!, 1]
tree_muts = df_tree_muts[!, 2]
#####################################################################################################################
#  Example of 1st, then 2nd column (i.e. tree_pangos & tree_muts) ——  AY.112.2 | AY.112 > G23012C > G4162T > G27516A
#####################################################################################################################
tree_pangos_set = Set(tree_pangos)
tree_muts_set = Set(tree_muts)
################################################
bad_pangos = Vector{String}()
################################################
OG_tree_pangos_set_len = length(tree_pangos_set)
println("OG_tree_pangos_set_len = $(OG_tree_pangos_set_len)")
excluded_bits = ["misc", "proposed", '_']
for tpango in tree_pangos_set
    for exclude in excluded_bits
        if occursin(exclude, tpango)
            push!(bad_pangos, tpango)
        end
    end
end
for baddy in bad_pangos
    delete!(tree_pangos_set, baddy)
end
NEW_tree_pangos_set_len = length(tree_pangos_set)
println("NEW_tree_pangos_set_len = $(NEW_tree_pangos_set_len)")
println("bad_pangos length = $(length(bad_pangos))")
bad_pangos_sort = sort(bad_pangos)
bad_pangos_sort_join = join(bad_pangos_sort, ", ")
bad_pangos_sort_join_newlines = join(bad_pangos_sort, "\n")
###########
println(bad_pangos_sort_join)
############################################################################################################################################################################
pango_unaliased_to_pango_prefix = Dict{String, String}()
pango_to_pango_unaliased_v2 = Dict{String, String}()
pango_unaliased_to_pango = Dict{String, String}()
##################################################################
pango_unaliased_predecessor_meta_dict = Dict{String, Dict{Int, String}}()
pango_predecessor_meta_dict = Dict{String, Dict{Int, String}}()
##################################################################
for (prefix, unaliased) in pango_prefix_to_pango_unaliased
    pango_unaliased_to_pango_prefix[unaliased] = prefix
end
for pango in tree_pangos_set
    dotpts_ct = count('.', pango)
    if dotpts_ct ≥ 1
        dotsplits = split(pango, ".")
        prefix = string(dotsplits[1])
        prefix_unaliased = get(pango_prefix_to_pango_unaliased, prefix, prefix)
        last_pts = join(dotsplits[2:end], ".")
        pango_to_pango_unaliased_v2[pango] = "$(prefix_unaliased).$(last_pts)"
    else
        pango_to_pango_unaliased_v2[pango] = pango
    end
end
#####################################################################
pango_to_pango_unaliased_v2_key_set = Set(collect(keys(pango_to_pango_unaliased_v2)))
no_key_pangos = Set(["C.1.2", "B.1.1.70", "B.1.177.46", "B.1.36", "B.1.551", "B.1.1.329", "B.1.1.170", "C.26", "B.1.214.3", "B.1.609", "B.55", "A.2.5.3", "B.1.1.117", "C.13", "B.1.1.523", "B.1.470", "B.1.1.67", "B.39", "B.1.36.31", "B.1.36.22", "B.4", "B.1.1.500", "B.1.258.17", "B.1.1.101", "B.1.1.141", "B.1.260", "B.1.1.360", "AE.1", "B.1.404", "B.1.351.1", "AE.8", "B.1.1.10", "B.1.36.19", "B.1.587", "B.1.465", "B.1.177.52", "B.1.1.404", "B.1.436", "A.5", "B.1.1.221", "B.1.396", "B.1.428", "C.4", "B.1.214.2", "B.1.568", "B.1.147", "B.1.1.442", "B.1.1.47", "B.1.575.1", "B.1.177.4", "B.1.527", "B.1.1.354", "B.1.530", "B.1.361", "B.1.36.36", "B.1.126", "B.1.1.297", "B.1.420", "B.1.1.406", "B.1.36.29", "B.1.560", "B.1.214.1", "W.1", "B.1.177.12", "B.1.629", "B.1.1.37", "B.1.1.192", "N.6", "A.23", "B.1.177.54", "A.2.5.1", "A.3", "B.1.1.263", "B.1.379", "B.1.1.376", "B.1.240.1", "B.1.466.1", "B.1.1.241", "B.1.1.368", "B.1.177.32", "B.1.1.420", "B.1.1.413", "B.1.236", "B.1.2", "B.23", "B.1.1.418", "B.1.416.1", "B.1.1.54", "B.1.540", "AZ.5", "B.1.389", "B.1.433", "B.1.1.205", "B.1.1.434", "B.1.1.359", "C.2", "B.1.600", "B.1.9", "B.1.575", "B.1.177.57", "B.1.627", "B.1.36.35", "A.19", "B.1.371", "B.1.462", "B.1.22", "B.1.111", "B.1.258", "B.1.427", "B.1.177.53", "B.1.626", "B.1.1.294", "B.1.36.38", "B.1.1.332", "B.1.1.417", "B.1.1.521", "B.1.558", "B.1.36.18", "B.1.544", "B.1.1.375", "B.1.1.412", "B.1.1.485", "B.1.1.326", "B.1.375", "B.1.1.243", "B.1.1.369", "B.1.336", "B.1.577", "B.1.409", "B.1.219", "B.1.1.301", "B.1.1.433", "B.1.128", "B.1.179", "B.1.1.519", "B.1.1.307", "B.1.537", "B.1.1.265", "C.38", "B.1.177.44", "B.1.620", "AD.2", "C.23", "B.1.189", "B.1.349", "B.1.567", "B.1.398", "AZ.2", "B.1.1.25", "B.1.1.163", "B.1.497", "B.1.480", "B.1.1.198", "B.1.320", "B.1.1.121", "B.1.402", "B.6.6", "B.1.8", "B.1.298", "B.1.177.8", "B.1.1.39", "B.1.324", "B.1.1.351", "B.1.1.161", "B.1.1.279", "B.1.1.46", "B.1.177.83", "B.1.177.15", "B.1.346", "B.1.566", "B.1.78", "B.1.239", "A", "B.1.1.318", "B.1.397", "B.1.177.10", "B.1.1.416", "B.1.1.27", "AZ.3", "B.1.319", "B.1.619", "B.1.177.7", "B.1.258.3", "B.1.1.317", "B.1.605", "B.1.146", "B.1.1.61", "A.25", "C.40", "B.3", "B.1.438", "B.1.1.33", "B.1.192", "B.1.468", "C.11", "B.1.177.18", "B.1.471", "B.1.1.52", "B.1.1.171", "C.1", "B.1.1.348", "B.1.1.507", "B.1.177.82", "B.1.533", "B.1.1.306", "B.1.524", "B.1.149", "B.1.619.1", "B.1.36.9", "B.1.570", "B.1.1.50", "B.1.517", "B.1.499", "B.1.177.60", "B.1.1.378", "B.1.177.62", "C.16", "C.36.3", "B.1.1.459", "B.1.177.75", "A.27", "B.1.1.207", "B.1.618", "B.1.391", "B.1.1.115", "A.29", "B.1.1.1", "L.3", "A.28", "B.1.1.302", "B.1.3", "C.2.1", "B.1.1.135", "B.1.1.316", "B.1.393", "B.1.177.87", "R.1", "B.1.1.176", "C.32", "B.1.636", "B.1.1.133", "B.1.1.462", "N.3", "B.1.399", "B.1.1.487", "B.1.1.311", "B.1.606", "B.1.617.1", "B.1.265", "B.1.177.86", "N.9", "B.1.1.374", "B.1.1.228", "B.1.177", "B.1.177.5", "A.2.5.2", "B.1.1.57", "C.33", "B.1.538", "B.1.258.14", "B.1.1.305", "B.1.505", "B.1.1.397", "A.1", "B.1.565", "AT.1", "B.1.1.214", "B.1.177.16", "C.35", "B.6", "B.1.356", "B.1.177.21", "B.1.1.153", "B.1.1.432", "B.1.1.181", "B.1.1.312", "B.1.1.216", "B.1.1.526", "A.21", "B.1.380", "B.1.210", "B.1.1.398", "B.1.232", "B.1.1.220", "C.36", "B.1.438.1", "B.1.503", "B.1.36.7", "B.1.426", "B.1.1.159", "B.1.1.231", "A.18", "B.1.177.77", "B.1.1.99", "B.1.1.298", "B.1.453", "B.1.1.253", "N.5", "B.1.367", "B.1.549", "C.14", "B.1.400", "B.1.1.284", "AZ.4", "B.1.459", "B.1.243", "C.36.3.1", "B.1.474", "B.1.221", "B.1.630", "C.29", "B.1.1.528", "B.1.1.203", "B.1.1.269", "B.1.340", "B.1.634", "B.1.1.174", "B.1.596.1", "B.1.311", "B.1.36.24", "B.1.509", "B.1.36.39", "B.1.1.189", "B.1.36.16", "B.1.441", "B.1.91", "B.1.416", "B.1.1.56", "B.1.258.11", "B.1.417", "B.1.525", "B.1.561", "B.1.1.525", "B.1.1.277", "B.1.177.35", "B.1.240", "B.1.1.382", "B.1.234", "B.1.1.273", "B.1.1.291", "N.4", "B.1.617.3", "A.2", "B.1.523", "B.1.279", "B.1.1.63", "B.1.1.51", "B.1.1.157", "B.1.415.1", "B.1.212", "B.1.177.73", "B.1.1.371", "A.2.5", "B.1.1.44", "B.1.1.464", "B.1.362", "D.2", "B.1.241", "B.1.1.486", "B.1.377", "B.1.1.53", "B.1.588", "B.1.1.391", "B.1.1.186", "B.1.1.274", "B.1.623", "B.1.36.1", "B.1.195", "B.1.237", "B.1.110", "B.1.1.111", "AZ.1", "B.1.395", "B.1.381", "C.39", "B.1.206", "B.1.177.81", "B.1.214", "B.40", "B.1.110.3", "B.1.1.83", "B.1.411", "B.1.625", "B.1.350"])
for pango in all_fucking_pangos
    if !haskey(pango_to_pango_unaliased_v2, pango)
        dotpts_ct = count('.', pango)
        if dotpts_ct ≥ 1
            dotsplits = split(pango, ".")
            prefix = string(dotsplits[1])
            prefix_unaliased = get(pango_prefix_to_pango_unaliased, prefix, prefix)
            last_pts = join(dotsplits[2:end], ".")
            pango_to_pango_unaliased_v2[pango] = "$(prefix_unaliased).$(last_pts)"
        end
    end
end
pango_to_pango_unaliased_v2["B.1.1.529"] = "B.1.1.529"
pango_to_pango_unaliased_v2["LF.3"] = "B.1.1.529.2.86.1.1.16.1.3"
pango_to_pango_unaliased_v2["LF.3.1"] = "B.1.1.529.2.86.1.1.16.1.3.1"
pango_to_pango_unaliased_v2["B.1.469"] = "B.1.469"
pango_to_pango_unaliased_v2["A"] = "A"
pango_to_pango_unaliased_v2["B"] = "B"
pango_to_pango_unaliased_v2["B.1.1.482"] = "B.1.1.482"
pango_to_pango_unaliased_v2["B.1.1.370"] = "B.1.1.370"
pango_to_pango_unaliased_v2["B.1.596"] = "B.1.596"
pango_to_pango_unaliased_v2["B.1.415"] = "B.1.415"
####################################################################
pango_unaliased_set = Set{String}()
for (pango, unaliased) in pango_to_pango_unaliased_v2
    pango_unaliased_to_pango[unaliased] = pango
    push!(pango_unaliased_set, unaliased)
end
####################################################################
all_fucking_pangos = union(all_fucking_pangos, pango_to_pango_unaliased_v2_key_set)
for pango in all_fucking_pangos
    unaliased = pango_to_pango_unaliased_v2[pango]
    pango_unaliased_predecessor_meta_dict[unaliased] = Dict{Int, String}()
#    println(pango)
    dotpts_ct = count('.', unaliased)
    if dotpts_ct ≥ 1
        dotsplits = split(unaliased, ".")
        for i in 1:dotpts_ct
            predecessor_i_unaliased = join(dotsplits[1:end-i], ".")
            pango_unaliased_predecessor_meta_dict[unaliased][i] = predecessor_i_unaliased
#            println("$(pango), $(unaliased), $(i) = $(predecessor_i_unaliased)")
        end
    end
end
pango_unaliased_predecessor_meta_dict["B.1.1.370"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.1.370"][1] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.1.370"][2] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.1.370"][3] = "B"
pango_unaliased_to_pango["B.1.1.370"] = "B.1.1.370"

pango_unaliased_predecessor_meta_dict["B.1.1.482"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.1.482"][1] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.1.482"][2] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.1.482"][3] = "B"
pango_unaliased_to_pango["B.1.1.482"] = "B.1.1.482"

pango_unaliased_predecessor_meta_dict["B.1.596"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.596"][1] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.596"][2] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.596"][3] = "B"
pango_unaliased_to_pango["B.1.596"] = "B.1.596"

pango_unaliased_predecessor_meta_dict["B.1.415"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.415"][1] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.415"][2] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.415"][3] = "B"
pango_unaliased_to_pango["B.1.415"] = "B.1.415"

for pango in all_fucking_pangos
    pango_predecessor_meta_dict[pango] = Dict{Int, String}()
    unaliased = pango_to_pango_unaliased_v2[pango]
    for (i, predecessor_i) in pango_unaliased_predecessor_meta_dict[unaliased]
        predecessor_i_pango = pango_unaliased_to_pango[predecessor_i]
        pango_predecessor_meta_dict[pango][i] = predecessor_i_pango
    end
end
#######################################################
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][1] = "B.1.1.529.2.86.1.1.16.1.3"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][2] = "B.1.1.529.2.86.1.1.16.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][3] = "B.1.1.529.2.86.1.1.16"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][4] = "B.1.1.529.2.86.1.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][5] = "B.1.1.529.2.86.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][6] = "B.1.1.529.2.86"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][7] = "B.1.1.529.2"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][8] = "B.1.1.529"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][9] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][10] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529.2.86.1.1.16.1.3.1"][11] = "B"
pango_predecessor_meta_dict["LF.3.1"] = Dict{Int, String}()
pango_predecessor_meta_dict["LF.3.1"][1] = "LF.3"
pango_predecessor_meta_dict["LF.3.1"][2] = "JN.1.16.1"
pango_predecessor_meta_dict["LF.3.1"][3] = "JN.1.16"
pango_predecessor_meta_dict["LF.3.1"][4] = "JN.1"
pango_predecessor_meta_dict["LF.3.1"][5] = "BA.2.86.1"
pango_predecessor_meta_dict["LF.3.1"][6] = "BA.2.86"
pango_predecessor_meta_dict["LF.3.1"][7] = "BA.2"
pango_predecessor_meta_dict["LF.3.1"][8] = "B.1.1.529"
pango_predecessor_meta_dict["LF.3.1"][9] = "B.1.1"
pango_predecessor_meta_dict["LF.3.1"][10] = "B.1"
pango_predecessor_meta_dict["LF.3.1"][11] = "B"
pango_unaliased_predecessor_meta_dict["B.1.1.529"] = Dict{Int, String}()
pango_unaliased_predecessor_meta_dict["B.1.1.529"][1] = "B.1.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529"][2] = "B.1"
pango_unaliased_predecessor_meta_dict["B.1.1.529"][3] = "B"
pango_predecessor_meta_dict["B.1.1.529"] = Dict{Int, String}()
pango_predecessor_meta_dict["B.1.1.529"][1] = "B.1.1"
pango_predecessor_meta_dict["B.1.1.529"][2] = "B.1"
pango_predecessor_meta_dict["B.1.1.529"][3] = "B"
############################################################################################################################################################################
gene_array = ["ORF1a", "ORF1b", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF9b"]
#######################################################################################################################################
forbidden_pangos = Set(["XCD", "XCT", "XEA", "XFR", "XFS", "XFT", "XFT.1", "XFU", "XGD"])
#######################################################################################################################################
pango_new_nuc_muts = Dict{String, Set{String}}()
pango_new_nuc_muts_WT = Dict{String, Set{String}}()
pango_new_nuc_muts_pos_only = Dict{String, Set{Int}}()
###########################
pango_new_AA_muts = Dict{String, Set{String}}()
pango_new_AA_muts_WT = Dict{String, Set{String}}()
pango_new_AA_muts_pos_only = Dict{String, Set{String}}()
###########################
nuc_mut_all_and_chr_set_missing = Set{String}()
new_pango_ct = 0
empty_string_pangos = Set{String}()
for i in 1:length(tree_muts)
    if tree_pangos[i] in tree_pangos_set
        pango = convert(String, tree_pangos[i])
        unaliased = pango_to_pango_unaliased_v2[pango]
        if ismissing(tree_muts[i])
            println("missing tree muts: $(pango)")
            continue
        end
        new_muts_pre0 = replace(tree_muts[i], r"        "=>",")
        new_muts_pre1 = replace(new_muts_pre0, r"[TCAGN] [TCAGN]"=>",")
        new_muts_pre2 = replace(new_muts_pre1, r"\s"=>"")
        new_muts_pre3 = replace(new_muts_pre2, ">"=>",")
        new_muts_pre4 = replace(new_muts_pre3, ",,"=>",")
#        println("new_muts_pre3 = $(new_muts_pre3)")
        comma_splits = string.(split(new_muts_pre3, ","))
        predecessor_vec = String[]
        predecessor = string(comma_splits[1])
        if '_' in predecessor
            predecessor = string(split(predecessor, "_")[1])
        end
#        predecessor_unaliased = pango_to_pango_unaliased_v2[predecessor]
        push!(predecessor_vec, predecessor)
#        println(predecessor)
        for j in 1:length(pango_unaliased_predecessor_meta_dict[unaliased])
            predecessor_j = pango_unaliased_predecessor_meta_dict[unaliased][j]
            predecessor_j_pango = pango_unaliased_to_pango[predecessor_j]
            push!(predecessor_vec, predecessor_j_pango)
        end
        new_muts_pre = comma_splits[2:end]
        new_muts = String[]
        for mut in new_muts_pre
            if !('N' in mut)
                push!(new_muts, mut)
            end
        end
#        println("new_muts = $(new_muts)")
#        if !haskey(pango_nuc_sub_private, pango) && !haskey(pango_AAsub_private, pango)
#            pango_nuc_sub_WT[pango] = Set{String}()
#            pango_nuc_del_WT[pango] = Set{String}()
#            pango_nuc_sub_private[pango] = Set{String}()
#            pango_nuc_del_private[pango] = Set{String}()
#            pango_AAsub_WT[pango] = Set{String}()
#            pango_AAsub_WT_pos_only[pango] = Set{String}()
#            pango_AAdel_WT[pango] = Set{String}()
#            pango_AAsub_private[pango] = Set{String}()
#            pango_AAdel_private[pango] = Set{String}()
#        end
#        new_muts = filter!(
        if !isempty(new_muts)
            for xmut in new_muts
                if xmut == ""
                    push!(empty_string_pangos, pango)
#                    println("empty string in $(pango)")
                    continue
                end
                if !(xmut in nuc_mut_all_and_chr_set)
                    push!(nuc_mut_all_and_chr_set_missing, xmut)
                    pos_int = split_nuc_pos_int(xmut)
                    refnuc = string(xmut[1])
                    qrynuc = string(xmut[end])
                    nuc_mut_int_dict[xmut] = pos_int
                    ref_nuc_dict[xmut] = refnuc
                    qry_nuc_dict[xmut] = qrynuc
                    nuc_mut_int_string_dict[xmut] = string(pos_int)
                end
            end
        end
        precedessor_vec_join = join(predecessor_vec, ", ")
        if pango == "BA.3.2"
            println("BA.3.2 new_muts = $(new_muts)")
            println("BA.3.2 precedessor_vec_join = $(precedessor_vec_join)")
        end
#        println(precedessor_vec_join)
        for predecessor_j in predecessor_vec
            if (!haskey(nuc_genome_pango_dict, pango) || !haskey(pango_AAsub_private, pango)) && haskey(nuc_genome_pango_dict, predecessor_j) && length(nuc_genome_pango_dict[predecessor_j]) ≥ 28600 # && predecessor_j in pango_consensus_set && !(pango in forbidden_pangos)
                new_pango_ct += 1
                if new_pango_ct%100 == 0
                    println("################## NEW PANGO COUNT = $(new_pango_ct) #######################")
                end
                gene_AA_pango_dict[pango] = Dict{String, String}()
                for gene in gene_array
                    gene_AA_pango_dict[pango][gene] = gene_AA_pango_dict[predecessor_j][gene]
                end
####################################################################################################################################################
#                println(new_muts)
                synonymous_nuc_AA_sorted_dict, nonsynonymous_nuc_AA_sorted_dict, all_nuc_AA_sorted_dict = SIMPLER_nuc_to_AA(predecessor_j, new_muts)
####################################################################################################################################################
                sorted_nuc_muts = String[]
                if !isempty(all_nuc_AA_sorted_dict)
                    pango_nuc_sub_private[pango] = Set{String}()
                    pango_nuc_sub_WT[pango] = Set{String}()
                    pango_new_nuc_muts_pos_only[pango] = Set{String}()
                    sorted_nuc_muts = [all_nuc_AA_sorted_dict[i][1] for i in 1:length(all_nuc_AA_sorted_dict)]
                    for mut in sorted_nuc_muts
                        nuc_mut_int = nuc_mut_int_dict[mut]
                        WT_ref_nuc = string(wuhan_ref_seq[nuc_mut_int])
#                        predecessor_ref_nuc = ref_nuc_dict[mut]
                        predecessor_ref_nuc = ref_nuc_dict[mut]
#                        qry_nuc = qry_nuc_dict[mut]
                        qry_nuc = qry_nuc_dict[mut]
                        nuc_mut_WT = ""
                        if WT_ref_nuc == predecessor_ref_nuc
                            nuc_mut_WT = mut
                        else
                            nuc_mut_WT = "$(WT_ref_nuc)$(nuc_mut_int)$(qry_nuc)"
                        end
                        push!(pango_nuc_sub_private[pango], mut)
                        push!(pango_nuc_sub_WT[pango], nuc_mut_WT)
                        push!(pango_new_nuc_muts_pos_only[pango], nuc_mut_int)
                    end
                end
###################################################################
                sorted_AA_muts = String[]
                if !isempty(nonsynonymous_nuc_AA_sorted_dict)
                    pango_AAsub_private[pango] = Set{String}()
                    pango_AAsub_WT[pango] = Set{String}()
                    pango_new_AA_muts_pos_only[pango] = Set{String}()
                    sorted_AA_muts = [nonsynonymous_nuc_AA_sorted_dict[i][2] for i in 1:length(nonsynonymous_nuc_AA_sorted_dict)]
                    for mut in sorted_AA_muts
                        mutgeen = AA_gene_dict[mut]
                        AAmut_int = AA_pos_dict[mut]
                        AAmut_pos_only = AA_gene_and_pos_dict[mut]
                        refAA = ref_AA_dict[mut]
                        qryAA = qry_AA_dict[mut]
                        new_gene_AA_seq = gene_AA_pango_dict[pango][mutgeen][1:AAmut_int-1]*qryAA*gene_AA_pango_dict[pango][mutgeen][AAmut_int+1:end]
                        gene_AA_pango_dict[pango][mutgeen] = new_gene_AA_seq
#                        pango_new_AA_muts[pango] = Set{String}()
#                        pango_new_AA_muts_WT[pango] = Set{String}()
#                        pango_new_AA_muts_pos_only[pango] = Set{String}()
                        WT_ref_AA = string(gene_AA_pango_dict[predecessor_j][mutgeen][AAmut_int])
                        AA_mut_WT = ""
                        if WT_ref_AA == refAA
                            AA_mut_WT = mut
                        else
                            AA_mut_WT = "$(mutgeen):$(WT_ref_AA)$(AAmut_int)$(qryAA)"
                        end 
                        push!(pango_AAsub_private[pango], mut)
                        push!(pango_AAsub_WT[pango], AA_mut_WT)
                        push!(pango_new_AA_muts_pos_only[pango], AAmut_pos_only)
                    end
                end
##################################################################
                nuc_genome_pango_dict[pango] = nuc_genome_pango_dict[predecessor_j]
                if !isempty(new_muts)
                    for mut in new_muts
                        if mut ≠ ""
                            qryNuc = qry_nuc_dict[mut]
                            nuc_int = nuc_mut_int_dict[mut]
                            new_nuc_seq = nuc_genome_pango_dict[pango][1:nuc_int-1]*qryNuc*nuc_genome_pango_dict[pango][nuc_int+1:end]
                            nuc_genome_pango_dict[pango] = new_nuc_seq
                        end
                    end
                    push!(pango_consensus_set, pango)
                end
                break
            end
        end
    end
end
#for pango in all_fucking_pangos
#    if !haskey(nuc_genome_pango_dict, pango)
#        unaliased = pango_to_pango_unaliased_v2[pango]
#        println("No nuc_genome_pango_dict for $(pango) | $(unaliased)")
#    end
#end
#for pango in all_fucking_pangos
#    if !haskey(nuc_genome_pango_dict, pango)
#        dot_ct = count('.', unaliased)
#        if dot_ct ≥ 1
#            for i in 1:dot_ct
#                predecessor_i = pango_unaliased_predecessor_meta_dict[pango][i]
#                if haskey(nuc_genome_pango_dict, predecessor_i)
#                    nuc_genome_pango_dict[pango] = nuc_genome_pango_dict[predecessor_i]
#                    break
#                end
#            end
#        end
#    end
#end
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
#########################################################__END NEW SECTION (2026-01-03)    #################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################### 2025-11-13 | Filling pango/clade/unaliased date/ct/country dicts  Runtime = 41.57 seconds ##################################################
############################################################################################################################################################################
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
index_len = length(values(tuple_to_index))
println("tuple_to_index_len = $(index_len)"); println("index_to_tuple[1] = $(index_to_tuple[1])"); println("index_to_tuple[2000] = $(index_to_tuple[2000])")
###########################################################################################################
pango_unaliased_date_index_ct = Dict{String, Dict{Int64, Int64}}()    
#                          clade     date_index  cumulative_count
clade_date_index_cumul = Dict{String, Dict{Int, Int}}()
pango_date_index_cumul = Dict{String, Dict{Int, Int}}()
pango_unaliased_date_index_cumul = Dict{String, Dict{Int, Int}}()
#                                country       clade    date_index  cumulative_count  
#country_clade_date_index_cumul = Dict{String, Dict{String, Dict{Int, Int}}}()
country_pango_date_index_cumul = Dict{String, Dict{String, Dict{Int, Int}}}()
country_pango_unaliased_date_index_cumul = Dict{String, Dict{String, Dict{Int, Int}}}()
#
clade_total = Dict{String, Int}()
pango_total = Dict{String, Int}()
pango_unaliased_total = Dict{String, Int}()
####################################################################################################
AAmut_date_index_cumul = Dict{String, Dict{Int, Int}}()
nucmut_date_index_cumul = Dict{String, Dict{Int, Int}}()
###################################################################################################################################
for clade in chronic_clade_set
    clade_date_index_cumul[clade] = Dict{Int, Int}()
    cumulative_clade_ct = 0
    for date_index in 1:2500
        if !haskey(clade_date_index_ct[clade], date_index)
            clade_date_index_cumul[clade][date_index] = cumulative_clade_ct
        end
        if haskey(clade_date_index_ct[clade], date_index)
            cumulative_clade_ct += clade_date_index_ct[clade][date_index]
            clade_date_index_cumul[clade][date_index] = cumulative_clade_ct
        end
    end
    clade_total[clade] = cumulative_clade_ct
end
################################################################################################################################################################
missing_pangos = Set{String}()
for pango in all_fucking_pangos
    if !haskey(pango_date_index_ct, pango)
        push!(missing_pangos, pango)
    end
end     
################################################################################################################################################################
for pango in all_fucking_pangos
    if pango in missing_pangos
        continue
    end
    pango_date_index_cumul[pango] = Dict{Int, Int}()
    cumulative_pango_ct = 0
    for date_index in 1:2500
        if !haskey(pango_date_index_ct[pango], date_index)
            pango_date_index_cumul[pango][date_index] = cumulative_pango_ct
        end
        if haskey(pango_date_index_ct[pango], date_index)
            cumulative_pango_ct += pango_date_index_ct[pango][date_index]
            pango_date_index_cumul[pango][date_index] = cumulative_pango_ct
        end
    end
    pango_total[pango] = cumulative_pango_ct
end
################################################################################
for pango in keys(pango_date_index_cumul)
    unaliased = pango_to_pango_unaliased_v2[pango]
    pango_unaliased_date_index_cumul[unaliased] = Dict{Int, Int}()
    for date_index in 1:2500
        pango_unaliased_date_index_cumul[unaliased][date_index] = pango_date_index_cumul[pango][date_index]
    end
    pango_unaliased_total[unaliased] = pango_total[pango]
end
println("Done Filling Pango Date Index Cts!")
################################################################################
clade_pct_date_index = Dict{String, Dict{Float64, Int}}()
clade_pct_date_tuple = Dict{String, Dict{Float64, Tuple{Int, Int, Int}}}()
clade_pct_date_string = Dict{String, Dict{Float64, String}}()
#####
pango_pct_date_index = Dict{String, Dict{Float64, Int}}()
pango_pct_date_tuple = Dict{String, Dict{Float64, Tuple{Int, Int, Int}}}()
pango_pct_date_string = Dict{String, Dict{Float64, String}}()
#####
pango_unaliased_pct_date_index = Dict{String, Dict{Float64, Int}}()
pango_unaliased_pct_date_tuple = Dict{String, Dict{Float64, Tuple{Int, Int, Int}}}()
pango_unaliased_pct_date_string = Dict{String, Dict{Float64, String}}()
######################################################################################################################
for clade in chronic_clade_set
    clade_pct_date_index[clade] = Dict{Float64, Int}()
    clade_pct_date_tuple[clade] = Dict{Float64, Tuple{Int, Int, Int}}()
    clade_pct_date_string[clade] = Dict{Float64, String}()
    cp_total = clade_total[clade]
    pct_date_index = 0
    pct_date_tuple = nothing
#    for pct in 950:999
#        pct = pct/10
    for pct in [1.0, 50.0, 95.0, 99.0]
        for date_index in 1:2500
            cumulative_ct = clade_date_index_cumul[clade][date_index]
            if 100*cumulative_ct/cp_total ≥ pct
                clade_pct_date_index[clade][pct] = date_index
                clade_pct_date_tuple[clade][pct] = index_to_tuple[date_index]
                clade_pct_date_string[clade][pct] = index_to_date_str[date_index]
                break
            end
        end
    end
end
##########################################################
for pango in all_fucking_pangos
    if pango in missing_pangos
        continue
    end
    pango_pct_date_index[pango] = Dict{Float64, Int}()
    pango_pct_date_tuple[pango] = Dict{Float64, Tuple{Int, Int, Int}}()
    pango_pct_date_string[pango] = Dict{Float64, String}()
    cp_total = get(pango_total, pango, 0)
    pct_date_index = 0
    pct_date_tuple = nothing
#    for pct in [10.0, 25.0, 50.0, 75.0, 90.0, 95.0, 98.0, 99.0, 99.5, 99.7, 99.8, 99.9]
    for pct in [1.0, 50.0, 95.0, 99.0]
        for date_index in 1:2500
            cumulative_ct = 0
            cumulative_ct = pango_date_index_cumul[pango][date_index]
            if 100*cumulative_ct/cp_total ≥ pct
                pango_pct_date_index[pango][pct] = date_index
                pango_pct_date_tuple[pango][pct] = index_to_tuple[date_index]
                pango_pct_date_string[pango][pct] = index_to_date_str[date_index]
                break
            end
        end
    end
end
println("Done Filling pango_pct_date_index/tuple/string!")
##########################################################
for pango in keys(pango_pct_date_index)
    if pango in missing_pangos
        continue
    end
    unaliased = pango_to_pango_unaliased_v2[pango]
    pango_unaliased_pct_date_index[unaliased] = Dict{Float64, Int}()
    pango_unaliased_pct_date_tuple[unaliased] = Dict{Float64, Tuple{Int, Int, Int}}()
    pango_unaliased_pct_date_string[unaliased] = Dict{Float64, String}()
    cp_total = get(pango_total, pango, 0)
    pct_date_index = 0
    pct_date_tuple = nothing
#    for pct in 950:999
#    for pct in [10.0, 25.0, 50.0, 75.0, 90.0, 95.0, 98.0, 99.0, 99.5, 99.7, 99.8, 99.9]
    for pct in [1.0, 50.0, 95.0, 99.0]
        for date_index in 1:2500
            cumulative_ct = 0
            if pango == "B.1.1.529"
                cumulative_ct = pango_date_index_cumul["BA.2"][date_index]
            elseif pango == "LF.3.1"
                cumulative_ct = pango_date_index_cumul["LF.3"][date_index]
            else
                cumulative_ct = pango_date_index_cumul[pango][date_index]
            end
            if 100*cumulative_ct/cp_total ≥ pct
                pango_unaliased_pct_date_index[unaliased][pct] = date_index
                pango_unaliased_pct_date_tuple[unaliased][pct] = index_to_tuple[date_index]
                pango_unaliased_pct_date_string[unaliased][pct] = index_to_date_str[date_index]
                break
            end
        end
    end
end
println("Done Filling pango_unaliased_pct_date_index/tuple/string!")
###########################################################################################################################################################################
###########################################################################################################################################################################
all_chr_seqs_pangos = Dict{String, Int}()
all_chr_seqs_inherited = Dict{String, Int}()
all_chr_seqs_inherited_pos_only = Dict{String, Int}()
for seq in all_unique_chr_seqs
    pango = seq_pango[seq]
    all_chr_seqs_pangos[pango] = get(all_chr_seqs_pangos, pango, 0) + 1
    if !haskey(pango_AAsub_WT, pango)
        if haskey(pango_predecessor_meta_dict, pango)
            if haskey(pango_predecessor_meta_dict[pango], 2)
                pango = pango_predecessor_meta_dict[pango][2]
                if !haskey(pango_AAsub_WT, pango)
                    if haskey(pango_predecessor_meta_dict, pango)
                        if haskey(pango_predecessor_meta_dict[pango], 3)
                            pango = pango_predecessor_meta_dict[pango][3]
                        end
                    end
                end
            end
        end
    end
    if pango == "B.1.1.529"
        pango_AAsub_WT[pango] = union(pango_AAsub_WT["BA.1"], pango_AAsub_WT["BA.2"])
        for mut in pango_AAsub_WT[pango]
            all_chr_seqs_inherited[mut] = get(all_chr_seqs_inherited, mut, 0) + 1
        end
    else
        for mut in pango_AAsub_WT[pango]
            all_chr_seqs_inherited[mut] = get(all_chr_seqs_inherited, mut, 0) + 1
        end
    end
end
println("Done Filling all_chr_seqs_pangos, all_chr_seqs_inherited, pango_AAsub_WT (leftovers)!")
############################################################################################################################################################################
############################################################################################################################################################################
seq_syn_nucs = Dict{String, Vector{String}}()
seq_noncoding_nucs = Dict{String, Vector{String}}()
for (seq, nuc_set) in seq_nuc_muts
    pango = seq_pango[seq]
#    if !haskey(nuc_genome_pango_dict, pango)
#        println("No nuc_genome_pango_dict key, $(pango)")
#    end
    synonymous_nucmuts, noncoding_nucmuts = SIMPLER_syn_noncoding_nuc(pango, nuc_set)
    seq_syn_nucs[seq] = synonymous_nucmuts
    seq_noncoding_nucs[seq] = noncoding_nucmuts
end
println("Done Filling seq_syn_nucs, seq_noncoding_nucs!")
############################################################################################################################################################################
nuc_genome_pango_dict["B.1.1.273"] = nuc_genome_pango_dict["B.1.1"]
nuc_genome_pango_dict["B.1.1.53"] = nuc_genome_pango_dict["B.1.1"]
############################################################################################################################################################################
finish = time() - start
finish_rd = round(digits=2, finish)
println("nuc_genome_pango_dict length = $(length(nuc_genome_pango_dict))")
println("Length all_fucking_pangos = $(length(all_fucking_pangos))")
print("\n"^1)
println("Total time to finish = $(finish_rd)"); print("\n"^1)
println("Finished!")
print("\n"^1)
temp_nuc_sort_key(n) = (length(n), n[2:end-1])
nuc_mut_all_and_chr_set_missing_sort = sort(collect(nuc_mut_all_and_chr_set_missing), by = x -> temp_nuc_sort_key(x))
nuc_mut_all_and_chr_set_missing_sort_join = join(nuc_mut_all_and_chr_set_missing_sort, ", ")
missing_AA_dict_keys_sort = sort(collect(missing_AA_dict_keys), by = x -> AA_gene_sortKey_2(x))
missing_AA_dict_keys_sort_join = join(missing_AA_dict_keys_sort, ", ")
empty_string_pangos_sort = sort(collect(empty_string_pangos))
empty_string_pangos_sort_join = join(empty_string_pangos_sort, ", ")
open("missing_nuc_dict_keys_$(date_now).txt", "w") do g
    println(g, "nuc_mut_all_and_chr_set_missing_sort_join = $(nuc_mut_all_and_chr_set_missing_sort_join)")
    println("nuc_mut_all_and_chr_set_missing_sort_join = $(nuc_mut_all_and_chr_set_missing_sort_join)")
    print("\n"^1)
    print(g, "\n"^1)
    println(g, "missing_AA_dict_keys_sort_join = $(missing_AA_dict_keys_sort_join)")
    println("missing_AA_dict_keys_sort_join = $(missing_AA_dict_keys_sort_join)")
    print("\n"^1)
    print(g, "\n"^1)
    println(g, "empty_string_pangos_sort_join = $(empty_string_pangos_sort_join)")
    println("empty_string_pangos_sort_join = $(empty_string_pangos_sort_join)")
end
print("\n"^1)
############################################################################################################################################################################
############################################################################################################################################################################

2026_02_07_922PM
nuc_genome_pango_dict length = 4730
2026_02_07__922PM
Number of Different Pango Lineages in Chronics = 839
Number of Different Pango_Unaliased Lineages in Chronics = 839
Number of Different Clades in Chronics = 49

XBB.1.5.33 = XBB.1.5.33
AY.127 = B.1.617.2.127
B.1.469 = B.1.469
BA.5.2.26 = B.1.1.529.5.2.26
MC.36 = B.1.1.529.2.86.1.1.11.1.3.1.1.36
XBB.1.5.20 = XBB.1.5.20
BA.5.2.43 = B.1.1.529.5.2.43
BQ.1.1.23 = B.1.1.529.5.3.1.1.1.1.1.1.23
XAB = XAB
XE = XE
JN.1.36.1 = B.1.1.529.2.86.1.1.36.1
KB.2 = XBB.1.9.2.5.1.8.2
BM.4 = B.1.1.529.2.75.3.4
AY.117 = B.1.617.2.117
length(keys(pango_to_pango_unaliased_v2) = 3392
2026_02_07__922PM
2026_02_07__922PM
Done Making Pango Consensus Sequences! (Part 1)

Total Missing Keys for AA_pos_only_gene_and_pos_dict = 1
missing_keys_for_WT_join = ||

OG_tree_pangos_set_len = 4432
NEW_tree_pangos_set_len = 4372
bad_pangos length = 60
AY.43.8_18086, B.1.1.529_dropout, B.1.1.7_dropout, B.1.160.16_13923, BA.1_22877_22878, BA.1_dropout, BA.2.

In [70]:
### Minimal DQ print——Run this before DQ Fx (*WAS* to be pasted into fx but was disaster)
# ********** Make sure  DQ_mut_thresh  &  date_pct_DQ_thresh  are same as used in main DQ Fx *********
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
abs_min_mut_thresh = 8
DQ_mut_thresh = 10
date_pct_DQ_thresh = 95
EPCI_qc_str = "$(abs_min_mut_thresh)_$(DQ_mut_thresh)_$(date_pct_DQ_thresh)"
################################################
date_disqualified_seqs = Set{String}()
AAmut_disqualified_seqs = Set{String}()
double_disqualified = Set{String}()
abs_mut_disqualified = Set{String}()
all_DQed_seqs = Set{String}()
################################################
all_unique_chr_seqs_len = 0
date_disqualified_seqs_len = 0
AAmut_disqualified_seqs_len = 0
double_disqualified_len = 0
abs_mut_disqualified_len = 0
all_DQ_total = 0
################################################
exempt_pangos = Set(["A", "B", "B.1", "B.1.1"])
open("unqualified_chronic_seqs_commas_$(date_pct_DQ_thresh)pct_$(date_now).txt", "w") do g
    for seq in all_unique_chr_seqs  # all_unique_chr_seqs   # all_qualifying_seqs_set
        if length(seq_AA_muts[seq]) + length(seq_AA_dels[seq]) < DQ_mut_thresh
            push!(AAmut_disqualified_seqs, seq)
        end
        if length(seq_AA_muts[seq]) + length(seq_AA_dels[seq]) < abs_min_mut_thresh
            push!(abs_mut_disqualified, seq)
        end
        dq_date_index = seq_date_index[seq]
        dq_date = seq_collection_date[seq]
        dq_pango = seq_pango[seq]
    #    dq_date_index = date_str_to_index[dq_date]
        if dq_pango == "JN.1.20" || dq_pango == "JN.1.16"
            dq_pango = "JN.1"
        elseif dq_pango == "BA.2.3.21"
            dq_pango = "BA.2.3"
        elseif dq_pango == "R.1"
            dq_pango = "B.1.1"
        elseif dq_pango == "BA.2.1"
            dq_pango = "BA.2"
        elseif dq_pango == "BA.1.1.2"
            dq_pango = "BA.1.1"
        elseif dq_pango == "B.1.1.529"
            dq_pango = "BA.2"
        elseif dq_pango == "BF.22"
            dq_pango = "BA.5.2.1"
        elseif dq_pango == "MC.7"
            dq_pango = "KP.3.1.1"
        elseif dq_pango == "LB.1.3.5"
            dq_pango = "LB.1.3"
        elseif dq_pango == "XBK"
            dq_pango = "BA.2"
        end
        if !haskey(pango_pct_date_index[dq_pango], date_pct_DQ_thresh)
            println("No $(date_pct_DQ_thresh) key for $(dq_pango)")
        end
        date_limit = pango_pct_date_index[dq_pango][date_pct_DQ_thresh]
        if dq_date_index < date_limit && !(dq_pango in exempt_pangos)
            push!(date_disqualified_seqs, seq)
            missed = date_limit - dq_date_index
            privAA = length(seq_AA_muts[seq]) + length(seq_AA_dels[seq])
    #        if privAA < 12
    #        if missed > 75
            missed_pad = lpad(missed, 3)
            date_limit_pad = lpad(date_limit, 4)
            index_pad = lpad(dq_date_index, 4)
            dq_pango_pad = lpad(dq_pango, 11)
            privAA_pad = lpad(privAA, 2)
    #        println("date_limit = $(date_limit_pad) | index = $(index_pad) | missed by = $(missed_pad) days | dq_pango = $(dq_pango_pad) | privAAmuts = $(privAA_pad) | $(seq)")
        end
    end
    print("\n"^1)
    for seq in date_disqualified_seqs
        if seq in AAmut_disqualified_seqs
            push!(double_disqualified, seq)
        end
    end
    all_unique_chr_seqs_len = length(all_unique_chr_seqs)
    date_disqualified_seqs_len = length(date_disqualified_seqs)
    AAmut_disqualified_seqs_len = length(AAmut_disqualified_seqs)
    double_disqualified_len = length(double_disqualified)
    abs_mut_disqualified_len = length(abs_mut_disqualified)
    println("Total Number of Date Disqualified Sequences = $(date_disqualified_seqs_len)")
    println("Total Number of AA-mut Disqualified Sequences = $(AAmut_disqualified_seqs_len)")
    println("Total Number of Double DQ'ed Sequences = $(double_disqualified_len)"); print("\n"^2)
    println("Total Number of Abs Min DQ'ed Sequences = $(abs_mut_disqualified_len)")
    print("\n"^2)
    all_DQed_seqs = union(double_disqualified, abs_mut_disqualified)
    all_DQ_total = length(all_DQed_seqs)
    println("Overall Total Sequences DQ'ed = $(all_DQ_total)")
    print("\n"^1)
    total_included_seqs = all_unique_chr_seqs_len - all_DQ_total
    println("Total Included Sequences = $(total_included_seqs)")
    print("\n"^1)
    double_disqualified_sort = sort(collect(double_disqualified), by = x -> (length(x), x))
    all_DQed_seqs = sort(collect(all_DQed_seqs), by = x -> (length(x), x))
    

    println(g, "Total Number of Date Disqualified Sequences = $(date_disqualified_seqs_len)")
    println(g, "Total Number of AA-mut Disqualified Sequences = $(AAmut_disqualified_seqs_len)")
    println(g, "Total Number of Double DQ'ed Sequences = $(double_disqualified_len)"); println(g)
    println(g, "########### Disqualified by BOTH, i.e. True DQs ($(double_disqualified_len) Total) ###########")
    for seq in double_disqualified_sort
        print(g, seq, ", ")
        print(seq, ", ")
    end
    println(g); println()
    open("disqualified_chronic_seqs_lines_$(date_pct_DQ_thresh)pct_$(date_now).txt", "w") do g
        println(g, "Total Number of Date Disqualified Sequences = $(date_disqualified_seqs_len)")
        println(g, "Total Number of AA-mut Disqualified Sequences = $(AAmut_disqualified_seqs_len)")
        println(g, "Total Number of Double DQ'ed Sequences = $(double_disqualified_len)"); println(g)
        println(g, "########### Disqualified by BOTH, i.e. True DQs ($(double_disqualified_len) Total) ###########")
        for seq in double_disqualified_sort
            println(g, seq)
    #        println(seq)
        end; println(g, "\n"^4)
    end
end
print("\n"^1)
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
print("\n"^1)
println("Finished!")
print("\n"^1)
##############################################################

2026_02_21_151PM

Total Number of Date Disqualified Sequences = 301
Total Number of AA-mut Disqualified Sequences = 2012
Total Number of Double DQ'ed Sequences = 278


Total Number of Abs Min DQ'ed Sequences = 1176


Overall Total Sequences DQ'ed = 1225

Total Included Sequences = 2258

EPI_ISL_517000, EPI_ISL_1477334, EPI_ISL_1626185, EPI_ISL_1853568, EPI_ISL_2285732, EPI_ISL_2566726, EPI_ISL_2790083, EPI_ISL_2858161, EPI_ISL_4193135, EPI_ISL_4203869, EPI_ISL_6113246, EPI_ISL_6262536, EPI_ISL_6281381, EPI_ISL_6628662, EPI_ISL_6938691, EPI_ISL_7175748, EPI_ISL_8057418, EPI_ISL_8627379, EPI_ISL_8828662, EPI_ISL_10251304, EPI_ISL_10590760, EPI_ISL_10717525, EPI_ISL_11242266, EPI_ISL_11248919, EPI_ISL_11349763, EPI_ISL_11424263, EPI_ISL_11789090, EPI_ISL_12021469, EPI_ISL_12284821, EPI_ISL_12401928, EPI_ISL_12467081, EPI_ISL_12467157, EPI_ISL_12602903, EPI_ISL_12628520, EPI_ISL_12652423, EPI_ISL_12754976, EPI_ISL_12809016, EPI_ISL_12809684, EPI_ISL_12879252, EPI_ISL_13028133, EPI_ISL_1308

In [71]:
### Fx: Main Function, Date_pct_DQed version ####################################
### To run this, you first have to run the normal Main chronic function, then load all the chronic info by running chronic_load_dicts2. 
### Then, run the cell directly above——making sure date_pct_DQ_thresh & mut_thresh are same as in Fx below
### Then you have to run the cell with Making pango_to_pango_unaliased_v2 & pango consensus Dicts
### And THEN you run this function.
#################################################################################################################################################
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
date = Dates.format(today(), "yyyy_mm_dd")
function chr_2025_11_15_date_DQ(ndjson::String, ndjson_name::String, date_pct_DQ_thresh::Int, rep_thresh::Int, revs_thresh::Int, print_ct_thresh::Int, DQ_mut_thresh::Int, abs_min_mut_thresh::Int, HQCS_qc_string::String)

    adj_score_ct_div = 8
    
    date = Dates.format(today(), "yyyy_mm_dd")
    save_folder = "$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_string)"
    mkdir(save_folder)
    
    all_qualifying_seqs_ct = 0
    total_chr_AA_subs = 0
    total_nuc_revs_seq = Dict{Int, Vector{String}}()
    for i in 0:35
        total_nuc_revs_seq[i] = Vector{String}()
    end
##                       EPI_ISL  #_of_total_nuc_reversions
    seq_nuc_total_revs = Dict{String, Int}()
#########################
##               #_of_total_AA_reversions,  all_seqs_with_that_many_AA_reversions
    total_AA_revs_seq = Dict{Int, Vector{String}}()
    for i in 0:35
        total_AA_revs_seq[i] = Vector{String}()
    end
##                       EPI_ISL  #_of_total_AA_reversions
    seq_AA_total_revs = Dict{String, Int}()
####################################################################################################################################
    other_exceptions = Set(["EPI_ISL_539541", "EPI_ISL_539542", "EPI_ISL_593478", "EPI_ISL_593479", "EPI_ISL_602912", "EPI_ISL_2397307", "EPI_ISL_2397308", "EPI_ISL_2397309", "EPI_ISL_2397310", "EPI_ISL_2397311", "EPI_ISL_6698637", "EPI_ISL_8725399", "EPI_ISL_8725400", "EPI_ISL_8725401", "EPI_ISL_8725402", "EPI_ISL_8725403", "EPI_ISL_13011225", "EPI_ISL_15385232", "EPI_ISL_17284796", "EPI_ISL_19415272"])
####################################################################################################################################
    
#################### Add This one to force_groups on the next EPCI update: Set(["EPI_ISL_15492887", "EPI_ISL_13244707", "EPI_ISL_11750262"]),  ########################
    ### Add These to force_groups on next EPCI update: 
#            Set(["EPI_ISL_15492887", "EPI_ISL_13244707", "EPI_ISL_11750262"]), 
#            Set(["EPI_ISL_20040623", "EPI_ISL_20040387", "EPI_ISL_19948061", "EPI_ISL_19946695", "EPI_ISL_20030503", "EPI_ISL_20027365", "EPI_ISL_20027953"]),  
    
    force_groups_prelim = Set([
            Set(["EPI_ISL_17437940", "EPI_ISL_20016121"]),
            Set(["EPI_ISL_13051970", "EPI_ISL_13052204", "EPI_ISL_13051740"]),
            Set(["EPI_ISL_17664370", "EPI_ISL_17789808"]),
            Set(["EPI_ISL_19267631", "EPI_ISL_19300385", "EPI_ISL_19415183", "EPI_ISL_19415333"]),
            Set(["EPI_ISL_7473129", "EPI_ISL_7473130", "EPI_ISL_7473131", "EPI_ISL_7473132", "EPI_ISL_7473133", "EPI_ISL_7473134", "EPI_ISL_7473135", "EPI_ISL_7473136", "EPI_ISL_7473137", "EPI_ISL_7473138", "EPI_ISL_7473139", "EPI_ISL_7473140", "EPI_ISL_7473141", "EPI_ISL_7473142", "EPI_ISL_7473143", "EPI_ISL_7473144"]),
            Set(["EPI_ISL_14922929", "EPI_ISL_15177334"]),
            Set(["EPI_ISL_13830121", "EPI_ISL_13830175", "EPI_ISL_14152696", "EPI_ISL_14601182", "EPI_ISL_14835057", "EPI_ISL_14846826", "EPI_ISL_14984311", "EPI_ISL_15076071", "EPI_ISL_15259998", "EPI_ISL_15415649", "EPI_ISL_15721045", "EPI_ISL_15866887", "EPI_ISL_16706080"]), 
            Set(["EPI_ISL_602912", "EPI_ISL_2397307", "EPI_ISL_2397308", "EPI_ISL_2397309", "EPI_ISL_2397310", "EPI_ISL_2397311", "EPI_ISL_2397312", "EPI_ISL_2397313", "EPI_ISL_14666760", "EPI_ISL_15292331", "EPI_ISL_15541751", "EPI_ISL_18127203", "EPI_ISL_18876158", "EPI_ISL_18876159", "EPI_ISL_18876160", "EPI_ISL_18876161", "EPI_ISL_18876171", "EPI_ISL_18876178"]),
            Set(["EPI_ISL_19032268", "EPI_ISL_19032269", "EPI_ISL_19032270", "EPI_ISL_19032271", "EPI_ISL_19032272", "EPI_ISL_19032273", "EPI_ISL_19032274", "EPI_ISL_19032275", "EPI_ISL_19032276", "EPI_ISL_19032277", "EPI_ISL_19032278", "EPI_ISL_19032279", "EPI_ISL_19032280", "EPI_ISL_19032281", "EPI_ISL_19032282", "EPI_ISL_19032283", "EPI_ISL_19032284", "EPI_ISL_20231309", "EPI_ISL_20231310", "EPI_ISL_20231311", "EPI_ISL_20231312", "EPI_ISL_20231313", "EPI_ISL_20231314", "EPI_ISL_20231315", "EPI_ISL_20231316"]),
            Set(["EPI_ISL_13299122", "EPI_ISL_13203425", "EPI_ISL_13299118", "EPI_ISL_13299108", "EPI_ISL_13301357", "EPI_ISL_13301368"]), 
            Set(["EPI_ISL_11356267", "EPI_ISL_12063600", "EPI_ISL_11356269", "EPI_ISL_12063601", "EPI_ISL_12063602", "EPI_ISL_12063598", "EPI_ISL_12808264", "EPI_ISL_12430022", "EPI_ISL_13354243", "EPI_ISL_12656485", "EPI_ISL_13172328", "EPI_ISL_13172329", "EPI_ISL_14243503", "EPI_ISL_14243471", "EPI_ISL_15096672", "EPI_ISL_15093817", "EPI_ISL_15093818"]),
            Set(["EPI_ISL_13893481", "EPI_ISL_14700285", "EPI_ISL_15109913", "EPI_ISL_15494897", "EPI_ISL_15837686"]),
            Set(["EPI_ISL_6397259", "EPI_ISL_8675368"]),
            Set(["EPI_ISL_13002811", "EPI_ISL_13002153", "EPI_ISL_13002802"]),
            Set(["EPI_ISL_19044119", "EPI_ISL_19109579", "EPI_ISL_19141756", "EPI_ISL_19557993", "EPI_ISL_19588050", "EPI_ISL_19613618"]), 
            Set(["EPI_ISL_16077353", "EPI_ISL_16181797", "EPI_ISL_16287690", "EPI_ISL_16528903", "EPI_ISL_17252934", "EPI_ISL_17387122", "EPI_ISL_17391460", "EPI_ISL_17441169", "EPI_ISL_17441208", "EPI_ISL_17612398", "EPI_ISL_17837432", "EPI_ISL_18051918", "EPI_ISL_18162567"]),
            Set(["EPI_ISL_19116728", "EPI_ISL_15405688"]),
            Set(["EPI_ISL_2228938", "EPI_ISL_2983871", "EPI_ISL_5692576", "EPI_ISL_5692832", "EPI_ISL_5693164"]),
            Set(["EPI_ISL_15424276", "EPI_ISL_16045410", "EPI_ISL_16045424"]),
            Set(["EPI_ISL_17976105", "EPI_ISL_17976114", "EPI_ISL_18044759", "EPI_ISL_18282077"]),
            Set(["EPI_ISL_6757093", "EPI_ISL_9869512", "EPI_ISL_12646116"]),
            Set(["EPI_ISL_14773345", "EPI_ISL_14995884", "EPI_ISL_15191475", "EPI_ISL_15349852", "EPI_ISL_18998583"]),
            Set(["EPI_ISL_15331994", "EPI_ISL_16231330", "EPI_ISL_16231873", "EPI_ISL_16479826"]),
            Set(["EPI_ISL_12501519", "EPI_ISL_18930681", "EPI_ISL_18966035", "EPI_ISL_19182893", "EPI_ISL_19217899", "EPI_ISL_19269083", "EPI_ISL_19348673"]),
            Set(["EPI_ISL_17190813", "EPI_ISL_17535979", "EPI_ISL_18147456"]),
            Set(["EPI_ISL_11621883", "EPI_ISL_13131091", "EPI_ISL_14561487", "EPI_ISL_14710834"]),
            Set(["EPI_ISL_14616543", "EPI_ISL_14616443", "EPI_ISL_14707948", "EPI_ISL_14766433"]),
            Set(["EPI_ISL_14260215", "EPI_ISL_14817985", "EPI_ISL_15507616"]),
            Set(["EPI_ISL_12535815", "EPI_ISL_12741485", "EPI_ISL_12742222", "EPI_ISL_13073691", "EPI_ISL_13605240", "EPI_ISL_14959923", "EPI_ISL_15169791", "EPI_ISL_15171102"]),
            Set(["EPI_ISL_12148503", "EPI_ISL_12146396", "EPI_ISL_12147521", "EPI_ISL_12146579", "EPI_ISL_14439649"]),
            Set(["EPI_ISL_10876034", "EPI_ISL_11295642"]), 
            Set(["EPI_ISL_13358809", "EPI_ISL_15887656"]),
            Set(["EPI_ISL_15420631", "EPI_ISL_16413336"]),
            Set(["EPI_ISL_19300400", "EPI_ISL_18990023", "EPI_ISL_19415272", "EPI_ISL_19415273"]), 
            Set(["EPI_ISL_19539586", "EPI_ISL_19539602"]), 
            Set(["EPI_ISL_17817657", "EPI_ISL_18042110"]), 
            Set(["EPI_ISL_1470396", "EPI_ISL_7711813", "EPI_ISL_12001179", "EPI_ISL_12001180"]), 
            Set(["EPI_ISL_19193606", "EPI_ISL_19193617", "EPI_ISL_19032056"]), 
            Set(["EPI_ISL_15436494", "EPI_ISL_16967082", "EPI_ISL_16967083", "EPI_ISL_17695348", "EPI_ISL_18345777", "EPI_ISL_18563181", "EPI_ISL_18814273", "EPI_ISL_18814272", "EPI_ISL_18814271", "EPI_ISL_19271166", "EPI_ISL_19271182"]), 
            Set(["EPI_ISL_2731960", "EPI_ISL_1048105", "EPI_ISL_1048102", "EPI_ISL_2732319", "EPI_ISL_1048104"]),
            Set(["EPI_ISL_6698637", "EPI_ISL_7205152", "EPI_ISL_7876524", "EPI_ISL_17276098", "EPI_ISL_15473942", "EPI_ISL_17658574"]),
            Set(["EPI_ISL_12639714", "EPI_ISL_12639788", "EPI_ISL_12926955", "EPI_ISL_13788917", "EPI_ISL_14292645", "EPI_ISL_14292615", "EPI_ISL_14292727", "EPI_ISL_14292796"]),
            Set(["EPI_ISL_17587657", "EPI_ISL_13830195", "EPI_ISL_14487315", "EPI_ISL_14847727"]),
            Set(["EPI_ISL_2281463", "EPI_ISL_979349"]),
            Set(["EPI_ISL_16428100", "EPI_ISL_16428101", "EPI_ISL_16428102", "EPI_ISL_16391752"]),
            Set(["EPI_ISL_13986497", "EPI_ISL_14175103", "EPI_ISL_18030395", "EPI_ISL_18876167", "EPI_ISL_12268492", "EPI_ISL_14599778", "EPI_ISL_14599779", "EPI_ISL_14599780", "EPI_ISL_17284794", "EPI_ISL_17284795", "EPI_ISL_17284796", "EPI_ISL_18876174", "EPI_ISL_18876176"]),
            Set(["EPI_ISL_18947086", "EPI_ISL_17276962", "EPI_ISL_17510495", "EPI_ISL_17686694", "EPI_ISL_17686736", "EPI_ISL_18029979", "EPI_ISL_18306254", "EPI_ISL_18947086", "EPI_ISL_18128931"]),
            Set(["EPI_ISL_15338015", "EPI_ISL_15463120", "EPI_ISL_16102479", "EPI_ISL_16102480", "EPI_ISL_16607452", "EPI_ISL_16688713"]),
            Set(["EPI_ISL_14665394", "EPI_ISL_15385232", "EPI_ISL_17201694", "EPI_ISL_16443874", "EPI_ISL_17780724"]),
            Set(["EPI_ISL_18039728", "EPI_ISL_18401313"]), 
            Set(["EPI_ISL_14901672", "EPI_ISL_14937654", "EPI_ISL_14694460"]),
            Set(["EPI_ISL_15911160", "EPI_ISL_16669829"]),
            Set(["EPI_ISL_14462783", "EPI_ISL_15215446", "EPI_ISL_16012424", "EPI_ISL_16863260"]),
            Set(["EPI_ISL_19030129", "EPI_ISL_19234843", "EPI_ISL_19706712"]),
            Set(["EPI_ISL_13011225", "EPI_ISL_13011226", "EPI_ISL_13358893", "EPI_ISL_13358894"]),
            Set(["EPI_ISL_18072343", "EPI_ISL_18225473"]),
            Set(["EPI_ISL_6227208", "EPI_ISL_11816904"]),
            Set(["EPI_ISL_18276415", "EPI_ISL_18414808"]),
            Set(["EPI_ISL_18326806", "EPI_ISL_18326807", "EPI_ISL_18525840"]),
            Set(["EPI_ISL_17669306", "EPI_ISL_19896098", "EPI_ISL_20063868", "EPI_ISL_17669304"]),
            Set(["EPI_ISL_3813731", "EPI_ISL_15568780", "EPI_ISL_15557544"]),
            Set(["EPI_ISL_10712909", "EPI_ISL_11296415", "EPI_ISL_12488441"]),
            Set(["EPI_ISL_528438", "EPI_ISL_686537", "EPI_ISL_2355027", "EPI_ISL_4114033"]),
            Set(["EPI_ISL_6977941", "EPI_ISL_6976497"]),
            Set(["EPI_ISL_539541", "EPI_ISL_539542", "EPI_ISL_8725399", "EPI_ISL_8725400", "EPI_ISL_8725401", "EPI_ISL_8725402", "EPI_ISL_8725403", "EPI_ISL_8725404", "EPI_ISL_8725405", "EPI_ISL_8725406", "EPI_ISL_8725407", "EPI_ISL_8725408", "EPI_ISL_8725409"]),
            Set(["EPI_ISL_593478", "EPI_ISL_593479", "EPI_ISL_593480", "EPI_ISL_593553", "EPI_ISL_593554", "EPI_ISL_593555", "EPI_ISL_593556", "EPI_ISL_593557", "EPI_ISL_593558"]),
            Set(["EPI_ISL_14574572", "EPI_ISL_14832276"]),
            Set(["EPI_ISL_18224410", "EPI_ISL_19173640", "EPI_ISL_19184858", "EPI_ISL_19192806", "EPI_ISL_19850000"]),
            Set(["EPI_ISL_19508878", "EPI_ISL_20066350", "EPI_ISL_20066351", "EPI_ISL_20066352", "EPI_ISL_20066353", "EPI_ISL_20066354", "EPI_ISL_20066355", "EPI_ISL_20066356", "EPI_ISL_20066357", "EPI_ISL_20066358", "EPI_ISL_20066359", "EPI_ISL_20066360"]),
            Set(["EPI_ISL_19912125", "EPI_ISL_19791384"]),
            Set(["EPI_ISL_20000997", "EPI_ISL_19875482", "EPI_ISL_20088904"]),
            Set(["EPI_ISL_15598104", "EPI_ISL_15667047", "EPI_ISL_16706498", "EPI_ISL_17022081", "EPI_ISL_17152522", "EPI_ISL_17370155", "EPI_ISL_18311951"]), 
            Set(["EPI_ISL_16119498", "EPI_ISL_16119508", "EPI_ISL_13830194", "EPI_ISL_16119512", "EPI_ISL_16119517", "EPI_ISL_13984460", "EPI_ISL_16119519", "EPI_ISL_15496641"]),
            Set(["EPI_ISL_11742572", "EPI_ISL_12701772", "EPI_ISL_12701858", "EPI_ISL_13129387", "EPI_ISL_13242155", "EPI_ISL_13470158", "EPI_ISL_13617475", "EPI_ISL_13740674", "EPI_ISL_14019093", "EPI_ISL_14019109"]), 
            Set(["EPI_ISL_11897546", "EPI_ISL_12422504", "EPI_ISL_12896994", "EPI_ISL_12961699"]) ])
    for fgroup in force_groups_prelim
        for seq in fgroup
            push!(other_exceptions, seq)
        end
    end
#####################################################################################################################################
    excluded_seqs = stringlist_to_strings_set("EPI_ISL_16969757, EPI_ISL_16969756, EPI_ISL_16583782, EPI_ISL_18059726, EPI_ISL_15865257, EPI_ISL_13215742, EPI_ISL_12293549, EPI_ISL_15945504, EPI_ISL_19227219, EPI_ISL_14203206, EPI_ISL_13338249, EPI_ISL_13358962, EPI_ISL_13464004, EPI_ISL_13716400, EPI_ISL_13729045, EPI_ISL_14599772, EPI_ISL_19239713, EPI_ISL_19506293, EPI_ISL_19506391")
    legends = Set(["EPI_ISL_17344660", "EPI_ISL_17997249", "EPI_ISL_19081416", "EPI_ISL_19081419", "EPI_ISL_19081422", "EPI_ISL_19081425", "EPI_ISL_19108198", "EPI_ISL_19108199", "EPI_ISL_19108200", "EPI_ISL_19108208", "EPI_ISL_19415272", "EPI_ISL_19415273"])
## To put into other_exceptions next time: "EPI_ISL_508674", "EPI_ISL_949208", "EPI_ISL_8725398",  
    AA_position(m) = split(m, ":")[2][2:end-1]
    date = Dates.format(today(), "yyyy_mm_dd")
    ref_seq = "..."
    
    excluded_pos = BitSet([1:60..., 76:78..., 686:694..., 2453, 3241, 5515, 7926, 11283:11295..., 14960, 15521, 19209:19210..., 19212, 19214, 19217, 21595, 21766, 21772, 21987, 21995:21996..., 22882, 24130, 27291, 28360:28373..., 29632, 29732:29736..., 29757, 29759:29761..., 29830:29900...])
    excluded_AA = Set(["ORF1a:F143M", "ORF1a:S142T", "ORF1a:I2107M", "ORF1a:N2111K", "ORF1a:S2114T", "ORF1a:S2151R", "ORF1a:N2155H", "ORF1a:T2152P", "ORF1a:T2153P", "ORF1a:T2154P", "ORF1a:T2158P", "ORF1b:N498I", "ORF1b:F685Y", "ORF1b:D1914E", "ORF1b:R1915G", "ORF1b:Y1916C", "ORF1b:P1917H", "S:S27A", "S:K113N", "S:Q115H", "S:N122K", "S:D142G", "S:Y145D", "S:S408R", "S:N417K", "S:K440N", "S:I794N"])
    reversion_exceptions = Set(["ORF1a:R135S", "ORF1a:I842T", "ORF1a:I3255T", "ORF1b:H264Y", "ORF1b:L314P", "ORF1b:V1566I", "S:N405D", "S:I19T", "S:F371S", "S:F375S", "S:A376T", "S:T444K", "S:S446G", "S:S455L", "S:K478T", "S:A484E", "S:V486F", "S:P486F", "S:P486L", "S:S490F", "S:R493Q", "S:S496G", "S:H505Y", "S:K764N", "E:I9T", "ORF7a:Y47H", "ORF7a:A82V", "N:L13P", "ORF9b:S10P", "ORF9b:G16D"])
    
    counted_seq_total_private_AA_subs = Dict{String, Int}()
    total_private_AA_subs_all_counted_chronics::Int = 0
    total_counted_sequences::Int = 0
    
    nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
    leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)

    date_nuc_mut_ct = Dict{Int, Dict{String, Int}}()
    date_nuc_mut_ct_no_dels = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct_no_dels = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct_pos_only_no_dels = Dict{Int, Dict{String, Int}}()

    seq_ct_by_year = Dict{Int, Int}(i=>0 for i in 2020:2027)
    seq_ct_by_year[0] = 0
    seq_ct_by_year_month = Dict{Tuple{Int, Int}, Int}()
    for i in 2020:2027
        for j in 0:12
            seq_ct_by_year_month[(i, j)] = 0
        end
    end
    seq_ct_by_year_month[(0, 0)] = 0
    seq_ct_by_year_month_day = Dict{Tuple{Int, Int, Int}, Int}()
    for i in 2020:2027
        if i%4 == 0
            for j in 0:12
                for k in 0:leap_month_day_dict[j]
                    seq_ct_by_year_month_day[(i, j, k)] = 0
                end
            end
        else
            for j in 0:12
                for k in 0:nonleap_month_day_dict[j]
                    seq_ct_by_year_month_day[(i, j, k)] = 0
                end
            end
        end
    end
    seq_ct_by_year_month_day[(0, 0, 0)] = 0
####################################################################################################################################
    domain_mut_density = Dict{String, Float64}()
    gene_mut_density = Dict{String, Float64}()
####################################################################################################################################
##                              seq          gene    AAs    site
#    seq_AA_insertions = Dict{String, Vector{Tuple{String, String, Int}}}()
    seq_AA_insertions_WT = Dict{String, Vector{Tuple{String, String, Int}}}()
##                              seq           nucs   site
#    seq_nuc_insertions = Dict{String, Vector{Tuple{String, Int}}}()
    seq_nuc_insertions_WT = Dict{String, Vector{Tuple{String, Int}}}()
####################################################################################################################################
    nuc_muts_seq = Dict{String, Set{String}}()
    nuc_dels_seq = Dict{String, Set{String}}()
    nuc_muts_seq_no_dels = Dict{String, Set{String}}()
    
    seq_nuc_muts = Dict{String, Set{String}}()
    seq_nuc_del_ranges = Dict{String, Set{String}}()
    seq_nuc_del_ranges_ct = Dict{String, Int}()
    seq_nuc_muts_no_dels = Dict{String, Set{String}}()
    
    seq_nuc_dropout = Dict{String, Set{Int}}()
    seq_mixed_nucs = Dict{String, Set{String}}()
####################################################################################################################################
    AA_muts_seq = Dict{String, Set{String}}()
    AA_dels_seq = Dict{String, Set{String}}()
    AA_muts_seq_no_dels = Dict{String, Set{String}}()
    AA_muts_seq_pos_only = Dict{String, Set{String}}()
    AA_muts_seq_pos_only_no_dels = Dict{String, Set{String}}()
    
    seq_AA_muts = Dict{String, Set{String}}()
    seq_AA_dels = Dict{String, Set{String}}()
    seq_nuc_del_ranges = Dict{String, Set{String}}()
    seq_AA_muts_no_dels = Dict{String, Set{String}}()
    seq_AA_muts_pos_only = Dict{String, Set{String}}()
    seq_AA_muts_pos_only_no_dels = Dict{String, Set{String}}()
    
    seq_unknown_AA = Dict{String, Set{String}}()
    seq_unknown_AA_ranges = Dict{String, Set{String}}()
    seq_mixed_AA_muts = Dict{String, Set{String}}()
#####################################################################################################################################    
## AA_muts_ct_pos_only: counts all muts at given AA position; e.g. "ORF1a:T1638" => 34
## seq_AA_muts_pos_only: keys=EPI_ISL, values=2-element array: 1=AA pos ("ORF1a:1638"), 2=AA res ("I")
## AA_muts_seq_pos_only: keys=AA pos ("ORF1a:1638"), values=EPI_ISLs
### WT means mutation relative to wild-type/ancestral
    seq_nuc_muts_WT = Dict{String, Set{String}}()
    seq_nuc_del_ranges_WT = Dict{String, Set{String}}()
    seq_AA_muts_WT = Dict{String, Set{String}}()
    seq_AA_dels_WT = Dict{String, Set{String}}()
    
    nuc_muts_seq_WT = Dict{String, Set{String}}()
    nuc_dels_seq_WT = Dict{String, Set{String}}()
    
    AA_muts_seq_WT = Dict{String, Set{String}}()
    AA_dels_seq_WT = Dict{String, Set{String}}()
    
    seq_AA_muts_WT_pos_only = Dict{String, Set{String}}()
    AA_muts_seq_WT_pos_only = Dict{String, Set{String}}()
####################################################################################################################################  
    nuc_muts_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_dels_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_muts_rep_seq_grps_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_muts = Dict{Int, Set{String}}()
    rep_seq_grps_dels = Dict{Int, Set{String}}()
    rep_seq_grps_muts_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_del_ranges_ct = Dict{String, Int}()
    
    rep_seq_grps_nuc_dropout = Dict{String, Set{Int}}()
    rep_seq_grps_mixed_nucs = Dict{String, Set{String}}()
#################################################################################################################################### 
    AA_muts_rep_seq_grps = Dict{String, Set{Int}}()
    AA_dels_rep_seq_grps = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_no_dels = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_pos_only = Dict{String, Set{Int}}()
    
    rep_seq_grps_AA = Dict{Int, Set{String}}()
    rep_seq_grps_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_unknown_AA_ranges = Dict{Int, Set{String}}()
#    rep_seq_grps_unknown_AA = Dict{Int, Set{String}}()
#    rep_seq_grps_mixed_AA_muts = Dict{Int, Set{String}}() 
#################################################################################################################################### 
    rep_seq_grps_nuc_muts_WT = Dict{String, Set{String}}()
    rep_seq_grps_nuc_dels_WT = Dict{String, Set{String}}()
    rep_seq_grps_AA_muts_WT = Dict{String, Set{String}}()
    rep_seq_grps_AA_dels_WT = Dict{String, Set{String}}()
    
    nuc_muts_rep_seq_grps_WT = Dict{String, Set{String}}()
    nuc_dels_rep_seq_grps_WT = Dict{String, Set{String}}()
    
    AA_muts_rep_seq_grps_WT = Dict{String, Set{String}}()
    AA_dels_rep_seq_grps_WT = Dict{String, Set{String}}()
    
    rep_seq_grps_AA_muts_WT_pos_only = Dict{String, Set{String}}()
    AA_muts_rep_seq_grps_WT_pos_only = Dict{String, Set{String}}()
#####################################################################################################################################
    NSP_muts = Dict{String, Dict{String, Int}}()
    NSP_muts_no_dels = Dict{String, Dict{String, Int}}()
    NSP_muts_sortByCt_Arr = []
    NSP_muts_sortByPos_Arr = []
    NSP_muts_no_dels_sortByCt_Arr = []
    NSP_muts_no_dels_sortByPos_Arr = []
#####################################################################################################################################
################################### AA Substitution Counts for Select Protein Domains  ##############################################
#####################################################################################################################################
    NSP3_domains = Set(["NSP3_Ubl1", "NSP3_HVR", "NSP3_Mac1", "NSP3_Mac2", "NSP3_Mac3", "NSP3_DPUP", "NSP3_Ubl2", "NSP3_PLpro", "NSP3_NAB", "NSP3_BSM", "NSP3_TM1", "NSP3_Ecto3", "NSP3_TM234HLX", "NSP3_Y1", "NSP3_CoVY"])
    NSP4_domains = Set(["NSP4_TM1", "NSP4_Ecto4", "NSP4_TM2_TM6", "NSP4_CTD"])
    NSP6_domains = Set(["NSP6_AmphHlx", "NSP6_MAE", "NSP6_cyto_CTD"])
    NSP12_domains = Set(["NSP12_NiRAN", "NSP12_intrfce", "NSP12_fingers", "NSP12_palm", "NSP12_palmLnk", "NSP12_thumb"])
    NSP13_domains = Set(["NSP13_ZBD", "NSP13_stalk", "NSP13_1B", "NSP13_RecA1", "NSP13_RecA2"])
    NSP14_domains = Set(["NSP14_nsp10", "NSP14_EXON", "NSP14_hinge1", "NSP14_hinge2", "NSP14_N7MTase"])
    NSP15_domains = Set(["NSP15_NTD", "NSP15_MD", "NSP15_endoU"])
    S_domains = Set(["S_S1", "S_S2", "S_NTD", "S_N2R", "S_RBD", "S_RBM", "S_SD1", "S_SD2", "S_630_loop", "S_FCS_region", "S_Beta1", "S_3H", "S_IL770", "S_FPPR", "S_FP", "S_HR1", "S_CH", "S_CD", "S_Beta2", "S_2turnHelix", "S_HR2", "S_TM", "S_CT"])
    ORF3a_domains = Set(["ORF3a_SignalP", "ORF3a_NTD", "ORF3a_TM1", "ORF3a_TM12Lnk", "ORF3a_TM2", "ORF3a_TM3", "ORF3a_cytosl1", "ORF3a_Loop", "ORF3a_3DB", "ORF3a_CTD"])
    E_domains = Set(["E_TM", "E_cytosol", "E_CTD"])
    N_domains = Set(["N_N1", "N_N2", "N_N3", "N_N4", "N_N5", "N_SR", "N_L_helix", "N_CBP", "N_9b_overlap"])
####################################################################################################################################
#################################### For AA ranges of each domain + sources, see 2 cells below #####################################
####################################################################################################################################
############ "NSP6_loop"=>75,     "NSP6_loop"=>NSP6_loop_ct,    "NSP6_loop"=>NSP6_loop_ct_no_dels,   "NSP6_loop"=>"NSP6:35-40|61-66|88-112|132-138|157-181|205-210",        "NSP6_loop"=>"BitSet(35:40; 61:66; 88:112; 132:138; 157:181; 205-210),  
############ "NSP6_TM"=>124,      "NSP6_TM"=>NSP6_TM_ct,        "NSP6_TM"=>NSP6_TM_ct_no_dels,       "NSP6_TM"=>"NSP6:12-34|41-60|67-87|113-131|139-156|182-204|211-230",       "NSP6_TM"=>BitSet(12:34; 41:60; 67:87; 113:131; 139:156; 182:204; 211:230),
                            ### AA: 10-27, 43-60, 63-80, 111-128, 130-150, 160-172, 176-190, 209-228 (alternate cource for TM: The multiple roles of nsp6 in the molecular pathogenesis of SARS-CoV-2, Antiviral Research; https://doi.org/10.1016/j.antiviral.2023.105590
#    NSP6_TM          ### AA: 12-34, 41-60, 67-87, 113-131, 139-156, 182-204, 211-230    Source: Feng S, O'Brien A, Chen DY, Saeed M, Baker SC. SARS-CoV-2 nonstructural protein 6 from Alpha to Omicron: evolution of a transmembrane protein. mBio. 2023;14(4):e0068823. doi:10.1128/mbio.00688-23
#    NSP6_loop        ### AA: 35-40, 61-66, 88-112, 132-138, 157-181, 205-210           Source: Feng S, O'Brien A, Chen DY, Saeed M, Baker SC. SARS-CoV-2 nonstructural protein 6 from Alpha to Omicron: evolution of a transmembrane protein. mBio. 2023;14(4):e0068823. doi:10.1128/mbio.00688-23
####################################################################################################################################
    gene_protein_length = Dict{String, Int}("NSP1"=>180, "NSP2"=>638, "NSP3"=>1945, "NSP4"=>500, "NSP5"=>306, "NSP6"=>290, "NSP7"=>83, "NSP8"=>198, "NSP9"=>113, "NSP10"=>139, "NSP12"=>932, "NSP13"=>601, "NSP14"=>527, "NSP15"=>346, "NSP16"=>298, "ORF3a"=>275, "ORF6"=>61, "ORF7a"=>121, "ORF7b"=>43, "ORF8"=>121, "ORF9b"=>97, "S"=>1273, "E"=>75, "M"=>222, "N"=>419)
    gene_protein_mut_ct = Dict{String, Int}("NSP1"=>0, "NSP2"=>0, "NSP3"=>0, "NSP4"=>0, "NSP5"=>0, "NSP6"=>0, "NSP7"=>0, "NSP8"=>0, "NSP9"=>0, "NSP10"=>0, "NSP12"=>0, "NSP13"=>0, "NSP14"=>0, "NSP15"=>0, "NSP16"=>0, "ORF3a"=>0, "ORF6"=>0, "ORF7a"=>0, "ORF7b"=>0, "ORF8"=>0, "ORF9b"=>0, "S"=>0, "E"=>0, "M"=>0, "N"=>0)
    gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)
####################################################################################################################################
    domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP6_AmphHlx"=>20, "NSP6_MAE"=>21, "NSP6_cyto_CTD"=>22, "NSP12_NiRAN"=>23, "NSP12_intrfce"=>24, "NSP12_fingers"=>25, "NSP12_palm"=>26, "NSP12_palmLnk"=>27, "NSP12_thumb"=>28, "NSP13_ZBD"=>29, "NSP13_stalk"=>30, "NSP13_1B"=>31, "NSP13_RecA1"=>32, "NSP13_RecA2"=>33, "NSP14_nsp10"=>34, "NSP14_EXON"=>35, "NSP14_hinge1"=>36, "NSP14_hinge2"=>37, "NSP14_N7MTase"=>38, "NSP15_NTD"=>39, "NSP15_MD"=>40, "NSP15_endoU"=>41, "S_S1"=>42, "S_S2"=>43, "S_NTD"=>44, "S_N2R"=>45, "S_RBD"=>46, "S_RBM"=>47, "S_SD1"=>48, "S_SD2"=>49, "S_630_loop"=>50, "S_FCS_region"=>51, "S_Beta1"=>52, "S_3H"=>53, "S_IL770"=>54, "S_FPPR"=>55, "S_FP"=>56, "S_HR1"=>57, "S_CH"=>58, "S_CD"=>59, "S_Beta2"=>60, "S_2turnHelix"=>61, "S_HR2"=>62, "S_TM"=>63, "S_CT"=>64, "ORF3a_SignalP"=>65, "ORF3a_NTD"=>66, "ORF3a_TM1"=>67, "ORF3a_TM12Lnk"=>68, "ORF3a_TM2"=>69, "ORF3a_TM3"=>70, "ORF3a_cytosl1"=>71, "ORF3a_Loop"=>72, "ORF3a_3DB"=>73, "ORF3a_CTD"=>74, "E_TM"=>75, "E_cytosol"=>76, "E_CTD"=>77, "N_N1"=>78, "N_N2"=>79, "N_N3"=>80, "N_N4"=>81, "N_N5"=>82, "N_SR"=>83, "N_L_helix"=>84, "N_CBP"=>85, "N_9b_overlap"=>86)
    domain_length = Dict{String, Int}("NSP3_Ubl1"=>111, "NSP3_HVR"=>95, "NSP3_Mac1"=>168, "NSP3_Mac2"=>176, "NSP3_Mac3"=>123, "NSP3_DPUP"=>70, "NSP3_Ubl2"=>60, "NSP3_PLpro"=>259, "NSP3_NAB"=>151, "NSP3_BSM"=>131, "NSP3_TM1"=>106, "NSP3_Ecto3"=>59, "NSP3_TM234HLX"=>87, "NSP3_Y1"=>179, "NSP3_CoVY"=>181, "NSP4_TM1"=>31, "NSP4_Ecto4"=>240, "NSP4_TM2_TM6"=>139, "NSP4_CTD"=>90, "NSP6_AmphHlx"=>18, "NSP6_MAE"=>40, "NSP6_cyto_CTD"=>14, "NSP12_NiRAN"=>250, "NSP12_intrfce"=>148, "NSP12_fingers"=>183, "NSP12_palm"=>171, "NSP12_palmLnk"=>60, "NSP12_thumb"=>120, "NSP13_ZBD"=>101, "NSP13_stalk"=>49, "NSP13_1B"=>76, "NSP13_RecA1"=>182, "NSP13_RecA2"=>159, "NSP14_nsp10"=>58, "NSP14_EXON"=>216, "NSP14_hinge1"=>15, "NSP14_hinge2"=>24, "NSP14_N7MTase"=>226, "NSP15_NTD"=>64, "NSP15_MD"=>118, "NSP15_endoU"=>141, "S_S1"=>685, "S_S2"=>587, "S_NTD"=>304, "S_N2R"=>29, "S_RBD"=>194, "S_RBM"=>69, "S_SD1"=>63, "S_SD2"=>83, "S_630_loop"=>24, "S_FCS_region"=>17, "S_Beta1"=>12, "S_3H"=>33, "S_IL770"=>62, "S_FPPR"=>35, "S_FP"=>43, "S_HR1"=>76, "S_CH"=>50, "S_CD"=>33, "S_Beta2"=>9, "S_2turnHelix"=>8, "S_HR2"=>48, "S_TM"=>23, "S_CT"=>39, "ORF3a_SignalP"=>14, "ORF3a_NTD"=>19, "ORF3a_TM1"=>23, "ORF3a_TM12Lnk"=>20, "ORF3a_TM2"=>23, "ORF3a_TM3"=>23, "ORF3a_cytosl1"=>34, "ORF3a_Loop"=>14, "ORF3a_3DB"=>53, "ORF3a_CTD"=>53, "E_TM"=>30, "E_cytosol"=>12, "E_CTD"=>7, "N_N1"=>43, "N_N2"=>131, "N_N3"=>71, "N_N4"=>118, "N_N5"=>55, "N_SR"=>31, "N_L_helix"=>21, "N_CBP"=>33, "N_9b_overlap"=>98)
    domain_mut_ct = Dict{String, Int}("NSP3_Ubl1"=>0, "NSP3_HVR"=>0, "NSP3_Mac1"=>0, "NSP3_Mac2"=>0, "NSP3_Mac3"=>0, "NSP3_DPUP"=>0, "NSP3_Ubl2"=>0, "NSP3_PLpro"=>0, "NSP3_NAB"=>0, "NSP3_BSM"=>0, "NSP3_TM1"=>0, "NSP3_Ecto3"=>0, "NSP3_TM234HLX"=>0, "NSP3_Y1"=>0, "NSP3_CoVY"=>0, "NSP4_TM1"=>0, "NSP4_Ecto4"=>0, "NSP4_TM2_TM6"=>0, "NSP4_CTD"=>0, "NSP6_AmphHlx"=>0, "NSP6_MAE"=>0, "NSP6_cyto_CTD"=>0, "NSP12_NiRAN"=>0, "NSP12_intrfce"=>0, "NSP12_fingers"=>0, "NSP12_palm"=>0, "NSP12_palmLnk"=>0, "NSP12_thumb"=>0, "NSP13_ZBD"=>0, "NSP13_stalk"=>0, "NSP13_1B"=>0, "NSP13_RecA1"=>0, "NSP13_RecA2"=>0, "NSP14_nsp10"=>0, "NSP14_EXON"=>0, "NSP14_hinge1"=>0, "NSP14_hinge2"=>0, "NSP14_N7MTase"=>0, "NSP15_NTD"=>0, "NSP15_MD"=>0, "NSP15_endoU"=>0, "S_S1"=>0, "S_S2"=>0, "S_NTD"=>0, "S_N2R"=>0, "S_RBD"=>0, "S_RBM"=>0, "S_SD1"=>0, "S_SD2"=>0, "S_630_loop"=>0, "S_FCS_region"=>0, "S_Beta1"=>0, "S_3H"=>0, "S_IL770"=>0, "S_FPPR"=>0, "S_FP"=>0, "S_HR1"=>0, "S_CH"=>0, "S_CD"=>0, "S_Beta2"=>0, "S_2turnHelix"=>0, "S_HR2"=>0, "S_TM"=>0, "S_CT"=>0, "ORF3a_SignalP"=>0, "ORF3a_NTD"=>0, "ORF3a_TM1"=>0, "ORF3a_TM12Lnk"=>0, "ORF3a_TM2"=>0, "ORF3a_TM3"=>0, "ORF3a_cytosl1"=>0, "ORF3a_Loop"=>0, "ORF3a_3DB"=>0, "ORF3a_CTD"=>0, "E_TM"=>0, "E_cytosol"=>0, "E_CTD"=>0, "N_N1"=>0, "N_N2"=>0, "N_N3"=>0, "N_N4"=>0, "N_N5"=>0, "N_SR"=>0, "N_L_helix"=>0, "N_CBP"=>0, "N_9b_overlap"=>0)
    domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM234HLX"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP6_AmphHlx"=>"NSP6:219-236", "NSP6_MAE"=>"NSP6:237-276", "NSP6_cyto_CTD"=>"NSP6:277-290", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "NSP13_ZBD"=>"NSP13:1-101", "NSP13_stalk"=>"NSP13:102-150", "NSP13_1B"=>"NSP13:151-226", "NSP13_RecA1"=>"NSP13:261-442", "NSP13_RecA2"=>"NSP13:443-601", "NSP14_nsp10"=>"NSP14:1-58", "NSP14_EXON"=>"NSP14:67-282 ", "NSP14_hinge1"=>"NSP14:286-300", "NSP14_hinge2"=>"NSP14:411-434", "NSP14_N7MTase"=>"NSP14:302-527", "NSP15_NTD"=>"NSP15:1-64", "NSP15_MD"=>"NSP15:65-182", "NSP15_endoU"=>"NSP15:207-347", "S_S1"=>"S:2-686", "S_S2"=>"S:687-1273", "S_NTD"=>"S:2-305", "S_N2R"=>"S:306-334", "S_RBD"=>"S:335-528", "S_RBM"=>"S:438-506", "S_SD1"=>"S:529-591", "S_SD2"=>"S:592-674", "S_630_loop"=>"S:619-642", "S_FCS_region"=>"S:675-691", "S_Beta1"=>"S:718-729", "S_3H"=>"S:747-769", "S_IL770"=>"S:770-831", "S_FPPR"=>"S:832-866", "S_FP"=>"S:867-909", "S_HR1"=>"S:910-985", "S_CH"=>"S:986-1035", "S_CD"=>"S:1036-1068", "S_Beta2"=>"S:1127-1135", "S_2turnHelix"=>"S:1148-1155", "S_HR2"=>"S:1164-1211", "S_TM"=>"S:1212-1234", "S_CT"=>"S:1235-1273", "ORF3a_SignalP"=>"ORF3a:1-14", "ORF3a_NTD"=>"ORF3a:15-33", "ORF3a_TM1"=>"ORF3a:34-56", "ORF3a_TM12Lnk"=>"ORF3a:57-76", "ORF3a_TM2"=>"ORF3a:77-99", "ORF3a_TM3"=>"ORF3a:103-125", "ORF3a_cytosl1"=>"ORF3a:126-169", "ORF3a_Loop"=>"ORF3a:170-183", "ORF3a_3DB"=>"ORF3a:170-222", "ORF3a_CTD"=>"ORF3a:223-275", "E_TM"=>"E:8-38", "E_cytosol"=>"E:54-65", "E_CTD"=>"E:69-75", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
    domain_residues_NSP_BitSet = Dict{String, BitSet}("NSP3_Ubl1"=>BitSet(1:111), "NSP3_HVR"=>BitSet(112:206), "NSP3_Mac1"=>BitSet(207:374), "NSP3_Mac2"=>BitSet(375:551), "NSP3_Mac3"=>BitSet(552:673), "NSP3_DPUP"=>BitSet(674:743), "NSP3_Ubl2"=>BitSet(744:803), "NSP3_PLpro"=>BitSet(804:1052), "NSP3_NAB"=>BitSet(1053:1203), "NSP3_BSM"=>BitSet(1204:1334), "NSP3_TM1"=>BitSet(1335:1440), "NSP3_Ecto3"=>BitSet(1441:1499), "NSP3_TM234HLX"=>BitSet(1500:1586), "NSP3_Y1"=>BitSet(1587:1764), "NSP3_CoVY"=>BitSet(1765:1945), "NSP4_TM1"=>BitSet(1:31), "NSP4_Ecto4"=>BitSet(32:271), "NSP4_TM2_TM6"=>BitSet(272:410), "NSP4_CTD"=>BitSet(411:500), "NSP6_AmphHlx"=>BitSet(219:236), "NSP6_MAE"=>BitSet(237:276), "NSP6_cyto_CTD"=>BitSet(277:290), "NSP12_NiRAN"=>BitSet(1:250), "NSP12_intrfce"=>BitSet(251:398), "NSP12_fingers"=>BitSet(399:581), "NSP12_palm"=>BitSet(582:627), "NSP12_palmLnk"=>BitSet(628:687), "NSP12_palm2"=>BitSet(688:812), "NSP12_thumb"=>BitSet(813:932), "NSP13_ZBD"=>BitSet(1:101), "NSP13_stalk"=>BitSet(102:150), "NSP13_1B"=>BitSet(151:226), "NSP13_RecA1"=>BitSet(261:442), "NSP13_RecA2"=>BitSet(443:601), "NSP14_nsp10"=>BitSet(1:58), "NSP14_EXON"=>BitSet(67:282), "NSP14_hinge1"=>BitSet(286:300), "NSP14_hinge2"=>BitSet(411:434), "NSP14_N7MTase"=>BitSet(302:527), "NSP15_NTD"=>BitSet(1:64), "NSP15_MD"=>BitSet(65:182), "NSP15_endoU"=>BitSet(207:347), "S_S1"=>BitSet(2:686), "S_S2"=>BitSet(687:1273), "S_NTD"=>BitSet(2:305), "S_N2R"=>BitSet(306:334), "S_RBD"=>BitSet(335:528), "S_RBM"=>BitSet(438:506), "S_SD1"=>BitSet(529:591), "S_SD2"=>BitSet(592:674), "S_630_loop"=>BitSet(619:642), "S_FCS_region"=>BitSet(675:691), "S_Beta1"=>BitSet(718:729), "S_3H"=>BitSet(747:769), "S_IL770"=>BitSet(770:831), "S_FPPR"=>BitSet(832:866), "S_FP"=>BitSet(867:909), "S_HR1"=>BitSet(910:985), "S_CH"=>BitSet(986:1035), "S_CD"=>BitSet(1036:1068), "S_Beta2"=>BitSet(1127:1135), "S_2turnHelix"=>BitSet(1148:1155), "S_HR2"=>BitSet(1164:1211), "S_TM"=>BitSet(1212:1234), "S_CT"=>BitSet(1235:1273), "ORF3a_SignalP"=>BitSet(1:14), "ORF3a_NTD"=>BitSet(15:33), "ORF3a_TM1"=>BitSet(34:56), "ORF3a_TM12Lnk"=>BitSet(57:76), "ORF3a_TM2"=>BitSet(77:99), "ORF3a_TM3"=>BitSet(103:125), "ORF3a_cytosl1"=>BitSet(126:169), "ORF3a_Loop"=>BitSet(170:183), "ORF3a_3DB"=>BitSet(170:222), "ORF3a_CTD"=>BitSet(223:275), "E_TM"=>BitSet(9:38), "E_cytosol"=>BitSet(54:65), "E_CTD"=>BitSet(69:75), "N_N1"=>BitSet(2:44), "N_N2"=>BitSet(45:175), "N_N3"=>BitSet(176:246), "N_N4"=>BitSet(247:364), "N_N5"=>BitSet(365:419), "N_SR"=>BitSet(176:206), "N_L_helix"=>BitSet(215:235), "N_CBP"=>BitSet(247:279), "N_9b_overlap"=>BitSet(4:101) )
    domain_residues_ORF = Dict{String, String}()
    domain_residues_ORF_BitSet = Dict{String, BitSet}()
####################################################################################################################################
    NSP_AA_size = Dict{String, Int}("NSP1"=>180, "NSP2"=>638, "NSP3"=>1945, "NSP4"=>500, "NSP5"=>306, "NSP6"=>290, "NSP7"=>83, "NSP8"=>198, "NSP9"=>113, "NSP10"=>139, "NSP11"=>0, "NSP12"=>932, "NSP13"=>601, "NSP14"=>527, "NSP15"=>346, "NSP16"=>299)                                                                # "NSP12"=>BitSet(1:923), 
    NSP_ranges = Dict{String, String}("NSP1"=>"ORF1a:1-180", "NSP2"=>"ORF1a:181-818", "NSP3"=>"ORF1a:819-2763", "NSP4"=>"ORF1a:2764-3263", "NSP5"=>"ORF1a:3264-3569", "NSP6"=>"ORF1a:3570-3859", "NSP7"=>"ORF1a:3860-3942", "NSP8"=>"ORF1a:3943-4140", "NSP9"=>"ORF1a:4141-4253", "NSP10"=>"ORF1a:4254-4392", "NSP11"=>"", "NSP12"=>"1a:4393-1b:923", "NSP13"=>"ORF1b:924-1524", "NSP14"=>"ORF1b:1525-2051", "NSP15"=>"ORF1b:2052-2397", "NSP16"=>"ORF1b:2398-2696", "S"=>"S:1-1274", "ORF3a"=>"ORF3a:1-276", "E"=>"E:1-76", "M"=>"M:1-223", "ORF6"=>"ORF6:1-62", "ORF7a"=>"ORF7a:1-122", "ORF7b"=>"ORF7b:1-44", "ORF8"=>"ORF8:1-122", "N"=>"N:1-420", "ORF9b"=>"ORF9b:1-98")
    NSP_ranges_num_only = Dict{String, BitSet}("NSP1"=>BitSet(1:180), "NSP2"=>BitSet(181:818), "NSP3"=>BitSet(819:2763), "NSP4"=>BitSet(2764:3263), "NSP5"=>BitSet(3264:3569), "NSP6"=>BitSet(3570:3859), "NSP7"=>BitSet(3860:3942), "NSP8"=>BitSet(3943:4140), "NSP9"=>BitSet(4141:4253), "NSP10"=>BitSet(4254:4392), "NSP11"=>BitSet(), "NSP12"=>BitSet([4393:4401; 1:923]), "NSP13"=>BitSet(924:1524), "NSP14"=>BitSet(1525:2051), "NSP15"=>BitSet(2052:2397), "NSP16"=>BitSet(2398:2696), "S"=>BitSet(1:1274), "ORF3a"=>BitSet(1:276), "E"=>BitSet(1:76), "M"=>BitSet(1:223), "ORF6"=>BitSet(1:62), "ORF7a"=>BitSet(1:122), "ORF7b"=>BitSet(1:44), "ORF8"=>BitSet(1:122), "N"=>BitSet(1:420), "ORF9b"=>BitSet(1:98) )
    NSP_ranges1a = Dict{Int, BitSet}(1=>BitSet(1:180), 2=>BitSet(181:818), 3=>BitSet(819:2763), 4=>BitSet(2764:3263), 5=>BitSet(3264:3569), 6=>BitSet(3570:3859), 7=>BitSet(3860:3942), 8=>BitSet(3943:4140), 9=>BitSet(4141:4253), 10=>BitSet(4254:4392), 12=>BitSet(4393:4401) )
    NSP_ranges1b = Dict{Int, BitSet}(12=>BitSet(1:923), 13=>BitSet(924:1524), 14=>BitSet(1525:2051), 15=>BitSet(2052:2397), 16=>BitSet(2398:2696) )
    NSP1a_add = Dict{Int, Int}(1=>0, 2=>180, 3=>818, 4=>2763, 5=>3263, 6=>3569, 7=>3859, 8=>3942, 9=>4140, 10=>4253, 11=>0, 12=>4392)
    NSP1b_add = Dict{Int, Int}(12=>-9, 13=>923, 14=>1524, 15=>2051, 16=>2397)
    NSP1ab_add = Dict{Int, Int}(1=>0, 2=>180, 3=>818, 4=>2763, 5=>3263, 6=>3569, 7=>3859, 8=>3942, 9=>4140, 10=>4353, 11=>0, 12=>-9, 13=>923, 14=>1524, 15=>2051, 16=>2397)
####################################################################################################################################
    function NSP_to_ORF(domain::String, NSP::Int)
        ORFadd = NSP1ab_add[NSP]
        first1 = minimum(domain_residues_NSP_BitSet[domain])
        last1 = maximum(domain_residues_NSP_BitSet[domain])
        first = first1 + ORFadd
        last = last1 + ORFadd
        ORF_range_BitSet = BitSet(first:last)
        firstORF = string(first)
        lastORF = string(last)
        ORF_range = ""
        if NSP < 12
            ORF_range = "ORF1a:$(firstORF)-$(lastORF)"
        else
            ORF_range = "ORF1b:$(firstORF)-$(lastORF)"
        end
        return ORF_range, ORF_range_BitSet
    end
#####################################################################################################################################
    NSP_dom_set_minus_NSP12NiRAN = Set(["NSP3_Ubl1", "NSP3_HVR", "NSP3_Mac1", "NSP3_Mac2", "NSP3_Mac3", "NSP3_DPUP", "NSP3_Ubl2", "NSP3_PLpro", "NSP3_NAB", "NSP3_BSM", "NSP3_TM1", "NSP3_Ecto3", "NSP3_TM234HLX", "NSP3_Y1", "NSP3_CoVY", "NSP4_TM1", "NSP4_Ecto4", "NSP4_TM2_TM6", "NSP4_CTD", "NSP6_AmphHlx", "NSP6_MAE", "NSP6_cyto_CTD", "NSP12_intrfce", "NSP12_fingers", "NSP12_palm", "NSP12_palmLnk", "NSP12_thumb", "NSP13_ZBD", "NSP13_stalk", "NSP13_1B", "NSP13_RecA1", "NSP13_RecA2", "NSP14_nsp10", "NSP14_EXON", "NSP14_hinge1", "NSP14_hinge2", "NSP14_N7MTase", "NSP15_NTD", "NSP15_MD", "NSP15_endoU"])
    non_NSP_dom_set = Set(["S_S1", "S_S2", "S_NTD", "S_N2R", "S_RBD", "S_RBM", "S_SD1", "S_SD2", "S_630_loop", "S_FCS_region", "S_Beta1", "S_3H", "S_IL770", "S_FPPR", "S_FP", "S_HR1", "S_CH", "S_CD", "S_Beta2", "S_2turnHelix", "S_HR2", "S_TM", "S_CT", "ORF3a_SignalP", "ORF3a_NTD", "ORF3a_TM1", "ORF3a_TM12Lnk", "ORF3a_TM2", "ORF3a_TM3", "ORF3a_cytosl1", "ORF3a_Loop", "ORF3a_3DB", "ORF3a_CTD", "E_TM", "E_cytosol", "E_CTD", "N_N1", "N_N2", "N_N3", "N_N4", "N_N5", "N_SR", "N_L_helix", "N_CBP", "N_9b_overlap"])   
    domain_residues_ORF["NSP12_NiRAN"] = "1a:4393-1b:241"
    domain_residues_ORF_BitSet["NSP12_NiRAN"] = BitSet([4393:4401; 1:241])
    for (dom, range) in domain_residues_NSP_BitSet
        NSP_part = string(split(dom, "_")[1])
        if dom in NSP_dom_set_minus_NSP12NiRAN
            if length(NSP_part) == 4
                NSP_num = parse(Int, string(NSP_part[4]))
            elseif length(NSP_part) == 5
                NSP_num = parse(Int, string(NSP_part[4:5]))
            end
            ORF, ORF_BitSet = NSP_to_ORF(dom, NSP_num)
            domain_residues_ORF_BitSet[dom] = ORF_BitSet
            domain_residues_ORF[dom] = ORF
        elseif dom in non_NSP_dom_set
            domain_residues_ORF[dom] = domain_residues_NSP[dom]
            domain_residues_ORF_BitSet[dom] = domain_residues_NSP_BitSet[dom]
        end
    end
##################################################################################################################################### 
    nuc_muts_ct = Dict{String, Int}()
    nuc_muts_ct_no_dels = Dict{String, Int}()
    nuc_muts_ct_no_dels_no_revs = Dict{String, Int}()
    nuc_muts_ct_pos_only = Dict{String, Int}()
    nuc_muts_ct_pos_only_no_dels = Dict{Int, Int}()
    nuc_muts_ct_pos_only_no_dels_no_revs = Dict{Int, Int}()
    
    nuc_muts_ct_adj = Dict{String, Float64}()
    nuc_muts_ct_adj_score = Dict{String, Float64}()
    nuc_muts_ct_adj_no_dels = Dict{String, Float64}()
    nuc_muts_ct_adj_score_no_dels = Dict{String, Float64}()

    nuc_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    nuc_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    nuc_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{String, Int}()
    
    nuc_muts_ct_WT = Dict{String, Int}()
    nuc_dels_ct_WT = Dict{String, Int}()
    
    nuc_muts_ct_sort_by_site = Vector{Pair{String, Int64}}()
    nuc_muts_ct_sort_by_seq_ct = Vector{Pair{String, Int64}}()
    
    nuc_muts_ct_adj_sort_by_site = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_score_sort_by_site = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_score_sort_by_score = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    nuc_muts_ct_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}()
####################################################################################################################################
    nuc_dels_ct = Dict{String, Int}()
    mixed_nucs_ct = Dict{String, Int}()
    nuc_dropout_ct = Dict{Int, Int}()
    
    AA_dels_ct = Dict{String, Int}()
    mixed_AA_ct = Dict{String, Int}()
    unknown_AA_ct = Dict{String, Int}()
####################################################################################################################################
    AA_muts_ct = Dict{String, Int}()
    AA_muts_ct_no_dels = Dict{String, Int}()
    AA_muts_ct_no_dels_no_revs = Dict{String, Int}()
    AA_muts_ct_pos_only = Dict{String, Int}()
    AA_muts_ct_pos_only_no_dels = Dict{String, Int}()
    AA_muts_ct_pos_only_no_dels_no_revs = Dict{String, Int}()
    AA_muts_ct_pos_only_reversion = Dict{String, Int}()

    AA_muts_ct_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{String, Int}()
    
    AA_muts_ct_WT = Dict{String, Int}()
    AA_dels_ct_WT = Dict{String, Int}()
    AA_muts_ct_WT_no_dels = Dict{String, Int}()
    AA_muts_ct_WT_pos_only = Dict{String, Int}()
    AA_muts_ct_WT_pos_only_no_dels = Dict{String, Int}()
    
    AA_muts_ct_adj = Dict{String, Float64}()
    AA_muts_ct_adj_no_dels = Dict{String, Float64}()
    AA_muts_ct_adj_score = Dict{String, Float64}()
    AA_muts_ct_adj_score_no_dels = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_no_dels = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_score = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_score_no_dels = Dict{String, Float64}()
    
    AA_muts_ct_sort_by_site = Vector{Pair{String, Int64}}()
    AA_muts_ct_sort_by_seq_ct = Vector{Pair{String, Int64}}()
    AA_muts_ct_no_dels_sort_by_site = Vector{Pair{String, Int64}}()
    AA_muts_ct_no_dels_sort_by_seq_ct = Vector{Pair{String, Int64}}()
    AA_muts_ct_no_dels_no_revs_sort_by_site = Vector{Pair{String, Int64}}()
    AA_muts_ct_no_dels_no_revs_sort_by_seq_ct = Vector{Pair{String, Int64}}()
    
    AA_muts_ct_adj_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    
    AA_muts_ct_adj_score_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_score_sort_by_score = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}()
    
    AA_muts_ct_pos_only_sort_by_site = Vector{Pair{String, Int64}}()
    AA_muts_ct_pos_only_sort_by_seq_ct = Vector{Pair{String, Int64}}()
    AA_muts_ct_pos_only_no_dels_sort_by_site = Vector{Pair{String, Int64}}()
    AA_muts_ct_pos_only_no_dels_sort_by_seq_ct = Vector{Pair{String, Int64}}()

    AA_muts_ct_pos_only_adj_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}()
    
    AA_muts_ct_pos_only_adj_score_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_score_sort_by_score = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}()
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}()
#####################################################################################################################################
    NSP1_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP2_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP3_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP4_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP5_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP6_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP7_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP8_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP9_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP10_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP11_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP12_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP13_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP14_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP15_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP16_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP1_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP2_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP3_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP4_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP5_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP6_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP7_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP8_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP9_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP10_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP11_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP12_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP13_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP14_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP15_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP16_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP1_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP2_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP3_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP4_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP5_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP6_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP7_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP8_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP9_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP10_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP11_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP12_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP13_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP14_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP15_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP16_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP1_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP2_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP3_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP4_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP5_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP6_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP7_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP8_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP9_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP10_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP11_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP12_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP13_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP14_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP15_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP16_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
#####################################################################################################################################
################################### AA Substitution Counts for Select Protein Domains  ##############################################
#####################################################################################################################################
    NSP3_domains = Set(["NSP3_Ubl1", "NSP3_HVR", "NSP3_Mac1", "NSP3_Mac2", "NSP3_Mac3", "NSP3_DPUP", "NSP3_Ubl2", "NSP3_PLpro", "NSP3_NAB", "NSP3_BSM", "NSP3_TM1", "NSP3_Ecto3", "NSP3_TM234HLX", "NSP3_Y1", "NSP3_CoVY"])
    NSP4_domains = Set(["NSP4_TM1", "NSP4_Ecto4", "NSP4_TM2_TM6", "NSP4_CTD"])
    NSP6_domains = Set(["NSP6_AmphHlx", "NSP6_MAE", "NSP6_cyto_CTD"])
    NSP12_domains = Set(["NSP12_NiRAN", "NSP12_intrfce", "NSP12_fingers", "NSP12_palm", "NSP12_palmLnk", "NSP12_thumb"])
    NSP13_domains = Set(["NSP13_ZBD", "NSP13_stalk", "NSP13_1B", "NSP13_RecA1", "NSP13_RecA2"])
    NSP14_domains = Set(["NSP14_nsp10", "NSP14_EXON", "NSP14_hinge1", "NSP14_hinge2", "NSP14_N7MTase"])
    NSP15_domains = Set(["NSP15_NTD", "NSP15_MD", "NSP15_endoU"])
    S_domains = Set(["S_S1", "S_S2", "S_NTD", "S_N2R", "S_RBD", "S_RBM", "S_SD1", "S_SD2", "S_630_loop", "S_FCS_region", "S_Beta1", "S_3H", "S_IL770", "S_FPPR", "S_FP", "S_HR1", "S_CH", "S_CD", "S_Beta2", "S_2turnHelix", "S_HR2", "S_TM", "S_CT"])
    ORF3a_domains = Set(["ORF3a_SignalP", "ORF3a_NTD", "ORF3a_TM1", "ORF3a_TM12Lnk", "ORF3a_TM2", "ORF3a_TM3", "ORF3a_cytosl1", "ORF3a_Loop", "ORF3a_3DB", "ORF3a_CTD"])
    E_domains = Set(["E_TM", "E_cytosol", "E_CTD"])
    N_domains = Set(["N_N1", "N_N2", "N_N3", "N_N4", "N_N5", "N_SR", "N_L_helix", "N_CBP", "N_9b_overlap"])
######################################################################################################################################
    seq_country = Dict{String, String}()
    seq_US_state = Dict{String, String}()
    seq_lab_dict = Dict{String, String}()
    seq_collection_date = Dict{String, String}()
    seq_date_index = Dict{String, Int}()
    seq_date_tuple = Dict{String, Tuple{Int, Int, Int}}()
    
    seq_clade = Dict{String, String}()
    seq_clade_display = Dict{String, String}()
    seq_pango = Dict{String, String}()
    seq_pango_unaliased = Dict{String, String}()

    rep_seq_grps_clade = Dict{Int, String}()
    rep_seq_grps_pango = Dict{Int, String}()
    rep_seq_grps_pango_unaliased = Dict{Int, String}()
    
    seq_clade_ct = Dict{String, Int}()
    seq_pango_ct = Dict{String, Int}()
    seq_pango_unaliased_ct = Dict{String, Int}()
    for clade in clade_set_complete
        seq_clade_ct[clade] = 0
    end

    seq_ct::Int = 0
    total_subs::Int = 0
    total_nuc_dels::Int = 0

    all_seqs = Vector{String}()
    all_seqs_set = Set{String}()
    all_qualifying_seqs = Vector{String}()
    all_qualifying_seqs_set = Set{String}()
    all_nonqualifying_seqs = Vector{String}()
    all_nonqualifying_seqs_set = Set{String}()    
    
    non_rep_seqs = Set{String}()
    rep_seqs = Set{String}()
    rep_seq_grps_seqs = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_seqs = Dict{Int, String}()

    rep_seq_grps_maxmut_nuc = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_nuc_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_del_ranges_ct = Dict{Int, Int}()
    rep_seq_grps_maxmut_nuc_dropout = Dict{Int, Set{Int}}()
    rep_seq_grps_maxmut_mixed_nucs = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_AA = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_pos_only_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_unknown_AA = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_unknown_AA_ranges = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_mixed_AA_muts = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_nuc_muts_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_nuc_dels_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_muts_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_dels_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_muts_WT_pos_only = Dict{Int, Set{String}}()

    rep_seq_grps_maxmut_clade = Dict{Int, String}()    
    rep_seq_grps_maxmut_pango = Dict{Int, String}()
    rep_seq_grps_maxmut_pango_unaliased = Dict{Int, String}()

    rep_seq_grps_maxmut_date_index = Dict{Int, Int}()
    rep_seq_grps_maxmut_date_tuple = Dict{Int, Tuple{Int, Int, Int}}()
############################################################################################################
    nuc_muts_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_dels_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_muts_rep_seq_grps_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_muts = Dict{Int, Set{String}}()
    rep_seq_grps_dels = Dict{Int, Set{String}}()
    rep_seq_grps_muts_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_del_ranges_ct = Dict{String, Int}()
    
    rep_seq_grps_nuc_dropout = Dict{String, Set{Int}}()
    rep_seq_grps_mixed_nucs = Dict{String, Set{String}}()
#################################################################################################################################### 
    AA_muts_rep_seq_grps = Dict{String, Set{Int}}()
    AA_dels_rep_seq_grps = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_no_dels = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_pos_only = Dict{String, Set{Int}}()
    
    rep_seq_grps_AA = Dict{Int, Set{String}}()
    rep_seq_grps_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only_no_dels = Dict{Int, Set{String}}()
#    rep_seq_grps_mixed_AA_muts = Dict{Int, Set{String}}() 
####################################################################################################################################
####################################################################################################################################
    non_rep_seqs_AA = Dict{String, Set{String}}()
    non_rep_seqs_AA_pos_only = Dict{String, Set{String}}()
    non_rep_seqs_AA_pos_only_no_dels = Dict{String, Set{String}}()
    
    global chronic_search_muts = Set{String}()
    global chronic_search_muts_min2 = Set{String}()
    global chronic_search_muts_min2_no_dels = Set{String}()
    global chronic_search_muts_v2 = Set{String}()
    
    too_many_reversions = Vector{String}()
    subs_9 = Vector{String}()
    subs_8 = Vector{String}()
    subs_7 = Vector{String}()
    subs_6 = Vector{String}()
    subs_5 = Vector{String}()
    subs_4 = Vector{String}()
    subs_3 = Vector{String}()
    subs_2 = Vector{String}()
    subs_9_ct::Int = 0
    subs_8_ct::Int = 0
    subs_7_ct::Int = 0
    subs_6_ct::Int = 0
    subs_5_ct::Int = 0
    subs_4_ct::Int = 0
    subs_3_ct::Int = 0
    subs_2_ct::Int = 0
####################################################################################################################################
    gene_AA_dict = Dict{String, String}()
    gene_AA_dict["ORF1a"] = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV*"
    gene_AA_dict["ORF1b"] = "RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
    gene_AA_dict["S"] = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
    gene_AA_dict["E"] = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
    gene_AA_dict["M"] = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
    gene_AA_dict["N"] = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
    gene_AA_dict["ORF3a"] = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
    gene_AA_dict["ORF6"] = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
    gene_AA_dict["ORF7a"] = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
    gene_AA_dict["ORF7b"] = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
    gene_AA_dict["ORF8"] = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
    gene_AA_dict["ORF9b"] = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
####################################################################################################################################
    NSP_AA_dict = Dict{Int, String}()
    for i in 1:10  "NSP$(string(i))"
        NSPkey = "NSP$(string(i))"
        first = minimum(NSP_ranges1a[i])
        last = maximum(NSP_ranges1a[i])
        NSP_AA_str = gene_AA_dict["ORF1a"][first:last]
        NSP_AA_dict[i] = NSP_AA_str
    end
    NSP_AA_dict[11] = ""
    NSP12_1a = gene_AA_dict["ORF1a"][4393:4401]
    NSP12_1b = gene_AA_dict["ORF1b"][1:923]
    NSP12_AA_str = "$(NSP12_1a)$(NSP12_1b)"
    NSP_AA_dict[12] = NSP12_AA_str
    for i in 13:16
        NSPkey = "NSP$(string(i))"
        first = minimum(NSP_ranges1b[i])
        last = maximum(NSP_ranges1b[i])
        NSP_AA_str = gene_AA_dict["ORF1b"][first:last]
        NSP_AA_dict[i] = NSP_AA_str
    end
    line_ct::Int = 0
######################################################################################################################################
######################################################################################################################################
    for line in eachline(ndjson)
        j = JSON3.read(line)
        name = EPI_ISL(j.seqName)
        if !(name in double_disqualified)
            push!(all_seqs_set, name)
            push!(all_seqs, name)
            seq_country[name] = country(j.seqName)
            if seq_country[name] == "USA" || seq_country[name] == "Canada"
                seq_US_state[name] = US_state(j.seqName)
            else
                seq_US_state[name] = ""
            end
            
            seq_clade[name] = j.clade
            seq_clade_display[name] = j.customNodeAttributes.clade_display
            seq_pango[name] = j.customNodeAttributes.Nextclade_pango
            seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
    #        seq_date = sequence_date(j.seqName)
            lab = ""
            if !isempty(seq_lab(j.seqName))
                lab = seq_lab(j.seqName)
            end
            seq_lab_dict[name] = lab
            seq_date = ""
    ####################################################
            try
                seq_clade[name] = j.clade
            catch
                seq_clade[name] = "Unknown"
            end
    ####################################################
            try
                seq_pango[name] = j.customNodeAttributes.Nextclade_pango
            catch
                seq_pango[name] = "Unknown"
            end
    ####################################################
            try
                seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
            catch
                seq_pango_unaliased[name] = "Unknown"
            end
    ####################################################
            seq_date = ""
            try
                seq_date = sequence_date(j.seqName)
            catch
                seq_date = "0-0-0"
            end
    ####################################################
            lab = ""
            if !isempty(seq_lab(j.seqName))
                lab = seq_lab(j.seqName)
            end
            seq_lab_dict[name] = lab
            seq_year::Int = 0        
            seq_month::Int = 0
            seq_day::Int = 0
            seq_yr = join(seq_date[1:4])
            if all(isdigit, seq_yr)
                seq_year = parse(Int, seq_yr)
            end
            if '-' in seq_date
                seq_mo = split(seq_date, "-")[2]
                if all(isdigit, seq_mo)
                    seq_month = parse(Int, seq_mo)
                end
            end
    ####################################################
            dash_ct::Int = 0
            for i in seq_date
                if i == '-'
                    dash_ct += 1
                end
            end
    ####################################################
            if dash_ct == 2
                seq_da = split(seq_date, "-")[3]
                if all(isdigit, seq_da)
                    seq_day = parse(Int, seq_da)
                end
            end
    ####################################################
            seq_collection_date[name] = seq_date
            if seq_month == 0 && seq_day == 0
                seq_collection_date[name] = "$(string(seq_year))-0-0"
            end
            if seq_month ≠ 0 && seq_day == 0
                seq_collection_date[name] = "$(string(seq_year))-$(string(seq_month))-0"
            end
            date_index = tuple_to_index[(seq_year, seq_month, seq_day)]
            seq_date_index[name] = date_index
            date_tuple = (seq_year, seq_month, seq_day)
            seq_date_tuple[name] = date_tuple
    ####################################################
    #        if length(seq_date) ≥ 7
    #            seq_mo = join(seq_date[6:7])
    #            if all(isdigit, seq_mo)
    #                seq_month = parse(Int, seq_mo)
    #            end
    #        end
    #        if length(seq_date) ≥ 10
    #            if all(isdigit, seq_da)
    #            seq_da = join(seq_date[9:10])
    #            seq_day = parse(Int, seq_da)
    #        end
    ####################################################
    ### Since there is——vexingly——no guide to what an ndjson file actually contains or what form it takes——I've had to spend 
    ###     dozens of hours documenting said info and forms and have inserted it here so I don't have to find & consult my text file
    ###     every time.
    #31 privateAaMutations
    #          Type: Dict{String, Any}
    #     j.privateAaMutations
    #          j.privateAaMutations.ORF1a  ...
    #          j.privateAaMutations.S
    #               j.privateAaMutations.S.totalReversionSubstitutions
    #               j.privateAaMutations.S.totalPrivateDeletions
    #               j.privateAaMutations.S.totalPrivateSubstitutions
    #               j.privateAaMutations.S.reversionSubstitutions
    #                    j.privateAaMutations.S.reversionSubstitutions.1  ...
    #                        j.privateAaMutations.S.reversionSubstitutions.1.refAa
    #                        j.privateAaMutations.S.reversionSubstitutions.1.cdsName
    #                        j.privateAaMutations.S.reversionSubstitutions.1.pos
    #                        j.privateAaMutations.S.reversionSubstitutions.1.qryAa
    #               j.privateAaMutations.S.privateDeletionRanges
    #                    j.privateAaMutations.S.privateDeletionRanges.1  ...
    #                        j.privateAaMutations.S.privateDeletionRanges.1.range.begin
    #                        j.privateAaMutations.S.privateDeletionRanges.1.range.end
            seq_ct += 1
####################################################    
            total_revs = j.privateNucMutations.totalReversionSubstitutions
            push!(total_nuc_revs_seq[total_revs], name)
            seq_nuc_total_revs[name] = total_revs
########################################################################################################
# #20 insertions
#          Type: Vector{Any}
#           j.insertions
#          Example: Any[Dict{String, Any}("ins" => "TCATGCCGCTGT", "pos" => 21607)]
# #22 aaInsertions
#           j.aaInsertions
#                j.aaInsertions.1
#                     j.aaInsertions.1.ins   eg "MPLF"
#                     j.aaInsertions.1.cds   eg "S"
#                     j.aaInsertions.1.pos   eg 15
#          Example: 
#          "aaInsertions" => Any[
#               Dict{String, Any}(
#                    "ins" => "MPLF", 
#                    "cds" => "S", 
#                    "pos" => 15)],
#          Example: Any[Dict{String, Any}("ins" => "MPLF", "cds" => "S", "pos" => 15)]
########################################################################################################
        nuc_insertions_WT = j.insertions
        AA_insertions_WT = j.aaInsertions
        seq_nuc_insertions_WT[name] = Vector{Tuple{String, Int}}()
        seq_AA_insertions_WT[name] = Vector{Tuple{String, String, Int}}()
        for i in nuc_insertions_WT
            insert = i.ins
            ins_pos = i.pos
            push!(seq_nuc_insertions_WT[name], (insert, ins_pos))
        end
        for i in AA_insertions_WT
            ins_gene = i.cds
            insert = i.ins
            ins_pos = i.pos
            push!(seq_AA_insertions_WT[name], (ins_gene, insert, ins_pos))
        end
########################################################################################################
            nuc_del_ranges = Set{String}()
            WT_nuc_del_ranges = Set{String}()
            genes = ["ORF1a", "ORF1b", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"]
            privAA = j.privateAaMutations
            AA_ORF1a = privAA.ORF1a
            AA_ORF1b = privAA.ORF1b
            AA_ORF3a = privAA.ORF3a
            AA_ORF6 = privAA.ORF6
            AA_ORF7a = privAA.ORF7a
            AA_ORF7b = privAA.ORF7b
            AA_ORF8 = privAA.ORF8
            AA_ORF9b = privAA.ORF9b
            AA_S = privAA.S
            AA_E = privAA.E
            AA_M = privAA.M
            AA_N = privAA.N
            gene_AA_subs = Set([AA_ORF1a.privateSubstitutions, AA_ORF1b.privateSubstitutions, AA_ORF3a.privateSubstitutions, AA_ORF6.privateSubstitutions, AA_ORF7a.privateSubstitutions, AA_ORF7b.privateSubstitutions, AA_ORF8.privateSubstitutions, AA_ORF9b.privateSubstitutions, AA_S.privateSubstitutions, AA_E.privateSubstitutions, AA_M.privateSubstitutions, AA_N.privateSubstitutions])
            total_AA_subs = AA_ORF1a.totalPrivateSubstitutions + AA_ORF1a.totalPrivateSubstitutions + AA_ORF3a.totalPrivateSubstitutions + AA_ORF6.totalPrivateSubstitutions + AA_ORF7a.totalPrivateSubstitutions + AA_ORF7b.totalPrivateSubstitutions + AA_ORF8.totalPrivateSubstitutions + AA_ORF9b.totalPrivateSubstitutions + AA_S.totalPrivateSubstitutions + AA_E.totalPrivateSubstitutions + AA_M.totalPrivateSubstitutions + AA_N.totalPrivateSubstitutions
            total_del_ranges = length(AA_ORF1a.privateDeletionRanges) + length(AA_ORF1a.privateDeletionRanges) + length(AA_ORF3a.privateDeletionRanges) + length(AA_ORF6.privateDeletionRanges) + length(AA_ORF7a.privateDeletionRanges) + length(AA_ORF7b.privateDeletionRanges) + length(AA_ORF8.privateDeletionRanges) + length(AA_ORF9b.privateDeletionRanges) + length(AA_S.privateDeletionRanges) + length(AA_E.privateDeletionRanges) + length(AA_M.privateDeletionRanges) + length(AA_N.privateDeletionRanges)
            total_AA_reversions = AA_ORF1a.totalReversionSubstitutions + AA_ORF1a.totalReversionSubstitutions + AA_ORF3a.totalReversionSubstitutions + AA_ORF6.totalReversionSubstitutions + AA_ORF7a.totalReversionSubstitutions + AA_ORF7b.totalReversionSubstitutions + AA_ORF8.totalReversionSubstitutions + AA_ORF9b.totalReversionSubstitutions + AA_S.totalReversionSubstitutions + AA_E.totalReversionSubstitutions + AA_M.totalReversionSubstitutions + AA_N.totalReversionSubstitutions
            total_AA_subs_plus_del_ranges_minus_reversions = total_AA_subs + total_del_ranges - total_AA_reversions
            
            seq_AA_total_revs[name] = total_AA_reversions
            push!(total_AA_revs_seq[total_AA_reversions], name)
            
            subs = Set(j.privateNucMutations.privateSubstitutions)
            dels = Set(j.privateNucMutations.privateDeletionRanges)
            nuc_dels = Set(j.privateNucMutations.privateDeletions)
            AA_deletions = Set([AA_ORF1a.privateDeletionRanges, AA_ORF1b.privateDeletionRanges, AA_ORF3a.privateDeletionRanges, AA_ORF6.privateDeletionRanges, AA_ORF7a.privateDeletionRanges, AA_ORF7b.privateDeletionRanges, AA_ORF8.privateDeletionRanges, AA_ORF9b.privateDeletionRanges, AA_S.privateDeletionRanges, AA_E.privateDeletionRanges, AA_M.privateDeletionRanges, AA_N.privateDeletionRanges])
    
            del_count::Int = 0
            for range in dels
                del_count += 1
            end
            seq_nuc_del_ranges_ct[name] = del_count
    
            subs_WT = Set(j.substitutions)
            # WT_dels = Set(j.deletions)
            nuc_dels_WT = Set(j.deletions)
            gene_AA_del_WT = j.aaDeletions
            del_ct::Int = 0
    
            function subs_n_ct(n, subs_n, subs_n_ct)
                if length(subs) + length(dels) == n
                    subs_n_ct +=1
                    push!(subs_n, name)
                end
            end
            subs_n_ct(9, subs_9, subs_9_ct)
            subs_n_ct(8, subs_8, subs_8_ct)
            subs_n_ct(7, subs_7, subs_7_ct)
            subs_n_ct(6, subs_6, subs_6_ct)
            subs_n_ct(5, subs_5, subs_5_ct)
            subs_n_ct(4, subs_4, subs_4_ct)
            subs_n_ct(3, subs_3, subs_3_ct)
            subs_n_ct(2, subs_2, subs_2_ct)
            if j.privateNucMutations.totalReversionSubstitutions ≥ revs_thresh && !(name in other_exceptions)
                push!(too_many_reversions, name)
                push!(all_nonqualifying_seqs, name)
                push!(all_nonqualifying_seqs_set, name)
            end
            if name in excluded_seqs && !(name in other_exceptions)
                push!(all_nonqualifying_seqs, name)
                push!(all_nonqualifying_seqs_set, name)
            end
    #        if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions < abs_min_mut_thresh && !(name in other_exceptions)
            if total_AA_subs_plus_del_ranges_minus_reversions < abs_min_mut_thresh && !(name in other_exceptions)
                push!(all_nonqualifying_seqs, name)
                push!(all_nonqualifying_seqs_set, name)
            end
    ############################################## BEGIN BEGIN BEGIN BEGIN ##############################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    ############################################## BEGIN BEGIN BEGIN BEGIN ##############################################################
            for k in subs
                pos = k.pos + 1
                if !(pos in excluded_pos) && !(k.refNuc == "-")
                    total_subs += 1
                    nuc_sub = "$(k.refNuc)$(string(pos))$(k.qryNuc)"
                    get!(seq_nuc_muts, name, Set{String}() )
                    push!(seq_nuc_muts[name], nuc_sub)
                    get!(nuc_muts_seq, nuc_sub, Set{String}() )
                    push!(nuc_muts_seq[nuc_sub], name)
    #                if k.qryNuc ≠ "-"
    #                    get!(seq_nuc_muts_no_dels, name, Set{String}() )
    #                    push!(seq_nuc_muts_no_dels[name], nuc_sub)
    #                end
                end
            end
            for k in subs_WT
                pos = k.pos + 1
                if !(pos in excluded_pos)
                    nuc_sub = "$(k.refNuc)$(string(pos))$(k.qryNuc)"  
                    get!(seq_nuc_muts_WT, name, Set{String}() )
                    push!(seq_nuc_muts_WT[name], nuc_sub)
                    get!(nuc_muts_seq_WT, nuc_sub, Set{String}() )
                    push!(nuc_muts_seq_WT[nuc_sub], name)
                end
            end
            for d in nuc_dels
                pos = d.pos + 1
                if !(pos in excluded_pos)
                    n_del = "$(d.refNuc)$(string(pos))-"
                    get!(seq_nuc_muts, name, Set{String}() )
                    push!(seq_nuc_muts[name], n_del)
                    get!(nuc_muts_seq, n_del, Set{String}() )
                    push!(nuc_muts_seq[n_del], name)
    #                get!(date_nuc_mut_ct, date_index, Dict{String, Int64}() )
    #                date_nuc_mut_ct[date_index][n_del] = get(date_nuc_mut_ct[date_index], n_del, 0) + 1
                end
            end
            for d in dels
                del_start = d.range[:begin] + 1
                del_end = d.range[:end]
                del_start_str = string(del_start)
                del_end_str = string(del_end)
                del_str = "$(del_start_str)-$(del_end_str)"
                push!(nuc_del_ranges, del_str)
            end
            for del in nuc_del_ranges
                get!(seq_nuc_del_ranges, name, Set{String}() )
                push!(seq_nuc_del_ranges[name], del)
                get!(nuc_dels_seq, del, Set{String}() )
                push!(nuc_dels_seq[del], name)
            end
            for d in nuc_dels_WT
                del_start = d.range[:begin] + 1
                del_end = d.range[:end]
                del_start_str = string(del_start)
                del_end_str = string(del_end)
                del_str = "$(del_start_str)-$(del_end_str)"
                push!(WT_nuc_del_ranges, del_str)
            end
            for del in WT_nuc_del_ranges
                get!(seq_nuc_del_ranges_WT, name, Set{String}() )
                push!(seq_nuc_del_ranges_WT[name], del)
                get!(nuc_dels_seq_WT, del, Set{String}() )
                push!(nuc_dels_seq_WT[del], name)
            end
    ######################################
            for i in gene_AA_subs
                for k in i
                    pos = k.pos + 1
                    AA_sub = "$(k.cdsName):$(k.refAa)$(string(pos))$(k.qryAa)"
                    if !(k.refAa == "-") && !(AA_sub in excluded_AA)
                        get!(seq_AA_muts, name, Set{String}() )
                        push!(seq_AA_muts[name], AA_sub)
    #                    if !(k.qryAa ≠ "-")
    #                        get!(seq_AA_muts_no_dels, name, Set{String}() )
    #                        push!(seq_AA_muts_no_dels[name], AA_sub)
    #                    end
    #######################################################################################################
    #                    get!(date_AA_mut_ct, date_index, Dict{String, Int64}() )
    #                    date_AA_mut_ct[date_index][AA_sub] = get(date_AA_mut_ct[date_index], AA_sub, 0) + 1
    #######################################################################################################
                        get!(AA_muts_seq, AA_sub, Set{String}() )
                        push!(AA_muts_seq[AA_sub], name)
                    end
    #               AA_sub_pos = "$(k.cdsName):$(string(pos))"
    #               if !(k.refAa == "-") && !(AA_sub in excluded_AA)
    #                   get!(seq_AA_muts_pos_only, AA_sub_pos, Set{String}() )
    #                   push!(seq_AA_muts_pos_only[name], AA_sub_pos)
    #                   get!(AA_muts_seq_pos_only, AA_sub_pos, Set{String}() )
    #                   push!(AA_muts_seq_pos_only[AA_sub_pos], name)
    ##############################################################################################################
    #                   if !(k.qryAa ≠ "-")
    #                       get!(seq_AA_muts_pos_only_no_dels, name)
    #                       push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
    #                       get!(AA_muts_seq_pos_only_no_dels, AA_sub_pos, Set{String}() )
    #                       push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
    #                   end
    #               end
                end 
            end
            gene_AA_subs_WT = j.aaSubstitutions
            for i in gene_AA_subs_WT
                pos = i.pos + 1
                AA_sub = "$(i.cdsName):$(i.refAa)$(string(pos))$(i.qryAa)"
                if !(i.refAa == "-") && !(AA_sub in excluded_AA)
                    get!(seq_AA_muts_WT, name, Set{String}() )
                    push!(seq_AA_muts_WT[name], AA_sub)
                    get!(AA_muts_seq_WT, AA_sub, Set{String}() )
                end
    #           AA_sub_pos = i.cdsName * ":" * string(pos)
    #           if !(i.refAa == "-") && !(AA_sub in excluded_AA)
    #               get!(seq_AA_muts_WT_pos_only, name, Set{String}() )
    #               push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
    #               get!(AA_muts_seq_WT_pos_only, AA_sub_pos, Set{String}() )
    #               push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
    #           end
            end
    ###################################################
            for i in gene_AA_del_WT
                jeen = i.cdsName
                refAa = i.refAa
                pos = string(i.pos + 1)
                AA_del_WT = "$(jeen):$(refAa)$(pos)-"
                get!(seq_AA_dels_WT, name, Set{String}() )
                push!(seq_AA_dels_WT[name], AA_del_WT)
                get!(AA_dels_seq_WT, AA_del_WT, Set{String}() )
                push!(AA_dels_seq_WT[AA_del_WT], name)
                get!(seq_AA_muts_WT, name, Set{String}() )
                push!(seq_AA_muts_WT[name], AA_del_WT)
                get!(AA_muts_seq_WT, AA_del_WT, Set{String}() )
                push!(AA_muts_seq_WT[AA_del_WT], name)
            end
    ##############################################################################################################
            for i in genes
                gene_AA_del_ranges = privAA[i].privateDeletionRanges
                for h in gene_AA_del_ranges
                    beg = h.range[:begin] + 1
                    fin = h.range[:end]
                    AA_ref_beg = ""
                    AA_ref_fin = ""
                    AA_del_res = privAA[i].privateDeletions
                    for n in AA_del_res
                        if n.pos == beg - 1
                           AA_ref_beg = n.refAa
                        end
                        if n.pos == fin
                            AA_ref_fin = n.refAa
                        end
                    end
                    beg_str = string(beg)
                    fin_str = string(fin)
                    AA_del_range = "$(i):$(beg_str)-$(fin_str)"
                    get!(seq_AA_dels, name, Set{String}() )
                    push!(seq_AA_dels[name], AA_del_range)
                    get!(AA_dels_seq, AA_del_range, Set{String}() )
                    push!(AA_dels_seq[AA_del_range], name)
                end
                AA_dels = privAA[i].privateDeletions
                for h in AA_dels
                    pos = h.pos + 1
                    del_AA = "$(i):$(h.refAa)$(string(pos))-"
                    get!(seq_AA_muts, name, Set{String}() )
                    push!(seq_AA_muts[name], del_AA)
                    get!(AA_muts_seq, del_AA, Set{String}() )
                    push!(AA_muts_seq[del_AA], name)
                end
            end
    #####################################################################################################################################
    #4 unknownAaRanges
    #         Type: Vector{Any}                                     ### = Array of dictionaries
    #          j.unknownAaRanges  j.unknownAaRanges
    #               j.unknownAaRanges.1  j.unknownAaRanges.1  ### = Dictionary, each # for 1 unknown AA range
    #               j.unknownAaRanges.2
    #               j.unknownAaRanges.3 (...etc)
    #                   j.unknownAaRanges.1.length
    #                   j.unknownAaRanges.1.character
    #                   j.unknownAaRanges.1.ranges           ### = Array of Dictionaries
    #                   j.unknownAaRanges.1.cdsName       
    #                       j.unknownAaRanges.1.ranges.1    ### = Dictionary (2 keys)
    #                       j.unknownAaRanges.1.ranges.2
    #                       j.unknownAaRanges.1.ranges.3   (...etc)
    #                            j.unknownAaRanges.1.ranges.1.character   
    #                            j.unknownAaRanges.1.ranges.1.range       ### = Dictionary (2 keys)
    #                                 j.unknownAaRanges.1.ranges.1.range.begin
    #                                 j.unknownAaRanges.1.ranges.1.range.end
    #####################################################################################################################################                    for i in j.unknownAaRanges
            unkn_AA_rng = j.unknownAaRanges
            for p in unkn_AA_rng
                unkn_gene = p.cdsName
                ranges = p.ranges
                for q in ranges
                    bgin = q.range[:begin] + 1
                    nd = q.range[:end]
                    for res in bgin:nd
                        res_str = string(res)
                        get!(seq_unknown_AA, name, Set{String}() )
                        push!(seq_unknown_AA[name], "$(unkn_gene):$(res_str)")
                        get!(seq_unknown_AA_ranges, name, Set{String}() )
                        push!(seq_unknown_AA_ranges[name], "$(unkn_gene):$(string(bgin))-$(string(nd))")
                    end
                end
            end
            if !haskey(seq_unknown_AA_ranges, name)
                seq_unknown_AA_ranges[name] = Set{String}()
            end
    #####################################################################################################################################
        #10 missing
    #        j["missing"][1]["character"]
    #        j["missing"][1]["range"]
    #        j["missing"][1]["range"]["begin"]
    #        j["missing"][1]["range"]["end"]
    #####################################################################################################################################
            for i in j.missing
                bgin = i.range[:begin] + 1
                nd = i.range[:end]
                for nuc_site in bgin:nd
                    get!(seq_nuc_dropout, name, Set{Int}() )
                    push!(seq_nuc_dropout[name], nuc_site)
                end
            end
    ############################################################################
            ref_seq_arr = Vector{String}()
            for i in 1:length(ref_seq)
                q = string(ref_seq[i])
                push!(ref_seq_arr, q)
            end
            mixed_nuc_vec = Vector{String}()
            for i in j.nonACGTNs
                mixed_nuc = i.character
                bgin = i.range[:begin] + 1
                nd = i.range[:end]
                if bgin > 20
                    for b in bgin:nd
                        mixed_mut = "$(ref_seq_arr[b])$(string(b))$(mixed_nuc)"
                        get!(seq_mixed_nucs, name, Set{String}() )
                        push!(seq_mixed_nucs[name], mixed_mut)
                        push!(mixed_nuc_vec, mixed_mut)
                    end
                end
            end
            mixed_nuc_vec_string = join(mixed_nuc_vec, ", ")
            if !isempty(mixed_nuc_vec)
                syn_nuc_to_AA_dict_sort, nonsyn_nuc_to_AA_dict_sort, all_nuc_to_AA_dict_sort = mixed_muts_to_AA(seq_pango[name], mixed_nuc_vec_string)
                mixed_AA_set = Set{String}()
                for nuc__AA in nonsyn_nuc_to_AA_dict_sort
                    mixy_AA = nuc__AA[2]
                    push!(mixed_AA_set, mixy_AA)
                end
                seq_mixed_AA_muts[name] = mixed_AA_set
            else
                seq_mixed_AA_muts[name] = Set{String}()
            end
    ################################################## END END END END END END ##########################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    #####################################################################################################################################
    ################################################## END END END END END END ##########################################################
    ## Below was to check for sequences in groups that were disqualified. They were put in an "exceptions" list. 
    #        for seq_epi in stringlist_to_strings_set("") 
    #            if seq_epi in all_nonqualifying_seqs_set
    #                if !(seq_epi in already_printed)
    #                    println(seq_epi)
    #                end
    #                push!(already_printed, seq_epi)
    #            end
    #        end
    ######################################################################################################################################
    ### Since there is——vexingly——no guide to what an ndjson file actually contains or what form it takes——I've had to spend 
    ###     dozens of hours documenting said info and forms and have occasionally inserted it here so I don't have to find & 
    ###        consult my text file every time I want to make a change.
    #44 privateNucMutations
    #          Type: Dict{String, Any}
    #      j.privateNucMutations
    #           j.privateNucMutations.totalLabeledSubstitutions
    #           j.privateNucMutations.totalUnlabeledSubstitutions
    #           j.privateNucMutations.totalReversionSubstitutions
    #           j.privateNucMutations.totalPrivateDeletions
    #           j.privateNucMutations.totalPrivateSubstitutions
    #           j.privateNucMutations.labeledSubstitutions
    #                j.privateNucMutations.labeledSubstitutions.1
    #                     j.privateNucMutations.labeledSubstitutions.1.substitution
    #                          j.privateNucMutations.labeledSubstitutions.1.substitution.refNuc
    #                          j.privateNucMutations.labeledSubstitutions.1.substitution.pos
    #                          j.privateNucMutations.labeledSubstitutions.1.substitution.qryNuc
    #                     j.privateNucMutations.labeledSubstitutions.1.labels  eg ["21F
    #           j.privateNucMutations.unlabeledSubstitutions
    #                j.privateNucMutations.unlabeledSubstitutions.1
    #                     j.privateNucMutations.unlabeledSubstitutions.1.refNuc
    #                     j.privateNucMutations.unlabeledSubstitutions.1.pos
    #                     j.privateNucMutations.unlabeledSubstitutions.1.qryNuc
    #           j.privateNucMutations.reversionSubstitutions
    #                j.privateNucMutations.reversionSubstitutions.1
    #                     j.privateNucMutations.reversionSubstitutions.1.refNuc
    #                     j.privateNucMutations.reversionSubstitutions.1.pos
    #                     j.privateNucMutations.reversionSubstitutions.1.qryNuc
    #           j.privateNucMutations.privateDeletionRanges
    #                j.privateNucMutations.privateDeletionRanges.1
    #                j.privateNucMutations.privateDeletionRanges.2  (...etc)
    #                     j.privateNucMutations.privateDeletionRanges.1
    #                         j.privateNucMutations.privateDeletionRanges.1.range.begin
    #                         j.privateNucMutations.privateDeletionRanges.1.range.end
    #           j.privateNucMutations.privateDeletions
    #                j.privateNucMutations.privateDeletions.1
    #                j.privateNucMutations.privateDeletions.2 ...etc
    #                     j.privateNucMutations.privateDeletions.1.refNuc
    #                     j.privateNucMutations.privateDeletions.1.pos
    #           j.privateNucMutations.privateSubstitutions
    #                j.privateNucMutations.privateSubstitutions.1
    #                j.privateNucMutations.privateSubstitutions.2 (...etc)
    #                     j.privateNucMutations.privateSubstitutions.1.refNuc
    #                     j.privateNucMutations.privateSubstitutions.1.pos
    #                     j.privateNucMutations.privateSubstitutions.1.qryNuc
    ######################################################################################################################################
            if j.privateNucMutations.totalReversionSubstitutions ≤ revs_thresh || name in legends || name in other_exceptions
                if !(name in excluded_seqs) || name in other_exceptions
    #                if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions ≥ abs_min_mut_thresh
                    if total_AA_subs_plus_del_ranges_minus_reversions ≥ abs_min_mut_thresh || name in other_exceptions
                        seq_ct_by_year[seq_year] += 1
                        seq_ct_by_year_month[(seq_year, seq_month)] += 1
                        seq_ct_by_year_month_day[(seq_year, seq_month, seq_day)] += 1
                        all_qualifying_seqs_ct += 1
                        push!(all_qualifying_seqs, name)
                        push!(all_qualifying_seqs_set, name)
    #                    for k in subs
    #                        pos = k.pos + 1
    #                        if !(pos in excluded_pos) && !(k.refNuc == "-")
    #                            total_subs += 1
    #                            nuc_sub = "$(k.refNuc)$(string(pos))$(k.qryNuc)"
    #                            get!(seq_nuc_muts, name, Set{String}() )
    #                            push!(seq_nuc_muts[name], nuc_sub)
    #                            get!(date_nuc_mut_ct, date_index, Dict{String, Int64}() )                            
    #                            date_nuc_mut_ct[date_index][nuc_sub] = get(date_nuc_mut_ct[date_index], nuc_sub, 0) + 1
    #                            get!(nuc_muts_seq, nuc_sub, Set{String}() )
    #                            push!(nuc_muts_seq[nuc_sub], name)
    #*#                          if k.qryNuc ≠ "-"                          
    #*#                              get!(nuc_muts_seq_no_dels, nuc_sub, Set{String}() )
    #*#                              push!(nuc_muts_seq_no_dels[nuc_sub], name)
    #*#                          end
    #                        end
    #                    end
    #                    for k in subs_WT
    #                        pos = k.pos + 1
    #                        if !(pos in excluded_pos)
    #                            nuc_sub = "$(k.refNuc)$(string(pos))$(k.qryNuc)"
    #                            get!(seq_nuc_muts_WT, name, Set{String}() )
    #                            push!(seq_nuc_muts_WT[name], nuc_sub)
    #                            get!(nuc_muts_seq_WT, nuc_sub, Set{String}() )
    #                            push!(nuc_muts_seq_WT[nuc_sub], name)
    #                        end
    #                    end
    #                    for d in nuc_dels
    #                        pos = d.pos + 1
    #                        if !(pos in excluded_pos)
    #                            total_nuc_dels += 1
    #                            n_del = "$(d.refNuc)$(string(pos))-"
    #                            get!(seq_nuc_muts, name, Set{String}() )
    #                            push!(seq_nuc_muts[name], n_del)
    #                            get!(nuc_muts_seq, n_del, Set{String}() )
    #                            push!(nuc_muts_seq[n_del], name)
    #                            get!(date_nuc_mut_ct, date_index, Dict{String, Int64}() )
    #                            date_nuc_mut_ct[date_index][n_del] = get(date_nuc_mut_ct[date_index], n_del, 0) + 1
    #                        end
    #                    end
    #                    for d in dels
    #                        del_start = d.range[:begin] + 1
    #                        del_end = d.range[:end]
    #                        del_start_str = string(del_start)
    #                        del_end_str = string(del_end)
    #                        del_str = "$(del_start_str)-$(del_end_str)"
    #                        push!(nuc_del_ranges, del_str)
    #                    end
    #                    for del in nuc_del_ranges
    #                        get!(seq_nuc_del_ranges, name, Set{String}() )
    #                        push!(seq_nuc_del_ranges[name], del)
    #                        get!(nuc_dels_seq, del, Set{String}() )
    #                        push!(nuc_dels_seq[del], name)
    #                    end
    #                    for d in nuc_dels_WT
    #                        del_start = d.range[:begin] + 1
    #                        del_end = d.range[:end]
    #                        del_start_str = string(del_start)
    #                        del_end_str = string(del_end)
    #                        del_str = "$(del_start_str)-$(del_end_str)"
    #                        push!(WT_nuc_del_ranges, del_str)
    #                    end
    #                    for del in WT_nuc_del_ranges
    #                        get!(seq_nuc_del_ranges_WT, name, Set{String}() )
    #                        push!(seq_nuc_del_ranges_WT[name], del)
    #                        get!(nuc_dels_seq_WT, del, Set{String}() )
    #                        push!(nuc_dels_seq_WT[del], name)
    #                    end
    
    #                    for i in gene_AA_subs
    #                        for k in i
    #                            pos = k.pos + 1
    #                            AA_sub = "$(k.cdsName):$(k.refAa)$(string(pos))$(k.qryAa)"
    #                            if !(k.refAa == "-") && !(AA_sub in excluded_AA)
    #                                get!(seq_AA_muts, name, Set{String}() )
    #                                push!(seq_AA_muts[name], AA_sub)
    #                                if !(k.qryAa ≠ "-")
    #                                    get!(seq_AA_muts_no_dels, name, Set{String}() )
    #                                    push!(seq_AA_muts_no_dels[name], AA_sub)
    #                                end
    #                                AA_muts_ct[AA_sub] = get(AA_muts_ct, AA_sub, 0) + 1
    #######################################################################################################
    #                                get!(date_AA_mut_ct, date_index, Dict{String, Int64}() )
    #                                date_AA_mut_ct[date_index][AA_sub] = get(date_AA_mut_ct[date_index], AA_sub, 0) + 1
    #######################################################################################################
    #                                get!(AA_muts_seq, AA_sub, Set{String}() )
    #                                push!(AA_muts_seq[AA_sub], name)
    #                            end
    #                            AA_sub_pos = "$(k.cdsName):$(string(pos))"
    #                            if !(k.refAa == "-") && !(AA_sub in excluded_AA)
    #                                get!(seq_AA_muts_pos_only, AA_sub_pos, Set{String}() )
    #                                push!(seq_AA_muts_pos_only[name], AA_sub_pos)
    #                                get!(AA_muts_seq_pos_only, AA_sub_pos, Set{String}() )
    #                                push!(AA_muts_seq_pos_only[AA_sub_pos], name)
    ##############################################################################################################
    #                                if !(k.qryAa ≠ "-")
    #                                    get!(seq_AA_muts_pos_only_no_dels, name)
    #                                    push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
    #                                    get!(AA_muts_seq_pos_only_no_dels, AA_sub_pos, Set{String}() )
    #                                    push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
    #                                end
    #                            end
    #                        end 
    #                    end
    #####################################################################################################################################
    #    27 aaSubstitutions
    #     Type: Vector{Any}
    #       j["aaSubstitutions"]
    #       j["aaSubstitutions"][1]
    #       j["aaSubstitutions"][2]
    #       j["aaSubstitutions"][3]  (...etc, very long list)
    #       j["aaSubstitutions"][1]["refAa"]
    #       j["aaSubstitutions"][1]["cdsName"]
    #       j["aaSubstitutions"][1]["pos"]
    #       j["aaSubstitutions"][1]["qryAa"]
    #####################################################################################################################################
    #                    gene_AA_subs_WT = j.aaSubstitutions
    #                    for i in gene_AA_subs_WT
    #                        pos = i.pos + 1
    #                        AA_sub = "$(i.cdsName):$(i.refAa)$(string(pos))$(i.qryAa)"
    #                        if !(i.refAa == "-") && !(AA_sub in excluded_AA)
    #                            get!(seq_AA_muts_WT, name, Set{String}() )
    #                            push!(seq_AA_muts_WT[name], AA_sub)
    #                            get!(AA_muts_seq_WT, AA_sub, Set{String}() )
    #                        end
    #                        AA_sub_pos = i.cdsName * ":" * string(pos)
    #                        if !(i.refAa == "-") && !(AA_sub in excluded_AA)
    #                            get!(seq_AA_muts_WT_pos_only, name, Set{String}() )
    #                            push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
    #                            get!(AA_muts_seq_WT_pos_only, AA_sub_pos, Set{String}() )
    #                            push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
    #                        end
    #                    end
    #####################################################################################################################################
        #46 aaDeletions
    #         Type: Vector{Any}
    #            j["aaDeletions"]
    #            j["aaDeletions"][1]
    #            j["aaDeletions"][2]
    #            j["aaDeletions"][3]  (...etc)
    #            j["aaDeletions"][1]["refAa"]
    #            j["aaDeletions"][1]["cdsName"]
    #            j["aaDeletions"][1]["pos"]
    #####################################################################################################################################
    #                    for i in gene_AA_del_WT
    #                        jeen = i.cdsName
    #                        refAa = i.refAa
    #                        pos = string(i.pos + 1)
    #                        AA_del_WT = "$(jeen):$(refAa)$(pos)-"
    #                        get!(seq_AA_dels_WT, name, Set{String}() )
    #                        push!(seq_AA_dels_WT[name], AA_del_WT)
    #                        get!(AA_dels_seq_WT, AA_del_WT, Set{String}() )
    #                        push!(AA_dels_seq_WT[AA_del_WT], name)
    #                        get!(seq_AA_muts_WT, name, Set{String}() )
    #                        push!(seq_AA_muts_WT[name], AA_del_WT)
    #                        get!(AA_muts_seq_WT, AA_del_WT, Set{String}() )
    #                        push!(AA_muts_seq_WT[AA_del_WT], name)
    #                    end
    ##############################################################################################################
    #                    for i in genes
    #                        gene_AA_del_ranges = privAA[i].privateDeletionRanges
    #                        for h in gene_AA_del_ranges
    #                            beg = h.range[:begin] + 1
    #                            fin = h.range[:end]
    #                            AA_ref_beg = ""
    #                            AA_ref_fin = ""
    #                            AA_del_res = privAA[i].privateDeletions
    #                            for n in AA_del_res
    #                                if n.pos == beg - 1
    #                                   AA_ref_beg = n.refAa
    #                                end
    #                                if n.pos == fin
    #                                    AA_ref_fin = n.refAa
    #                                end
    #                            end
    #                            beg_str = string(beg)
    #                            fin_str = string(fin)
    #                            AA_del_range = "$(i):$(beg_str)-$(fin_str)"
    #                            get!(seq_AA_dels, name, Set{String}() )
    #                            push!(seq_AA_dels[name], AA_del_range)
    #                            get!(AA_dels_seq, AA_del_range, Set{String}() )
    #                            push!(AA_dels_seq[AA_del_range], name)
    #                        end
    #                        AA_dels = privAA[i].privateDeletions
    #                        for h in AA_dels
    #                            pos = h.pos + 1
    #                            del_AA = "$(i):$(h.refAa)$(string(pos))-"
    #                            get!(seq_AA_muts, name, Set{String}() )
    #                            push!(seq_AA_muts[name], del_AA)
    #                            get!(AA_muts_seq, del_AA, Set{String}() )
    #                            push!(AA_muts_seq[del_AA], name)
    #                        end
    #####################################################################################################################################
        #4 unknownAaRanges
    #          Type: Vector{Any}                                   #    ## = Array of dictionaries
    #              j["unknownAaRanges"]
    #              j["unknownAaRanges"][1]                         #    ## = Dictionary, each # for 1 unknown AA range
    #              j["unknownAaRanges"][2]
    #              j["unknownAaRanges"][3] (...etc)
    #              j["unknownAaRanges"][1]["length"]
    #              j["unknownAaRanges"][1]["character"]
    #              j["unknownAaRanges"][1]["ranges"]               #    ## = Array of Dictionaries
    #              j["unknownAaRanges"][1]["cdsName"]       
    #              j["unknownAaRanges"][1]["ranges"][1]            #    ## = Dictionary (2 keys)
    #              j["unknownAaRanges"][1]["ranges"][2]
    #              j["unknownAaRanges"][1]["ranges"][3]   (...etc)
    #              j["unknownAaRanges"][1]["ranges"][1]["character"]   
    #              j["unknownAaRanges"][1]["ranges"][1]["range"]   #    ## = Dictionary (2 keys)
    #              j["unknownAaRanges"][1]["ranges"][1]["range"]["begin"]
    #              j["unknownAaRanges"][1]["ranges"][1]["range"]["end"]
    #####################################################################################################################################                    for i in j.unknownAaRanges
    #                        unkn_gene = i.cdsName
    #                        for p in i.ranges
    #                            bgin = p.range[:begin] + 1
    #                            nd = p.range[:end]
    #                            for res in bgin:nd
    #                                res_str = string(res)
    #                                get!(seq_unknown_AA, name, Set{String}() )
    #                                push!(seq_unknown_AA[name], "$(unkn_gene):$(res_str)")
    #                            end
    #                            get!(seq_unknown_AA_ranges, name, Set{String}() )
    #                            push!(seq_unknown_AA_ranges[name], "$(unkn_gene):$(string(bgin))-$(string(nd))")
    #                        end
    #                    end
    #                    if !haskey(seq_unknown_AA_ranges, name)
    #                        seq_unknown_AA_ranges[name] = Set{String}()
    #                    end
    #####################################################################################################################################
        #10 missing
    #        j["missing"][1]["character"]
    #        j["missing"][1]["range"]
    #        j["missing"][1]["range"]["begin"]
    #        j["missing"][1]["range"]["end"]
    #####################################################################################################################################
    #                    for i in j.missing
    #                        bgin = i.range[:begin] + 1
    #                        nd = i.range[:end]
    #                        for nuc_site in bgin:nd
    #                            get!(seq_nuc_dropout, name, Set{Int}() )
    #                            push!(seq_nuc_dropout[name], nuc_site)
    #                        end
    #                    end
    ############################################################################
    #                    ref_seq_arr = Vector{String}()
    #                    for i in 1:length(ref_seq)
    #                        q = string(ref_seq[i])
    #                        push!(ref_seq_arr, q)
    #                    end
    #                    mixed_nuc_vec = Vector{String}()
    #                    for i in j.nonACGTNs
    #                        mixed_nuc = i.character
    #                        bgin = i.range[:begin] + 1
    #                        nd = i.range[:end]
    #                        if bgin > 20
    #                            for b in bgin:nd
    #                                mixed_mut = "$(ref_seq_arr[b])$(string(b))$(mixed_nuc)"
    #                                get!(seq_mixed_nucs, name, Set{String}() )
    #                                push!(seq_mixed_nucs[name], mixed_mut)
    #                                push!(mixed_nuc_vec, mixed_mut)
    #                            end
    #                        end
    #                    end
    #                    mixed_nuc_vec_string = join(mixed_nuc_vec, ", ")
    #                    if !isempty(mixed_nuc_vec)
    #                        mixed_AA_set, mixed_AA_sort = mixed_muts_to_AA(seq_pango[name], mixed_nuc_vec)
    #                        seq_mixed_AA_muts[name] = mixed_AA_set
    #                    else
    #                        seq_mixed_AA_muts[name] = Set{String}()
    #                    end
                    end
                end
            end
        end
    end
################################################################################################################################
    for (name, mutset) in seq_nuc_muts
        for nuc_sub in mutset
            if nuc_sub[end] ≠ '-'
                get!(seq_nuc_muts_no_dels, name, Set{String}() )
                push!(seq_nuc_muts_no_dels[name], nuc_sub)
            end
        end
    end
    for (name, mutset) in seq_AA_muts
    seq_AA_muts_pos_only[name] = Set{String}()
    seq_AA_muts_no_dels[name] = Set{String}()
    seq_AA_muts_pos_only_no_dels[name] = Set{String}()
        for AA_sub in mutset
            AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
            push!(seq_AA_muts_pos_only[name], AA_sub_pos)
            if AA_sub[end] ≠ '-'
                total_chr_AA_subs += 1
                push!(seq_AA_muts_no_dels[name], AA_sub)
                push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
            end
        end
    end
    for (AA_sub, name_set) in AA_muts_seq
        AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
        for name in name_set
            get!(AA_muts_seq_pos_only, AA_sub_pos, Set{String}() )
            push!(AA_muts_seq_pos_only[AA_sub_pos], name)
            if AA_sub[end] ≠ '-'
                get!(AA_muts_seq_pos_only_no_dels, AA_sub_pos, Set{String}() )
                push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
            end
        end
    end
    for (name, mutset) in seq_AA_muts_WT
        for AA_sub in mutset
            AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
            get!(seq_AA_muts_WT_pos_only, name, Set{String}() )
            push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
        end
    end
    for (AA_sub, name_set) in AA_muts_seq_WT
        AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
        for name in name_set
            get!(AA_muts_seq_WT_pos_only, AA_sub_pos, Set{String}() )
            push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
        end
    end                             
    for seq in all_qualifying_seqs_set
        if !haskey(seq_nuc_dropout, seq)
            seq_nuc_dropout[seq] = Set{Int}()
        end
        if !haskey(seq_mixed_nucs, seq)
            seq_mixed_nucs[seq] = Set{String}()
        end
        if !haskey(seq_unknown_AA, seq)
            seq_unknown_AA[seq] = Set{String}()
        end
        if !haskey(seq_nuc_del_ranges, seq)
            seq_nuc_del_ranges[seq] = Set{String}()
        end
        if !haskey(seq_AA_dels, seq)
            seq_AA_dels[seq] = Set{String}()
        end
        if !haskey(seq_nuc_del_ranges_WT, seq)
            seq_nuc_del_ranges_WT[seq] = Set{String}()
        end
        if !haskey(seq_AA_dels_WT, seq)
            seq_AA_dels_WT[seq] = Set{String}()
        end
    end
####################################################################################################################################
####################################################################################################################################
### force_groups = groups of sequences that are definitely related but may need to be forced into a group. 

       
    ### Add These to force_groups on next EPCI update: 
#     Set(["EPI_ISL_15492887", "EPI_ISL_13244707", "EPI_ISL_11750262"]), 
#     Set(["EPI_ISL_20040623", "EPI_ISL_20040387", "EPI_ISL_19948061", "EPI_ISL_19946695", "EPI_ISL_20030503", "EPI_ISL_20027365", "EPI_ISL_20027953"]),  
    
force_groups = [
    Set(["EPI_ISL_17437940", "EPI_ISL_20016121"]),
    Set(["EPI_ISL_13051970", "EPI_ISL_13052204", "EPI_ISL_13051740"]),
    Set(["EPI_ISL_17664370", "EPI_ISL_17789808"]),
    Set(["EPI_ISL_19267631", "EPI_ISL_19300385", "EPI_ISL_19415183", "EPI_ISL_19415333"]),
    Set(["EPI_ISL_7473129", "EPI_ISL_7473130", "EPI_ISL_7473131", "EPI_ISL_7473132", "EPI_ISL_7473133", "EPI_ISL_7473134", "EPI_ISL_7473135", "EPI_ISL_7473136", "EPI_ISL_7473137", "EPI_ISL_7473138", "EPI_ISL_7473139", "EPI_ISL_7473140", "EPI_ISL_7473141", "EPI_ISL_7473142", "EPI_ISL_7473143", "EPI_ISL_7473144"]),
    Set(["EPI_ISL_14922929", "EPI_ISL_15177334"]),
    Set(["EPI_ISL_13830121", "EPI_ISL_13830175", "EPI_ISL_14152696", "EPI_ISL_14601182", "EPI_ISL_14835057", "EPI_ISL_14846826", "EPI_ISL_14984311", "EPI_ISL_15076071", "EPI_ISL_15259998", "EPI_ISL_15415649", "EPI_ISL_15721045", "EPI_ISL_15866887", "EPI_ISL_16706080"]), 
    Set(["EPI_ISL_602912", "EPI_ISL_2397307", "EPI_ISL_2397308", "EPI_ISL_2397309", "EPI_ISL_2397310", "EPI_ISL_2397311", "EPI_ISL_2397312", "EPI_ISL_2397313", "EPI_ISL_14666760", "EPI_ISL_15292331", "EPI_ISL_15541751", "EPI_ISL_18127203", "EPI_ISL_18876158", "EPI_ISL_18876159", "EPI_ISL_18876160", "EPI_ISL_18876161", "EPI_ISL_18876171", "EPI_ISL_18876178"]),
    Set(["EPI_ISL_19032268", "EPI_ISL_19032269", "EPI_ISL_19032270", "EPI_ISL_19032271", "EPI_ISL_19032272", "EPI_ISL_19032273", "EPI_ISL_19032274", "EPI_ISL_19032275", "EPI_ISL_19032276", "EPI_ISL_19032277", "EPI_ISL_19032278", "EPI_ISL_19032279", "EPI_ISL_19032280", "EPI_ISL_19032281", "EPI_ISL_19032282", "EPI_ISL_19032283", "EPI_ISL_19032284", "EPI_ISL_20231309", "EPI_ISL_20231310", "EPI_ISL_20231311", "EPI_ISL_20231312", "EPI_ISL_20231313", "EPI_ISL_20231314", "EPI_ISL_20231315", "EPI_ISL_20231316"]),
    Set(["EPI_ISL_13299122", "EPI_ISL_13203425", "EPI_ISL_13299118", "EPI_ISL_13299108", "EPI_ISL_13301357", "EPI_ISL_13301368"]), 
    Set(["EPI_ISL_11356267", "EPI_ISL_12063600", "EPI_ISL_11356269", "EPI_ISL_12063601", "EPI_ISL_12063602", "EPI_ISL_12063598", "EPI_ISL_12808264", "EPI_ISL_12430022", "EPI_ISL_13354243", "EPI_ISL_12656485", "EPI_ISL_13172328", "EPI_ISL_13172329", "EPI_ISL_14243503", "EPI_ISL_14243471", "EPI_ISL_15096672", "EPI_ISL_15093817", "EPI_ISL_15093818"]),
    Set(["EPI_ISL_13893481", "EPI_ISL_14700285", "EPI_ISL_15109913", "EPI_ISL_15494897", "EPI_ISL_15837686"]),
    Set(["EPI_ISL_6397259", "EPI_ISL_8675368"]),
    Set(["EPI_ISL_13002811", "EPI_ISL_13002153", "EPI_ISL_13002802"]),
    Set(["EPI_ISL_19044119", "EPI_ISL_19109579", "EPI_ISL_19141756", "EPI_ISL_19557993", "EPI_ISL_19588050", "EPI_ISL_19613618"]), 
    Set(["EPI_ISL_16077353", "EPI_ISL_16181797", "EPI_ISL_16287690", "EPI_ISL_16528903", "EPI_ISL_17252934", "EPI_ISL_17387122", "EPI_ISL_17391460", "EPI_ISL_17441169", "EPI_ISL_17441208", "EPI_ISL_17612398", "EPI_ISL_17837432", "EPI_ISL_18051918", "EPI_ISL_18162567"]),
    Set(["EPI_ISL_19116728", "EPI_ISL_15405688"]),
    Set(["EPI_ISL_2228938", "EPI_ISL_2983871", "EPI_ISL_5692576", "EPI_ISL_5692832", "EPI_ISL_5693164"]),
    Set(["EPI_ISL_15424276", "EPI_ISL_16045410", "EPI_ISL_16045424"]),
    Set(["EPI_ISL_17976105", "EPI_ISL_17976114", "EPI_ISL_18044759", "EPI_ISL_18282077"]),
    Set(["EPI_ISL_6757093", "EPI_ISL_9869512", "EPI_ISL_12646116"]),
    Set(["EPI_ISL_14773345", "EPI_ISL_14995884", "EPI_ISL_15191475", "EPI_ISL_15349852", "EPI_ISL_18998583"]),
    Set(["EPI_ISL_15331994", "EPI_ISL_16231330", "EPI_ISL_16231873", "EPI_ISL_16479826"]),
    Set(["EPI_ISL_12501519", "EPI_ISL_18930681", "EPI_ISL_18966035", "EPI_ISL_19182893", "EPI_ISL_19217899", "EPI_ISL_19269083", "EPI_ISL_19348673"]),
    Set(["EPI_ISL_17190813", "EPI_ISL_17535979", "EPI_ISL_18147456"]),
    Set(["EPI_ISL_11621883", "EPI_ISL_13131091", "EPI_ISL_14561487", "EPI_ISL_14710834"]),
    Set(["EPI_ISL_14616543", "EPI_ISL_14616443", "EPI_ISL_14707948", "EPI_ISL_14766433"]),
    Set(["EPI_ISL_14260215", "EPI_ISL_14817985", "EPI_ISL_15507616"]),
    Set(["EPI_ISL_12535815", "EPI_ISL_12741485", "EPI_ISL_12742222", "EPI_ISL_13073691", "EPI_ISL_13605240", "EPI_ISL_14959923", "EPI_ISL_15169791", "EPI_ISL_15171102"]),
    Set(["EPI_ISL_12148503", "EPI_ISL_12146396", "EPI_ISL_12147521", "EPI_ISL_12146579", "EPI_ISL_14439649"]),
    Set(["EPI_ISL_10876034", "EPI_ISL_11295642"]), 
    Set(["EPI_ISL_13358809", "EPI_ISL_15887656"]), 
    Set(["EPI_ISL_15420631", "EPI_ISL_16413336"]), 
    Set(["EPI_ISL_19300400", "EPI_ISL_18990023", "EPI_ISL_19415272", "EPI_ISL_19415273"]), 
    Set(["EPI_ISL_19539586", "EPI_ISL_19539602"]), 
    Set(["EPI_ISL_17817657", "EPI_ISL_18042110"]), 
    Set(["EPI_ISL_1470396", "EPI_ISL_7711813", "EPI_ISL_12001179", "EPI_ISL_12001180"]), 
    Set(["EPI_ISL_19193606", "EPI_ISL_19193617", "EPI_ISL_19032056"]), 
    Set(["EPI_ISL_15436494", "EPI_ISL_16967082", "EPI_ISL_16967083", "EPI_ISL_17695348", "EPI_ISL_18345777", "EPI_ISL_18563181", "EPI_ISL_18814273", "EPI_ISL_18814272", "EPI_ISL_18814271", "EPI_ISL_19271166", "EPI_ISL_19271182"]), 
    Set(["EPI_ISL_2731960", "EPI_ISL_1048105", "EPI_ISL_1048102", "EPI_ISL_2732319", "EPI_ISL_1048104"]),
    Set(["EPI_ISL_6698637", "EPI_ISL_7205152", "EPI_ISL_7876524", "EPI_ISL_17276098", "EPI_ISL_15473942", "EPI_ISL_17658574"]),
    Set(["EPI_ISL_12639714", "EPI_ISL_12639788", "EPI_ISL_12926955", "EPI_ISL_13788917", "EPI_ISL_14292645", "EPI_ISL_14292615", "EPI_ISL_14292727", "EPI_ISL_14292796"]),
    Set(["EPI_ISL_17587657", "EPI_ISL_13830195", "EPI_ISL_14487315", "EPI_ISL_14847727"]),
    Set(["EPI_ISL_2281463", "EPI_ISL_979349"]),
    Set(["EPI_ISL_16428100", "EPI_ISL_16428101", "EPI_ISL_16428102", "EPI_ISL_16391752"]),
    Set(["EPI_ISL_13986497", "EPI_ISL_14175103", "EPI_ISL_18030395", "EPI_ISL_18876167", "EPI_ISL_12268492", "EPI_ISL_14599778", "EPI_ISL_14599779", "EPI_ISL_14599780", "EPI_ISL_17284794", "EPI_ISL_17284795", "EPI_ISL_17284796", "EPI_ISL_18876174", "EPI_ISL_18876176"]),
    Set(["EPI_ISL_18947086", "EPI_ISL_17276962", "EPI_ISL_17510495", "EPI_ISL_17686694", "EPI_ISL_17686736", "EPI_ISL_18029979", "EPI_ISL_18306254", "EPI_ISL_18947086", "EPI_ISL_18128931"]),
    Set(["EPI_ISL_15338015", "EPI_ISL_15463120", "EPI_ISL_16102479", "EPI_ISL_16102480", "EPI_ISL_16607452", "EPI_ISL_16688713"]),
    Set(["EPI_ISL_14665394", "EPI_ISL_15385232", "EPI_ISL_17201694", "EPI_ISL_16443874", "EPI_ISL_17780724"]),
    Set(["EPI_ISL_18039728", "EPI_ISL_18401313"]), 
    Set(["EPI_ISL_14901672", "EPI_ISL_14937654", "EPI_ISL_14694460"]),
    Set(["EPI_ISL_15911160", "EPI_ISL_16669829"]),
    Set(["EPI_ISL_14462783", "EPI_ISL_15215446", "EPI_ISL_16012424", "EPI_ISL_16863260"]),
    Set(["EPI_ISL_19030129", "EPI_ISL_19234843", "EPI_ISL_19706712"]),
    Set(["EPI_ISL_13011225", "EPI_ISL_13011226", "EPI_ISL_13358893", "EPI_ISL_13358894"]),
    Set(["EPI_ISL_18072343", "EPI_ISL_18225473"]),
    Set(["EPI_ISL_6227208", "EPI_ISL_11816904"]),
    Set(["EPI_ISL_18276415", "EPI_ISL_18414808"]),
    Set(["EPI_ISL_18326806", "EPI_ISL_18326807", "EPI_ISL_18525840"]),
    Set(["EPI_ISL_17669306", "EPI_ISL_19896098", "EPI_ISL_20063868", "EPI_ISL_17669304"]),
    Set(["EPI_ISL_3813731", "EPI_ISL_15568780", "EPI_ISL_15557544"]),
    Set(["EPI_ISL_10712909", "EPI_ISL_11296415", "EPI_ISL_12488441"]),
    Set(["EPI_ISL_528438", "EPI_ISL_686537", "EPI_ISL_2355027", "EPI_ISL_4114033"]),
    Set(["EPI_ISL_6977941", "EPI_ISL_6976497"]),
    Set(["EPI_ISL_539541", "EPI_ISL_539542", "EPI_ISL_8725399", "EPI_ISL_8725400", "EPI_ISL_8725401", "EPI_ISL_8725402", "EPI_ISL_8725403", "EPI_ISL_8725404", "EPI_ISL_8725405", "EPI_ISL_8725406", "EPI_ISL_8725407", "EPI_ISL_8725408", "EPI_ISL_8725409"]),
    Set(["EPI_ISL_593478", "EPI_ISL_593479", "EPI_ISL_593480", "EPI_ISL_593553", "EPI_ISL_593554", "EPI_ISL_593555", "EPI_ISL_593556", "EPI_ISL_593557", "EPI_ISL_593558"]),
    Set(["EPI_ISL_14574572", "EPI_ISL_14832276"]),
    Set(["EPI_ISL_18224410", "EPI_ISL_19173640", "EPI_ISL_19184858", "EPI_ISL_19192806", "EPI_ISL_19850000"]),
    Set(["EPI_ISL_19508878", "EPI_ISL_20066350", "EPI_ISL_20066351", "EPI_ISL_20066352", "EPI_ISL_20066353", "EPI_ISL_20066354", "EPI_ISL_20066355", "EPI_ISL_20066356", "EPI_ISL_20066357", "EPI_ISL_20066358", "EPI_ISL_20066359", "EPI_ISL_20066360"]),
    Set(["EPI_ISL_19912125", "EPI_ISL_19791384"]),
    Set(["EPI_ISL_20000997", "EPI_ISL_19875482", "EPI_ISL_20088904"]),
    Set(["EPI_ISL_15598104", "EPI_ISL_15667047", "EPI_ISL_16706498", "EPI_ISL_17022081", "EPI_ISL_17152522", "EPI_ISL_17370155", "EPI_ISL_18311951"]), 
    Set(["EPI_ISL_16119498", "EPI_ISL_16119508", "EPI_ISL_13830194", "EPI_ISL_16119512", "EPI_ISL_16119517", "EPI_ISL_13984460", "EPI_ISL_16119519", "EPI_ISL_15496641"]), 
    Set(["EPI_ISL_11742572", "EPI_ISL_12701772", "EPI_ISL_12701858", "EPI_ISL_13129387", "EPI_ISL_13242155", "EPI_ISL_13470158", "EPI_ISL_13617475", "EPI_ISL_13740674", "EPI_ISL_14019093", "EPI_ISL_14019109"]),
    Set(["EPI_ISL_11897546", "EPI_ISL_12422504", "EPI_ISL_12896994", "EPI_ISL_12961699"]) ]
#        Set(["", ""]), Belongs in 4th-to-last grp above: "EPI_ISL_508674", "EPI_ISL_949208", "EPI_ISL_8725398", (with EPI_ISL_539542, etc.)
    for i in 1:length(force_groups)
        forced_group = force_groups[i]
        for seq in forced_group
            if haskey(seq_nuc_muts, seq)
                for mut in seq_nuc_muts[seq]
                    push!(rep_seqs, seq)
                    rep_seq_grps_seqs[i] = forced_group
                    rep_seq_grps_muts[i] = Set{String}()
                    rep_seq_grps_muts_no_dels[i] = Set{String}()
                    rep_seq_grps_dels[i] = Set{String}()
                    rep_seq_grps_AA[i] = Set{String}()
                    rep_seq_grps_AA_no_dels[i] = Set{String}()
                    rep_seq_grps_AA_dels[i] = Set{String}()
                    rep_seq_grps_clade[i] = seq_clade[seq]
                    rep_seq_grps_pango[i] = seq_pango[seq]
                    rep_seq_grps_pango_unaliased[i] = seq_pango_unaliased[seq]
                    for m in seq_nuc_muts[seq]
                        push!(rep_seq_grps_muts[i], m)
                        if m[end] ≠ '-'
                            push!(rep_seq_grps_muts_no_dels[i], m)
                            elseif m[end] == '-'
                            push!(rep_seq_grps_dels[i], m)
                        end
                    end
                    for m in seq_AA_muts[seq]
                        push!(rep_seq_grps_AA[i], m)
                        if m[end] ≠ '-'
                            push!(rep_seq_grps_AA_no_dels[i], m)
                            elseif m[end] == '-'
                            push!(rep_seq_grps_AA_dels[i], m)
                        end
                    end
                end
            end
        end
    end
### singlet_exceptions is for sequences that the function groups together but which don't actually belong together (i.e. they seem likely to have come from separate chronic infections)
    singlet_exceptions = Set(["EPI_ISL_18205057"])
### group_exceptions is for rare related sequences that belong together but don't fit criteria (e.g. 1 seq is from different country or US state)
    group_exceptions = Set(["EPI_ISL_14574572", "EPI_ISL_14832276", "EPI_ISL_18889879", "EPI_ISL_18326806", "EPI_ISL_18326807", "EPI_ISL_18525840", "EPI_ISL_14847727", "EPI_ISL_13830195", "EPI_ISL_14487315", "EPI_ISL_18224410", "EPI_ISL_19173640", "EPI_ISL_19184858", "EPI_ISL_20000997", "EPI_ISL_19875482", "EPI_ISL_19791384", "EPI_ISL_19912125", "EPI_ISL_20088904"])
    for n in 1:length(all_qualifying_seqs) - 1
#        yep_already_in_repSeqGrpSeqs::Int = 0
        NthSeq = all_qualifying_seqs[n]
        if !(NthSeq in rep_seqs) && !(NthSeq in singlet_exceptions)
            if haskey(rep_seq_grps_muts, 1) && haskey(rep_seq_grps_seqs, 1)
                keez = keys(rep_seq_grps_muts)
                for i in keys(rep_seq_grps_seqs)
                    Grp_i_MutsSet = rep_seq_grps_muts_no_dels[i]
                    Grp_i_AAMutsSet = rep_seq_grps_AA_no_dels[i]
                    Grp_i_EpisSet = rep_seq_grps_seqs[i]
                    Grp_i_MutsSetNoDels = Set{String}()
                    Grp_i_AAMutsSetNoDels = Set{String}()
                    for grp_i_seq in Grp_i_EpisSet
                        if haskey(seq_nuc_muts_no_dels, grp_i_seq)
                            for mut in seq_nuc_muts_no_dels[grp_i_seq]
                                push!(Grp_i_MutsSetNoDels, mut)
                            end
                            for AA in seq_AA_muts_no_dels[grp_i_seq]
                                push!(Grp_i_AAMutsSetNoDels, AA)
                            end
                        end
                    end
                    firstSeq = first(Grp_i_EpisSet)
                    NthSeqMuts = seq_nuc_muts_no_dels[NthSeq]
                    NthSeqAAMuts = seq_AA_muts_no_dels[NthSeq]
                    NthSeqMuts_Tot = length(NthSeqMuts)
                    NthSeqAAMuts_Tot = length(NthSeqAAMuts)
                    Grp_i_Muts_Tot = length(Grp_i_MutsSetNoDels)
                    Grp_i_AAMuts_Tot = length(Grp_i_AAMutsSetNoDels)
                    GrpSize = length(Grp_i_EpisSet)
                    common = intersect(seq_nuc_muts_no_dels[NthSeq], Grp_i_MutsSetNoDels)
                    commonAA = intersect(seq_AA_muts_no_dels[NthSeq], Grp_i_MutsSetNoDels)
                    sharedMuts_tot = length(common)
                    sharedMuts_NthSeq_prop = sharedMuts_tot/NthSeqMuts_Tot
                    sharedMuts_Grp_prop = sharedMuts_tot/Grp_i_Muts_Tot
                    if GrpSize < 3 && sharedMuts_Grp_prop ≤ 0.3
                        continue
                    end
                    if sharedMuts_NthSeq_prop ≤ 0.3
                        continue
                    end
                    US_state_check = "no"
                    if seq_country[NthSeq] == seq_country[firstSeq] && (seq_country[NthSeq] == "USA" || seq_country[NthSeq] == "Canada") && seq_US_state[NthSeq] == seq_US_state[firstSeq]
                        US_state_check = "yes"
                    end 
                    if seq_country[NthSeq] == seq_country[firstSeq] && seq_country[NthSeq] ≠ "USA" && seq_country[NthSeq] ≠ "Canada"
                        US_state_check = "yes"
                    end
                    if NthSeq in group_exceptions
                        US_state_check = "yes"
                    end
                    if sharedMuts_tot ≥ rep_thresh && seq_clade[NthSeq] == seq_clade[firstSeq] && US_state_check == "yes"
                        push!(rep_seqs, NthSeq)
                        push!(rep_seq_grps_seqs[i], NthSeq)
                        for mut in seq_nuc_muts[NthSeq]
                            push!(rep_seq_grps_muts[i], mut)
                            if mut[end] ≠ '-'
                                push!(rep_seq_grps_muts_no_dels[i], mut)
                            end
                        end
                        for mut in seq_AA_muts[NthSeq]
                            push!(rep_seq_grps_AA[i], mut)
                            if mut[end] ≠ '-'
                                push!(rep_seq_grps_AA_no_dels[i], mut)
                            end
                        end
                    elseif sharedMuts_tot ≥ rep_thresh && seq_clade[NthSeq] == seq_clade[firstSeq] && NthSeq in group_exceptions
                        push!(rep_seqs, NthSeq)
                        push!(rep_seq_grps_seqs[i], NthSeq)
                        for mut in seq_nuc_muts[NthSeq]
                            push!(rep_seq_grps_muts[i], mut)
                            if mut[end] ≠ '-'
                                push!(rep_seq_grps_muts_no_dels[i], mut)
                            end
                        end
                        for mut in seq_AA_muts[NthSeq]
                            push!(rep_seq_grps_AA[i], mut)
                            if mut[end] ≠ '-'
                                push!(rep_seq_grps_AA_no_dels[i], mut)
                            end
                        end
                    end
                end
            end
        end
        for p in n+1:length(all_qualifying_seqs)
            if !(NthSeq in rep_seqs) && !(NthSeq in singlet_exceptions)
                PthSeq = all_qualifying_seqs[p]
                if !(PthSeq in rep_seqs) && !(PthSeq in singlet_exceptions)
                    common_muts = intersect(seq_AA_muts_no_dels[NthSeq], seq_AA_muts_no_dels[PthSeq])
                    US_state_check = "no"
                    if seq_country[NthSeq] == seq_country[PthSeq] && (seq_country[NthSeq] == "USA" || seq_country[NthSeq] == "Canada") && seq_US_state[NthSeq] == seq_US_state[PthSeq]
                        US_state_check = "yes"
                    end 
                    if seq_country[NthSeq] == seq_country[PthSeq] && seq_country[NthSeq] ≠ "USA" && seq_country[NthSeq] ≠ "Canada"
                        US_state_check = "yes"
                    end  
                    if NthSeq in group_exceptions
                        US_state_check = "yes"
                    end
                    if length(common_muts) ≥ rep_thresh && seq_clade[NthSeq] == seq_clade[PthSeq] && US_state_check == "yes"
                        push!(rep_seqs, NthSeq)
                        push!(rep_seqs, PthSeq)
                        if !haskey(rep_seq_grps_seqs, 1)
                            rep_seq_grps_seqs[1] = Set{String}()
                            rep_seq_grps_muts[1] = Set{String}()
                            rep_seq_grps_muts_no_dels[1] = Set{String}()
                            rep_seq_grps_dels[1] = Set{String}()
                            rep_seq_grps_AA[1] = Set{String}()
                            rep_seq_grps_AA_no_dels[1] = Set{String}()
                            rep_seq_grps_AA_dels[1] = Set{String}()
                            push!(rep_seq_grps_seqs[1], NthSeq)
                            push!(rep_seq_grps_seqs[1], PthSeq)
                            rep_seq_grps_clade[1] = seq_clade[PthSeq]
                            rep_seq_grps_pango[1] = seq_pango[PthSeq]
                            rep_seq_grps_pango_unaliased[1] = seq_pango_unaliased[PthSeq]
                            for m in seq_nuc_muts[NthSeq]
                                push!(rep_seq_grps_muts[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[1], m)
                                end
                            end
                            for m in seq_nuc_muts[PthSeq]
                                push!(rep_seq_grps_muts[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[1], m)
                                end
                            end
                            for m in seq_AA_muts[NthSeq]
                                push!(rep_seq_grps_AA[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[1], m)
                                end
                            end
                            for m in seq_AA_muts[PthSeq]
                                push!(rep_seq_grps_AA[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[1], m)
                                end
                            end
                        else
                            Grp_Nums = keys(rep_seq_grps_seqs)
                            new_grp_num = length(Grp_Nums) + 1
                            rep_seq_grps_clade[new_grp_num] = seq_clade[PthSeq]
                            rep_seq_grps_pango[new_grp_num] = seq_pango[PthSeq]
                            rep_seq_grps_pango_unaliased[new_grp_num] = seq_pango_unaliased[PthSeq]
                            rep_seq_grps_seqs[new_grp_num] = Set{String}()
                            rep_seq_grps_muts[new_grp_num] = Set{String}()
                            rep_seq_grps_muts_no_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_AA[new_grp_num] = Set{String}()
                            rep_seq_grps_AA_no_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_AA_dels[new_grp_num] = Set{String}()
                            push!(rep_seq_grps_seqs[new_grp_num], NthSeq)
                            push!(rep_seq_grps_seqs[new_grp_num], PthSeq)
                            for m in seq_nuc_muts[NthSeq]
                                push!(rep_seq_grps_muts[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_nuc_muts[PthSeq]
                                push!(rep_seq_grps_muts[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_AA_muts[NthSeq]
                                push!(rep_seq_grps_AA[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_AA_muts[PthSeq]
                                push!(rep_seq_grps_AA[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[new_grp_num], m)
                                end
                            end
                        end
                    end
                end
            end
        end
    end
#####################################################################################################################################
    for (grp_num, seq_set) in rep_seq_grps_seqs
        max_AA_muts::Int = 0
        max_AA_mut_seq = ""
        for seq in seq_set
            if haskey(seq_AA_muts, seq)
                seq_total_priv_AA_muts = length(seq_AA_muts[seq])
                if seq_total_priv_AA_muts > max_AA_muts
                    max_AA_muts = seq_total_priv_AA_muts
                    max_AA_mut_seq = seq
                end
            end
        end
        rep_seq_grps_maxmut_seqs[grp_num] = max_AA_mut_seq
    end
################
    for (grp_num, seq) in rep_seq_grps_maxmut_seqs
        rep_seq_grps_maxmut_date_index[grp_num] = seq_date_index[seq]
        rep_seq_grps_maxmut_date_tuple[grp_num] = seq_date_tuple[seq]
        rep_seq_grps_maxmut_nuc[grp_num] = seq_nuc_muts[seq]
        rep_seq_grps_maxmut_nuc_no_dels[grp_num] = seq_nuc_muts_no_dels[seq]
        rep_seq_grps_maxmut_dels[grp_num] = seq_nuc_del_ranges[seq]
        rep_seq_grps_maxmut_del_ranges_ct[grp_num] = seq_nuc_del_ranges_ct[seq]
        rep_seq_grps_maxmut_nuc_dropout[grp_num] = Set{Int}()
        if haskey(seq_nuc_dropout, seq)
            for num in seq_nuc_dropout[seq]
                push!(rep_seq_grps_maxmut_nuc_dropout[grp_num], num)
            end
        end
        rep_seq_grps_maxmut_mixed_nucs[grp_num] = seq_mixed_nucs[seq]
        rep_seq_grps_maxmut_AA[grp_num] = seq_AA_muts[seq]
        rep_seq_grps_maxmut_AA_no_dels[grp_num] = seq_AA_muts_no_dels[seq]
        rep_seq_grps_maxmut_AA_dels[grp_num] = seq_AA_dels[seq]
        rep_seq_grps_maxmut_AA_pos_only[grp_num] = seq_AA_muts_pos_only[seq]
        rep_seq_grps_maxmut_AA_pos_only_no_dels[grp_num] = seq_AA_muts_pos_only_no_dels[seq]
        rep_seq_grps_maxmut_unknown_AA[grp_num] = Set{String}()
        if haskey(seq_unknown_AA, seq)
            for AAsite in seq_unknown_AA[seq]
                push!(rep_seq_grps_maxmut_unknown_AA[grp_num], AAsite)
            end
        end
################
        rep_seq_grps_maxmut_unknown_AA_ranges[grp_num] = seq_unknown_AA_ranges[seq]
        rep_seq_grps_maxmut_mixed_AA_muts[grp_num] = seq_mixed_AA_muts[seq]
        rep_seq_grps_maxmut_nuc_muts_WT[grp_num] = seq_nuc_muts_WT[seq]
        rep_seq_grps_maxmut_nuc_dels_WT[grp_num] = seq_nuc_del_ranges_WT[seq]
        rep_seq_grps_maxmut_AA_muts_WT[grp_num] = seq_AA_muts_WT[seq]
        rep_seq_grps_maxmut_AA_dels_WT[grp_num] = seq_AA_dels_WT[seq]
        rep_seq_grps_maxmut_AA_muts_WT_pos_only[grp_num] = seq_AA_muts_WT_pos_only[seq]
        rep_seq_grps_maxmut_clade[grp_num] = seq_clade[seq]
        rep_seq_grps_maxmut_pango[grp_num] = seq_pango[seq]
        rep_seq_grps_maxmut_pango_unaliased[grp_num] = seq_pango_unaliased[seq]
    end
    for i in 1:length(rep_seq_grps_seqs)
        date_index = rep_seq_grps_maxmut_date_index[i]
        for m in rep_seq_grps_maxmut_nuc[i]
            nuc_muts_ct[m] = get(nuc_muts_ct, m, 0) + 1
            get!(date_nuc_mut_ct, date_index, Dict{String, Int64}() )
            date_nuc_mut_ct[date_index][m] = get(date_nuc_mut_ct[date_index], m, 0) + 1
            if m[end] ≠ '-'
                get!(date_nuc_mut_ct_no_dels, date_index, Dict{String, Int64}() )
                date_nuc_mut_ct_no_dels[date_index][m] = get(date_nuc_mut_ct_no_dels[date_index], m, 0) + 1
            end
        end
#*        for m in rep_seq_grps_maxmut_nuc[i]
#*            if m[end] ≠ '-'
#*                nuc_muts_ct_no_dels[m] = get(nuc_muts_ct_no_dels, m, 0) + 1
#*            end
#*        end
        for m in rep_seq_grps_maxmut_dels[i]
            nuc_dels_ct[m] = get(nuc_dels_ct, m, 0) + 1
        end
        for m in rep_seq_grps_maxmut_mixed_nucs[i]
            mixed_nucs_ct[m] = get(mixed_nucs_ct, m, 0) + 1
        end
##############################################################################################################
        for m in rep_seq_grps_maxmut_AA[i]
            AA_muts_ct[m] = get(AA_muts_ct, m, 0) + 1
            get!(date_AA_mut_ct, date_index, Dict{String, Int64}() )
            date_AA_mut_ct[date_index][m] = get(date_AA_mut_ct[date_index], m, 0) + 1
            if m[end] ≠ '-'
                get!(date_AA_mut_ct_no_dels, date_index, Dict{String, Int64}() )
                date_AA_mut_ct_no_dels[date_index][m] = get(date_AA_mut_ct_no_dels[date_index], m, 0) + 1
                AApos = AA_position(m)
                get!(date_AA_mut_ct_pos_only_no_dels, date_index, Dict{String, Int64}() )
                date_AA_mut_ct_pos_only_no_dels[date_index][m] = get(date_AA_mut_ct_pos_only_no_dels[date_index], m, 0) + 1
            end
        end
        for m in rep_seq_grps_maxmut_AA_dels[i]
            AA_dels_ct[m] = get(AA_dels_ct, m, 0) + 1
        end
        for m in rep_seq_grps_maxmut_unknown_AA[i]
            unknown_AA_ct[m] = get(unknown_AA_ct, m, 0) + 1
        end
##############################################################################################################
        for m in rep_seq_grps_maxmut_nuc_muts_WT[i]
            nuc_muts_ct_WT[m] = get(nuc_muts_ct_WT, m, 0) + 1
        end
        for m in rep_seq_grps_maxmut_nuc_dels_WT[i]
            nuc_dels_ct_WT[m] = get(nuc_dels_ct_WT, m, 0) + 1
        end
##############################################################################################################
        for m in rep_seq_grps_maxmut_AA_muts_WT[i]
            AA_muts_ct_WT[m] = get(AA_muts_ct_WT, m, 0) + 1
        end
        for m in rep_seq_grps_maxmut_AA_dels_WT[i]
            AA_dels_ct_WT[m] = get(AA_dels_ct_WT, m, 0) + 1
        end
        for m in rep_seq_grps_maxmut_AA[i]
            jeen = split(m, ":")[1]
            AA_pos = split(m, ":")[2][2:end-1]
            mpos = "$(jeen):$(AA_pos)"
            qry_AA = string(m[end])
            ref_AA = string(split(m, ":")[2][1])
            if m[end] == ref_AA
                AA_muts_ct_pos_only_reversion[mpos] = get(AA_muts_ct_pos_only_reversion, mpos, 0) + 1
            end
        end
        clade = rep_seq_grps_maxmut_clade[i]
        pango = rep_seq_grps_maxmut_pango[i]
        pango_unaliased = rep_seq_grps_maxmut_pango_unaliased[i]
        seq_clade_ct[pango] = get(seq_clade_ct, clade, 0) + 1
        seq_pango_ct[pango] = get(seq_pango_ct, pango, 0) + 1
        seq_pango_unaliased_ct[pango_unaliased] = get(seq_pango_unaliased_ct, pango, 0) + 1
    end
##############################################################################################################
#**#    rep_seq_grps_nuc_mut_keys = keys(rep_seq_grps_muts)
#**#    rep_seq_grps_nuc_mut_keys_ln = length(rep_seq_grps_nuc_mut_keys)
#**#    for i in 1:rep_seq_grps_nuc_mut_keys_ln
#**#        for m in rep_seq_grps_muts[i]
#**#            if m[end] ≠ '-'
#**#                nuc_muts_ct_no_dels[m] = get(nuc_muts_ct_no_dels, m, 0) + 1
#**#            end
#**#        end
#**#    end
#    for (grp_num, clade) in rep_seq_grps_clade
#        seq_clade_ct[clade] = get(seq_clade_ct, clade, 0) + 1
#    end
#    for (grp_num, pango) in rep_seq_grps_pango
#        seq_pango_ct[pango] = get(seq_pango_ct, pango, 0) + 1
#    end
#    for (grp_num, pango_unaliased) in rep_seq_grps_pango_unaliased
#        seq_pango_unaliased_ct[pango_unaliased] = get(seq_pango_unaliased_ct, pango_unaliased, 0) + 1
#    end
    println("Number of Sequences in rep_seqs list: ", length(rep_seqs))
    print("\n"^2)
    gene_name_fx(m) = split(m, ":")[1]
    for grp_num in keys(rep_seq_grps_AA)
        rep_seq_grps_AA_pos_only[grp_num] = Set{String}()
        rep_seq_grps_AA_pos_only_no_dels[grp_num] = Set{String}()
    end
    for (grp_num, mutset) in rep_seq_grps_AA
        for m in mutset
            AA_pos = AA_mut_to_AA_pos(m)
            push!(rep_seq_grps_AA_pos_only[grp_num], AA_pos)
        end
        for m in mutset
            if m[end] ≠ '-'
                AA_pos = AA_mut_to_AA_pos(m)
                push!(rep_seq_grps_AA_pos_only_no_dels[grp_num], AA_pos)
            end
        end
        for m in mutset
            if !haskey(AA_muts_rep_seq_grps, m)
                AA_muts_rep_seq_grps[m] = Set{Int}()
            end
            push!(AA_muts_rep_seq_grps[m], grp_num)
            if m[end] ≠ '-'
                if !haskey(AA_muts_rep_seq_grps_no_dels, m)
                    AA_muts_rep_seq_grps_no_dels[m] = Set{Int}()
                end
                push!(AA_muts_rep_seq_grps_no_dels[m], grp_num)
            end
            if m[end] == '-'
                get!(AA_dels_rep_seq_grps, m, Set{Int}() )
                push!(AA_dels_rep_seq_grps[m], grp_num)
            end
        end
    end
    for (grp_num, mutpos_set) in rep_seq_grps_AA_pos_only
        for m in mutpos_set
            get!(AA_muts_rep_seq_grps_pos_only, m, Set{Int}() )
            push!(AA_muts_rep_seq_grps_pos_only[m], grp_num)
        end      
    end
##############################################################################
    for seq in all_qualifying_seqs_set
        if !(seq in rep_seqs)
            push!(non_rep_seqs, seq)
        end
    end    
    for seq in non_rep_seqs
        date_index = seq_date_index[seq]
        clade = seq_clade[seq]
        pango = seq_pango[seq]
        pango_unaliased = seq_pango_unaliased[seq]
        seq_clade_ct[clade] = get(seq_clade_ct, clade, 0) + 1
        seq_pango_ct[pango] = get(seq_pango_ct, pango, 0) + 1
        seq_pango_unaliased_ct[pango_unaliased] = get(seq_pango_unaliased_ct, pango_unaliased, 0) + 1
        for m in seq_nuc_muts[seq]
            nuc_muts_ct[m] = get(nuc_muts_ct, m, 0) + 1
            get!(date_nuc_mut_ct, date_index, Dict{String, Int64}() )
            date_nuc_mut_ct[date_index][m] = get(date_nuc_mut_ct[date_index], m, 0) + 1
            if m[end] ≠ '-'
                get!(date_nuc_mut_ct_no_dels, date_index, Dict{String, Int64}() )
                date_nuc_mut_ct_no_dels[date_index][m] = get(date_nuc_mut_ct_no_dels[date_index], m, 0) + 1
            end
        end
        for m in seq_nuc_muts_WT[seq]
            nuc_muts_ct_WT[m] = get(nuc_muts_ct_WT, m, 0) + 1
        end
        for m in seq_nuc_del_ranges_WT[seq]
            nuc_dels_ct_WT[m] = get(nuc_dels_ct_WT, m, 0) + 1
        end
        for m in seq_mixed_nucs[seq]
            mixed_nucs_ct[m] = get(mixed_nucs_ct, m, 0) + 1
        end
        for m in seq_AA_muts[seq]
            AA_muts_ct[m] = get(AA_muts_ct, m, 0) + 1
            get!(date_AA_mut_ct, date_index, Dict{String, Int64}() )
            date_AA_mut_ct[date_index][m] = get(date_AA_mut_ct[date_index], m, 0) + 1
            if m[end] ≠ '-'
                get!(date_AA_mut_ct_no_dels, date_index, Dict{String, Int64}() )
                date_AA_mut_ct_no_dels[date_index][m] = get(date_AA_mut_ct_no_dels[date_index], m, 0) + 1
                AApos = AA_position(m)
                get!(date_AA_mut_ct_pos_only_no_dels, date_index, Dict{String, Int64}() )
                date_AA_mut_ct_pos_only_no_dels[date_index][m] = get(date_AA_mut_ct_pos_only_no_dels[date_index], m, 0) + 1
            end
            get!(non_rep_seqs_AA, seq, Set{String}() )
            push!(non_rep_seqs_AA[seq], m)
        end
        for m in seq_AA_muts_WT[seq]
            AA_muts_ct_WT[m] = get(AA_muts_ct_WT, m, 0) + 1
        end
        for m in seq_AA_dels_WT[seq]
            AA_dels_ct_WT[m] = get(AA_dels_ct_WT, m, 0) + 1
        end
        for m in seq_unknown_AA[seq]
            unknown_AA_ct[m] = get(unknown_AA_ct, m, 0) + 1
        end
        for m in seq_AA_muts[seq]
            jeen = split(m, ":")[1]
            AA_pos = split(m, ":")[2][2:end-1]
            mpos = "$(jeen):$(AA_pos)"
            qry_AA = string(m[end])
            ref_AA = string(split(m, ":")[2][1])
            if m[end] == ref_AA
                AA_muts_ct_pos_only_reversion[mpos] = get(AA_muts_ct_pos_only_reversion, mpos, 0) + 1
            end
        end
        for (EPI, mutSet) in non_rep_seqs_AA
            for m in mutSet
                AA_pos = AA_mut_to_AA_pos(m)
                get!(non_rep_seqs_AA_pos_only, EPI, Set{String}() )
                push!(non_rep_seqs_AA_pos_only[EPI], AA_pos)
            end
            for m in mutSet
                if m[end] ≠ '-'
                    AA_pos = AA_mut_to_AA_pos(m)
                    get!(non_rep_seqs_AA_pos_only_no_dels, EPI, Set{String}() )
                    push!(non_rep_seqs_AA_pos_only_no_dels[EPI], AA_pos)
                end
            end
        end
    end
#####################################################################################################################################
#####################################################################################################################################
    nuc_mut_pos(m) = parse(Int, m[2:end-1])
    for (mut, ct) in nuc_muts_ct
        nuc_pos = nuc_mut_pos(mut)
        if nuc_pos in BitSet([1:29850...])
            REFNUC = string(ref_seq[nuc_pos])
            QRYNUC = string(mut[end])
            if mut[end] ≠ '-'
                nuc_muts_ct_no_dels[mut] = ct
                nuc_muts_ct_pos_only_no_dels[nuc_pos] = get(nuc_muts_ct_pos_only_no_dels, nuc_pos, 0) + ct
                if !(QRYNUC == REFNUC) && !(mut in reversion_exceptions)
                    nuc_muts_ct_no_dels_no_revs[mut] = ct
                    nuc_muts_ct_pos_only_no_dels_no_revs[nuc_pos] = get(nuc_muts_ct_pos_only_no_dels_no_revs, nuc_pos, 0) + ct
                end
            end
        end
    end
    for (mut, ct) in AA_muts_ct
        AA_pos = AA_mut_to_AA_pos(mut)
        AA_mut_pos_int = parse(Int, split(mut, ":")[2][2:end-1])
        gene = split(mut, ":")[1]
        gene = string(gene)
        refaa = gene_AA_dict[gene][AA_mut_pos_int]
        refAA = string(refaa)
        qryAA = string(mut[end])
        AA_muts_ct_pos_only[AA_pos] = get(AA_muts_ct_pos_only, AA_pos, 0) + ct
        if mut[end] ≠ '-'
            AA_muts_ct_no_dels[mut] = ct
            AA_muts_ct_pos_only_no_dels[AA_pos] = get(AA_muts_ct_pos_only_no_dels, AA_pos, 0) + ct
            if !(qryAA == refAA) && !(mut in reversion_exceptions)
                AA_muts_ct_no_dels_no_revs[mut] = ct
                AA_muts_ct_pos_only_no_dels_no_revs[AA_pos] = get(AA_muts_ct_pos_only_no_dels_no_revs, AA_pos, 0) + ct
            end
        end
    end
    for (mut, ct) in AA_muts_ct_WT
        AA_pos = AA_mut_to_AA_pos(mut)
        AA_muts_ct_WT_pos_only[AA_pos] = get(AA_muts_ct_WT_pos_only, AA_pos, 0) + ct
        if mut[end] ≠ '-'
            AA_muts_ct_WT_no_dels[mut] = ct
            AA_muts_ct_WT_pos_only_no_dels[AA_pos] = get(AA_muts_ct_WT_pos_only_no_dels, AA_pos, 0) + ct
        end
    end
#####################################################################################################################################
#####################################################################################################################################
######################################### Begin, Nuc Chr-All Ratio Section ##########################################################
#####################################################################################################################################
#####################################################################################################################################
    nuc_muts_ct_pos_only_no_dels_all = Dict{Int, Int}()
    for i in 1:30000
        nuc_muts_ct_pos_only_no_dels_all[i] = 0
    end
    for (nuc_mut, count) in nuc_muts_ct_no_dels_all
        nuc_pos = nuc_sort_key(nuc_mut)
        nuc_muts_ct_pos_only_no_dels_all[nuc_pos] += count
    end
#####################################################################################################################################
    nuc_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    nuc_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    nuc_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{Int, Float64}()
    nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio = Dict{Int, Float64}()
    
    non_rep_seq_ct = length(non_rep_seqs)
    rep_seq_grps_maxmut_seqs_ct = length(keys(rep_seq_grps_maxmut_seqs))
    total_qualifying_chr_seq_ct = non_rep_seq_ct + rep_seq_grps_maxmut_seqs_ct
    all_ct = qualifying_seq_ct_all
    total_circ_nuc_mut_ct::Int = 0
    total_chr_nuc_ct::Int = 0
    total_chr_nuc_no_revs_ct::Int = 0
    for (mut, count) in nuc_muts_ct_no_dels_all
        total_circ_nuc_mut_ct += count
    end
    for (mut, count) in nuc_muts_ct_no_dels
        total_chr_nuc_ct += count
    end
    for (mut, count) in nuc_muts_ct_no_dels_no_revs
        total_chr_nuc_no_revs_ct += count
    end
    avg_nuc_subs_per_chr_seq = total_chr_nuc_ct/total_qualifying_chr_seq_ct
    avg_nuc_subs_per_chr_seq_rd = round(digits=2, avg_nuc_subs_per_chr_seq)
    avg_nuc_subs_per_circ_seq = total_circ_nuc_mut_ct/all_ct
    avg_nuc_subs_per_circ_seq_rd = round(digits=2, avg_nuc_subs_per_circ_seq)
    avg_nuc_subs_per_chr_seq_no_revs = total_chr_nuc_no_revs_ct/total_qualifying_chr_seq_ct
    avg_nuc_subs_per_chr_seq_no_revs_rd = round(digits=2, avg_nuc_subs_per_chr_seq_no_revs)
    println()
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic Nuc Substitution Ct = $(total_chr_nuc_ct)  **********************")
    println("********************** Total Circulating Nuc Substitution Ct = $(total_circ_nuc_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic Nuc Substitution Ct = $(total_chr_nuc_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating Nuc Substitution Ct = $(total_circ_nuc_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
    chr_to_circ_mut_ratio_adj_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_nuc_subs_per_circ_seq/avg_nuc_subs_per_chr_seq)
    for (mut, chr_ct) in nuc_muts_ct_no_dels
        if haskey(nuc_muts_ct_no_dels_all, mut)
            circ_ct = nuc_muts_ct_no_dels_all[mut]
            nuc_muts_ct_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            nuc_muts_ct_no_dels_all[mut] = 0
            nuc_muts_ct_no_dels_chr_all_ratio[mut] = chr_ct
        end
    end
#####################################################################################################################################
    nuc_muts_ct_no_dels_chr_all_ratio_pos_sort = sort(collect(nuc_muts_ct_no_dels_chr_all_ratio), by = x -> nuc_sort_key(x[1]))
    nuc_muts_ct_no_dels_chr_all_ratio_ct_sort = sort(collect(nuc_muts_ct_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    ratio_print_ct::Int = 0
    println()
    println("10 random nuc_muts_ct_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in nuc_muts_ct_no_dels_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", lpad(ratio, 7))
        end
    end
    print("\n"^8)
#####################################################################################################################################
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic Nuc Substitution Ct No Revs = $(total_chr_nuc_no_revs_ct)  **********************")
    println("********************** Total Circulating Nuc Substitution Ct = $(total_circ_nuc_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic Nuc Substitution Ct, No Revs = $(total_chr_nuc_no_revs_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating Nuc Substitution Ct = $(total_circ_nuc_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
    chr_to_circ_mut_ratio_adj_factor_no_revs = (all_ct/total_qualifying_chr_seq_ct)*(avg_nuc_subs_per_circ_seq/avg_nuc_subs_per_chr_seq_no_revs)
    for (mut, chr_ct_no_revs) in nuc_muts_ct_no_dels_no_revs
        if haskey(nuc_muts_ct_no_dels_all, mut)
            circ_ct = nuc_muts_ct_no_dels_all[mut]
            nuc_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            nuc_muts_ct_no_dels_all[mut] = 0
            nuc_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs
        end
    end
#####################################################################################################################################
    nuc_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(nuc_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> nuc_sort_key(x[1]))
    nuc_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(nuc_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    ratio_print_ct = 0
    println()
    println("10 random nuc_muts_ct_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in nuc_muts_ct_no_dels_no_revs_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", lpad(ratio_no_revs, 7))
        end
    end
######################################################################################################################################
    total_circ_nuc_pos_only_ct::Int = 0
    total_chr_nuc_pos_only_ct::Int = 0
    total_chr_nuc_pos_only_no_revs_ct::Int = 0
    for (mut, count) in nuc_muts_ct_pos_only_no_dels_all
        total_circ_nuc_pos_only_ct += count
    end
    for (mut, count) in nuc_muts_ct_pos_only_no_dels
        total_chr_nuc_pos_only_ct += count
    end
    for (mut, count) in nuc_muts_ct_pos_only_no_dels_no_revs
        total_chr_nuc_pos_only_no_revs_ct += count
    end
    avg_nuc_subs_pos_only_per_circ_seq = total_circ_nuc_pos_only_ct/all_ct
    avg_nuc_subs_pos_only_per_circ_seq_rd = round(digits=2, avg_nuc_subs_pos_only_per_circ_seq)
    avg_nuc_subs_pos_only_per_chr_seq = total_chr_nuc_pos_only_ct/total_qualifying_chr_seq_ct
    avg_nuc_subs_pos_only_per_chr_seq_rd = round(digits=2, avg_nuc_subs_pos_only_per_chr_seq)
    avg_nuc_subs_pos_only_no_revs_per_chr_seq = total_chr_nuc_pos_only_no_revs_ct/total_qualifying_chr_seq_ct
    avg_nuc_subs_pos_only_no_revs_per_chr_seq_rd = round(digits=2, avg_nuc_subs_pos_only_no_revs_per_chr_seq)
    println()
    println("********************** Total Counted Chronic Sequences = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Sequences = $(qualifying_seq_ct_all)  **********************")
    println()
    println("********************** Total Chronic Nuc Subs Pos Only Ct = $(total_chr_nuc_pos_only_ct)  **********************")
    println("********************** Total Circulating Nuc Subs Pos Only Ct = $(total_circ_nuc_pos_only_ct)  **********************")
    println()
    println("********************** Avg Chronic Nuc Subs Pos Only Ct = $(avg_nuc_subs_pos_only_per_chr_seq_rd)  **********************")
    println("********************** Avg Circulating Nuc Subs Pos Only Ct = $(avg_nuc_subs_pos_only_per_circ_seq_rd)  **********************")
    println()
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_nuc_subs_pos_only_per_circ_seq/avg_nuc_subs_pos_only_per_chr_seq)
    for (mut, chr_ct) in nuc_muts_ct_pos_only_no_dels
        if haskey(nuc_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = nuc_muts_ct_pos_only_no_dels_all[mut]
            nuc_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            nuc_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = chr_ct
            nuc_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
#####################################################################################################################################
    nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort = sort(collect(nuc_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> x[1])
    nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort = sort(collect(nuc_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    pos_only_ratio_print_ct::Int = 0
    println()
    println("10 random nuc_muts_ct_pos_only_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in nuc_muts_ct_pos_only_no_dels_chr_all_ratio
        pos_only_ratio_print_ct += 1
        if pos_only_ratio_print_ct < 10
            println(rpad(mut, 10), " = ", lpad(ratio, 7))
        end
    end
    for mut in keys(nuc_muts_ct_pos_only_no_dels_chr_all_ratio)
        if !haskey(nuc_muts_ct_pos_only_no_dels, mut)
            nuc_muts_ct_pos_only_no_dels[mut] = 0
        end
    end
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_no_revs_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_nuc_subs_pos_only_per_circ_seq/avg_nuc_subs_pos_only_no_revs_per_chr_seq)
    for (mut, chr_ct_no_revs) in nuc_muts_ct_pos_only_no_dels_no_revs
        if haskey(nuc_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = nuc_muts_ct_pos_only_no_dels_all[mut]
            nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs
            nuc_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
#####################################################################################################################################
    nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> x[1])
    nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    pos_only_no_revs_ratio_print_ct::Int = 0
    println("10 random nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio
        pos_only_no_revs_ratio_print_ct += 1
        if pos_only_no_revs_ratio_print_ct < 10
            println(rpad(mut, 10), " = ", lpad(ratio_no_revs, 7))
        end
    end
    println()
    for mut in keys(nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
        if !haskey(nuc_muts_ct_pos_only_no_dels_no_revs, mut)
            nuc_muts_ct_pos_only_no_dels_no_revs[mut] = 0
        end
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_nuc_Chr_Circ_Ratios.txt", "w") do g
        println(g, "*******************************************************************************************************")
        println(g, "******************* Nuc Chronic/Circulating Ratios (no deletions), Sorted by Ratio ********************")
        println(g, "*******************************************************************************************************")
        for w in nuc_muts_ct_no_dels_chr_all_ratio_ct_sort
             nuc_mut = w[1]
             ratio = w[2]
             if nuc_muts_ct_no_dels[nuc_mut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nuc_mut[2:end-1])
                qry_nuc_str = string(nuc_mut[end])
                ref_seq_nuc_str = string(nuc_mut[1])
                nucmut_pad = rpad(nuc_mut, 7)
                ratiopad = lpad(w[2], 6)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_no_dels[nuc_mut])]", 5)
                if qry_nuc_str ≠ ref_seq_nuc_str 
                    mutpad = rpad(string(nuc_mut), 12)
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
                else
                    mutpad = rpad(string(nuc_mut), 12)
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "*******************************************************************************************************")
        println(g, "************** Nuc Chronic/Circulating Ratios (no deletions), Sorted by Genome Position ***************")
        println(g, "*******************************************************************************************************")
        for w in nuc_muts_ct_no_dels_chr_all_ratio_pos_sort
             nuc_mut = w[1]
             ratio = w[2]
             if nuc_muts_ct_no_dels[nuc_mut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nuc_mut[2:end-1])
                qry_nuc_str = string(nuc_mut[end])
                ref_seq_nuc_str = string(nuc_mut[1])
                nucmut_pad = rpad(nuc_mut, 7)
                ratiopad = lpad(w[2], 6)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_no_dels[nuc_mut])]", 5)
                if qry_nuc_str ≠ ref_seq_nuc_str 
                    mutpad = rpad(string(nuc_mut), 12)
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
                else
                    mutpad = rpad(string(nuc_mut), 12)
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_nuc_Chr_Circ_Ratios_Pos_Only.txt", "w") do g
        println(g, "*******************************************************************************************************")
        println(g, "************** Nuc Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Ratio ****************")
        println(g, "*******************************************************************************************************")
        for w in nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort
            nuc_mut_pos_int = w[1]
            ratio = w[2]
            if nuc_muts_ct_pos_only_no_dels[nuc_mut_pos_int] ≥ print_ct_thresh
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_pos_only_no_dels[nuc_mut_pos_int])]", 5)
                ratiopad = lpad(w[2], 6)
                println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
        println(g, "*******************************************************************************************************")
        println(g, "******** Nuc Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Genome Position ************")
        println(g, "*******************************************************************************************************")
        for w in nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort
            nuc_mut_pos_int = w[1]
            count = w[2]
            if nuc_muts_ct_pos_only_no_dels[nuc_mut_pos_int] ≥ print_ct_thresh
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_pos_only_no_dels[nuc_mut_pos_int])]", 5)
                ratiopad = lpad(w[2], 6)
                println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_nuc_Chr_Circ_Ratios_No_Revs.txt", "w") do g
        println(g, "*************************************************************************************************************************************")
        println(g, "************** Nuc Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Ratio ***************")
        println(g, "*************************************************************************************************************************************")
        for w in nuc_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort
             nuc_mut = w[1]
             ratio = w[2]
             if nuc_muts_ct_no_dels_no_revs[nuc_mut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nuc_mut[2:end-1])
                qry_nuc_str = string(nuc_mut[end])
                ref_seq_nuc_str = string(nuc_mut[1])
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_no_dels_no_revs[nuc_mut])]", 5)
                ratiopad = lpad(w[2], 6)
                if qry_nuc_str ≠ ref_seq_nuc_str 
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
                else
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "***********************************************************************************************************************************************")
        println(g, "************** Nuc Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Genome Position ***************")
        println(g, "***********************************************************************************************************************************************")
        for w in nuc_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort
             nuc_mut = w[1]
             ratio = w[2]
             if nuc_muts_ct_no_dels_no_revs[nuc_mut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nuc_mut[2:end-1])
                qry_nuc_str = string(nuc_mut[end])
                ref_seq_nuc_str = string(nuc_mut[1])
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_no_dels_no_revs[nuc_mut])]", 5)
                ratiopad = lpad(w[2], 6)
                if qry_nuc_str ≠ ref_seq_nuc_str 
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
                else
                    println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_nuc_Chr_Circ_Ratios_Pos_Only_No_Revs.txt", "w") do g
        println(g, "***********************************************************************************************************************************")
        println(g, "************** Nuc Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Ratio ****************")
        println(g, "***********************************************************************************************************************************")
        for w in nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort
            nuc_mut_pos_int = w[1]
            ratio = w[2]
            if nuc_muts_ct_pos_only_no_dels_no_revs[nuc_mut_pos_int] ≥ print_ct_thresh
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_pos_only_no_dels_no_revs[nuc_mut_pos_int])]", 5)
                ratiopad = lpad(w[2], 6)
                println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
        println(g, "***********************************************************************************************************************************")
        println(g, "********* Nuc Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Genome Position ***********")
        println(g, "***********************************************************************************************************************************")
        for w in nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort
            nuc_mut_pos_int = w[1]
            ratio = w[2]
            if nuc_muts_ct_pos_only_no_dels_no_revs[nuc_mut_pos_int] ≥ print_ct_thresh
                mutpad = rpad(string(nuc_mut_pos_int), 5)
                nuc_muts_ct_pad = rpad("[$(nuc_muts_ct_pos_only_no_dels_no_revs[nuc_mut_pos_int])]", 5)
                ratiopad = lpad(w[2], 6)
                println(g, "$(mutpad) = $(ratiopad) $(nuc_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
#####################################################################################################################################
######################################### End, Nuc Chr-All Ratio Section ############################################################
#####################################################################################################################################
#####################################################################################################################################

#####################################################################################################################################
#####################################################################################################################################
######################################### Begin, AA Chr-All Ratio Section ###########################################################
#####################################################################################################################################
#####################################################################################################################################
    AA_muts_ct_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    
    non_rep_seq_ct = length(non_rep_seqs)
    rep_seq_grps_maxmut_seqs_ct = length(keys(rep_seq_grps_maxmut_seqs))
    total_qualifying_chr_seq_ct = non_rep_seq_ct + rep_seq_grps_maxmut_seqs_ct
    all_ct = qualifying_seq_ct_all
    total_circ_AA_mut_ct::Int = 0
    total_chr_AA_ct::Int = 0
    total_chr_AA_no_revs_ct::Int = 0
    for (mut, count) in AA_muts_ct_no_dels_all
        total_circ_AA_mut_ct += count
    end
    for (mut, count) in AA_muts_ct_no_dels
        total_chr_AA_ct += count
    end
    for (mut, count) in AA_muts_ct_no_dels_no_revs
        total_chr_AA_no_revs_ct += count
    end
    avg_AA_subs_per_chr_seq = total_chr_AA_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_per_chr_seq_rd = round(digits=2, avg_AA_subs_per_chr_seq)
    avg_AA_subs_per_circ_seq = total_circ_AA_mut_ct/all_ct
    avg_AA_subs_per_circ_seq_rd = round(digits=2, avg_AA_subs_per_circ_seq)
    avg_AA_subs_per_chr_seq_no_revs = total_chr_AA_no_revs_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_per_chr_seq_no_revs_rd = round(digits=2, avg_AA_subs_per_chr_seq_no_revs)
    println()
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic AA Substitution Ct = $(total_chr_AA_ct)  **********************")
    println("********************** Total Circulating AA Substitution Ct = $(total_circ_AA_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Substitution Ct = $(total_chr_AA_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating AA Substitution Ct = $(total_circ_AA_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_per_circ_seq/avg_AA_subs_per_chr_seq)
    for (mut, chr_ct) in AA_muts_ct_no_dels
        if haskey(AA_muts_ct_no_dels_all, mut)
            circ_ct = AA_muts_ct_no_dels_all[mut]
            AA_muts_ct_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            AA_muts_ct_no_dels_all[mut] = 0
            AA_muts_ct_no_dels_chr_all_ratio[mut] = chr_ct
        end
    end
#####################################################################################################################################
    AA_muts_ct_no_dels_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_no_dels_chr_all_ratio), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_no_dels_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    ratio_no_dels_print_ct = 0
    println()
    println("10 random AA_muts_ct_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in AA_muts_ct_no_dels_chr_all_ratio
        ratio_no_dels_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", rpad(ratio, 7))
        end
    end
    print("\n"^5)
#####################################################################################################################################
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_per_circ_seq/avg_AA_subs_per_chr_seq)
    for (mut, chr_ct) in AA_muts_ct
        if haskey(AA_muts_ct_all, mut)
            circ_ct = AA_muts_ct_all[mut]
            AA_muts_ct_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            AA_muts_ct_all[mut] = 0
            AA_muts_ct_chr_all_ratio[mut] = chr_ct
        end
    end
#####################################################################################################################################
    AA_muts_ct_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_chr_all_ratio), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_chr_all_ratio Keys and Values")
    for (mut, ratio) in AA_muts_ct_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", rpad(ratio, 7))
        end
    end
    print("\n"^5)
#####################################################################################################################################
#####################################################################################################################################
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic AA Substitution Ct No Revs = $(total_chr_AA_no_revs_ct)  **********************")
    println("********************** Total Circulating AA Substitution Ct = $(total_circ_AA_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Substitution Ct, No Revs = $(total_chr_AA_no_revs_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating AA Substitution Ct = $(total_circ_AA_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
    chr_to_circ_mut_ratio_adj_factor_no_revs = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_per_circ_seq/avg_AA_subs_per_chr_seq_no_revs)
    for (mut, chr_ct_no_revs) in AA_muts_ct_no_dels_no_revs
        if haskey(AA_muts_ct_no_dels_all, mut)
            circ_ct = AA_muts_ct_no_dels_all[mut]
            AA_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            AA_muts_ct_no_dels_all[mut] = 0
            AA_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs
        end
    end
#####################################################################################################################################
    AA_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in AA_muts_ct_no_dels_no_revs_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", rpad(ratio_no_revs, 7))
        end
    end
######################################################################################################################################
    total_circ_AA_pos_only_ct::Int = 0
    total_chr_AA_pos_only_ct::Int = 0
    total_chr_AA_pos_only_no_revs_ct::Int = 0
    for (mut, count) in AA_muts_ct_pos_only_no_dels_all
        total_circ_AA_pos_only_ct += count
    end
    for (mut, count) in AA_muts_ct_pos_only_no_dels
        total_chr_AA_pos_only_ct += count
    end
    for (mut, count) in AA_muts_ct_pos_only_no_dels_no_revs
        total_chr_AA_pos_only_no_revs_ct += count
    end
    avg_AA_subs_pos_only_per_circ_seq = total_circ_AA_pos_only_ct/all_ct
    avg_AA_subs_pos_only_per_circ_seq_rd = round(digits=2, avg_AA_subs_pos_only_per_circ_seq)
    avg_AA_subs_pos_only_per_chr_seq = total_chr_AA_pos_only_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_pos_only_per_chr_seq_rd = round(digits=2, avg_AA_subs_pos_only_per_chr_seq)
    avg_AA_subs_pos_only_no_revs_per_chr_seq = total_chr_AA_pos_only_no_revs_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_pos_only_no_revs_per_chr_seq_rd = round(digits=2, avg_AA_subs_pos_only_no_revs_per_chr_seq)
    println()
    println("********************** Total Counted Chronic Sequences = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Sequences = $(qualifying_seq_ct_all)  **********************")
    println()
    println("********************** Total Chronic AA Subs Pos Only Ct = $(total_chr_AA_pos_only_ct)  **********************")
    println("********************** Total Circulating AA Subs Pos Only Ct = $(total_circ_AA_pos_only_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Subs Pos Only Ct = $(avg_AA_subs_pos_only_per_chr_seq_rd)  **********************")
    println("********************** Avg Circulating AA Subs Pos Only Ct = $(avg_AA_subs_pos_only_per_circ_seq_rd)  **********************")
    println()
#####################################################################################################################################
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_pos_only_per_circ_seq/avg_AA_subs_pos_only_per_chr_seq)
    for (mut, chr_ct) in AA_muts_ct_pos_only_no_dels
        if haskey(AA_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = AA_muts_ct_pos_only_no_dels_all[mut]
            AA_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            AA_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = chr_ct
            AA_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
#####################################################################################################################################
    AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    pos_only_ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_pos_only_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in AA_muts_ct_pos_only_no_dels_chr_all_ratio
        pos_only_ratio_print_ct += 1
        if pos_only_ratio_print_ct < 10
            println(rpad(mut, 11), " = ", rpad(ratio, 7))
        end
    end
    println()
    for mut in keys(AA_muts_ct_pos_only_no_dels_chr_all_ratio)
        if !haskey(AA_muts_ct_pos_only_no_dels, mut)
            AA_muts_ct_pos_only_no_dels[mut] = 0
        end
    end
#####################################################################################################################################
#####################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_no_revs_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_pos_only_per_circ_seq/avg_AA_subs_pos_only_no_revs_per_chr_seq)
    for (mut, chr_ct_no_revs) in AA_muts_ct_pos_only_no_dels_no_revs
        if haskey(AA_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = AA_muts_ct_pos_only_no_dels_all[mut]
            AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs
            AA_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
#####################################################################################################################################
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
#####################################################################################################################################
    pos_only_no_revs_ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio
        pos_only_no_revs_ratio_print_ct += 1
        if pos_only_no_revs_ratio_print_ct < 10
            println(rpad(mut, 11), " = ", rpad(ratio_no_revs, 7))
        end
    end
    println()
    for mut in keys(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
        if !haskey(AA_muts_ct_pos_only_no_dels_no_revs, mut)
            AA_muts_ct_pos_only_no_dels_no_revs[mut] = 0
        end
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_Chr_Circ_Ratios.txt", "w") do g
        println(g, "******************************************************************************************************")
        println(g, "******************* AA Chronic/Circulating Ratios (no deletions), Sorted by Ratio ********************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_no_dels_chr_all_ratio_ct_sort
             if AA_muts_ct_no_dels[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct_no_dels[w[1]])]", 5)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
                else
                    mutpad = rpad(w[1], 12)
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "******************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios (no deletions), Sorted by Genome Position ***************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_no_dels_chr_all_ratio_pos_sort
             if AA_muts_ct_no_dels[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12) 
                AA_muts_ct_pad = lpad("[$(AA_muts_ct_no_dels[w[1]])]", 5)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
                else
                    mutpad = rpad(w[1], 12)
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_Chr_Circ_Ratios_WITH_DELS.txt", "w") do g
        println(g, "******************************************************************************************************")
        println(g, "******************* AA Chronic/Circulating Ratios (WITH deletions), Sorted by Ratio ********************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_chr_all_ratio_ct_sort
             if AA_muts_ct[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct[w[1]])]", 5)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
                else
                    mutpad = rpad(w[1], 12)
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "******************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios (WITH deletions), Sorted by Genome Position ***************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_chr_all_ratio_pos_sort
             if AA_muts_ct[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12) 
                AA_muts_ct_pad = lpad("[$(AA_muts_ct[w[1]])]", 5)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
                else
                    mutpad = rpad(w[1], 12)
                    println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_Chr_Circ_Ratios_Pos_Only.txt", "w") do g
        println(g, "******************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Ratio ****************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort
            if !(haskey(AA_muts_ct_pos_only_no_dels, w[1]))
                AA_muts_ct_pos_only_no_dels[w[1]] = 0
            end
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                mutpad = rpad(w[1], 12)
                ratiopad = lpad(w[2], 7)
                println(g, "$(mutpad) = $(ratiopad)")
            end
        end
        print(g, "\n"^3)
        println(g, "******************************************************************************************************")
        println(g, "******** AA Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Genome Position ************")
        println(g, "******************************************************************************************************")
        for w in AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                mutpad = rpad(w[1], 12)
                ratiopad = lpad(w[2], 7)
                println(g, "$(mutpad) = $(ratiopad)")
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_Chr_Circ_Ratios_No_Revs.txt", "w") do g
        println(g, "************************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Ratio ***************")
        println(g, "************************************************************************************************************************************")
        for w in AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort
             if AA_muts_ct_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12) 
                if qry_AA ≠ ref_AA 
                    println(g, "$(mutpad) = $(ratiopad)")
                else
                    println(g, "$(mutpad) = $(ratiopad) REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "**********************************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Genome Position ***************")
        println(g, "**********************************************************************************************************************************************")
        for w in AA_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort
             if AA_muts_ct_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                ratiopad = lpad(w[2], 6)
                mutpad = rpad(w[1], 12) 
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(ratiopad)")
                else
                    println(g, "$(mutpad) = $(ratiopad) REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_Chr_Circ_Ratios_Pos_Only_No_Revs.txt", "w") do g
        println(g, "**********************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Ratio ****************")
        println(g, "**********************************************************************************************************************************")
        for w in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort
            if !(haskey(AA_muts_ct_pos_only_no_dels_no_revs, w[1]))
                AA_muts_ct_pos_only_no_dels_no_revs[w[1]] = 0
            end
            if AA_muts_ct_pos_only_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                mutpad = rpad(w[1], 12)
                ratiopad = lpad(w[2], 6)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct_pos_only_no_dels_no_revs[w[1]])]", 5)
                println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
        println(g, "**********************************************************************************************************************************")
        println(g, "********* AA Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Genome Position ***********")
        println(g, "**********************************************************************************************************************************")
        for w in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort
            if AA_muts_ct_pos_only_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                mutpad = rpad(w[1], 12)
                ratiopad = lpad(w[2], 6)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct_pos_only_no_dels_no_revs[w[1]])]", 5)
                println(g, "$(mutpad) = $(ratiopad) $(AA_muts_ct_pad)")
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
#####################################################################################################################################
######################################### End, AA Chr-All Ratio Section #############################################################
#####################################################################################################################################
#####################################################################################################################################
    for (nucmut, Ct) in nuc_muts_ct
        if haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)$(site5)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int]
                        v_all = nuc_muts_ct_all[nucmut]
                        nuc_muts_ct_adj[nucmut] = round(digits=1, Ct/v_all)
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, Ct/v_all)
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = "$(site1)$(site2)$(site3)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, Ct/v_all)
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = "$(site1)$(site2)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, Ct/v_all)
                end
            end
        end
        if !haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)$(site5)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int]
                        nuc_muts_ct_adj[nucmut] = 1001*Ct
                    end
                    if nucmut[end] == ref_seq[nuc_site_int]
                        nuc_muts_ct_adj[nucmut] = 50*Ct
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = "$(site1)$(site2)$(site3)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = "$(site1)$(site2)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
        end
    end
    for (mut, adj) in nuc_muts_ct_adj
        adj_pt = sqrt(nuc_muts_ct_adj[mut])
        ct_pt = (nuc_muts_ct[mut])/adj_score_ct_div
        score = adj_pt + ct_pt
        score_rd = round(digits=2, score)
        nuc_muts_ct_adj_score[mut] = score_rd
    end
    for (nucmut, Ct) in nuc_muts_ct
        if haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)$(site5)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        v_all = nuc_muts_ct_all[nucmut]
                        nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, Ct/v_all)
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, Ct/v_all)
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = "$(site1)$(site2)$(site3)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, Ct/v_all)
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = "$(site1)$(site2)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, Ct/v_all)
                end
            end
        end
        if !haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)$(site5)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                    end
                    if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = "$(site1)$(site2)$(site3)$(site4)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = "$(site1)$(site2)$(site3)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = "$(site1)$(site2)"
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
        end
    end
    for (mut, adj) in nuc_muts_ct_adj
        if mut[end] ≠ '-'
            adj_pt = sqrt(nuc_muts_ct_adj_no_dels[mut])
            ct_pt = (nuc_muts_ct[mut])/adj_score_ct_div
            #adj_pt_rd = round(digits=2, adj_pt)
            #ct_pt_rd = round(digits=2, ct_pt)
            #score = adj_pt_rd + ct_pt_rd
            score = adj_pt + ct_pt
            score_rd = round(digits=2, score)
            nuc_muts_ct_adj_score_no_dels[mut] = score_rd
        end
    end
#####################################################################################################################################
    minimum_count_1 = 50
    minimum_score_1 = 10
    min_comb_score = 100
    for (AAmut, Ct) in AA_muts_ct
        ref_AA = string(split(AAmut, ":")[2][1])
        qry_AA = string(AAmut[end])
        if haskey(AA_muts_ct_all, AAmut)
            v_all = AA_muts_ct_all[AAmut]
            AA_muts_ct_adj[AAmut] = round(digits=1, Ct/v_all)
        end
        if !haskey(AA_muts_ct_all, AAmut)
            if qry_AA == ref_AA 
                AA_muts_ct_adj[AAmut] = round(digits=1, 50*Ct)
            else
                AA_muts_ct_adj[AAmut] = round(digits=1, 1001*Ct)
            end
        end
#####################################################################################################################################    
        if haskey(AA_muts_ct_all, AAmut) && AAmut[end] ≠ '-'
            v_all = AA_muts_ct_all[AAmut]
            AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 1000*Ct/v_all)
        end
        if !haskey(AA_muts_ct_all, AAmut) && AAmut[end] ≠ '-'
            if qry_AA == ref_AA 
                AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 50*Ct)
            else
                AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 1001*Ct)
            end
        end
    end
#####################################################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only
        rev_ct::Int = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
        if haskey(AA_muts_ct_pos_only_all, AAmutpos)
            v_all = AA_muts_ct_pos_only_all[AAmutpos]
            non_rev_adj = 1000*non_rev_ct/v_all
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj[AAmutpos] = round(digits=1, total_adj)
        end
        if !haskey(AA_muts_ct_pos_only_all, AAmutpos)
            non_rev_adj = 1001*non_rev_ct
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj[AAmutpos] = round(digits=1, total_adj)
        end
    end
#####################################################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only_no_dels
        rev_ct::Int = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
        if haskey(AA_muts_ct_pos_only_all, AAmutpos)
            v_all = AA_muts_ct_pos_only_all[AAmutpos]
            non_rev_adj = 1000*non_rev_ct/v_all
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj_no_dels[AAmutpos] = round(digits=1, total_adj)
        end
        if !haskey(AA_muts_ct_pos_only_all, AAmutpos)
            non_rev_adj = 1001*non_rev_ct
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj_no_dels[AAmutpos] = round(digits=1, total_adj)
        end
    end
#####################################################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only_no_dels
        rev_ct::Int = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
    end
#####################################################################################################################################
    for (mut, adj) in AA_muts_ct_adj
        if mut[end] ≠ '*'
            adj_pt = sqrt(AA_muts_ct_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_adj_score[mut] = score_rd
        end
    end
#####################################################################################################################################
    for (mut, adj) in AA_muts_ct_adj
        if mut[end] ≠ '*' && mut[end] ≠ '-'
            adj_pt = sqrt(AA_muts_ct_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_adj_score_no_dels[mut] = score_rd
        end
    end
#####################################################################################################################################  
    for (mut, adj) in AA_muts_ct_pos_only_adj
        if mut[end] ≠ '*'
            adj_pt = sqrt(AA_muts_ct_pos_only_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct_pos_only_adj[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_pos_only_adj_score[mut] = score_rd                
        end
    end
#####################################################################################################################################
    for (mut, adj) in AA_muts_ct_pos_only_adj
        if mut[end] ≠ '*' && mut[end] ≠ '-'
            adj_pt = sqrt(AA_muts_ct_pos_only_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct_pos_only_adj[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_pos_only_adj_score_no_dels[mut] = score_rd                
        end
    end
    AA_pos_int(m) = parse(Int, AA_position(m))      
#####################################################################################################################################
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "M"
            gene_protein_mut_ct["M"] += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF6"
            gene_protein_mut_ct["ORF6"] += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF7a"
            gene_protein_mut_ct["ORF7a"] += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF7b"
            gene_protein_mut_ct["ORF7b"] += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF8"
            gene_protein_mut_ct["ORF8"] += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF9b"
            gene_protein_mut_ct["ORF9b"] += AAmutct
        end
    end
####################################################################################################################################
    for (mut, AAmutct) in AA_muts_ct_no_dels
        if gene_name_fx(mut) == "ORF1a"
            if AA_pos_int(mut) ≤ 180
                gene_protein_mut_ct["NSP1"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(181:818)        
                gene_protein_mut_ct["NSP2"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(819:2763)
                gene_protein_mut_ct["NSP3"] += AAmutct
                for dom in NSP3_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(2764:3263)
                gene_protein_mut_ct["NSP4"] += AAmutct
                for dom in NSP4_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(3264:3569)
                gene_protein_mut_ct["NSP5"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(3570:3859)
                gene_protein_mut_ct["NSP6"] += AAmutct
                for dom in NSP6_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(3860:3942)
                gene_protein_mut_ct["NSP7"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(3943:4140)
                gene_protein_mut_ct["NSP8"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(4141:4253)
                gene_protein_mut_ct["NSP9"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(4253:4392)
                gene_protein_mut_ct["NSP10"] += AAmutct
            elseif AA_pos_int(mut) in BitSet(4392:4400)
                gene_protein_mut_ct["NSP12"] += AAmutct
                domain_mut_ct["NSP12_NiRAN"] += AAmutct
            end
        end
        if gene_name_fx(mut) == "ORF1b"
            if AA_pos_int(mut) ≤ 923
                gene_protein_mut_ct["NSP12"] += AAmutct
                for dom in NSP12_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(924:1524)
                gene_protein_mut_ct["NSP13"] += AAmutct
                for dom in NSP13_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(1525:2051)
                gene_protein_mut_ct["NSP14"] += AAmutct
                for dom in NSP14_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) in BitSet(2052:2397)
                gene_protein_mut_ct["NSP15"] += AAmutct
                for dom in NSP15_domains
                    if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                        domain_mut_ct[dom] += AAmutct
                    end
                end
            elseif AA_pos_int(mut) ≥ 2398
                gene_protein_mut_ct["NSP16"] += AAmutct
            end
        end
        if gene_name_fx(mut) == "S"
            gene_protein_mut_ct["S"] += AAmutct
            for dom in S_domains
                if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                    domain_mut_ct[dom] += AAmutct
                end
            end
        end
        if gene_name_fx(mut) == "ORF3a"
            gene_protein_mut_ct["ORF3a"] += AAmutct
            for dom in ORF3a_domains
                if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                    domain_mut_ct[dom] += AAmutct
                end
            end
        end
        if gene_name_fx(mut) == "E"
            gene_protein_mut_ct["E"] += AAmutct
            for dom in E_domains
                if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                    domain_mut_ct[dom] += AAmutct
                end
            end
        end
        if gene_name_fx(mut) == "N"
            gene_protein_mut_ct["N"] += AAmutct
            for dom in N_domains
                if AA_pos_int(mut) in domain_residues_ORF_BitSet[dom]
                    domain_mut_ct[dom] += AAmutct
                end
            end
        end
    end
##################################################################################################################################
    subs_9_sort = sort(subs_9)
    subs_8_sort = sort(subs_8)
    subs_7_sort = sort(subs_7)
    subs_6_sort = sort(subs_6)
    subs_5_sort = sort(subs_5)
    subs_4_sort = sort(subs_4)
    subs_3_sort = sort(subs_3)
    subs_2_sort = sort(subs_2)
    too_many_reversions_sort = sort(too_many_reversions)
    rep_seq_grps_maxmut_seqs_set = collect(values(rep_seq_grps_maxmut_seqs))
    all_unique_chr_seqs = union(rep_seq_grps_maxmut_seqs_set, non_rep_seqs)
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)__Too_Many_Reversions.txt", "w") do g
        println("######################### Sequences with more than $(revs_thresh) Reversions #########################")
        println(g, "######################### Sequences with more than $(revs_thresh) Reversions #########################")
        for s in too_many_reversions_sort
            print(s, ", "); print(g, s, ", ")
        end
        println(); println(g)
        function seqs_with_n_subs(n::Int, subs_n_sort::Vector{String})
            println(); println(g)
            println("EPI_ISLs for sequences with $(n) private subs + del ranges")
            println(g, "EPI_ISLs for sequences with $(n) private subs + del ranges")
            for seq in subs_n_sort
                if seq in all_unique_chr_seqs
                    print(seq, ", "); print(g, seq, ", ")
                end
            end
            println(); println(g)
        end
        seqs_with_n_subs(8, subs_8_sort)
        seqs_with_n_subs(7, subs_7_sort)
        seqs_with_n_subs(6, subs_6_sort)
        seqs_with_n_subs(5, subs_5_sort)
        seqs_with_n_subs(4, subs_4_sort)
        seqs_with_n_subs(3, subs_3_sort)
        seqs_with_n_subs(2, subs_2_sort)
        println(); print(g, "\n"^2)
    end
##################################################################################################################################
    function AA_mutations_in_seq(g::IO, EPI_ISL::String)        
        println(g, "Mutations in sequence $(EPI_ISL)")
        collected_muts = collect(seq_AA_muts[EPI_ISL])
        mut_list = join(collected_muts, ", ")
        println(g, mut_list)
    end
    function print_seqs_with_AA_mut_plus_their_other_AA_muts(g::IO, mut::String)
        println(g, "$(mut) sequences")
        if haskey(AA_muts_seq, mut)
            seq_list = collect(AA_muts_seq[mut])
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************** Mutations in Sequences with $(mut) **********************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            for seq in seq_list
                AA_mutations_in_seq(g, seq)
                seq_mut_total = length(seq_AA_muts[seq])
                seq_pad = rpad(seq, 16)
                seq_mut_total_pad = lpad(seq_mut_total, 2)
                println(g, "Total Private Mutations in $(seq_pad) = $(seq_mut_total_pad)")
                println(g)
            end
        end
    end
    function nuc_muts_in_seq(g::IO, EPI_ISL)
        println(g, "Nuc Mutations in sequence ", EPI_ISL)
        seq_mut_list = join(collect(seq_AA_muts[EPI_ISL]), ", ")
        println(g, seq_mut_list)
    end
    function print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g::IO, nuc_mut::String)
        println("$(nuc_mut) sequences"); println(g, "$(nuc_mut) sequences")
        if haskey(nuc_muts_seq, nuc_mut)
            seq_list = collect(nuc_muts_seq[nuc_mut])
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "******************** Mutations in Sequences with $(nuc_mut) ********************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            for seq in seq_list
                nuc_muts_in_seq(g, seq)
                seq_mut_total = length(seq_nuc_muts[seq])
                seq_pad = rpad(seq, 16)
                seq_mut_total_pad = lpad(seq_mut_total, 2)
                println(g, "Total Private Mutations in $(seq_pad) = $(seq_mut_total_pad)"); println(g)
            end
        end
    end
    function print_pos_only_adj(g::IO, gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
        print(g, "\n"^2)
        println(g, "***************** $(gene_name) AA Mutations *POS ONLY* Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) *****************")
        for w in AA_muts_ct_pos_only_adj_sort_by_seq_ct
            AAmut = string(w[1])
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            AAmutct_pad = lpad("[$(AAmutct)]", 5)
            len = length(gene_char_num)
            if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)    ## get(collection, key, default_value)
                if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                    mutpad = rpad(AAmut, 12)
                    countpad = lpad(AAct, 3)
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                end
            end
        end
    end
    function print_pos_only_adj_score(g::IO, gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
        print(g, "\n"^2)
        println(g, "***************** $(gene_name) AA Mutations *POS ONLY* Sorted by Adjusted SCORE (Ratio of Chronic:Circulating Private Muts) *****************")
        for w in AA_muts_ct_pos_only_adj_score_sort_by_score
            AAmut = string(w[1])
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            AAmutct_pad = lpad("[$(AAmutct)]", 5)
            len = length(gene_char_num)
            if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)    ## get(collection, key, default_value)
                if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                    mutpad = rpad(AAmut, 12)
                    countpad = lpad(AAct, 3)
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                end
            end
        end
    end
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_rep_seq_grps.txt", "w") do g
        println(g)
        rep_seq_grps_seqs_grp_size_sort = sort(collect(rep_seq_grps_seqs), by = x -> length(x[2]), rev=true)
        println(g, "#############################################################################################")
        println(g, "############################### Groups with ≥5 Sequences ####################################")
        println(g, "#############################################################################################")
        for (grp_num, seq_EPIs_set) in rep_seq_grps_seqs_grp_size_sort
            seq_EPIs_arr = collect(seq_EPIs_set)
            seq_EPIs_sort = sort(seq_EPIs_arr)
            seq_EPIs_sort_str = join(seq_EPIs_sort, ", ")
            seq_EPI_set_len = length(seq_EPIs_set)
            if seq_EPI_set_len ≥ 5
                println(g)
                println(g, "#############################################################################################")
                println(g, "Group Number ", grp_num, "    Total Sequences in Group = ", seq_EPI_set_len)
                println(g, "EPI_ISLs in Group Number ", grp_num, " = ", seq_EPIs_sort_str)
                println(g)
                rep_seq_grps_AA_sort = sort(collect(rep_seq_grps_AA[grp_num]))
                rep_seq_grps_muts_sort = sort(collect(rep_seq_grps_muts[grp_num]))
                println(g, "Collective Mutations in Group Number $(grp_num)")
                for mut in rep_seq_grps_muts_sort
                    print(g, mut, ", ")
                end
                println(g)
                println(g, "Collective AA Mutations in Group Number $(grp_num)")
                for AA in rep_seq_grps_AA_sort
                    print(g, AA, ", ")
                end
                println(g)
            end
            println(g, "#############################################################################################")
            println(g, "############################### Groups with <5 Sequences ####################################")
            println(g, "#############################################################################################")
            if seq_EPI_set_len < 5
                println(g)
                println(g, "#############################################################################################")
                println(g, "Group Number ", grp_num, "    Total Sequences in Group = ", seq_EPI_set_len)
                println(g, "EPI_ISLs in Group Number ", grp_num, " = ", seq_EPIs_sort_str)
                println(g)
                rep_seq_grps_AA_sort = sort(collect(rep_seq_grps_AA[grp_num]))
                rep_seq_grps_muts_sort = sort(collect(rep_seq_grps_muts[grp_num]))                        
                println(g, "Collective Mutations in Group Number $(grp_num)")
                for mut in rep_seq_grps_muts_sort
                    print(g, mut, ", ")
                end
                println(g)
                println(g, "Collective AA Mutations in Group Number $(grp_num)")
                for AA in rep_seq_grps_AA_sort
                    print(g, AA, ", ")
                end
                println(g)
            end
        end
    end
    nuc_num_from_key(k) = parse(Int, k[2:end-1])
    nuc_muts_ct_sort_by_site = sort(collect(nuc_muts_ct), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_sort_by_seq_ct = sort(collect(nuc_muts_ct), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_sort_by_site = sort(collect(nuc_muts_ct_adj), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_sort_by_seq_ct = sort(collect(nuc_muts_ct_adj), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_no_dels_sort_by_site = sort(collect(nuc_muts_ct_adj_no_dels), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_no_dels_sort_by_seq_ct = sort(collect(nuc_muts_ct_adj_no_dels), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_score_sort_by_site = sort(collect(nuc_muts_ct_adj_score), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_score_sort_by_score = sort(collect(nuc_muts_ct_adj_score), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_score_no_dels_sort_by_site = sort(collect(nuc_muts_ct_adj_score_no_dels), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_score_no_dels_sort_by_score = sort(collect(nuc_muts_ct_adj_score_no_dels), by = x -> x[2], rev = true)
#####################################################################################################################################
    date = Dates.format(today(), "yyyy_mm_dd")
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_NUCS.txt", "a") do g       
        print(g, "\n"^2)
        println(g, "************** TOP NUC MUTATIONS Sorted by Count ***************")
        for w in nuc_muts_ct_adj_sort_by_seq_ct
            nucmut = w[1]
            nucct = w[2]
            nucmut_pad = rpad(nucmut, 7)
            nucct_pad = lpad(nucct, 3)
            nuc_muts_ct_pad = lpad("[$(nuc_muts_ct[nucmut])]", 3)
            if nuc_muts_ct[nucmut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nucmut[2:end-1])
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc_str = string(nucmut[end])
                    ref_seq_nuc_str = string(nucmut[1])
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, "$(nucmut_pad) = $(nucct_pad) $(nuc_muts_ct_pad)") 
                    else
                        println(g, "$(nucmut_pad) = $(nucct_pad) $(nuc_muts_ct_pad) REVERSION)") 
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "************** TOP NUC MUTATIONS Sorted by Genome Position ***************")
        for w in nuc_muts_ct_adj_sort_by_site
            nucmut = w[1]
            nucct = w[2]
            nucmut_pad = rpad(nucmut, 7)
            nucct_pad = lpad(nucct, 3)
            nuc_muts_ct_pad = lpad("[$(nuc_muts_ct[nucmut])]", 3)
            if nuc_muts_ct[nucmut] ≥ print_ct_thresh
                nuc_mut_pos_int = parse(Int, nucmut[2:end-1])
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc_str = string(nucmut[end])
                    ref_seq_nuc_str = string(nucmut[1])
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, "$(nucmut_pad) = $(nucct_pad) $(nuc_muts_ct_pad)") 
                    else
                        println(g, "$(nucmut_pad) = $(nucct_pad) $(nuc_muts_ct_pad) REVERSION)") 
                    end
                end
            end
        end
    end
#####################################################################################################################################
    AA_muts_ct_sort_by_site = sort(collect(AA_muts_ct), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_sort_by_seq_ct = sort(collect(AA_muts_ct), by = x -> x[2], rev = true)

    AA_muts_ct_no_dels_sort_by_site = sort(collect(AA_muts_ct_no_dels), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_no_dels), by = x -> x[2], rev = true)

    AA_muts_ct_pos_only_sort_by_site = sort(collect(AA_muts_ct_pos_only), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only), by = x -> x[2], rev = true)
    AA_muts_ct_pos_only_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_no_dels), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_no_dels), by = x -> x[2], rev = true)
    
    AA_muts_ct_adj_sort_by_site = sort(collect(AA_muts_ct_adj), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_adj_sort_by_seq_ct = sort(collect(AA_muts_ct_adj), by = x -> x[2], rev = true)
    AA_muts_ct_adj_no_dels_sort_by_site = sort(collect(AA_muts_ct_adj_no_dels), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_adj_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_adj_no_dels), by = x -> x[2], rev = true)

    AA_muts_ct_adj_score_sort_by_site = sort(collect(AA_muts_ct_adj_score), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_adj_score_sort_by_score = sort(collect(AA_muts_ct_adj_score), by = x -> x[2], rev = true)
    AA_muts_ct_adj_score_no_dels_sort_by_site = sort(collect(AA_muts_ct_adj_score_no_dels), by = x -> AA_gene_sortKey_2(x[1]))
    AA_muts_ct_adj_score_no_dels_sort_by_score = sort(collect(AA_muts_ct_adj_score_no_dels), by = x -> x[2], rev = true)
#    AA_dels_ct_sort_by_site = sort(collect(AA_dels_ct), by = x -> AA_gene_sortKey_2(x[1]))
#    AA_dels_ct_sort_by_ct = sort(collect(AA_dels_ct), by = x -> x[2], rev=true)
#####################################################################################################################################  
#    NSP1 = 1:180       NSP2 = 181:818     NSP3 = 819:2763    NSP4 = 2764:3264   
#    NSP5 = 3264:3569   NSP6 = 3570:3859   NSP7 = 3860:3942   NSP8 = 3943:4140
#    NSP9 = 4141:4253   NSP10 = 4254:4392  NSP12 = 4392:4401, 1:923
#    NSP13 = 924:1524   NSP14 = 1525:2051  NSP15 = 2052:2397  NSP16 = 2397:2696
#####################################################################################################################################
    NSPmutPosKey(n) = parse(Int, split(n, ":")[2][2:end-1])
    NSPmutPosKey2(n) = parse(Int, split(n, ":")[2])
    for i in 1:16
        if i ≠ 11
            NSP_muts["NSP$(i)"] = Dict{String, Int}()
            NSP_muts_no_dels["NSP$(i)"] = Dict{String, Int}()
        end
    end  
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_TOP_NSP_MUTS.txt", "w") do g 
        print(g, "\n"^2)
        println(g, "********************** TOP AA MUTS by NSP ************************")
        for w in AA_muts_ct
            AAmut = string(w[1])
            AAct = w[2]
            jeen = string(split(AAmut, ":")[1])
            AA_mut_pos_int = parse(Int, split(AAmut, ":")[2][2:end-1])
            ref_AA_str = string(split(AAmut, ":")[2][1])
            qry_AA = string(AAmut[end])
            ref_AA = string(split(AAmut, ":")[2][1])
            for i in 1:12
                if i ≠ 11
                    if jeen == "ORF1a" && AA_mut_pos_int in NSP_ranges1a[i]
                        NSP_pos = AA_mut_pos_int - NSP1a_add[i]
                        if qry_AA == ref_AA
                            NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            if qry_AA ≠ "-"
                                NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            end
                        else
                            NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            if qry_AA ≠ "-"
                                NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            end
                        end
                    end
                end
            end
            for i in 12:16
                if jeen == "ORF1b" && AA_mut_pos_int in NSP_ranges1b[i]
                    NSP_pos = AA_mut_pos_int - NSP1b_add[i]
                    if qry_AA == ref_AA
                        NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        if qry_AA ≠ "-"
                            NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        end
                    else
                        NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        if qry_AA ≠ "-"
                            NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        end
                    end
                end
            end
        end
        NSP1_muts_sortByCt = sort(collect(NSP_muts["NSP1"]), by = x -> x[2], rev=true)
        NSP1_muts_sortByPos = sort(collect(NSP_muts["NSP1"]), by = x -> NSPmutPosKey(x[1]))
        NSP2_muts_sortByCt = sort(collect(NSP_muts["NSP2"]), by = x -> x[2], rev=true)
        NSP2_muts_sortByPos = sort(collect(NSP_muts["NSP2"]), by = x -> NSPmutPosKey(x[1]))
        NSP3_muts_sortByCt = sort(collect(NSP_muts["NSP3"]), by = x -> x[2], rev=true)
        NSP3_muts_sortByPos = sort(collect(NSP_muts["NSP3"]), by = x -> NSPmutPosKey(x[1]))
        NSP4_muts_sortByCt = sort(collect(NSP_muts["NSP4"]), by = x -> x[2], rev=true)
        NSP4_muts_sortByPos = sort(collect(NSP_muts["NSP4"]), by = x -> NSPmutPosKey(x[1]))
        NSP5_muts_sortByCt = sort(collect(NSP_muts["NSP5"]), by = x -> x[2], rev=true)
        NSP5_muts_sortByPos = sort(collect(NSP_muts["NSP5"]), by = x -> NSPmutPosKey(x[1]))
        NSP6_muts_sortByCt = sort(collect(NSP_muts["NSP6"]), by = x -> x[2], rev=true)
        NSP6_muts_sortByPos = sort(collect(NSP_muts["NSP6"]), by = x -> NSPmutPosKey(x[1]))
        NSP7_muts_sortByCt = sort(collect(NSP_muts["NSP7"]), by = x -> x[2], rev=true)
        NSP7_muts_sortByPos = sort(collect(NSP_muts["NSP7"]), by = x -> NSPmutPosKey(x[1]))
        NSP8_muts_sortByCt = sort(collect(NSP_muts["NSP8"]), by = x -> x[2], rev=true)
        NSP8_muts_sortByPos = sort(collect(NSP_muts["NSP8"]), by = x -> NSPmutPosKey(x[1]))
        NSP9_muts_sortByCt = sort(collect(NSP_muts["NSP9"]), by = x -> x[2], rev=true)
        NSP9_muts_sortByPos = sort(collect(NSP_muts["NSP9"]), by = x -> NSPmutPosKey(x[1]))
        NSP10_muts_sortByCt = sort(collect(NSP_muts["NSP10"]), by = x -> x[2], rev=true)
        NSP10_muts_sortByPos = sort(collect(NSP_muts["NSP10"]), by = x -> NSPmutPosKey(x[1]))
        NSP12_muts_sortByCt = sort(collect(NSP_muts["NSP12"]), by = x -> x[2], rev=true)
        NSP12_muts_sortByPos = sort(collect(NSP_muts["NSP12"]), by = x -> NSPmutPosKey(x[1]))
        NSP13_muts_sortByCt = sort(collect(NSP_muts["NSP13"]), by = x -> x[2], rev=true)
        NSP13_muts_sortByPos = sort(collect(NSP_muts["NSP13"]), by = x -> NSPmutPosKey(x[1]))
        NSP14_muts_sortByCt = sort(collect(NSP_muts["NSP14"]), by = x -> x[2], rev=true)
        NSP14_muts_sortByPos = sort(collect(NSP_muts["NSP14"]), by = x -> NSPmutPosKey(x[1]))
        NSP15_muts_sortByCt = sort(collect(NSP_muts["NSP15"]), by = x -> x[2], rev=true)
        NSP15_muts_sortByPos = sort(collect(NSP_muts["NSP15"]), by = x -> NSPmutPosKey(x[1]))
        NSP16_muts_sortByCt = sort(collect(NSP_muts["NSP16"]), by = x -> x[2], rev=true)
        NSP16_muts_sortByPos = sort(collect(NSP_muts["NSP16"]), by = x -> NSPmutPosKey(x[1]))
######################################################################################################################################
        NSP11_placeholder = Vector{Pair{String, Int64}}
######################################################################################################################################
        NSP1_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP1"]), by = x -> x[2], rev=true)
        NSP1_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP1"]), by = x -> NSPmutPosKey(x[1]))
        NSP2_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP2"]), by = x -> x[2], rev=true)
        NSP2_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP2"]), by = x -> NSPmutPosKey(x[1]))
        NSP3_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP3"]), by = x -> x[2], rev=true)
        NSP3_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP3"]), by = x -> NSPmutPosKey(x[1]))
        NSP4_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP4"]), by = x -> x[2], rev=true)
        NSP4_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP4"]), by = x -> NSPmutPosKey(x[1]))
        NSP5_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP5"]), by = x -> x[2], rev=true)
        NSP5_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP5"]), by = x -> NSPmutPosKey(x[1]))
        NSP6_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP6"]), by = x -> x[2], rev=true)
        NSP6_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP6"]), by = x -> NSPmutPosKey(x[1]))
        NSP7_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP7"]), by = x -> x[2], rev=true)
        NSP7_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP7"]), by = x -> NSPmutPosKey(x[1]))
        NSP8_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP8"]), by = x -> x[2], rev=true)
        NSP8_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP8"]), by = x -> NSPmutPosKey(x[1]))
        NSP9_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP9"]), by = x -> x[2], rev=true)
        NSP9_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP9"]), by = x -> NSPmutPosKey(x[1]))
        NSP10_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP10"]), by = x -> x[2], rev=true)
        NSP10_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP10"]), by = x -> NSPmutPosKey(x[1]))
        NSP12_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP12"]), by = x -> x[2], rev=true)
        NSP12_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP12"]), by = x -> NSPmutPosKey(x[1]))
        NSP13_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP13"]), by = x -> x[2], rev=true)
        NSP13_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP13"]), by = x -> NSPmutPosKey(x[1]))
        NSP14_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP14"]), by = x -> x[2], rev=true)
        NSP14_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP14"]), by = x -> NSPmutPosKey(x[1]))
        NSP15_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP15"]), by = x -> x[2], rev=true)
        NSP15_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP15"]), by = x -> NSPmutPosKey(x[1]))
        NSP16_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP16"]), by = x -> x[2], rev=true)
        NSP16_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP16"]), by = x -> NSPmutPosKey(x[1]))
######################################################################################################################################
        NSP_muts_sortByCt_Arr = [NSP1_muts_sortByCt, NSP2_muts_sortByCt, NSP3_muts_sortByCt, NSP4_muts_sortByCt, NSP5_muts_sortByCt, NSP6_muts_sortByCt, NSP7_muts_sortByCt, NSP8_muts_sortByCt, NSP9_muts_sortByCt, NSP10_muts_sortByCt, NSP11_placeholder, NSP12_muts_sortByCt, NSP13_muts_sortByCt, NSP14_muts_sortByCt, NSP15_muts_sortByCt, NSP16_muts_sortByCt]
        NSP_muts_sortByPos_Arr = [NSP1_muts_sortByPos, NSP2_muts_sortByPos, NSP3_muts_sortByPos, NSP4_muts_sortByPos, NSP5_muts_sortByPos, NSP6_muts_sortByPos, NSP7_muts_sortByPos, NSP8_muts_sortByPos, NSP9_muts_sortByPos, NSP10_muts_sortByPos, NSP11_placeholder, NSP12_muts_sortByPos, NSP13_muts_sortByPos, NSP14_muts_sortByPos, NSP15_muts_sortByPos, NSP16_muts_sortByPos]
        NSP_muts_no_dels_sortByCt_Arr = [NSP1_muts_no_dels_sortByCt, NSP2_muts_no_dels_sortByCt, NSP3_muts_no_dels_sortByCt, NSP4_muts_no_dels_sortByCt, NSP5_muts_no_dels_sortByCt, NSP6_muts_no_dels_sortByCt, NSP7_muts_no_dels_sortByCt, NSP8_muts_no_dels_sortByCt, NSP9_muts_no_dels_sortByCt, NSP10_muts_no_dels_sortByCt, NSP11_placeholder, NSP12_muts_no_dels_sortByCt, NSP13_muts_no_dels_sortByCt, NSP14_muts_no_dels_sortByCt, NSP15_muts_no_dels_sortByCt, NSP16_muts_no_dels_sortByCt]
        NSP_muts_no_dels_sortByPos_Arr = [NSP1_muts_no_dels_sortByPos, NSP2_muts_no_dels_sortByPos, NSP3_muts_no_dels_sortByPos, NSP4_muts_no_dels_sortByPos, NSP5_muts_no_dels_sortByPos, NSP6_muts_no_dels_sortByPos, NSP7_muts_no_dels_sortByPos, NSP8_muts_no_dels_sortByPos, NSP9_muts_no_dels_sortByPos, NSP10_muts_no_dels_sortByPos, NSP11_placeholder, NSP12_muts_no_dels_sortByPos, NSP13_muts_no_dels_sortByPos, NSP14_muts_no_dels_sortByPos, NSP15_muts_no_dels_sortByPos, NSP16_muts_no_dels_sortByPos]
        for i in 1:16
            if i ≠ 11
                println(g)
                println(g, "************* NSP$(i) Muts Sorted by Ct *************")
                for w in NSP_muts_sortByCt_Arr[i]
                    mutpad = rpad(w[1], 12)
                    ctpad = lpad(w[2], 3)
                    println(g, "$(mutpad) = $(ctpad)")
                end
            end
        end
        for i in 1:16
            if i ≠ 11
                println(g)
                println(g, "************* NSP$(i) Muts Sorted by Pos *************")
                for w in NSP_muts_sortByPos_Arr[i]
                    mutpad = rpad(w[1], 12)
                    ctpad = lpad(w[2], 3)
                    println(g, "$(mutpad) = $(ctpad)")
                end
            end
        end      
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_NUC_COUNTS.txt", "w") do g   
        print(g, "\n"^3)
        println(g, "************** TOP NUC MUTATIONS Sorted by Count ***************")
        for w in nuc_muts_ct_sort_by_seq_ct
            if nuc_muts_ct[w[1]] ≥ print_ct_thresh
                nucmut = string(w[1])
                nucmut_pad = rpad(nucmut, 7)
                nuc_mut_pos_int = parse(Int, w[1][2:end-1])
                nucct_pad = lpad("$(w[2])", 3)
                nuc_muts_ct_pad = lpad("[$(nuc_muts_ct[nucmut])", 3)
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc_str = string(w[1][end])
                    ref_seq_nuc_str = string(nucmut[1])
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, "$(nucmut) = $(nucct_pad) $(nuc_muts_ct_pad)")
                    else
                        println(g, "$(nucmut) = $(nucct_pad) $(nuc_muts_ct_pad) REVERSION") 
                    end
                end
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP NUC MUTATIONS Sorted by Genome Position ***************")
        for w in nuc_muts_ct_sort_by_site
            if nuc_muts_ct[w[1]] ≥ print_ct_thresh
                nucmut = string(w[1])
                nucmut_pad = rpad(nucmut, 7)
                nuc_mut_pos_int = parse(Int, w[1][2:end-1])
                nucct_pad = lpad("$(w[2])", 3)
                nuc_muts_ct_pad = lpad("[$(nuc_muts_ct[nucmut])", 3)
                if nuc_mut_pos_int ≤ 29850    
                    qry_nuc_str = string(w[1][end])
                    ref_seq_nuc_str = string(nucmut[1])
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, "$(nucmut) = $(nucct_pad) $(nuc_muts_ct_pad)")
                    else
                        println(g, "$(nucmut) = $(nucct_pad) $(nuc_muts_ct_pad) REVERSION") 
                    end
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    gene_array = ["ORF1a", "ORF1b", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF9b"]
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_COUNTS.txt", "w") do g               
        print(g, "\n"^3)
        println(g, "************** TOP AA MUTATIONS Sorted by Count ***************")
        for w in AA_muts_ct_sort_by_seq_ct
            if AA_muts_ct[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                countpad = lpad(w[2], 3)
                mutpad = rpad(w[1], 12)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct[w[1]])]", 5)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(countpad)")
                else
                    mutpad = rpad(w[1], 12)
                     println(g, "$(mutpad) = $(countpad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP AA MUTATIONS Sorted by Genome Position ***************")
        for w in AA_muts_ct_sort_by_site
             if AA_muts_ct[w[1]] ≥ print_ct_thresh
                AAmut = string(w[1])
                AAmut = string(w[1])
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                countpad = lpad(w[2], 3)
                mutpad = rpad(w[1], 12)
                AA_muts_ct_pad = lpad("[$(AA_muts_ct[w[1]])]", 3)
                if qry_AA ≠ ref_AA
                    println(g, "$(mutpad) = $(countpad)")
                else
                    mutpad = rpad(w[1], 12)
                    println(g, "$(mutpad) = $(countpad) REVERSION")
                end
            end
        end
        print(g, "\n"^3)
        for orf in gene_array
            println(g, "**************** $(orf) AA Mutations Sorted by Count *****************")
            for w in AA_muts_ct_sort_by_seq_ct
                gn = string(split(w[1], ":")[1])
                if gn == orf
                    AAmut = string(w[1])
                    ref_AA = string(split(AAmut, ":")[2][1])
                    qry_AA = string(AAmut[end])
                    mutpad = rpad(w[1], 12)
                    countpad = lpad(w[2], 3)
                    all_ct_pad = lpad("[$(AA_muts_ct[AAmut])]", 3)
                    if qry_AA ≠ ref_AA
                        println(g, "$(mutpad) = $(countpad) $(all_ct_pad)")
                    else
                        println(g, "$(mutpad) = $(countpad) $(all_ct_pad) REVERSION")
                    end
                end
            end
            print(g, "\n"^4)
        end
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_COUNTS_POS_ONLY_NO_DELS.txt", "w") do g               
        print(g, "\n"^3)
        println(g, "************** TOP AA Mutations Pos Only No Dels Sorted by Count ***************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            AAmut = string(w[1])
            countpad = lpad(w[2], 3)
            mutpad = rpad(AAmut, 12)
            println(g, "$(mutpad) = $(countpad)")
        end
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                mutpad = rpad(string(w[1]), 12)
                ct_pad = lpad(w[2], 3)
                println(g, "$(mutpad) = $(ct_pad)")
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP AA Mutations Pos Only No Dels Sorted by Genome Position ***************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_site
             if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                mutpad = rpad(w[1], 12)
                ct_pad = lpad(w[2], 7)
                println(g, "$(mutpad) = $(ct_pad)")
             end
        end
    end
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_COUNTS_POS_ONLY_NO_DELS.txt", "w") do g               
        for orf in gene_array
            println(g, "**************** $(orf) AA Mutations Pos Only No Dels, Sorted by Count *****************")
            for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
                AAmut = string(w[1])
                jeen = string(split(AAmut, ":")[1])
                if jeen == orf && AA_muts_ct_pos_only_no_dels[AAmut] ≥ print_ct_thresh
                    countpad = lpad(w[2], 3)
                    mutpad = lpad(AAmut, 12)
                    println(g, "$(mutpad) = $(countpad)")
                end
            end
        end
        print(g, "\n"^3)
    end
#####################################################################################################################################
    date = Dates.format(today(), "yyyy_mm_dd")   
    minimum_count_1 = 50
    minimum_score_1 = 10
    min_comb_score = 100
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_CHRONIC_SEARCH_MUTS.txt", "w") do g        
        print(g, "\n"^2)
        println(g, "********************** TOP AA MUTS for Chronic Search, CovSpectrum ************************")
        key_pre2(n) = (length(n), split(n, ":")[2][2:end-1])
        Cov_sortkey(n) = (split(n, ":")[1], key_pre2(n))
        CovSpec_muts = Set{String}()
#        open("CovSpectrum_Search_Muts.txt", "w") do h
#            for w in AA_muts_ct_adj_sort_by_seq_ct
#                AAmut = string(w[1])
#                AAct = w[2]
#                qry_AA = string(AAmut[end])
#                ref_AA = string(split(AAmut, ":")[2][1])
#                countpad = lpad(w[2], 3)
#                if AA_muts_ct[AAmut] > 5 && AAct > 20 || AA_muts_ct[AAmut] > 5 && AA_muts_ct[AAmut] + AAct > min_comb_score
#                    if qry_AA ≠ ref_AA && AAmut[end] ≠ '-' && AAmut[end] ≠ '*'
#                        push!(CovSpec_muts, AAmut)
#                    end
#                end
#            end
#            CovSpec_sort = sort(collect(CovSpec_muts), by = x -> Cov_sortkey(x))
#            for mut in CovSpec_sort
#                print(h, mut, ", ")
#            end
#        end
        print(g, "\n"^2)
        println(g, "********************** TOP AA MUTS for Chronic Search ************************")
        for w in AA_muts_ct_adj_sort_by_seq_ct 
            AAmut = string(w[1])
            AAct = w[2]
            qry_AA = string(AAmut[end])
            ref_AA = string(split(AAmut, ":")[2][1])
            AAmutct = AA_muts_ct[AAmut]
            AAmutct_pad = lpad("[$(AAmutct)]", 5)
            if AA_muts_ct[AAmut] + AAct*10 ≥ min_comb_score || AAct > 20
                if qry_AA ≠ ref_AA
                    push!(chronic_search_muts, AAmut)
                    mutpad = rpad(AAmut, 12)
                    countpad = lpad(AAct, 3)
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                else
                    mutpad = rpad(AAmut, 12)
                    countpad = lpad(AAct, 3)
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad), REVERSION")
                end
            end
        end
    end
#####################################################################################################################################
######################################### Chronic Search Muts Version 2, Improved #####################################################
#####################################################################################################################################
    minimum_count_1 = 2
#    minimum_score_1 = 10
    min_comb_score = 125
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_CHRONIC_SEARCH_MUTS_v2_improved.txt", "w") do g
        println(g, "********************** TOP AA MUTS for Chronic Search: New & Improved Version ************************")
        for AAmut___ratio in AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort 
            AAmut = string(AAmut___ratio[1])
            ratio = AAmut___ratio[2]
            qry_AA = string(AAmut[end])
            ref_AA_str = string(split(AAmut, ":")[2][1])
            ref_AA = string(split(AAmut, ":")[2][1])
            AAmutct = AA_muts_ct_no_dels_no_revs[AAmut]
            AAmutct_pad = lpad("[$(AAmutct)]", 5)
            ratiopad = lpad(ratio, 6)
            if AA_muts_ct_no_dels_no_revs[AAmut]*ratio ≥ min_comb_score || ratio > 10 && AAmutct ≥ minimum_count_1
                if qry_AA ≠ ref_AA
                    push!(chronic_search_muts_v2, AAmut)
                    mutpad = rpad(AAmut, 12)
                    println(g, "$(mutpad) = $(ratio), $(AAmutct_pad)")
                else
                    push!(chronic_search_muts_v2, AAmut)
                    mutpad = rpad(AAmut, 12)
                    println(g, "$(mutpad) = $(ratio), $(AAmutct_pad), REVERSION")
                end
            end
        end
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_ADJ_SORTED_BY_POS.txt", "w") do g        
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS, Adjusted, Sorted by Genome Position ***************")
        for w in AA_muts_ct_adj_sort_by_site
            AAmut = string(w[1])
            if AA_muts_ct[AAmut] ≥ print_ct_thresh
                AAct = w[2]
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                countpad = lpad(AAct, 3)
                AAmutct = AA_muts_ct[AAmut]
                AAmutct_pad = lpad("[$(AAmutct)]", 5)
                mutpad = rpad(AAmut, 12)
                if qry_AA ≠ ref_AA 
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                else
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad), REVERSION")
                end
            end
        end
        print(g, "\n"^2)
    end  
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_ADJ_SCORE.txt", "w") do g        
        println(g, "************** TOP AA MUTATIONS Sorted by Adjusted SCORE = sqrt(chronic/circulating ratio) + (raw chronic mut ct)/$(adj_score_ct_div) ***************")
        for w in AA_muts_ct_adj_score_sort_by_score
            AAmut = string(w[1])
            AAct = w[2]            
            ref_AA = string(split(AAmut, ":")[2][1])
            qry_AA = string(AAmut[end])
            mutpad = rpad(AAmut, 12)
            adj_score_pad = lpad(AAct, 6)
            raw_ct_pad = lpad(AA_muts_ct[AAmut], 3)
            adj_ct_pad = lpad(AA_muts_ct_adj[AAmut], 6)
            if qry_AA ≠ ref_AA 
                mutpad = rpad(AAmut, 12)
                println(g, "$(mutpad) = $(adj_score_pad) ($(raw_ct_pad), $(adj_ct_pad))")
            else
                mutpad = rpad(AAmut, 12)
                println(g, "$(mutpad) = $(adj_score_pad) ($(raw_ct_pad), $(adj_ct_pad)), REVERSION")
            end
        end
        print(g, "\n"^2)   
        println(g, "************** TOP AA MUTATIONS Adjusted Sorted by Genome Position, w/Adjusted SCORE = sqrt(chronic/circulating ratio) + (raw chronic mut ct)/$(adj_score_ct_div) ***************")  
        for w in AA_muts_ct_adj_score_sort_by_site
            AAmut = string(w[1])
            if AA_muts_ct[AAmut] ≥ 2
                AAct = w[2]
                ref_AA = string(split(AAmut, ":")[2][1])
                qry_AA = string(AAmut[end])
                mutpad = rpad(AAmut, 12)
                adj_score_pad = lpad(AAct, 6)
                raw_ct_pad = lpad(AA_muts_ct[AAmut], 3)
                adj_ct_pad = lpad(AA_muts_ct_adj[AAmut], 6)
                if qry_AA ≠ ref_AA 
                    println(g, "$(mutpad) = $(adj_score_pad) ($(raw_ct_pad), $(adj_ct_pad))")
                else
                    println(g, "$(mutpad) = $(adj_score_pad) ($(raw_ct_pad), $(adj_ct_pad)), REVERSION")
                end
            end
        end
        print(g, "\n"^2)
        mutkey_1000 = AA_muts_ct_adj_score_sort_by_score[1000][1]
        mut_score_1000 = AA_muts_ct_adj_score_sort_by_score[1000][2]
        function nth_ranked_score(n)
            mutkey = AA_muts_ct_adj_score_sort_by_score[n][1]
            println(g)
            mut_score = AA_muts_ct_adj_score_sort_by_score[n][2]
            adj_ct = AA_muts_ct_adj[mutkey][1]
            raw_ct = AA_muts_ct[mutkey]
            println(g, "$(n)th-ranked AA Mut Score, (Adj_ct, Raw Ct); $(mutkey) = $(mut_score), ($(adj_ct), $(raw_ct))")
            println(g)
        end
        nth_ranked_score(2000); nth_ranked_score(1000); nth_ranked_score(750); nth_ranked_score(500)
        nth_ranked_score(400);  nth_ranked_score(300);  nth_ranked_score(200); nth_ranked_score(100)
        nth_ranked_score(50);   nth_ranked_score(25);   nth_ranked_score(10);  nth_ranked_score(5);   nth_ranked_score(1)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_AA_ADJ_SORT_BY_COUNT.txt", "w") do g            
        println(g, "************** TOP AA MUTATIONS Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) ***************")
        for w in AA_muts_ct_adj_sort_by_seq_ct
            AAmut = string(w[1])
            if AA_muts_ct[AAmut] ≥ print_ct_thresh
                AAct = w[2]
                qry_AA = string(AAmut[end])
                ref_AA = string(split(AAmut, ":")[2][1])
                AAmutct = AA_muts_ct[AAmut]
                AAmutct_pad = lpad("[$(AAmutct)]", 5)
                mutpad = rpad(AAmut, 12)
                countpad = lpad(AAct, 3)
                if qry_AA ≠ ref_AA 
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                else
                    println(g, "$(mutpad) = $(countpad) $(AAmutct_pad), REVERSION")
                end
            end
        end
        print(g, "\n"^2)
        function AA_median_Q1_Q3_etc(key, key_name)
            println(g, "$(key_name) AA Adj Number = ", key)
            println(g, "$(key_name) AA Adj Mutation = ", AA_muts_ct_adj_sort_by_seq_ct[key][2])
            println(g, "$(key_name) AA adj Score = ", AA_muts_ct_adj_sort_by_seq_ct[key][1])
            println(g)
        end
#####################################################################################################################################
        kz = keys(AA_muts_ct_adj_sort_by_seq_ct)
        kzln = length(kz)
        half_k = round(digits=0, kzln÷2)
        half_ke = string(half_k)
        half_kee = split(half_ke, ".")[1]
        half_key = parse(Int, half_kee)
        d9k = round(digits=0, kzln÷(10)÷1)
        d9ke = string(d9k)
        d9kee = split(d9ke, ".")[1]
        d9key = parse(Int, d9kee)
        Q3mutk = round(digits=0, half_k÷2)
        Q3mutke = string(Q3mutk)
        Q3mutkee = split(Q3mutke, ".")[1]
        Q3mutkey = parse(Int, Q3mutkee)
        Q1mutk = round(digits=0, half_k + kzln÷2)
        Q1mutke = string(Q1mutk)
        Q1mutkee = split(Q1mutke, ".")[1]
        Q1mutkey = parse(Int, Q1mutkee)
        fac = 10/9
        d1k = round(digits=0, kzln÷fac)
        d1ke = string(d1k)
        d1kee = split(d1ke, ".")[1]
        d1key = parse(Int, d1kee)
#####################################################################################################################################
        AA_median_Q1_Q3_etc(d1key, "d9")
        AA_median_Q1_Q3_etc(Q1mutkey, "Q3")
        AA_median_Q1_Q3_etc(half_key, "Median")
        AA_median_Q1_Q3_etc(Q3mutkey, "Q1")
        AA_median_Q1_Q3_etc(d9key, "d1")
#####################################################################################################################################
        function print_AA_adj_ct_sorted(gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
            println(g)
            println(g, "**************** $(gene_name) AA Mutations Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) *****************")
            for w in AA_muts_ct_adj_sort_by_seq_ct
                AAmut = string(w[1])
                AAct = w[2]
                len = length(gene_char_num)
                if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)     
                    AAmutct = AA_muts_ct[AAmut]
                    AAmutct_pad = lpad("[$(AAmutct)]", 5)
                    qry_AA = AAmut[end]
                    ref_AA = string(split(AAmut, ":")[2][1])
                    if AA_muts_ct[AAmut] ≥ print_ct_thresh
                        mutpad = rpad(AAmut, 12)
                        countpad = lpad(AAct, 3)
                        if qry_AA ≠ ref_AA
                            println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
                        else
                            println(g, "$(mutpad) = $(countpad) $(AAmutct_pad), REVERSION")
                        end
                    end
                end
            end
        end
        print_AA_adj_ct_sorted("ORF1a", [4, 5], ['1', 'a'])
        print_AA_adj_ct_sorted("ORF1b", [4, 5], ['1', 'b'])
        print_AA_adj_ct_sorted("Spike", [1], ['S'])
        print_AA_adj_ct_sorted("E", [1], ['E'])
        print_AA_adj_ct_sorted("M", [1], ['M'])
        print_AA_adj_ct_sorted("N", [1], ['N'])
        print_AA_adj_ct_sorted("ORF3a", [2, 4], ['R', '3'])
        print_AA_adj_ct_sorted("ORF6", [2, 4], ['R', '6'])
        print_AA_adj_ct_sorted("ORF7a", [4, 5], ['7', 'a'])
        print_AA_adj_ct_sorted("ORF7b", [4, 5], ['7', 'b'])
        print_AA_adj_ct_sorted("ORF8", [2, 4], ['R', '8'])
        print_AA_adj_ct_sorted("ORF9b", [4, 5], ['9', 'b'])
    end
#####################################################################################################################################
    gene_set = Set(["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"])
    domain_set = Set(["NSP3_Ubl1", "NSP3_HVR", "NSP3_Mac1", "NSP3_Mac2", "NSP3_Mac3", "NSP3_DPUP", "NSP3_Ubl2", "NSP3_PLpro", "NSP3_NAB", "NSP3_BSM", "NSP3_TM1", "NSP3_Ecto3", "NSP3_TM234HLX", "NSP3_Y1", "NSP3_CoVY", "NSP4_TM1", "NSP4_Ecto4", "NSP4_TM2_TM6", "NSP4_CTD", "NSP6_AmphHlx", "NSP6_MAE", "NSP6_cyto_CTD", "NSP12_NiRAN", "NSP12_intrfce", "NSP12_fingers", "NSP12_palm", "NSP12_palmLnk", "NSP12_thumb", "NSP13_ZBD", "NSP13_stalk", "NSP13_1B", "NSP13_RecA1", "NSP13_RecA2", "NSP14_nsp10", "NSP14_EXON", "NSP14_hinge1", "NSP14_hinge2", "NSP14_N7MTase", "NSP15_NTD", "NSP15_MD", "NSP15_endoU", "S_S1", "S_S2", "S_NTD", "S_N2R", "S_RBD", "S_RBM", "S_SD1", "S_SD2", "S_630_loop", "S_FCS_region", "S_Beta1", "S_3H", "S_IL770", "S_FPPR", "S_FP", "S_HR1", "S_CH", "S_CD", "S_Beta2", "S_2turnHelix", "S_HR2", "S_TM", "S_CT", "E_TM", "E_cytosol", "E_CTD", "N_N1", "N_N2", "N_N3", "N_N4", "N_N5", "N_SR", "N_L_helix", "N_CBP", "N_9b_overlap"])
    for geen in gene_set
        gene_mut_density[geen] = round(digits=2, gene_protein_mut_ct[geen]/gene_protein_length[geen])
    end
    for dom in domain_set
        domain_mut_density[dom] = round(digits=2, domain_mut_ct[dom]/domain_length[dom])
    end
    gene_sort(n) = gene_protein_order[n]
    gene_mut_density_sort_by_gene = sort(collect(gene_mut_density), by = x -> gene_sort(x[1]))
    gene_mut_density_sort_by_density = sort(collect(gene_mut_density), by = x -> x[2], rev=true)
    protein_dom_sort(n) = domain_order[n]
    domain_mut_density_sort_by_gene = sort(collect(domain_mut_density), by = x -> protein_dom_sort(x[1]))
    domain_mut_density_sort_by_density = sort(collect(domain_mut_density), by = x -> x[2], rev=true)
#############################################################################################################
    date = Dates.format(today(), "yyyy_mm_dd")
    function density_print(d)
        ds = string(d)
        ds_dec = split(ds, ".")[2]
        if length(ds_dec) < 2
            ds = "$(ds)0"
        end
        return ds
    end
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_GENE_MUT_DENSITY.txt", "w") do g
        print(g, "\n"^2)
        println(g, "*************** Protein Mut Density, Sorted by Gene ***************")
        for (prot, density) in gene_mut_density_sort_by_gene
            println(g, rpad("$(prot) = ", 8), lpad("$(density_print(density))", 5) )
        end
        println(g)
        println(g, "*************** Protein Mut Density, Sorted by Mut Density ***************")
        for (prot, density) in gene_mut_density_sort_by_density
            println(g, rpad("$(prot) = ", 8), lpad("$(density_print(density))", 5) )
        end
        print(g, "\n"^2)
        println(g, "*************** Protein Domain Mut Density, Sorted by Gene, Domain ***************")
        for (domain, density) in domain_mut_density_sort_by_gene
            println(g, rpad("$(domain) = ", 20), lpad("$(density_print(density))", 5) )
        end
        println(g)
        println(g, "*************** Protein Domain Mut Density, Sorted by Mut Density ***************")
        for (domain, density) in domain_mut_density_sort_by_density
            println(g, rpad("$(domain) = ", 20), lpad("$(density_print(density))", 5) )
        end
        print(g, "\n"^2)
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_MISCELLANEOUS.txt", "w") do g        
        println(g, "Number of Sequences in rep_seqs list: ", length(rep_seqs))
        println(g, "Number of keys in rep_seq_grps_seqs: ", length(keys(rep_seq_grps_seqs)))
        println(g)
        related_grps = []
        for (k, v) in rep_seq_grps_seqs
            z = []
            p = sort(collect(v))
            q = join(p, ", ")
            push!(z, k)
            push!(z, q)
            push!(related_grps, z)
        end
        println(g)
        print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g, "T241C")
        print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g, "A28877T")
        print_seqs_with_AA_mut_plus_their_other_AA_muts(g, "ORF7a:R39I")
        print_seqs_with_AA_mut_plus_their_other_AA_muts(g, "S:A372T")
        AA_mutations_in_seq(g, "EPI_ISL_18337738")
        nuc_muts_in_seq(g, "EPI_ISL_18337738")
    end
#####################################################################################################################################
    AA_muts_ct_pos_only_adj_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_adj_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_adj), by = x -> x[2], rev = true)

    AA_muts_ct_pos_only_adj_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_no_dels), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_adj_no_dels), by = x -> x[2], rev = true)
    
    AA_muts_ct_pos_only_adj_score_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_score), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_adj_score_sort_by_score = sort(collect(AA_muts_ct_pos_only_adj_score), by = x -> x[2], rev = true)
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_score_no_dels), by = x -> AA_gene_pos_sortKey_2(x[1]))
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score = sort(collect(AA_muts_ct_pos_only_adj_score_no_dels), by = x -> x[2], rev = true)
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_POS_ONLY_ADJ_COUNT.txt", "w") do g       
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS *POS ONLY* Sorted by Count ***************")
        for w in AA_muts_ct_pos_only_adj_sort_by_seq_ct
            AAmut = string(w[1])
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                mutpad = rpad(string(AAmut), 12)
                countpad = lpad(AAct, 3)
                AAmutct_pad = lpad("[$(AAmutct)]", 5)
                println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
            end
        end
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS ADJ *POS ONLY* Sorted by Genome Position ***************")
        for w in AA_muts_ct_pos_only_adj_sort_by_site
            AAmut = string(w[1])
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                mutpad = rpad(AAmut, 12)
                countpad = lpad(AAct, 3)
                AAmutct_pad = lpad("[$(AAmutct)]", 5)
                println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
            end
        end
        print_pos_only_adj(g, "ORF1a", [4, 5], ['1', 'a'])
        print_pos_only_adj(g, "ORF1b", [4, 5], ['1', 'b'])
        print_pos_only_adj(g, "Spike", [1], ['S'])
        print_pos_only_adj(g, "E", [1], ['E'])
        print_pos_only_adj(g, "M", [1], ['M'])
        print_pos_only_adj(g, "N", [1], ['N'])
        print_pos_only_adj(g, "ORF3a", [2, 4], ['R', '3'])
        print_pos_only_adj(g, "ORF6", [2, 4], ['R', '6'])
        print_pos_only_adj(g, "ORF7a", [4, 5], ['7', 'a'])
        print_pos_only_adj(g, "ORF7b", [4, 5], ['7', 'b'])
        print_pos_only_adj(g, "ORF8", [2, 4], ['R', '8'])
        print_pos_only_adj(g, "ORF9b", [4, 5], ['9', 'b'])
    end
#####################################################################################################################################
    for (date, nuc_mut_ct_dict) in date_nuc_mut_ct
        for (mut, ct) in nuc_mut_ct_dict
            if !haskey(date_nuc_mut_ct_no_dels, date)
                date_nuc_mut_ct_no_dels[date] = Dict{String, Int}()
            end
            if mut[end] ≠ '-'
                date_nuc_mut_ct_no_dels[date][mut] = ct
            end
        end
    end
    for (date, AA_mut_ct_dict) in date_AA_mut_ct
        for (mut, ct) in AA_mut_ct_dict
            AA_pos = AA_mut_to_AA_pos(mut)
            if mut[end] ≠ '-'
                if !haskey(date_AA_mut_ct_no_dels, date)
                    date_AA_mut_ct_no_dels[date] = Dict{String, Int}()
                end
                date_AA_mut_ct_no_dels[date][mut] = ct
                if !haskey(date_AA_mut_ct_pos_only_no_dels, date)
                    date_AA_mut_ct_pos_only_no_dels[date] = Dict{String, Int}()
                end
####################################################
                date_AA_mut_ct_pos_only_no_dels[date][AA_pos] = get(date_AA_mut_ct_pos_only_no_dels[date], AA_pos, 0) + ct
#                if haskey(date_AA_mut_ct_pos_only_no_dels[date], AA_pos)
#                    date_AA_mut_ct_pos_only_no_dels[date][AA_pos] += ct
#                else
#                    date_AA_mut_ct_pos_only_no_dels[date][AA_pos] = ct
#                end
####################################################
            end
        end
    end
#####################################################################################################################################
    open("$(save_folder)/$(date)_$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep$(rep_thresh)_rev$(revs_thresh)_POS_ONLY_ADJ_RATIO.txt", "w") do g       
        print(g, "\n"^2)
        println(g, "******************************************************************************************************************")  
        println(g, "* AA Mutations Ratio (Proportion of Chronic/Proportion of Circulating Muts) *POS ONLY* Sorted by Genome Position *")
        println(g, "******************************************************************************************************************")
        for w in AA_muts_ct_pos_only_adj_score_sort_by_site
            AAmut = string(w[1])
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                mutpad = rpad(AAmut, 12)
                countpad = lpad(AAct, 3)
                AAmutct_pad = lpad("[$(AAmutct)]", 5)
                println(g, "$(mutpad) = $(countpad) $(AAmutct_pad)")
            end
        end
        print_pos_only_adj_score(g, "ORF1a", [4, 5], ['1', 'a'])
        print_pos_only_adj_score(g, "ORF1b", [4, 5], ['1', 'b'])
        print_pos_only_adj_score(g, "Spike", [1], ['S'])
        print_pos_only_adj_score(g, "E", [1], ['E'])
        print_pos_only_adj_score(g, "M", [1], ['M'])
        print_pos_only_adj_score(g, "N", [1], ['N'])
        print_pos_only_adj_score(g, "ORF3a", [2, 4], ['R', '3'])
        print_pos_only_adj_score(g, "ORF6", [2, 4], ['R', '6'])
        print_pos_only_adj_score(g, "ORF7a", [4, 5], ['7', 'a'])
        print_pos_only_adj_score(g, "ORF7b", [4, 5], ['7', 'b'])
        print_pos_only_adj_score(g, "ORF8", [2, 4], ['R', '8'])
        print_pos_only_adj_score(g, "ORF9b", [4, 5], ['9', 'b'])
    end
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__total_chr_AA_subs.jld2", "total_chr_AA_subs", total_chr_AA_subs)
######################################################################################################################################   
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__total_nuc_revs_seq.jld2", "total_nuc_revs_seq", total_nuc_revs_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_total_revs.jld2", "seq_nuc_total_revs", seq_nuc_total_revs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__total_AA_revs_seq.jld2", "total_AA_revs_seq", total_AA_revs_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_total_revs.jld2", "seq_AA_total_revs", seq_AA_total_revs)
######################################################################################################################################   
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_no_revs.jld2", "AA_muts_ct_no_dels_no_revs", AA_muts_ct_no_dels_no_revs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_no_dels_no_revs_chr_all_ratio", AA_muts_ct_no_dels_no_revs_chr_all_ratio)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio", AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_chr_all_ratio.jld2", "AA_muts_ct_chr_all_ratio", AA_muts_ct_chr_all_ratio)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_chr_all_ratio.jld2", "AA_muts_ct_no_dels_chr_all_ratio", AA_muts_ct_no_dels_chr_all_ratio)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio", AA_muts_ct_pos_only_no_dels_chr_all_ratio)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels_no_revs.jld2", "nuc_muts_ct_no_dels_no_revs", nuc_muts_ct_no_dels_no_revs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "nuc_muts_ct_no_dels_no_revs_chr_all_ratio", nuc_muts_ct_no_dels_no_revs_chr_all_ratio)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio", nuc_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels_chr_all_ratio.jld2", "nuc_muts_ct_no_dels_chr_all_ratio", nuc_muts_ct_no_dels_chr_all_ratio)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio", nuc_muts_ct_pos_only_no_dels_chr_all_ratio)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__gene_mut_density.jld2", "gene_mut_density", gene_mut_density)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__domain_mut_density.jld2", "domain_mut_density", domain_mut_density)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__date_nuc_mut_ct.jld2", "date_nuc_mut_ct", date_nuc_mut_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels", date_nuc_mut_ct_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__date_AA_mut_ct.jld2", "date_AA_mut_ct", date_AA_mut_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels", date_AA_mut_ct_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels", date_AA_mut_ct_pos_only_no_dels)

    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day", seq_ct_by_year_month_day)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_ct_by_year_month.jld2", "seq_ct_by_year_month", seq_ct_by_year_month)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_ct_by_year.jld2", "seq_ct_by_year", seq_ct_by_year)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_seq.jld2", "nuc_muts_seq", nuc_muts_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_dels_seq.jld2", "nuc_dels_seq", nuc_dels_seq)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_seq.jld2", "AA_muts_seq", AA_muts_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_seq_pos_only.jld2", "AA_muts_seq_pos_only", AA_muts_seq_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_dels_seq.jld2", "AA_dels_seq", AA_dels_seq)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_muts.jld2", "seq_nuc_muts", seq_nuc_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_del_ranges.jld2", "seq_nuc_del_ranges", seq_nuc_del_ranges)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts.jld2", "seq_AA_muts", seq_AA_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_revs.jld2", "seq_AA_revs", seq_AA_revs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts_no_dels.jld2", "seq_AA_muts_no_dels", seq_AA_muts_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts_pos_only.jld2", "seq_AA_muts_pos_only", seq_AA_muts_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_dels.jld2", "seq_AA_dels", seq_AA_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_mixed_AA_muts.jld2", "seq_mixed_AA_muts", seq_mixed_AA_muts)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_muts_WT.jld2", "seq_nuc_muts_WT", seq_nuc_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_del_ranges_WT.jld2", "seq_nuc_del_ranges_WT", seq_nuc_del_ranges_WT)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts_WT.jld2", "seq_AA_muts_WT", seq_AA_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts_WT_pos_only.jld2", "seq_AA_muts_WT_pos_only", seq_AA_muts_WT_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_dels_WT.jld2", "seq_AA_dels_WT", seq_AA_dels_WT)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_seq_WT.jld2", "nuc_muts_seq_WT", nuc_muts_seq_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_dels_seq_WT.jld2", "nuc_dels_seq_WT", nuc_dels_seq_WT)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_seq_WT.jld2", "AA_muts_seq_WT", AA_muts_seq_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_seq_WT_pos_only.jld2", "AA_muts_seq_WT_pos_only", AA_muts_seq_WT_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_dels_seq_WT.jld2", "AA_dels_seq_WT", AA_dels_seq_WT)

    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_insertions_WT.jld2", "seq_AA_insertions_WT", seq_AA_insertions_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_insertions_WT.jld2", "seq_nuc_insertions_WT", seq_nuc_insertions_WT)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_country.jld2", "seq_country", seq_country)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_US_state.jld2", "seq_US_state", seq_US_state)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_collection_date.jld2", "seq_collection_date", seq_collection_date)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_date_index.jld2", "seq_date_index", seq_date_index)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_date_tuple.jld2", "seq_date_tuple", seq_date_tuple)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_lab_dict.jld2", "seq_lab_dict", seq_lab_dict)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_unknown_AA.jld2", "seq_unknown_AA", seq_unknown_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_unknown_AA_ranges.jld2", "seq_unknown_AA_ranges", seq_unknown_AA_ranges)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_nuc_dropout.jld2", "seq_nuc_dropout", seq_nuc_dropout)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_mixed_nucs.jld2", "seq_mixed_nucs", seq_mixed_nucs)
###################################################################################################################################### 
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts.jld2", "NSP_muts", NSP_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts_no_dels.jld2", "NSP_muts_no_dels", NSP_muts_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_clade.jld2", "seq_clade", seq_clade)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_clade_display.jld2", "seq_clade_display", seq_clade_display)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_pango.jld2", "seq_pango", seq_pango)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_pango_unaliased.jld2", "seq_pango_unaliased", seq_pango_unaliased)

    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_clade.jld2", "rep_seq_grps_clade", rep_seq_grps_clade)

    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_clade_ct.jld2", "seq_clade_ct", seq_clade_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_pango_ct.jld2", "seq_pango_ct", seq_pango_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_pango_unaliased_ct.jld2", "seq_pango_unaliased_ct", seq_pango_unaliased_ct)  
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_seqs.jld2", "rep_seq_grps_seqs", rep_seq_grps_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_muts.jld2", "rep_seq_grps_muts", rep_seq_grps_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA.jld2", "rep_seq_grps_AA", rep_seq_grps_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_pos_only.jld2", "rep_seq_grps_AA_pos_only", rep_seq_grps_AA_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_pos_only_no_dels.jld2", "rep_seq_grps_AA_pos_only_no_dels", rep_seq_grps_AA_pos_only_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__non_rep_seqs_AA.jld2", "non_rep_seqs_AA", non_rep_seqs_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__non_rep_seqs_AA_pos_only.jld2", "non_rep_seqs_AA_pos_only", non_rep_seqs_AA_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__non_rep_seqs_AA_pos_only_no_dels.jld2", "non_rep_seqs_AA_pos_only_no_dels", non_rep_seqs_AA_pos_only_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_dels_ct.jld2", "nuc_dels_ct", nuc_dels_ct)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct.jld2", "nuc_muts_ct", nuc_muts_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels", nuc_muts_ct_no_dels)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj.jld2", "nuc_muts_ct_adj", nuc_muts_ct_adj)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_no_dels.jld2", "nuc_muts_ct_adj_no_dels", nuc_muts_ct_adj_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score.jld2", "nuc_muts_ct_adj_score", nuc_muts_ct_adj_score)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score_no_dels.jld2", "nuc_muts_ct_adj_score_no_dels", nuc_muts_ct_adj_score_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_dels_ct.jld2", "AA_dels_ct", AA_dels_ct)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct.jld2", "AA_muts_ct", AA_muts_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels", AA_muts_ct_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only", AA_muts_ct_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels", AA_muts_ct_pos_only_no_dels)
    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj.jld2", "AA_muts_ct_adj", AA_muts_ct_adj)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_no_dels.jld2", "AA_muts_ct_adj_no_dels", AA_muts_ct_adj_no_dels)    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj.jld2", "AA_muts_ct_pos_only_adj", AA_muts_ct_pos_only_adj)    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_no_dels.jld2", "AA_muts_ct_pos_only_adj_no_dels", AA_muts_ct_pos_only_adj_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score.jld2", "AA_muts_ct_adj_score", AA_muts_ct_adj_score)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score_no_dels.jld2", "AA_muts_ct_adj_score_no_dels", AA_muts_ct_adj_score_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score.jld2", "AA_muts_ct_pos_only_adj_score", AA_muts_ct_pos_only_adj_score)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score_no_dels.jld2", "AA_muts_ct_pos_only_adj_score_no_dels", AA_muts_ct_pos_only_adj_score_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_seq_pos_only_no_dels.jld2", "AA_muts_seq_pos_only_no_dels", AA_muts_seq_pos_only_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__seq_AA_muts_pos_only_no_dels.jld2", "seq_AA_muts_pos_only_no_dels", seq_AA_muts_pos_only_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_rep_seq_grps.jld2", "nuc_muts_rep_seq_grps", nuc_muts_rep_seq_grps)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_dels_rep_seq_grps.jld2", "nuc_dels_rep_seq_grps", nuc_dels_rep_seq_grps)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_rep_seq_grps_no_dels.jld2", "nuc_muts_rep_seq_grps_no_dels", nuc_muts_rep_seq_grps_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_dels.jld2", "rep_seq_grps_dels", rep_seq_grps_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_muts_no_dels.jld2", "rep_seq_grps_muts_no_dels", rep_seq_grps_muts_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_del_ranges_ct.jld2", "rep_seq_grps_del_ranges_ct", rep_seq_grps_del_ranges_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_nuc_dropout.jld2", "rep_seq_grps_nuc_dropout", rep_seq_grps_nuc_dropout)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_mixed_nucs.jld2", "rep_seq_grps_mixed_nucs", rep_seq_grps_mixed_nucs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_dels.jld2", "rep_seq_grps_AA_dels", rep_seq_grps_AA_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_no_dels.jld2", "rep_seq_grps_AA_no_dels", rep_seq_grps_AA_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_rep_seq_grps.jld2", "AA_muts_rep_seq_grps", AA_muts_rep_seq_grps)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_dels_rep_seq_grps.jld2", "AA_dels_rep_seq_grps", AA_dels_rep_seq_grps)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_rep_seq_grps_no_dels.jld2", "AA_muts_rep_seq_grps_no_dels", AA_muts_rep_seq_grps_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_rep_seq_grps_pos_only.jld2", "AA_muts_rep_seq_grps_pos_only", AA_muts_rep_seq_grps_pos_only)
#    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_unknown_AA.jld2", "rep_seq_grps_unknown_AA", rep_seq_grps_unknown_AA)
#    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_mixed_AA_muts.jld2", "rep_seq_grps_mixed_AA_muts", rep_seq_grps_mixed_AA_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_nuc_muts_WT.jld2", "rep_seq_grps_nuc_muts_WT", rep_seq_grps_nuc_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_nuc_dels_WT.jld2", "rep_seq_grps_nuc_dels_WT", rep_seq_grps_nuc_dels_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_muts_WT.jld2", "rep_seq_grps_AA_muts_WT", rep_seq_grps_AA_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_dels_WT.jld2", "rep_seq_grps_AA_dels_WT", rep_seq_grps_AA_dels_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_rep_seq_grps_WT.jld2", "nuc_muts_rep_seq_grps_WT", nuc_muts_rep_seq_grps_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_dels_rep_seq_grps_WT.jld2", "nuc_dels_rep_seq_grps_WT", nuc_dels_rep_seq_grps_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_rep_seq_grps_WT.jld2", "AA_muts_rep_seq_grps_WT", AA_muts_rep_seq_grps_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_dels_rep_seq_grps_WT.jld2", "AA_dels_rep_seq_grps_WT", AA_dels_rep_seq_grps_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_AA_muts_WT_pos_only.jld2", "rep_seq_grps_AA_muts_WT_pos_only", rep_seq_grps_AA_muts_WT_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_rep_seq_grps_WT_pos_only.jld2", "AA_muts_rep_seq_grps_WT_pos_only", AA_muts_rep_seq_grps_WT_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_pango.jld2", "rep_seq_grps_pango", rep_seq_grps_pango)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_pango_unaliased.jld2", "rep_seq_grps_pango_unaliased", rep_seq_grps_pango_unaliased)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_seqs.jld2", "rep_seq_grps_maxmut_seqs", rep_seq_grps_maxmut_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_clade.jld2", "rep_seq_grps_maxmut_clade", rep_seq_grps_maxmut_clade)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_pango.jld2", "rep_seq_grps_maxmut_pango", rep_seq_grps_maxmut_pango)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_pango_unaliased.jld2", "rep_seq_grps_maxmut_pango_unaliased", rep_seq_grps_maxmut_pango_unaliased)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc.jld2", "rep_seq_grps_maxmut_nuc", rep_seq_grps_maxmut_nuc)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_no_dels.jld2", "rep_seq_grps_maxmut_nuc_no_dels", rep_seq_grps_maxmut_nuc_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_dels.jld2", "rep_seq_grps_maxmut_dels", rep_seq_grps_maxmut_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_dropout.jld2", "rep_seq_grps_maxmut_nuc_dropout", rep_seq_grps_maxmut_nuc_dropout)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_mixed_nucs.jld2", "rep_seq_grps_maxmut_mixed_nucs", rep_seq_grps_maxmut_mixed_nucs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA.jld2", "rep_seq_grps_maxmut_AA", rep_seq_grps_maxmut_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_no_dels.jld2", "rep_seq_grps_maxmut_AA_no_dels", rep_seq_grps_maxmut_AA_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_dels.jld2", "rep_seq_grps_maxmut_AA_dels", rep_seq_grps_maxmut_AA_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_pos_only.jld2", "rep_seq_grps_maxmut_AA_pos_only", rep_seq_grps_maxmut_AA_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_pos_only_no_dels.jld2", "rep_seq_grps_maxmut_AA_pos_only_no_dels", rep_seq_grps_maxmut_AA_pos_only_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_unknown_AA_ranges.jld2", "rep_seq_grps_maxmut_unknown_AA_ranges", rep_seq_grps_maxmut_unknown_AA_ranges)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_mixed_AA_muts.jld2", "rep_seq_grps_maxmut_mixed_AA_muts", rep_seq_grps_maxmut_mixed_AA_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_seqs.jld2", "rep_seq_grps_maxmut_seqs", rep_seq_grps_maxmut_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc.jld2", "rep_seq_grps_maxmut_nuc", rep_seq_grps_maxmut_nuc)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_no_dels.jld2", "rep_seq_grps_maxmut_nuc_no_dels", rep_seq_grps_maxmut_nuc_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_dels.jld2", "rep_seq_grps_maxmut_dels", rep_seq_grps_maxmut_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_del_ranges_ct.jld2", "rep_seq_grps_maxmut_del_ranges_ct", rep_seq_grps_maxmut_del_ranges_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_dropout.jld2", "rep_seq_grps_maxmut_nuc_dropout", rep_seq_grps_maxmut_nuc_dropout)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_mixed_nucs.jld2", "rep_seq_grps_maxmut_mixed_nucs", rep_seq_grps_maxmut_mixed_nucs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA.jld2", "rep_seq_grps_maxmut_AA", rep_seq_grps_maxmut_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_no_dels.jld2", "rep_seq_grps_maxmut_AA_no_dels", rep_seq_grps_maxmut_AA_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_dels.jld2", "rep_seq_grps_maxmut_AA_dels", rep_seq_grps_maxmut_AA_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_pos_only.jld2", "rep_seq_grps_maxmut_AA_pos_only", rep_seq_grps_maxmut_AA_pos_only)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_pos_only_no_dels.jld2", "rep_seq_grps_maxmut_AA_pos_only_no_dels", rep_seq_grps_maxmut_AA_pos_only_no_dels)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_unknown_AA.jld2", "rep_seq_grps_maxmut_unknown_AA", rep_seq_grps_maxmut_unknown_AA)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_muts_WT.jld2", "rep_seq_grps_maxmut_nuc_muts_WT", rep_seq_grps_maxmut_nuc_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_nuc_dels_WT.jld2", "rep_seq_grps_maxmut_nuc_dels_WT", rep_seq_grps_maxmut_nuc_dels_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_muts_WT.jld2", "rep_seq_grps_maxmut_AA_muts_WT", rep_seq_grps_maxmut_AA_muts_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_dels_WT.jld2", "rep_seq_grps_maxmut_AA_dels_WT", rep_seq_grps_maxmut_AA_dels_WT)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seq_grps_maxmut_AA_muts_WT_pos_only.jld2", "rep_seq_grps_maxmut_AA_muts_WT_pos_only", rep_seq_grps_maxmut_AA_muts_WT_pos_only)
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_seqs.jld2", "all_seqs", all_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_seqs_set.jld2", "all_seqs_set", all_seqs_set)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_qualifying_seqs.jld2", "all_qualifying_seqs", all_qualifying_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_qualifying_seqs_set.jld2", "all_qualifying_seqs_set", all_qualifying_seqs_set)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_nonqualifying_seqs.jld2", "all_nonqualifying_seqs", all_nonqualifying_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__all_nonqualifying_seqs_set.jld2", "all_nonqualifying_seqs_set", all_nonqualifying_seqs_set)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__rep_seqs.jld2", "rep_seqs", rep_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__non_rep_seqs.jld2", "non_rep_seqs", non_rep_seqs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__too_many_reversions.jld2", "too_many_reversions", too_many_reversions)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts_sortByCt_Arr.jld2", "NSP_muts_sortByCt_Arr", NSP_muts_sortByCt_Arr)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts_sortByPos_Arr.jld2", "NSP_muts_sortByPos_Arr", NSP_muts_sortByPos_Arr)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts_no_dels_sortByCt_Arr.jld2", "NSP_muts_no_dels_sortByCt_Arr", NSP_muts_no_dels_sortByCt_Arr)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP_muts_no_dels_sortByPos_Arr.jld2", "NSP_muts_no_dels_sortByPos_Arr", NSP_muts_no_dels_sortByPos_Arr)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP1_muts_sortByCt.jld2", "NSP1_muts_sortByCt", NSP1_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP2_muts_sortByCt.jld2", "NSP2_muts_sortByCt", NSP2_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP3_muts_sortByCt.jld2", "NSP3_muts_sortByCt", NSP3_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP4_muts_sortByCt.jld2", "NSP4_muts_sortByCt", NSP4_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP5_muts_sortByCt.jld2", "NSP5_muts_sortByCt", NSP5_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP6_muts_sortByCt.jld2", "NSP6_muts_sortByCt", NSP6_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP7_muts_sortByCt.jld2", "NSP7_muts_sortByCt", NSP7_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP8_muts_sortByCt.jld2", "NSP8_muts_sortByCt", NSP8_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP9_muts_sortByCt.jld2", "NSP9_muts_sortByCt", NSP9_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP10_muts_sortByCt.jld2", "NSP10_muts_sortByCt", NSP10_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP11_muts_sortByCt.jld2", "NSP11_muts_sortByCt", NSP11_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP12_muts_sortByCt.jld2", "NSP12_muts_sortByCt", NSP12_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP13_muts_sortByCt.jld2", "NSP13_muts_sortByCt", NSP13_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP14_muts_sortByCt.jld2", "NSP14_muts_sortByCt", NSP14_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP15_muts_sortByCt.jld2", "NSP15_muts_sortByCt", NSP15_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP16_muts_sortByCt.jld2", "NSP16_muts_sortByCt", NSP16_muts_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP1_muts_sortByPos.jld2", "NSP1_muts_sortByPos", NSP1_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP2_muts_sortByPos.jld2", "NSP2_muts_sortByPos", NSP2_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP3_muts_sortByPos.jld2", "NSP3_muts_sortByPos", NSP3_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP4_muts_sortByPos.jld2", "NSP4_muts_sortByPos", NSP4_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP5_muts_sortByPos.jld2", "NSP5_muts_sortByPos", NSP5_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP6_muts_sortByPos.jld2", "NSP6_muts_sortByPos", NSP6_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP7_muts_sortByPos.jld2", "NSP7_muts_sortByPos", NSP7_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP8_muts_sortByPos.jld2", "NSP8_muts_sortByPos", NSP8_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP9_muts_sortByPos.jld2", "NSP9_muts_sortByPos", NSP9_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP10_muts_sortByPos.jld2", "NSP10_muts_sortByPos", NSP10_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP11_muts_sortByPos.jld2", "NSP11_muts_sortByPos", NSP11_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP12_muts_sortByPos.jld2", "NSP12_muts_sortByPos", NSP12_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP13_muts_sortByPos.jld2", "NSP13_muts_sortByPos", NSP13_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP14_muts_sortByPos.jld2", "NSP14_muts_sortByPos", NSP14_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP15_muts_sortByPos.jld2", "NSP15_muts_sortByPos", NSP15_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP16_muts_sortByPos.jld2", "NSP16_muts_sortByPos", NSP16_muts_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP1_muts_no_dels_sortByCt.jld2", "NSP1_muts_no_dels_sortByCt", NSP1_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP2_muts_no_dels_sortByCt.jld2", "NSP2_muts_no_dels_sortByCt", NSP2_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP3_muts_no_dels_sortByCt.jld2", "NSP3_muts_no_dels_sortByCt", NSP3_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP4_muts_no_dels_sortByCt.jld2", "NSP4_muts_no_dels_sortByCt", NSP4_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP5_muts_no_dels_sortByCt.jld2", "NSP5_muts_no_dels_sortByCt", NSP5_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP6_muts_no_dels_sortByCt.jld2", "NSP6_muts_no_dels_sortByCt", NSP6_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP7_muts_no_dels_sortByCt.jld2", "NSP7_muts_no_dels_sortByCt", NSP7_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP8_muts_no_dels_sortByCt.jld2", "NSP8_muts_no_dels_sortByCt", NSP8_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP9_muts_no_dels_sortByCt.jld2", "NSP9_muts_no_dels_sortByCt", NSP9_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP10_muts_no_dels_sortByCt.jld2", "NSP10_muts_no_dels_sortByCt", NSP10_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP11_muts_no_dels_sortByCt.jld2", "NSP11_muts_no_dels_sortByCt", NSP11_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP12_muts_no_dels_sortByCt.jld2", "NSP12_muts_no_dels_sortByCt", NSP12_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP13_muts_no_dels_sortByCt.jld2", "NSP13_muts_no_dels_sortByCt", NSP13_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP14_muts_no_dels_sortByCt.jld2", "NSP14_muts_no_dels_sortByCt", NSP14_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP15_muts_no_dels_sortByCt.jld2", "NSP15_muts_no_dels_sortByCt", NSP15_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP16_muts_no_dels_sortByCt.jld2", "NSP16_muts_no_dels_sortByCt", NSP16_muts_no_dels_sortByCt)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP1_muts_no_dels_sortByPos.jld2", "NSP1_muts_no_dels_sortByPos", NSP1_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP2_muts_no_dels_sortByPos.jld2", "NSP2_muts_no_dels_sortByPos", NSP2_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP3_muts_no_dels_sortByPos.jld2", "NSP3_muts_no_dels_sortByPos", NSP3_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP4_muts_no_dels_sortByPos.jld2", "NSP4_muts_no_dels_sortByPos", NSP4_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP5_muts_no_dels_sortByPos.jld2", "NSP5_muts_no_dels_sortByPos", NSP5_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP6_muts_no_dels_sortByPos.jld2", "NSP6_muts_no_dels_sortByPos", NSP6_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP7_muts_no_dels_sortByPos.jld2", "NSP7_muts_no_dels_sortByPos", NSP7_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP8_muts_no_dels_sortByPos.jld2", "NSP8_muts_no_dels_sortByPos", NSP8_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP9_muts_no_dels_sortByPos.jld2", "NSP9_muts_no_dels_sortByPos", NSP9_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP10_muts_no_dels_sortByPos.jld2", "NSP10_muts_no_dels_sortByPos", NSP10_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP11_muts_no_dels_sortByPos.jld2", "NSP11_muts_no_dels_sortByPos", NSP11_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP12_muts_no_dels_sortByPos.jld2", "NSP12_muts_no_dels_sortByPos", NSP12_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP13_muts_no_dels_sortByPos.jld2", "NSP13_muts_no_dels_sortByPos", NSP13_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP14_muts_no_dels_sortByPos.jld2", "NSP14_muts_no_dels_sortByPos", NSP14_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP15_muts_no_dels_sortByPos.jld2", "NSP15_muts_no_dels_sortByPos", NSP15_muts_no_dels_sortByPos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__NSP16_muts_no_dels_sortByPos.jld2", "NSP16_muts_no_dels_sortByPos", NSP16_muts_no_dels_sortByPos)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_chr_all_ratio_ct_sort", AA_muts_ct_chr_all_ratio_ct_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_chr_all_ratio_pos_sort", AA_muts_ct_chr_all_ratio_pos_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_no_dels_chr_all_ratio_ct_sort", AA_muts_ct_no_dels_chr_all_ratio_ct_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_no_dels_chr_all_ratio_pos_sort", AA_muts_ct_no_dels_chr_all_ratio_pos_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort", AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort", AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__avg_AA_subs_per_chr_seq.jld2", "avg_AA_subs_per_chr_seq", avg_AA_subs_per_chr_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__avg_AA_subs_per_circ_seq.jld2", "avg_AA_subs_per_circ_seq", avg_AA_subs_per_circ_seq)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels_chr_all_ratio_ct_sort.jld2", "nuc_muts_ct_no_dels_chr_all_ratio_ct_sort", nuc_muts_ct_no_dels_chr_all_ratio_ct_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_no_dels_chr_all_ratio_pos_sort.jld2", "nuc_muts_ct_no_dels_chr_all_ratio_pos_sort", nuc_muts_ct_no_dels_chr_all_ratio_pos_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort", nuc_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort.jld2", "nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort", nuc_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__avg_nuc_subs_per_chr_seq.jld2", "avg_nuc_subs_per_chr_seq", avg_nuc_subs_per_chr_seq)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__avg_nuc_subs_per_circ_seq.jld2", "avg_nuc_subs_per_circ_seq", avg_nuc_subs_per_circ_seq)
#####################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__gene_mut_density_sort_by_gene.jld2", "gene_mut_density_sort_by_gene", gene_mut_density_sort_by_gene)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__gene_mut_density_sort_by_density.jld2", "gene_mut_density_sort_by_density", gene_mut_density_sort_by_density)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__domain_mut_density_sort_by_gene.jld2", "domain_mut_density_sort_by_gene", domain_mut_density_sort_by_gene)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__domain_mut_density_sort_by_density.jld2", "domain_mut_density_sort_by_density", domain_mut_density_sort_by_density)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__excluded_pos.jld2", "excluded_pos", excluded_pos)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__excluded_AA.jld2", "excluded_AA", excluded_AA)
#####################################################################################################################################    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_sort_by_site.jld2", "nuc_muts_ct_sort_by_site", nuc_muts_ct_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_sort_by_seq_ct.jld2", "nuc_muts_ct_sort_by_seq_ct", nuc_muts_ct_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_sort_by_site.jld2", "nuc_muts_ct_adj_sort_by_site", nuc_muts_ct_adj_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_sort_by_seq_ct.jld2", "nuc_muts_ct_adj_sort_by_seq_ct", nuc_muts_ct_adj_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_no_dels_sort_by_site.jld2", "nuc_muts_ct_adj_no_dels_sort_by_site", nuc_muts_ct_adj_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", "nuc_muts_ct_adj_no_dels_sort_by_seq_ct", nuc_muts_ct_adj_no_dels_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score_sort_by_site.jld2", "nuc_muts_ct_adj_score_sort_by_site", nuc_muts_ct_adj_score_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score_sort_by_score.jld2", "nuc_muts_ct_adj_score_sort_by_score", nuc_muts_ct_adj_score_sort_by_score)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score_no_dels_sort_by_site.jld2", "nuc_muts_ct_adj_score_no_dels_sort_by_site", nuc_muts_ct_adj_score_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__nuc_muts_ct_adj_score_no_dels_sort_by_score.jld2", "nuc_muts_ct_adj_score_no_dels_sort_by_score", nuc_muts_ct_adj_score_no_dels_sort_by_score)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_sort_by_site.jld2", "AA_muts_ct_sort_by_site", AA_muts_ct_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_sort_by_seq_ct.jld2", "AA_muts_ct_sort_by_seq_ct", AA_muts_ct_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_sort_by_site.jld2", "AA_muts_ct_no_dels_sort_by_site", AA_muts_ct_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_sort_by_seq_ct", AA_muts_ct_no_dels_sort_by_seq_ct)
#####################################################################################################################################    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_sort_by_site.jld2", "AA_muts_ct_adj_sort_by_site", AA_muts_ct_adj_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_sort_by_seq_ct.jld2", "AA_muts_ct_adj_sort_by_seq_ct", AA_muts_ct_adj_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_no_dels_sort_by_site.jld2", "AA_muts_ct_adj_no_dels_sort_by_site", AA_muts_ct_adj_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_adj_no_dels_sort_by_seq_ct", AA_muts_ct_adj_no_dels_sort_by_seq_ct)
#####################################################################################################################################        
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score_sort_by_score.jld2", "AA_muts_ct_adj_score_sort_by_score", AA_muts_ct_adj_score_sort_by_score) 
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score_sort_by_site.jld2", "AA_muts_ct_adj_score_sort_by_site", AA_muts_ct_adj_score_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score_no_dels_sort_by_score.jld2", "AA_muts_ct_adj_score_no_dels_sort_by_score", AA_muts_ct_adj_score_no_dels_sort_by_score) 
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_adj_score_no_dels_sort_by_site.jld2", "AA_muts_ct_adj_score_no_dels_sort_by_site", AA_muts_ct_adj_score_no_dels_sort_by_site)
#####################################################################################################################################    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_sort_by_site.jld2", "AA_muts_ct_pos_only_sort_by_site", AA_muts_ct_pos_only_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_sort_by_seq_ct", AA_muts_ct_pos_only_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_site", AA_muts_ct_pos_only_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_seq_ct", AA_muts_ct_pos_only_no_dels_sort_by_seq_ct)
#####################################################################################################################################    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_sort_by_site", AA_muts_ct_pos_only_adj_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_adj_sort_by_seq_ct", AA_muts_ct_pos_only_adj_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_no_dels_sort_by_site", AA_muts_ct_pos_only_adj_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct", AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct)
#####################################################################################################################################    
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_score_sort_by_site", AA_muts_ct_pos_only_adj_score_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score_sort_by_score.jld2", "AA_muts_ct_pos_only_adj_score_sort_by_score", AA_muts_ct_pos_only_adj_score_sort_by_score)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site.jld2", "AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site", AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score.jld2", "AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score", AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score)
######################################################################################################################################  
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__chronic_search_muts.jld2", "chronic_search_muts", chronic_search_muts)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__chronic_search_muts_min2.jld2", "chronic_search_muts_min2", chronic_search_muts_min2)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__chronic_search_muts_min2_no_dels.jld2", "chronic_search_muts_min2_no_dels", chronic_search_muts_min2_no_dels)
######################################################################################################################################
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_no_revs_sort_by_site.jld2", "AA_muts_ct_no_dels_no_revs_sort_by_site", AA_muts_ct_no_dels_no_revs_sort_by_site)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__AA_muts_ct_no_dels_no_revs_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_no_revs_sort_by_seq_ct", AA_muts_ct_no_dels_no_revs_sort_by_seq_ct)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__avg_AA_subs_per_chr_seq_no_revs.jld2", "avg_AA_subs_per_chr_seq_no_revs", avg_AA_subs_per_chr_seq_no_revs)
    save("$(save_folder)/$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)__chronic_search_muts_v2.jld2", "chronic_search_muts_v2", chronic_search_muts_v2)
end
#####################################################################################################################################    

2026_02_21_151PM


chr_2025_11_15_date_DQ (generic function with 1 method)

In [72]:
### Execute Main Chronic Fx, Date Pct DQ Version | chronics_2026_02_01__6153seq | - Runtime: ~10 min #############
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
nowtime = Dates.format(now(), "I:MM.SS_p"); println(nowtime)
date = Dates.format(today(), "yyyy_mm_dd")
chr_date_fx_start = time()
rep_thresh = 5
revs_thresh = 6

abs_min_mut_thresh = 8
DQ_mut_thresh = 10
DQ_pct = 95

print_ct_thresh = 1
ndjson = "chronics_2026_02_01__6153seq_v2.ndjson"
ndjson_name = "chronics_2026_02_01__6153seq_v2"
folder_name = "$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_string)"
chr_2025_11_15_date_DQ(ndjson, ndjson_name, DQ_pct, rep_thresh, revs_thresh, print_ct_thresh, DQ_mut_thresh, abs_min_mut_thresh, HQCS_qc_str)
chr_date_fx_runtime = round(digits=1, time() - chr_date_fx_start)
println(chr_date_fx_runtime, " seconds")
chr_date_fx_hms1, chr_date_fx_hms2 = seconds_to_hrs_min_sec(chr_date_fx_runtime)
println("Time to Run Chronic Date Fx = $(chr_date_fx_hms1)")
println("Time to Run Chronic Date Fx = $(chr_date_fx_hms2)")
print("\n"^2)
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now)
nowtime = Dates.format(now(), "I:MM.SS_p"); println(nowtime)
print("\n"^2)
# (ndjson::String, ndjson_name::String, date_pct_DQ_thresh::Int, adj_score_ct_div::Int, rep_thresh::Int, revs_thresh::Int, print_ct_thresh::Int, DQ_mut_thresh::Int, abs_min_mut_thresh::Int)
# 
# Runtime = 
# Runtime = 
# Runtime = 0 hr, 05 min, 11.20 sec  2026_02_20 
# Runtime = 0 hr, 08 min, 08.30 sec  2026_01_18
# Runtime = 0 hr, 12 min, 59.30 sec  2025_12_29
# Time to Run Chronic Date Fx = 0 hr, 06 min, 30.90 sec  2025_11_17
# Time to Run Chronic Date Fx = 0 hr, 04 min, 49.20 sec  2025_09_06
# Time to Run Chronic Date Fx = 0 hr, 04 min, 31.97 sec  2025_08_19
# Time to Run Chronic Date Fx = 0 hr, 04 min, 47.71 sec
################################################################################################################################################
################################################################################################################################################

2026_02_21_151PM
1:51.41_PM
Number of Sequences in rep_seqs list: 2564



********************** Total Counted Chronic Seq Ct = 2469  **********************
********************** Total Counted Circulating Seq Ct = 14631051  **********************

********************** Total Chronic Nuc Substitution Ct = 69376  **********************
********************** Total Circulating Nuc Substitution Ct = 67293282  **********************

********************** Avg Chronic Nuc Substitution Ct = 28.098825435398947  **********************
********************** Avg Circulating Nuc Substitution Ct = 4.5993471009020475  **********************


10 random nuc_muts_ct_no_dels_chr_all_ratio Keys and Values
A24922T      =    0.76
A13237G      =     0.8
G10073A      =    0.88
A12307G      =    1.13
G27556T      =    0.14
C8650T       =    0.88
A22776G      =   61.39
C12242A      =  107.78
A299G        =   47.82








********************** Total Counted Chronic Seq Ct = 2469  **********************
*

In [ ]:
############################## Protein Domain ranges & sources ######################################################
    #    NSP3_Ubl1      ### NSP3:1-111
    #    NSP3_HVR       ### NSP3:112-206
    #    NSP3_Mac1      ### NSP3:207-374
    #    NSP3_Mac2      ### NSP3:375-551
    #    NSP3_Mac3      ### NSP3:552-673
    #    NSP3_DPUP      ### NSP3:674-743
    #    NSP3_Ubl2      ### NSP3:744-803
    #    NSP3_PLpro     ### NSP3:804-1052
    #    NSP3_NAB       ### NSP3:1053-1203
    #    NSP3_BSM       ### NSP3:1204-1334
    #    NSP3_TM1       ### NSP3:1335-1440
    #    NSP3_Ecto3     ### NSP3:1441-1499
    #    NSP3_TM234HLX  ### NSP3:1500-1586
    #    NSP3_Y1        ### NSP3:1587-1764
    #    NSP3_CoVY      ### NSP3:1765-1945
####################################################################################################################################
############ "NSP6_loop"=>75,     "NSP6_loop"=>NSP6_loop_ct,    "NSP6_loop"=>NSP6_loop_ct_no_dels,   "NSP6_loop"=>"NSP6:35-40|61-66|88-112|132-138|157-181|205-210",        "NSP6_loop"=>"BitSet(35:40; 61:66; 88:112; 132:138; 157:181; 205-210),  
############ "NSP6_TM"=>124,      "NSP6_TM"=>NSP6_TM_ct,        "NSP6_TM"=>NSP6_TM_ct_no_dels,       "NSP6_TM"=>"NSP6:12-34|41-60|67-87|113-131|139-156|182-204|211-230",       "NSP6_TM"=>BitSet(12:34; 41:60; 67:87; 113:131; 139:156; 182:204; 211:230),
                            ### AA: 10-27, 43-60, 63-80, 111-128, 130-150, 160-172, 176-190, 209-228 (alternate cource for TM: The multiple roles of nsp6 in the molecular pathogenesis of SARS-CoV-2, Antiviral Research; https://doi.org/10.1016/j.antiviral.2023.105590
#    NSP6_TM          ### AA: 12-34, 41-60, 67-87, 113-131, 139-156, 182-204, 211-230    Source: Feng S, O'Brien A, Chen DY, Saeed M, Baker SC. SARS-CoV-2 nonstructural protein 6 from Alpha to Omicron: evolution of a transmembrane protein. mBio. 2023;14(4):e0068823. doi:10.1128/mbio.00688-23
#    NSP6_loop        ### AA: 35-40, 61-66, 88-112, 132-138, 157-181, 205-210           Source: Feng S, O'Brien A, Chen DY, Saeed M, Baker SC. SARS-CoV-2 nonstructural protein 6 from Alpha to Omicron: evolution of a transmembrane protein. mBio. 2023;14(4):e0068823. doi:10.1128/mbio.00688-23
####################################################################################################################################
#    NSP6_AmphHlx  ### Amphipathic Helix: 219-236   Source: Ricciardi S, Guarino AM, Giaquinto L, et al. The role of NSP6 in the biogenesis of the SARS-CoV-2 replication organelle. Nature. 2022;606(7915):761-768. doi:10.1038/s41586-022-04835-6
#    NSP6_MAE         ### membrane-associated element: 237-276      Source: Han Y, Yuan Z, Yi Z. Identification of a membrane-associated element (MAE) in the C-terminal region of SARS-CoV-2 nsp6 that is essential for viral replication. J Virol. 2024;98(5):e0034924. doi:10.1128/jvi.00349-24
#    NSP6_cyto_CTD    ### Cytoplasmic CTD: 277-290
####################################################################################################################################
#    NSP12_NiRAN      ### NSP12:1-250    ORF1a:4393-4401 + ORF1b:1-241
#    NSP12_intrfce    ### NSP12:251-398  ORF1b:242-389
#    NSP12_fingers    ### NSP12:399-581  ORF1b:390-572
#    NSP12_palm       ### NSP12:582-627  ORF1b:573-618
#    NSP12_palmLnk    ### NSP12:628-687  ORF1b:619-678
#    NSP12_palm2      ### NSP12:688-812  ORF1b:679-803
#    NSP12_thumb      ### NSP12:813-932  ORF1b:804-923
#####################################################################################################################################
#    NSP13_ZBD     ### NSP13:1-101    ORF1b:924-1024
#    NSP13_stalk   ### NSP13:102-150  ORF1b:1025-1073
#    NSP13_1B      ### NSP13:151-226  ORF1b:1074-1149
#    NSP13_RecA1   ### NSP13:261-442  ORF1b:1174-1365
#    NSP13_RecA2   ### NSP13:443-601  ORF1b:1366-1524
#####################################################################################################################################
#    NSP14_nsp10  ### NSP14:1-58
#    NSP14_EXON       ### NSP14:67-282 
#    NSP14_hinge1     ### NSP14:286-300
#    NSP14_hinge2     ### NSP14:411-434
#    NSP14_N7MTase    ### NSP14:302-527
#####################################################################################################################################
#### NSP15 domain source: Pillon MC, Frazier MN, Dillard LB, et al. Cryo-EM structures of the SARS-CoV-2 endoribonuclease Nsp15 reveal insight into nuclease specificity and dynamics. Nat Commun. 2021;12(1):636. Published 2021 Jan 27. doi:10.1038/s41467-020-20608-z
#    NSP15_NTD    ### 1:64
#    NSP15_MD     ### 65:182, middle domain
#    NSP15_endoU  ### 207-347, Endonuclease, poly-U specific
#####################################################################################################################################
#    S_S1          ### S:2-686
#    S_S2          ### S:687-1273  
#    S_NTD         ### S:2-305      N-terminal domain
#    S_N2R         ### S:306-334 =  NTD-to-RBD linker  Source: Structural diversity of the SARS-CoV-2 Omicron spike; Gobeil SMC, et al, Molecular Cell, 2022-6-2; https://doi.org/10.1016/j.molcel.2022.03.028
#    S_RBD         ### S:335-528    receptor-binding domain
#    S_RBM         ### S:438-506    receptor-binding motif
#    S_SD1         ### S:529-591    subdomain 2 (AKA CTD1, RBD-s)
#    S_SD2         ### S:592-674    subdomain 2 (AKA CTD2, NTD-s)
#    S_630_loop    ### S:619-642    Source: Structural diversity of the SARS-CoV-2 Omicron spike; Gobeil SMC, et al, Molecular Cell, 2022-6-2; https://doi.org/10.1016/j.molcel.2022.03.028
#    S_FCS_region  ### S:675-691    furin cleavage site region
######## Source for all regions below: Xing L, Liu Z, Wang X, et al. Early fusion intermediate of ACE2-using coronavirus spike acting as an antiviral target. Cell. 2025;188(5):1297-1314.e24. doi:10.1016/j.cell.2025.01.012
#    S_Beta1        ### S:718-729    beta-strand; 
#    S_3H          ### S:737-769    3-helix; 
#    S_IL770       ### S:770-831    IL770
#    S_FPPR        ### S:832–866    fusion peptide-proximal region
#    S_FP          ### S:867–909    fusion peptide; 
#    S_HR1         ### S:910-985    heptad repeat 1; 
#    S_CH          ### S:986-1035   central helix
#    S_CD          ### S:1036-1068  connector domain
#    S_Beta2       ### S:1127-1135  beta-strand
#    S_2turnHelix  ### S:1148-1155  "two-turn helix"
#    S_HR2         ### S:1164-1211  heptad repeat 2
#    S_TM          ### S:1212-1234  transmembrane
#    S_CT          ### S:1235-1273  cytoplasmic tail
#####################################################################################################################################
## Source for all three E regions: Pearson GJ, Mears HV, Broncel M, Snijders AP, Bauer DLV, Carlton JG. ER-export and ARFRP1/AP-1-dependent delivery of SARS-CoV-2 Envelope to lysosomes controls late stages of viral replication. Sci Adv. 2024;10(14):eadl5012. doi:10.1126/sciadv.adl5012
#    E_TM        ## E:9-38   transmembrane region
#    E_cytosol   ## E:54-65  with YxxØ & quasi FxxFxxxR motifs "responsible for Golgi-to-lysosome trafficking" 
#    E_CTD       ## E:69-75  motif "drives ER export" 
#####################################################################################################################################
## ORF3a TM source: Source: Miao G, Zhao H, Li Y, et al. ORF3a of the COVID-19 virus SARS-CoV-2 blocks HOPS complex-mediated assembly of the SNARE complex required for autolysosome formation. Dev Cell. 2021;56(4):427-442.e5. doi:10.1016/j.devcel.2020.12.010
# "ORF3a_SignalP"=>BitSet(1:14), "ORF3a_NTD"=>BitSet(15:33), "ORF3a_TM1"=>BitSet(34:56), "ORF3a_TM12Lnk"=>BitSet(57:76), "ORF3a_TM2"=>BitSet(77:99), "ORF3a_TM3"=>BitSet(103:125), "ORF3a_cytosl1"=>BitSet(126:169), "ORF3a_Loop"=>BitSet(170:183), "ORF3a_3DB"=>BitSet(170:222), "ORF3a_CTD"=>BitSet(223:275),
#    ORF3a_SignalP   ### ORF3a:1-14
#    ORF3a_NTD       ### ORF3a:15-33
#    ORF3a_TM1       ### ORF3a:34-56
#    ORF3a_TM12Lnk  ### ORF3a:57-76
#    ORF3a_TM2       ### ORF3a:77-99
#    ORF3a_TM3       ### ORF3a:103-125
#    ORF3a_cytosl1  ### ORF3a:126-169
#    ORF3a_Loop      ### ORF3a:170-183
#    ORF3a_3DB       ### ORF3a:170-222
#    ORF3a_CTD       ### ORF3a:223-275
#####################################################################################################################################
#    N_N1          ### N:2-44
#    N_N2          ### N:45-175
#    N_N3          ### N:176-246
#    N_N4          ### N:247-364
#    N_N5          ### N:365-419
#    N_SR          ### N:176-206
#    N_L_helix     ### N:215-235
#    N_CBP         ### N:247-279
#    N_9b_overlap  ### N:4-101
####################################################################################################################################

In [43]:
### Dataframe and CSV File stuff for chronic_search_muts chronic search muts
gene(n) = split(n, ":")[1]
poz(n) = parse(Int, string(split(n, ":")[2][2:end-1]))
chronic_search_muts_v2_arr = Vector{String}()
for mut in chronic_search_muts_v2
    push!(chronic_search_muts_v2_arr, mut)
end
chr_srch_mut_sort = sort(chronic_search_muts_v2_arr, by = x -> (gene(x), poz(x)))
chr_srch_mut_sort_len = length(chr_srch_mut_sort)
println("Number of Mutations = $(chr_srch_mut_sort_len)")
#######################################
df_chr_search_muts_v2 = DataFrame("Chronic Search Mutations" => chr_srch_mut_sort)
#display(df_chr_search_muts_v2)
CSV.write("__CSV_Files/df_chr_search_muts_v2.csv", df_chr_search_muts_v2)
########################################
chronic_search_muts_table = pretty_table(df_chr_search_muts_v2)
########################################

Number of Mutations = 1735
┌──────────────────────────┐
│ Chronic Search Mutations │
│                   String │
├──────────────────────────┤
│                    E:V5D │
│                    E:V5A │
│                    E:V5F │
│                    E:V5L │
│                    E:S6L │
│                    E:E8A │
│                   E:G10S │
│                   E:G10C │
│                   E:G10A │
│                   E:G10V │
│                   E:T11A │
│                   E:V14A │
│                   E:N15I │
│                   E:N15K │
│                   E:N15H │
│                   E:N15Y │
│                   E:N15T │
│                   E:N15S │
│                   E:S16I │
│                   E:S16R │
│                   E:S16N │
│                   E:L18R │
│                   E:L19P │
│                   E:L19V │
│                   E:L19F │
│                   E:L19S │
│                   E:F20S │
│                   E:A22D │
│                   E:A22T │
│               

In [12]:
### Fx: chronic_search + Chronic cross-check list stuff
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp")
date = Dates.format(today(),"yyyy_mm_dd")
println(date_now)
function chronic_epis(chronic_ndjson)
    chr_epi_set = Set{String}()
    for line in eachline(chronic_ndjson)
        j = JSON3.read(line)
        name = EPI_ISL(j.seqName)
        push!(chr_epi_set, name)
        # print(name, ", ")
    end
    return chr_epi_set
end
function multiepi_to_epis(multi)  
    epi_num_only_pre(n) = split(n, "_")[3]
    epi_num_only_first(n) = parse(Int, split(epi_num_only_pre(n), "-")[1])
    epi_num_only_last(n) = parse(Int, split(epi_num_only_pre(n), "-")[2])
    first = epi_num_only_first(multi)
    last = epi_num_only_last(multi)
    epi_arr = Vector{String}()
    for i in first:last
        i_str = string(i)
        epi = "EPI_ISL_$(i_str)"
        push!(epi_arr, epi)
    end
    return epi_arr
end
function stringlist_to_strings(txt)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    arr_of_strings1 = Vector{String}()
    arr_of_strings2 = Vector{String}()
    no_newlines = replace(txt, 
        "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(arr_of_strings2, mseq)
            end
        else 
            push!(arr_of_strings2, seq)
        end
    end
    sort_arr_of_strings2 = sort(collect(arr_of_strings2), by = x -> epi_sortkey(x))
    return sort_arr_of_strings2
end
function stringlist_to_strings_set(txt::String)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    set_of_strings = Set{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(set_of_strings, mseq)
            end
        else 
            push!(set_of_strings, seq)
        end
    end
    return set_of_strings
end 
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); print("\n"^1); println(date_now)
############################################################################################################################################################################
abs_min_mut_thresh = 8
DQ_mut_thresh = 10
date_pct_DQ_thresh = 95
rep_thresh = 5
revs_thresh = 6
HQCS_qc_str = "5_1_5"
EPCI_qc_str = "$(abs_min_mut_thresh)_$(DQ_mut_thresh)_$(date_pct_DQ_thresh)"
print_ct_thresh = 1
date = "2026_02_20"
ndjson_name = "chronics_2026_02_01__6153seq_v2"
folder_name = "$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)"
println("EPCI_qc_str = $(EPCI_qc_str) | HQCS_qc_str = $(HQCS_qc_str)")
global all_seqs_set_DQ = load("$(folder_name)/$(folder_name)__all_seqs_set.jld2", "all_seqs_set")

date_pct_DQ_thresh = 1
DQ_mut_thresh = 1
abs_min_mut_thresh = 1
EPCI_qc_str = "$(abs_min_mut_thresh)_$(DQ_mut_thresh)_$(date_pct_DQ_thresh)"
folder_name = "$(date)__$(ndjson_name)__EPCI_$(EPCI_qc_str)__HQCS_$(HQCS_qc_str)"
global all_seqs_set = load("$(folder_name)/$(folder_name)__all_seqs_set.jld2", "all_seqs_set")
############################################################################################################################################################################
new_chronic = stringlist_to_strings_set("EPI_ISL_17159284, EPI_ISL_17159370, EPI_ISL_17159456, EPI_ISL_17159198")
Russian_Kaluga_duplicate = Set(["EPI_ISL_18647516"])
interesting_wastewater = stringlist_to_strings_set("EPI_ISL_18111774, EPI_ISL_12836539")
Good_But_Not_Chronic = stringlist_to_strings_set("EPI_ISL_5336026, EPI_ISL_1419150, EPI_ISL_894228, EPI_ISL_894229, EPI_ISL_1165744, EPI_ISL_1232443, EPI_ISL_1370821")
mink = stringlist_to_strings_set("EPI_ISL_7082794, EPI_ISL_3494590, EPI_ISL_3494726, EPI_ISL_3496945, EPI_ISL_3496948, EPI_ISL_7082794, EPI_ISL_7083492, EPI_ISL_8619831, EPI_ISL_13107810, EPI_ISL_8174956, EPI_ISL_8189787-8189788, EPI_ISL_13107809, EPI_ISL_13107811, EPI_ISL_13110241, EPI_ISL_8174953-8174955, EPI_ISL_8175130, EPI_ISL_8175132-8175133, EPI_ISL_8514993-8514997, EPI_ISL_8527347-8527353, EPI_ISL_8564587, EPI_ISL_8564590, EPI_ISL_8619819-8619830, EPI_ISL_8786093-8786094, EPI_ISL_10217391-10217398, EPI_ISL_10217400-10217424, EPI_ISL_10437323-10437361, EPI_ISL_10509691, EPI_ISL_10571267, EPI_ISL_10576571, EPI_ISL_10580472, EPI_ISL_11826874-11826875, EPI_ISL_11826908, EPI_ISL_11827547-11827549, EPI_ISL_11827576, EPI_ISL_11827583, EPI_ISL_11827615, EPI_ISL_11827892, EPI_ISL_11827942-11827943, EPI_ISL_11828060, EPI_ISL_11828282, EPI_ISL_11828393, EPI_ISL_13107812-13107816, EPI_ISL_13107818-13107822, EPI_ISL_13110242-13110249, EPI_ISL_13466466-13466470, EPI_ISL_13467028-13467031, EPI_ISL_19096357, EPI_ISL_16811138, EPI_ISL_16811146, EPI_ISL_16811151, EPI_ISL_16811154, EPI_ISL_16994016, EPI_ISL_16994017, EPI_ISL_16994018, EPI_ISL_16994087")
deer = stringlist_to_strings_set("EPI_ISL_19259473, EPI_ISL_19259477, EPI_ISL_19259485")
############################################################################################################################################################################    
Angola_A = stringlist_to_strings_set("EPI_ISL_1347940, EPI_ISL_1347941, EPI_ISL_1347942, EPI_ISL_2419105, EPI_ISL_2746031")
AT1 = stringlist_to_strings_set("EPI_ISL_2385089, EPI_ISL_18014433, EPI_ISL_18014355, EPI_ISL_3454837, EPI_ISL_2245684, EPI_ISL_2385137, EPI_ISL_18016392, EPI_ISL_18027763, EPI_ISL_1652622, EPI_ISL_18027770, EPI_ISL_2245728, EPI_ISL_2258756, EPI_ISL_2450459, EPI_ISL_2245675, EPI_ISL_2698607, EPI_ISL_3026520, EPI_ISL_2385127, EPI_ISL_11246080, EPI_ISL_1652580, EPI_ISL_2245775, EPI_ISL_2385430, EPI_ISL_2385240, EPI_ISL_18014354, EPI_ISL_2258757, EPI_ISL_2385264, EPI_ISL_2245737, EPI_ISL_2385132, EPI_ISL_2038887, EPI_ISL_2245644, EPI_ISL_11267839, EPI_ISL_1491610, EPI_ISL_1919575, EPI_ISL_1652607, EPI_ISL_3101247, EPI_ISL_2816166, EPI_ISL_3101347, EPI_ISL_3101296, EPI_ISL_2385268, EPI_ISL_1491611, EPI_ISL_2245680, EPI_ISL_1652608, EPI_ISL_18014306, EPI_ISL_3076755, EPI_ISL_2694579, EPI_ISL_3040116, EPI_ISL_2385306, EPI_ISL_2385222, EPI_ISL_3021542, EPI_ISL_1652639, EPI_ISL_2385317, EPI_ISL_2609255, EPI_ISL_2609191, EPI_ISL_2609371, EPI_ISL_18018216, EPI_ISL_2258962, EPI_ISL_2385285, EPI_ISL_18015138, EPI_ISL_2385294, EPI_ISL_11267844, EPI_ISL_2523392, EPI_ISL_2385305, EPI_ISL_12389148, EPI_ISL_11267840, EPI_ISL_2385426, EPI_ISL_2385141, EPI_ISL_1652545, EPI_ISL_3101302, EPI_ISL_2363834, EPI_ISL_2616208, EPI_ISL_2385265, EPI_ISL_2642249, EPI_ISL_18014342, EPI_ISL_2363825, EPI_ISL_2523425, EPI_ISL_18018316, EPI_ISL_3040109, EPI_ISL_2385427, EPI_ISL_2245760, EPI_ISL_1548041, EPI_ISL_4093675, EPI_ISL_2609203, EPI_ISL_2385115, EPI_ISL_2385223, EPI_ISL_2385086, EPI_ISL_2967111, EPI_ISL_18014405, EPI_ISL_2757968, EPI_ISL_18015053, EPI_ISL_11267843, EPI_ISL_18017208, EPI_ISL_11225838, EPI_ISL_2657493, EPI_ISL_2385327, EPI_ISL_1652585, EPI_ISL_2032642, EPI_ISL_2523721, EPI_ISL_2245670, EPI_ISL_18029419, EPI_ISL_1823179, EPI_ISL_7850437, EPI_ISL_2385286, EPI_ISL_3076760, EPI_ISL_2485133, EPI_ISL_18018233, EPI_ISL_18027824, EPI_ISL_2385376, EPI_ISL_2038912, EPI_ISL_2385269, EPI_ISL_3040113, EPI_ISL_2385325, EPI_ISL_2887541, EPI_ISL_3454841, EPI_ISL_2245716, EPI_ISL_2523667, EPI_ISL_2385234, EPI_ISL_2689418, EPI_ISL_2663108, EPI_ISL_2609134, EPI_ISL_2967104, EPI_ISL_5049878, EPI_ISL_2245717, EPI_ISL_2644858, EPI_ISL_2523696, EPI_ISL_2385238, EPI_ISL_18014312, EPI_ISL_2887421, EPI_ISL_2245679, EPI_ISL_2385171, EPI_ISL_13287205, EPI_ISL_3798354, EPI_ISL_3101286, EPI_ISL_2523759, EPI_ISL_3101315, EPI_ISL_1919459, EPI_ISL_2385201, EPI_ISL_8112527, EPI_ISL_1259282, EPI_ISL_2245761, EPI_ISL_2385198, EPI_ISL_2385307, EPI_ISL_1263541, EPI_ISL_3135785, EPI_ISL_5415169, EPI_ISL_18014259, EPI_ISL_2038886, EPI_ISL_2263172, EPI_ISL_2385216, EPI_ISL_2245665, EPI_ISL_2245649, EPI_ISL_3040110, EPI_ISL_2245663, EPI_ISL_7850438, EPI_ISL_11246081, EPI_ISL_1259283, EPI_ISL_11267841, EPI_ISL_2608656, EPI_ISL_2694559, EPI_ISL_2385421, EPI_ISL_18014137, EPI_ISL_2450562, EPI_ISL_2038871, EPI_ISL_2609088, EPI_ISL_2385082, EPI_ISL_11267842, EPI_ISL_3076749, EPI_ISL_6686962, EPI_ISL_2608657, EPI_ISL_1652635, EPI_ISL_12389150, EPI_ISL_2385109, EPI_ISL_3101381, EPI_ISL_2385229, EPI_ISL_1664699, EPI_ISL_2245763, EPI_ISL_2385110, EPI_ISL_11267838, EPI_ISL_18015144, EPI_ISL_2450534, EPI_ISL_1664729, EPI_ISL_2450556, EPI_ISL_2523448, EPI_ISL_2626347, EPI_ISL_2245677, EPI_ISL_1548040, EPI_ISL_2363838, EPI_ISL_2523400, EPI_ISL_2698617, EPI_ISL_3101291, EPI_ISL_2609233, EPI_ISL_1823178, EPI_ISL_2887542, EPI_ISL_18014235, EPI_ISL_18014407, EPI_ISL_2457502, EPI_ISL_2816398, EPI_ISL_2385263, EPI_ISL_2608651, EPI_ISL_2385431, EPI_ISL_2523611, EPI_ISL_2450517, EPI_ISL_1652557")
AV1 = stringlist_to_strings_set("EPI_ISL_4566929, EPI_ISL_2125482, EPI_ISL_2239779, EPI_ISL_2004704, EPI_ISL_2240090, EPI_ISL_2240151, EPI_ISL_2122546, EPI_ISL_2198382, EPI_ISL_1987067, EPI_ISL_2021628, EPI_ISL_2624764, EPI_ISL_2434445, EPI_ISL_2127047, EPI_ISL_2236823, EPI_ISL_2520581, EPI_ISL_2236967, EPI_ISL_2705045, EPI_ISL_2120895, EPI_ISL_1857951, EPI_ISL_2237157, EPI_ISL_2239580, EPI_ISL_2171756, EPI_ISL_2434675, EPI_ISL_2237958, EPI_ISL_2239457, EPI_ISL_2518284, EPI_ISL_2240292, EPI_ISL_2570109, EPI_ISL_2457326, EPI_ISL_2706055, EPI_ISL_2625296, EPI_ISL_2518253, EPI_ISL_2518281, EPI_ISL_2517875, EPI_ISL_2021658, EPI_ISL_2625337, EPI_ISL_2625442, EPI_ISL_2318105, EPI_ISL_2518496, EPI_ISL_2721970, EPI_ISL_2236548, EPI_ISL_1857666, EPI_ISL_2253970, EPI_ISL_2092569, EPI_ISL_2437210, EPI_ISL_2514457, EPI_ISL_2517895, EPI_ISL_1857609, EPI_ISL_2434746, EPI_ISL_2355937, EPI_ISL_2127052, EPI_ISL_2240008, EPI_ISL_2434695, EPI_ISL_2240274, EPI_ISL_2240118, EPI_ISL_2518887, EPI_ISL_2004581, EPI_ISL_2239978, EPI_ISL_2518712, EPI_ISL_2717065, EPI_ISL_2138347, EPI_ISL_2126474, EPI_ISL_2517941, EPI_ISL_2347631, EPI_ISL_2356078, EPI_ISL_2004900, EPI_ISL_2356178, EPI_ISL_2171691, EPI_ISL_2239677, EPI_ISL_2707108, EPI_ISL_2356235, EPI_ISL_2240054, EPI_ISL_2240678, EPI_ISL_2517872, EPI_ISL_2518852, EPI_ISL_2625597, EPI_ISL_2005439, EPI_ISL_3291235, EPI_ISL_2434355, EPI_ISL_2238485, EPI_ISL_2239850, EPI_ISL_1912100, EPI_ISL_2517910, EPI_ISL_2355982, EPI_ISL_2092318, EPI_ISL_2664976, EPI_ISL_2518249, EPI_ISL_2122397, EPI_ISL_2394252, EPI_ISL_2434509, EPI_ISL_2742613, EPI_ISL_2240131, EPI_ISL_2355981, EPI_ISL_2434451, EPI_ISL_2625398, EPI_ISL_2394056, EPI_ISL_2355966, EPI_ISL_2120972, EPI_ISL_2353408, EPI_ISL_2199581, EPI_ISL_2240019, EPI_ISL_2743887, EPI_ISL_2623351, EPI_ISL_2487128, EPI_ISL_2120960, EPI_ISL_2138441, EPI_ISL_2240680, EPI_ISL_1980365, EPI_ISL_2128461, EPI_ISL_2240079, EPI_ISL_2119669, EPI_ISL_2518053, EPI_ISL_1698537, EPI_ISL_2434465, EPI_ISL_2236937, EPI_ISL_2239489, EPI_ISL_2348421, EPI_ISL_2518704, EPI_ISL_2355974, EPI_ISL_2236504, EPI_ISL_2356883, EPI_ISL_2434473, EPI_ISL_2518367, EPI_ISL_2004376, EPI_ISL_2518126, EPI_ISL_2518270, EPI_ISL_1857753, EPI_ISL_2199394, EPI_ISL_2239848, EPI_ISL_2240116, EPI_ISL_2366158, EPI_ISL_2022294, EPI_ISL_2120273, EPI_ISL_2723162, EPI_ISL_2138170, EPI_ISL_2356106, EPI_ISL_2239443, EPI_ISL_1980414, EPI_ISL_2240058, EPI_ISL_2240677, EPI_ISL_2121628, EPI_ISL_2356045, EPI_ISL_1980353, EPI_ISL_2392572, EPI_ISL_2706009, EPI_ISL_2437243, EPI_ISL_2022182, EPI_ISL_2138472, EPI_ISL_2199542, EPI_ISL_1921237, EPI_ISL_2239816")
B1_243_2 = stringlist_to_strings_set("EPI_ISL_5059975, EPI_ISL_5778139, EPI_ISL_6253803, EPI_ISL_6437714, EPI_ISL_6437731, EPI_ISL_6810194, EPI_ISL_4370844, EPI_ISL_19101329, EPI_ISL_19101331, EPI_ISL_19101352, EPI_ISL_2341030, EPI_ISL_2341032, EPI_ISL_2341042, EPI_ISL_2598009, EPI_ISL_2779620, EPI_ISL_2790212, EPI_ISL_2861192, EPI_ISL_2966365, EPI_ISL_2966381, EPI_ISL_2966385, EPI_ISL_2966388, EPI_ISL_2966389, EPI_ISL_2966390, EPI_ISL_2966391, EPI_ISL_2966393, EPI_ISL_2966394, EPI_ISL_2966395, EPI_ISL_2966397, EPI_ISL_2966398, EPI_ISL_2970391, EPI_ISL_3010940, EPI_ISL_3010945, EPI_ISL_3010951, EPI_ISL_3062847, EPI_ISL_3062855, EPI_ISL_3062858, EPI_ISL_3133773, EPI_ISL_3133784, EPI_ISL_3162228, EPI_ISL_3162233, EPI_ISL_3162237, EPI_ISL_3162259, EPI_ISL_3268387, EPI_ISL_3268388, EPI_ISL_3371410, EPI_ISL_3371411, EPI_ISL_3371412, EPI_ISL_3371413, EPI_ISL_3509975, EPI_ISL_3557000, EPI_ISL_3557006, EPI_ISL_3568782, EPI_ISL_3568783, EPI_ISL_3743213, EPI_ISL_3743216, EPI_ISL_3753944, EPI_ISL_3769234, EPI_ISL_3827105, EPI_ISL_3848024, EPI_ISL_3857817, EPI_ISL_5655176, EPI_ISL_5217681, EPI_ISL_5925120, EPI_ISL_6327614, EPI_ISL_4299659, EPI_ISL_4370973, EPI_ISL_4959931, EPI_ISL_4706700, EPI_ISL_4706704, EPI_ISL_4571920, EPI_ISL_4571923, EPI_ISL_4508892, EPI_ISL_4508893, EPI_ISL_4508918, EPI_ISL_4359868, EPI_ISL_2245655, EPI_ISL_2385087, EPI_ISL_2385112, EPI_ISL_2385117, EPI_ISL_2385134, EPI_ISL_2450564, EPI_ISL_3015043, EPI_ISL_17105781, EPI_ISL_17105784, EPI_ISL_17105799, EPI_ISL_17105800, EPI_ISL_17105801")
B1_616 = stringlist_to_strings_set("EPI_ISL_1259297, EPI_ISL_1262772, EPI_ISL_1292807, EPI_ISL_1292808, EPI_ISL_1443902, EPI_ISL_1526787, EPI_ISL_1526780, EPI_ISL_2178439, EPI_ISL_2178448, EPI_ISL_2178432, EPI_ISL_1655920, EPI_ISL_1739315, EPI_ISL_1655921, EPI_ISL_1118893, EPI_ISL_2178447, EPI_ISL_1443880, EPI_ISL_1526779, EPI_ISL_1111064, EPI_ISL_1409708, EPI_ISL_1118892, EPI_ISL_1381829, EPI_ISL_1259307, EPI_ISL_1489729, EPI_ISL_1526783, EPI_ISL_1739314, EPI_ISL_1739317, EPI_ISL_1526778, EPI_ISL_1318289, EPI_ISL_1526786, EPI_ISL_1655918, EPI_ISL_1540445, EPI_ISL_2281746, EPI_ISL_1526793, EPI_ISL_1526794, EPI_ISL_2178418, EPI_ISL_1381830, EPI_ISL_1739313, EPI_ISL_2178417, EPI_ISL_1110211, EPI_ISL_1655919, EPI_ISL_1239370, EPI_ISL_1525263")
B1_632 = stringlist_to_strings_set("EPI_ISL_3326947, EPI_ISL_2942176, EPI_ISL_3610014, EPI_ISL_20118501, EPI_ISL_2343008, EPI_ISL_2402068, EPI_ISL_2479894, EPI_ISL_2479898, EPI_ISL_2479906, EPI_ISL_2490409, EPI_ISL_2490411, EPI_ISL_2616958, EPI_ISL_2658951, EPI_ISL_2658952, EPI_ISL_2658953, EPI_ISL_2671630, EPI_ISL_2681236, EPI_ISL_2681271, EPI_ISL_2681324, EPI_ISL_2778011, EPI_ISL_2801662, EPI_ISL_2801665, EPI_ISL_2801886, EPI_ISL_2835074, EPI_ISL_2835075, EPI_ISL_2835076, EPI_ISL_2835085, EPI_ISL_2835086, EPI_ISL_2835087, EPI_ISL_2835088, EPI_ISL_2920739, EPI_ISL_2926853, EPI_ISL_2928207, EPI_ISL_2942474, EPI_ISL_2942481, EPI_ISL_2942483, EPI_ISL_2942635, EPI_ISL_2942683, EPI_ISL_2942695, EPI_ISL_2970046, EPI_ISL_2978610, EPI_ISL_2987720, EPI_ISL_3048080, EPI_ISL_3067876, EPI_ISL_3067941, EPI_ISL_3084636, EPI_ISL_3084750, EPI_ISL_3085041, EPI_ISL_3085047, EPI_ISL_3110921, EPI_ISL_3155372, EPI_ISL_3155430, EPI_ISL_3155433, EPI_ISL_3160927, EPI_ISL_3219930, EPI_ISL_3239305, EPI_ISL_3239820, EPI_ISL_3265665, EPI_ISL_3265672, EPI_ISL_3265675, EPI_ISL_3277764, EPI_ISL_3277772, EPI_ISL_3327183, EPI_ISL_3328423, EPI_ISL_3330563, EPI_ISL_3330564, EPI_ISL_3347599, EPI_ISL_3347600, EPI_ISL_3347601, EPI_ISL_3347602, EPI_ISL_3347603, EPI_ISL_3347604, EPI_ISL_3347611, EPI_ISL_3385228, EPI_ISL_3388731, EPI_ISL_3393484, EPI_ISL_3408494, EPI_ISL_3408975, EPI_ISL_3428767, EPI_ISL_3431392, EPI_ISL_3491927, EPI_ISL_3492216, EPI_ISL_3494886, EPI_ISL_3495427, EPI_ISL_3496863, EPI_ISL_3501228, EPI_ISL_3503159, EPI_ISL_3503427, EPI_ISL_3503636, EPI_ISL_3520843, EPI_ISL_3556922, EPI_ISL_3556959, EPI_ISL_3556963, EPI_ISL_3610010, EPI_ISL_3610012, EPI_ISL_3713850, EPI_ISL_3713864, EPI_ISL_3715035, EPI_ISL_3721729, EPI_ISL_3721730, EPI_ISL_3721785, EPI_ISL_3721792, EPI_ISL_3721953, EPI_ISL_3757608, EPI_ISL_3769078, EPI_ISL_3805701, EPI_ISL_3805702, EPI_ISL_3814483, EPI_ISL_3817319, EPI_ISL_3817320, EPI_ISL_3848578, EPI_ISL_3848863, EPI_ISL_3851025, EPI_ISL_3855894, EPI_ISL_3905876, EPI_ISL_3912953, EPI_ISL_3913297, EPI_ISL_3945907, EPI_ISL_4006304, EPI_ISL_4006313, EPI_ISL_4035603, EPI_ISL_4071438, EPI_ISL_4158860, EPI_ISL_4158861, EPI_ISL_4232402, EPI_ISL_4232459, EPI_ISL_4276964, EPI_ISL_4298883, EPI_ISL_4298887, EPI_ISL_4412175, EPI_ISL_4455120, EPI_ISL_4466556, EPI_ISL_4507562, EPI_ISL_4550213, EPI_ISL_4743240, EPI_ISL_4743997, EPI_ISL_4917854, EPI_ISL_4917855, EPI_ISL_5014464, EPI_ISL_5193918, EPI_ISL_5337621, EPI_ISL_5338716, EPI_ISL_5340253, EPI_ISL_5340620, EPI_ISL_5342459, EPI_ISL_5344003, EPI_ISL_5344344, EPI_ISL_5344451, EPI_ISL_5517977, EPI_ISL_5781530, EPI_ISL_6475949, EPI_ISL_6483957, EPI_ISL_6486767, EPI_ISL_7812871, EPI_ISL_7813075, EPI_ISL_7813116, EPI_ISL_7813229, EPI_ISL_7813230, EPI_ISL_7813236, EPI_ISL_9540705, EPI_ISL_9545466, EPI_ISL_9546569, EPI_ISL_9548140, EPI_ISL_11090742, EPI_ISL_19570451, EPI_ISL_19587372")
B1_633 = stringlist_to_strings_set("EPI_ISL_4132923, EPI_ISL_3006978, EPI_ISL_5978028, EPI_ISL_2556441, EPI_ISL_2253528, EPI_ISL_3006966, EPI_ISL_2880232, EPI_ISL_3277096, EPI_ISL_2461370, EPI_ISL_2556087, EPI_ISL_6024311, EPI_ISL_2881152, EPI_ISL_3267406, EPI_ISL_3267216, EPI_ISL_2461371, EPI_ISL_2880231, EPI_ISL_3277103")
B1_637_1 = stringlist_to_strings_set("EPI_ISL_8062698, EPI_ISL_7769956, EPI_ISL_7025451, EPI_ISL_7350400, EPI_ISL_7580569, EPI_ISL_7718156, EPI_ISL_6210089, EPI_ISL_6603205, EPI_ISL_6754699, EPI_ISL_6792660, EPI_ISL_6813070, EPI_ISL_6826473, EPI_ISL_6888292, EPI_ISL_6892560, EPI_ISL_7025451, EPI_ISL_7126024, EPI_ISL_7126268, EPI_ISL_7126596, EPI_ISL_7128688, EPI_ISL_7145825, EPI_ISL_7158721, EPI_ISL_7251933, EPI_ISL_7350400, EPI_ISL_7447025, EPI_ISL_7447510, EPI_ISL_7447532, EPI_ISL_7498572, EPI_ISL_7567411, EPI_ISL_7580569, EPI_ISL_7602745, EPI_ISL_7602755, EPI_ISL_7649030, EPI_ISL_7698214, EPI_ISL_7698366, EPI_ISL_7698724, EPI_ISL_7698761, EPI_ISL_7699601, EPI_ISL_7699963, EPI_ISL_7718156, EPI_ISL_7742408, EPI_ISL_7751808, EPI_ISL_7751853, EPI_ISL_7851125, EPI_ISL_7851482, EPI_ISL_7851594, EPI_ISL_7851775, EPI_ISL_7851928, EPI_ISL_7852675, EPI_ISL_7907991-7907993, EPI_ISL_7908113, EPI_ISL_7929082, EPI_ISL_7946679, EPI_ISL_7948596, EPI_ISL_7962082, EPI_ISL_7981526, EPI_ISL_8011329, EPI_ISL_8017095, EPI_ISL_8017382, EPI_ISL_8017451, EPI_ISL_8026270, EPI_ISL_8060502, EPI_ISL_8062840, EPI_ISL_8093540, EPI_ISL_8093960, EPI_ISL_8146663, EPI_ISL_8146698-8146699, EPI_ISL_8186328, EPI_ISL_8207383, EPI_ISL_8211035, EPI_ISL_8304874, EPI_ISL_8318401, EPI_ISL_8374349, EPI_ISL_8749229, EPI_ISL_8937903, EPI_ISL_9394541, EPI_ISL_13787627, EPI_ISL_15328188, EPI_ISL_15328191, EPI_ISL_15328196")
B1_638 = stringlist_to_strings_set("EPI_ISL_2693841, EPI_ISL_2693834, EPI_ISL_2955458, EPI_ISL_3717990, EPI_ISL_3451089, EPI_ISL_2693841, EPI_ISL_3718070, EPI_ISL_3451090, EPI_ISL_2955458, EPI_ISL_3717989, EPI_ISL_3717991, EPI_ISL_3451094, EPI_ISL_3451091, EPI_ISL_3451093, EPI_ISL_3451092")
C12 = stringlist_to_strings_set("EPI_ISL_12934369, EPI_ISL_18141392, EPI_ISL_5510293, EPI_ISL_6966915, EPI_ISL_13830454, EPI_ISL_4877022, EPI_ISL_4572846, EPI_ISL_3451295, EPI_ISL_4505691, EPI_ISL_4029943, EPI_ISL_6201919, EPI_ISL_17118749, EPI_ISL_5416314, EPI_ISL_4301822, EPI_ISL_6422305, EPI_ISL_4029946, EPI_ISL_3411457, EPI_ISL_4649835, EPI_ISL_6966431, EPI_ISL_3237233, EPI_ISL_6010139, EPI_ISL_3451195, EPI_ISL_3132623, EPI_ISL_3746871, EPI_ISL_4301836, EPI_ISL_3149300, EPI_ISL_2770450, EPI_ISL_9439179, EPI_ISL_5250003, EPI_ISL_4030023, EPI_ISL_5918093, EPI_ISL_3451362, EPI_ISL_3482536, EPI_ISL_3730315, EPI_ISL_3717993, EPI_ISL_7310266, EPI_ISL_3261918, EPI_ISL_3236953, EPI_ISL_6327852, EPI_ISL_3643862, EPI_ISL_5196557, EPI_ISL_6202863, EPI_ISL_2988404, EPI_ISL_17118744, EPI_ISL_4498625, EPI_ISL_4575101, EPI_ISL_3643966, EPI_ISL_6422315, EPI_ISL_6966423, EPI_ISL_11435082, EPI_ISL_6704861, EPI_ISL_5416342, EPI_ISL_5196625, EPI_ISL_4301774, EPI_ISL_2988405, EPI_ISL_5510296, EPI_ISL_3451144, EPI_ISL_3799102, EPI_ISL_3729072, EPI_ISL_4651575, EPI_ISL_4121731, EPI_ISL_4816930, EPI_ISL_5501810, EPI_ISL_3838520, EPI_ISL_7015167, EPI_ISL_3838614, EPI_ISL_4030024, EPI_ISL_5196561, EPI_ISL_3411459, EPI_ISL_2803815, EPI_ISL_3451214, EPI_ISL_3746811, EPI_ISL_3342732, EPI_ISL_10716937, EPI_ISL_5249928, EPI_ISL_4029912, EPI_ISL_4301764, EPI_ISL_4651914, EPI_ISL_9201638, EPI_ISL_3281602, EPI_ISL_5416309, EPI_ISL_7971616, EPI_ISL_5918098, EPI_ISL_8801146, EPI_ISL_6327942, EPI_ISL_5510294, EPI_ISL_6308315, EPI_ISL_16078910, EPI_ISL_6900134, EPI_ISL_6585209, EPI_ISL_5603133, EPI_ISL_6966611, EPI_ISL_16073449, EPI_ISL_5510477, EPI_ISL_5416370, EPI_ISL_7971718, EPI_ISL_5918082, EPI_ISL_4572320, EPI_ISL_7751200, EPI_ISL_3859880, EPI_ISL_6693554, EPI_ISL_10878448, EPI_ISL_2989113, EPI_ISL_4029941, EPI_ISL_3746752, EPI_ISL_17118742, EPI_ISL_11376225, EPI_ISL_4029927, EPI_ISL_3236186, EPI_ISL_6203560, EPI_ISL_4572840, EPI_ISL_7456442, EPI_ISL_5917927, EPI_ISL_5918091, EPI_ISL_3859890, EPI_ISL_3838321, EPI_ISL_12954333, EPI_ISL_3827640, EPI_ISL_3643965, EPI_ISL_3451544, EPI_ISL_4891676, EPI_ISL_7337403, EPI_ISL_4891529, EPI_ISL_4253663, EPI_ISL_5250000, EPI_ISL_8182904, EPI_ISL_3722231, EPI_ISL_5416341, EPI_ISL_8801148, EPI_ISL_4473177, EPI_ISL_7605669, EPI_ISL_3447713, EPI_ISL_3411467, EPI_ISL_5429037, EPI_ISL_12954319, EPI_ISL_4572340, EPI_ISL_8182902, EPI_ISL_6202177, EPI_ISL_8801145, EPI_ISL_7852873, EPI_ISL_3411463, EPI_ISL_16073445, EPI_ISL_3342730, EPI_ISL_3236951, EPI_ISL_6585226, EPI_ISL_6202861, EPI_ISL_4301828, EPI_ISL_3451391, EPI_ISL_6327902, EPI_ISL_11673422, EPI_ISL_6966874, EPI_ISL_4030022, EPI_ISL_4473159, EPI_ISL_6327957, EPI_ISL_4572321, EPI_ISL_4572845, EPI_ISL_5099204, EPI_ISL_11978044, EPI_ISL_7456446, EPI_ISL_3237063, EPI_ISL_3447717, EPI_ISL_5099171, EPI_ISL_4884442, EPI_ISL_3411681, EPI_ISL_3149306, EPI_ISL_9439165, EPI_ISL_7337381, EPI_ISL_4572347, EPI_ISL_3237084, EPI_ISL_4121681, EPI_ISL_3799162, EPI_ISL_5510297, EPI_ISL_5250001, EPI_ISL_7740878, EPI_ISL_7337385, EPI_ISL_7605582, EPI_ISL_4891546, EPI_ISL_4650213, EPI_ISL_2942287, EPI_ISL_3838556, EPI_ISL_4968419, EPI_ISL_5416369, EPI_ISL_5098944, EPI_ISL_3033753, EPI_ISL_4891545, EPI_ISL_4253610, EPI_ISL_3261970, EPI_ISL_6010138, EPI_ISL_5416297, EPI_ISL_3447773, EPI_ISL_5098812, EPI_ISL_3447714, EPI_ISL_3451378, EPI_ISL_17758039, EPI_ISL_3149313, EPI_ISL_5098807, EPI_ISL_17758080, EPI_ISL_6203099, EPI_ISL_4968413, EPI_ISL_6308256, EPI_ISL_2841668, EPI_ISL_4968415, EPI_ISL_8801151, EPI_ISL_3447758, EPI_ISL_4193905, EPI_ISL_3722270, EPI_ISL_5249995, EPI_ISL_3722264, EPI_ISL_5510295, EPI_ISL_5416523, EPI_ISL_2931281, EPI_ISL_7456406, EPI_ISL_7337388, EPI_ISL_7337401, EPI_ISL_6327890, EPI_ISL_4575118, EPI_ISL_6327851, EPI_ISL_13037920, EPI_ISL_3838375, EPI_ISL_5196617, EPI_ISL_5540191, EPI_ISL_3281600, EPI_ISL_3149299, EPI_ISL_5501902, EPI_ISL_14594777, EPI_ISL_3149307, EPI_ISL_3838569, EPI_ISL_5196616, EPI_ISL_5249952, EPI_ISL_12954336, EPI_ISL_8957611, EPI_ISL_3451358, EPI_ISL_7337390, EPI_ISL_5018646, EPI_ISL_3287712, EPI_ISL_4253656, EPI_ISL_4029935, EPI_ISL_3746874, EPI_ISL_3643842, EPI_ISL_6327948, EPI_ISL_6585204, EPI_ISL_12954342, EPI_ISL_4968411, EPI_ISL_6202316, EPI_ISL_4336900, EPI_ISL_2984801, EPI_ISL_9439008, EPI_ISL_5603137, EPI_ISL_4253637, EPI_ISL_5196531, EPI_ISL_3717995, EPI_ISL_4030021, EPI_ISL_3860015, EPI_ISL_6436394, EPI_ISL_5429026, EPI_ISL_3237098, EPI_ISL_3697115, EPI_ISL_6308257, EPI_ISL_3132566, EPI_ISL_3799095, EPI_ISL_3342734, EPI_ISL_9230060, EPI_ISL_3342733, EPI_ISL_3451569, EPI_ISL_3237092, EPI_ISL_2718062, EPI_ISL_6327850, EPI_ISL_4876926, EPI_ISL_3717994, EPI_ISL_4029948, EPI_ISL_17758081, EPI_ISL_5918092, EPI_ISL_3149312, EPI_ISL_5917886, EPI_ISL_11435083, EPI_ISL_12954141, EPI_ISL_6201944, EPI_ISL_3746788, EPI_ISL_7881950, EPI_ISL_8801147, EPI_ISL_3219805, EPI_ISL_4650090, EPI_ISL_4029923, EPI_ISL_7310320, EPI_ISL_3663539, EPI_ISL_4891705, EPI_ISL_3074033, EPI_ISL_3799035, EPI_ISL_3838541, EPI_ISL_17758085, EPI_ISL_6585202, EPI_ISL_6782048, EPI_ISL_18215881, EPI_ISL_3482519, EPI_ISL_3451222, EPI_ISL_13963277, EPI_ISL_8616711, EPI_ISL_4891659, EPI_ISL_5249934, EPI_ISL_4572381, EPI_ISL_3411458, EPI_ISL_4301791, EPI_ISL_3451369, EPI_ISL_3453877, EPI_ISL_9439176, EPI_ISL_6436401, EPI_ISL_3799163, EPI_ISL_3643860, EPI_ISL_3506424, EPI_ISL_3128775, EPI_ISL_5416371, EPI_ISL_3746772, EPI_ISL_2828749, EPI_ISL_6203297, EPI_ISL_5477673, EPI_ISL_4003563, EPI_ISL_2827937, EPI_ISL_3237237, EPI_ISL_3746842, EPI_ISL_3838512, EPI_ISL_3132608, EPI_ISL_3838515, EPI_ISL_6203350, EPI_ISL_2726854, EPI_ISL_6327853, EPI_ISL_4474416, EPI_ISL_3860060, EPI_ISL_3859884, EPI_ISL_6966660, EPI_ISL_4003556, EPI_ISL_6693553, EPI_ISL_5098934, EPI_ISL_4651145, EPI_ISL_7971590, EPI_ISL_4108306, EPI_ISL_3643903, EPI_ISL_4968412, EPI_ISL_4651357, EPI_ISL_3451301, EPI_ISL_6966959, EPI_ISL_4301837, EPI_ISL_11516060, EPI_ISL_3219868, EPI_ISL_12954357, EPI_ISL_4968418, EPI_ISL_3149301, EPI_ISL_4029964, EPI_ISL_3411589, EPI_ISL_11435084, EPI_ISL_5540181, EPI_ISL_5917846, EPI_ISL_7456429, EPI_ISL_5250002, EPI_ISL_2988409, EPI_ISL_4650166, EPI_ISL_3838489, EPI_ISL_3722284, EPI_ISL_4473174, EPI_ISL_6900144, EPI_ISL_3838621, EPI_ISL_5099194, EPI_ISL_17118750, EPI_ISL_12954142, EPI_ISL_2695610, EPI_ISL_9439051, EPI_ISL_6906407, EPI_ISL_6201931, EPI_ISL_11435085, EPI_ISL_3717932, EPI_ISL_6203343, EPI_ISL_3101505, EPI_ISL_6201939, EPI_ISL_6327854, EPI_ISL_6913983, EPI_ISL_3281601, EPI_ISL_6966691, EPI_ISL_4575181, EPI_ISL_4030025, EPI_ISL_4474432, EPI_ISL_4336918, EPI_ISL_2726855, EPI_ISL_3799137")
############################################################################################################################################################################    
UTEP = stringlist_to_strings_set("EPI_ISL_17041205, EPI_ISL_17041227, EPI_ISL_17051747, EPI_ISL_17051806, EPI_ISL_17051826, EPI_ISL_17051828, EPI_ISL_17051854, EPI_ISL_17052015, EPI_ISL_17052019, EPI_ISL_17052041, EPI_ISL_17052055, EPI_ISL_17052095, EPI_ISL_17052100, EPI_ISL_17052106, EPI_ISL_17052113, EPI_ISL_17052117, EPI_ISL_17052122, EPI_ISL_17052124, EPI_ISL_17052147, EPI_ISL_17052159-17052160, EPI_ISL_17052165, EPI_ISL_17052170, EPI_ISL_17052174, EPI_ISL_17052182, EPI_ISL_17052203, EPI_ISL_17052210, EPI_ISL_17052224-17052225, EPI_ISL_17052254, EPI_ISL_17052259, EPI_ISL_17052271, EPI_ISL_17052273, EPI_ISL_17052278, EPI_ISL_17052324, EPI_ISL_17052329, EPI_ISL_17052353, EPI_ISL_17052358, EPI_ISL_17052372, EPI_ISL_17052393-17052394, EPI_ISL_17052408, EPI_ISL_17052416, EPI_ISL_17052421, EPI_ISL_17052429, EPI_ISL_17052435, EPI_ISL_17052437-17052438, EPI_ISL_17052440, EPI_ISL_17052453, EPI_ISL_17052469, EPI_ISL_17052471-17052472, EPI_ISL_17052475, EPI_ISL_17057354, EPI_ISL_17057357, EPI_ISL_17057376, EPI_ISL_17057396, EPI_ISL_17057412, EPI_ISL_17057625, EPI_ISL_17057810, EPI_ISL_17057818, EPI_ISL_17057859, EPI_ISL_17057867, EPI_ISL_17057921, EPI_ISL_17057925, EPI_ISL_17057951, EPI_ISL_17057958, EPI_ISL_17061101, EPI_ISL_17061104, EPI_ISL_17061115, EPI_ISL_17061117, EPI_ISL_17061140, EPI_ISL_17061147, EPI_ISL_17061172, EPI_ISL_17061186, EPI_ISL_17061239, EPI_ISL_17061244, EPI_ISL_17061290, EPI_ISL_17061334, EPI_ISL_17061353, EPI_ISL_17061374, EPI_ISL_17061464, EPI_ISL_17061484, EPI_ISL_17061492, EPI_ISL_17061509, EPI_ISL_17061519, EPI_ISL_17061615-17061616")
India = stringlist_to_strings_set("EPI_ISL_479561, EPI_ISL_14192861, EPI_ISL_1415150, EPI_ISL_8804019, EPI_ISL_14408725, EPI_ISL_10944447, EPI_ISL_14408782, EPI_ISL_18936212, EPI_ISL_17047867, EPI_ISL_9091138, EPI_ISL_15157217, EPI_ISL_14441047, EPI_ISL_14227227, EPI_ISL_10943846, EPI_ISL_14227415, EPI_ISL_14227383, EPI_ISL_14227399, EPI_ISL_14227382, EPI_ISL_14227381, EPI_ISL_13391738, EPI_ISL_14441094, EPI_ISL_10924576, EPI_ISL_11333412, EPI_ISL_17372596, EPI_ISL_17565160, EPI_ISL_10205296, EPI_ISL_18094389, EPI_ISL_14999073, EPI_ISL_17372568, EPI_ISL_9419009, EPI_ISL_10729443, EPI_ISL_10729438, EPI_ISL_10729442, EPI_ISL_10729421, EPI_ISL_10066198, EPI_ISL_14679032, EPI_ISL_12906191-12906640, EPI_ISL_17466156-17466179, EPI_ISL_10716749-10716816, EPI_ISL_10772350, EPI_ISL_13718890, EPI_ISL_13847287, EPI_ISL_8527556, EPI_ISL_9782216, EPI_ISL_10080769, EPI_ISL_14211650-14211651, EPI_ISL_14211653, EPI_ISL_14211655, EPI_ISL_14211668, EPI_ISL_14211686, EPI_ISL_14211693, EPI_ISL_14211701, EPI_ISL_14211740, EPI_ISL_14211775, EPI_ISL_14211779, EPI_ISL_14211813, EPI_ISL_14211816, EPI_ISL_14211820, EPI_ISL_14211828, EPI_ISL_8806428, EPI_ISL_15293844, EPI_ISL_14677091, EPI_ISL_14681074, EPI_ISL_14595732, EPI_ISL_10306128, EPI_ISL_14443132, EPI_ISL_8882813, EPI_ISL_11468761, EPI_ISL_11468762, EPI_ISL_479776, EPI_ISL_2547229, EPI_ISL_1703710, EPI_ISL_541731, EPI_ISL_577654, EPI_ISL_2966131, EPI_ISL_11106543, EPI_ISL_11887241, EPI_ISL_11887313, EPI_ISL_11887728, EPI_ISL_11887754, EPI_ISL_11887820, EPI_ISL_12291249, EPI_ISL_12291339, EPI_ISL_9782271, EPI_ISL_8049924, EPI_ISL_8049928, EPI_ISL_17372400, EPI_ISL_17372447, EPI_ISL_17372505, EPI_ISL_17372556, EPI_ISL_17372557, EPI_ISL_17372567, EPI_ISL_15294031, EPI_ISL_15363265, EPI_ISL_14910936, EPI_ISL_14495310, EPI_ISL_14495319, EPI_ISL_14495329, EPI_ISL_14495334, EPI_ISL_14148984, EPI_ISL_13158620, EPI_ISL_13158678, EPI_ISL_13158687, EPI_ISL_13158700, EPI_ISL_12494547, EPI_ISL_12494558, EPI_ISL_12494582, EPI_ISL_12494585, EPI_ISL_12494587, EPI_ISL_12494596, EPI_ISL_12494638, EPI_ISL_12494874, EPI_ISL_10149349, EPI_ISL_14975997, EPI_ISL_14975999, EPI_ISL_14976001, EPI_ISL_14976002, EPI_ISL_14976004, EPI_ISL_14976005, EPI_ISL_14976006, EPI_ISL_14976008, EPI_ISL_14976014, EPI_ISL_14976017, EPI_ISL_14976018, EPI_ISL_14976094, EPI_ISL_14976096, EPI_ISL_14976099, EPI_ISL_14976106, EPI_ISL_14976107, EPI_ISL_14976109, EPI_ISL_14976111, EPI_ISL_14976112, EPI_ISL_14976113, EPI_ISL_14976123, EPI_ISL_14976129, EPI_ISL_14976139, EPI_ISL_14976142, EPI_ISL_14976146, EPI_ISL_14976147, EPI_ISL_14976148, EPI_ISL_14976150, EPI_ISL_14976153, EPI_ISL_14976154, EPI_ISL_14986408, EPI_ISL_14986472, EPI_ISL_14986473, EPI_ISL_14986476, EPI_ISL_14986478, EPI_ISL_14986480, EPI_ISL_14986481, EPI_ISL_14986482, EPI_ISL_14986484, EPI_ISL_14986486, EPI_ISL_14986487, EPI_ISL_14986488, EPI_ISL_14986489, EPI_ISL_14986490, EPI_ISL_14986493, EPI_ISL_14986505, EPI_ISL_14986525, EPI_ISL_14986529, EPI_ISL_14986530, EPI_ISL_14986532, EPI_ISL_14986533, EPI_ISL_14986539, EPI_ISL_14986542, EPI_ISL_14986543, EPI_ISL_14986544, EPI_ISL_14986545, EPI_ISL_14986547, EPI_ISL_14986550, EPI_ISL_14986551, EPI_ISL_14986554, EPI_ISL_14986555, EPI_ISL_14986556, EPI_ISL_14986560, EPI_ISL_14986563, EPI_ISL_14986567, EPI_ISL_14986568, EPI_ISL_14986570, EPI_ISL_14986574, EPI_ISL_14986580, EPI_ISL_14986583, EPI_ISL_14986591, EPI_ISL_14986594, EPI_ISL_14986600, EPI_ISL_14986601, EPI_ISL_14986606, EPI_ISL_14986608, EPI_ISL_14986612, EPI_ISL_14986613, EPI_ISL_14986619, EPI_ISL_14986623, EPI_ISL_14986624, EPI_ISL_14986625, EPI_ISL_14986630, EPI_ISL_14986631, EPI_ISL_14986633, EPI_ISL_14992926, EPI_ISL_14992971, EPI_ISL_14993025, EPI_ISL_14993053, EPI_ISL_14993093, EPI_ISL_14993097, EPI_ISL_14993098, EPI_ISL_14993100, EPI_ISL_14993111, EPI_ISL_14993113, EPI_ISL_14993114, EPI_ISL_14993139, EPI_ISL_14993140, EPI_ISL_14993141, EPI_ISL_14993142, EPI_ISL_14993146, EPI_ISL_14993173, EPI_ISL_14993174, EPI_ISL_15172564, EPI_ISL_15172596, EPI_ISL_15172598, EPI_ISL_15172634, EPI_ISL_15172657, EPI_ISL_15172658, EPI_ISL_15172662, EPI_ISL_15172663, EPI_ISL_15172678, EPI_ISL_15172680, EPI_ISL_15172702, EPI_ISL_15172709, EPI_ISL_15172719, EPI_ISL_15172720, EPI_ISL_15172726, EPI_ISL_15172727, EPI_ISL_15172729, EPI_ISL_15172730, EPI_ISL_15172733, EPI_ISL_15172741, EPI_ISL_15172743, EPI_ISL_15172745, EPI_ISL_15190291, EPI_ISL_15262286, EPI_ISL_15293825, EPI_ISL_15293919, EPI_ISL_15293920, EPI_ISL_15293921, EPI_ISL_15293922, EPI_ISL_15293923, EPI_ISL_15293924, EPI_ISL_15293951, EPI_ISL_15293996, EPI_ISL_15294000, EPI_ISL_15294013, EPI_ISL_15294014, EPI_ISL_15294017, EPI_ISL_15294021, EPI_ISL_15294023, EPI_ISL_15294024, EPI_ISL_15294025, EPI_ISL_15294027, EPI_ISL_15294028, EPI_ISL_11336342, EPI_ISL_11336416, EPI_ISL_11359809, EPI_ISL_11359818, EPI_ISL_11373282, EPI_ISL_11373303, EPI_ISL_11373308, EPI_ISL_11373310, EPI_ISL_11373330, EPI_ISL_11373343, EPI_ISL_11373365, EPI_ISL_11373384, EPI_ISL_11373430, EPI_ISL_11507507, EPI_ISL_11736106, EPI_ISL_11736364, EPI_ISL_11736368, EPI_ISL_11736370, EPI_ISL_11736383, EPI_ISL_11736392, EPI_ISL_11736399, EPI_ISL_11736401, EPI_ISL_11737568, EPI_ISL_11737648, EPI_ISL_11737649, EPI_ISL_11737670, EPI_ISL_11737703, EPI_ISL_11737704, EPI_ISL_11737705, EPI_ISL_11737757, EPI_ISL_11738154, EPI_ISL_11743906, EPI_ISL_11743950, EPI_ISL_11743951, EPI_ISL_11743952, EPI_ISL_11743956, EPI_ISL_11743957, EPI_ISL_11743964, EPI_ISL_11743967, EPI_ISL_11743971, EPI_ISL_11743972, EPI_ISL_11743980, EPI_ISL_11743981, EPI_ISL_11743982, EPI_ISL_11743984, EPI_ISL_11743985, EPI_ISL_11743987, EPI_ISL_11743992, EPI_ISL_11744005, EPI_ISL_11744006, EPI_ISL_11744012, EPI_ISL_11744047, EPI_ISL_11744059, EPI_ISL_11746406, EPI_ISL_11746517, EPI_ISL_11746523, EPI_ISL_11746556, EPI_ISL_11746561, EPI_ISL_11746579, EPI_ISL_11746642, EPI_ISL_11746682, EPI_ISL_11746691, EPI_ISL_11767469, EPI_ISL_11767883, EPI_ISL_11777028, EPI_ISL_11777627, EPI_ISL_11777640, EPI_ISL_11777683, EPI_ISL_11777799, EPI_ISL_11777817, EPI_ISL_11789967, EPI_ISL_11790070, EPI_ISL_11790073, EPI_ISL_11790078, EPI_ISL_11790082, EPI_ISL_11790083, EPI_ISL_11790088, EPI_ISL_11790094, EPI_ISL_11790096, EPI_ISL_11790105, EPI_ISL_11790135, EPI_ISL_11790203, EPI_ISL_11790222, EPI_ISL_11790247, EPI_ISL_11790250, EPI_ISL_11790253, EPI_ISL_11790259, EPI_ISL_11790274, EPI_ISL_11790278, EPI_ISL_11790290, EPI_ISL_11790294, EPI_ISL_11790305, EPI_ISL_11790310, EPI_ISL_11802438, EPI_ISL_11802439, EPI_ISL_11802546, EPI_ISL_11802599, EPI_ISL_11802600, EPI_ISL_11802602, EPI_ISL_11802603, EPI_ISL_11802608, EPI_ISL_11802616, EPI_ISL_11802639, EPI_ISL_11803132, EPI_ISL_11803143, EPI_ISL_11803157, EPI_ISL_11803274, EPI_ISL_11803277, EPI_ISL_11803351, EPI_ISL_11803374, EPI_ISL_11803386, EPI_ISL_12906222, EPI_ISL_12906238, EPI_ISL_12906247, EPI_ISL_12906251, EPI_ISL_12906255, EPI_ISL_12906258, EPI_ISL_12906259, EPI_ISL_12906261, EPI_ISL_12906265, EPI_ISL_12906267, EPI_ISL_12906274, EPI_ISL_12906280, EPI_ISL_12906286, EPI_ISL_12906290, EPI_ISL_12906293, EPI_ISL_12906296, EPI_ISL_12906299, EPI_ISL_12906303, EPI_ISL_12906304, EPI_ISL_12906309, EPI_ISL_12906313, EPI_ISL_12906314, EPI_ISL_12906369, EPI_ISL_12906374, EPI_ISL_12906383, EPI_ISL_12906455, EPI_ISL_12906464, EPI_ISL_12906470, EPI_ISL_12906479, EPI_ISL_12906498, EPI_ISL_12906503, EPI_ISL_12906506, EPI_ISL_12906509, EPI_ISL_12906511, EPI_ISL_12906518, EPI_ISL_12906542, EPI_ISL_12906559, EPI_ISL_12906562, EPI_ISL_12906590, EPI_ISL_12906592, EPI_ISL_12906597, EPI_ISL_12906599, EPI_ISL_12906600, EPI_ISL_12906601, EPI_ISL_12906602, EPI_ISL_12906604, EPI_ISL_12906609, EPI_ISL_12906618, EPI_ISL_12906619, EPI_ISL_12906622, EPI_ISL_12906627, EPI_ISL_12906628, EPI_ISL_12906632, EPI_ISL_12906633, EPI_ISL_12906636, EPI_ISL_12906637, EPI_ISL_12906638, EPI_ISL_12906640, EPI_ISL_12906210, EPI_ISL_12906211, EPI_ISL_12906212, EPI_ISL_12906214, EPI_ISL_12906218, EPI_ISL_12906219, EPI_ISL_12906220, EPI_ISL_12906235, EPI_ISL_12906237, EPI_ISL_12906240, EPI_ISL_12906242, EPI_ISL_12906243, EPI_ISL_12906244, EPI_ISL_12906248, EPI_ISL_12906249, EPI_ISL_12906250, EPI_ISL_12906253, EPI_ISL_12906254, EPI_ISL_12906257, EPI_ISL_12906260, EPI_ISL_12906262, EPI_ISL_12906266, EPI_ISL_12906269, EPI_ISL_12906275, EPI_ISL_12906277, EPI_ISL_12906278, EPI_ISL_12906279, EPI_ISL_12906291, EPI_ISL_12906295, EPI_ISL_12906297, EPI_ISL_12906298, EPI_ISL_12906308, EPI_ISL_12906324, EPI_ISL_12906326, EPI_ISL_12906327, EPI_ISL_12906328, EPI_ISL_12906329, EPI_ISL_12906330, EPI_ISL_12906331, EPI_ISL_12906332, EPI_ISL_12906333, EPI_ISL_12906334, EPI_ISL_12906335, EPI_ISL_12906336, EPI_ISL_12906337, EPI_ISL_12906338, EPI_ISL_12906340, EPI_ISL_12906341, EPI_ISL_12906342, EPI_ISL_12906343, EPI_ISL_12906344, EPI_ISL_12906345, EPI_ISL_12906346, EPI_ISL_12906347, EPI_ISL_12906348, EPI_ISL_12906349, EPI_ISL_12906350, EPI_ISL_12906351, EPI_ISL_12906352, EPI_ISL_12906353, EPI_ISL_12906354, EPI_ISL_12906355, EPI_ISL_12906356, EPI_ISL_12906357, EPI_ISL_12906358, EPI_ISL_12906359, EPI_ISL_12906360, EPI_ISL_12906361, EPI_ISL_12906362, EPI_ISL_12906363, EPI_ISL_12906364, EPI_ISL_12906365, EPI_ISL_12906366, EPI_ISL_12906367, EPI_ISL_12906368, EPI_ISL_12906370, EPI_ISL_12906371, EPI_ISL_12906372, EPI_ISL_12906373, EPI_ISL_12906375, EPI_ISL_12906376, EPI_ISL_12906377, EPI_ISL_12906378, EPI_ISL_12906379, EPI_ISL_12906380, EPI_ISL_12906381, EPI_ISL_12906382, EPI_ISL_12906384, EPI_ISL_12906385, EPI_ISL_12906386, EPI_ISL_12906387, EPI_ISL_12906388, EPI_ISL_12906389, EPI_ISL_12906390, EPI_ISL_12906391, EPI_ISL_12906392, EPI_ISL_12906393, EPI_ISL_12906394, EPI_ISL_12906395, EPI_ISL_12906396, EPI_ISL_12906397, EPI_ISL_12906398, EPI_ISL_12906399, EPI_ISL_12906400, EPI_ISL_12906401, EPI_ISL_12906402, EPI_ISL_12906403, EPI_ISL_12906404, EPI_ISL_12906405, EPI_ISL_12906406, EPI_ISL_12906407, EPI_ISL_12906408, EPI_ISL_12906409, EPI_ISL_12906410, EPI_ISL_12906411, EPI_ISL_12906412, EPI_ISL_12906413, EPI_ISL_12906414, EPI_ISL_12906415, EPI_ISL_12906416, EPI_ISL_12906417, EPI_ISL_12906418, EPI_ISL_12906419, EPI_ISL_12906420, EPI_ISL_12906421, EPI_ISL_12906422, EPI_ISL_12906423, EPI_ISL_12906424, EPI_ISL_12906425, EPI_ISL_12906426, EPI_ISL_12906427, EPI_ISL_12906428, EPI_ISL_12906429, EPI_ISL_12906430, EPI_ISL_12906431, EPI_ISL_12906432, EPI_ISL_12906433, EPI_ISL_12906434, EPI_ISL_12906435, EPI_ISL_12906436, EPI_ISL_12906437, EPI_ISL_12906438, EPI_ISL_12906439, EPI_ISL_12906440, EPI_ISL_12906441, EPI_ISL_12906444, EPI_ISL_12906451, EPI_ISL_12906452, EPI_ISL_12906453, EPI_ISL_12906454, EPI_ISL_12906456, EPI_ISL_12906457, EPI_ISL_12906458, EPI_ISL_12906459, EPI_ISL_12906460, EPI_ISL_12906461, EPI_ISL_12906462, EPI_ISL_12906463, EPI_ISL_12906465, EPI_ISL_12906466, EPI_ISL_12906467, EPI_ISL_12906468, EPI_ISL_12906469, EPI_ISL_12906471, EPI_ISL_12906472, EPI_ISL_12906473, EPI_ISL_12906474, EPI_ISL_12906475, EPI_ISL_12906476, EPI_ISL_12906477, EPI_ISL_12906478, EPI_ISL_12906480, EPI_ISL_12906481, EPI_ISL_12906482, EPI_ISL_12906483, EPI_ISL_12906484, EPI_ISL_12906485, EPI_ISL_12906486, EPI_ISL_12906487, EPI_ISL_12906488, EPI_ISL_12906489, EPI_ISL_12906490, EPI_ISL_12906491, EPI_ISL_12906492, EPI_ISL_12906493, EPI_ISL_12906494, EPI_ISL_12906495, EPI_ISL_12906496, EPI_ISL_12906497, EPI_ISL_12906499, EPI_ISL_12906500, EPI_ISL_12906501, EPI_ISL_12906502, EPI_ISL_12906504, EPI_ISL_12906505, EPI_ISL_12906507, EPI_ISL_12906508, EPI_ISL_12906510, EPI_ISL_12906512, EPI_ISL_12906513, EPI_ISL_12906514, EPI_ISL_12906515, EPI_ISL_12906516, EPI_ISL_12906517, EPI_ISL_12906519, EPI_ISL_12906520, EPI_ISL_12906521, EPI_ISL_12906522, EPI_ISL_12906523, EPI_ISL_12906524, EPI_ISL_12906525, EPI_ISL_12906526, EPI_ISL_12906527, EPI_ISL_12906528, EPI_ISL_12906529, EPI_ISL_12906530, EPI_ISL_12906531, EPI_ISL_12906532, EPI_ISL_12906533, EPI_ISL_12906534, EPI_ISL_12906535, EPI_ISL_12906536, EPI_ISL_12906537, EPI_ISL_12906538, EPI_ISL_12906539, EPI_ISL_12906540, EPI_ISL_12906541, EPI_ISL_12906543, EPI_ISL_12906544, EPI_ISL_12906545, EPI_ISL_12906546, EPI_ISL_12906547, EPI_ISL_12906548, EPI_ISL_12906549, EPI_ISL_12906550, EPI_ISL_12906551, EPI_ISL_12906552, EPI_ISL_12906553, EPI_ISL_12906554, EPI_ISL_12906555, EPI_ISL_12906556, EPI_ISL_12906557, EPI_ISL_12906558, EPI_ISL_12906560, EPI_ISL_12906561, EPI_ISL_12906563, EPI_ISL_12906564, EPI_ISL_12906565, EPI_ISL_12906566, EPI_ISL_12906567, EPI_ISL_12906568, EPI_ISL_12906569, EPI_ISL_12906570, EPI_ISL_12906571, EPI_ISL_12906572, EPI_ISL_12906573, EPI_ISL_12906574, EPI_ISL_12906575, EPI_ISL_12906576, EPI_ISL_12906577, EPI_ISL_12906578, EPI_ISL_12906579, EPI_ISL_12906580, EPI_ISL_12906581, EPI_ISL_12906582, EPI_ISL_12906583, EPI_ISL_12906584, EPI_ISL_12906585, EPI_ISL_12906586, EPI_ISL_12906587, EPI_ISL_12906588, EPI_ISL_12906594, EPI_ISL_12906595, EPI_ISL_12906603, EPI_ISL_12906605, EPI_ISL_12906606, EPI_ISL_12906607, EPI_ISL_12906608, EPI_ISL_12906611, EPI_ISL_12906612, EPI_ISL_12906613, EPI_ISL_12906614, EPI_ISL_12906615, EPI_ISL_12906616, EPI_ISL_12906617, EPI_ISL_12906620, EPI_ISL_12906623, EPI_ISL_12906624, EPI_ISL_12906625, EPI_ISL_12906629, EPI_ISL_12906630, EPI_ISL_12906631, EPI_ISL_12906634")
Indonesia = stringlist_to_strings_set("EPI_ISL_3019945, EPI_ISL_2854687, EPI_ISL_2854775, EPI_ISL_2868873, EPI_ISL_2868876, EPI_ISL_1824617, EPI_ISL_8889354, EPI_ISL_14583732, EPI_ISL_14584687, EPI_ISL_14584697")
Mexico = stringlist_to_strings_set("EPI_ISL_13398383, EPI_ISL_7716412, EPI_ISL_7716415, EPI_ISL_7716416, EPI_ISL_7716417, EPI_ISL_7716418, EPI_ISL_7716419, EPI_ISL_7716421, EPI_ISL_7716425, EPI_ISL_7716427, EPI_ISL_7716429, EPI_ISL_7716431, EPI_ISL_7716432, EPI_ISL_7716433, EPI_ISL_7716435, EPI_ISL_7716439, EPI_ISL_7716441, EPI_ISL_7716442, EPI_ISL_7716450, EPI_ISL_7716454, EPI_ISL_7716455, EPI_ISL_7716456, EPI_ISL_7716459, EPI_ISL_7716460, EPI_ISL_7716482, EPI_ISL_7716483, EPI_ISL_7716485, EPI_ISL_7716486, EPI_ISL_7716487, EPI_ISL_7716488, EPI_ISL_7716489, EPI_ISL_7730969")
Texas = stringlist_to_strings_set("EPI_ISL_1236653, EPI_ISL_1078985, EPI_ISL_1079742, EPI_ISL_17061113, EPI_ISL_1079482, EPI_ISL_18817042, EPI_ISL_1079887, EPI_ISL_16762249, EPI_ISL_16762269, EPI_ISL_16542938, EPI_ISL_16398020, EPI_ISL_16012710, EPI_ISL_16397783, EPI_ISL_16762458, EPI_ISL_16762297, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599980, EPI_ISL_16599842, EPI_ISL_16599826, EPI_ISL_16599959, EPI_ISL_16599794, EPI_ISL_16599636, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599492, EPI_ISL_16599498, EPI_ISL_16599495, EPI_ISL_16599649, EPI_ISL_16599554, EPI_ISL_16599460, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599299, EPI_ISL_16599467, EPI_ISL_16542453, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16398022, EPI_ISL_16397883, EPI_ISL_16398016, EPI_ISL_16397988, EPI_ISL_16397860, EPI_ISL_16397721, EPI_ISL_16397554, EPI_ISL_16397720, EPI_ISL_16397699, EPI_ISL_16271386, EPI_ISL_16271102, EPI_ISL_16271048, EPI_ISL_16271255, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16113336, EPI_ISL_16113255, EPI_ISL_16113399, EPI_ISL_16013449, EPI_ISL_16013309, EPI_ISL_16013227, EPI_ISL_16012987, EPI_ISL_16012954, EPI_ISL_16012943, EPI_ISL_16013209, EPI_ISL_16013267, EPI_ISL_16599525, EPI_ISL_16762084, EPI_ISL_16599288, EPI_ISL_16599747, EPI_ISL_16270893, EPI_ISL_16762700, EPI_ISL_16762753, EPI_ISL_16762699, EPI_ISL_16762631, EPI_ISL_16762654, EPI_ISL_16762714, EPI_ISL_16762735, EPI_ISL_16762653, EPI_ISL_16762559, EPI_ISL_16762726, EPI_ISL_16762652, EPI_ISL_16762600, EPI_ISL_16762614, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762327, EPI_ISL_16762455, EPI_ISL_16762310, EPI_ISL_16762611, EPI_ISL_16762475, EPI_ISL_16762245, EPI_ISL_16762132, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762273, EPI_ISL_16762070, EPI_ISL_16762022, EPI_ISL_16762148, EPI_ISL_16761962, EPI_ISL_16762255, EPI_ISL_16762066, EPI_ISL_16761978, EPI_ISL_16761930, EPI_ISL_16761912, EPI_ISL_16542189, EPI_ISL_16542369, EPI_ISL_16542267, EPI_ISL_16762176, EPI_ISL_16762028, EPI_ISL_16762462, EPI_ISL_16599325, EPI_ISL_16542552, EPI_ISL_16271333, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16919306, EPI_ISL_16919337, EPI_ISL_16919315, EPI_ISL_16919296, EPI_ISL_16919348, EPI_ISL_16919365, EPI_ISL_16919303, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919362, EPI_ISL_16919367, EPI_ISL_16919388, EPI_ISL_16919412, EPI_ISL_16919396, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919430, EPI_ISL_16919399, EPI_ISL_16919496, EPI_ISL_16919468, EPI_ISL_16919526, EPI_ISL_16919523, EPI_ISL_16919467, EPI_ISL_16919558, EPI_ISL_16919504, EPI_ISL_16919456, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919586, EPI_ISL_16919660, EPI_ISL_16919546, EPI_ISL_16919662, EPI_ISL_16919695, EPI_ISL_16919680, EPI_ISL_16919670, EPI_ISL_16919640, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919794, EPI_ISL_16919804, EPI_ISL_16919796, EPI_ISL_16919806, EPI_ISL_16919772, EPI_ISL_16919841, EPI_ISL_16919706, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919810, EPI_ISL_16919894, EPI_ISL_16919852, EPI_ISL_16919902, EPI_ISL_16919932, EPI_ISL_16919919, EPI_ISL_16919944, EPI_ISL_16919906, EPI_ISL_16919897, EPI_ISL_16919870, EPI_ISL_16919856, EPI_ISL_16919962")
Thailand = stringlist_to_strings_set("EPI_ISL_11720069, EPI_ISL_17785884, EPI_ISL_6136058, EPI_ISL_6599859, EPI_ISL_5916960, EPI_ISL_5916998, EPI_ISL_3636272, EPI_ISL_4348475")
Turkey = stringlist_to_strings_set("EPI_ISL_12321732, EPI_ISL_12146153, EPI_ISL_12321719, EPI_ISL_14298818, EPI_ISL_14456697, EPI_ISL_12610886, EPI_ISL_10858468, EPI_ISL_11507543, EPI_ISL_7874635, EPI_ISL_10858493, EPI_ISL_12146142, EPI_ISL_12320637, EPI_ISL_13112964, EPI_ISL_13622800, EPI_ISL_10397169, EPI_ISL_10397203, EPI_ISL_10678671, EPI_ISL_10678679, EPI_ISL_12319870, EPI_ISL_12584102, EPI_ISL_13074466, EPI_ISL_13112909, EPI_ISL_13176001, EPI_ISL_13621057, EPI_ISL_15927979, EPI_ISL_2102044, EPI_ISL_15681715, EPI_ISL_15295325")
############################################################################################################################################################################
K113N_Q115H = stringlist_to_strings_set("EPI_ISL_16932114, EPI_ISL_16869750, EPI_ISL_16931402, EPI_ISL_16201312, EPI_ISL_16536270, EPI_ISL_16957026, EPI_ISL_16675021, EPI_ISL_16675029, EPI_ISL_16675040, EPI_ISL_16675042, EPI_ISL_16675044, EPI_ISL_16675045, EPI_ISL_16675046, EPI_ISL_16675048, EPI_ISL_16675051, EPI_ISL_16675052, EPI_ISL_16675054, EPI_ISL_16675056, EPI_ISL_16675058, EPI_ISL_16675061, EPI_ISL_16675066, EPI_ISL_16675070, EPI_ISL_16201278, EPI_ISL_16201285, EPI_ISL_16201296, EPI_ISL_16201314, EPI_ISL_16201319, EPI_ISL_16201326, EPI_ISL_16201331, EPI_ISL_16201334, EPI_ISL_16913912, EPI_ISL_16536267, EPI_ISL_16536288, EPI_ISL_16536294, EPI_ISL_16536298, EPI_ISL_16536307, EPI_ISL_16536310, EPI_ISL_16177586, EPI_ISL_16176426, EPI_ISL_16177390, EPI_ISL_16177585, EPI_ISL_16177885, EPI_ISL_16182290, EPI_ISL_16181062, EPI_ISL_16181097, EPI_ISL_16182291, EPI_ISL_16182292, EPI_ISL_16194844, EPI_ISL_16927816, EPI_ISL_16927912, EPI_ISL_16928225, EPI_ISL_16928351, EPI_ISL_16929433, EPI_ISL_16929568, EPI_ISL_16933210, EPI_ISL_16933982, EPI_ISL_16934074, EPI_ISL_16934187, EPI_ISL_16934367, EPI_ISL_16930495, EPI_ISL_16396196, EPI_ISL_16041963, EPI_ISL_16193316, EPI_ISL_16510293, EPI_ISL_16117273, EPI_ISL_16117275, EPI_ISL_16504003, EPI_ISL_16019601, EPI_ISL_16766839, EPI_ISL_16564702, EPI_ISL_16638878, EPI_ISL_16564608, EPI_ISL_16548907, EPI_ISL_16760558, EPI_ISL_16038755, EPI_ISL_16139659, EPI_ISL_16371647, EPI_ISL_16370921, EPI_ISL_16396871, EPI_ISL_16896486, EPI_ISL_16896496, EPI_ISL_16170925, EPI_ISL_16170920, EPI_ISL_16386231, EPI_ISL_16118427, EPI_ISL_16535377, EPI_ISL_16260047, EPI_ISL_16401734, EPI_ISL_16815607, EPI_ISL_16269297, EPI_ISL_16815605, EPI_ISL_16815315, EPI_ISL_16977445, EPI_ISL_672597, EPI_ISL_12293485, EPI_ISL_12293533, EPI_ISL_12293547, EPI_ISL_12293573, EPI_ISL_12293576, EPI_ISL_12293585, EPI_ISL_12293623, EPI_ISL_12623202, EPI_ISL_12623213, EPI_ISL_12623228, EPI_ISL_12623250, EPI_ISL_12623253, EPI_ISL_12623256, EPI_ISL_12623264, EPI_ISL_12623269, EPI_ISL_12784708, EPI_ISL_12784712, EPI_ISL_12784743, EPI_ISL_12784755, EPI_ISL_12784767, EPI_ISL_12955745, EPI_ISL_12955833, EPI_ISL_13135506, EPI_ISL_13612385, EPI_ISL_13774112-13774113, EPI_ISL_13774116-13774117, EPI_ISL_13774120, EPI_ISL_13774122, EPI_ISL_13774124-13774126, EPI_ISL_13774133-13774134, EPI_ISL_13774137, EPI_ISL_13774142-13774143, EPI_ISL_14807247, EPI_ISL_17372592")
EPI_8mil_9mil = stringlist_to_strings_set("EPI_ISL_8001937, EPI_ISL_8014799, EPI_ISL_8017918, EPI_ISL_8029713, EPI_ISL_8030450, EPI_ISL_8030625, EPI_ISL_8038409, EPI_ISL_8044393, EPI_ISL_8046586, EPI_ISL_8049906, EPI_ISL_8049915, EPI_ISL_8049917, EPI_ISL_8049918, EPI_ISL_8049919, EPI_ISL_8060345, EPI_ISL_8060406, EPI_ISL_8060833, EPI_ISL_8060894, EPI_ISL_8060974, EPI_ISL_8062145, EPI_ISL_8063083, EPI_ISL_8068107, EPI_ISL_8078285, EPI_ISL_8079424, EPI_ISL_8079488, EPI_ISL_8079555, EPI_ISL_8080096, EPI_ISL_8080419, EPI_ISL_8082607, EPI_ISL_8082623, EPI_ISL_8082825, EPI_ISL_8087834, EPI_ISL_8091177, EPI_ISL_8092036, EPI_ISL_8097113, EPI_ISL_8097119, EPI_ISL_8106398, EPI_ISL_8109581, EPI_ISL_8111073, EPI_ISL_8112683, EPI_ISL_8115748, EPI_ISL_8116089, EPI_ISL_8116090, EPI_ISL_8122388, EPI_ISL_8126914, EPI_ISL_8132420, EPI_ISL_8133923, EPI_ISL_8149958, EPI_ISL_8157487, EPI_ISL_8159346, EPI_ISL_8163293, EPI_ISL_8163307, EPI_ISL_8163414, EPI_ISL_8166517, EPI_ISL_8170854, EPI_ISL_8180529, EPI_ISL_8182884, EPI_ISL_8183017, EPI_ISL_8183131, EPI_ISL_8183263, EPI_ISL_8185416, EPI_ISL_8186670, EPI_ISL_8188582, EPI_ISL_8188583, EPI_ISL_8188584, EPI_ISL_8188585, EPI_ISL_8188586, EPI_ISL_8188587, EPI_ISL_8188588, EPI_ISL_8188589, EPI_ISL_8189141, EPI_ISL_8193597, EPI_ISL_8200642, EPI_ISL_8206829, EPI_ISL_8206840, EPI_ISL_8206864, EPI_ISL_8207740, EPI_ISL_8229935, EPI_ISL_8229990, EPI_ISL_8233602, EPI_ISL_8236893, EPI_ISL_8238189, EPI_ISL_8238279, EPI_ISL_8241972, EPI_ISL_8242673, EPI_ISL_8243729, EPI_ISL_8248465, EPI_ISL_8248962, EPI_ISL_8255974, EPI_ISL_8257133, EPI_ISL_8257994, EPI_ISL_8258023, EPI_ISL_8259443, EPI_ISL_8260129, EPI_ISL_8260467, EPI_ISL_8260727, EPI_ISL_8260969, EPI_ISL_8261006, EPI_ISL_8261145, EPI_ISL_8261170, EPI_ISL_8261175, EPI_ISL_8261182, EPI_ISL_8261191, EPI_ISL_8261196, EPI_ISL_8261207, EPI_ISL_8261636, EPI_ISL_8262817, EPI_ISL_8263131, EPI_ISL_8265180, EPI_ISL_8267137, EPI_ISL_8276676, EPI_ISL_8280181, EPI_ISL_8281741, EPI_ISL_8282119, EPI_ISL_8282662, EPI_ISL_8287151, EPI_ISL_8287534, EPI_ISL_8289103, EPI_ISL_8289305, EPI_ISL_8289538, EPI_ISL_8289922, EPI_ISL_8290718, EPI_ISL_8290776, EPI_ISL_8293444, EPI_ISL_8294105, EPI_ISL_8295523, EPI_ISL_8303487, EPI_ISL_8304935, EPI_ISL_8305718, EPI_ISL_8306738, EPI_ISL_8307368, EPI_ISL_8307404, EPI_ISL_8307812, EPI_ISL_8308045, EPI_ISL_8308655, EPI_ISL_8308964, EPI_ISL_8309887, EPI_ISL_8309894, EPI_ISL_8310980, EPI_ISL_8311137, EPI_ISL_8311149, EPI_ISL_8311897, EPI_ISL_8314392, EPI_ISL_8315214, EPI_ISL_8317836, EPI_ISL_8321859, EPI_ISL_8328897, EPI_ISL_8328910, EPI_ISL_8329948, EPI_ISL_8336265, EPI_ISL_8336433, EPI_ISL_8336449, EPI_ISL_8336450, EPI_ISL_8337681, EPI_ISL_8337736, EPI_ISL_8338155, EPI_ISL_8351717, EPI_ISL_8352253, EPI_ISL_8353002, EPI_ISL_8353108, EPI_ISL_8354726, EPI_ISL_8355349, EPI_ISL_8355392, EPI_ISL_8356546, EPI_ISL_8356698, EPI_ISL_8357683, EPI_ISL_8358788, EPI_ISL_8364636, EPI_ISL_8365056, EPI_ISL_8365900, EPI_ISL_8373815, EPI_ISL_8373830, EPI_ISL_8373837, EPI_ISL_8374787, EPI_ISL_8374830, EPI_ISL_8374840, EPI_ISL_8374865, EPI_ISL_8374872, EPI_ISL_8374901, EPI_ISL_8377503, EPI_ISL_8381143, EPI_ISL_8381594, EPI_ISL_8381916, EPI_ISL_8381956, EPI_ISL_8384493, EPI_ISL_8392598, EPI_ISL_8393372, EPI_ISL_8393961, EPI_ISL_8394061, EPI_ISL_8396883, EPI_ISL_8402134, EPI_ISL_8402159, EPI_ISL_8404859, EPI_ISL_8404938, EPI_ISL_8404940, EPI_ISL_8404941, EPI_ISL_8405256, EPI_ISL_8405273, EPI_ISL_8405275, EPI_ISL_8405278, EPI_ISL_8405315, EPI_ISL_8405318, EPI_ISL_8405319, EPI_ISL_8405320, EPI_ISL_8405322, EPI_ISL_8405323, EPI_ISL_8405380, EPI_ISL_8405381, EPI_ISL_8405386, EPI_ISL_8410467, EPI_ISL_8410970, EPI_ISL_8411771, EPI_ISL_8416220, EPI_ISL_8416492, EPI_ISL_8418256, EPI_ISL_8420864, EPI_ISL_8424966, EPI_ISL_8425444, EPI_ISL_8425523, EPI_ISL_8425592, EPI_ISL_8425672, EPI_ISL_8425704, EPI_ISL_8425738, EPI_ISL_8426120, EPI_ISL_8426360, EPI_ISL_8427960, EPI_ISL_8428965, EPI_ISL_8433278, EPI_ISL_8433519, EPI_ISL_8434288, EPI_ISL_8434905, EPI_ISL_8436270, EPI_ISL_8449160, EPI_ISL_8456090, EPI_ISL_8457545, EPI_ISL_8458445, EPI_ISL_8462049, EPI_ISL_8465951, EPI_ISL_8467603, EPI_ISL_8468086, EPI_ISL_8468812, EPI_ISL_8470809, EPI_ISL_8471061, EPI_ISL_8471158, EPI_ISL_8471718, EPI_ISL_8471777, EPI_ISL_8472812, EPI_ISL_8472952, EPI_ISL_8475117, EPI_ISL_8480319, EPI_ISL_8482448, EPI_ISL_8484894, EPI_ISL_8494317, EPI_ISL_8494323, EPI_ISL_8494345, EPI_ISL_8495309, EPI_ISL_8508586, EPI_ISL_8512920, EPI_ISL_8513858, EPI_ISL_8513971, EPI_ISL_8514031, EPI_ISL_8517222, EPI_ISL_8517765, EPI_ISL_8517847, EPI_ISL_8521955, EPI_ISL_8524441, EPI_ISL_8524457, EPI_ISL_8525646, EPI_ISL_8525691, EPI_ISL_8527625, EPI_ISL_8527626, EPI_ISL_8529690, EPI_ISL_8530292, EPI_ISL_8530350, EPI_ISL_8530555, EPI_ISL_8530614, EPI_ISL_8538969, EPI_ISL_8541464, EPI_ISL_8541468, EPI_ISL_8541479, EPI_ISL_8546707, EPI_ISL_8546869, EPI_ISL_8546880, EPI_ISL_8546881, EPI_ISL_8546882, EPI_ISL_8546887, EPI_ISL_8546891, EPI_ISL_8546904, EPI_ISL_8546914, EPI_ISL_8546921, EPI_ISL_8546924, EPI_ISL_8546931, EPI_ISL_8546941, EPI_ISL_8546942, EPI_ISL_8546943, EPI_ISL_8546950, EPI_ISL_8546961, EPI_ISL_8546971, EPI_ISL_8546974, EPI_ISL_8546975, EPI_ISL_8546976, EPI_ISL_8546980, EPI_ISL_8546991, EPI_ISL_8546994, EPI_ISL_8547002, EPI_ISL_8547004, EPI_ISL_8551701, EPI_ISL_8551761, EPI_ISL_8552453, EPI_ISL_8552532, EPI_ISL_8552581, EPI_ISL_8552772, EPI_ISL_8554484, EPI_ISL_8554672, EPI_ISL_8554757, EPI_ISL_8555445, EPI_ISL_8555469, EPI_ISL_8556379, EPI_ISL_8557231, EPI_ISL_8573331, EPI_ISL_8578788, EPI_ISL_8581082, EPI_ISL_8581168, EPI_ISL_8581612, EPI_ISL_8581634, EPI_ISL_8584284, EPI_ISL_8584288, EPI_ISL_8584342, EPI_ISL_8584781, EPI_ISL_8584953, EPI_ISL_8585021, EPI_ISL_8588323, EPI_ISL_8588361, EPI_ISL_8588364, EPI_ISL_8588380, EPI_ISL_8588388, EPI_ISL_8588391, EPI_ISL_8588427, EPI_ISL_8588430, EPI_ISL_8588432, EPI_ISL_8588435, EPI_ISL_8591364, EPI_ISL_8592531, EPI_ISL_8593220, EPI_ISL_8603155, EPI_ISL_8605978, EPI_ISL_8606026, EPI_ISL_8611160, EPI_ISL_8623916, EPI_ISL_8624527, EPI_ISL_8625737, EPI_ISL_8632712, EPI_ISL_8637660, EPI_ISL_8637762, EPI_ISL_8647028, EPI_ISL_8647980, EPI_ISL_8648923, EPI_ISL_8650390, EPI_ISL_8652362, EPI_ISL_8652364, EPI_ISL_8652383, EPI_ISL_8652384, EPI_ISL_8654313, EPI_ISL_8658036, EPI_ISL_8662478, EPI_ISL_8665279, EPI_ISL_8665287, EPI_ISL_8668093, EPI_ISL_8674922, EPI_ISL_8674923, EPI_ISL_8674925, EPI_ISL_8674927, EPI_ISL_8674928, EPI_ISL_8674929, EPI_ISL_8674930, EPI_ISL_8674933, EPI_ISL_8674934, EPI_ISL_8674936, EPI_ISL_8674938, EPI_ISL_8676067, EPI_ISL_8676068, EPI_ISL_8676069, EPI_ISL_8676070, EPI_ISL_8676071, EPI_ISL_8676072, EPI_ISL_8676073, EPI_ISL_8676074, EPI_ISL_8676075, EPI_ISL_8676076, EPI_ISL_8676078, EPI_ISL_8676079, EPI_ISL_8676080, EPI_ISL_8676081, EPI_ISL_8676082, EPI_ISL_8676083, EPI_ISL_8676084, EPI_ISL_8676085, EPI_ISL_8676086, EPI_ISL_8676087, EPI_ISL_8676088, EPI_ISL_8676089, EPI_ISL_8676091, EPI_ISL_8676092, EPI_ISL_8676093, EPI_ISL_8676094, EPI_ISL_8676095, EPI_ISL_8676096, EPI_ISL_8678185, EPI_ISL_8678971, EPI_ISL_8679259, EPI_ISL_8679268, EPI_ISL_8679715, EPI_ISL_8679849, EPI_ISL_8681601, EPI_ISL_8683706, EPI_ISL_8683806, EPI_ISL_8684846, EPI_ISL_8685784, EPI_ISL_8686045, EPI_ISL_8686128, EPI_ISL_8686207, EPI_ISL_8686269, EPI_ISL_8686300, EPI_ISL_8686328, EPI_ISL_8686493, EPI_ISL_8686597, EPI_ISL_8686682, EPI_ISL_8686690, EPI_ISL_8686754, EPI_ISL_8686761, EPI_ISL_8686789, EPI_ISL_8686819, EPI_ISL_8686839, EPI_ISL_8686847, EPI_ISL_8687000, EPI_ISL_8687449, EPI_ISL_8687931, EPI_ISL_8687964, EPI_ISL_8688006, EPI_ISL_8688043, EPI_ISL_8688224, EPI_ISL_8688225, EPI_ISL_8688245, EPI_ISL_8700224, EPI_ISL_8703745, EPI_ISL_8703751, EPI_ISL_8703820, EPI_ISL_8704528, EPI_ISL_8705856, EPI_ISL_8707751, EPI_ISL_8707755, EPI_ISL_8707757, EPI_ISL_8707765, EPI_ISL_8713502, EPI_ISL_8717411, EPI_ISL_8717997, EPI_ISL_8718031, EPI_ISL_8718035, EPI_ISL_8718060, EPI_ISL_8718446, EPI_ISL_8719196, EPI_ISL_8719218, EPI_ISL_8719280, EPI_ISL_8719337, EPI_ISL_8719483, EPI_ISL_8719489, EPI_ISL_8719543, EPI_ISL_8719614, EPI_ISL_8720079, EPI_ISL_8720127, EPI_ISL_8733411, EPI_ISL_8733650, EPI_ISL_8739680, EPI_ISL_8739918, EPI_ISL_8742288, EPI_ISL_8742542, EPI_ISL_8744107, EPI_ISL_8746666, EPI_ISL_8746676, EPI_ISL_8746677, EPI_ISL_8746824, EPI_ISL_8748244, EPI_ISL_8754017, EPI_ISL_8766584, EPI_ISL_8766632, EPI_ISL_8766766, EPI_ISL_8767028, EPI_ISL_8767346, EPI_ISL_8768280, EPI_ISL_8769246, EPI_ISL_8769320, EPI_ISL_8769322, EPI_ISL_8769379, EPI_ISL_8769462, EPI_ISL_8769569, EPI_ISL_8769627, EPI_ISL_8769628, EPI_ISL_8769648, EPI_ISL_8769680, EPI_ISL_8769788, EPI_ISL_8769790, EPI_ISL_8769824, EPI_ISL_8769847, EPI_ISL_8769857, EPI_ISL_8769918, EPI_ISL_8770192, EPI_ISL_8781930, EPI_ISL_8781931, EPI_ISL_8784470, EPI_ISL_8784572, EPI_ISL_8790835, EPI_ISL_8790931, EPI_ISL_8792919, EPI_ISL_8799640, EPI_ISL_8800910, EPI_ISL_8800997, EPI_ISL_8801344, EPI_ISL_8801345, EPI_ISL_8802474, EPI_ISL_8802508, EPI_ISL_8804682, EPI_ISL_8809124, EPI_ISL_8809149, EPI_ISL_8809538, EPI_ISL_8809542, EPI_ISL_8809554, EPI_ISL_8809566, EPI_ISL_8809593, EPI_ISL_8809600, EPI_ISL_8809623, EPI_ISL_8809629, EPI_ISL_8809822, EPI_ISL_8809828, EPI_ISL_8809841, EPI_ISL_8809843, EPI_ISL_8809845, EPI_ISL_8809848, EPI_ISL_8809852, EPI_ISL_8809853, EPI_ISL_8809855, EPI_ISL_8809857, EPI_ISL_8809858, EPI_ISL_8809859, EPI_ISL_8809861, EPI_ISL_8809862, EPI_ISL_8809864, EPI_ISL_8809897, EPI_ISL_8809906, EPI_ISL_8809914, EPI_ISL_8809918, EPI_ISL_8809920, EPI_ISL_8809921, EPI_ISL_8809922, EPI_ISL_8809925, EPI_ISL_8809929, EPI_ISL_8809933, EPI_ISL_8809939, EPI_ISL_8809941, EPI_ISL_8809973, EPI_ISL_8809974, EPI_ISL_8809978, EPI_ISL_8809989, EPI_ISL_8809995, EPI_ISL_8814315, EPI_ISL_8814384, EPI_ISL_8814429, EPI_ISL_8814445, EPI_ISL_8814732, EPI_ISL_8815416, EPI_ISL_8815595, EPI_ISL_8815638, EPI_ISL_8815659, EPI_ISL_8818869, EPI_ISL_8819629, EPI_ISL_8825863, EPI_ISL_8826031, EPI_ISL_8826840, EPI_ISL_8827045, EPI_ISL_8827050, EPI_ISL_8827103, EPI_ISL_8827128, EPI_ISL_8827454, EPI_ISL_8827488, EPI_ISL_8827489, EPI_ISL_8827490, EPI_ISL_8827491, EPI_ISL_8827728, EPI_ISL_8827959, EPI_ISL_8827960, EPI_ISL_8828192, EPI_ISL_8828260, EPI_ISL_8829541, EPI_ISL_8829676, EPI_ISL_8829684, EPI_ISL_8830413, EPI_ISL_8830421, EPI_ISL_8830422, EPI_ISL_8830425, EPI_ISL_8830429, EPI_ISL_8830757, EPI_ISL_8830989, EPI_ISL_8831792, EPI_ISL_8831848, EPI_ISL_8831882, EPI_ISL_8834694, EPI_ISL_8835906, EPI_ISL_8835936, EPI_ISL_8837359, EPI_ISL_8837952, EPI_ISL_8841708, EPI_ISL_8843285, EPI_ISL_8851373, EPI_ISL_8867091, EPI_ISL_8867332, EPI_ISL_8869019, EPI_ISL_8869364, EPI_ISL_8870629, EPI_ISL_8870728, EPI_ISL_8871465, EPI_ISL_8871471, EPI_ISL_8872057, EPI_ISL_8872528, EPI_ISL_8876645, EPI_ISL_8876658, EPI_ISL_8877091, EPI_ISL_8878410, EPI_ISL_8881555, EPI_ISL_8881557, EPI_ISL_8881656, EPI_ISL_8881657, EPI_ISL_8881665, EPI_ISL_8881732, EPI_ISL_8881761, EPI_ISL_8881764, EPI_ISL_8881774, EPI_ISL_8881783, EPI_ISL_8881789, EPI_ISL_8881790, EPI_ISL_8881792, EPI_ISL_8881795, EPI_ISL_8881796, EPI_ISL_8881802, EPI_ISL_8881803, EPI_ISL_8881825, EPI_ISL_8881826, EPI_ISL_8881828, EPI_ISL_8881832, EPI_ISL_8881834, EPI_ISL_8881836, EPI_ISL_8881845, EPI_ISL_8881846, EPI_ISL_8881850, EPI_ISL_8881852, EPI_ISL_8881854, EPI_ISL_8885399, EPI_ISL_8887525, EPI_ISL_8887824, EPI_ISL_8892298, EPI_ISL_8897108, EPI_ISL_8898058, EPI_ISL_8901948, EPI_ISL_8901957, EPI_ISL_8911076, EPI_ISL_8917553, EPI_ISL_8917567, EPI_ISL_8917574, EPI_ISL_8917631, EPI_ISL_8917683, EPI_ISL_8917706, EPI_ISL_8917711, EPI_ISL_8917729, EPI_ISL_8917754, EPI_ISL_8917822, EPI_ISL_8917831, EPI_ISL_8921555, EPI_ISL_8921859, EPI_ISL_8921872, EPI_ISL_8921892, EPI_ISL_8921935, EPI_ISL_8922141, EPI_ISL_8934076, EPI_ISL_8940932, EPI_ISL_8941919, EPI_ISL_8942426, EPI_ISL_8942952, EPI_ISL_8943696, EPI_ISL_8944656, EPI_ISL_8945151, EPI_ISL_8945557, EPI_ISL_8946014, EPI_ISL_8947563, EPI_ISL_8949486, EPI_ISL_8950554, EPI_ISL_8950655, EPI_ISL_8950704, EPI_ISL_8950735, EPI_ISL_8951104, EPI_ISL_8951359, EPI_ISL_8957606, EPI_ISL_8960442, EPI_ISL_8962449, EPI_ISL_8974204, EPI_ISL_8975553, EPI_ISL_8976190, EPI_ISL_8978128, EPI_ISL_8978129, EPI_ISL_8984121, EPI_ISL_8990258, EPI_ISL_8991619")
EPI_9mil_10mil = stringlist_to_strings_set("EPI_ISL_9005352, EPI_ISL_9005356, EPI_ISL_9009178, EPI_ISL_9010456, EPI_ISL_9011268, EPI_ISL_9011275, EPI_ISL_9011283, EPI_ISL_9011289, EPI_ISL_9011290, EPI_ISL_9011298, EPI_ISL_9011310, EPI_ISL_9012199, EPI_ISL_9012200, EPI_ISL_9012288, EPI_ISL_9012305, EPI_ISL_9013737, EPI_ISL_9025020, EPI_ISL_9028109, EPI_ISL_9029649, EPI_ISL_9029749, EPI_ISL_9030071, EPI_ISL_9031908, EPI_ISL_9034666, EPI_ISL_9034671, EPI_ISL_9034712, EPI_ISL_9034715, EPI_ISL_9035097, EPI_ISL_9036190, EPI_ISL_9036685, EPI_ISL_9036935, EPI_ISL_9037616, EPI_ISL_9037649, EPI_ISL_9038072, EPI_ISL_9040293, EPI_ISL_9041163, EPI_ISL_9044369, EPI_ISL_9044458, EPI_ISL_9044535, EPI_ISL_9044608, EPI_ISL_9045074, EPI_ISL_9045075, EPI_ISL_9045087, EPI_ISL_9045225, EPI_ISL_9046021, EPI_ISL_9046155, EPI_ISL_9046511, EPI_ISL_9046768, EPI_ISL_9047380, EPI_ISL_9047385, EPI_ISL_9047396, EPI_ISL_9047413, EPI_ISL_9047424, EPI_ISL_9047438, EPI_ISL_9047479, EPI_ISL_9047560, EPI_ISL_9047929, EPI_ISL_9048577, EPI_ISL_9048645, EPI_ISL_9048813, EPI_ISL_9048832, EPI_ISL_9049061, EPI_ISL_9049124, EPI_ISL_9051207, EPI_ISL_9053050, EPI_ISL_9053093, EPI_ISL_9058521, EPI_ISL_9061430, EPI_ISL_9063947, EPI_ISL_9064017, EPI_ISL_9064261, EPI_ISL_9065596, EPI_ISL_9066315, EPI_ISL_9068610, EPI_ISL_9074323, EPI_ISL_9074392, EPI_ISL_9074669, EPI_ISL_9075663, EPI_ISL_9076234, EPI_ISL_9078947, EPI_ISL_9080382, EPI_ISL_9086997, EPI_ISL_9087004, EPI_ISL_9087015, EPI_ISL_9087019, EPI_ISL_9090751, EPI_ISL_9090837, EPI_ISL_9090929, EPI_ISL_9090961, EPI_ISL_9090962, EPI_ISL_9090967, EPI_ISL_9090980, EPI_ISL_9090984, EPI_ISL_9090991, EPI_ISL_9090992, EPI_ISL_9090993, EPI_ISL_9090995, EPI_ISL_9091000, EPI_ISL_9091036, EPI_ISL_9091066, EPI_ISL_9091067, EPI_ISL_9091077, EPI_ISL_9091088, EPI_ISL_9091091, EPI_ISL_9091097, EPI_ISL_9091117, EPI_ISL_9091195, EPI_ISL_9091242, EPI_ISL_9092486, EPI_ISL_9092543, EPI_ISL_9093024, EPI_ISL_9093324, EPI_ISL_9100643, EPI_ISL_9100660, EPI_ISL_9100671, EPI_ISL_9100690, EPI_ISL_9103139, EPI_ISL_9104320, EPI_ISL_9108186, EPI_ISL_9109762, EPI_ISL_9109898, EPI_ISL_9110303, EPI_ISL_9120288, EPI_ISL_9121666, EPI_ISL_9121730, EPI_ISL_9126398, EPI_ISL_9129541, EPI_ISL_9131755, EPI_ISL_9133265, EPI_ISL_9133362, EPI_ISL_9133533, EPI_ISL_9133534, EPI_ISL_9133560, EPI_ISL_9133651, EPI_ISL_9133676, EPI_ISL_9133770, EPI_ISL_9133811, EPI_ISL_9133912, EPI_ISL_9134047, EPI_ISL_9134101, EPI_ISL_9134165, EPI_ISL_9134203, EPI_ISL_9134204, EPI_ISL_9134714, EPI_ISL_9135550, EPI_ISL_9135558, EPI_ISL_9135571, EPI_ISL_9139305, EPI_ISL_9139589, EPI_ISL_9139682, EPI_ISL_9139789, EPI_ISL_9139793, EPI_ISL_9139806, EPI_ISL_9139841, EPI_ISL_9139906, EPI_ISL_9139953, EPI_ISL_9146287, EPI_ISL_9147675, EPI_ISL_9151196, EPI_ISL_9151405, EPI_ISL_9151417, EPI_ISL_9151754, EPI_ISL_9152643, EPI_ISL_9156804, EPI_ISL_9157476, EPI_ISL_9163218, EPI_ISL_9166254, EPI_ISL_9168068, EPI_ISL_9169546, EPI_ISL_9169574, EPI_ISL_9169859, EPI_ISL_9170032, EPI_ISL_9172208, EPI_ISL_9176038, EPI_ISL_9180127, EPI_ISL_9182632, EPI_ISL_9182645, EPI_ISL_9182662, EPI_ISL_9187834, EPI_ISL_9187843, EPI_ISL_9187872, EPI_ISL_9187891, EPI_ISL_9191078, EPI_ISL_9191147, EPI_ISL_9191309, EPI_ISL_9196222, EPI_ISL_9196330, EPI_ISL_9196339, EPI_ISL_9196351, EPI_ISL_9196358, EPI_ISL_9196364, EPI_ISL_9196485, EPI_ISL_9196486, EPI_ISL_9196487, EPI_ISL_9196489, EPI_ISL_9196490, EPI_ISL_9196491, EPI_ISL_9196493, EPI_ISL_9196494, EPI_ISL_9196495, EPI_ISL_9196496, EPI_ISL_9196497, EPI_ISL_9196498, EPI_ISL_9196499, EPI_ISL_9196500, EPI_ISL_9196502, EPI_ISL_9196508, EPI_ISL_9197135, EPI_ISL_9197952, EPI_ISL_9198020, EPI_ISL_9198230, EPI_ISL_9198767, EPI_ISL_9199183, EPI_ISL_9200048, EPI_ISL_9200866, EPI_ISL_9202367, EPI_ISL_9202404, EPI_ISL_9202411, EPI_ISL_9202498, EPI_ISL_9202934, EPI_ISL_9202999, EPI_ISL_9204848, EPI_ISL_9205605, EPI_ISL_9206643, EPI_ISL_9210610, EPI_ISL_9210834, EPI_ISL_9210948, EPI_ISL_9211469, EPI_ISL_9214019, EPI_ISL_9214153, EPI_ISL_9217742, EPI_ISL_9218276, EPI_ISL_9227643, EPI_ISL_9227757, EPI_ISL_9229700, EPI_ISL_9229876, EPI_ISL_9230962, EPI_ISL_9232065, EPI_ISL_9232175, EPI_ISL_9232315, EPI_ISL_9232396, EPI_ISL_9232404, EPI_ISL_9232441, EPI_ISL_9232442, EPI_ISL_9233990, EPI_ISL_9237445, EPI_ISL_9244079, EPI_ISL_9244229, EPI_ISL_9244241, EPI_ISL_9244346, EPI_ISL_9244347, EPI_ISL_9244349, EPI_ISL_9244352, EPI_ISL_9244354, EPI_ISL_9244356, EPI_ISL_9244361, EPI_ISL_9244362, EPI_ISL_9244364, EPI_ISL_9244365, EPI_ISL_9244366, EPI_ISL_9244367, EPI_ISL_9244368, EPI_ISL_9244371, EPI_ISL_9244393, EPI_ISL_9244407, EPI_ISL_9244414, EPI_ISL_9244430, EPI_ISL_9244436, EPI_ISL_9244438, EPI_ISL_9244443, EPI_ISL_9257372, EPI_ISL_9263448, EPI_ISL_9268007, EPI_ISL_9272940, EPI_ISL_9273139, EPI_ISL_9275801, EPI_ISL_9287159, EPI_ISL_9301892, EPI_ISL_9303894, EPI_ISL_9320000, EPI_ISL_9320048, EPI_ISL_9320701, EPI_ISL_9322799, EPI_ISL_9323194, EPI_ISL_9324184, EPI_ISL_9324261, EPI_ISL_9324445, EPI_ISL_9325956, EPI_ISL_9329574, EPI_ISL_9329597, EPI_ISL_9329613, EPI_ISL_9329709, EPI_ISL_9329949, EPI_ISL_9332338, EPI_ISL_9340194, EPI_ISL_9346264, EPI_ISL_9357293, EPI_ISL_9361245, EPI_ISL_9366656, EPI_ISL_9366674, EPI_ISL_9366695, EPI_ISL_9366701, EPI_ISL_9366748, EPI_ISL_9369408, EPI_ISL_9369483, EPI_ISL_9369654, EPI_ISL_9369696, EPI_ISL_9369748, EPI_ISL_9369840, EPI_ISL_9374568, EPI_ISL_9375639, EPI_ISL_9375644, EPI_ISL_9393238, EPI_ISL_9393371, EPI_ISL_9393552, EPI_ISL_9393561, EPI_ISL_9393590, EPI_ISL_9393601, EPI_ISL_9393617, EPI_ISL_9393658, EPI_ISL_9397486, EPI_ISL_9397545, EPI_ISL_9397580, EPI_ISL_9397591, EPI_ISL_9402130, EPI_ISL_9402568, EPI_ISL_9402729, EPI_ISL_9402868, EPI_ISL_9405606, EPI_ISL_9406530, EPI_ISL_9406661, EPI_ISL_9406745, EPI_ISL_9407327, EPI_ISL_9408663, EPI_ISL_9408966, EPI_ISL_9408994, EPI_ISL_9409022, EPI_ISL_9409061, EPI_ISL_9409074, EPI_ISL_9409098, EPI_ISL_9409875, EPI_ISL_9409936, EPI_ISL_9410965, EPI_ISL_9412364, EPI_ISL_9416349, EPI_ISL_9416383, EPI_ISL_9416496, EPI_ISL_9416811, EPI_ISL_9417829, EPI_ISL_9417856, EPI_ISL_9418351, EPI_ISL_9429907, EPI_ISL_9429910, EPI_ISL_9429914, EPI_ISL_9429916, EPI_ISL_9429917, EPI_ISL_9429928, EPI_ISL_9429953, EPI_ISL_9430010, EPI_ISL_9430074, EPI_ISL_9430075, EPI_ISL_9430083, EPI_ISL_9430088, EPI_ISL_9430093, EPI_ISL_9430095, EPI_ISL_9430097, EPI_ISL_9430100, EPI_ISL_9430104, EPI_ISL_9430105, EPI_ISL_9430142, EPI_ISL_9430149, EPI_ISL_9430156, EPI_ISL_9430807, EPI_ISL_9433224, EPI_ISL_9436219, EPI_ISL_9436655, EPI_ISL_9436667, EPI_ISL_9436800, EPI_ISL_9436819, EPI_ISL_9439014, EPI_ISL_9443344, EPI_ISL_9445585, EPI_ISL_9445587, EPI_ISL_9448294, EPI_ISL_9448303, EPI_ISL_9450747, EPI_ISL_9451725, EPI_ISL_9454967, EPI_ISL_9455817, EPI_ISL_9455959, EPI_ISL_9456039, EPI_ISL_9456611, EPI_ISL_9457664, EPI_ISL_9457860, EPI_ISL_9458247, EPI_ISL_9459219, EPI_ISL_9460511, EPI_ISL_9465685, EPI_ISL_9466125, EPI_ISL_9466330, EPI_ISL_9466376, EPI_ISL_9466488, EPI_ISL_9466563, EPI_ISL_9466712, EPI_ISL_9466730, EPI_ISL_9466742, EPI_ISL_9466762, EPI_ISL_9466822, EPI_ISL_9466824, EPI_ISL_9466827, EPI_ISL_9467584, EPI_ISL_9468165, EPI_ISL_9471815, EPI_ISL_9471816, EPI_ISL_9471828, EPI_ISL_9471831, EPI_ISL_9471852, EPI_ISL_9471897, EPI_ISL_9471996, EPI_ISL_9471999, EPI_ISL_9472306, EPI_ISL_9472308, EPI_ISL_9472322, EPI_ISL_9472338, EPI_ISL_9472385, EPI_ISL_9472416, EPI_ISL_9472426, EPI_ISL_9472506, EPI_ISL_9472507, EPI_ISL_9472531, EPI_ISL_9472535, EPI_ISL_9475129, EPI_ISL_9487490, EPI_ISL_9487619, EPI_ISL_9487621, EPI_ISL_9487630, EPI_ISL_9487635, EPI_ISL_9487640, EPI_ISL_9487643, EPI_ISL_9487651, EPI_ISL_9487652, EPI_ISL_9487655, EPI_ISL_9487657, EPI_ISL_9487660, EPI_ISL_9487662, EPI_ISL_9487676, EPI_ISL_9487692, EPI_ISL_9487693, EPI_ISL_9487694, EPI_ISL_9487703, EPI_ISL_9487706, EPI_ISL_9487716, EPI_ISL_9487722, EPI_ISL_9487739, EPI_ISL_9487741, EPI_ISL_9487742, EPI_ISL_9487743, EPI_ISL_9487750, EPI_ISL_9487753, EPI_ISL_9487768, EPI_ISL_9487787, EPI_ISL_9487793, EPI_ISL_9487803, EPI_ISL_9487831, EPI_ISL_9487857, EPI_ISL_9487898, EPI_ISL_9487928, EPI_ISL_9487970, EPI_ISL_9487982, EPI_ISL_9487983, EPI_ISL_9488006, EPI_ISL_9488029, EPI_ISL_9488474, EPI_ISL_9488477, EPI_ISL_9488501, EPI_ISL_9488515, EPI_ISL_9488519, EPI_ISL_9488524, EPI_ISL_9488545, EPI_ISL_9488570, EPI_ISL_9488575, EPI_ISL_9488581, EPI_ISL_9488619, EPI_ISL_9488636, EPI_ISL_9488661, EPI_ISL_9488677, EPI_ISL_9488757, EPI_ISL_9488782, EPI_ISL_9488784, EPI_ISL_9488805, EPI_ISL_9488811, EPI_ISL_9488829, EPI_ISL_9488834, EPI_ISL_9489247, EPI_ISL_9489252, EPI_ISL_9497113, EPI_ISL_9504005, EPI_ISL_9504259, EPI_ISL_9510032, EPI_ISL_9510046, EPI_ISL_9511693, EPI_ISL_9511841, EPI_ISL_9511897, EPI_ISL_9511947, EPI_ISL_9511949, EPI_ISL_9511951, EPI_ISL_9512035, EPI_ISL_9512183, EPI_ISL_9512188, EPI_ISL_9512193, EPI_ISL_9512195, EPI_ISL_9512200, EPI_ISL_9512213, EPI_ISL_9512376, EPI_ISL_9519799, EPI_ISL_9523248, EPI_ISL_9532527, EPI_ISL_9533075, EPI_ISL_9533563, EPI_ISL_9533609, EPI_ISL_9549192, EPI_ISL_9549456, EPI_ISL_9561760, EPI_ISL_9569515, EPI_ISL_9569535, EPI_ISL_9572644, EPI_ISL_9573034, EPI_ISL_9573172, EPI_ISL_9587585, EPI_ISL_9587875, EPI_ISL_9589057, EPI_ISL_9594728, EPI_ISL_9594734, EPI_ISL_9594736, EPI_ISL_9594746, EPI_ISL_9594748, EPI_ISL_9594790, EPI_ISL_9594793, EPI_ISL_9594813, EPI_ISL_9594816, EPI_ISL_9594822, EPI_ISL_9594823, EPI_ISL_9594825, EPI_ISL_9596778, EPI_ISL_9596984, EPI_ISL_9601628, EPI_ISL_9601700, EPI_ISL_9601744, EPI_ISL_9604897, EPI_ISL_9609009, EPI_ISL_9609069, EPI_ISL_9610381, EPI_ISL_9610390, EPI_ISL_9610393, EPI_ISL_9610402, EPI_ISL_9613674, EPI_ISL_9614658, EPI_ISL_9617758, EPI_ISL_9620216, EPI_ISL_9623090, EPI_ISL_9625693, EPI_ISL_9625727, EPI_ISL_9625751, EPI_ISL_9625759, EPI_ISL_9625770, EPI_ISL_9625891, EPI_ISL_9636300, EPI_ISL_9637481, EPI_ISL_9640020, EPI_ISL_9640740, EPI_ISL_9642609, EPI_ISL_9647293, EPI_ISL_9650799, EPI_ISL_9651096, EPI_ISL_9651353, EPI_ISL_9653335, EPI_ISL_9656370, EPI_ISL_9666874, EPI_ISL_9668614, EPI_ISL_9673566, EPI_ISL_9674935, EPI_ISL_9678134, EPI_ISL_9678135, EPI_ISL_9679129, EPI_ISL_9679144, EPI_ISL_9679158, EPI_ISL_9679269, EPI_ISL_9679403, EPI_ISL_9683434, EPI_ISL_9683447, EPI_ISL_9683466, EPI_ISL_9683467, EPI_ISL_9683478, EPI_ISL_9683483, EPI_ISL_9683486, EPI_ISL_9683487, EPI_ISL_9689797, EPI_ISL_9691942, EPI_ISL_9695242, EPI_ISL_9695386, EPI_ISL_9696667, EPI_ISL_9696682, EPI_ISL_9696732, EPI_ISL_9696744, EPI_ISL_9696756, EPI_ISL_9696794, EPI_ISL_9696809, EPI_ISL_9697283, EPI_ISL_9697342, EPI_ISL_9697401, EPI_ISL_9697409, EPI_ISL_9697412, EPI_ISL_9697419, EPI_ISL_9697420, EPI_ISL_9697622, EPI_ISL_9697669, EPI_ISL_9697674, EPI_ISL_9697678, EPI_ISL_9697695, EPI_ISL_9697740, EPI_ISL_9697742, EPI_ISL_9697750, EPI_ISL_9697760, EPI_ISL_9697779, EPI_ISL_9697780, EPI_ISL_9697786, EPI_ISL_9697797, EPI_ISL_9697801, EPI_ISL_9697802, EPI_ISL_9697803, EPI_ISL_9697810, EPI_ISL_9697813, EPI_ISL_9697815, EPI_ISL_9697816, EPI_ISL_9702292, EPI_ISL_9705038, EPI_ISL_9705152, EPI_ISL_9707770, EPI_ISL_9709081, EPI_ISL_9709115, EPI_ISL_9709126, EPI_ISL_9709128, EPI_ISL_9709139, EPI_ISL_9713712, EPI_ISL_9714143, EPI_ISL_9714174, EPI_ISL_9715503, EPI_ISL_9715636, EPI_ISL_9715645, EPI_ISL_9721311, EPI_ISL_9721351, EPI_ISL_9721359, EPI_ISL_9721365, EPI_ISL_9721370, EPI_ISL_9721377, EPI_ISL_9721381, EPI_ISL_9721390, EPI_ISL_9727117, EPI_ISL_9727142, EPI_ISL_9727150, EPI_ISL_9727169, EPI_ISL_9727186, EPI_ISL_9727194, EPI_ISL_9727223, EPI_ISL_9727268, EPI_ISL_9728278, EPI_ISL_9729033, EPI_ISL_9729063, EPI_ISL_9729127, EPI_ISL_9729128, EPI_ISL_9736232, EPI_ISL_9743775, EPI_ISL_9746931, EPI_ISL_9754259, EPI_ISL_9754610, EPI_ISL_9770596, EPI_ISL_9770856, EPI_ISL_9770870, EPI_ISL_9771473, EPI_ISL_9773064, EPI_ISL_9775562, EPI_ISL_9781644, EPI_ISL_9781649, EPI_ISL_9781675, EPI_ISL_9781690, EPI_ISL_9783712, EPI_ISL_9788022, EPI_ISL_9791141, EPI_ISL_9792573, EPI_ISL_9806761, EPI_ISL_9806777, EPI_ISL_9806780, EPI_ISL_9806795, EPI_ISL_9806825, EPI_ISL_9806831, EPI_ISL_9806849, EPI_ISL_9806890, EPI_ISL_9806897, EPI_ISL_9806909, EPI_ISL_9806941, EPI_ISL_9806943, EPI_ISL_9806949, EPI_ISL_9806960, EPI_ISL_9806977, EPI_ISL_9806985, EPI_ISL_9806992, EPI_ISL_9806995, EPI_ISL_9808141, EPI_ISL_9808146, EPI_ISL_9808159, EPI_ISL_9808311, EPI_ISL_9808318, EPI_ISL_9813896, EPI_ISL_9814061, EPI_ISL_9817504, EPI_ISL_9818106, EPI_ISL_9818145, EPI_ISL_9831433, EPI_ISL_9831863, EPI_ISL_9832011, EPI_ISL_9832049, EPI_ISL_9832223, EPI_ISL_9832265, EPI_ISL_9832266, EPI_ISL_9833929, EPI_ISL_9835767, EPI_ISL_9835963, EPI_ISL_9837460, EPI_ISL_9837718, EPI_ISL_9837719, EPI_ISL_9843631, EPI_ISL_9844747, EPI_ISL_9844816, EPI_ISL_9844820, EPI_ISL_9849476, EPI_ISL_9849487, EPI_ISL_9849501, EPI_ISL_9854654, EPI_ISL_9854923, EPI_ISL_9863925, EPI_ISL_9873050, EPI_ISL_9873051, EPI_ISL_9873829, EPI_ISL_9873872, EPI_ISL_9873933, EPI_ISL_9873964, EPI_ISL_9877792, EPI_ISL_9880735, EPI_ISL_9882111, EPI_ISL_9884555, EPI_ISL_9886029, EPI_ISL_9886100, EPI_ISL_9911424, EPI_ISL_9911578, EPI_ISL_9919496, EPI_ISL_9925948, EPI_ISL_9925959, EPI_ISL_9925965, EPI_ISL_9925989, EPI_ISL_9925993, EPI_ISL_9925995, EPI_ISL_9925997, EPI_ISL_9925999, EPI_ISL_9926001, EPI_ISL_9926003, EPI_ISL_9926004, EPI_ISL_9926007, EPI_ISL_9926010, EPI_ISL_9926017, EPI_ISL_9926019, EPI_ISL_9926022, EPI_ISL_9926024, EPI_ISL_9926030, EPI_ISL_9926058, EPI_ISL_9926078, EPI_ISL_9926099, EPI_ISL_9926112, EPI_ISL_9926118, EPI_ISL_9926127, EPI_ISL_9926154, EPI_ISL_9926162, EPI_ISL_9926327, EPI_ISL_9926334, EPI_ISL_9940202, EPI_ISL_9961706, EPI_ISL_9961927, EPI_ISL_9963072, EPI_ISL_9966882, EPI_ISL_9970992, EPI_ISL_9979109, EPI_ISL_9983756, EPI_ISL_9983883, EPI_ISL_9987524, EPI_ISL_9994180")
EPI_10mil_11mil = stringlist_to_strings_set("EPI_ISL_10005959, EPI_ISL_10006826, EPI_ISL_10008204, EPI_ISL_10008459, EPI_ISL_10008460, EPI_ISL_10008465, EPI_ISL_10009329, EPI_ISL_10011197, EPI_ISL_10013260, EPI_ISL_10015927, EPI_ISL_10021110, EPI_ISL_10022743, EPI_ISL_10023162, EPI_ISL_10023287, EPI_ISL_10023339, EPI_ISL_10025493, EPI_ISL_10026752, EPI_ISL_10026977, EPI_ISL_10027022, EPI_ISL_10027035, EPI_ISL_10027100, EPI_ISL_10027276, EPI_ISL_10028336, EPI_ISL_10029029, EPI_ISL_10034133, EPI_ISL_10034481, EPI_ISL_10034593, EPI_ISL_10034637, EPI_ISL_10045301, EPI_ISL_10058780, EPI_ISL_10058783, EPI_ISL_10063882, EPI_ISL_10065608, EPI_ISL_10065768, EPI_ISL_10065974, EPI_ISL_10065986, EPI_ISL_10065987, EPI_ISL_10065994, EPI_ISL_10066000, EPI_ISL_10066001, EPI_ISL_10066002, EPI_ISL_10066010, EPI_ISL_10066015, EPI_ISL_10066022, EPI_ISL_10066023, EPI_ISL_10066029, EPI_ISL_10066034, EPI_ISL_10066036, EPI_ISL_10066048, EPI_ISL_10066050, EPI_ISL_10066056, EPI_ISL_10066057, EPI_ISL_10066058, EPI_ISL_10066059, EPI_ISL_10066061, EPI_ISL_10066062, EPI_ISL_10066068, EPI_ISL_10066070, EPI_ISL_10066073, EPI_ISL_10066084, EPI_ISL_10066090, EPI_ISL_10066091, EPI_ISL_10066093, EPI_ISL_10066102, EPI_ISL_10066130, EPI_ISL_10066134, EPI_ISL_10066137, EPI_ISL_10066150, EPI_ISL_10066153, EPI_ISL_10066164, EPI_ISL_10066168, EPI_ISL_10066186, EPI_ISL_10066213, EPI_ISL_10066243, EPI_ISL_10066246, EPI_ISL_10066254, EPI_ISL_10066267, EPI_ISL_10072285, EPI_ISL_10072674, EPI_ISL_10072698, EPI_ISL_10073148, EPI_ISL_10074087, EPI_ISL_10074702, EPI_ISL_10074707, EPI_ISL_10077456, EPI_ISL_10077457, EPI_ISL_10077740, EPI_ISL_10077789, EPI_ISL_10077805, EPI_ISL_10077939, EPI_ISL_10077958, EPI_ISL_10077960, EPI_ISL_10077963, EPI_ISL_10077965, EPI_ISL_10077966, EPI_ISL_10077980, EPI_ISL_10077987, EPI_ISL_10078149, EPI_ISL_10078155, EPI_ISL_10078205, EPI_ISL_10078216, EPI_ISL_10078218, EPI_ISL_10078365, EPI_ISL_10078399, EPI_ISL_10078400, EPI_ISL_10078405, EPI_ISL_10078406, EPI_ISL_10078749, EPI_ISL_10078881, EPI_ISL_10078882, EPI_ISL_10078977, EPI_ISL_10079008, EPI_ISL_10079124, EPI_ISL_10079482, EPI_ISL_10079485, EPI_ISL_10079730, EPI_ISL_10080088, EPI_ISL_10080659, EPI_ISL_10081351, EPI_ISL_10081394, EPI_ISL_10081402, EPI_ISL_10081662, EPI_ISL_10081729, EPI_ISL_10081881, EPI_ISL_10081886, EPI_ISL_10082052, EPI_ISL_10082630, EPI_ISL_10082989, EPI_ISL_10083756, EPI_ISL_10083777, EPI_ISL_10083787, EPI_ISL_10083812, EPI_ISL_10083866, EPI_ISL_10083944, EPI_ISL_10083960, EPI_ISL_10083972, EPI_ISL_10083979, EPI_ISL_10083981, EPI_ISL_10083985, EPI_ISL_10083988, EPI_ISL_10084004, EPI_ISL_10084007, EPI_ISL_10084008, EPI_ISL_10084093, EPI_ISL_10085377, EPI_ISL_10087734, EPI_ISL_10087752, EPI_ISL_10092537, EPI_ISL_10093299, EPI_ISL_10093694, EPI_ISL_10101118, EPI_ISL_10101120, EPI_ISL_10101124, EPI_ISL_10121115, EPI_ISL_10121312, EPI_ISL_10121522, EPI_ISL_10123102, EPI_ISL_10123125, EPI_ISL_10123126, EPI_ISL_10123127, EPI_ISL_10123136, EPI_ISL_10123138, EPI_ISL_10123144, EPI_ISL_10123149, EPI_ISL_10123151, EPI_ISL_10123184, EPI_ISL_10123237, EPI_ISL_10123242, EPI_ISL_10123246, EPI_ISL_10123256, EPI_ISL_10123278, EPI_ISL_10129960, EPI_ISL_10138346, EPI_ISL_10138347, EPI_ISL_10138351, EPI_ISL_10138509, EPI_ISL_10149308, EPI_ISL_10149344, EPI_ISL_10149570, EPI_ISL_10173852, EPI_ISL_10174207, EPI_ISL_10174960, EPI_ISL_10175390, EPI_ISL_10182087, EPI_ISL_10182644, EPI_ISL_10182902, EPI_ISL_10184975, EPI_ISL_10185121, EPI_ISL_10185137, EPI_ISL_10185347, EPI_ISL_10188771, EPI_ISL_10188772, EPI_ISL_10188801, EPI_ISL_10188808, EPI_ISL_10188819, EPI_ISL_10188829, EPI_ISL_10192722, EPI_ISL_10192735, EPI_ISL_10198777, EPI_ISL_10204977, EPI_ISL_10204983, EPI_ISL_10204986, EPI_ISL_10204992, EPI_ISL_10205004, EPI_ISL_10205039, EPI_ISL_10205073, EPI_ISL_10205094, EPI_ISL_10205098, EPI_ISL_10205106, EPI_ISL_10205142, EPI_ISL_10205178, EPI_ISL_10205180, EPI_ISL_10205182, EPI_ISL_10205253, EPI_ISL_10205289, EPI_ISL_10205338, EPI_ISL_10205375, EPI_ISL_10207392, EPI_ISL_10207536, EPI_ISL_10207539, EPI_ISL_10207777, EPI_ISL_10210308, EPI_ISL_10210437, EPI_ISL_10210486, EPI_ISL_10212487, EPI_ISL_10212488, EPI_ISL_10213774, EPI_ISL_10213775, EPI_ISL_10214094, EPI_ISL_10214108, EPI_ISL_10214427, EPI_ISL_10214452, EPI_ISL_10214463, EPI_ISL_10220260, EPI_ISL_10224838, EPI_ISL_10246896, EPI_ISL_10246913, EPI_ISL_10246929, EPI_ISL_10247077, EPI_ISL_10247741, EPI_ISL_10248161, EPI_ISL_10251088, EPI_ISL_10251304, EPI_ISL_10255340, EPI_ISL_10255376, EPI_ISL_10283352, EPI_ISL_10283711, EPI_ISL_10283724, EPI_ISL_10290459, EPI_ISL_10293291, EPI_ISL_10293513, EPI_ISL_10295769, EPI_ISL_10296275, EPI_ISL_10298437, EPI_ISL_10298438, EPI_ISL_10298464, EPI_ISL_10298491, EPI_ISL_10298494, EPI_ISL_10300795, EPI_ISL_10306024, EPI_ISL_10306120, EPI_ISL_10306134, EPI_ISL_10306140, EPI_ISL_10306148, EPI_ISL_10306161, EPI_ISL_10306164, EPI_ISL_10306559, EPI_ISL_10306869, EPI_ISL_10309965, EPI_ISL_10311702, EPI_ISL_10311711, EPI_ISL_10311753, EPI_ISL_10312030, EPI_ISL_10312031, EPI_ISL_10312032, EPI_ISL_10312033, EPI_ISL_10312042, EPI_ISL_10312044, EPI_ISL_10312052, EPI_ISL_10312056, EPI_ISL_10312057, EPI_ISL_10312059, EPI_ISL_10312060, EPI_ISL_10312064, EPI_ISL_10312065, EPI_ISL_10312066, EPI_ISL_10312081, EPI_ISL_10312085, EPI_ISL_10312086, EPI_ISL_10312087, EPI_ISL_10312094, EPI_ISL_10312096, EPI_ISL_10312370, EPI_ISL_10312525, EPI_ISL_10312548, EPI_ISL_10312553, EPI_ISL_10312557, EPI_ISL_10312559, EPI_ISL_10312565, EPI_ISL_10312568, EPI_ISL_10312573, EPI_ISL_10312582, EPI_ISL_10312584, EPI_ISL_10312585, EPI_ISL_10312592, EPI_ISL_10312596, EPI_ISL_10312597, EPI_ISL_10312598, EPI_ISL_10312615, EPI_ISL_10312617, EPI_ISL_10312618, EPI_ISL_10312623, EPI_ISL_10312624, EPI_ISL_10312626, EPI_ISL_10312627, EPI_ISL_10312635, EPI_ISL_10312638, EPI_ISL_10312754, EPI_ISL_10313149, EPI_ISL_10313150, EPI_ISL_10313183, EPI_ISL_10313845, EPI_ISL_10314883, EPI_ISL_10314886, EPI_ISL_10314887, EPI_ISL_10314899, EPI_ISL_10314904, EPI_ISL_10314914, EPI_ISL_10314915, EPI_ISL_10314916, EPI_ISL_10314924, EPI_ISL_10314930, EPI_ISL_10314931, EPI_ISL_10314940, EPI_ISL_10314941, EPI_ISL_10314942, EPI_ISL_10314943, EPI_ISL_10314944, EPI_ISL_10316983, EPI_ISL_10317379, EPI_ISL_10327362, EPI_ISL_10327716, EPI_ISL_10328262, EPI_ISL_10328772, EPI_ISL_10369251, EPI_ISL_10379783, EPI_ISL_10382204, EPI_ISL_10386226, EPI_ISL_10391525, EPI_ISL_10392765, EPI_ISL_10398645, EPI_ISL_10401810, EPI_ISL_10401837, EPI_ISL_10401856, EPI_ISL_10401865, EPI_ISL_10401868, EPI_ISL_10401984, EPI_ISL_10404067, EPI_ISL_10404103, EPI_ISL_10404131, EPI_ISL_10405495, EPI_ISL_10431884, EPI_ISL_10431934, EPI_ISL_10431989, EPI_ISL_10433657, EPI_ISL_10435634, EPI_ISL_10437641, EPI_ISL_10441356, EPI_ISL_10443386, EPI_ISL_10443404, EPI_ISL_10443409, EPI_ISL_10443411, EPI_ISL_10443425, EPI_ISL_10443430, EPI_ISL_10443433, EPI_ISL_10443434, EPI_ISL_10443436, EPI_ISL_10443439, EPI_ISL_10443440, EPI_ISL_10443441, EPI_ISL_10443444, EPI_ISL_10443446, EPI_ISL_10443447, EPI_ISL_10443448, EPI_ISL_10443450, EPI_ISL_10443452, EPI_ISL_10443454, EPI_ISL_10443456, EPI_ISL_10443458, EPI_ISL_10443460, EPI_ISL_10443461, EPI_ISL_10443466, EPI_ISL_10443469, EPI_ISL_10443471, EPI_ISL_10443490, EPI_ISL_10443494, EPI_ISL_10443495, EPI_ISL_10443503, EPI_ISL_10443505, EPI_ISL_10443506, EPI_ISL_10443507, EPI_ISL_10443510, EPI_ISL_10443511, EPI_ISL_10443512, EPI_ISL_10443516, EPI_ISL_10443517, EPI_ISL_10443518, EPI_ISL_10443519, EPI_ISL_10443520, EPI_ISL_10443523, EPI_ISL_10443525, EPI_ISL_10443526, EPI_ISL_10443533, EPI_ISL_10443535, EPI_ISL_10443563, EPI_ISL_10443572, EPI_ISL_10443573, EPI_ISL_10443575, EPI_ISL_10443577, EPI_ISL_10443584, EPI_ISL_10443592, EPI_ISL_10443595, EPI_ISL_10443599, EPI_ISL_10443601, EPI_ISL_10443603, EPI_ISL_10443605, EPI_ISL_10443606, EPI_ISL_10443608, EPI_ISL_10443611, EPI_ISL_10443613, EPI_ISL_10443615, EPI_ISL_10443617, EPI_ISL_10443623, EPI_ISL_10443627, EPI_ISL_10443632, EPI_ISL_10443636, EPI_ISL_10443647, EPI_ISL_10443648, EPI_ISL_10443649, EPI_ISL_10443656, EPI_ISL_10443658, EPI_ISL_10443661, EPI_ISL_10443671, EPI_ISL_10443674, EPI_ISL_10443675, EPI_ISL_10443678, EPI_ISL_10450198, EPI_ISL_10451247, EPI_ISL_10455028, EPI_ISL_10457041, EPI_ISL_10481797, EPI_ISL_10482336, EPI_ISL_10482850, EPI_ISL_10497475, EPI_ISL_10502293, EPI_ISL_10504027, EPI_ISL_10504064, EPI_ISL_10504107, EPI_ISL_10504108, EPI_ISL_10504167, EPI_ISL_10504187, EPI_ISL_10504203, EPI_ISL_10504205, EPI_ISL_10504239, EPI_ISL_10504258, EPI_ISL_10504269, EPI_ISL_10504319, EPI_ISL_10504339, EPI_ISL_10504345, EPI_ISL_10504347, EPI_ISL_10504381, EPI_ISL_10504387, EPI_ISL_10504420, EPI_ISL_10504441, EPI_ISL_10504471, EPI_ISL_10504485, EPI_ISL_10504647, EPI_ISL_10514956, EPI_ISL_10519096, EPI_ISL_10523937, EPI_ISL_10524685, EPI_ISL_10524790, EPI_ISL_10524808, EPI_ISL_10543999, EPI_ISL_10544042, EPI_ISL_10544102, EPI_ISL_10544144, EPI_ISL_10544474, EPI_ISL_10544507, EPI_ISL_10544984, EPI_ISL_10544986, EPI_ISL_10545423, EPI_ISL_10545452, EPI_ISL_10546178, EPI_ISL_10546476, EPI_ISL_10547155, EPI_ISL_10547219, EPI_ISL_10547244, EPI_ISL_10547258, EPI_ISL_10547266, EPI_ISL_10547268, EPI_ISL_10547296, EPI_ISL_10547299, EPI_ISL_10547314, EPI_ISL_10547344, EPI_ISL_10547683, EPI_ISL_10548055, EPI_ISL_10548071, EPI_ISL_10548077, EPI_ISL_10548443, EPI_ISL_10548461, EPI_ISL_10549286, EPI_ISL_10555394, EPI_ISL_10555425, EPI_ISL_10555459, EPI_ISL_10555540, EPI_ISL_10555586, EPI_ISL_10555615, EPI_ISL_10555636, EPI_ISL_10555668, EPI_ISL_10559721, EPI_ISL_10560171, EPI_ISL_10565822, EPI_ISL_10567103, EPI_ISL_10568889, EPI_ISL_10571268, EPI_ISL_10579550, EPI_ISL_10579638, EPI_ISL_10579782, EPI_ISL_10580591, EPI_ISL_10580610, EPI_ISL_10580791, EPI_ISL_10580798, EPI_ISL_10590704, EPI_ISL_10594394, EPI_ISL_10596188, EPI_ISL_10596244, EPI_ISL_10623440, EPI_ISL_10630173, EPI_ISL_10630208, EPI_ISL_10630286, EPI_ISL_10630294, EPI_ISL_10630387, EPI_ISL_10630426, EPI_ISL_10630509, EPI_ISL_10631057, EPI_ISL_10631087, EPI_ISL_10631441, EPI_ISL_10632161, EPI_ISL_10633951, EPI_ISL_10634024, EPI_ISL_10638400, EPI_ISL_10638438, EPI_ISL_10638461, EPI_ISL_10638464, EPI_ISL_10638466, EPI_ISL_10638483, EPI_ISL_10638490, EPI_ISL_10638493, EPI_ISL_10638507, EPI_ISL_10638513, EPI_ISL_10638522, EPI_ISL_10638528, EPI_ISL_10638529, EPI_ISL_10638534, EPI_ISL_10638544, EPI_ISL_10638548, EPI_ISL_10638569, EPI_ISL_10638681, EPI_ISL_10638682, EPI_ISL_10638684, EPI_ISL_10638687, EPI_ISL_10639921, EPI_ISL_10640459, EPI_ISL_10645365, EPI_ISL_10646183, EPI_ISL_10646223, EPI_ISL_10646224, EPI_ISL_10646625, EPI_ISL_10646659, EPI_ISL_10646660, EPI_ISL_10646664, EPI_ISL_10646678, EPI_ISL_10652475, EPI_ISL_10661114, EPI_ISL_10661119, EPI_ISL_10661136, EPI_ISL_10667120, EPI_ISL_10673551, EPI_ISL_10676563, EPI_ISL_10678674, EPI_ISL_10682237, EPI_ISL_10682241, EPI_ISL_10682298, EPI_ISL_10682367, EPI_ISL_10682374, EPI_ISL_10699409, EPI_ISL_10703587, EPI_ISL_10703923, EPI_ISL_10707346, EPI_ISL_10714913, EPI_ISL_10714914, EPI_ISL_10715650, EPI_ISL_10716749, EPI_ISL_10716750, EPI_ISL_10716751, EPI_ISL_10716752, EPI_ISL_10716753, EPI_ISL_10716754, EPI_ISL_10716755, EPI_ISL_10716756, EPI_ISL_10716757, EPI_ISL_10716758, EPI_ISL_10716759, EPI_ISL_10716760, EPI_ISL_10716761, EPI_ISL_10716762, EPI_ISL_10716763, EPI_ISL_10716764, EPI_ISL_10716765, EPI_ISL_10716766, EPI_ISL_10716767, EPI_ISL_10716768, EPI_ISL_10716769, EPI_ISL_10716770, EPI_ISL_10716771, EPI_ISL_10716772, EPI_ISL_10716773, EPI_ISL_10716774, EPI_ISL_10716775, EPI_ISL_10716776, EPI_ISL_10716777, EPI_ISL_10716778, EPI_ISL_10716779, EPI_ISL_10716780, EPI_ISL_10716781, EPI_ISL_10716782, EPI_ISL_10716783, EPI_ISL_10716784, EPI_ISL_10716785, EPI_ISL_10716786, EPI_ISL_10716787, EPI_ISL_10716788, EPI_ISL_10716789, EPI_ISL_10716790, EPI_ISL_10716791, EPI_ISL_10716793, EPI_ISL_10716794, EPI_ISL_10716795, EPI_ISL_10716796, EPI_ISL_10716797, EPI_ISL_10716798, EPI_ISL_10716799, EPI_ISL_10716800, EPI_ISL_10716801, EPI_ISL_10716802, EPI_ISL_10716803, EPI_ISL_10716804, EPI_ISL_10716805, EPI_ISL_10716806, EPI_ISL_10716807, EPI_ISL_10716809, EPI_ISL_10716810, EPI_ISL_10716811, EPI_ISL_10716812, EPI_ISL_10716813, EPI_ISL_10716814, EPI_ISL_10716816, EPI_ISL_10720898, EPI_ISL_10720899, EPI_ISL_10720917, EPI_ISL_10720921, EPI_ISL_10720922, EPI_ISL_10720936, EPI_ISL_10720939, EPI_ISL_10720942, EPI_ISL_10720945, EPI_ISL_10720959, EPI_ISL_10720964, EPI_ISL_10720967, EPI_ISL_10720972, EPI_ISL_10720980, EPI_ISL_10720994, EPI_ISL_10721000, EPI_ISL_10721008, EPI_ISL_10721019, EPI_ISL_10721027, EPI_ISL_10721028, EPI_ISL_10721030, EPI_ISL_10721034, EPI_ISL_10721045, EPI_ISL_10721046, EPI_ISL_10721063, EPI_ISL_10721065, EPI_ISL_10721071, EPI_ISL_10721081, EPI_ISL_10721083, EPI_ISL_10721087, EPI_ISL_10721089, EPI_ISL_10721092, EPI_ISL_10721094, EPI_ISL_10721096, EPI_ISL_10721097, EPI_ISL_10721098, EPI_ISL_10721101, EPI_ISL_10721107, EPI_ISL_10721109, EPI_ISL_10721112, EPI_ISL_10721115, EPI_ISL_10721130, EPI_ISL_10721131, EPI_ISL_10721136, EPI_ISL_10721137, EPI_ISL_10721139, EPI_ISL_10721144, EPI_ISL_10721149, EPI_ISL_10721152, EPI_ISL_10721155, EPI_ISL_10721157, EPI_ISL_10721163, EPI_ISL_10721178, EPI_ISL_10721181, EPI_ISL_10721192, EPI_ISL_10721199, EPI_ISL_10721211, EPI_ISL_10721221, EPI_ISL_10721236, EPI_ISL_10721237, EPI_ISL_10721238, EPI_ISL_10724195, EPI_ISL_10729907, EPI_ISL_10729913, EPI_ISL_10729914, EPI_ISL_10729923, EPI_ISL_10729942, EPI_ISL_10729960, EPI_ISL_10729962, EPI_ISL_10729987, EPI_ISL_10730596, EPI_ISL_10737360, EPI_ISL_10737642, EPI_ISL_10737702, EPI_ISL_10737703, EPI_ISL_10737959, EPI_ISL_10738130, EPI_ISL_10738152, EPI_ISL_10739357, EPI_ISL_10743603, EPI_ISL_10744853, EPI_ISL_10745444, EPI_ISL_10745973, EPI_ISL_10746047, EPI_ISL_10746077, EPI_ISL_10746272, EPI_ISL_10746292, EPI_ISL_10746326, EPI_ISL_10746390, EPI_ISL_10746411, EPI_ISL_10746428, EPI_ISL_10746508, EPI_ISL_10746552, EPI_ISL_10746590, EPI_ISL_10746695, EPI_ISL_10746698, EPI_ISL_10747159, EPI_ISL_10747334, EPI_ISL_10748103, EPI_ISL_10748266, EPI_ISL_10748279, EPI_ISL_10748300, EPI_ISL_10748460, EPI_ISL_10748490, EPI_ISL_10748497, EPI_ISL_10748499, EPI_ISL_10748731, EPI_ISL_10748742, EPI_ISL_10748765, EPI_ISL_10748789, EPI_ISL_10748796, EPI_ISL_10749120, EPI_ISL_10749363, EPI_ISL_10751290, EPI_ISL_10752622, EPI_ISL_10752779, EPI_ISL_10753373, EPI_ISL_10761887, EPI_ISL_10765065, EPI_ISL_10766003, EPI_ISL_10767929, EPI_ISL_10774009, EPI_ISL_10778001, EPI_ISL_10786839, EPI_ISL_10787136, EPI_ISL_10792746, EPI_ISL_10794919, EPI_ISL_10794943, EPI_ISL_10795137, EPI_ISL_10804609, EPI_ISL_10808697, EPI_ISL_10816282, EPI_ISL_10818640, EPI_ISL_10824995, EPI_ISL_10825050, EPI_ISL_10825072, EPI_ISL_10825080, EPI_ISL_10827078, EPI_ISL_10827098, EPI_ISL_10827110, EPI_ISL_10827811, EPI_ISL_10828375, EPI_ISL_10830097, EPI_ISL_10832248, EPI_ISL_10832490, EPI_ISL_10832873, EPI_ISL_10834277, EPI_ISL_10834286, EPI_ISL_10834301, EPI_ISL_10834303, EPI_ISL_10834339, EPI_ISL_10835529, EPI_ISL_10835646, EPI_ISL_10836497, EPI_ISL_10839829, EPI_ISL_10846979, EPI_ISL_10846993, EPI_ISL_10847003, EPI_ISL_10847047, EPI_ISL_10856645, EPI_ISL_10858431, EPI_ISL_10858434, EPI_ISL_10858504, EPI_ISL_10858534, EPI_ISL_10858576, EPI_ISL_10859516, EPI_ISL_10863825, EPI_ISL_10863916, EPI_ISL_10864352, EPI_ISL_10864402, EPI_ISL_10864691, EPI_ISL_10869496, EPI_ISL_10875769, EPI_ISL_10876749, EPI_ISL_10885204, EPI_ISL_10889622, EPI_ISL_10889699, EPI_ISL_10894052, EPI_ISL_10899907, EPI_ISL_10901492, EPI_ISL_10901542, EPI_ISL_10901650, EPI_ISL_10902039, EPI_ISL_10902076, EPI_ISL_10903411, EPI_ISL_10905381, EPI_ISL_10905390, EPI_ISL_10906700, EPI_ISL_10907939, EPI_ISL_10908621, EPI_ISL_10909171, EPI_ISL_10910544, EPI_ISL_10910885, EPI_ISL_10911406, EPI_ISL_10911449, EPI_ISL_10916131, EPI_ISL_10916325, EPI_ISL_10916342, EPI_ISL_10916454, EPI_ISL_10916811, EPI_ISL_10916964, EPI_ISL_10917724, EPI_ISL_10918548, EPI_ISL_10918663, EPI_ISL_10918757, EPI_ISL_10918877, EPI_ISL_10918897, EPI_ISL_10919073, EPI_ISL_10919348, EPI_ISL_10919390, EPI_ISL_10919417, EPI_ISL_10920188, EPI_ISL_10920207, EPI_ISL_10920295, EPI_ISL_10920318, EPI_ISL_10920323, EPI_ISL_10920387, EPI_ISL_10920420, EPI_ISL_10920485, EPI_ISL_10929576, EPI_ISL_10929737, EPI_ISL_10931583, EPI_ISL_10940696, EPI_ISL_10944316, EPI_ISL_10944320, EPI_ISL_10944473, EPI_ISL_10944567, EPI_ISL_10944816, EPI_ISL_10945552, EPI_ISL_10952144, EPI_ISL_10952168, EPI_ISL_10952171, EPI_ISL_10953120, EPI_ISL_10958600, EPI_ISL_10958671, EPI_ISL_10958693, EPI_ISL_10958704, EPI_ISL_10958730, EPI_ISL_10969481, EPI_ISL_10981425, EPI_ISL_10984859, EPI_ISL_10985594, EPI_ISL_10985626, EPI_ISL_10989445, EPI_ISL_10989720, EPI_ISL_10989838, EPI_ISL_10989839, EPI_ISL_10993276, EPI_ISL_10993277, EPI_ISL_10993342, EPI_ISL_10993606, EPI_ISL_10993614, EPI_ISL_10993840, EPI_ISL_10996854, EPI_ISL_10996875, EPI_ISL_10996887, EPI_ISL_10996895, EPI_ISL_10997448, EPI_ISL_10999171")
EPI_11mil_12mil = stringlist_to_strings_set("EPI_ISL_11522614, EPI_ISL_11001378, EPI_ISL_11001397, EPI_ISL_11001406, EPI_ISL_11001410, EPI_ISL_11001422, EPI_ISL_11001431, EPI_ISL_11001508, EPI_ISL_11001616, EPI_ISL_11001834, EPI_ISL_11001864, EPI_ISL_11001898, EPI_ISL_11001909, EPI_ISL_11001910, EPI_ISL_11001940, EPI_ISL_11002001, EPI_ISL_11002017, EPI_ISL_11002051, EPI_ISL_11002068, EPI_ISL_11007865, EPI_ISL_11011841, EPI_ISL_11011842, EPI_ISL_11011899, EPI_ISL_11011902, EPI_ISL_11011912, EPI_ISL_11011916, EPI_ISL_11011921, EPI_ISL_11011925, EPI_ISL_11011943, EPI_ISL_11011947, EPI_ISL_11011948, EPI_ISL_11011954, EPI_ISL_11011956, EPI_ISL_11011959, EPI_ISL_11011977, EPI_ISL_11011979, EPI_ISL_11011981, EPI_ISL_11011985, EPI_ISL_11012065, EPI_ISL_11012067, EPI_ISL_11012071, EPI_ISL_11012083, EPI_ISL_11012095, EPI_ISL_11012096, EPI_ISL_11012105, EPI_ISL_11012146, EPI_ISL_11012162, EPI_ISL_11012215, EPI_ISL_11012226, EPI_ISL_11012236, EPI_ISL_11012240, EPI_ISL_11012245, EPI_ISL_11012261, EPI_ISL_11012315, EPI_ISL_11012413, EPI_ISL_11012456, EPI_ISL_11012472, EPI_ISL_11012512, EPI_ISL_11012531, EPI_ISL_11012543, EPI_ISL_11012621, EPI_ISL_11012652, EPI_ISL_11012696, EPI_ISL_11012965, EPI_ISL_11015460, EPI_ISL_11015499, EPI_ISL_11016046, EPI_ISL_11022678, EPI_ISL_11023750, EPI_ISL_11023776, EPI_ISL_11023778, EPI_ISL_11027868, EPI_ISL_11035832, EPI_ISL_11036960, EPI_ISL_11036979, EPI_ISL_11037012, EPI_ISL_11037077, EPI_ISL_11057571, EPI_ISL_11057573, EPI_ISL_11057577, EPI_ISL_11057578, EPI_ISL_11057581, EPI_ISL_11057586, EPI_ISL_11057587, EPI_ISL_11057588, EPI_ISL_11057589, EPI_ISL_11057593, EPI_ISL_11057599, EPI_ISL_11057601, EPI_ISL_11057616, EPI_ISL_11057681, EPI_ISL_11057697, EPI_ISL_11057712, EPI_ISL_11057720, EPI_ISL_11057741, EPI_ISL_11057775, EPI_ISL_11057776, EPI_ISL_11057791, EPI_ISL_11057819, EPI_ISL_11057998, EPI_ISL_11062632, EPI_ISL_11063127, EPI_ISL_11063642, EPI_ISL_11063656, EPI_ISL_11064020, EPI_ISL_11068208, EPI_ISL_11069952, EPI_ISL_11070263, EPI_ISL_11072733, EPI_ISL_11072760, EPI_ISL_11072840, EPI_ISL_11073708, EPI_ISL_11074533, EPI_ISL_11074547, EPI_ISL_11074949, EPI_ISL_11075203, EPI_ISL_11075204, EPI_ISL_11075208, EPI_ISL_11075232, EPI_ISL_11095421, EPI_ISL_11101743, EPI_ISL_11102862, EPI_ISL_11102868, EPI_ISL_11102875, EPI_ISL_11102889, EPI_ISL_11102890, EPI_ISL_11102891, EPI_ISL_11102900, EPI_ISL_11102924, EPI_ISL_11102931, EPI_ISL_11102937, EPI_ISL_11102940, EPI_ISL_11102941, EPI_ISL_11102942, EPI_ISL_11102950, EPI_ISL_11102956, EPI_ISL_11102957, EPI_ISL_11103042, EPI_ISL_11103086, EPI_ISL_11104664, EPI_ISL_11104679, EPI_ISL_11104750, EPI_ISL_11104787, EPI_ISL_11104817, EPI_ISL_11104830, EPI_ISL_11104878, EPI_ISL_11104879, EPI_ISL_11104936, EPI_ISL_11104946, EPI_ISL_11104949, EPI_ISL_11104969, EPI_ISL_11105281, EPI_ISL_11105304, EPI_ISL_11105376, EPI_ISL_11105377, EPI_ISL_11105589, EPI_ISL_11105593, EPI_ISL_11105661, EPI_ISL_11105665, EPI_ISL_11106012, EPI_ISL_11106554, EPI_ISL_11106847, EPI_ISL_11108553, EPI_ISL_11108569, EPI_ISL_11108594, EPI_ISL_11108623, EPI_ISL_11108654, EPI_ISL_11108683, EPI_ISL_11108722, EPI_ISL_11108748, EPI_ISL_11108767, EPI_ISL_11108772, EPI_ISL_11108774, EPI_ISL_11108803, EPI_ISL_11108815, EPI_ISL_11108817, EPI_ISL_11133720, EPI_ISL_11133721, EPI_ISL_11133722, EPI_ISL_11134084, EPI_ISL_11134695, EPI_ISL_11141677, EPI_ISL_11141682, EPI_ISL_11144326, EPI_ISL_11145334, EPI_ISL_11148047, EPI_ISL_11168875, EPI_ISL_11168895, EPI_ISL_11168902, EPI_ISL_11168912, EPI_ISL_11168920, EPI_ISL_11168923, EPI_ISL_11168926, EPI_ISL_11168929, EPI_ISL_11168930, EPI_ISL_11168932, EPI_ISL_11168988, EPI_ISL_11168992, EPI_ISL_11168995, EPI_ISL_11169013, EPI_ISL_11169014, EPI_ISL_11169015, EPI_ISL_11169017, EPI_ISL_11169022, EPI_ISL_11169028, EPI_ISL_11173013, EPI_ISL_11173017, EPI_ISL_11173018, EPI_ISL_11173041, EPI_ISL_11173049, EPI_ISL_11173060, EPI_ISL_11173065, EPI_ISL_11173069, EPI_ISL_11173070, EPI_ISL_11173076, EPI_ISL_11173079, EPI_ISL_11173084, EPI_ISL_11173088, EPI_ISL_11173353, EPI_ISL_11173361, EPI_ISL_11173417, EPI_ISL_11179918, EPI_ISL_11179924, EPI_ISL_11209814, EPI_ISL_11220075, EPI_ISL_11220597, EPI_ISL_11222931, EPI_ISL_11225546, EPI_ISL_11225824, EPI_ISL_11228601, EPI_ISL_11228602, EPI_ISL_11228630, EPI_ISL_11228665, EPI_ISL_11228678, EPI_ISL_11228700, EPI_ISL_11235456, EPI_ISL_11235471, EPI_ISL_11235483, EPI_ISL_11235496, EPI_ISL_11235499, EPI_ISL_11235563, EPI_ISL_11235578, EPI_ISL_11235580, EPI_ISL_11235581, EPI_ISL_11235583, EPI_ISL_11235627, EPI_ISL_11235843, EPI_ISL_11235856, EPI_ISL_11235861, EPI_ISL_11235862, EPI_ISL_11235863, EPI_ISL_11235865, EPI_ISL_11235879, EPI_ISL_11235890, EPI_ISL_11235919, EPI_ISL_11235925, EPI_ISL_11235929, EPI_ISL_11235937, EPI_ISL_11236108, EPI_ISL_11236172, EPI_ISL_11236449, EPI_ISL_11237359, EPI_ISL_11246565, EPI_ISL_11260036, EPI_ISL_11260040, EPI_ISL_11260049, EPI_ISL_11260136, EPI_ISL_11260148, EPI_ISL_11260154, EPI_ISL_11262602, EPI_ISL_11268266, EPI_ISL_11268277, EPI_ISL_11268631, EPI_ISL_11270606, EPI_ISL_11271349, EPI_ISL_11278381, EPI_ISL_11278382, EPI_ISL_11284205, EPI_ISL_11291795, EPI_ISL_11299954, EPI_ISL_11299975, EPI_ISL_11311005, EPI_ISL_11323423, EPI_ISL_11323501, EPI_ISL_11323538, EPI_ISL_11323548, EPI_ISL_11323552, EPI_ISL_11323584, EPI_ISL_11323592, EPI_ISL_11323651, EPI_ISL_11323672, EPI_ISL_11323696, EPI_ISL_11323720, EPI_ISL_11325780, EPI_ISL_11330485, EPI_ISL_11333623, EPI_ISL_11333761, EPI_ISL_11344101, EPI_ISL_11349130, EPI_ISL_11351998, EPI_ISL_11369287, EPI_ISL_11381121, EPI_ISL_11381144, EPI_ISL_11397865, EPI_ISL_11404058, EPI_ISL_11404085, EPI_ISL_11404186, EPI_ISL_11404235, EPI_ISL_11404242, EPI_ISL_11404262, EPI_ISL_11404274, EPI_ISL_11404733, EPI_ISL_11410657, EPI_ISL_11414660, EPI_ISL_11414941, EPI_ISL_11415188, EPI_ISL_11416082, EPI_ISL_11436707, EPI_ISL_11441807, EPI_ISL_11442637, EPI_ISL_11450574, EPI_ISL_11451045, EPI_ISL_11451135, EPI_ISL_11451317, EPI_ISL_11451340, EPI_ISL_11451721, EPI_ISL_11451747, EPI_ISL_11452037, EPI_ISL_11452179, EPI_ISL_11452204, EPI_ISL_11454385, EPI_ISL_11454553, EPI_ISL_11454667, EPI_ISL_11454932, EPI_ISL_11455246, EPI_ISL_11466767, EPI_ISL_11468033, EPI_ISL_11468040, EPI_ISL_11468560, EPI_ISL_11468703, EPI_ISL_11468704, EPI_ISL_11468712, EPI_ISL_11468998, EPI_ISL_11469089, EPI_ISL_11471268, EPI_ISL_11471391, EPI_ISL_11471530, EPI_ISL_11471595, EPI_ISL_11472799, EPI_ISL_11473010, EPI_ISL_11473294, EPI_ISL_11473344, EPI_ISL_11474040, EPI_ISL_11474069, EPI_ISL_11474248, EPI_ISL_11474345, EPI_ISL_11474631, EPI_ISL_11474689, EPI_ISL_11477582, EPI_ISL_11477660, EPI_ISL_11478575, EPI_ISL_11480313, EPI_ISL_11483474, EPI_ISL_11483944, EPI_ISL_11483969, EPI_ISL_11486118, EPI_ISL_11486134, EPI_ISL_11486916, EPI_ISL_11487008, EPI_ISL_11490008, EPI_ISL_11494181, EPI_ISL_11495508, EPI_ISL_11503117, EPI_ISL_11505283, EPI_ISL_11505390, EPI_ISL_11507500, EPI_ISL_11507601, EPI_ISL_11519651, EPI_ISL_11520066, EPI_ISL_11521151, EPI_ISL_11521416, EPI_ISL_11521422, EPI_ISL_11521427, EPI_ISL_11521431, EPI_ISL_11521432, EPI_ISL_11521468, EPI_ISL_11521537, EPI_ISL_11521679, EPI_ISL_11522492, EPI_ISL_11522623, EPI_ISL_11522626, EPI_ISL_11525794, EPI_ISL_11525892, EPI_ISL_11525987, EPI_ISL_11526041, EPI_ISL_11532512, EPI_ISL_11533032, EPI_ISL_11533037, EPI_ISL_11533038, EPI_ISL_11533039, EPI_ISL_11533042, EPI_ISL_11533048, EPI_ISL_11533055, EPI_ISL_11533059, EPI_ISL_11533060, EPI_ISL_11533069, EPI_ISL_11533082, EPI_ISL_11533083, EPI_ISL_11533084, EPI_ISL_11533107, EPI_ISL_11533112, EPI_ISL_11533950, EPI_ISL_11534031, EPI_ISL_11534153, EPI_ISL_11534154, EPI_ISL_11538263, EPI_ISL_11543286, EPI_ISL_11543382, EPI_ISL_11543420, EPI_ISL_11543431, EPI_ISL_11555534, EPI_ISL_11555548, EPI_ISL_11555594, EPI_ISL_11559326, EPI_ISL_11559410, EPI_ISL_11559413, EPI_ISL_11559417, EPI_ISL_11559448, EPI_ISL_11563413, EPI_ISL_11563417, EPI_ISL_11563418, EPI_ISL_11565882, EPI_ISL_11565885, EPI_ISL_11566152, EPI_ISL_11566753, EPI_ISL_11575298, EPI_ISL_11575324, EPI_ISL_11575447, EPI_ISL_11575462, EPI_ISL_11575466, EPI_ISL_11575478, EPI_ISL_11575481, EPI_ISL_11578018, EPI_ISL_11579649, EPI_ISL_11579784, EPI_ISL_11579804, EPI_ISL_11579812, EPI_ISL_11579831, EPI_ISL_11580138, EPI_ISL_11580145, EPI_ISL_11582258, EPI_ISL_11582259, EPI_ISL_11582428, EPI_ISL_11582444, EPI_ISL_11582544, EPI_ISL_11582650, EPI_ISL_11582673, EPI_ISL_11583316, EPI_ISL_11583338, EPI_ISL_11583341, EPI_ISL_11584302, EPI_ISL_11584306, EPI_ISL_11584307, EPI_ISL_11584311, EPI_ISL_11584334, EPI_ISL_11584335, EPI_ISL_11586351, EPI_ISL_11586743, EPI_ISL_11588831, EPI_ISL_11588876, EPI_ISL_11591856, EPI_ISL_11593249, EPI_ISL_11594770, EPI_ISL_11595467, EPI_ISL_11598753, EPI_ISL_11610925, EPI_ISL_11612817, EPI_ISL_11612972, EPI_ISL_11612981, EPI_ISL_11613645, EPI_ISL_11614059, EPI_ISL_11615640, EPI_ISL_11616224, EPI_ISL_11618940, EPI_ISL_11619152, EPI_ISL_11619182, EPI_ISL_11619311, EPI_ISL_11619523, EPI_ISL_11621075, EPI_ISL_11636756, EPI_ISL_11651621, EPI_ISL_11651726, EPI_ISL_11651735, EPI_ISL_11654722, EPI_ISL_11656053, EPI_ISL_11656172, EPI_ISL_11656610, EPI_ISL_11656659, EPI_ISL_11657596, EPI_ISL_11657892, EPI_ISL_11657965, EPI_ISL_11662146, EPI_ISL_11663835, EPI_ISL_11666737, EPI_ISL_11666773, EPI_ISL_11667158, EPI_ISL_11667216, EPI_ISL_11672579, EPI_ISL_11672755, EPI_ISL_11672970, EPI_ISL_11677016, EPI_ISL_11681633, EPI_ISL_11682511, EPI_ISL_11687875, EPI_ISL_11687889, EPI_ISL_11687901, EPI_ISL_11735262, EPI_ISL_11738966, EPI_ISL_11738979, EPI_ISL_11738981, EPI_ISL_11739588, EPI_ISL_11744191, EPI_ISL_11744197, EPI_ISL_11744203, EPI_ISL_11744225, EPI_ISL_11744241, EPI_ISL_11745458, EPI_ISL_11750312, EPI_ISL_11754783, EPI_ISL_11755178, EPI_ISL_11764066, EPI_ISL_11764145, EPI_ISL_11764370, EPI_ISL_11765223, EPI_ISL_11765233, EPI_ISL_11765239, EPI_ISL_11765314, EPI_ISL_11776032, EPI_ISL_11777698, EPI_ISL_11778037, EPI_ISL_11780300, EPI_ISL_11780326, EPI_ISL_11780428, EPI_ISL_11792579, EPI_ISL_11792580, EPI_ISL_11799231, EPI_ISL_11799253, EPI_ISL_11799254, EPI_ISL_11799412, EPI_ISL_11799422, EPI_ISL_11799425, EPI_ISL_11799426, EPI_ISL_11799427, EPI_ISL_11799428, EPI_ISL_11799429, EPI_ISL_11799448, EPI_ISL_11799451, EPI_ISL_11799459, EPI_ISL_11799460, EPI_ISL_11799463, EPI_ISL_11799472, EPI_ISL_11799473, EPI_ISL_11799482, EPI_ISL_11799483, EPI_ISL_11800533, EPI_ISL_11805895, EPI_ISL_11816334, EPI_ISL_11817226, EPI_ISL_11817393, EPI_ISL_11818911, EPI_ISL_11822283, EPI_ISL_11822392, EPI_ISL_11822405, EPI_ISL_11822410, EPI_ISL_11822441, EPI_ISL_11822445, EPI_ISL_11822452, EPI_ISL_11833006, EPI_ISL_11833041, EPI_ISL_11833043, EPI_ISL_11833045, EPI_ISL_11834694, EPI_ISL_11834787, EPI_ISL_11835704, EPI_ISL_11838824, EPI_ISL_11838984, EPI_ISL_11839213, EPI_ISL_11839231, EPI_ISL_11839266, EPI_ISL_11840926, EPI_ISL_11841862, EPI_ISL_11851894, EPI_ISL_11851906, EPI_ISL_11852879, EPI_ISL_11854563, EPI_ISL_11857754, EPI_ISL_11857995, EPI_ISL_11858417, EPI_ISL_11858533, EPI_ISL_11858709, EPI_ISL_11858787, EPI_ISL_11858901, EPI_ISL_11860869, EPI_ISL_11861621, EPI_ISL_11873762, EPI_ISL_11887370, EPI_ISL_11887719, EPI_ISL_11887726, EPI_ISL_11890159, EPI_ISL_11890160, EPI_ISL_11890193, EPI_ISL_11897161, EPI_ISL_11897201, EPI_ISL_11897281, EPI_ISL_11897326, EPI_ISL_11897379, EPI_ISL_11898663, EPI_ISL_11898850, EPI_ISL_11899077, EPI_ISL_11901662, EPI_ISL_11901676, EPI_ISL_11901688, EPI_ISL_11901702, EPI_ISL_11901703, EPI_ISL_11901787, EPI_ISL_11905620, EPI_ISL_11908129, EPI_ISL_11924873, EPI_ISL_11927468, EPI_ISL_11927549, EPI_ISL_11927559, EPI_ISL_11927927, EPI_ISL_11929258, EPI_ISL_11929260, EPI_ISL_11929325, EPI_ISL_11929348, EPI_ISL_11929386, EPI_ISL_11930116, EPI_ISL_11936029, EPI_ISL_11937846, EPI_ISL_11946506, EPI_ISL_11946951, EPI_ISL_11947876, EPI_ISL_11947967, EPI_ISL_11948994, EPI_ISL_11950245, EPI_ISL_11952162, EPI_ISL_11953205, EPI_ISL_11953779, EPI_ISL_11954832, EPI_ISL_11969387, EPI_ISL_11969394, EPI_ISL_11971083, EPI_ISL_11976102, EPI_ISL_11988824, EPI_ISL_11999927")
EPI_16mil_17mil = stringlist_to_strings_set("EPI_ISL_16815237, EPI_ISL_16897661, EPI_ISL_16371526, EPI_ISL_16654387, EPI_ISL_16654388, EPI_ISL_16670567, EPI_ISL_16742612, EPI_ISL_16991534, EPI_ISL_16876388, EPI_ISL_16206521, EPI_ISL_16097562, EPI_ISL_16201324, EPI_ISL_16472666, EPI_ISL_16897651, EPI_ISL_16012710, EPI_ISL_16012943, EPI_ISL_16012954, EPI_ISL_16012987, EPI_ISL_16013209, EPI_ISL_16013227, EPI_ISL_16013267, EPI_ISL_16013309, EPI_ISL_16013449, EPI_ISL_16113255, EPI_ISL_16113336, EPI_ISL_16113399, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16270893, EPI_ISL_16271048, EPI_ISL_16271102, EPI_ISL_16271255, EPI_ISL_16271333, EPI_ISL_16271386, EPI_ISL_16397554, EPI_ISL_16397699, EPI_ISL_16397720, EPI_ISL_16397721, EPI_ISL_16397783, EPI_ISL_16397860, EPI_ISL_16397883, EPI_ISL_16397988, EPI_ISL_16398016, EPI_ISL_16398020, EPI_ISL_16398022, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16542189, EPI_ISL_16542267, EPI_ISL_16542369, EPI_ISL_16542453, EPI_ISL_16542552, EPI_ISL_16542938, EPI_ISL_16599288, EPI_ISL_16599299, EPI_ISL_16599325, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599460, EPI_ISL_16599467, EPI_ISL_16599492, EPI_ISL_16599495, EPI_ISL_16599498, EPI_ISL_16599525, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599554, EPI_ISL_16599636, EPI_ISL_16599649, EPI_ISL_16599747, EPI_ISL_16599794, EPI_ISL_16599826, EPI_ISL_16599842, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599959, EPI_ISL_16599980, EPI_ISL_16761912, EPI_ISL_16761930, EPI_ISL_16761962, EPI_ISL_16761978, EPI_ISL_16762022, EPI_ISL_16762028, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762066, EPI_ISL_16762070, EPI_ISL_16762084, EPI_ISL_16762132, EPI_ISL_16762148, EPI_ISL_16762176, EPI_ISL_16762245, EPI_ISL_16762249, EPI_ISL_16762255, EPI_ISL_16762269, EPI_ISL_16762273, EPI_ISL_16762297, EPI_ISL_16762310, EPI_ISL_16762327, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762455, EPI_ISL_16762458, EPI_ISL_16762462, EPI_ISL_16762475, EPI_ISL_16762559, EPI_ISL_16762600, EPI_ISL_16762611, EPI_ISL_16762614, EPI_ISL_16762631, EPI_ISL_16762652, EPI_ISL_16762653, EPI_ISL_16762654, EPI_ISL_16762699, EPI_ISL_16762700, EPI_ISL_16762714, EPI_ISL_16762726, EPI_ISL_16762735, EPI_ISL_16762753, EPI_ISL_16919296, EPI_ISL_16919303, EPI_ISL_16919306, EPI_ISL_16919315, EPI_ISL_16919337, EPI_ISL_16919348, EPI_ISL_16919362, EPI_ISL_16919365, EPI_ISL_16919367, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919388, EPI_ISL_16919396, EPI_ISL_16919399, EPI_ISL_16919412, EPI_ISL_16919430, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919456, EPI_ISL_16919467, EPI_ISL_16919468, EPI_ISL_16919496, EPI_ISL_16919504, EPI_ISL_16919523, EPI_ISL_16919526, EPI_ISL_16919546, EPI_ISL_16919558, EPI_ISL_16919586, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919640, EPI_ISL_16919660, EPI_ISL_16919662, EPI_ISL_16919670, EPI_ISL_16919680, EPI_ISL_16919695, EPI_ISL_16919706, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919772, EPI_ISL_16919794, EPI_ISL_16919796, EPI_ISL_16919804, EPI_ISL_16919806, EPI_ISL_16919810, EPI_ISL_16919841, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919852, EPI_ISL_16919856, EPI_ISL_16919870, EPI_ISL_16919894, EPI_ISL_16919897, EPI_ISL_16919902, EPI_ISL_16919906, EPI_ISL_16919919, EPI_ISL_16919932, EPI_ISL_16919944, EPI_ISL_16919962, EPI_ISL_16001302, EPI_ISL_16020838, EPI_ISL_16024683, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16088909, EPI_ISL_16089016, EPI_ISL_16092776, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16148354, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16173291, EPI_ISL_16173302, EPI_ISL_16189719, EPI_ISL_16204016, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16248743, EPI_ISL_16260886, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16276826, EPI_ISL_16281961, EPI_ISL_16281966, EPI_ISL_16293327, EPI_ISL_16302637, EPI_ISL_16327676, EPI_ISL_16364514, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16378441, EPI_ISL_16381367, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16382491, EPI_ISL_16390727, EPI_ISL_16402352, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16496661, EPI_ISL_16499231, EPI_ISL_16507911, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16528798, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16570393, EPI_ISL_16574023, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16608560, EPI_ISL_16612080, EPI_ISL_16613416, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668033, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16729840, EPI_ISL_16734294, EPI_ISL_16742171, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760525, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760564, EPI_ISL_16760565, EPI_ISL_16763262, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16825422, EPI_ISL_16835014, EPI_ISL_16845486, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16860526, EPI_ISL_16861003, EPI_ISL_16870757, EPI_ISL_16879371, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16893748, EPI_ISL_16895710, EPI_ISL_16895713, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907588, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16921497, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943850, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16966278, EPI_ISL_16968369, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16020838, EPI_ISL_16041723, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16089016, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16189719, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16212482, EPI_ISL_16226002, EPI_ISL_16248743, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16292066, EPI_ISL_16292121, EPI_ISL_16293327, EPI_ISL_16328978, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16390727, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449525, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16459438, EPI_ISL_16465329, EPI_ISL_16465341, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16513739, EPI_ISL_16527367, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570393, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16607325, EPI_ISL_16612080, EPI_ISL_16612088, EPI_ISL_16613416, EPI_ISL_16638962, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16724327, EPI_ISL_16729840, EPI_ISL_16734284, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760565, EPI_ISL_16760567, EPI_ISL_16765385, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16835014, EPI_ISL_16837809, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16861003, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16895710, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16900417, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943847, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16097754, EPI_ISL_16119602, EPI_ISL_16119610, EPI_ISL_16119629, EPI_ISL_16171387, EPI_ISL_16204016, EPI_ISL_16212482, EPI_ISL_16307563, EPI_ISL_16357154, EPI_ISL_16368857, EPI_ISL_16369646, EPI_ISL_16377998, EPI_ISL_16382423, EPI_ISL_16390790, EPI_ISL_16402352, EPI_ISL_16440038, EPI_ISL_16441289, EPI_ISL_16449517, EPI_ISL_16457740, EPI_ISL_16465341, EPI_ISL_16475710, EPI_ISL_16483723, EPI_ISL_16496661, EPI_ISL_16510324, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16529241, EPI_ISL_16536287, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16574237, EPI_ISL_16606996, EPI_ISL_16607319, EPI_ISL_16607322, EPI_ISL_16607325, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16670793, EPI_ISL_16702339, EPI_ISL_16702350, EPI_ISL_16702360, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702393, EPI_ISL_16702396, EPI_ISL_16733307, EPI_ISL_16734294, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760541, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760562, EPI_ISL_16760565, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16823432, EPI_ISL_16845486, EPI_ISL_16870757, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907559, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907602, EPI_ISL_16915893, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943857, EPI_ISL_16943858, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16993581, EPI_ISL_16073446, EPI_ISL_16861820, EPI_ISL_16510293, EPI_ISL_16815208, EPI_ISL_16457333, EPI_ISL_16937010, EPI_ISL_16536298, EPI_ISL_16536270, EPI_ISL_16536294, EPI_ISL_16536310, EPI_ISL_16536267, EPI_ISL_16913912, EPI_ISL_16170920, EPI_ISL_16201285, EPI_ISL_16201296, EPI_ISL_16201312, EPI_ISL_16201314, EPI_ISL_16201319, EPI_ISL_16201326, EPI_ISL_16201331, EPI_ISL_16201278, EPI_ISL_16201334, EPI_ISL_16940694, EPI_ISL_16861820, EPI_ISL_16012710, EPI_ISL_16012943, EPI_ISL_16012954, EPI_ISL_16012987, EPI_ISL_16013209, EPI_ISL_16013227, EPI_ISL_16013267, EPI_ISL_16013309, EPI_ISL_16013449, EPI_ISL_16113255, EPI_ISL_16113336, EPI_ISL_16113399, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16270893, EPI_ISL_16271048, EPI_ISL_16271102, EPI_ISL_16271255, EPI_ISL_16271333, EPI_ISL_16271386, EPI_ISL_16397554, EPI_ISL_16397699, EPI_ISL_16397720, EPI_ISL_16397721, EPI_ISL_16397783, EPI_ISL_16397860, EPI_ISL_16397883, EPI_ISL_16397988, EPI_ISL_16398016, EPI_ISL_16398020, EPI_ISL_16398022, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16542189, EPI_ISL_16542267, EPI_ISL_16542369, EPI_ISL_16542453, EPI_ISL_16542552, EPI_ISL_16542938, EPI_ISL_16599288, EPI_ISL_16599299, EPI_ISL_16599325, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599460, EPI_ISL_16599467, EPI_ISL_16599492, EPI_ISL_16599495, EPI_ISL_16599498, EPI_ISL_16599525, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599554, EPI_ISL_16599636, EPI_ISL_16599649, EPI_ISL_16599747, EPI_ISL_16599794, EPI_ISL_16599826, EPI_ISL_16599842, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599959, EPI_ISL_16599980, EPI_ISL_16761912, EPI_ISL_16761930, EPI_ISL_16761962, EPI_ISL_16761978, EPI_ISL_16762022, EPI_ISL_16762028, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762066, EPI_ISL_16762070, EPI_ISL_16762084, EPI_ISL_16762132, EPI_ISL_16762148, EPI_ISL_16762176, EPI_ISL_16762245, EPI_ISL_16762249, EPI_ISL_16762255, EPI_ISL_16762269, EPI_ISL_16762273, EPI_ISL_16762297, EPI_ISL_16762310, EPI_ISL_16762327, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762455, EPI_ISL_16762458, EPI_ISL_16762462, EPI_ISL_16762475, EPI_ISL_16762559, EPI_ISL_16762600, EPI_ISL_16762611, EPI_ISL_16762614, EPI_ISL_16762631, EPI_ISL_16762652, EPI_ISL_16762653, EPI_ISL_16762654, EPI_ISL_16762699, EPI_ISL_16762700, EPI_ISL_16762714, EPI_ISL_16762726, EPI_ISL_16762735, EPI_ISL_16762753, EPI_ISL_16919296, EPI_ISL_16919303, EPI_ISL_16919306, EPI_ISL_16919315, EPI_ISL_16919337, EPI_ISL_16919348, EPI_ISL_16919362, EPI_ISL_16919365, EPI_ISL_16919367, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919388, EPI_ISL_16919396, EPI_ISL_16919399, EPI_ISL_16919412, EPI_ISL_16919430, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919456, EPI_ISL_16919467, EPI_ISL_16919468, EPI_ISL_16919496, EPI_ISL_16919504, EPI_ISL_16919523, EPI_ISL_16919526, EPI_ISL_16919546, EPI_ISL_16919558, EPI_ISL_16919586, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919640, EPI_ISL_16919660, EPI_ISL_16919662, EPI_ISL_16919670, EPI_ISL_16919680, EPI_ISL_16919695, EPI_ISL_16919706, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919772, EPI_ISL_16919794, EPI_ISL_16919796, EPI_ISL_16919804, EPI_ISL_16919806, EPI_ISL_16919810, EPI_ISL_16919841, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919852, EPI_ISL_16919856, EPI_ISL_16919870, EPI_ISL_16919894, EPI_ISL_16919897, EPI_ISL_16919902, EPI_ISL_16919906, EPI_ISL_16919919, EPI_ISL_16919932, EPI_ISL_16919944, EPI_ISL_16919962, EPI_ISL_16001302, EPI_ISL_16020838, EPI_ISL_16024683, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16088909, EPI_ISL_16089016, EPI_ISL_16092776, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16148354, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16173291, EPI_ISL_16173302, EPI_ISL_16189719, EPI_ISL_16204016, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16248743, EPI_ISL_16260886, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16276826, EPI_ISL_16281961, EPI_ISL_16281966, EPI_ISL_16293327, EPI_ISL_16302637, EPI_ISL_16327676, EPI_ISL_16364514, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16378441, EPI_ISL_16381367, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16382491, EPI_ISL_16390727, EPI_ISL_16402352, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16496661, EPI_ISL_16499231, EPI_ISL_16507911, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16528798, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16570393, EPI_ISL_16574023, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16608560, EPI_ISL_16612080, EPI_ISL_16613416, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668033, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16729840, EPI_ISL_16734294, EPI_ISL_16742171, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760525, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760564, EPI_ISL_16760565, EPI_ISL_16763262, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16825422, EPI_ISL_16835014, EPI_ISL_16845486, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16860526, EPI_ISL_16861003, EPI_ISL_16870757, EPI_ISL_16879371, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16893748, EPI_ISL_16895710, EPI_ISL_16895713, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907588, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16921497, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943850, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16966278, EPI_ISL_16968369, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16020838, EPI_ISL_16041723, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16089016, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16189719, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16212482, EPI_ISL_16226002, EPI_ISL_16248743, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16292066, EPI_ISL_16292121, EPI_ISL_16293327, EPI_ISL_16328978, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16390727, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449525, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16459438, EPI_ISL_16465329, EPI_ISL_16465341, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16513739, EPI_ISL_16527367, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570393, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16607325, EPI_ISL_16612080, EPI_ISL_16612088, EPI_ISL_16613416, EPI_ISL_16638962, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16724327, EPI_ISL_16729840, EPI_ISL_16734284, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760565, EPI_ISL_16760567, EPI_ISL_16765385, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16835014, EPI_ISL_16837809, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16861003, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16895710, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16900417, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943847, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16097754, EPI_ISL_16119602, EPI_ISL_16119610, EPI_ISL_16119629, EPI_ISL_16171387, EPI_ISL_16204016, EPI_ISL_16212482, EPI_ISL_16307563, EPI_ISL_16357154, EPI_ISL_16368857, EPI_ISL_16369646, EPI_ISL_16377998, EPI_ISL_16382423, EPI_ISL_16390790, EPI_ISL_16402352, EPI_ISL_16440038, EPI_ISL_16441289, EPI_ISL_16449517, EPI_ISL_16457740, EPI_ISL_16465341, EPI_ISL_16475710, EPI_ISL_16483723, EPI_ISL_16496661, EPI_ISL_16510324, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16529241, EPI_ISL_16536287, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16574237, EPI_ISL_16606996, EPI_ISL_16607319, EPI_ISL_16607322, EPI_ISL_16607325, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16670793, EPI_ISL_16702339, EPI_ISL_16702350, EPI_ISL_16702360, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702393, EPI_ISL_16702396, EPI_ISL_16733307, EPI_ISL_16734294, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760541, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760562, EPI_ISL_16760565, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16823432, EPI_ISL_16845486, EPI_ISL_16870757, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907559, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907602, EPI_ISL_16915893, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943857, EPI_ISL_16943858, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16993581")
Spike_I794N = stringlist_to_strings_set("EPI_ISL_1407032, EPI_ISL_1646281, EPI_ISL_2221068, EPI_ISL_2661550-2661609, EPI_ISL_2681488-2681550, EPI_ISL_2970972, EPI_ISL_2970981, EPI_ISL_2970989, EPI_ISL_2970996, EPI_ISL_2970999, EPI_ISL_2971051, EPI_ISL_2971057, EPI_ISL_2971077, EPI_ISL_2971085-2971086, EPI_ISL_2971089, EPI_ISL_2971099, EPI_ISL_2971123, EPI_ISL_2971141, EPI_ISL_2971157, EPI_ISL_2971165, EPI_ISL_2996654, EPI_ISL_2996657, EPI_ISL_2996702, EPI_ISL_2996712, EPI_ISL_2996720, EPI_ISL_3373334, EPI_ISL_3613280, EPI_ISL_3613461, EPI_ISL_3613545, EPI_ISL_3613599, EPI_ISL_3613648, EPI_ISL_3613711, EPI_ISL_3613807, EPI_ISL_3614113, EPI_ISL_3759977, EPI_ISL_3801323, EPI_ISL_3923183, EPI_ISL_3998693-3998695, EPI_ISL_3998698-3998700, EPI_ISL_3998702, EPI_ISL_3998704-3998705, EPI_ISL_3998708, EPI_ISL_3998710-3998711, EPI_ISL_3998713-3998714, EPI_ISL_3998716, EPI_ISL_3998718-3998720, EPI_ISL_3998722-3998724, EPI_ISL_3998726, EPI_ISL_3998728, EPI_ISL_3998730-3998735, EPI_ISL_3998737-3998741, EPI_ISL_3998743-3998745, EPI_ISL_3998747-3998748, EPI_ISL_3998751-3998752, EPI_ISL_3998754-3998757, EPI_ISL_3998760-3998768, EPI_ISL_3998771-3998772, EPI_ISL_3998775-3998777, EPI_ISL_3998781, EPI_ISL_3998784, EPI_ISL_4470628-4470629, EPI_ISL_4470633, EPI_ISL_4470646-4470647, EPI_ISL_5350573, EPI_ISL_6566104, EPI_ISL_6596059, EPI_ISL_6596063, EPI_ISL_6596067-6596072, EPI_ISL_6596074-6596076, EPI_ISL_6596079-6596080, EPI_ISL_6596082-6596088, EPI_ISL_6596090-6596091, EPI_ISL_6596094, EPI_ISL_6596096, EPI_ISL_6596098, EPI_ISL_6596100-6596101, EPI_ISL_6596103, EPI_ISL_6596106-6596107, EPI_ISL_6596109, EPI_ISL_6596111-6596117, EPI_ISL_6596119, EPI_ISL_6596122-6596124, EPI_ISL_6596127, EPI_ISL_6596129-6596131, EPI_ISL_6596133, EPI_ISL_6596135, EPI_ISL_6596137, EPI_ISL_6601688, EPI_ISL_6601692-6601693, EPI_ISL_6601695-6601698, EPI_ISL_6601701-6601708, EPI_ISL_6601710, EPI_ISL_6601712-6601713, EPI_ISL_6601715-6601716, EPI_ISL_6601718, EPI_ISL_6601720-6601722, EPI_ISL_6601724, EPI_ISL_6601727-6601731, EPI_ISL_6601733, EPI_ISL_6601735-6601740, EPI_ISL_6601742-6601744, EPI_ISL_6601747-6601748, EPI_ISL_6601750-6601759, EPI_ISL_6601762-6601766, EPI_ISL_6601770, EPI_ISL_6601774, EPI_ISL_6601776, EPI_ISL_6601778-6601780, EPI_ISL_6601782-6601783, EPI_ISL_6601785-6601786, EPI_ISL_6601788, EPI_ISL_6601790-6601793, EPI_ISL_6601795-6601797, EPI_ISL_6601799-6601802, EPI_ISL_6601805, EPI_ISL_6601807-6601808, EPI_ISL_6601810-6601822, EPI_ISL_6601824-6601833, EPI_ISL_6601835, EPI_ISL_6601837-6601839, EPI_ISL_6601841-6601842, EPI_ISL_6601844-6601846, EPI_ISL_6601849, EPI_ISL_6601851-6601858, EPI_ISL_6601860, EPI_ISL_6601862-6601866, EPI_ISL_6601872-6601880, EPI_ISL_6601884-6601886, EPI_ISL_6601891, EPI_ISL_6601894, EPI_ISL_6601899-6601906, EPI_ISL_6601908-6601909, EPI_ISL_6601911-6601912, EPI_ISL_6601914-6601922, EPI_ISL_6601924, EPI_ISL_6601926-6601929, EPI_ISL_6601931-6601932, EPI_ISL_6601935-6601936, EPI_ISL_6601938-6601939, EPI_ISL_6601941-6601944, EPI_ISL_6601948-6601949, EPI_ISL_6601951-6601957, EPI_ISL_6601961-6601965, EPI_ISL_6601969-6601979, EPI_ISL_6601981-6601982, EPI_ISL_6601985-6601991, EPI_ISL_6601993-6602003, EPI_ISL_6602005-6602006, EPI_ISL_6602008, EPI_ISL_6602010, EPI_ISL_6602012-6602021, EPI_ISL_6602023-6602028, EPI_ISL_6602032-6602033, EPI_ISL_6602035-6602036, EPI_ISL_6602038-6602039, EPI_ISL_6602041, EPI_ISL_6603775-6603778, EPI_ISL_6603780-6603782, EPI_ISL_6603784-6603786, EPI_ISL_6603790-6603793, EPI_ISL_6603795-6603800, EPI_ISL_6603802-6603809, EPI_ISL_6603813-6603823, EPI_ISL_6603825-6603829, EPI_ISL_6603831-6603833, EPI_ISL_6603835, EPI_ISL_6603837-6603842, EPI_ISL_6603847, EPI_ISL_6603850-6603857, EPI_ISL_6603860-6603862, EPI_ISL_6603864, EPI_ISL_6603866-6603874, EPI_ISL_6603876-6603880, EPI_ISL_6603882-6603884, EPI_ISL_6603886-6603888, EPI_ISL_6603891-6603894, EPI_ISL_6603896-6603898, EPI_ISL_6603901-6603903, EPI_ISL_6603905-6603906, EPI_ISL_6603911-6603925, EPI_ISL_6603927-6603940, EPI_ISL_6603943, EPI_ISL_6603945-6603949, EPI_ISL_6645158, EPI_ISL_6645161, EPI_ISL_6645169, EPI_ISL_6645175, EPI_ISL_6645183, EPI_ISL_6645192, EPI_ISL_6645196, EPI_ISL_6645211, EPI_ISL_6645216, EPI_ISL_6645223, EPI_ISL_6645228, EPI_ISL_6645246-6645247, EPI_ISL_6645252, EPI_ISL_6645260, EPI_ISL_6645266, EPI_ISL_6645279, EPI_ISL_6645291, EPI_ISL_6645307, EPI_ISL_6645326, EPI_ISL_6645338, EPI_ISL_6645344, EPI_ISL_6645360, EPI_ISL_6645371, EPI_ISL_6645381, EPI_ISL_6645396, EPI_ISL_6645403, EPI_ISL_6645409, EPI_ISL_6645419, EPI_ISL_6645434, EPI_ISL_6645441, EPI_ISL_6645462, EPI_ISL_6645471, EPI_ISL_6645487, EPI_ISL_6645499, EPI_ISL_6645504, EPI_ISL_6645513, EPI_ISL_6645547, EPI_ISL_6645554, EPI_ISL_6645560, EPI_ISL_6645570, EPI_ISL_6645574, EPI_ISL_6645590, EPI_ISL_6645600, EPI_ISL_6645603, EPI_ISL_6645613, EPI_ISL_6645625, EPI_ISL_6645635, EPI_ISL_6645645, EPI_ISL_6645653, EPI_ISL_6645664, EPI_ISL_6645673, EPI_ISL_6645678, EPI_ISL_6645685, EPI_ISL_6645695, EPI_ISL_6645700, EPI_ISL_6645705, EPI_ISL_6645714, EPI_ISL_6645742, EPI_ISL_6645749, EPI_ISL_6645769, EPI_ISL_6645778, EPI_ISL_6645785, EPI_ISL_6645801, EPI_ISL_6645814, EPI_ISL_6645821, EPI_ISL_6645827, EPI_ISL_6645836, EPI_ISL_6645845, EPI_ISL_6645852, EPI_ISL_6645861, EPI_ISL_6645868, EPI_ISL_6645875, EPI_ISL_6645881, EPI_ISL_6645885, EPI_ISL_6645891, EPI_ISL_6645896, EPI_ISL_6645909, EPI_ISL_6645916, EPI_ISL_6645924, EPI_ISL_6645932, EPI_ISL_6645937, EPI_ISL_6645943, EPI_ISL_6645950, EPI_ISL_6645971, EPI_ISL_6645984-6645985, EPI_ISL_6645997, EPI_ISL_6646022, EPI_ISL_6646030, EPI_ISL_6646034, EPI_ISL_6646044, EPI_ISL_6646052, EPI_ISL_6646057, EPI_ISL_6646065, EPI_ISL_6646084, EPI_ISL_6646092, EPI_ISL_6646101, EPI_ISL_6646107, EPI_ISL_6646110, EPI_ISL_6646123, EPI_ISL_6646127, EPI_ISL_6646135, EPI_ISL_6646145, EPI_ISL_6646152, EPI_ISL_6646160, EPI_ISL_6646168, EPI_ISL_6646180, EPI_ISL_6646188-6646189, EPI_ISL_6646199, EPI_ISL_6646211, EPI_ISL_6646224-6646225, EPI_ISL_6711328, EPI_ISL_6711340, EPI_ISL_6711344, EPI_ISL_6711349, EPI_ISL_6711357, EPI_ISL_6711362, EPI_ISL_6711380, EPI_ISL_6711386, EPI_ISL_6711392, EPI_ISL_6711400, EPI_ISL_6711422, EPI_ISL_6711433, EPI_ISL_6711448, EPI_ISL_6711452, EPI_ISL_6711458, EPI_ISL_6711460-6711463, EPI_ISL_6711465-6711467, EPI_ISL_6711469-6711471, EPI_ISL_6711473, EPI_ISL_6711476, EPI_ISL_6711478, EPI_ISL_6711482, EPI_ISL_6711484, EPI_ISL_6711486-6711489, EPI_ISL_6711492-6711494, EPI_ISL_6711497-6711498, EPI_ISL_6711500-6711501, EPI_ISL_6711515-6711516, EPI_ISL_6711527, EPI_ISL_6711533, EPI_ISL_6711537, EPI_ISL_6711547, EPI_ISL_6711562, EPI_ISL_6711569, EPI_ISL_6711571, EPI_ISL_6930232-6930233, EPI_ISL_6954331-6954332, EPI_ISL_6954334-6954337, EPI_ISL_6954339-6954341, EPI_ISL_6954344-6954345, EPI_ISL_6954347, EPI_ISL_6954349, EPI_ISL_6954351-6954354, EPI_ISL_6954356-6954362, EPI_ISL_6954367, EPI_ISL_6954370-6954372, EPI_ISL_6954374-6954377, EPI_ISL_6954379-6954381, EPI_ISL_6954383-6954397, EPI_ISL_6954399, EPI_ISL_6954401-6954409, EPI_ISL_6954411-6954414, EPI_ISL_6954416-6954422, EPI_ISL_6954424-6954425, EPI_ISL_6954427-6954429, EPI_ISL_6954431-6954443, EPI_ISL_6954445-6954449, EPI_ISL_6954451-6954455, EPI_ISL_6954458-6954459, EPI_ISL_6954461-6954470, EPI_ISL_6954472-6954492, EPI_ISL_7239290, EPI_ISL_7239302-7239303, EPI_ISL_7239307, EPI_ISL_7239312, EPI_ISL_7239332, EPI_ISL_7239344, EPI_ISL_7239351, EPI_ISL_7239381-7239382, EPI_ISL_7239408, EPI_ISL_7239420-7239421, EPI_ISL_7239427, EPI_ISL_7239446, EPI_ISL_7239455, EPI_ISL_7239463, EPI_ISL_7239472, EPI_ISL_7239489, EPI_ISL_7239498, EPI_ISL_7239506, EPI_ISL_7239513, EPI_ISL_7239522, EPI_ISL_7239529, EPI_ISL_7239572, EPI_ISL_7239578, EPI_ISL_7239596, EPI_ISL_7239605, EPI_ISL_7239618, EPI_ISL_7239628-7239629, EPI_ISL_7239636, EPI_ISL_7239646, EPI_ISL_7239657, EPI_ISL_7239662, EPI_ISL_7239673, EPI_ISL_7239684-7239685, EPI_ISL_7239694, EPI_ISL_7239701, EPI_ISL_7239708, EPI_ISL_7239721, EPI_ISL_7239738, EPI_ISL_7239756, EPI_ISL_7239761-7239762, EPI_ISL_7338768, EPI_ISL_8476117-8476121, EPI_ISL_8476124-8476125, EPI_ISL_8674922-8674940, EPI_ISL_8674971, EPI_ISL_8676018, EPI_ISL_8676067-8676096, EPI_ISL_10678677, EPI_ISL_10834506, EPI_ISL_11096096, EPI_ISL_11173595, EPI_ISL_11323484, EPI_ISL_12293481-12293484, EPI_ISL_12293486-12293532, EPI_ISL_12293534-12293546, EPI_ISL_12293548, EPI_ISL_12293550-12293572, EPI_ISL_12293574-12293575, EPI_ISL_12293577-12293584, EPI_ISL_12293586-12293607, EPI_ISL_12293609-12293622, EPI_ISL_12293624-12293633, EPI_ISL_12367081, EPI_ISL_12623196-12623201, EPI_ISL_12623203-12623204, EPI_ISL_12623206-12623212, EPI_ISL_12623214-12623218, EPI_ISL_12623220-12623227, EPI_ISL_12623229-12623249, EPI_ISL_12623251-12623252, EPI_ISL_12623254-12623255, EPI_ISL_12623257-12623258, EPI_ISL_12623260-12623263, EPI_ISL_12623266-12623268, EPI_ISL_12623270, EPI_ISL_12771908, EPI_ISL_12784697-12784707, EPI_ISL_12784709-12784711, EPI_ISL_12784713-12784742, EPI_ISL_12784744-12784754, EPI_ISL_12784756-12784766, EPI_ISL_12784768-12784839, EPI_ISL_12784841-12784849, EPI_ISL_12784851-12784863, EPI_ISL_12911909-12911910, EPI_ISL_13773508-13773509, EPI_ISL_13773511-13773512, EPI_ISL_13773515, EPI_ISL_13773518, EPI_ISL_13773520-13773521, EPI_ISL_13774111, EPI_ISL_13774114-13774115, EPI_ISL_13774118-13774119, EPI_ISL_13774121, EPI_ISL_13774123, EPI_ISL_13774127-13774132, EPI_ISL_13774135-13774136, EPI_ISL_13774138-13774141, EPI_ISL_13778407, EPI_ISL_13778414, EPI_ISL_13812267, EPI_ISL_14253453, EPI_ISL_14643049, EPI_ISL_14643088-14643093, EPI_ISL_16680115, EPI_ISL_16762665, EPI_ISL_17546505, EPI_ISL_18032876, EPI_ISL_18936208, EPI_ISL_18936222")
############################################################################################################################################################################
MOV = stringlist_to_strings_set("EPI_ISL_12080356, EPI_ISL_1092344, EPI_ISL_2968466, EPI_ISL_3304784, EPI_ISL_3326806, EPI_ISL_3464544, EPI_ISL_3758018, EPI_ISL_4012065, EPI_ISL_4171202, EPI_ISL_4201810, EPI_ISL_4536418, EPI_ISL_5115827, EPI_ISL_5511463, EPI_ISL_5581788, EPI_ISL_6082708, EPI_ISL_6162906, EPI_ISL_6298830, EPI_ISL_6831874, EPI_ISL_7472964, EPI_ISL_7614031, EPI_ISL_8243566, EPI_ISL_8261058, EPI_ISL_8401858, EPI_ISL_9292221, EPI_ISL_9352281, EPI_ISL_9382677, EPI_ISL_9560035, EPI_ISL_9564988, EPI_ISL_9775279, EPI_ISL_9961511, EPI_ISL_10012883, EPI_ISL_10127751, EPI_ISL_10213024, EPI_ISL_10280217, EPI_ISL_10441882, EPI_ISL_10512732, EPI_ISL_10513884, EPI_ISL_10625202, EPI_ISL_10651538, EPI_ISL_10654545, EPI_ISL_10668128, EPI_ISL_10738796, EPI_ISL_10766361, EPI_ISL_10896078, EPI_ISL_10910076, EPI_ISL_11022200, EPI_ISL_11030076, EPI_ISL_11030366, EPI_ISL_11050901, EPI_ISL_11065112, EPI_ISL_11084474, EPI_ISL_11174220, EPI_ISL_11224116, EPI_ISL_11230018, EPI_ISL_11255409, EPI_ISL_11282729, EPI_ISL_11353416, EPI_ISL_11416915, EPI_ISL_11442788, EPI_ISL_11514566, EPI_ISL_11621053, EPI_ISL_11630027, EPI_ISL_11631155, EPI_ISL_11664910, EPI_ISL_11782428, EPI_ISL_11855687, EPI_ISL_11855713, EPI_ISL_11879008, EPI_ISL_11892804, EPI_ISL_11956947, EPI_ISL_12014556, EPI_ISL_12043591, EPI_ISL_12107836, EPI_ISL_12180755, EPI_ISL_12252373, EPI_ISL_12260199, EPI_ISL_12338023, EPI_ISL_12352282, EPI_ISL_12391660, EPI_ISL_12447732, EPI_ISL_12456735, EPI_ISL_12486036, EPI_ISL_12559054, EPI_ISL_12582161, EPI_ISL_12583464, EPI_ISL_12601110, EPI_ISL_12626284, EPI_ISL_12626387, EPI_ISL_12630743, EPI_ISL_12636130, EPI_ISL_12640988, EPI_ISL_12666444, EPI_ISL_12666642, EPI_ISL_12674754, EPI_ISL_12681076, EPI_ISL_12830302, EPI_ISL_12837375, EPI_ISL_12845916, EPI_ISL_12849707, EPI_ISL_12850240, EPI_ISL_12883279, EPI_ISL_12905490, EPI_ISL_12933035, EPI_ISL_12943505, EPI_ISL_12980006, EPI_ISL_13037364, EPI_ISL_13056045, EPI_ISL_13087460, EPI_ISL_13106533, EPI_ISL_13107201, EPI_ISL_13214811, EPI_ISL_13217407, EPI_ISL_13261618, EPI_ISL_13274125, EPI_ISL_13274357, EPI_ISL_13276187, EPI_ISL_13276436, EPI_ISL_13289724, EPI_ISL_13291196, EPI_ISL_13294593, EPI_ISL_13358749, EPI_ISL_13358801, EPI_ISL_13358804, EPI_ISL_13376289, EPI_ISL_13390266, EPI_ISL_13449336, EPI_ISL_13458696, EPI_ISL_13464576, EPI_ISL_13465017, EPI_ISL_13467791, EPI_ISL_13496740, EPI_ISL_13498905, EPI_ISL_13501768, EPI_ISL_13577993, EPI_ISL_13613050, EPI_ISL_13614366, EPI_ISL_13614367, EPI_ISL_13614401, EPI_ISL_13614610, EPI_ISL_13691164, EPI_ISL_13694672, EPI_ISL_13709091, EPI_ISL_13710220, EPI_ISL_13723805, EPI_ISL_13734677, EPI_ISL_13737005, EPI_ISL_13745743, EPI_ISL_13763841, EPI_ISL_13859056, EPI_ISL_13867663, EPI_ISL_13884501, EPI_ISL_13884648, EPI_ISL_13890658, EPI_ISL_13892609, EPI_ISL_13893030, EPI_ISL_13907925, EPI_ISL_13947241, EPI_ISL_13968198, EPI_ISL_13979237, EPI_ISL_13992801, EPI_ISL_14001467, EPI_ISL_14016773, EPI_ISL_14021631, EPI_ISL_14030106, EPI_ISL_14069091, EPI_ISL_14120400, EPI_ISL_14136918, EPI_ISL_14138850, EPI_ISL_14163050, EPI_ISL_14179475, EPI_ISL_14189255, EPI_ISL_14218792, EPI_ISL_14251033, EPI_ISL_14278233, EPI_ISL_14285562, EPI_ISL_14285646, EPI_ISL_14298694, EPI_ISL_14312503, EPI_ISL_14346370, EPI_ISL_14377405, EPI_ISL_14382457, EPI_ISL_14388832, EPI_ISL_14409577, EPI_ISL_14429354, EPI_ISL_14429373, EPI_ISL_14466720, EPI_ISL_14466939, EPI_ISL_14467031, EPI_ISL_14509715, EPI_ISL_14516309, EPI_ISL_14526287, EPI_ISL_14536391, EPI_ISL_14541651, EPI_ISL_14548126, EPI_ISL_14548186, EPI_ISL_14548996, EPI_ISL_14573187, EPI_ISL_14580651, EPI_ISL_14589022, EPI_ISL_14601448, EPI_ISL_14616144, EPI_ISL_14617298, EPI_ISL_14620632, EPI_ISL_14636356, EPI_ISL_14654283, EPI_ISL_14667656, EPI_ISL_14709471, EPI_ISL_14716868, EPI_ISL_14717521, EPI_ISL_14788088, EPI_ISL_14789011, EPI_ISL_14794264, EPI_ISL_14804388, EPI_ISL_14809350, EPI_ISL_14810089, EPI_ISL_14818681, EPI_ISL_14830201, EPI_ISL_14835350, EPI_ISL_14913266, EPI_ISL_14916287, EPI_ISL_14916417, EPI_ISL_14916471, EPI_ISL_14924994, EPI_ISL_14929774, EPI_ISL_14930743, EPI_ISL_14932144, EPI_ISL_14937861, EPI_ISL_14945818, EPI_ISL_14946908, EPI_ISL_14952616, EPI_ISL_14989193, EPI_ISL_15010302, EPI_ISL_15020116, EPI_ISL_15077793, EPI_ISL_15081472, EPI_ISL_15081501, EPI_ISL_15090054, EPI_ISL_15098796, EPI_ISL_15106590, EPI_ISL_15106739, EPI_ISL_15106758, EPI_ISL_15111369, EPI_ISL_15111552, EPI_ISL_15113210, EPI_ISL_15118530, EPI_ISL_15118652, EPI_ISL_15118683, EPI_ISL_15119550, EPI_ISL_15119586, EPI_ISL_15119587, EPI_ISL_15119798, EPI_ISL_15119826, EPI_ISL_15119830, EPI_ISL_15119899, EPI_ISL_15120114, EPI_ISL_15120173, EPI_ISL_15125352, EPI_ISL_15145266, EPI_ISL_15150344, EPI_ISL_15157642, EPI_ISL_15160596, EPI_ISL_15165861, EPI_ISL_15166985, EPI_ISL_15167502, EPI_ISL_15176036, EPI_ISL_15185400, EPI_ISL_15185557, EPI_ISL_15185627, EPI_ISL_15192815, EPI_ISL_15192964, EPI_ISL_15212355, EPI_ISL_15212402, EPI_ISL_15228995, EPI_ISL_15240404, EPI_ISL_15248977, EPI_ISL_15258694, EPI_ISL_15258697, EPI_ISL_15258702, EPI_ISL_15258714, EPI_ISL_15258754, EPI_ISL_15260772, EPI_ISL_15260783, EPI_ISL_15268354, EPI_ISL_15270718, EPI_ISL_15276586, EPI_ISL_15276610, EPI_ISL_15292147, EPI_ISL_15301009, EPI_ISL_15301025, EPI_ISL_15341577, EPI_ISL_15343208, EPI_ISL_15343310, EPI_ISL_15352141, EPI_ISL_15354679, EPI_ISL_15356322, EPI_ISL_15357173, EPI_ISL_15357186, EPI_ISL_15357564, EPI_ISL_15377262, EPI_ISL_15385709, EPI_ISL_15385769, EPI_ISL_15386088, EPI_ISL_15395718, EPI_ISL_15410430, EPI_ISL_15416260, EPI_ISL_15416332, EPI_ISL_15422263, EPI_ISL_15428954, EPI_ISL_15471118, EPI_ISL_15471127, EPI_ISL_15471292, EPI_ISL_15472544, EPI_ISL_15475787, EPI_ISL_15476096, EPI_ISL_15476838, EPI_ISL_15476890, EPI_ISL_15477002, EPI_ISL_15477006, EPI_ISL_15477014, EPI_ISL_15477080, EPI_ISL_15477087, EPI_ISL_15477089, EPI_ISL_15477122, EPI_ISL_15477123, EPI_ISL_15477124, EPI_ISL_15477133, EPI_ISL_15477167, EPI_ISL_15488495, EPI_ISL_15488731, EPI_ISL_15489332, EPI_ISL_15494887, EPI_ISL_15523458, EPI_ISL_15531670, EPI_ISL_15532754, EPI_ISL_15532755, EPI_ISL_15532764, EPI_ISL_15532776, EPI_ISL_15532778, EPI_ISL_15533551, EPI_ISL_15541106, EPI_ISL_15546710, EPI_ISL_15546790, EPI_ISL_15578381, EPI_ISL_15581363, EPI_ISL_15581659, EPI_ISL_15582663, EPI_ISL_15609973, EPI_ISL_15610206, EPI_ISL_15614395, EPI_ISL_15616465, EPI_ISL_15617522, EPI_ISL_15627130, EPI_ISL_15629811, EPI_ISL_15641087, EPI_ISL_15642469, EPI_ISL_15652740, EPI_ISL_15653305, EPI_ISL_15666161, EPI_ISL_15670984, EPI_ISL_15673084, EPI_ISL_15673196, EPI_ISL_15673587, EPI_ISL_15673829, EPI_ISL_15693069, EPI_ISL_15693720, EPI_ISL_15697288, EPI_ISL_15718497, EPI_ISL_15722243, EPI_ISL_15729046, EPI_ISL_15729147, EPI_ISL_15731617, EPI_ISL_15731631, EPI_ISL_15731918, EPI_ISL_15732415, EPI_ISL_15734089, EPI_ISL_15736020, EPI_ISL_15741410, EPI_ISL_15749655, EPI_ISL_15755946, EPI_ISL_15755997, EPI_ISL_15756004, EPI_ISL_15756044, EPI_ISL_15756058, EPI_ISL_15758276, EPI_ISL_15765732, EPI_ISL_15797427, EPI_ISL_15797464, EPI_ISL_15797751, EPI_ISL_15797823, EPI_ISL_15797872, EPI_ISL_15803022, EPI_ISL_15803045, EPI_ISL_15805129, EPI_ISL_15808072, EPI_ISL_15820336, EPI_ISL_15820414, EPI_ISL_15820574, EPI_ISL_15825048, EPI_ISL_15825627, EPI_ISL_15829108, EPI_ISL_15829598, EPI_ISL_15830052, EPI_ISL_15838528, EPI_ISL_15857468, EPI_ISL_15865651, EPI_ISL_15870196, EPI_ISL_15870952, EPI_ISL_15871047, EPI_ISL_15871064, EPI_ISL_15875255, EPI_ISL_15876857, EPI_ISL_15891417, EPI_ISL_15894376, EPI_ISL_15895664, EPI_ISL_15895805, EPI_ISL_15895830, EPI_ISL_15896932, EPI_ISL_15896947, EPI_ISL_15898266, EPI_ISL_15905654, EPI_ISL_15906500, EPI_ISL_15907744, EPI_ISL_15908757, EPI_ISL_15910110, EPI_ISL_15910797, EPI_ISL_15926723, EPI_ISL_15938365, EPI_ISL_15938543, EPI_ISL_15940018, EPI_ISL_15941958, EPI_ISL_15955458, EPI_ISL_15955461, EPI_ISL_15956078, EPI_ISL_15956334, EPI_ISL_15957375, EPI_ISL_15958000, EPI_ISL_15958880, EPI_ISL_15959194, EPI_ISL_15979054, EPI_ISL_15979220, EPI_ISL_15980444, EPI_ISL_15980454, EPI_ISL_15980534, EPI_ISL_15983623, EPI_ISL_16004112, EPI_ISL_16004139, EPI_ISL_16004173, EPI_ISL_16004201, EPI_ISL_16004221, EPI_ISL_16004267, EPI_ISL_16008150, EPI_ISL_16008489, EPI_ISL_16008877, EPI_ISL_16018930, EPI_ISL_16026658, EPI_ISL_16036592, EPI_ISL_16037058, EPI_ISL_16046816, EPI_ISL_16048672, EPI_ISL_16050127, EPI_ISL_16050317, EPI_ISL_16050568, EPI_ISL_16054786, EPI_ISL_16058939, EPI_ISL_16059102, EPI_ISL_16059617, EPI_ISL_16070082, EPI_ISL_16072907, EPI_ISL_16074147, EPI_ISL_16077673, EPI_ISL_16077721, EPI_ISL_16077819, EPI_ISL_16077827, EPI_ISL_16078136, EPI_ISL_16081297, EPI_ISL_16092096, EPI_ISL_16093705, EPI_ISL_16101357, EPI_ISL_16119461, EPI_ISL_16119652, EPI_ISL_16119687, EPI_ISL_16119697, EPI_ISL_16121748, EPI_ISL_16121818, EPI_ISL_16131605, EPI_ISL_16133519, EPI_ISL_16133537, EPI_ISL_16133850, EPI_ISL_16134022, EPI_ISL_16137724, EPI_ISL_16138501, EPI_ISL_16151550, EPI_ISL_16172206, EPI_ISL_16172628, EPI_ISL_16172642, EPI_ISL_16173251, EPI_ISL_16175780, EPI_ISL_16191277, EPI_ISL_16193751, EPI_ISL_16196595, EPI_ISL_16197780, EPI_ISL_16205170, EPI_ISL_16207275, EPI_ISL_16208574, EPI_ISL_16210148, EPI_ISL_16210278, EPI_ISL_16210314, EPI_ISL_16210315, EPI_ISL_16210321, EPI_ISL_16210344, EPI_ISL_16210352, EPI_ISL_16210356, EPI_ISL_16219790, EPI_ISL_16226994, EPI_ISL_16230801, EPI_ISL_16243417, EPI_ISL_16246433, EPI_ISL_16246605, EPI_ISL_16257294, EPI_ISL_16267849, EPI_ISL_16272927, EPI_ISL_16273053, EPI_ISL_16273094, EPI_ISL_16273106, EPI_ISL_16273691, EPI_ISL_16273904, EPI_ISL_16279440, EPI_ISL_16279922, EPI_ISL_16285032, EPI_ISL_16285249, EPI_ISL_16287235, EPI_ISL_16289354, EPI_ISL_16290866, EPI_ISL_16297102, EPI_ISL_16298573, EPI_ISL_16298574, EPI_ISL_16315643, EPI_ISL_16315645, EPI_ISL_16315710, EPI_ISL_16332693, EPI_ISL_16341231, EPI_ISL_16342178, EPI_ISL_16342216, EPI_ISL_16342762, EPI_ISL_16356910, EPI_ISL_16359576, EPI_ISL_16359648, EPI_ISL_16359676, EPI_ISL_16359695, EPI_ISL_16359707, EPI_ISL_16359708, EPI_ISL_16359709, EPI_ISL_16359721, EPI_ISL_16359724, EPI_ISL_16359730, EPI_ISL_16359737, EPI_ISL_16359739, EPI_ISL_16360495, EPI_ISL_16360560, EPI_ISL_16360578, EPI_ISL_16360615, EPI_ISL_16360631, EPI_ISL_16364267, EPI_ISL_16364779, EPI_ISL_16366337, EPI_ISL_16374061, EPI_ISL_16374062, EPI_ISL_16374063, EPI_ISL_16374064, EPI_ISL_16374077, EPI_ISL_16374103, EPI_ISL_16378479, EPI_ISL_16386434, EPI_ISL_16386695, EPI_ISL_16391752, EPI_ISL_16394844, EPI_ISL_16394849, EPI_ISL_16394918, EPI_ISL_16419696, EPI_ISL_16433986, EPI_ISL_16434219, EPI_ISL_16434682, EPI_ISL_16435411, EPI_ISL_16435758, EPI_ISL_16435776, EPI_ISL_16435834, EPI_ISL_16435848, EPI_ISL_16436045, EPI_ISL_16448121, EPI_ISL_16449474, EPI_ISL_16455847, EPI_ISL_16466074, EPI_ISL_16468471, EPI_ISL_16474400, EPI_ISL_16480279, EPI_ISL_16482060, EPI_ISL_16491962, EPI_ISL_16493846, EPI_ISL_16493881, EPI_ISL_16494105, EPI_ISL_16495627, EPI_ISL_16495643, EPI_ISL_16507760, EPI_ISL_16521266, EPI_ISL_16521681, EPI_ISL_16522889, EPI_ISL_16523533, EPI_ISL_16523552, EPI_ISL_16523565, EPI_ISL_16523595, EPI_ISL_16527181, EPI_ISL_16527529, EPI_ISL_16535383, EPI_ISL_16542512, EPI_ISL_16546118, EPI_ISL_16548794, EPI_ISL_16554670, EPI_ISL_16557447, EPI_ISL_16559981, EPI_ISL_16568182, EPI_ISL_16568197, EPI_ISL_16570236, EPI_ISL_16570568, EPI_ISL_16570602, EPI_ISL_16579916, EPI_ISL_16581363, EPI_ISL_16581388, EPI_ISL_16581425, EPI_ISL_16581431, EPI_ISL_16581432, EPI_ISL_16581578, EPI_ISL_16585008, EPI_ISL_16585433, EPI_ISL_16593038, EPI_ISL_16607627, EPI_ISL_16611238, EPI_ISL_16611373, EPI_ISL_16611412, EPI_ISL_16611422, EPI_ISL_16611506, EPI_ISL_16611514, EPI_ISL_16611550, EPI_ISL_16611563, EPI_ISL_16611624, EPI_ISL_16611667, EPI_ISL_16611673, EPI_ISL_16611682, EPI_ISL_16611698, EPI_ISL_16611778, EPI_ISL_16611838, EPI_ISL_16611841, EPI_ISL_16611848, EPI_ISL_16611855, EPI_ISL_16612037, EPI_ISL_16616330, EPI_ISL_16617719, EPI_ISL_16617775, EPI_ISL_16633089, EPI_ISL_16633129, EPI_ISL_16639468, EPI_ISL_16639472, EPI_ISL_16641545, EPI_ISL_16645057, EPI_ISL_16645120, EPI_ISL_16664013, EPI_ISL_16675393, EPI_ISL_16678683, EPI_ISL_16691130, EPI_ISL_16691138, EPI_ISL_16691144, EPI_ISL_16691244, EPI_ISL_16702617, EPI_ISL_16702731, EPI_ISL_16711169, EPI_ISL_16716050, EPI_ISL_16716112, EPI_ISL_16716666, EPI_ISL_16716667, EPI_ISL_16717190, EPI_ISL_16723860, EPI_ISL_16729430, EPI_ISL_16742981, EPI_ISL_16746019, EPI_ISL_16783884, EPI_ISL_16806139, EPI_ISL_16824117, EPI_ISL_16825886, EPI_ISL_16825887, EPI_ISL_16825902, EPI_ISL_16825911, EPI_ISL_16825973, EPI_ISL_16825975, EPI_ISL_16826011, EPI_ISL_16826162, EPI_ISL_16827461, EPI_ISL_16829544, EPI_ISL_16830824, EPI_ISL_16831676, EPI_ISL_16832813, EPI_ISL_16836551, EPI_ISL_16840631, EPI_ISL_16846006, EPI_ISL_16850006, EPI_ISL_16850282, EPI_ISL_16856565, EPI_ISL_16886081, EPI_ISL_16887730, EPI_ISL_16887870, EPI_ISL_16888824, EPI_ISL_16889192, EPI_ISL_16900219, EPI_ISL_16901108, EPI_ISL_16902309, EPI_ISL_16902361, EPI_ISL_16902363, EPI_ISL_16902379, EPI_ISL_16904536, EPI_ISL_16909958, EPI_ISL_16910033, EPI_ISL_16910057, EPI_ISL_16910058, EPI_ISL_16910139, EPI_ISL_16910202, EPI_ISL_16910382, EPI_ISL_16910409, EPI_ISL_16912192, EPI_ISL_16912224, EPI_ISL_16912699, EPI_ISL_16925018, EPI_ISL_16931898, EPI_ISL_16939136, EPI_ISL_16939502, EPI_ISL_16939509, EPI_ISL_16939520, EPI_ISL_16947370, EPI_ISL_16949310, EPI_ISL_16949324, EPI_ISL_16949325, EPI_ISL_16949387, EPI_ISL_16949791, EPI_ISL_16950012, EPI_ISL_16950629, EPI_ISL_16953777, EPI_ISL_16953781, EPI_ISL_16953798, EPI_ISL_16953799, EPI_ISL_16953818, EPI_ISL_16953822, EPI_ISL_16953828, EPI_ISL_16953830, EPI_ISL_16953838, EPI_ISL_16953839, EPI_ISL_16953840, EPI_ISL_16953847, EPI_ISL_16953922, EPI_ISL_16954432, EPI_ISL_16957344, EPI_ISL_16957451, EPI_ISL_16964941, EPI_ISL_16969564, EPI_ISL_16970275, EPI_ISL_16971354, EPI_ISL_16973054, EPI_ISL_16973491, EPI_ISL_16973514, EPI_ISL_16974198, EPI_ISL_16974315, EPI_ISL_16974382, EPI_ISL_16974837, EPI_ISL_16976164, EPI_ISL_16977847, EPI_ISL_16983895, EPI_ISL_16993160, EPI_ISL_16994359, EPI_ISL_16994360, EPI_ISL_16994528, EPI_ISL_16994534, EPI_ISL_16994641, EPI_ISL_16994642, EPI_ISL_16994643, EPI_ISL_16994655, EPI_ISL_16997635, EPI_ISL_17000779, EPI_ISL_17001818, EPI_ISL_17013912, EPI_ISL_17013920, EPI_ISL_17014551, EPI_ISL_17014690, EPI_ISL_17015386, EPI_ISL_17015403, EPI_ISL_17015508, EPI_ISL_17024862, EPI_ISL_17027606, EPI_ISL_17027639, EPI_ISL_17027948, EPI_ISL_17027949, EPI_ISL_17028105, EPI_ISL_17028190, EPI_ISL_17029568, EPI_ISL_17029620, EPI_ISL_17040153, EPI_ISL_17040170, EPI_ISL_17040185, EPI_ISL_17042013, EPI_ISL_17046849, EPI_ISL_17047075, EPI_ISL_17047300, EPI_ISL_17053152, EPI_ISL_17053374, EPI_ISL_17058072, EPI_ISL_17058073, EPI_ISL_17062470, EPI_ISL_17062513, EPI_ISL_17068606, EPI_ISL_17076266, EPI_ISL_17079912, EPI_ISL_17089619, EPI_ISL_17090505, EPI_ISL_17090748, EPI_ISL_17091144, EPI_ISL_17092036, EPI_ISL_17092037, EPI_ISL_17092038, EPI_ISL_17120889, EPI_ISL_17124976, EPI_ISL_17131649, EPI_ISL_17144768, EPI_ISL_17146391, EPI_ISL_17150326, EPI_ISL_17150374, EPI_ISL_17150380, EPI_ISL_17154893, EPI_ISL_17156691, EPI_ISL_17162825, EPI_ISL_17162929, EPI_ISL_17163260, EPI_ISL_17164036, EPI_ISL_17164425, EPI_ISL_17168296, EPI_ISL_17171052, EPI_ISL_17171054, EPI_ISL_17171074, EPI_ISL_17173504, EPI_ISL_17180537, EPI_ISL_17185359, EPI_ISL_17185391, EPI_ISL_17185393, EPI_ISL_17185521, EPI_ISL_17187061, EPI_ISL_17187082, EPI_ISL_17187115, EPI_ISL_17187405, EPI_ISL_17190415, EPI_ISL_17193320, EPI_ISL_17193409, EPI_ISL_17197288, EPI_ISL_17198749, EPI_ISL_17198796, EPI_ISL_17198959, EPI_ISL_17198971, EPI_ISL_17205507, EPI_ISL_17212042, EPI_ISL_17212254, EPI_ISL_17212450, EPI_ISL_17213648, EPI_ISL_17213677, EPI_ISL_17233016, EPI_ISL_17237758, EPI_ISL_17239970, EPI_ISL_17241376, EPI_ISL_17243067, EPI_ISL_17244428, EPI_ISL_17244430, EPI_ISL_17245777, EPI_ISL_17249116, EPI_ISL_17259971, EPI_ISL_17273231, EPI_ISL_17273546, EPI_ISL_17279072, EPI_ISL_17283738, EPI_ISL_17288373, EPI_ISL_17292738, EPI_ISL_17292778, EPI_ISL_17292812, EPI_ISL_17295274, EPI_ISL_17299688, EPI_ISL_17305358, EPI_ISL_17322295, EPI_ISL_17326431, EPI_ISL_17329306, EPI_ISL_17342779, EPI_ISL_17350941, EPI_ISL_17372713, EPI_ISL_17373645, EPI_ISL_17373894, EPI_ISL_17374733, EPI_ISL_17374799, EPI_ISL_17374806, EPI_ISL_17374807, EPI_ISL_17375007, EPI_ISL_17382768, EPI_ISL_17387866, EPI_ISL_17388895, EPI_ISL_17388906, EPI_ISL_17393458, EPI_ISL_17404084, EPI_ISL_17409157, EPI_ISL_17413069, EPI_ISL_17413882, EPI_ISL_17419568, EPI_ISL_17422869, EPI_ISL_17430511, EPI_ISL_17433569, EPI_ISL_17463210, EPI_ISL_17464836, EPI_ISL_17465459, EPI_ISL_17468440, EPI_ISL_17477106, EPI_ISL_17485616, EPI_ISL_17485727, EPI_ISL_17485728, EPI_ISL_17493590, EPI_ISL_17501569, EPI_ISL_17503268, EPI_ISL_17504112, EPI_ISL_17511739, EPI_ISL_17516925, EPI_ISL_17517664, EPI_ISL_17521432, EPI_ISL_17527442, EPI_ISL_17527471, EPI_ISL_17527478, EPI_ISL_17527567, EPI_ISL_17539348, EPI_ISL_17544585, EPI_ISL_17544916, EPI_ISL_17547750, EPI_ISL_17551316, EPI_ISL_17556166, EPI_ISL_17558242, EPI_ISL_17558487, EPI_ISL_17558870, EPI_ISL_17559934, EPI_ISL_17592407, EPI_ISL_17604727, EPI_ISL_17604844, EPI_ISL_17605096, EPI_ISL_17621484, EPI_ISL_17622137, EPI_ISL_17628855, EPI_ISL_17642930, EPI_ISL_17647324, EPI_ISL_17649109, EPI_ISL_17656014, EPI_ISL_17656052, EPI_ISL_17657007, EPI_ISL_17657210, EPI_ISL_17660818, EPI_ISL_17661045, EPI_ISL_17661237, EPI_ISL_17661261, EPI_ISL_17661270, EPI_ISL_17662899, EPI_ISL_17666331, EPI_ISL_17682891, EPI_ISL_17682942, EPI_ISL_17683117, EPI_ISL_17683214, EPI_ISL_17683243, EPI_ISL_17685027, EPI_ISL_17692258, EPI_ISL_17695126, EPI_ISL_17695180, EPI_ISL_17696632, EPI_ISL_17696673, EPI_ISL_17696719, EPI_ISL_17698434, EPI_ISL_17703866, EPI_ISL_17714932, EPI_ISL_17716296, EPI_ISL_17718464, EPI_ISL_17719091, EPI_ISL_17720695, EPI_ISL_17722014, EPI_ISL_17724271, EPI_ISL_17724304, EPI_ISL_17737041, EPI_ISL_17740508, EPI_ISL_17742125, EPI_ISL_17762760, EPI_ISL_17762820, EPI_ISL_17764084, EPI_ISL_17765632, EPI_ISL_17765706, EPI_ISL_17765717, EPI_ISL_17775797, EPI_ISL_17775798, EPI_ISL_17776606, EPI_ISL_17777067, EPI_ISL_17784540, EPI_ISL_17784548, EPI_ISL_17784550, EPI_ISL_17784552, EPI_ISL_17784569, EPI_ISL_17784593, EPI_ISL_17790241, EPI_ISL_17791006, EPI_ISL_17793597, EPI_ISL_17793709, EPI_ISL_17796704, EPI_ISL_17802162, EPI_ISL_17802448, EPI_ISL_17802471, EPI_ISL_17802863, EPI_ISL_17803152, EPI_ISL_17806624, EPI_ISL_17806780, EPI_ISL_17806785, EPI_ISL_17807899, EPI_ISL_17818059, EPI_ISL_17821941, EPI_ISL_17822275, EPI_ISL_17822422, EPI_ISL_17822859, EPI_ISL_17822940, EPI_ISL_17823538, EPI_ISL_17824267, EPI_ISL_17824292, EPI_ISL_17824608, EPI_ISL_17824670, EPI_ISL_17826031, EPI_ISL_17826225, EPI_ISL_17826355, EPI_ISL_17826392, EPI_ISL_17827176, EPI_ISL_17827221, EPI_ISL_17827261, EPI_ISL_17829319, EPI_ISL_17829398, EPI_ISL_17832368, EPI_ISL_17849659, EPI_ISL_17851798, EPI_ISL_17857957, EPI_ISL_17859873, EPI_ISL_17859887, EPI_ISL_17879222, EPI_ISL_17882742, EPI_ISL_17960272, EPI_ISL_17975460, EPI_ISL_17977889, EPI_ISL_17977890, EPI_ISL_17977939, EPI_ISL_17980375, EPI_ISL_17980376, EPI_ISL_17980377, EPI_ISL_17985070, EPI_ISL_17987531, EPI_ISL_17994215, EPI_ISL_17997273, EPI_ISL_17999525, EPI_ISL_17999535, EPI_ISL_17999631, EPI_ISL_17999795, EPI_ISL_17999836, EPI_ISL_18000297, EPI_ISL_18001169, EPI_ISL_18010323, EPI_ISL_18014681, EPI_ISL_18031645, EPI_ISL_18032375, EPI_ISL_18039667, EPI_ISL_18040500, EPI_ISL_18041690, EPI_ISL_18042063, EPI_ISL_18043809, EPI_ISL_18051511, EPI_ISL_18052776, EPI_ISL_18057695, EPI_ISL_18060832, EPI_ISL_18063266, EPI_ISL_18063275, EPI_ISL_18063343, EPI_ISL_18063346, EPI_ISL_18063361, EPI_ISL_18063384, EPI_ISL_18063710, EPI_ISL_18063741, EPI_ISL_18063779, EPI_ISL_18063898, EPI_ISL_18064050, EPI_ISL_18064055, EPI_ISL_18064103, EPI_ISL_18064142, EPI_ISL_18064155, EPI_ISL_18064235, EPI_ISL_18064253, EPI_ISL_18064299, EPI_ISL_18064307, EPI_ISL_18064328, EPI_ISL_18064337, EPI_ISL_18064383, EPI_ISL_18065323, EPI_ISL_18066648, EPI_ISL_18066669, EPI_ISL_18066701, EPI_ISL_18076272, EPI_ISL_18077730, EPI_ISL_18082776, EPI_ISL_18093842, EPI_ISL_18100370, EPI_ISL_18100403, EPI_ISL_18100404, EPI_ISL_18100416, EPI_ISL_18100417, EPI_ISL_18100418, EPI_ISL_18108242, EPI_ISL_18112575, EPI_ISL_18112668, EPI_ISL_18115399, EPI_ISL_18117671, EPI_ISL_18127619, EPI_ISL_18127909, EPI_ISL_18127961, EPI_ISL_18134274, EPI_ISL_18134756, EPI_ISL_18138526, EPI_ISL_18139339, EPI_ISL_18139659, EPI_ISL_18166414, EPI_ISL_18208967, EPI_ISL_18213578, EPI_ISL_18215762, EPI_ISL_18215772, EPI_ISL_18216065, EPI_ISL_18218555, EPI_ISL_18218775, EPI_ISL_18218935, EPI_ISL_18218936, EPI_ISL_18225292, EPI_ISL_18226914, EPI_ISL_18237064, EPI_ISL_18247242, EPI_ISL_18265252, EPI_ISL_18272809, EPI_ISL_18280058, EPI_ISL_18285922, EPI_ISL_18285953, EPI_ISL_18300222, EPI_ISL_18300587, EPI_ISL_18306489, EPI_ISL_18316786, EPI_ISL_18320556, EPI_ISL_18343894, EPI_ISL_18347703, EPI_ISL_18350437, EPI_ISL_18352397, EPI_ISL_18354145, EPI_ISL_18355510, EPI_ISL_18365833, EPI_ISL_18367086, EPI_ISL_18375185, EPI_ISL_18375538, EPI_ISL_18376234, EPI_ISL_18382930, EPI_ISL_18382993, EPI_ISL_18383001, EPI_ISL_18390814, EPI_ISL_18423785, EPI_ISL_18432517, EPI_ISL_18432566, EPI_ISL_18434659, EPI_ISL_18435860, EPI_ISL_18440660, EPI_ISL_18449837, EPI_ISL_18461692, EPI_ISL_18466251, EPI_ISL_18471046, EPI_ISL_18475421, EPI_ISL_18475423, EPI_ISL_18475424, EPI_ISL_18475534, EPI_ISL_18498513, EPI_ISL_18502004, EPI_ISL_18505899, EPI_ISL_18509397, EPI_ISL_18509398, EPI_ISL_18509399, EPI_ISL_18512167, EPI_ISL_18512168, EPI_ISL_18512169, EPI_ISL_18520761, EPI_ISL_18520956, EPI_ISL_18551236, EPI_ISL_18551245, EPI_ISL_18566482, EPI_ISL_18566696, EPI_ISL_18575585, EPI_ISL_18623910, EPI_ISL_18624229, EPI_ISL_18624532, EPI_ISL_18633916, EPI_ISL_18639764, EPI_ISL_18650558, EPI_ISL_18650648, EPI_ISL_18662705, EPI_ISL_18687076, EPI_ISL_18687807, EPI_ISL_18687813, EPI_ISL_18687815, EPI_ISL_18697820, EPI_ISL_18711358, EPI_ISL_18712220, EPI_ISL_18715624, EPI_ISL_18715897, EPI_ISL_18720533, EPI_ISL_18720935, EPI_ISL_18730535, EPI_ISL_18742836, EPI_ISL_18743070, EPI_ISL_18744493, EPI_ISL_18747739, EPI_ISL_18749619, EPI_ISL_18751663, EPI_ISL_18751845, EPI_ISL_18751862, EPI_ISL_18758080, EPI_ISL_18760281, EPI_ISL_18760288, EPI_ISL_18760322, EPI_ISL_18792280, EPI_ISL_18792992, EPI_ISL_18793204, EPI_ISL_18798820, EPI_ISL_18799141, EPI_ISL_18843955, EPI_ISL_18857091, EPI_ISL_18877237, EPI_ISL_18878774, EPI_ISL_18886497, EPI_ISL_18895660, EPI_ISL_18904633, EPI_ISL_18904635, EPI_ISL_18922726, EPI_ISL_18924174, EPI_ISL_18931648, EPI_ISL_18931705, EPI_ISL_18931859, EPI_ISL_18932507, EPI_ISL_18932551, EPI_ISL_18960210, EPI_ISL_18967490, EPI_ISL_18992540, EPI_ISL_18994815, EPI_ISL_19005505, EPI_ISL_19009620, EPI_ISL_19009621, EPI_ISL_19009623, EPI_ISL_19009624, EPI_ISL_19015609, EPI_ISL_19021722, EPI_ISL_19024035, EPI_ISL_19024044, EPI_ISL_19024068, EPI_ISL_19024081, EPI_ISL_19037171, EPI_ISL_19050132, EPI_ISL_19054784, EPI_ISL_19057886, EPI_ISL_19059047, EPI_ISL_19071524, EPI_ISL_19080112, EPI_ISL_19082638, EPI_ISL_19084871, EPI_ISL_19084883, EPI_ISL_19085033, EPI_ISL_19085070, EPI_ISL_19085078, EPI_ISL_19085135, EPI_ISL_19085355, EPI_ISL_19096287, EPI_ISL_19108340, EPI_ISL_19122484, EPI_ISL_19135284, EPI_ISL_19140391, EPI_ISL_19150867, EPI_ISL_19150870, EPI_ISL_19168138, EPI_ISL_19169477, EPI_ISL_19169870, EPI_ISL_19169976, EPI_ISL_19175684, EPI_ISL_19180359, EPI_ISL_19180367, EPI_ISL_19197718, EPI_ISL_19205639, EPI_ISL_19226538, EPI_ISL_19239735, EPI_ISL_19269562, EPI_ISL_19269713, EPI_ISL_19281165, EPI_ISL_19289747, EPI_ISL_19292254, EPI_ISL_19298398, EPI_ISL_19300597, EPI_ISL_19305340, EPI_ISL_19308874, EPI_ISL_19308875, EPI_ISL_19308876, EPI_ISL_19308877, EPI_ISL_19323028, EPI_ISL_19328684, EPI_ISL_19333890, EPI_ISL_19337826, EPI_ISL_19340412, EPI_ISL_19340513, EPI_ISL_19340884, EPI_ISL_19363557, EPI_ISL_19369244, EPI_ISL_19374121, EPI_ISL_19374151, EPI_ISL_19374152, EPI_ISL_19374153, EPI_ISL_19374154, EPI_ISL_19377637, EPI_ISL_19377761, EPI_ISL_19377778, EPI_ISL_19383580, EPI_ISL_19383680, EPI_ISL_19392805, EPI_ISL_19405488, EPI_ISL_19427278, EPI_ISL_19434372, EPI_ISL_19444130, EPI_ISL_19472214, EPI_ISL_19473889, EPI_ISL_19506224, EPI_ISL_19515856, EPI_ISL_19517266, EPI_ISL_19522346, EPI_ISL_19526058, EPI_ISL_19526073, EPI_ISL_19526074, EPI_ISL_19527099, EPI_ISL_19531769, EPI_ISL_19531772, EPI_ISL_19531776, EPI_ISL_19534803, EPI_ISL_19555103, EPI_ISL_19572558, EPI_ISL_19585472, EPI_ISL_19588464, EPI_ISL_19591990, EPI_ISL_19620039, EPI_ISL_19697717, EPI_ISL_19707682, EPI_ISL_19750637, EPI_ISL_19778907, EPI_ISL_19783495, EPI_ISL_19783524, EPI_ISL_19788441, EPI_ISL_19799392, EPI_ISL_19799712, EPI_ISL_19799810, EPI_ISL_19800089, EPI_ISL_19800094, EPI_ISL_19800098, EPI_ISL_19816731, EPI_ISL_19852829, EPI_ISL_19856862, EPI_ISL_19878325, EPI_ISL_19913944, EPI_ISL_19914209, EPI_ISL_20048939, EPI_ISL_20056146, EPI_ISL_20067911, EPI_ISL_20076112, EPI_ISL_20084491, EPI_ISL_20133315")
Idaho = stringlist_to_strings_set("EPI_ISL_3588790, EPI_ISL_3923181, EPI_ISL_6117526, EPI_ISL_9352281, EPI_ISL_632847-632889, EPI_ISL_1164782-1164793, EPI_ISL_1164795-1164815, EPI_ISL_1190729-1190737, EPI_ISL_1219911-1219923, EPI_ISL_1363168-1363171, EPI_ISL_1363175-1363194, EPI_ISL_1366701-1366713, EPI_ISL_1405711-1405728, EPI_ISL_1492620-1492636, EPI_ISL_1510260-1510277, EPI_ISL_1532066-1532088, EPI_ISL_1627285-1627319, EPI_ISL_1696491-1696496, EPI_ISL_1806183, EPI_ISL_1806186, EPI_ISL_1806188, EPI_ISL_1806191, EPI_ISL_1806194, EPI_ISL_1806197, EPI_ISL_1806200, EPI_ISL_1806203, EPI_ISL_1806206, EPI_ISL_1806209, EPI_ISL_1806212, EPI_ISL_1806215, EPI_ISL_1806218, EPI_ISL_1806220, EPI_ISL_1806223, EPI_ISL_1806226, EPI_ISL_1806229, EPI_ISL_1806232, EPI_ISL_1806235, EPI_ISL_1806238, EPI_ISL_1806241, EPI_ISL_1806244, EPI_ISL_1806247, EPI_ISL_1806250, EPI_ISL_1806253, EPI_ISL_1806256, EPI_ISL_1806259, EPI_ISL_1806262, EPI_ISL_1806265, EPI_ISL_1806268, EPI_ISL_1806270, EPI_ISL_1806274, EPI_ISL_1806277, EPI_ISL_1806280, EPI_ISL_1806283, EPI_ISL_1806286, EPI_ISL_1806289, EPI_ISL_1806292, EPI_ISL_1806296, EPI_ISL_1806298, EPI_ISL_1806301, EPI_ISL_1806305, EPI_ISL_1806307, EPI_ISL_1806310, EPI_ISL_1806313, EPI_ISL_1806317, EPI_ISL_1806320, EPI_ISL_1806323, EPI_ISL_1806326, EPI_ISL_1806329, EPI_ISL_1806332, EPI_ISL_1806335, EPI_ISL_1806338, EPI_ISL_1806341, EPI_ISL_1806345, EPI_ISL_1806347, EPI_ISL_1825119-1825140, EPI_ISL_1969000, EPI_ISL_2031593-2031635, EPI_ISL_2090629-2090647, EPI_ISL_2095693-2095700, EPI_ISL_2140147-2140169, EPI_ISL_2229207, EPI_ISL_2229209-2229210, EPI_ISL_2229212-2229215, EPI_ISL_2229217-2229218, EPI_ISL_2229220, EPI_ISL_2229222, EPI_ISL_2229224-2229225, EPI_ISL_2229227-2229228, EPI_ISL_2229230-2229231, EPI_ISL_2229233-2229234, EPI_ISL_2598120-2598134, EPI_ISL_2689737-2689754, EPI_ISL_2834621-2834678, EPI_ISL_2934924-2934987, EPI_ISL_2976189-2976299, EPI_ISL_3010500-3010583, EPI_ISL_3052920, EPI_ISL_3064153-3064241, EPI_ISL_3066910-3066987, EPI_ISL_3086849-3086850, EPI_ISL_3090099, EPI_ISL_3090757-3090848, EPI_ISL_3098710, EPI_ISL_3098719, EPI_ISL_3098762, EPI_ISL_3100632-3100677, EPI_ISL_3156454-3156546, EPI_ISL_3164497, EPI_ISL_3190942-3191016, EPI_ISL_3191021, EPI_ISL_3191024, EPI_ISL_3191027, EPI_ISL_3191029-3191030, EPI_ISL_3191035-3191044, EPI_ISL_3217177-3217269, EPI_ISL_3242917-3242999, EPI_ISL_3272351-3272449, EPI_ISL_3272452, EPI_ISL_3272454, EPI_ISL_3272457, EPI_ISL_3272460, EPI_ISL_3272463, EPI_ISL_3272466, EPI_ISL_3272469, EPI_ISL_3272471, EPI_ISL_3272474, EPI_ISL_3272477, EPI_ISL_3272480, EPI_ISL_3272483, EPI_ISL_3272485, EPI_ISL_3272489, EPI_ISL_3272491, EPI_ISL_3272494, EPI_ISL_3272497, EPI_ISL_3272500, EPI_ISL_3272503, EPI_ISL_3272506, EPI_ISL_3272509, EPI_ISL_3272512, EPI_ISL_3272515, EPI_ISL_3272518, EPI_ISL_3272521, EPI_ISL_3272523, EPI_ISL_3272527, EPI_ISL_3272530, EPI_ISL_3272972, EPI_ISL_3273142, EPI_ISL_3273147, EPI_ISL_3273175-3273176, EPI_ISL_3273180, EPI_ISL_3273187, EPI_ISL_3273193, EPI_ISL_3273206, EPI_ISL_3273222, EPI_ISL_3275054-3275138, EPI_ISL_3385856-3385941, EPI_ISL_3404610-3404730, EPI_ISL_3464593-3464683, EPI_ISL_3502770-3502844, EPI_ISL_3502887, EPI_ISL_3543363-3543450, EPI_ISL_3556822-3556911, EPI_ISL_3588721-3588813, EPI_ISL_3640689-3640701, EPI_ISL_3640705, EPI_ISL_3640709, EPI_ISL_3640713, EPI_ISL_3640717, EPI_ISL_3640720, EPI_ISL_3640722, EPI_ISL_3640726, EPI_ISL_3640730, EPI_ISL_3640734, EPI_ISL_3640738, EPI_ISL_3640742, EPI_ISL_3640747, EPI_ISL_3640752, EPI_ISL_3640755, EPI_ISL_3640760, EPI_ISL_3640765, EPI_ISL_3640769, EPI_ISL_3640773, EPI_ISL_3640777, EPI_ISL_3640781, EPI_ISL_3640784, EPI_ISL_3640788, EPI_ISL_3640792, EPI_ISL_3640796, EPI_ISL_3640800, EPI_ISL_3640804, EPI_ISL_3640809, EPI_ISL_3640814, EPI_ISL_3640818, EPI_ISL_3640823, EPI_ISL_3640827, EPI_ISL_3640831, EPI_ISL_3640835, EPI_ISL_3640839, EPI_ISL_3640844, EPI_ISL_3640849, EPI_ISL_3640853, EPI_ISL_3640857, EPI_ISL_3640862, EPI_ISL_3640866, EPI_ISL_3640870, EPI_ISL_3640874, EPI_ISL_3640878, EPI_ISL_3640883, EPI_ISL_3640887, EPI_ISL_3640892, EPI_ISL_3640896, EPI_ISL_3640900, EPI_ISL_3640904, EPI_ISL_3640909, EPI_ISL_3640911, EPI_ISL_3640915, EPI_ISL_3640919, EPI_ISL_3640923, EPI_ISL_3640927, EPI_ISL_3640932, EPI_ISL_3640937, EPI_ISL_3640942, EPI_ISL_3640947, EPI_ISL_3640953, EPI_ISL_3640958, EPI_ISL_3640965, EPI_ISL_3640971, EPI_ISL_3640976, EPI_ISL_3640981, EPI_ISL_3640986, EPI_ISL_3640991, EPI_ISL_3640996, EPI_ISL_3641001, EPI_ISL_3641007, EPI_ISL_3641012, EPI_ISL_3641017, EPI_ISL_3641022, EPI_ISL_3641027, EPI_ISL_3641032, EPI_ISL_3641037, EPI_ISL_3641042, EPI_ISL_3641046, EPI_ISL_3641051, EPI_ISL_3641056, EPI_ISL_3641062, EPI_ISL_3641066, EPI_ISL_3641071, EPI_ISL_3641076, EPI_ISL_3641080, EPI_ISL_3641084, EPI_ISL_3641088, EPI_ISL_3641092, EPI_ISL_3641097, EPI_ISL_3641103, EPI_ISL_3641108, EPI_ISL_3641113, EPI_ISL_3641118, EPI_ISL_3641123, EPI_ISL_3641128, EPI_ISL_3641133, EPI_ISL_3641138, EPI_ISL_3641144, EPI_ISL_3641148, EPI_ISL_3641153, EPI_ISL_3641157, EPI_ISL_3641162, EPI_ISL_3641167, EPI_ISL_3641171, EPI_ISL_3641176, EPI_ISL_3641181, EPI_ISL_3641185, EPI_ISL_3641191, EPI_ISL_3641195, EPI_ISL_3641200, EPI_ISL_3641204, EPI_ISL_3641209, EPI_ISL_3641212, EPI_ISL_3641217, EPI_ISL_3642831-3642867, EPI_ISL_3642872-3642921, EPI_ISL_3664290-3664293, EPI_ISL_3664321-3664328, EPI_ISL_3722342-3722427, EPI_ISL_3743004, EPI_ISL_3743006, EPI_ISL_3743008-3743009, EPI_ISL_3743011, EPI_ISL_3743013-3743014, EPI_ISL_3743016, EPI_ISL_3743018-3743019, EPI_ISL_3743021, EPI_ISL_3743023-3743024, EPI_ISL_3743026, EPI_ISL_3743028-3743029, EPI_ISL_3743031, EPI_ISL_3743033-3743034, EPI_ISL_3743037, EPI_ISL_3743039-3743040, EPI_ISL_3743042-3743044, EPI_ISL_3743046, EPI_ISL_3743048, EPI_ISL_3743050, EPI_ISL_3743053, EPI_ISL_3743056-3743057, EPI_ISL_3743059, EPI_ISL_3743061, EPI_ISL_3743063-3743064, EPI_ISL_3743066, EPI_ISL_3743068, EPI_ISL_3743070-3743071, EPI_ISL_3743074-3743075, EPI_ISL_3743077, EPI_ISL_3743080, EPI_ISL_3743083, EPI_ISL_3743085, EPI_ISL_3743088, EPI_ISL_3743090, EPI_ISL_3743092, EPI_ISL_3743095, EPI_ISL_3743098, EPI_ISL_3743101-3743102, EPI_ISL_3743104, EPI_ISL_3743106, EPI_ISL_3743108-3743109, EPI_ISL_3743111, EPI_ISL_3743113-3743114, EPI_ISL_3743116, EPI_ISL_3743118, EPI_ISL_3743120, EPI_ISL_3743122, EPI_ISL_3743125, EPI_ISL_3743128, EPI_ISL_3743130-3743131, EPI_ISL_3743133, EPI_ISL_3743135, EPI_ISL_3743137-3743138, EPI_ISL_3743140, EPI_ISL_3743142-3743143, EPI_ISL_3743145, EPI_ISL_3743147-3743148, EPI_ISL_3743150, EPI_ISL_3743152, EPI_ISL_3743154-3743155, EPI_ISL_3743157, EPI_ISL_3743159-3743160, EPI_ISL_3743162-3743163, EPI_ISL_3743165, EPI_ISL_3743167, EPI_ISL_3743169-3743170, EPI_ISL_3743172, EPI_ISL_3743174, EPI_ISL_3756615-3756699, EPI_ISL_3756838-3756923, EPI_ISL_3759954-3760044, EPI_ISL_3905577-3905658, EPI_ISL_3905666-3905748, EPI_ISL_3922775-3922943, EPI_ISL_3923011-3923095, EPI_ISL_3923164-3923189, EPI_ISL_3923232-3923337, EPI_ISL_4084621-4084700, EPI_ISL_4091126-4091212, EPI_ISL_4175470, EPI_ISL_4175472-4175473, EPI_ISL_4175475, EPI_ISL_4175477, EPI_ISL_4175479, EPI_ISL_4175481, EPI_ISL_4175483, EPI_ISL_4175485-4175486, EPI_ISL_4175488, EPI_ISL_4175490, EPI_ISL_4175492, EPI_ISL_4175494, EPI_ISL_4175496, EPI_ISL_4175498, EPI_ISL_4175500-4175501, EPI_ISL_4175503, EPI_ISL_4175505, EPI_ISL_4175507, EPI_ISL_4175509, EPI_ISL_4175511, EPI_ISL_4175513-4175514, EPI_ISL_4175516, EPI_ISL_4175518, EPI_ISL_4175520-4175521, EPI_ISL_4175524-4175525, EPI_ISL_4175527, EPI_ISL_4175529, EPI_ISL_4175531, EPI_ISL_4175533, EPI_ISL_4175535-4175536, EPI_ISL_4175538, EPI_ISL_4175540, EPI_ISL_4175542, EPI_ISL_4175544-4175545, EPI_ISL_4175547, EPI_ISL_4175549, EPI_ISL_4175551, EPI_ISL_4175553, EPI_ISL_4175555, EPI_ISL_4175557, EPI_ISL_4175559, EPI_ISL_4175561, EPI_ISL_4175563, EPI_ISL_4175565, EPI_ISL_4175567, EPI_ISL_4175569-4175570, EPI_ISL_4175572, EPI_ISL_4175574, EPI_ISL_4175576, EPI_ISL_4175578, EPI_ISL_4175580, EPI_ISL_4175582, EPI_ISL_4175584, EPI_ISL_4175586, EPI_ISL_4175588, EPI_ISL_4175590, EPI_ISL_4175592, EPI_ISL_4175594-4175595, EPI_ISL_4175597, EPI_ISL_4175599, EPI_ISL_4175601, EPI_ISL_4175603-4175604, EPI_ISL_4175606, EPI_ISL_4175608, EPI_ISL_4175610, EPI_ISL_4175612, EPI_ISL_4175614, EPI_ISL_4175616, EPI_ISL_4175618-4175619, EPI_ISL_4175621, EPI_ISL_4175623, EPI_ISL_4175625, EPI_ISL_4175627, EPI_ISL_4211457-4211476, EPI_ISL_4219243-4219259, EPI_ISL_4256841-4256887, EPI_ISL_4435380-4435467, EPI_ISL_4469547-4469582, EPI_ISL_4474929-4475022, EPI_ISL_4494135-4494208, EPI_ISL_4496487-4496542, EPI_ISL_4730623-4730627, EPI_ISL_4730633, EPI_ISL_4730636-4730641, EPI_ISL_4730645-4730648, EPI_ISL_4878312, EPI_ISL_4889660-4889729, EPI_ISL_4945672-4945674, EPI_ISL_4945679-4945735, EPI_ISL_4945737, EPI_ISL_4955745-4955746, EPI_ISL_4978374-4978449, EPI_ISL_5057881-5057962, EPI_ISL_5071787-5071831, EPI_ISL_5071834-5071881, EPI_ISL_5072680-5072681, EPI_ISL_5072683-5072684, EPI_ISL_5072687, EPI_ISL_5072689, EPI_ISL_5072691-5072692, EPI_ISL_5072694, EPI_ISL_5072696, EPI_ISL_5072698-5072699, EPI_ISL_5072701, EPI_ISL_5072703-5072704, EPI_ISL_5072706, EPI_ISL_5072708, EPI_ISL_5072710-5072711, EPI_ISL_5072713, EPI_ISL_5072715-5072716, EPI_ISL_5072718-5072719, EPI_ISL_5072721-5072722, EPI_ISL_5072724, EPI_ISL_5072726, EPI_ISL_5072728, EPI_ISL_5072730-5072731, EPI_ISL_5072733-5072734, EPI_ISL_5072736-5072738, EPI_ISL_5072740-5072741, EPI_ISL_5072743-5072744, EPI_ISL_5072746, EPI_ISL_5072748-5072749, EPI_ISL_5072751-5072752, EPI_ISL_5072754, EPI_ISL_5072756, EPI_ISL_5072759, EPI_ISL_5072762, EPI_ISL_5072767, EPI_ISL_5072769, EPI_ISL_5072771-5072772, EPI_ISL_5072774-5072775, EPI_ISL_5072777-5072780, EPI_ISL_5072782-5072783, EPI_ISL_5072785, EPI_ISL_5072787-5072788, EPI_ISL_5072790-5072791, EPI_ISL_5072793, EPI_ISL_5072795-5072796, EPI_ISL_5072798, EPI_ISL_5072800-5072801, EPI_ISL_5072803-5072804, EPI_ISL_5072806, EPI_ISL_5072808, EPI_ISL_5072810-5072811, EPI_ISL_5072813, EPI_ISL_5072815, EPI_ISL_5072817-5072818, EPI_ISL_5072820, EPI_ISL_5072822-5072823, EPI_ISL_5072825, EPI_ISL_5072827, EPI_ISL_5072829, EPI_ISL_5072831, EPI_ISL_5072833-5072834, EPI_ISL_5072836, EPI_ISL_5095655-5095734, EPI_ISL_5114936-5115013, EPI_ISL_5115092-5115134, EPI_ISL_5115136-5115172, EPI_ISL_5334248, EPI_ISL_5334587, EPI_ISL_5348547, EPI_ISL_5348554-5348555, EPI_ISL_5348560-5348561, EPI_ISL_5348566-5348567, EPI_ISL_5348572-5348573, EPI_ISL_5348575, EPI_ISL_5348582-5348583, EPI_ISL_5348591-5348592, EPI_ISL_5348595-5348596, EPI_ISL_5348599-5348600, EPI_ISL_5348605-5348606, EPI_ISL_5348612-5348613, EPI_ISL_5348619-5348620, EPI_ISL_5348628-5348629, EPI_ISL_5348636-5348637, EPI_ISL_5348643-5348644, EPI_ISL_5348649-5348650, EPI_ISL_5348657, EPI_ISL_5348662, EPI_ISL_5348666, EPI_ISL_5348673-5348674, EPI_ISL_5348679-5348680, EPI_ISL_5348687-5348688, EPI_ISL_5348692-5348693, EPI_ISL_5348702-5348704, EPI_ISL_5348708-5348709, EPI_ISL_5348713, EPI_ISL_5348720-5348721, EPI_ISL_5348728, EPI_ISL_5348732-5348733, EPI_ISL_5348739-5348740, EPI_ISL_5348746-5348748, EPI_ISL_5348757-5348758, EPI_ISL_5348763, EPI_ISL_5348769-5348770, EPI_ISL_5348774-5348775, EPI_ISL_5348782-5348783, EPI_ISL_5348787-5348788, EPI_ISL_5441241-5441315, EPI_ISL_5511689-5511695, EPI_ISL_5696764-5696843, EPI_ISL_5845388, EPI_ISL_5850374-5850416, EPI_ISL_5850421-5850446, EPI_ISL_5851328-5851417, EPI_ISL_5851420-5851498, EPI_ISL_5851502-5851603, EPI_ISL_5851605-5851655, EPI_ISL_5851723-5851815, EPI_ISL_5851820-5851910, EPI_ISL_5851915-5851938, EPI_ISL_5868823-5868907, EPI_ISL_5868910-5869000, EPI_ISL_5869003-5869082, EPI_ISL_5869084-5869122, EPI_ISL_5924689-5924693, EPI_ISL_5931619-5931621, EPI_ISL_5934840-5934848, EPI_ISL_5990147-5990204, EPI_ISL_6102162, EPI_ISL_6340373-6340424, EPI_ISL_6340462-6340523, EPI_ISL_6436860-6436862, EPI_ISL_6477199, EPI_ISL_6477201, EPI_ISL_6477203, EPI_ISL_6477205-6477206, EPI_ISL_6477208-6477209, EPI_ISL_6477211, EPI_ISL_6477213, EPI_ISL_6477215-6477216, EPI_ISL_6477218, EPI_ISL_6477220, EPI_ISL_6477222-6477223, EPI_ISL_6477225, EPI_ISL_6477227, EPI_ISL_6477229-6477230, EPI_ISL_6477232, EPI_ISL_6477234, EPI_ISL_6477236-6477237, EPI_ISL_6585694, EPI_ISL_6606438, EPI_ISL_6606446, EPI_ISL_6606455, EPI_ISL_6606466, EPI_ISL_6606470, EPI_ISL_6606476, EPI_ISL_6606485, EPI_ISL_6606492, EPI_ISL_6606495, EPI_ISL_6606500, EPI_ISL_6606510, EPI_ISL_6606516, EPI_ISL_6606523, EPI_ISL_6606529, EPI_ISL_6606533, EPI_ISL_6606539, EPI_ISL_6606549, EPI_ISL_6606559, EPI_ISL_6606568, EPI_ISL_6606575, EPI_ISL_6606580, EPI_ISL_6606586, EPI_ISL_6606594, EPI_ISL_6606601, EPI_ISL_6606607, EPI_ISL_6606614, EPI_ISL_6606618, EPI_ISL_6606621, EPI_ISL_6606628, EPI_ISL_6606632, EPI_ISL_6606636, EPI_ISL_6606644, EPI_ISL_6606648, EPI_ISL_6606656, EPI_ISL_6606660, EPI_ISL_6606664, EPI_ISL_6606675, EPI_ISL_6606678, EPI_ISL_6606688, EPI_ISL_6606696, EPI_ISL_6606704, EPI_ISL_6606710, EPI_ISL_6606717, EPI_ISL_6606724, EPI_ISL_6606733, EPI_ISL_6606737, EPI_ISL_6606743, EPI_ISL_6606747, EPI_ISL_6606757, EPI_ISL_6606763, EPI_ISL_6606769, EPI_ISL_6606777, EPI_ISL_6606786, EPI_ISL_6606793, EPI_ISL_6606800, EPI_ISL_6606810, EPI_ISL_6606815, EPI_ISL_6606822, EPI_ISL_6606831, EPI_ISL_6606840, EPI_ISL_6606848, EPI_ISL_6606856, EPI_ISL_6647949-6647951, EPI_ISL_6794949-6794979, EPI_ISL_6795087-6795148, EPI_ISL_6913893-6913894, EPI_ISL_7017893-7017966, EPI_ISL_7017968, EPI_ISL_7163309-7163367, EPI_ISL_7163370-7163380, EPI_ISL_7229474, EPI_ISL_7229486, EPI_ISL_7313992-7314004, EPI_ISL_7314006-7314067, EPI_ISL_7314153-7314192, EPI_ISL_7314194-7314205, EPI_ISL_7314208-7314228, EPI_ISL_7314233-7314252, EPI_ISL_7314257-7314268, EPI_ISL_7314273-7314278, EPI_ISL_7314281-7314297, EPI_ISL_7314303-7314304, EPI_ISL_7338767-7338768, EPI_ISL_7355476, EPI_ISL_7355482, EPI_ISL_7355490, EPI_ISL_7355497, EPI_ISL_7355526, EPI_ISL_7355554, EPI_ISL_7355581, EPI_ISL_7355606, EPI_ISL_7355634, EPI_ISL_7355659, EPI_ISL_7355690, EPI_ISL_7355719, EPI_ISL_7355739, EPI_ISL_7355768, EPI_ISL_7355785, EPI_ISL_7355799, EPI_ISL_7355825, EPI_ISL_7355848, EPI_ISL_7355879, EPI_ISL_7355899, EPI_ISL_7355922, EPI_ISL_7355946, EPI_ISL_7355968, EPI_ISL_7355986, EPI_ISL_7356008, EPI_ISL_7356032, EPI_ISL_7356055, EPI_ISL_7356080, EPI_ISL_7356110, EPI_ISL_7356132, EPI_ISL_7356167, EPI_ISL_7356194, EPI_ISL_7356220, EPI_ISL_7356245, EPI_ISL_7356269, EPI_ISL_7356291, EPI_ISL_7356318, EPI_ISL_7356335, EPI_ISL_7356360, EPI_ISL_7356382, EPI_ISL_7356405, EPI_ISL_7356433, EPI_ISL_7356458, EPI_ISL_7356478, EPI_ISL_7356495, EPI_ISL_7356520, EPI_ISL_7356535, EPI_ISL_7356564, EPI_ISL_7356594, EPI_ISL_7356614, EPI_ISL_7356636, EPI_ISL_7356664, EPI_ISL_7356688, EPI_ISL_7356706, EPI_ISL_7356724, EPI_ISL_7356747, EPI_ISL_7356770, EPI_ISL_7356808, EPI_ISL_7356834, EPI_ISL_7356838, EPI_ISL_7356846, EPI_ISL_7356851, EPI_ISL_7356859, EPI_ISL_7356862, EPI_ISL_7356871, EPI_ISL_7356877, EPI_ISL_7356887, EPI_ISL_7356894, EPI_ISL_7356903, EPI_ISL_7356912, EPI_ISL_7356921, EPI_ISL_7356930, EPI_ISL_7356938, EPI_ISL_7356947, EPI_ISL_7356954, EPI_ISL_7356964, EPI_ISL_7356975, EPI_ISL_7356984, EPI_ISL_7356992, EPI_ISL_7456985-7457002, EPI_ISL_7457006-7457060, EPI_ISL_7492647-7492671, EPI_ISL_7492675-7492698, EPI_ISL_7492700-7492724, EPI_ISL_7492728-7492735, EPI_ISL_7496679-7496724, EPI_ISL_7496729-7496734, EPI_ISL_7496737-7496739, EPI_ISL_7622544-7622596, EPI_ISL_7622598-7622618, EPI_ISL_7622620-7622671, EPI_ISL_7622676-7622734, EPI_ISL_7622737-7622755, EPI_ISL_7622760-7622860, EPI_ISL_7648887, EPI_ISL_7648903, EPI_ISL_7648905, EPI_ISL_7648908, EPI_ISL_7648910, EPI_ISL_7648912, EPI_ISL_7648914, EPI_ISL_7648916, EPI_ISL_7648919, EPI_ISL_7648921, EPI_ISL_7648924, EPI_ISL_7648926, EPI_ISL_7648929, EPI_ISL_7648931, EPI_ISL_7648934, EPI_ISL_7648936, EPI_ISL_7648938, EPI_ISL_7648941, EPI_ISL_7648943, EPI_ISL_7648946, EPI_ISL_7648948, EPI_ISL_7648951, EPI_ISL_7648953, EPI_ISL_7648956, EPI_ISL_7648959, EPI_ISL_7648962, EPI_ISL_7648965, EPI_ISL_7648968, EPI_ISL_7648971, EPI_ISL_7648974, EPI_ISL_7648977, EPI_ISL_7648986, EPI_ISL_7648994, EPI_ISL_7648996, EPI_ISL_7648999, EPI_ISL_7671313, EPI_ISL_7674334-7674342, EPI_ISL_7674344-7674538, EPI_ISL_7674540-7674627, EPI_ISL_7778211-7778286, EPI_ISL_7792103, EPI_ISL_7832645-7832693, EPI_ISL_7950352-7950400, EPI_ISL_7983814, EPI_ISL_8130198-8130248, EPI_ISL_8131550-8131628, EPI_ISL_8166892-8166971, EPI_ISL_8184775-8184777, EPI_ISL_8186683-8186688, EPI_ISL_8186703-8186707, EPI_ISL_8318248-8318314, EPI_ISL_8318980-8318983, EPI_ISL_8324898-8324970, EPI_ISL_8329270-8329290, EPI_ISL_8329292-8329357, EPI_ISL_8351203-8351216, EPI_ISL_8629099-8629173, EPI_ISL_8629214-8629274, EPI_ISL_8639330-8639333, EPI_ISL_8639360-8639364, EPI_ISL_8681649, EPI_ISL_8681651-8681686, EPI_ISL_8681688-8681711, EPI_ISL_8717997-8718070, EPI_ISL_8750817-8750866, EPI_ISL_8883001-8883179, EPI_ISL_8900516-8900852, EPI_ISL_8901883-8901884, EPI_ISL_8975267-8975320, EPI_ISL_9030095-9030105, EPI_ISL_9047087-9047116, EPI_ISL_9093135-9093196, EPI_ISL_9095780-9095858, EPI_ISL_9100789-9100798, EPI_ISL_9112401-9112441, EPI_ISL_9112443-9112488, EPI_ISL_9143403-9143405, EPI_ISL_9152928-9153007, EPI_ISL_9203155-9203227, EPI_ISL_9271651-9271736, EPI_ISL_9348922-9348994, EPI_ISL_9352263-9352342, EPI_ISL_9356683-9356764, EPI_ISL_9410495-9410498, EPI_ISL_9410500-9410575, EPI_ISL_9437768-9437843, EPI_ISL_9471578-9471588, EPI_ISL_9471590-9471633, EPI_ISL_9487023-9487112, EPI_ISL_9520523-9520538, EPI_ISL_9520540-9520604, EPI_ISL_9551856-9551940, EPI_ISL_9636984-9637044, EPI_ISL_9805649, EPI_ISL_9873088-9873103, EPI_ISL_9873105-9873126, EPI_ISL_9873128-9873162, EPI_ISL_9873507-9873560, EPI_ISL_9873562-9873578, EPI_ISL_9928485-9928549, EPI_ISL_9928551-9928553, EPI_ISL_10057438-10057491, EPI_ISL_10057493-10057510, EPI_ISL_10082603-10082686, EPI_ISL_10239441-10239503, EPI_ISL_10239657-10239720, EPI_ISL_10239779-10239859, EPI_ISL_10239861-10239944, EPI_ISL_10283576, EPI_ISL_10284172, EPI_ISL_10284716-10284718, EPI_ISL_10402739-10402769, EPI_ISL_10575103-10575384, EPI_ISL_10706201-10706250, EPI_ISL_10792673-10792679, EPI_ISL_11096036-11096037, EPI_ISL_11096039-11096077, EPI_ISL_11096079-11096102, EPI_ISL_11567772-11567824, EPI_ISL_11799908-11799969, EPI_ISL_12001704-12001732, EPI_ISL_12036973-12038956, EPI_ISL_12166970-12167007, EPI_ISL_12167009-12167021, EPI_ISL_12471173-12471217, EPI_ISL_12573620-12573681, EPI_ISL_12599428-12599506, EPI_ISL_12617586-12617667, EPI_ISL_12869399-12869426, EPI_ISL_13044520-13044590, EPI_ISL_13125307-13125374, EPI_ISL_13191623-13191689, EPI_ISL_13282059-13282111, EPI_ISL_13346203-13346242, EPI_ISL_13413796-13413798, EPI_ISL_13436070-13436089, EPI_ISL_13453572-13453655, EPI_ISL_13563917-13563933, EPI_ISL_13647726-13647802, EPI_ISL_13811828-13811835, EPI_ISL_13812248-13812307, EPI_ISL_13829359-13829445, EPI_ISL_13871229-13871292, EPI_ISL_14064584, EPI_ISL_14069795-14069801, EPI_ISL_14069805-14069806, EPI_ISL_14224066-14224143, EPI_ISL_14253402-14253470, EPI_ISL_14287960-14288034, EPI_ISL_14333784-14333843, EPI_ISL_14488398-14488461, EPI_ISL_14588225-14588292, EPI_ISL_14881049-14881080, EPI_ISL_14907032-14907098, EPI_ISL_15068083-15068135, EPI_ISL_15279926-15279960, EPI_ISL_15377008-15377077, EPI_ISL_15415338-15415391, EPI_ISL_15416614-15416655, EPI_ISL_15659017-15659058, EPI_ISL_15692996-15693038, EPI_ISL_15734312-15734353, EPI_ISL_15783595-15783654, EPI_ISL_15998820-15998859, EPI_ISL_16049757-16049846, EPI_ISL_16137517-16137564, EPI_ISL_16225759-16225826, EPI_ISL_16314961-16315008, EPI_ISL_16372293-16372321, EPI_ISL_16553477-16553528, EPI_ISL_16852846-16852883, EPI_ISL_16893894-16893896, EPI_ISL_16946504-16946522, EPI_ISL_16947535-16947556, EPI_ISL_16987766-16987798, EPI_ISL_17089345-17089388, EPI_ISL_17162889-17162920, EPI_ISL_17260602-17260660, EPI_ISL_17273394-17273433, EPI_ISL_17463038-17463055, EPI_ISL_17546498-17546515, EPI_ISL_18231364-18231376, EPI_ISL_18550551-18550565, EPI_ISL_18551259-18551267, EPI_ISL_18744051-18744057, EPI_ISL_18840357-18840365, EPI_ISL_19315398-19315403, EPI_ISL_19364518-19364520")
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################






manual = stringlist_to_strings_set("EPI_ISL_9322998, EPI_ISL_9323008, EPI_ISL_12422422, EPI_ISL_10129290, EPI_ISL_10129589, EPI_ISL_10129579, EPI_ISL_10786232, EPI_ISL_11071429, EPI_ISL_4945669, EPI_ISL_8187764, EPI_ISL_8692773, EPI_ISL_9322967, EPI_ISL_9322982, EPI_ISL_11355302, EPI_ISL_12065180, EPI_ISL_10744459, EPI_ISL_14227250, EPI_ISL_14227280, EPI_ISL_14227291, EPI_ISL_14227297-14227298, EPI_ISL_14227305, EPI_ISL_14227313, EPI_ISL_14227330, EPI_ISL_14227332, EPI_ISL_14227349, EPI_ISL_14227387, EPI_ISL_14227425-14227426, EPI_ISL_14227432, EPI_ISL_14227439, EPI_ISL_15041542, EPI_ISL_2312495, EPI_ISL_9087735, EPI_ISL_16982968, EPI_ISL_1167755, EPI_ISL_10013220, EPI_ISL_10053707, EPI_ISL_10053713, EPI_ISL_10068325, EPI_ISL_10240338, EPI_ISL_10240343, EPI_ISL_10240341, EPI_ISL_10240347, EPI_ISL_10240348, EPI_ISL_10393956, EPI_ISL_10393957, EPI_ISL_10013201, EPI_ISL_10053707, EPI_ISL_10013220, EPI_ISL_10053713, EPI_ISL_10671808, EPI_ISL_10671820, EPI_ISL_10671866, EPI_ISL_10683168, EPI_ISL_10671869, EPI_ISL_10757766, EPI_ISL_10757832, EPI_ISL_10758397, EPI_ISL_10882803, EPI_ISL_10882818, EPI_ISL_10882481, EPI_ISL_10925738, EPI_ISL_11000947, EPI_ISL_11376303, EPI_ISL_11376327, EPI_ISL_11586645, EPI_ISL_12658807, EPI_ISL_12918863, EPI_ISL_1544032, EPI_ISL_16215631, EPI_ISL_16454076, EPI_ISL_17372598, EPI_ISL_18098342, EPI_ISL_19456706, EPI_ISL_20175638, EPI_ISL_20213414, EPI_ISL_3613477, EPI_ISL_4719710, EPI_ISL_1096990, EPI_ISL_5049873, EPI_ISL_6463191, EPI_ISL_7153257, EPI_ISL_7121833, EPI_ISL_8135152, EPI_ISL_8287710, EPI_ISL_8517218, EPI_ISL_877764, EPI_ISL_8561137, EPI_ISL_8829793, EPI_ISL_8829653, EPI_ISL_9053095, EPI_ISL_9143287, EPI_ISL_9288279, EPI_ISL_9288355, EPI_ISL_9588089, EPI_ISL_9835867, EPI_ISL_9835886, EPI_ISL_9835879, EPI_ISL_9835902, EPI_ISL_9835920, EPI_ISL_18743306, EPI_ISL_18846640, EPI_ISL_18743470, EPI_ISL_19747527, EPI_ISL_18816571, EPI_ISL_18821499, EPI_ISL_18868309, EPI_ISL_18743431, EPI_ISL_19468169, EPI_ISL_19168836, EPI_ISL_18816651, EPI_ISL_18809900, EPI_ISL_18983272, EPI_ISL_18716638, EPI_ISL_18708779, EPI_ISL_18793700, EPI_ISL_18159982, EPI_ISL_19645563, EPI_ISL_19302341, EPI_ISL_18607562, EPI_ISL_18423762, EPI_ISL_12084022, EPI_ISL_15682771, EPI_ISL_18454240, EPI_ISL_4325655, EPI_ISL_14578769, EPI_ISL_18065435, EPI_ISL_10222131, EPI_ISL_15682788, EPI_ISL_15637296, EPI_ISL_14996195, EPI_ISL_5049921, EPI_ISL_15316450, EPI_ISL_5049921, EPI_ISL_16407945, EPI_ISL_2304079, EPI_ISL_20090417, EPI_ISL_19189685, EPI_ISL_19192500, EPI_ISL_17057500, EPI_ISL_8788657, EPI_ISL_804043, EPI_ISL_18141461, EPI_ISL_11565253, EPI_ISL_11070505, EPI_ISL_8035694, EPI_ISL_12171528, EPI_ISL_9261356, EPI_ISL_14239213, EPI_ISL_4945775, EPI_ISL_3464481, EPI_ISL_14689915, EPI_ISL_20125502, EPI_ISL_10306135, EPI_ISL_2222502, EPI_ISL_4186372, EPI_ISL_2221132, EPI_ISL_2190385, EPI_ISL_20125501, EPI_ISL_17047780, EPI_ISL_11355408, EPI_ISL_18936212, EPI_ISL_12914782, EPI_ISL_12911275, EPI_ISL_12914780, EPI_ISL_8812967, EPI_ISL_12934364, EPI_ISL_8806479, EPI_ISL_8812973, EPI_ISL_7551897, EPI_ISL_12934332, EPI_ISL_17213928, EPI_ISL_11355275, EPI_ISL_8804024, EPI_ISL_6262430, EPI_ISL_6938247, EPI_ISL_10306135, EPI_ISL_14680808, EPI_ISL_19243172, EPI_ISL_8812983, EPI_ISL_8813006, EPI_ISL_8813057, EPI_ISL_479747, EPI_ISL_8813018, EPI_ISL_8813069, EPI_ISL_15292382, EPI_ISL_13493579, EPI_ISL_15157267, EPI_ISL_12582783, EPI_ISL_15292690, EPI_ISL_9205859, EPI_ISL_14595398, EPI_ISL_14677326, EPI_ISL_2296053, EPI_ISL_2650472, EPI_ISL_7892890, EPI_ISL_16814977, EPI_ISL_1312352, EPI_ISL_4186372, EPI_ISL_2221132, EPI_ISL_13114890, EPI_ISL_2193347, EPI_ISL_13992956, EPI_ISL_10831828, EPI_ISL_10823290, EPI_ISL_11236803, EPI_ISL_10993946, EPI_ISL_9205057, EPI_ISL_9405597, EPI_ISL_10707897, EPI_ISL_10246702, EPI_ISL_5400333, EPI_ISL_5664187, EPI_ISL_7757661, EPI_ISL_7757684, EPI_ISL_8222762, EPI_ISL_8222967, EPI_ISL_8670567, EPI_ISL_9213667, EPI_ISL_9213692, EPI_ISL_9229258, EPI_ISL_9443826, EPI_ISL_9444069, EPI_ISL_9732707, EPI_ISL_10030009, EPI_ISL_10030176, EPI_ISL_10310444, EPI_ISL_10765993, EPI_ISL_10905742, EPI_ISL_11460188, EPI_ISL_12014754, EPI_ISL_12015959, EPI_ISL_12363744, EPI_ISL_12364499, EPI_ISL_12524891, EPI_ISL_12735789, EPI_ISL_12818860, EPI_ISL_12823932, EPI_ISL_13236709, EPI_ISL_13237591, EPI_ISL_13384126, EPI_ISL_13384129, EPI_ISL_13384137, EPI_ISL_13384148-13384149, EPI_ISL_13384153, EPI_ISL_13384156, EPI_ISL_13384160, EPI_ISL_13384167, EPI_ISL_13384175, EPI_ISL_13384180, EPI_ISL_13384183, EPI_ISL_13384192, EPI_ISL_13384211-13384212, EPI_ISL_13384216, EPI_ISL_13384220, EPI_ISL_13384230, EPI_ISL_13384250, EPI_ISL_13384252, EPI_ISL_13384272, EPI_ISL_13384285, EPI_ISL_13384292-13384293, EPI_ISL_13384305, EPI_ISL_13384320, EPI_ISL_13384323, EPI_ISL_13384325, EPI_ISL_13384328, EPI_ISL_13384336, EPI_ISL_13384342, EPI_ISL_13384349, EPI_ISL_13384366, EPI_ISL_13384371, EPI_ISL_13384374, EPI_ISL_13384395-13384396, EPI_ISL_13384401-13384402, EPI_ISL_13384407, EPI_ISL_13384413, EPI_ISL_13384416, EPI_ISL_13384420, EPI_ISL_13384425, EPI_ISL_10069072, EPI_ISL_4470082, EPI_ISL_15457520, EPI_ISL_17712917, EPI_ISL_12658797, EPI_ISL_17712420, EPI_ISL_17057556, EPI_ISL_13714340, EPI_ISL_13633130, EPI_ISL_12829598, EPI_ISL_17642296, EPI_ISL_17708677, EPI_ISL_7733214, EPI_ISL_8146261, EPI_ISL_11449568, EPI_ISL_15457520, EPI_ISL_18572269, EPI_ISL_5049871, EPI_ISL_19013131, EPI_ISL_17372563, EPI_ISL_8164976, EPI_ISL_17372562, EPI_ISL_17698885, EPI_ISL_14936053, EPI_ISL_11584217, EPI_ISL_3613544, EPI_ISL_18979607, EPI_ISL_8422512, EPI_ISL_15588441, EPI_ISL_16613984, EPI_ISL_15588441, EPI_ISL_12065504, EPI_ISL_12065507, EPI_ISL_12065513, EPI_ISL_12071130, EPI_ISL_5522696, EPI_ISL_19465491, EPI_ISL_5031397, EPI_ISL_16993458, EPI_ISL_20244086, EPI_ISL_1755027, EPI_ISL_15588324, EPI_ISL_8688192, EPI_ISL_10551553, EPI_ISL_20248007, EPI_ISL_2966185, EPI_ISL_10834859, EPI_ISL_17583367, EPI_ISL_18365480, EPI_ISL_18365364, EPI_ISL_16946489, EPI_ISL_2221881, EPI_ISL_3299465, EPI_ISL_3299471, EPI_ISL_5309835, EPI_ISL_3299192, EPI_ISL_18341474, EPI_ISL_18328446, EPI_ISL_951315, EPI_ISL_2654003, EPI_ISL_4112436, EPI_ISL_1960932, EPI_ISL_11345967, EPI_ISL_1401804, EPI_ISL_1415145, EPI_ISL_8812981, EPI_ISL_10943972, EPI_ISL_10944001, EPI_ISL_10950962, EPI_ISL_10951229, EPI_ISL_12914785, EPI_ISL_12914805, EPI_ISL_18418390, EPI_ISL_17110455, EPI_ISL_16355474, EPI_ISL_3033651, EPI_ISL_1905078, EPI_ISL_7733492, EPI_ISL_8422475, EPI_ISL_8422486, EPI_ISL_19837250, EPI_ISL_19837306, EPI_ISL_17323946, EPI_ISL_12792509, EPI_ISL_12792527, EPI_ISL_12792542, EPI_ISL_12792549, EPI_ISL_12792598, EPI_ISL_12792604, EPI_ISL_16699250, EPI_ISL_14192854, EPI_ISL_16314191, EPI_ISL_17812445, EPI_ISL_16546030, EPI_ISL_17989364, EPI_ISL_19704256, EPI_ISL_2382994, EPI_ISL_17505863, EPI_ISL_17989356, EPI_ISL_17505855, EPI_ISL_17989361, EPI_ISL_10013207, EPI_ISL_10013313, EPI_ISL_17697547, EPI_ISL_5297454, EPI_ISL_2332780, EPI_ISL_10925740, EPI_ISL_17664185, EPI_ISL_3242601, EPI_ISL_9288672, EPI_ISL_10925732, EPI_ISL_10758363, EPI_ISL_9835876, EPI_ISL_2382940, EPI_ISL_16680126, EPI_ISL_11469663, EPI_ISL_10882762, EPI_ISL_2304098, EPI_ISL_1936614, EPI_ISL_9835815, EPI_ISL_9835863, EPI_ISL_9223704, EPI_ISL_3236785, EPI_ISL_3775918, EPI_ISL_17047962, EPI_ISL_6684991, EPI_ISL_12792520, EPI_ISL_10013207, EPI_ISL_7547082, EPI_ISL_1475614, EPI_ISL_11355259, EPI_ISL_12174847, EPI_ISL_10560968, EPI_ISL_14570652, EPI_ISL_10830779, EPI_ISL_11933005, EPI_ISL_15588423, EPI_ISL_2230703, EPI_ISL_11254727, EPI_ISL_18098347, EPI_ISL_16258489, EPI_ISL_17530377, EPI_ISL_16394748, EPI_ISL_18936196, EPI_ISL_18936257, EPI_ISL_11887814, EPI_ISL_18936201, EPI_ISL_18255176, EPI_ISL_18831502, EPI_ISL_11214367, EPI_ISL_6631807, EPI_ISL_1253646, EPI_ISL_8766773, EPI_ISL_3020336, EPI_ISL_470837, EPI_ISL_12005443, EPI_ISL_9286565, EPI_ISL_11254727, EPI_ISL_7232349, EPI_ISL_2392166, EPI_ISL_11824693, EPI_ISL_3150227, EPI_ISL_3150233, EPI_ISL_3150234, EPI_ISL_3246603, EPI_ISL_3254309, EPI_ISL_14474356, EPI_ISL_754195, EPI_ISL_20125507, EPI_ISL_20125617, EPI_ISL_15091235, EPI_ISL_1133158, EPI_ISL_5384514, EPI_ISL_10729421, EPI_ISL_10835322, EPI_ISL_5881213, EPI_ISL_11514235, EPI_ISL_463893, EPI_ISL_1406836, EPI_ISL_6938332, EPI_ISL_7551888, EPI_ISL_11355432, EPI_ISL_12911269, EPI_ISL_1036813, EPI_ISL_1036822-1036823, EPI_ISL_1036933, EPI_ISL_1711176, EPI_ISL_1711203, EPI_ISL_2715298, EPI_ISL_2715306, EPI_ISL_2715310, EPI_ISL_2715355, EPI_ISL_2966759, EPI_ISL_3940164, EPI_ISL_11294820, EPI_ISL_8579199, EPI_ISL_19348679, EPI_ISL_17275324, EPI_ISL_1475561, EPI_ISL_1249228, EPI_ISL_8724990, EPI_ISL_1051710, EPI_ISL_1051768, EPI_ISL_20082490, EPI_ISL_20082488, EPI_ISL_15637858, EPI_ISL_5484593, EPI_ISL_17759731, EPI_ISL_6810514, EPI_ISL_10671881, EPI_ISL_12955779, EPI_ISL_18736013, EPI_ISL_17565160, EPI_ISL_18936264, EPI_ISL_18255100, EPI_ISL_16258490, EPI_ISL_18098341, EPI_ISL_12914784, EPI_ISL_11776067, EPI_ISL_4170073, EPI_ISL_12582706, EPI_ISL_12914795, EPI_ISL_17057936, EPI_ISL_8813083, EPI_ISL_8813033, EPI_ISL_11746640, EPI_ISL_16424022, EPI_ISL_15846162, EPI_ISL_18936264, EPI_ISL_18255100, EPI_ISL_18115504, EPI_ISL_16407946, EPI_ISL_12146072, EPI_ISL_14298842, EPI_ISL_10149346, EPI_ISL_10149333, EPI_ISL_17601585, EPI_ISL_961353, EPI_ISL_19804505, EPI_ISL_13895599, EPI_ISL_17236435, EPI_ISL_18056797, EPI_ISL_18071577, EPI_ISL_10889661, EPI_ISL_15756983, EPI_ISL_12319878, EPI_ISL_13988923, EPI_ISL_11023738, EPI_ISL_4056154, EPI_ISL_17236482, EPI_ISL_11022632, EPI_ISL_11881600, EPI_ISL_5050881, EPI_ISL_7509807, EPI_ISL_1079237, EPI_ISL_2445044, EPI_ISL_16761948, EPI_ISL_11776056, EPI_ISL_12836675, EPI_ISL_2628241, EPI_ISL_18840307, EPI_ISL_1696737, EPI_ISL_14753968, EPI_ISL_1051606, EPI_ISL_10443097, EPI_ISL_2690331, EPI_ISL_4104706, EPI_ISL_5802416, EPI_ISL_10830359, EPI_ISL_2628258, EPI_ISL_5433662, EPI_ISL_1840693, EPI_ISL_16687715, EPI_ISL_14699173, EPI_ISL_10826836, EPI_ISL_2695990, EPI_ISL_1077253, EPI_ISL_10826719, EPI_ISL_14954510, EPI_ISL_14871997, EPI_ISL_2227569, EPI_ISL_561368, EPI_ISL_13787687, EPI_ISL_18347556, EPI_ISL_9526798, EPI_ISL_1904297, EPI_ISL_2080874, EPI_ISL_2174728, EPI_ISL_2468944, EPI_ISL_2650481, EPI_ISL_2652051, EPI_ISL_3299465, EPI_ISL_3299521, EPI_ISL_16560204, EPI_ISL_17028423, EPI_ISL_3299528, EPI_ISL_3792382, EPI_ISL_18829937, EPI_ISL_12308695, EPI_ISL_11736389, EPI_ISL_13544269, EPI_ISL_19212892, EPI_ISL_14749461, EPI_ISL_18936191, EPI_ISL_12934315, EPI_ISL_14211821, EPI_ISL_14211671, EPI_ISL_17275299, EPI_ISL_7547071, EPI_ISL_11291877, EPI_ISL_1979030, EPI_ISL_8812993, EPI_ISL_3790381, EPI_ISL_14211783, EPI_ISL_6262420, EPI_ISL_999172, EPI_ISL_10951217, EPI_ISL_1097025, EPI_ISL_3794870, EPI_ISL_14211793, EPI_ISL_8193918, EPI_ISL_7547083, EPI_ISL_14749035, EPI_ISL_4198509, EPI_ISL_12658798, EPI_ISL_6262415, EPI_ISL_951358, EPI_ISL_12658867, EPI_ISL_3793003, EPI_ISL_7507374, EPI_ISL_18226327, EPI_ISL_3393273, EPI_ISL_7551912, EPI_ISL_10729410, EPI_ISL_7551929, EPI_ISL_17565330, EPI_ISL_5049870, EPI_ISL_3421721, EPI_ISL_6938327, EPI_ISL_12658806, EPI_ISL_14211649, EPI_ISL_3794866, EPI_ISL_14749439, EPI_ISL_12934326, EPI_ISL_14749463, EPI_ISL_3393126, EPI_ISL_19212649, EPI_ISL_413695, EPI_ISL_14211663, EPI_ISL_17427519, EPI_ISL_3421717, EPI_ISL_8813039, EPI_ISL_12934445, EPI_ISL_17372611, EPI_ISL_1167194, EPI_ISL_19212796, EPI_ISL_14749448, EPI_ISL_17275312, EPI_ISL_1167188, EPI_ISL_17236581, EPI_ISL_999097, EPI_ISL_8813042, EPI_ISL_9114647, EPI_ISL_4530134, EPI_ISL_14749465, EPI_ISL_14749025, EPI_ISL_5049922, EPI_ISL_3164296, EPI_ISL_17372618, EPI_ISL_8812990, EPI_ISL_11355423, EPI_ISL_3078866, EPI_ISL_17592069, EPI_ISL_14211728, EPI_ISL_4254661, EPI_ISL_14211661, EPI_ISL_5934822, EPI_ISL_17020828, EPI_ISL_19013431, EPI_ISL_14770231, EPI_ISL_12934291, EPI_ISL_8812965, EPI_ISL_17275321, EPI_ISL_4198510, EPI_ISL_19212916, EPI_ISL_953938, EPI_ISL_3793007, EPI_ISL_2903010, EPI_ISL_6262428, EPI_ISL_1109908, EPI_ISL_6357208, EPI_ISL_7509709, EPI_ISL_10944012, EPI_ISL_17425904, EPI_ISL_8803764, EPI_ISL_8812978, EPI_ISL_17565368, EPI_ISL_3421852, EPI_ISL_14211838, EPI_ISL_17020822, EPI_ISL_3792996, EPI_ISL_3793009, EPI_ISL_950844, EPI_ISL_14749011, EPI_ISL_1910755, EPI_ISL_15157245, EPI_ISL_13736428, EPI_ISL_12934366, EPI_ISL_12934246, EPI_ISL_14211711, EPI_ISL_6262412, EPI_ISL_14749469, EPI_ISL_951124, EPI_ISL_3386462, EPI_ISL_16258470, EPI_ISL_867117, EPI_ISL_17049000, EPI_ISL_950782, EPI_ISL_17275314, EPI_ISL_1165084, EPI_ISL_14211773, EPI_ISL_8803733, EPI_ISL_6938386, EPI_ISL_4198507, EPI_ISL_17372609, EPI_ISL_3794580, EPI_ISL_17592058, EPI_ISL_19192594, EPI_ISL_2399424, EPI_ISL_12694374, EPI_ISL_17592097, EPI_ISL_14211787, EPI_ISL_12934328, EPI_ISL_2707330, EPI_ISL_10943998, EPI_ISL_7493985, EPI_ISL_12658800, EPI_ISL_951455, EPI_ISL_15292716, EPI_ISL_1979036, EPI_ISL_6938399, EPI_ISL_10080731, EPI_ISL_6262427, EPI_ISL_3421731, EPI_ISL_19212725, EPI_ISL_3078917, EPI_ISL_3052905, EPI_ISL_12658803, EPI_ISL_14211788, EPI_ISL_12934421, EPI_ISL_1097027, EPI_ISL_17275330, EPI_ISL_19192556, EPI_ISL_17252820, EPI_ISL_14211785, EPI_ISL_7547072, EPI_ISL_20125615, EPI_ISL_3793004, EPI_ISL_3793312, EPI_ISL_1387214, EPI_ISL_17275327, EPI_ISL_3078951, EPI_ISL_3079049, EPI_ISL_14749430, EPI_ISL_3393200, EPI_ISL_14211665, EPI_ISL_4198508, EPI_ISL_16258498, EPI_ISL_17020817, EPI_ISL_840352, EPI_ISL_2404761, EPI_ISL_12934358, EPI_ISL_3792995, EPI_ISL_7547069, EPI_ISL_12658810, EPI_ISL_3568588, EPI_ISL_1663023, EPI_ISL_10944002, EPI_ISL_3393163, EPI_ISL_1178701, EPI_ISL_12934314, EPI_ISL_17744298, EPI_ISL_12954355, EPI_ISL_950902, EPI_ISL_6262431, EPI_ISL_14211790, EPI_ISL_3052909, EPI_ISL_1914289, EPI_ISL_12934360, EPI_ISL_906121, EPI_ISL_8812963, EPI_ISL_12934313, EPI_ISL_14770243, EPI_ISL_15334776, EPI_ISL_1475544, EPI_ISL_7508846, EPI_ISL_1165085, EPI_ISL_18831499, EPI_ISL_1178703, EPI_ISL_16258468, EPI_ISL_6262421, EPI_ISL_14211856, EPI_ISL_1415157, EPI_ISL_8803750, EPI_ISL_17502968, EPI_ISL_999421, EPI_ISL_12658812, EPI_ISL_3393236, EPI_ISL_15157236, EPI_ISL_20125505, EPI_ISL_3079030, EPI_ISL_12934162, EPI_ISL_954750, EPI_ISL_13857951, EPI_ISL_6125398, EPI_ISL_7509667, EPI_ISL_1167190, EPI_ISL_12658794, EPI_ISL_8193604, EPI_ISL_7792652, EPI_ISL_13374359, EPI_ISL_12248761, EPI_ISL_9090788, EPI_ISL_8085714, EPI_ISL_9487744, EPI_ISL_10746637, EPI_ISL_10996872, EPI_ISL_12363357, EPI_ISL_14018741, EPI_ISL_14260636, EPI_ISL_14793479, EPI_ISL_14936557, EPI_ISL_13469794, EPI_ISL_417446, EPI_ISL_406592, EPI_ISL_2775392, EPI_ISL_2775401, EPI_ISL_12754223, EPI_ISL_12754227, EPI_ISL_12754233, EPI_ISL_18052875, EPI_ISL_18936206, EPI_ISL_18936263, EPI_ISL_18936280, EPI_ISL_2312645, EPI_ISL_2349009, EPI_ISL_7637886, EPI_ISL_7644315, EPI_ISL_7644402, EPI_ISL_9130454-9130455, EPI_ISL_9189916-9189917, EPI_ISL_10079271, EPI_ISL_10079312, EPI_ISL_10786190, EPI_ISL_10808713, EPI_ISL_10811722, EPI_ISL_11383862, EPI_ISL_16258481, EPI_ISL_16370707, EPI_ISL_16741333, EPI_ISL_17275329, EPI_ISL_17275340, EPI_ISL_17275345, EPI_ISL_17275364, EPI_ISL_5049884, EPI_ISL_1167187, EPI_ISL_1909249, EPI_ISL_4106141, EPI_ISL_1908152, EPI_ISL_2965871, EPI_ISL_2775396, EPI_ISL_4646106, EPI_ISL_1039244, EPI_ISL_2775409, EPI_ISL_4085143, EPI_ISL_2775408, EPI_ISL_3717099, EPI_ISL_3806776, EPI_ISL_13351575, EPI_ISL_18743060, EPI_ISL_10858520, EPI_ISL_12145040, EPI_ISL_18436051, EPI_ISL_18228604, EPI_ISL_12584044, EPI_ISL_4050580, EPI_ISL_12740221, EPI_ISL_2885435, EPI_ISL_10858451, EPI_ISL_1585044, EPI_ISL_19507701, EPI_ISL_11541387, EPI_ISL_14718231, EPI_ISL_19507698, EPI_ISL_12999280, EPI_ISL_3664332, EPI_ISL_12955767, EPI_ISL_12955757, EPI_ISL_12999198, EPI_ISL_12955759, EPI_ISL_12320470, EPI_ISL_17057905, EPI_ISL_18743097, EPI_ISL_7591211, EPI_ISL_12146282, EPI_ISL_5510405, EPI_ISL_18228600, EPI_ISL_19130834, EPI_ISL_18228592, EPI_ISL_1257834, EPI_ISL_12152776, EPI_ISL_12955309, EPI_ISL_12955323, EPI_ISL_12955358, EPI_ISL_12955755, EPI_ISL_2185097, EPI_ISL_18228572, EPI_ISL_12955362, EPI_ISL_3047545, EPI_ISL_2861894, EPI_ISL_2375796, EPI_ISL_17675888, EPI_ISL_4367333, EPI_ISL_2762420, EPI_ISL_3547039, EPI_ISL_15348882, EPI_ISL_2462557, EPI_ISL_12999295, EPI_ISL_2185139, EPI_ISL_19721400, EPI_ISL_13131330, EPI_ISL_4635662, EPI_ISL_12955798, EPI_ISL_3305534, EPI_ISL_12320477, EPI_ISL_3032638, EPI_ISL_10523958, EPI_ISL_3410096, EPI_ISL_1684491, EPI_ISL_12999303, EPI_ISL_14495416, EPI_ISL_18912398, EPI_ISL_12223844, EPI_ISL_7711714, EPI_ISL_2987814, EPI_ISL_3315446, EPI_ISL_3402433, EPI_ISL_3402507, EPI_ISL_18821489, EPI_ISL_18821483, EPI_ISL_18821485, EPI_ISL_18821484, EPI_ISL_18909127, EPI_ISL_18677245, EPI_ISL_18743285, EPI_ISL_1703642, EPI_ISL_7337934, EPI_ISL_14148380, EPI_ISL_14148392, EPI_ISL_14148370, EPI_ISL_19213104, EPI_ISL_1585048, EPI_ISL_5049881, EPI_ISL_19014027, EPI_ISL_1096997, EPI_ISL_1109909, EPI_ISL_1305139, EPI_ISL_11415298, EPI_ISL_19022021, EPI_ISL_11359469, EPI_ISL_5237178, EPI_ISL_1811222, EPI_ISL_10657798, EPI_ISL_14934379, EPI_ISL_3164401, EPI_ISL_5069757, EPI_ISL_5056578, EPI_ISL_2634081, EPI_ISL_2761448, EPI_ISL_18268996, EPI_ISL_11776065, EPI_ISL_9305848, EPI_ISL_19906489, EPI_ISL_7810567, EPI_ISL_17048351, EPI_ISL_17048373, EPI_ISL_7459802, EPI_ISL_7459808, EPI_ISL_7459826, EPI_ISL_7459829, EPI_ISL_5324097, EPI_ISL_8165195, EPI_ISL_19143502, EPI_ISL_17017049, EPI_ISL_17017054, EPI_ISL_17017060, EPI_ISL_17612350, EPI_ISL_17821657, EPI_ISL_1127139, EPI_ISL_10905976, EPI_ISL_8151965, EPI_ISL_15213516, EPI_ISL_13905991, EPI_ISL_2989612, EPI_ISL_2990625, EPI_ISL_2990638, EPI_ISL_2990640, EPI_ISL_2990657, EPI_ISL_2990682, EPI_ISL_2990723, EPI_ISL_2991008, EPI_ISL_3655594, EPI_ISL_3656320, EPI_ISL_3657281, EPI_ISL_4366250, EPI_ISL_4562321, EPI_ISL_4562335, EPI_ISL_4562352, EPI_ISL_4562623, EPI_ISL_4562651, EPI_ISL_4921913, EPI_ISL_13092657, EPI_ISL_6313521, EPI_ISL_3003316, EPI_ISL_3178943, EPI_ISL_18394018, EPI_ISL_2804368, EPI_ISL_14217903, EPI_ISL_14218187, EPI_ISL_19614951, EPI_ISL_17522539, EPI_ISL_15316164, EPI_ISL_17245100, EPI_ISL_5339021, EPI_ISL_19536023, EPI_ISL_15015723, EPI_ISL_16397957, EPI_ISL_17549547, EPI_ISL_13462014, EPI_ISL_4473854, EPI_ISL_14578586, EPI_ISL_16397957, EPI_ISL_18138937, EPI_ISL_13912700, EPI_ISL_14361317, EPI_ISL_14432028, EPI_ISL_10380147, EPI_ISL_15154103, EPI_ISL_9215228, EPI_ISL_19573850, EPI_ISL_11760260, EPI_ISL_11017250, EPI_ISL_14551457, EPI_ISL_9940216, EPI_ISL_15038834, EPI_ISL_16125619, EPI_ISL_16528621, EPI_ISL_16528622, EPI_ISL_13368486, EPI_ISL_19838229, EPI_ISL_18274129, EPI_ISL_9261274, EPI_ISL_9261275, EPI_ISL_9261315, EPI_ISL_9261333, EPI_ISL_9261343, EPI_ISL_9261351, EPI_ISL_9261365, EPI_ISL_19032958, EPI_ISL_5388899, EPI_ISL_2557866, EPI_ISL_6262413, EPI_ISL_6262414, EPI_ISL_6262417, EPI_ISL_6262422, EPI_ISL_6262422, EPI_ISL_8812980, EPI_ISL_8812987, EPI_ISL_8812988, EPI_ISL_8812994, EPI_ISL_8812998, EPI_ISL_8813000, EPI_ISL_8813010, EPI_ISL_8813011, EPI_ISL_8813020, EPI_ISL_8813036, EPI_ISL_8813037, EPI_ISL_8813043, EPI_ISL_8813047, EPI_ISL_8813049, EPI_ISL_8813061, EPI_ISL_8813062, EPI_ISL_8813063, EPI_ISL_8813071, EPI_ISL_6938252, EPI_ISL_6938255, EPI_ISL_6938305, EPI_ISL_6938365, EPI_ISL_6938417, EPI_ISL_7493988, EPI_ISL_7547073, EPI_ISL_7547080, EPI_ISL_7547084, EPI_ISL_7551871, EPI_ISL_12658842, EPI_ISL_12658843, EPI_ISL_12658853, EPI_ISL_12934134, EPI_ISL_12934143, EPI_ISL_12934185, EPI_ISL_12934261, EPI_ISL_12934297, EPI_ISL_12934298, EPI_ISL_12934299, EPI_ISL_12934321, EPI_ISL_12934333, EPI_ISL_12934336, EPI_ISL_12934369, EPI_ISL_12934450, EPI_ISL_12934455, EPI_ISL_12934456, EPI_ISL_17562669, EPI_ISL_3588762, EPI_ISL_3743152, EPI_ISL_3588812, EPI_ISL_3756845, EPI_ISL_3743157, EPI_ISL_3588790, EPI_ISL_12583842, EPI_ISL_3543373, EPI_ISL_3588749, EPI_ISL_3588802, EPI_ISL_5511689, EPI_ISL_14701411, EPI_ISL_16120995, EPI_ISL_16553517, EPI_ISL_16814940, EPI_ISL_16814949, EPI_ISL_16815034, EPI_ISL_16599636, EPI_ISL_16271048, EPI_ISL_17275297, EPI_ISL_16699252, EPI_ISL_15473936, EPI_ISL_9629866, EPI_ISL_1109976, EPI_ISL_482775, EPI_ISL_12511485, EPI_ISL_7945646, EPI_ISL_7944977, EPI_ISL_7792628, EPI_ISL_5470943, EPI_ISL_467299, EPI_ISL_1517192, EPI_ISL_13384362, EPI_ISL_1034151, EPI_ISL_7947025, EPI_ISL_2754678, EPI_ISL_1625031, EPI_ISL_9629922, EPI_ISL_9077212, EPI_ISL_17597388, EPI_ISL_13544323, EPI_ISL_955140, EPI_ISL_12736844, EPI_ISL_2887404, EPI_ISL_5934793, EPI_ISL_17034838, EPI_ISL_12955760, EPI_ISL_7591215, EPI_ISL_7591217, EPI_ISL_7591219, EPI_ISL_7591222, EPI_ISL_1696687, EPI_ISL_1696692, EPI_ISL_1696714, EPI_ISL_1696715, EPI_ISL_1133157, EPI_ISL_1704072, EPI_ISL_17238817, EPI_ISL_3299585, EPI_ISL_2095083, EPI_ISL_2095085, EPI_ISL_2095099, EPI_ISL_2095111, EPI_ISL_2095114, EPI_ISL_2095125, EPI_ISL_19013684, EPI_ISL_18110550, EPI_ISL_17785884, EPI_ISL_17821633, EPI_ISL_17821640, EPI_ISL_17821665, EPI_ISL_1960965, EPI_ISL_1960981, EPI_ISL_1960987, EPI_ISL_1961000, EPI_ISL_8422490, EPI_ISL_12582703, EPI_ISL_18572143, EPI_ISL_2931767, EPI_ISL_12146277, EPI_ISL_12320529, EPI_ISL_12320662, EPI_ISL_10697706, EPI_ISL_10697708, EPI_ISL_10697709, EPI_ISL_10697711, EPI_ISL_10697714, EPI_ISL_10697716, EPI_ISL_10697718, EPI_ISL_10697720, EPI_ISL_10697721, EPI_ISL_10697722, EPI_ISL_10697723, EPI_ISL_10697724, EPI_ISL_18352009, EPI_ISL_18139468, EPI_ISL_15316453, EPI_ISL_10432651, EPI_ISL_3941723, EPI_ISL_1133149, EPI_ISL_12152767, EPI_ISL_12321743, EPI_ISL_12906443, EPI_ISL_17372560, EPI_ISL_19014042, EPI_ISL_17097728, EPI_ISL_14552727, EPI_ISL_14584747, EPI_ISL_14584752, EPI_ISL_14584768, EPI_ISL_14584783, EPI_ISL_961464, EPI_ISL_1704083, EPI_ISL_2106988, EPI_ISL_2547087, EPI_ISL_3631655, EPI_ISL_7873234, EPI_ISL_8445235, EPI_ISL_8445256, EPI_ISL_8975271, EPI_ISL_9093175, EPI_ISL_9112420, EPI_ISL_9305884, EPI_ISL_9348945, EPI_ISL_10121550, EPI_ISL_10716792, EPI_ISL_10716808, EPI_ISL_10716815, EPI_ISL_11336335, EPI_ISL_11799946, EPI_ISL_12906191, EPI_ISL_12906192, EPI_ISL_12906193, EPI_ISL_12906194, EPI_ISL_12906195, EPI_ISL_12906196, EPI_ISL_12906197, EPI_ISL_12906198, EPI_ISL_12906199, EPI_ISL_12906200, EPI_ISL_12906201, EPI_ISL_12906202, EPI_ISL_12906203, EPI_ISL_12906204, EPI_ISL_12906205, EPI_ISL_12906206, EPI_ISL_12906207, EPI_ISL_12906208, EPI_ISL_12906209, EPI_ISL_12906213, EPI_ISL_12906215, EPI_ISL_12906216, EPI_ISL_12906217, EPI_ISL_12906221, EPI_ISL_12906223, EPI_ISL_12906224, EPI_ISL_12906225, EPI_ISL_12906226, EPI_ISL_12906227, EPI_ISL_12906228, EPI_ISL_12906229, EPI_ISL_12906230, EPI_ISL_12906231, EPI_ISL_12906232, EPI_ISL_12906233, EPI_ISL_12906234, EPI_ISL_12906236, EPI_ISL_12906239, EPI_ISL_12906241, EPI_ISL_12906245, EPI_ISL_12906246, EPI_ISL_12906252, EPI_ISL_12906256, EPI_ISL_12906263, EPI_ISL_12906264, EPI_ISL_12906268, EPI_ISL_12906270, EPI_ISL_12906271, EPI_ISL_12906272, EPI_ISL_12906273, EPI_ISL_12906276, EPI_ISL_12906281, EPI_ISL_12906282, EPI_ISL_12906283, EPI_ISL_12906284, EPI_ISL_12906285, EPI_ISL_12906287, EPI_ISL_12906288, EPI_ISL_12906289, EPI_ISL_12906292, EPI_ISL_12906294, EPI_ISL_12906300, EPI_ISL_12906301, EPI_ISL_12906302, EPI_ISL_12906305, EPI_ISL_12906306, EPI_ISL_12906307, EPI_ISL_12906310, EPI_ISL_12906311, EPI_ISL_12906312, EPI_ISL_12906315, EPI_ISL_12906316, EPI_ISL_12906317, EPI_ISL_12906318, EPI_ISL_12906319, EPI_ISL_12906320, EPI_ISL_12906321, EPI_ISL_12906322, EPI_ISL_12906323, EPI_ISL_12906325, EPI_ISL_12906339, EPI_ISL_12906442, EPI_ISL_12906445, EPI_ISL_12906446, EPI_ISL_12906447, EPI_ISL_12906448, EPI_ISL_12906449, EPI_ISL_12906450, EPI_ISL_12906589, EPI_ISL_12906591, EPI_ISL_12906593, EPI_ISL_12906596, EPI_ISL_12906598, EPI_ISL_12906610, EPI_ISL_12906621, EPI_ISL_12906626, EPI_ISL_12906635, EPI_ISL_12906639, EPI_ISL_12934416, EPI_ISL_12955769, EPI_ISL_12999272, EPI_ISL_13304547, EPI_ISL_15038277, EPI_ISL_15251283, EPI_ISL_15348547, EPI_ISL_16407943, EPI_ISL_17275302, EPI_ISL_17301938, EPI_ISL_17301940, EPI_ISL_17301944, EPI_ISL_17302062, EPI_ISL_17302064, EPI_ISL_17302068, EPI_ISL_17302070, EPI_ISL_17302071, EPI_ISL_17466156, EPI_ISL_17466157, EPI_ISL_17466158, EPI_ISL_17466159, EPI_ISL_17466160, EPI_ISL_17466161, EPI_ISL_17466162, EPI_ISL_17466163, EPI_ISL_17466164, EPI_ISL_17466165, EPI_ISL_17466166, EPI_ISL_17466167, EPI_ISL_17466168, EPI_ISL_17466169, EPI_ISL_17466170, EPI_ISL_17466171, EPI_ISL_17466172, EPI_ISL_17466173, EPI_ISL_17466174, EPI_ISL_17466175, EPI_ISL_17466176, EPI_ISL_17466177, EPI_ISL_17466178, EPI_ISL_17466179, EPI_ISL_12583843, EPI_ISL_12146044, EPI_ISL_12146227, EPI_ISL_1957278, EPI_ISL_16061073, EPI_ISL_1173555, EPI_ISL_17682075, EPI_ISL_17989401, EPI_ISL_14561987, EPI_ISL_12194616, EPI_ISL_449800, EPI_ISL_8688196, EPI_ISL_2389452, EPI_ISL_2375846, EPI_ISL_1825065, EPI_ISL_2375511, EPI_ISL_11468163, EPI_ISL_10551554, EPI_ISL_10551562, EPI_ISL_13478523, EPI_ISL_10314946, EPI_ISL_5484223, EPI_ISL_17445536, EPI_ISL_18385487, EPI_ISL_1703777, EPI_ISL_1703826, EPI_ISL_16354122, EPI_ISL_15348741, EPI_ISL_15415391, EPI_ISL_9861349, EPI_ISL_1133148, EPI_ISL_1166096, EPI_ISL_1166102, EPI_ISL_1166104, EPI_ISL_1469239, EPI_ISL_10523160, EPI_ISL_10851659, EPI_ISL_10851661, EPI_ISL_10858436, EPI_ISL_10858527, EPI_ISL_10858541, EPI_ISL_11333353, EPI_ISL_11507483, EPI_ISL_11507489, EPI_ISL_11507524, EPI_ISL_11793451, EPI_ISL_12146193, EPI_ISL_12291347, EPI_ISL_12308587, EPI_ISL_12321607, EPI_ISL_12583926, EPI_ISL_12694427, EPI_ISL_12955756, EPI_ISL_12955775, EPI_ISL_12955828, EPI_ISL_13176027, EPI_ISL_13307898, EPI_ISL_13847367, EPI_ISL_14999066, EPI_ISL_15038548, EPI_ISL_1185776, EPI_ISL_4551799, EPI_ISL_1184558, EPI_ISL_1185776, EPI_ISL_18098351, EPI_ISL_8827492, EPI_ISL_9196492, EPI_ISL_8193909, EPI_ISL_8251487, EPI_ISL_5686636, EPI_ISL_3743165, EPI_ISL_1915533, EPI_ISL_11887232, EPI_ISL_11336333, EPI_ISL_12753427, EPI_ISL_12753104, EPI_ISL_3775926, EPI_ISL_6599771, EPI_ISL_12955519, EPI_ISL_14847621, EPI_ISL_9658674, EPI_ISL_9202505, EPI_ISL_9202539, EPI_ISL_9030329, EPI_ISL_9746968, EPI_ISL_9518504, EPI_ISL_9133317, EPI_ISL_9300549, EPI_ISL_9437783, EPI_ISL_9669302, EPI_ISL_9669737, EPI_ISL_9519903, EPI_ISL_9437783, EPI_ISL_9133317, EPI_ISL_9518504, EPI_ISL_9746968, EPI_ISL_9030329, EPI_ISL_9202505, EPI_ISL_9202539, EPI_ISL_9437783, EPI_ISL_9093609, EPI_ISL_9093690, EPI_ISL_7566355, EPI_ISL_6949207, EPI_ISL_6117425, EPI_ISL_6898986, EPI_ISL_6208342, EPI_ISL_6208343, EPI_ISL_6208344, EPI_ISL_6208345, EPI_ISL_6208346, EPI_ISL_6368268, EPI_ISL_6582943, EPI_ISL_5736276, EPI_ISL_5736289, EPI_ISL_5736297, EPI_ISL_5736302, EPI_ISL_5736307, EPI_ISL_5736314, EPI_ISL_5736811, EPI_ISL_5738176, EPI_ISL_5367954, EPI_ISL_5123044, EPI_ISL_5384223, EPI_ISL_5344440, EPI_ISL_5854760, EPI_ISL_5536462, EPI_ISL_5536463, EPI_ISL_5599480, EPI_ISL_5599487, EPI_ISL_5599496, EPI_ISL_5599500, EPI_ISL_5867029, EPI_ISL_5324096, EPI_ISL_5070742, EPI_ISL_5073213, EPI_ISL_5390697, EPI_ISL_2428882, EPI_ISL_2428956, EPI_ISL_7370200, EPI_ISL_7380510, EPI_ISL_7456430, EPI_ISL_7456435, EPI_ISL_7456462, EPI_ISL_7547036, EPI_ISL_7566326, EPI_ISL_7566334, EPI_ISL_7566340, EPI_ISL_7566342, EPI_ISL_7566350, EPI_ISL_7566352, EPI_ISL_7566354, EPI_ISL_7566356, EPI_ISL_7566358, EPI_ISL_7566359, EPI_ISL_7566361, EPI_ISL_7566362, EPI_ISL_7566367, EPI_ISL_7566369, EPI_ISL_7566372, EPI_ISL_7566374, EPI_ISL_7566375, EPI_ISL_7566381, EPI_ISL_7566383, EPI_ISL_7566386, EPI_ISL_7568581, EPI_ISL_7568583, EPI_ISL_7568585, EPI_ISL_7568586, EPI_ISL_7568588, EPI_ISL_7568590, EPI_ISL_7568595, EPI_ISL_7568596, EPI_ISL_7568598, EPI_ISL_7575090, EPI_ISL_7594185, EPI_ISL_7605583, EPI_ISL_7605585, EPI_ISL_7615994, EPI_ISL_7634556, EPI_ISL_7660560, EPI_ISL_7698118, EPI_ISL_7716422, EPI_ISL_7716481, EPI_ISL_7719334, EPI_ISL_7729239, EPI_ISL_7729406, EPI_ISL_7734039, EPI_ISL_7734070, EPI_ISL_7734081, EPI_ISL_7738202, EPI_ISL_7743919, EPI_ISL_7747278, EPI_ISL_7762218, EPI_ISL_7763437, EPI_ISL_7795567, EPI_ISL_7807933, EPI_ISL_7808511, EPI_ISL_7818414, EPI_ISL_7842429, EPI_ISL_7848702, EPI_ISL_7850777, EPI_ISL_7850785, EPI_ISL_7860076, EPI_ISL_7862534, EPI_ISL_7862924, EPI_ISL_7863030, EPI_ISL_7863256, EPI_ISL_7868872, EPI_ISL_7869228, EPI_ISL_7882980, EPI_ISL_7895735, EPI_ISL_7948596, EPI_ISL_7952860, EPI_ISL_7957890, EPI_ISL_7961502, EPI_ISL_7961514, EPI_ISL_7961912, EPI_ISL_7963486, EPI_ISL_7969832, EPI_ISL_7971033, EPI_ISL_7976101, EPI_ISL_7977800, EPI_ISL_7977825, EPI_ISL_7977909, EPI_ISL_7978204, EPI_ISL_7978212, EPI_ISL_7978362, EPI_ISL_7978364, EPI_ISL_7978604, EPI_ISL_7979198, EPI_ISL_7979224, EPI_ISL_7980072, EPI_ISL_7992055, EPI_ISL_7994652, EPI_ISL_7994984, EPI_ISL_7605585, EPI_ISL_7337601, EPI_ISL_7850777, EPI_ISL_7850785, EPI_ISL_7698118, EPI_ISL_7734039, EPI_ISL_7734070, EPI_ISL_7734081, EPI_ISL_7977800, EPI_ISL_7977825, EPI_ISL_7977909, EPI_ISL_7978204, EPI_ISL_7978212, EPI_ISL_7978362, EPI_ISL_7978364, EPI_ISL_7978604, EPI_ISL_7979198, EPI_ISL_7979224, EPI_ISL_7980072, EPI_ISL_7738202, EPI_ISL_7848702, EPI_ISL_7842429, EPI_ISL_7969832, EPI_ISL_7868872, EPI_ISL_7807933, EPI_ISL_7808511, EPI_ISL_7615994, EPI_ISL_7594185, EPI_ISL_7265940, EPI_ISL_7952860, EPI_ISL_7869228, EPI_ISL_7994984, EPI_ISL_7994652, EPI_ISL_7992055, EPI_ISL_7795567, EPI_ISL_7895735, EPI_ISL_7862534, EPI_ISL_7862924, EPI_ISL_7863030, EPI_ISL_7863256, EPI_ISL_7575090, EPI_ISL_7963486, EPI_ISL_7762218, EPI_ISL_7763437, EPI_ISL_7818414, EPI_ISL_7634556, EPI_ISL_7719334, EPI_ISL_7860076, EPI_ISL_7605583, EPI_ISL_7337522, EPI_ISL_7337591, EPI_ISL_7167723, EPI_ISL_7747278, EPI_ISL_7716422, EPI_ISL_7716481, EPI_ISL_7976101, EPI_ISL_7566355, EPI_ISL_14210694, EPI_ISL_15846254, EPI_ISL_15945485, EPI_ISL_9456848, EPI_ISL_9456983, EPI_ISL_9156804, EPI_ISL_15709506, EPI_ISL_15025376, EPI_ISL_15940837, EPI_ISL_15975710, EPI_ISL_15025373, EPI_ISL_15270610, EPI_ISL_15670087, EPI_ISL_15250585, EPI_ISL_15709506, EPI_ISL_15880471, EPI_ISL_15879843, EPI_ISL_15897084, EPI_ISL_15897053, EPI_ISL_15886020, EPI_ISL_15866417, EPI_ISL_15874062, EPI_ISL_15179344, EPI_ISL_15237955, EPI_ISL_15399812, EPI_ISL_15494652, EPI_ISL_15482029, EPI_ISL_15947183, EPI_ISL_15982525, EPI_ISL_15846260, EPI_ISL_15831957")






############################################################################################################################################################################
############################################################################################################################################################################
############################################################################################################################################################################
combined_lists = union(all_seqs_set, all_seqs_set_DQ, new_chronic, Russian_Kaluga_duplicate, interesting_wastewater, Angola_A, AT1, AV1, B1_243_2, B1_616, B1_632, B1_633, B1_637_1, B1_638, C12, manual, EPI_8mil_9mil, EPI_9mil_10mil, EPI_10mil_11mil, EPI_11mil_12mil, EPI_16mil_17mil, Spike_I794N, K113N_Q115H, Idaho, India, Indonesia, Mexico, Texas, Thailand, Turkey, UTEP, mink, deer)
baddies = union(Angola_A, AT1, AV1, B1_243_2, B1_616, B1_632, B1_633, B1_637_1, B1_638, C12, manual, EPI_8mil_9mil, EPI_9mil_10mil, EPI_10mil_11mil, EPI_11mil_12mil, EPI_16mil_17mil, Spike_I794N, K113N_Q115H, Idaho, India, Indonesia, Mexico, Texas, Thailand, Turkey, UTEP, mink, deer)
############################################################################################################################################################################
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp"); println(date_now); println("Finished"); print("\n"^1)
############################################################################################################################################################################
############################################################################################################################################################################
    
    ql(n) = (length(n), n)
    sort_by_length_then_num(n) = sort(collect(n), by = x -> ql(x))

function chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)
    date = Dates.format(today(), "yyyy_mm_dd")
    excluded_AA = Set(["ORF1a:F143M", "ORF1a:S142T", "S:T19I", "S:A67V", "S:V67A", "S:T95I", "S:G142D", "S:D142G", "S:Y145D", "S:L212I", "S:G339D", "S:S371F", "S:S371L", "S:L371F", "S:S373P", "S:S375F", "S:F375S", "S:T376A", "S:A376T", "S:D405N", "S:N405D", "S:R408S", "S:S408R", "S:K417N", "S:N417K", "S:K440N", "S:Q493R", "S:G496S", "S:N679K", "S:I794N", "S:N856K", "S:Q954H", "S:N969K", "S:L981F", "ORF1b:N498I", "ORF1b:F685Y", "S:S27A"])
    epi_key(m) = (length(split(m, "_")[3]), parse(Int, split(m, "_")[3]))
    seq_country = Dict{String, String}()
    seq_US_state = Dict{String, String}()
    seq_clade = Dict{String, String}()
    seq_pango = Dict{String, String}()
    seq_pango_unaliased = Dict{String, String}()
    seq_collection_date = Dict{String, String}()
    seq_date_index = Dict{String, Int}()
    seq_date_tuple = Dict{String, Tuple{Int, Int, Int}}()
    seq_lab_dict = Dict{String, String}()
    seq_nuc_del_ranges_ct = Dict{String, Int}()

    chr_candidates = Vector{String}()
    bad_chr_candidates = Vector{String}()
    bad_country_chr_candidates = Vector{String}()
    #chr_epis_list = chronic_epis("chronics_combined_2024_11_23__3902_seq_v2.ndjson")
    chr_epis_list = combined_lists
    no_date_to_index_key::Int = 0
    for line in eachline(ndjson)
        batcheck = split(line, "/")[2]
        if batcheck ≠ "bat"
            j = JSON3.read(line)
            name = EPI_ISL(j.seqName)
            
            seq_country[name] = country(j.seqName)
            seq_US_state[name] = US_state(j.seqName)
            seq_date = ""
            try
                seq_clade[name] = j.clade
            catch
                seq_clade[name] = "Unknown"
            end
            try
                seq_pango[name] = j.customNodeAttributes.Nextclade_pango
            catch
                seq_pango[name] = "Unknown"
            end
            try
                seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
            catch
                seq_pango_unaliased[name] = "Unknown"
            end
            seq_date = "0-0-0"
            try
                seq_date = sequence_date(j.seqName)
            catch
                seq_date = "0-0-0"
            end
            lab = ""
            if !isempty(seq_lab(j.seqName))
                lab = seq_lab(j.seqName)
            end
            seq_lab_dict[name] = lab
            seq_yr = join(seq_date[1:4])
            seq_year::Int = 0
            if all(isdigit, seq_yr)
                seq_year = parse(Int, seq_yr)
            else
                seq_year = 0
            end
            seq_month::Int = 0
            seq_day::Int = 0
            if length(seq_date) ≥ 7
                seq_mo = join(seq_date[6:7])
                if all(isdigit, seq_mo)
                    seq_month = parse(Int, seq_mo)
                end
            end
            if length(seq_date) ≥ 10
                seq_da = join(seq_date[9:10])
                if all(isdigit, seq_da)
                    seq_day = parse(Int, seq_da)
                end
            end
            seq_collection_date[name] = seq_date
#            date_index = ""
#            if haskey(tuple_to_index, (seq_year, seq_month, seq_day))
#                date_index = tuple_to_index[(seq_year, seq_month, seq_day)]
#            else
#                no_date_to_index_key += 1
#            end
#            seq_date_index[name] = date_index
            date_tuple = (seq_year, seq_month, seq_day)
            seq_date_tuple[name] = date_tuple
            chr_muts_ct::Int = 0
            labeled_subs_total::Int = 0
            privAA = j.privateAaMutations
            AA_ORF1a = privAA.ORF1a
            AA_ORF1b = privAA.ORF1b
            AA_ORF3a = privAA.ORF3a
            AA_ORF6 = privAA.ORF6
            AA_ORF7a = privAA.ORF7a
            AA_ORF7b = privAA.ORF7b
            AA_ORF8 = privAA.ORF8
            AA_ORF9b = privAA.ORF9b
            AA_S = privAA.S
            AA_E = privAA.E
            AA_M = privAA.M
            AA_N = privAA.N
            bad_countries = Set(["Kuwait", "India", "Bangladesh", "Thailand", "Pakistan", "Saudi Arabia", "INDIA", "Senegal", "Palestine", "Mongolia"])
            if !(name in combined_lists) && haskey(j, "privateAaMutations")
                if haskey(privAA, "ORF1a") && haskey(privAA, "ORF1b") && haskey(privAA, "S") && haskey(privAA, "ORF3a") && haskey(privAA, "ORF6")  && haskey(privAA, "E")  && haskey(privAA, "M") && haskey(privAA, "ORF7a") && haskey(privAA, "ORF7b") && haskey(privAA, "ORF8") && haskey(privAA, "N") && haskey(privAA, "ORF9b")
                    genes = ["ORF1a", "ORF1b", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"]
                    gene_AA_subs = Set([AA_ORF1a.privateSubstitutions, AA_ORF1b.privateSubstitutions, AA_ORF3a.privateSubstitutions, AA_ORF6.privateSubstitutions, AA_ORF7a.privateSubstitutions, AA_ORF7b.privateSubstitutions, AA_ORF8.privateSubstitutions, AA_ORF9b.privateSubstitutions, AA_S.privateSubstitutions, AA_E.privateSubstitutions, AA_M.privateSubstitutions, AA_N.privateSubstitutions])
                    gene_AA_subs_WT = j.aaSubstitutions
                    labeled_subs_total = j.privateNucMutations.totalLabeledSubstitutions
                    subs = Set(j.privateNucMutations.privateSubstitutions)
                    total_private_nuc_subs = length(subs)
                    dels = Set(j.privateNucMutations.privateDeletionRanges)
                    nuc_dels = Set(j.privateNucMutations.privateDeletions)
                    AA_deletions = Set([AA_ORF1a.privateDeletionRanges, AA_ORF1b.privateDeletionRanges, AA_ORF3a.privateDeletionRanges, AA_ORF6.privateDeletionRanges, AA_ORF7a.privateDeletionRanges, AA_ORF7b.privateDeletionRanges, AA_ORF8.privateDeletionRanges, AA_ORF9b.privateDeletionRanges, AA_S.privateDeletionRanges, AA_E.privateDeletionRanges, AA_M.privateDeletionRanges, AA_N.privateDeletionRanges])

                    seq_nuc_del_ranges_ct[name] = length(dels)

                    subs_WT = Set(j.substitutions)
                    # WT_dels = Set(j.deletions)
                    nuc_dels_WT = Set(j.deletions)
                    gene_AA_del_WT = j.aaDeletions
                    del_ct::Int = 0
                    if j.privateNucMutations.totalReversionSubstitutions < revs_thresh
                        if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions ≥ minmut && j.privateNucMutations.totalPrivateSubstitutions ≤ maxmut
#                            if seq_pango[name] ≠ "B" seq_pango[name] ≠ "B.1" && seq_pango[name] ≠ "B.1.1"
                            for i in gene_AA_subs
                                for k in i
                                    pos = k.pos + 1
                                    AA_sub = "$(k.cdsName):$(k.refAa)$(string(pos))$(k.qryAa)"
                                    if !(k.refAa == "-") && !(AA_sub in excluded_AA)
                                        if AA_sub in chronic_search_muts_v2
                                            chr_muts_ct += 1
                                        end
                                    end
                                end
                            end
                        end
                    end
                    if chr_muts_ct ≥ min_chr_mut
                        if labeled_subs_total/total_private_nuc_subs ≤ 0.3 && !(seq_country in bad_countries)
                            push!(chr_candidates, name)
                        end
                        if labeled_subs_total/total_private_nuc_subs > 0.3
                            push!(bad_chr_candidates, name)
                        end
                    end
                    if chr_muts_ct ≥ min_chr_mut && seq_country in bad_countries && labeled_subs_total/total_private_nuc_subs ≤ 0.3
                        push!(bad_country_chr_candidates, name)
                    end
                end
            end
        end
    end
    println("no_date_to_index_key count = $(no_date_to_index_key)")
    open("$(date)_$(ndjson_name)_minMut$(minmut)_minChrMut$(min_chr_mut)_$(date).txt", "w") do g
        ln = length(chr_candidates)
        ln_bad = length(bad_chr_candidates)
        println("Number of New Chronic Candidates = $(ln)")
        println(g, "Number of New Chronic Candidates = $(ln)")
        chr_candidates_sort = sort(chr_candidates)
        bad_chr_candidates_sort = sort(bad_chr_candidates)
        bad_country_chr_candidates_sort = sort(bad_country_chr_candidates)
        for seq in chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^15)
        print(g, "\n"^15)
        println("Number of Likely Bad New Chronic Candidates = $(ln_bad)")
        println(g, "Number of Likely Bad New Chronic Candidates = $(ln_bad)")
        for seq in bad_chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^15)
        print(g, "\n"^15)
        println("Number of Bad-Country New Chronic Candidates = $(ln_bad)")
        println(g, "Number of Bad-Country New Chronic Candidates = $(ln_bad)")
        for seq in bad_chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^5)
        print(g, "\n"^5)
    end
end

2025-08-06_727PM
5754


chronic_search (generic function with 1 method)

In [148]:
### Execute chronic_search 
date_now = Dates.format(now(), "yyyy_mm_dd_IMMp")
println(date_now)
@time begin
    chronic_search("EPI_ISL_16000001-17000000.ndjson", "16_17mil_chr_candidates", 7, 120, 4, 3)
end
#   chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)
# 786.434928 seconds
# 648.830901 seconds
# 673.509452 seconds
# 870.509452 seconds
# 1165.752907 seconds
#

2025-04-10_716PM
no_date_to_index_key count = 0
Number of New Chronic Candidates = 296
EPI_ISL_16004831, EPI_ISL_16004970, EPI_ISL_16005246, EPI_ISL_16008663, EPI_ISL_16019629, EPI_ISL_16019756, EPI_ISL_16026774, EPI_ISL_16034832, EPI_ISL_16036598, EPI_ISL_16041249, EPI_ISL_16045424, EPI_ISL_16046320, EPI_ISL_16053307, EPI_ISL_16059748, EPI_ISL_16064284, EPI_ISL_16067377, EPI_ISL_16068583, EPI_ISL_16072774, EPI_ISL_16073472, EPI_ISL_16074714, EPI_ISL_16075627, EPI_ISL_16076461, EPI_ISL_16080261, EPI_ISL_16082206, EPI_ISL_16092756, EPI_ISL_16102479, EPI_ISL_16109108, EPI_ISL_16121996, EPI_ISL_16122454, EPI_ISL_16127331, EPI_ISL_16131989, EPI_ISL_16136652, EPI_ISL_16140011, EPI_ISL_16142211, EPI_ISL_16151441, EPI_ISL_16153726, EPI_ISL_16158374, EPI_ISL_16160419, EPI_ISL_16161381, EPI_ISL_16161395, EPI_ISL_16161404, EPI_ISL_16167391, EPI_ISL_16170579, EPI_ISL_16176110, EPI_ISL_16177082, EPI_ISL_16178844, EPI_ISL_16179499, EPI_ISL_16179863, EPI_ISL_16181575, EPI_ISL_16187400, EPI_ISL_16194

In [73]:
### Execute chronic_search 
@time begin
    chronic_search("EPI_ISL_17000001-18000000.ndjson", "EPI_ISL_17000001-18000000", 7, 120, 4, 3)
end
#   chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)

no_date_to_index_key count = 0
Number of New Chronic Candidates = 508
EPI_ISL_17002766, EPI_ISL_17004753, EPI_ISL_17008157, EPI_ISL_17016399, EPI_ISL_17017196, EPI_ISL_17017197, EPI_ISL_17018522, EPI_ISL_17022622, EPI_ISL_17025055, EPI_ISL_17029347, EPI_ISL_17029348, EPI_ISL_17031992, EPI_ISL_17040128, EPI_ISL_17041089, EPI_ISL_17041247, EPI_ISL_17044437, EPI_ISL_17047368, EPI_ISL_17048681, EPI_ISL_17056860, EPI_ISL_17062186, EPI_ISL_17062229, EPI_ISL_17062846, EPI_ISL_17068251, EPI_ISL_17072716, EPI_ISL_17073551, EPI_ISL_17074818, EPI_ISL_17077825, EPI_ISL_17082133, EPI_ISL_17082136, EPI_ISL_17097533, EPI_ISL_17105780, EPI_ISL_17105789, EPI_ISL_17109829, EPI_ISL_17109854, EPI_ISL_17109878, EPI_ISL_17109887, EPI_ISL_17110214, EPI_ISL_17119024, EPI_ISL_17119248, EPI_ISL_17129408, EPI_ISL_17137526, EPI_ISL_17139897, EPI_ISL_17144205, EPI_ISL_17144575, EPI_ISL_17148613, EPI_ISL_17152858, EPI_ISL_17154778, EPI_ISL_17158600, EPI_ISL_17159198, EPI_ISL_17159284, EPI_ISL_17159370, EPI_ISL_1715